In [2]:
from datetime import time
from collections import Counter
import requests
from selenium.webdriver.firefox import webdriver
%run ParsePatent.ipynb
from selenium import webdriver
import time
import pandas as pd
import xlrd
import xlwt
from openpyxl import load_workbook
from xlwt import Workbook
#from xlsxwriter import Workbook


def TopGenesInPatents(targets):
    
    #print(targets)

    my_dict = {i:targets.count(i) for i in targets}
    sor = sorted(my_dict.items(), key=lambda x: x[1], reverse=True)
    sor = list(sor)

    targetList = []
    amountList = []
    total = 0
    for target in sor:
        target = list(target)
        targetList.append(target[0])
        amountList.append(target[1])
        total += target[1]

    df = pd.DataFrame.from_dict({'Column1': targetList, 'Column2': amountList})
    writer = pd.ExcelWriter('Popular2.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Genes', index=False)
    writer.save()

    return sor

def RunningCompanyData():
    patents = PatentsFromFile()
    #patents = testFile.manual_patents

    return testmanual(patents)

def RunningCompanyFinal(info):
    diseases = diseasesfromfile()

    return TopGenesInPatents(info)

def testmanual(patents):
    newPatents = []

    for patent in patents:
        patent = str(patent)
        newPatent = patent.replace('-', '')

        if len(str(newPatent)) == 14:
            newPatent = newPatent[0:6] + "0" + newPatent[6:]
        newPatents.append(newPatent)

    wb = Workbook()
    sheet1 = wb.add_sheet('Sheet 1')
    i = 0
    driver = webdriver.Firefox(
        executable_path=r"C:\Python\Python38\geckodriver.exe")

    aliases = newaliasesfromfile()
    genes = genesfromfile()
    firstlist = {}

    while i < len(newPatents):
        driver.get("https://patents.google.com/patent/" + newPatents[i])
        time.sleep(5)

        targets = parse(driver, aliases, genes, 3, 10)
        k = 0

        if len(targets) > 13:
            i = i + 1
            continue

        sheet1.write(i + 1, 0, newPatents[i])

        for target in targets:
            sheet1.write(i + 1, k + 1, target)
            k = k + 1
        wb.save('CompanyData.xls')

        firstlist[newPatents[i]] = targets
        print(i)
        i = i + 1

    driver.close
    return firstlist

def newaliasesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xls")
    # print(pepega["Unnamed: 0"].values[0])
    # print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog]) - 1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers


def genesfromfile():
    pepega = pd.read_excel("Gene_Aliases_All.xls")
    return pepega["Unnamed: 0"]


def diseasesfromfile():
    pepega = pd.read_excel("Gene_Conditions_All.xls")
    # print(pepega["Unnamed: 0"].values[0])
    # print(pepega.values[0])
    poogers = []
    for dog in range(len(pepega.values)):
        peepo = []
        for cow in range(len(pepega.values[dog]) - 1):
            if pd.isnull(pepega.values[dog][cow]):
                break
            peepo.append(pepega.values[dog][cow])
        poogers.append(peepo)
    return poogers


def diseasesFromTarget(foundTarget, diseases):

    for target in diseases:
        if target[0] == foundTarget:
            return target[1:]
        
def PatentsFromFile():
    pepega = pd.read_excel("JJ_Raw_Data.xls")
    firstColumn = pepega["search URL:"]
    firstColumn.pop(0)

    return firstColumn

def CompanyGenesFromFile():
    pepega = pd.read_excel("CompanyData.xls")
    firstColumn = list(pepega["Unnamed: 1"]) + list(pepega["Unnamed: 2"]) + list(pepega["Unnamed: 3"]) + list(pepega["Unnamed: 4"]) + list(pepega["Unnamed: 5"]) + list(pepega["Unnamed: 6"]) + list(pepega["Unnamed: 7"]) + list(pepega["Unnamed: 8"]) + list(pepega["Unnamed: 9"])
    newlist = [x for x in firstColumn if pd.isnull(x) == False and x != 'nan']

    return newlist

def CompanyDictionary(company, dictionary, distribution, keys):
    dictionary[str(company)] = {}
    count = 0
    for dic in distribution:
        count += 1
        #index = distribution.index(dic)
        if count == len(keys):
            count = 0
        #print(count)
        insert = {}
        if company in dic:
            insert[keys[count]] = dic[company]
        else:
            insert[keys[count]] = None
        dictionary[str(company)].update(insert)
        
    #return count
    

def ReadFromGeneData():
    genes = []
    anotherOne = []
    geneForCompanies = []
    numberForCompanies = []
    companies = []
    companies_kept = []
    pepega = pd.read_excel("CompanyGeneData.xls")
    #print(pepega)
    #x = list(pepega["unnamed: 1"])
    '''
    i = 0
    while i < 163:
        if i % 4 == 1:
            genes += list(pepega["Unnamed: " + str(i)])
        i += 1
    genes = [x for x in genes if pd.isnull(x) == False and x != 'nan']
    #print(genes)
    genes = list(set(genes))
    #print(genes)
    '''
    k = 0
    while k < 163:
        if k % 4 == 0:
            companies += list(pepega["Unnamed: " + str(k)])
            companies = [x for x in companies if pd.isnull(x) == False and x != 'nan']
        elif k % 4 == 1:
            geneForCompanies += list(pepega["Unnamed: " + str(k)])
            geneForCompanies = [x for x in geneForCompanies if pd.isnull(x) == False and x != 'nan']
        elif k % 4 == 2:
            numberForCompanies += list(pepega["Unnamed: " + str(k)])
            numberForCompanies = [x for x in numberForCompanies if pd.isnull(x) == False and x != 'nan']
        elif k % 4 == 3:
            
            for gene in geneForCompanies:
                indexGene = geneForCompanies.index(gene)
                returnList = []
                returnList.append(gene)
                returnList.append(companies[0])
                returnList.append(int(numberForCompanies[indexGene]))
                anotherOne.append(returnList)
                companies_kept.append(companies[0])
                
            '''
            for gene in genes:
                if gene in geneForCompanies:
                    returnList = []
                    indexGene = geneForCompanies.index(gene)
                    returnList.append(gene)
                    returnList.append(companies[0])
                    returnList.append(int(numberForCompanies[indexGene]))
                    anotherOne.append(returnList)
                    #print(returnList)
            '''
            companies.clear()
            geneForCompanies.clear()
            numberForCompanies.clear()
            
        k += 1

    res = []
    [res.append(x) for x in companies_kept if x not in res]
    #print(res)
    #anodaOne = anotherOne.sort(key=lambda x: str(x[0]))
    #print(anotherOne)
    sortedList = sorted(anotherOne, key=lambda x: x[0])
    #print(sortedList)
    list1 = [item[0] for item in sortedList]
    #print(list1)
    list2 = [item[1] for item in sortedList]
    list3 = [item[2] for item in sortedList]
    unique_genes = []
    '''
    for gene in list1:
        if gene not in unique_genes:
            unique_genes.append(gene)
    #print(len(list1))
    '''
    genes_with_patents = {}
    genes_with_companies = {}
    for data in sortedList:
        if data[0] not in unique_genes:
            unique_genes.append(data[0])
            genes_with_patents[data[0]] = data[2]
            dic = {}
            dic[data[1]] = data[2]
            genes_with_companies[data[0]] = dic
        else: 
            genes_with_patents[data[0]] = genes_with_patents[data[0]] + data[2]
            dic = {}
            dic[data[1]] = data[2]
            genes_with_companies[data[0]].update(dic)
    #print(sortedList)
    #print(genes_with_companies)
    
    

            
    ### CANNOT BE USED ON PYTHON 3.7 ###
    genes_with_patents_sorted = dict(sorted(genes_with_patents.items(), key=lambda item: item[1], reverse = True))
    #print(genes_with_patents_sorted)
    #print(unique_genes)
    '''
    for gene in unique_genes:
        print(genes_with_patents[gene])
    '''
        
    pepega1 = pd.read_excel("Gene_Conditions_All.xls")
    disease_genes = list(pepega1["Unnamed: 0"])
    #diseases = []
    diseases = [['breast adenocarcinoma', 'glioblastoma multiforme', 'rheumatoid arthritis', 'refractory anemia', 'non-small cell lung carcinoma'], ['Triple A syndrome', 'influenza', 'HIV infection', 'viral disease', 'Histiocytosis'], ['glioblastoma multiforme', 'androgenetic alopecia', 'hair shape measurement', 'vital capacity', 'forced expiratory volume'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'mean reticulocyte volume', 'mean corpuscular hemoglobin'], ["Hashimoto's thyroiditis", 'thyroid volume', 'high density lipoprotein cholesterol measurement', 'physical activity', 'macrophage inflammatory protein 1b measurement'], ['Tangier disease', 'Apolipoprotein A-I deficiency', 'hyperlipidemia', 'Hypercholesterolemia', 'high density lipoprotein cholesterol measurement'], ['Stargardt disease', 'Cone rod dystrophy', 'age-related macular degeneration', 'Severe early-childhood-onset retinal dystrophy', 'Retinitis pigmentosa'], ['non-small cell lung carcinoma', 'blood protein measurement', 'body height', "Alzheimer's disease", 'cognitive decline measurement'], ['Dyschromatosis universalis', 'Familial pseudohyperkalemia', 'Colobomatous microphthalmia', 'microphthalmia', 'Coloboma of iris'], ['X-linked sideroblastic anemia with ataxia', 'sideroblastic anemia', 'Spinocerebellar ataxia type 1', 'genetic disorder', 'vaginal discharge'], ['Dubin-Johnson syndrome', 'intrahepatic cholestasis', 'genetic disorder', 'erythrocyte count', 'Pseudoxanthoma elasticum'], ['Pseudoxanthoma elasticum', 'Generalized arterial calcification of infancy', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count'], ['hyperinsulinemic hypoglycemia, familial, 1', 'type II diabetes mellitus', 'diabetes mellitus, permanent neonatal 3', 'Familial hyperinsulinism', 'Transient neonatal diabetes mellitus'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['influenza A (H1N1)', 'Graves disease', 'serum IgM measurement', 'QRS duration', 'hemoglobin measurement'], ['neuritic plaque measurement', 'birth weight', 'endometrial cancer', 'high density lipoprotein cholesterol measurement', 'cutaneous melanoma'], ['gout', 'urate measurement', 'uric acid measurement', 'body mass index', 'underweight body mass index status'], ['mean platelet volume', 'gut microbiome measurement', 'salivary gland disease', 'Atrophy', 'coronary artery disease'], ['breast adenocarcinoma', 'periodontal measurement', 'Palmar Fibromatosis', 'monocyte count', 'narcolepsy with cataplexy'], ['breast adenocarcinoma', 'ovarian cancer', 'platelet component distribution width', 'platelet crit', 'non-small cell lung carcinoma'], ['chronic myelogenous leukemia', 'congenital heart defects and skeletal malformations syndrome', 'acute lymphoblastic leukemia', 'neoplasm', 'Gastrointestinal stromal tumor'], ['kidney neoplasm', 'lymphoid neoplasm', 'bile duct carcinoma', 'hemangioblastoma', 'basal cell carcinoma'], ['thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['lymphocyte count', 'hematocrit', 'low grade glioma', 'thyroid gland adenocarcinoma', 'Skull Fractures'], ['autism spectrum disorder', 'triglyceride measurement', 'breast cancer', 'cancer', 'Obesity due to melanocortin 4 receptor deficiency'], ['Isobutyryl-CoA dehydrogenase deficiency', 'body fat percentage', 'body height', 'fat body mass', 'lean body mass'], ['Spondyloepimetaphyseal dysplasia, aggrecan type', 'Spondyloepiphyseal dysplasia, Kimberley type', 'Familial osteochondritis dissecans', 'Osteochondritis Dissecans', 'body height'], ['Ketoacidosis due to beta-ketothiolase deficiency', 'beta-ketothiolase deficiency', 'genetic disorder', 'rheumatoid arthritis', 'nonpapillary renal cell carcinoma'], ['IGA glomerulonephritis', 'pulse pressure measurement', 'systolic blood pressure', 'hair shape measurement', 'dry eye syndrome'], ['Dyskeratosis congenita', 'Hoyeraal-Hreidarsson syndrome', 'Hereditary isolated aplastic anemia', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['diabetic nephropathy', 'hypertension', 'Renal tubular dysgenesis of genetic origin', 'cardiovascular disease', 'type II diabetes mellitus'], ['COVID-19', 'severe acute respiratory syndrome', 'endometrial cancer', 'cutaneous melanoma', 'squamous cell lung carcinoma'], ["Alzheimer's disease", 'Dementia', 'Myasthenia gravis', "Parkinson's disease", 'Cognitive impairment'], ['breast adenocarcinoma', 'endometrial cancer', 'ovarian cancer', 'low grade glioma', 'glioblastoma multiforme'], ['eosinophil count', 'monocyte count', 'monocyte percentage of leukocytes', 'leukocyte count', 'myeloid white cell count'], ['cardiovascular disease', 'coronary artery disease', 'familial hypercholesterolemia', 'hyperlipidemia', 'reticulocyte count'], ['erythrocyte count', 'red blood cell density measurement', 'grip strength measurement', 'body fat percentage', "Parkinson's disease"], ['Peroxisomal acyl-CoA oxidase deficiency', 'Mitchell syndrome', 'Muscle weakness', 'chronic kidney disease', 'urinary metabolite measurement'], ['low molecular weight phosphotyrosine protein phosphatase measurement', 'blood protein measurement', 'appendicular lean mass', 'high density lipoprotein cholesterol measurement', 'protein measurement'], ['prostate cancer', 'self reported educational attainment', 'prostate adenocarcinoma', 'hematocrit', 'hemoglobin measurement'], ['cutaneous melanoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'infect'], ['platelet count', 'red blood cell distribution width', 'prostate carcinoma', 'vital capacity', 'heel bone mineral density'], ['mean corpuscular hemoglobin concentration', 'reticulocyte count', 'hematocrit', 'hemoglobin measurement', 'erythrocyte count'], ['type II diabetes mellitus', 'age at menopause', 'mean platelet volume', 'platelet crit', 'menopause'], ['Oral cleft', 'cutaneous melanoma', 'glomerular filtration rate', 'coronary artery disease', 'forced expiratory volume'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Moyamoya disease', 'Multisystemic smooth muscle dysfunction syndrome', 'Rare disease with thoracic aortic aneurysm and aortic dissection', 'Rare genetic vascular disease'], ['Baraitser-Winter syndrome', 'Developmental malformations - deafness - dystonia', 'cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'focal segmental glomerulosclerosis', 'neuroimaging measurement', 'insulin like growth factor measurement', 'brain measurement'], ['Fibrodysplasia ossificans progressiva', 'genetic disorder', 'kidney neoplasm', 'endometrial cancer', 'carcinoma of liver and intrahepatic biliary tract'], ['pancreatic carcinoma', 'FEV/FEC ratio', 'wellbeing measurement', 'CC16 measurement', 'lung adenocarcinoma'], ['colorectal adenocarcinoma', 'pancreatic adenocarcinoma', 'ulcerative colitis', 'Hepatobiliary Neoplasm', 'gastric carcinoma'], ['visceral heterotaxy', 'Heterotaxia', 'glomerular filtration rate', 'Familial isolated congenital asplenia', 'Double outlet right ventricle'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'epistaxis', 'pulmonary arteriovenous malformation (disease)', 'Heritable pulmonary arterial hypertension'], ['Neurological conditions associated with aminoacylase 1 deficiency', 'Aminoacylase 1 deficiency', 'Intellectual disability', 'toxic encephalopathy', 'body height'], ['Severe combined immunodeficiency due to adenosine deaminase deficiency', 'Severe combined immunodeficiency', 'partial adenosine deaminase deficiency', 'Omenn syndrome', 'chronic lymphocytic leukemia'], ['polyarteritis nodosa, childhoood-onset', 'Sneddon syndrome', 'vasculitis', 'Blackfan-Diamond anemia', "Behcet's syndrome"], ['neoplasm', 'mucous membrane pemphigoid', 'pancreatic carcinoma', 'hepatocellular carcinoma', 'Malignant Pancreatic Neoplasm'], ['Reticulate acropigmentation of Kitamura', "Alzheimer's disease", 'body mass index', 'family history of Alzheimer’s disease', 'schizophrenia'], ['pancreatic adenocarcinoma', 'refractive error', 'body height', 'lean body mass', 'fat body mass'], ['Neonatal inflammatory skin and bowel disease', 'Recurrent infections', 'inflammatory bowel disease', 'inflammatory skin disease', 'genetic disorder'], ['primary ovarian insufficiency', 'vital capacity', 'breast adenocarcinoma', 'hair colour measurement', "Alzheimer's disease"], ['eosinophil count', 'eosinophil percentage of leukocytes', 'head and neck squamous cell carcinoma', 'asthma', "Alzheimer's disease"], ['body height', 'a disintegrin and metalloproteinase with thrombospondin motifs 5 measurement', 'blood protein measurement', 'BMI-adjusted waist circumference', 'appendicular lean mass'], ['Congenital thrombotic thrombocytopenic purpura due to ADAMTS-13 deficiency', 'congenital thrombotic thrombocytopenic purpura', 'thrombotic thrombocytopenic purpura', 'Thrombocytopenia', 'Abnormal bleeding'], ['Dyschromatosis symmetrica hereditaria', 'Aicardi-Goutières syndrome', 'genetic disorder', 'dystonic disorder', 'Familial infantile bilateral striatal necrosis'], ['ulcerative colitis', 'autoimmune disease', 'eosinophil percentage of leukocytes', 'inflammatory bowel disease', "Crohn's disease"], ['body mass index', 'monocyte count', 'lymphocyte count', 'monocyte percentage of leukocytes', 'body height'], ['insulin like growth factor measurement', 'protein measurement', 'fetal genotype effect measurement', 'soluble interleukin-2 receptor subunit alpha measurement', 'gestational serum measurement'], ['DNA methylation', 'gut microbiome measurement', 'refractory celiac disease', 'thyroid gland adenocarcinoma', 'ovarian cancer'], ['Inherited congenital spastic tetraplegia', 'self reported educational attainment', 'cerebral palsy', 'eosinophil count', 'adolescent idiopathic scoliosis'], ['vibratory urticaria', 'ovarian cancer', 'glioblastoma multiforme', 'low grade glioma', 'Hiatus hernia'], ['self reported educational attainment', 'chronotype measurement', 'PR interval', 'risk-taking behaviour', 'Abnormality of refraction'], ['eosinophil count', 'neutrophil count', 'red blood cell distribution width', 'leukocyte count', 'lymphocyte percentage of leukocytes'], ['neoplasm', 'malaria', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm', 'prostate cancer'], ['adiponectin measurement', 'BMI-adjusted adiponectin measurement', 'blood protein measurement', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['Hypermethioninemia encephalopathy due to adenosine kinase deficiency', 'body mass index', 'tooth eruption', 'lean body mass', 'odontogenesis'], ['thyroid gland adenocarcinoma', 'diastolic blood pressure', 'lean body mass', 'adrenomedullin measurement', 'apolipoprotein A 1 measurement'], ['cardiovascular disease', 'hypertension', 'benign prostatic hyperplasia', 'myocardial infarction', 'orthostatic hypotension'], ['unipolar depression', 'pain', 'attention deficit hyperactivity disorder', 'hypertension', 'delirium'], ['chronic obstructive pulmonary disease', 'asthma', 'myocardial infarction', 'hypertension', 'open-angle glaucoma'], ['acute lymphoblastic leukemia', 'B-cell acute lymphoblastic leukemia', 'chronic myelogenous leukemia', 'triglyceride measurement', 'heel bone mineral density'], ['Cognitive impairment-coarse facies-heart defects-obesity-pulmonary involvement-short stature-skeletal dysplasia syndrome', '3q26 microduplication syndrome', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm', 'HER2 Positive Breast Carcinoma'], ['prostate carcinoma', 'blood protein measurement', 'afamin measurement', 'peripheral nerve injury', 'cancer biomarker measurement'], ['Congenital deficiency in alpha-fetoprotein', 'Hereditary persistence of alpha-fetoprotein', 'primary ovarian insufficiency', 'cancer biomarker measurement', 'blood protein measurement'], ['influenza A (H1N1)', 'myeloid white cell count', 'glioblastoma multiforme', 'eosinophil count', 'neutrophil count'], ['Glycogen storage disease due to glycogen debranching enzyme deficiency', 'Glycogen storage disease', 'genetic disorder', 'hypertrophic cardiomyopathy', 'endometrial cancer'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'mood instability measurement', 'hypothyroidism'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'waist circumference', 'risk-taking behaviour', 'phospholipid measurement'], ['protein measurement', 'acute myeloid leukemia', 'Tinnitus', 'pancreatic carcinoma', 'neoplasm'], ['Renal tubular dysgenesis of genetic origin', 'hypertension', 'Bilateral renal dysplasia', 'diastolic blood pressure', 'systolic blood pressure'], ['hypertension', 'Renal tubular dysgenesis of genetic origin', 'primary hypertension', 'kidney disease', 'congestive heart failure'], ['X-linked non-syndromic intellectual disability', 'Inguinal hernia', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['Primary hyperoxaluria type 1', 'Primary hyperoxaluria', 'nephrolithiasis', 'Nephrocalcinosis', 'body height'], ['Joubert syndrome with ocular defect', 'Joubert syndrome', 'Retinitis pigmentosa', 'Retinal dystrophy', 'multiple sclerosis'], ['Retinitis pigmentosa', 'eosinophil count', 'skin pigmentation', 'cutaneous melanoma', 'eosinophil percentage of leukocytes'], ['leukocyte count', 'apolipoprotein A 1 measurement', 'neoplasm', 'infect', 'infection'], ['Severe X-linked mitochondrial encephalomyopathy', 'X-linked hereditary sensory and autonomic neuropathy with deafness', 'X-linked Charcot-Marie-Tooth disease type 4', 'Spondyloepimetaphyseal dysplasia, Bieganski type', 'Leukoencephalopathy - metaphyseal chondrodysplasia'], ['Reticular dysgenesis', 'Intellectual disability', 'gut microbiome measurement', 'neoplasm', 'cancer'], ['glomerular filtration rate', 'neoplasm', 'cancer', 'acute myeloid leukemia', 'colorectal carcinoma'], ['glioblastoma multiforme', 'blood protein measurement', 'pulse pressure measurement', 'reticulocyte count', 'FEV/FEC ratio'], ['diabetes mellitus', 'ovarian cancer', 'squamous cell lung carcinoma', 'blood pressure', 'white matter growth measurement'], ['46,XY disorder of sex development due to isolated 17,20 lyase deficiency', '46,XY complete gonadal dysgenesis', 'testosterone measurement', 'lean body mass', 'appendicular lean mass'], ['lung adenocarcinoma', 'heel bone mineral density', 'metabolite measurement', 'cutaneous melanoma', 'head and neck squamous cell carcinoma'], ['triglyceride measurement', 'hemoglobin measurement', 'sex hormone-binding globulin measurement', 'lean body mass', 'testosterone measurement'], ['Proteus syndrome', 'breast carcinoma', 'Cowden syndrome', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['neoplasm', 'hepatocellular carcinoma', 'gastric cancer', 'head and neck squamous cell carcinoma', 'kidney neoplasm'], ['type II diabetes mellitus', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'cancer', 'prostate carcinoma', 'Familial partial lipodystrophy due to AKT2 mutations'], ['Megalencephaly - polymicrogyria - postaxial polydactyly - hydrocephalus', 'Megalencephaly-capillary malformation-polymicrogyria syndrome', 'cancer', 'glioma', 'schizophrenia'], ['hyperthyroxinemia', 'Congenital analbuminemia', 'liver cancer', 'Ehlers-Danlos syndrome, arthrochalasic type', 'Ehlers-Danlos syndrome type 7A'], ['glioblastoma multiforme', 'cutaneous melanoma', 'urinary bladder cancer', 'age at menarche', 'neoplasm'], ['Isolated anophthalmia - microphthalmia', 'Anophthalmia - microphthalmia', 'microphthalmia', 'autism', 'Breast hypertrophy'], ['alcohol dependence', 'parasitic infection', 'cocaine dependence', 'alcohol drinking', 'alcohol consumption measurement'], ['Sjögren-Larsson syndrome', 'Intellectual disability', 'mean platelet volume', 'mean corpuscular hemoglobin', 'gonorrhea'], ['Pyridoxine-dependent epilepsy', 'Seizure', 'Ventriculomegaly', 'genetic disorder', 'osteoporosis'], ['ALG3-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type I', 'genetic disorder', 'Intellectual disability'], ['neuroblastoma', 'non-small cell lung carcinoma', 'neuroblastoma, susceptibility to, 3', 'neoplasm', 'anaplastic large cell lymphoma'], ['dental caries', 'protein measurement', 'concussion', 'HIV-1 infection', 'viral load'], ['rheumatoid arthritis', 'ulcerative colitis', 'asthma', "Crohn's disease", 'juvenile idiopathic arthritis'], ['bipolar disorder', 'risk-taking behaviour', 'cutaneous melanoma', 'asthma', 'inflammatory bowel disease'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'Nasal Cavity Polyp', 'Eczema', 'sum of eosinophil and basophil counts'], ['inflammatory bowel disease', 'glioblastoma multiforme', 'cutaneous melanoma', 'breast adenocarcinoma', 'blood protein measurement'], ['hypertrophic cardiomyopathy', 'cardiomyopathy', 'PR interval', 'P wave terminal force measurement', 'Left ventricular mass to end-diastolic volume ratio'], ['Hypophosphatasia', 'Adult hypophosphatasia', 'Odontohypophosphatasia', 'Infantile hypophosphatasia', 'Childhood-onset hypophosphatasia'], ['blood protein measurement', 'body height', 'lean body mass', 'fat body mass', 'DNA methylation'], ['lifestyle measurement', 'cutaneous melanoma', 'squamous cell lung carcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['self reported educational attainment', 'neoplasm', 'risk-taking behaviour', 'body fat percentage', 'prostate cancer'], ['Hypomaturation amelogenesis imperfecta', 'Amelogenesis imperfecta', 'Hypoplastic amelogenesis imperfecta', 'amelogenesis imperfecta, type ij', 'Hypomaturation-hypoplastic amelogenesis imperfecta with taurodontism'], ['Gräsbeck-Imerslund disease', 'Imerslund-Grasbeck syndrome type 2', 'megaloblastic anemia (disease)', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['amyotrophic lateral sclerosis', 'type I diabetes mellitus', 'angiogenin measurement', 'familial amyotrophic lateral sclerosis', 'blood protein measurement'], ['intraocular pressure measurement', 'protein measurement', 'open-angle glaucoma', 'glaucoma', 'mean corpuscular volume'], ['heel bone mineral density', 'intraocular pressure measurement', 'glomerular filtration rate', 'ovarian cancer', 'fallopian tube cancer'], ['food allergy measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['vital capacity', 'forced expiratory volume', 'brain measurement', 'neuroimaging measurement', 'blood protein measurement'], ['Hypobetalipoproteinemia', 'familial hypobetalipoproteinemia 1', 'Homozygous familial hypercholesterolemia', 'cutaneous melanoma', 'triglyceride measurement'], ['triglyceride measurement', 'coronary artery disease', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['intraocular pressure measurement', 'refractive error measurement', 'open-angle glaucoma', 'glaucoma', 'corneal topography'], ['high density lipoprotein cholesterol measurement', 'type II diabetes mellitus', 'type I diabetes mellitus', 'triglyceride measurement', 'total cholesterol measurement'], ['Familial long QT syndrome', 'Romano-Ward syndrome', 'genetic disorder', 'complex neurodevelopmental disorder', 'Death in infancy'], ['chondrocalcinosis 2', 'Craniometaphyseal dysplasia', 'Chondrocalcinosis', 'type II diabetes mellitus', 'glaucoma'], ['Autosomal recessive primary microcephaly', 'Microcephaly', 'mental retardation', 'Hypotonia', 'Intellectual disability'], ['chronotype measurement', 'diet measurement', 'brain volume measurement', 'reticulocyte count', 'putamen volume'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration'], ['Diarrhea', 'eosinophil count', 'appendicular lean mass', 'Recurrent infection of the gastrointestinal tract', 'Chronic diarrhea'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'eosinophil count'], ['intraocular pressure measurement', 'serum alanine aminotransferase measurement', 'blood metabolite measurement', 'gallstones', 'body height'], ['GAPO syndrome', 'anthrax infection', 'Familial capillary hemangioma', 'capillary hemangioma', 'body height'], ['Juvenile hyaline fibromatosis', 'hyaline fibromatosis syndrome', 'anthrax infection', 'lymphocyte count', 'lymphocyte percentage of leukocytes'], ['age at menarche', 'brain volume measurement', 'annexin A1 measurement', 'body height', 'breast adenocarcinoma'], ['lymphocyte count', 'monocyte count', 'lymphocyte percentage of leukocytes', 'breast adenocarcinoma', 'inflammatory bowel disease'], ['type II diabetes mellitus', 'lean body mass', 'appendicular lean mass', 'diabetes mellitus', 'body fat percentage'], ['glioblastoma multiforme', 'atherosclerosis', 'Hypercholesterolemia', 'pulse pressure measurement', "Alzheimer's disease"], ['urinary bladder cancer', 'pulse pressure measurement', 'chronic kidney disease', 'atherosclerosis', 'colorectal carcinoma'], ['atrial fibrillation', 'appendicular lean mass', 'body height', 'polycystic ovary syndrome', 'snoring measurement'], ['KID syndrome', 'ichthyosiform erythroderma, corneal involvement, and deafness', 'HIV infection', 'MEDNIK syndrome', 'Failure to thrive'], ['X-linked intellectual disability - Dandy-Walker malformation - basal ganglia disease - Seizures', 'X-linked syndromic intellectual disability', 'Fried syndrome', 'HIV infection', 'X-linked intellectual disability - hypotonia - facial dysmorphism - aggressive behavior'], ['Generalized pustular psoriasis', 'HIV infection', 'Pustulosis palmaris et plantaris', 'Acrodermatitis continua suppurativa of Hallopeau', 'asthma'], ['intelligence', 'self reported educational attainment', 'adolescent idiopathic scoliosis', 'schizophrenia', 'mathematical ability'], ['white matter microstructure measurement', 'body height', 'sitting height measurement', 'vital capacity', 'monocyte count'], ['Familial adenomatous polyposis', 'colorectal adenocarcinoma', 'Gardner syndrome', 'APC-related attenuated familial adenomatous polyposis', 'desmoid tumor'], ['Hypotrichosis simplex', 'hair shape measurement', 'balding measurement', 'heel bone mineral density', 'Alopecia'], ["Crohn's disease", 'ulcerative colitis', 'inflammatory bowel disease', 'blood protein measurement', 'hepatocyte growth factor-like protein measurement'], ['attention deficit hyperactivity disorder', 'glioma', 'pulmonary arterial hypertension', 'glioblastoma multiforme', 'neoplasm'], ['breast adenocarcinoma', 'endometrial cancer', 'neoplasm', 'cancer', 'hepatocellular carcinoma'], ['neoplasm', 'coronary artery disease', 'hepatocellular carcinoma', 'proliferative diabetic retinopathy', 'glioblastoma multiforme'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'hypertension', 'coronary artery disease'], ['blood protein measurement', 'breast adenocarcinoma', 'pancreatic adenocarcinoma', 'glioblastoma multiforme', 'esophageal cancer'], ['Familial renal amyloidosis', 'Apolipoprotein A-I deficiency', 'Tangier disease', 'AL amyloidosis', 'Hypoalphalipoproteinemia'], ['AL amyloidosis', 'triglyceride measurement', 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement', 'apolipoprotein A-IV measurement'], ['familial hypercholesterolemia', 'Hypobetalipoproteinemia', 'hypercholesterolemia, autosomal dominant, type B', 'familial hypobetalipoproteinemia 1', 'cardiovascular disease'], ['colon adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'cutaneous melanoma', 'hepatocellular carcinoma', 'bladder carcinoma'], ['cutaneous melanoma', 'cancer', 'breast cancer', 'multiple myeloma', 'myeloid white cell count'], ['Familial apolipoprotein C-II deficiency', 'familial chylomicronemia syndrome', 'amyloidosis', 'Hyperlipoproteinemia type 1', 'Hyperlipoproteinemia type 4'], ['Cholesterol-ester transfer protein deficiency', 'coronary artery disease', 'Hyperalphalipoproteinemia', 'platelet crit', 'phosphatidylinositol measurement'], ['Hyperlipoproteinemia type 3', 'Lipoprotein glomerulopathy', 'coronary artery disease', 'Sea-blue histiocytosis', 'Early-onset autosomal dominant Alzheimer disease'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'glomerulonephritis (disease)', 'Proteinuria', 'chronic kidney disease'], ['cutaneous melanoma', 'breast adenocarcinoma', 'coronary artery disease', 'systemic lupus erythematosus', 'Sepsis'], ['Early-onset autosomal dominant Alzheimer disease', "Alzheimer's disease", 'Hereditary cerebral hemorrhage with amyloidosis, Dutch type', 'Hereditary cerebral hemorrhage with amyloidosis, Iowa type', 'Hereditary cerebral hemorrhage with amyloidosis, Italian type'], ['Adenine phosphoribosyltransferase deficiency', 'Purine nucleoside phosphorylase deficiency', 'chronic kidney disease', 'interstitial nephritis', 'red blood cell distribution width'], ['Nephrogenic diabetes insipidus', 'allergy', 'allergic rhinitis', 'asthma', 'cyst'], ['glomerular filtration rate', 'hematocrit', 'hemoglobin measurement', 'chronic kidney disease', 'blood urea nitrogen measurement'], ['Androgen insensitivity syndrome', 'Partial androgen insensitivity syndrome', 'prostate carcinoma', 'Kennedy disease', 'prostate adenocarcinoma'], ['short stature, rhizomelic, with microcephaly, micrognathia, and developmental delay', 'myeloid white cell count', 'total cholesterol measurement', 'Microcephaly', 'breast adenocarcinoma'], ['Argininemia', 'glomerular filtration rate', 'type II diabetes mellitus', 'diabetes mellitus', 'creatinine measurement'], ['glioblastoma multiforme', 'cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'chronic myelogenous leukemia'], ['prostate carcinoma', 'varicose disease', 'vein disease', 'lymphatic system disease', 'Varicose veins'], ['basophil percentage of leukocytes', 'inflammatory bowel disease', 'hepatocellular carcinoma', 'acute myeloid leukemia', 'platelet crit'], ['Juvenile Myelomonocytic Leukemia', 'eosinophil count', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma', 'Hepatobiliary Neoplasm'], ['immunodeficiency 62', 'immunodeficiency disease', 'breast adenocarcinoma', 'Combined immunodeficiency with skin granulomas', 'Neutrophil immunodeficiency syndrome'], ['neurodevelopmental disorder with midbrain and hindbrain malformations', 'thyroid gland adenocarcinoma', 'testosterone measurement', 'breast adenocarcinoma', 'Microcephaly'], ['thyroid gland adenocarcinoma', 'pancreatic adenocarcinoma', 'glioblastoma multiforme', 'body mass index', 'lung adenocarcinoma'], ['breast adenocarcinoma', 'cardiovascular disease', 'glioblastoma multiforme', 'systolic blood pressure', 'hematocrit'], ['sex hormone-binding globulin measurement', 'reticulocyte count', 'diet measurement', 'hearing loss', 'reticulocyte measurement'], ['Coffin-Siris syndrome', 'mental retardation', 'gastric adenocarcinoma', 'colorectal adenocarcinoma', 'urinary bladder cancer'], ['age at menopause', 'glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'gastric adenocarcinoma'], ['acute myeloid leukemia', 'breast adenocarcinoma', 'monocyte count', 'lean body mass', 'endometrial cancer'], ['Joubert syndrome', 'Retinitis pigmentosa', 'neutrophil count', 'prostate carcinoma', 'BMI-adjusted waist-hip ratio'], ['age-related macular degeneration', 'wet macular degeneration', 'atrophic macular degeneration', 'retinopathy', 'diabetic retinopathy'], ['neuroticism measurement', 'worry measurement', 'body mass index', 'waist-hip ratio', 'body fat percentage'], ['Metachromatic leukodystrophy', 'Metachromatic leukodystrophy, juvenile form', 'Metachromatic leukodystrophy, adult form', 'Metachromatic leukodystrophy, late infantile form', 'genetic disorder'], ['Mucopolysaccharidosis type 6', 'Mucopolysaccharidosis', 'Mucopolysaccharidosis type 6, slowly progressing', 'Metachromatic leukodystrophy', 'genetic disorder'], ['attention deficit hyperactivity disorder', 'obesity', 'atrial fibrillation', 'breast cancer', 'Interauricular communication'], ['chronic myelogenous leukemia', "Barrett's esophagus", 'hepatocellular carcinoma', 'psoriasis', 'neoplasm'], ['hepatocellular carcinoma', 'neuroblastoma', 'cancer', 'non-small cell lung carcinoma', 'neoplasm'], ['intelligence', 'self reported educational attainment', 'neoplasm', 'pancreatic carcinoma', 'hepatocellular carcinoma'], ['Farber lipogranulomatosis', 'Hereditary myoclonus - progressive distal muscular atrophy', 'Rolandic epilepsy', 'Keloid', 'Fetal akinesia deformation sequence'], ['blood protein measurement', 'neutral ceramidase measurement', 'cerebrospinal fluid biomarker measurement', 'red blood cell distribution width', 'pulse pressure measurement'], ['mathematical ability', 'self reported educational attainment', 'body mass index', 'acute myeloid leukemia', 'body fat percentage'], ['clear cell renal carcinoma', 'hematocrit', 'red blood cell density measurement', 'bipolar disorder', 'body fat distribution'], ['spinal muscular atrophy with congenital bone fractures 2', 'Fetal akinesia deformation sequence', 'spinal muscular atrophy', 'arthrogryposis', 'contracture'], ['Haddad syndrome', 'Ondine syndrome', 'amino acid measurement', 'phenylalanine measurement', 'sex hormone-binding globulin measurement'], ['mean platelet volume', 'erythrocyte count', 'testosterone measurement', 'platelet count', 'self reported educational attainment'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'apolipoprotein B measurement', 'interleukin-27 measurement', 'HMG CoA reductase inhibitor use measurement'], ['blood protein measurement', 'Hypercholesterolemia', 'red blood cell distribution width', 'X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12RB1 deficiency'], ['breast adenocarcinoma', 'Seizure', 'head and neck squamous cell carcinoma', 'breast cancer', 'nonpapillary renal cell carcinoma'], ['body mass index', 'lean body mass', 'puberty onset measurement', 'fat body mass', 'hypertension'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'mean reticulocyte volume', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['Argininosuccinic aciduria', 'blood protein measurement', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'neutrophil count'], ['amino acid measurement', 'asparagine measurement', 'chronic kidney disease', 'serum metabolite measurement', 'hair colour measurement'], ['vital capacity', 'prostate carcinoma', 'mean platelet volume', 'neoplasm', 'platelet count'], ['alveolar rhabdomyosarcoma', 'alveolar soft part sarcoma', 'renal carcinoma', 'Paraganglioma', 'Chordoma'], ['esophageal adenocarcinoma', "Barrett's esophagus", 'sex interaction measurement', 'waist-hip ratio', 'BMI-adjusted waist-hip ratio'], ['appendicular lean mass', 'body height', 'glioblastoma multiforme', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'lung adenocarcinoma', 'mean corpuscular hemoglobin', 'neurotic disorder'], ['Achromatopsia', 'Cone rod dystrophy', 'Macular dystrophy', 'basophil percentage of leukocytes', 'drug use measurement'], ['complement C4 measurement', 'self reported educational attainment', 'mean corpuscular volume', 'Peeling skin syndrome type A', 'Acral peeling skin syndrome'], ['AICA-ribosiduria', 'Intellectual disability', 'anaplastic large cell lymphoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['Ataxia-telangiectasia', 'Hereditary breast cancer', 'colorectal adenocarcinoma', 'breast carcinoma', 'prostate adenocarcinoma'], ['facial morphology measurement', 'colorectal cancer', 'colorectal adenoma', 'hearing loss', 'intelligence'], ['gastroesophageal reflux disease', 'gastric ulcer', 'Helicobacter pylori infectious disease', 'duodenal ulcer', 'Peptic ulcer'], ['hematocrit', 'hair color', 'hemoglobin measurement', 'aspartate aminotransferase measurement', 'eosinophil count'], ['mitochondrial complex V deficiency, nuclear type 6', 'Isolated ATP synthase deficiency', 'Leigh syndrome', 'malaria', 'Sepsis'], ['immunodeficiency 47', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Granular Cell Tumor', 'cancer'], ['X-linked intellectual disability, Hedera type', 'X-linked parkinsonism-spasticity syndrome', 'X-linked intellectual disability - epilepsy', 'congenital disorder of glycosylation type II', 'ATP6AP2-related disorder'], ['Wilson disease', 'dystonic disorder', 'genetic disorder', 'mean corpuscular volume', 'Epileptic encephalopathy'], ['Dysequilibrium syndrome', 'Rare hereditary ataxia', 'Intellectual disability', 'genetic disorder', 'fatty acid measurement'], ['Progressive familial intrahepatic cholestasis', 'Benign recurrent intrahepatic cholestasis', 'Progressive familial intrahepatic cholestasis type 1', 'Benign recurrent intrahepatic cholestasis type 1', 'Intrahepatic cholestasis of pregnancy'], ['systolic blood pressure', 'cystic fibrosis associated meconium ileus', 'diastolic blood pressure', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['Seckel syndrome', 'Seckel syndrome 1', 'Familial cutaneous telangiectasia and oropharyngeal predisposition cancer syndrome', 'cutaneous melanoma', 'lung carcinoma'], ['Alpha-thalassemia - X-linked intellectual disability syndrome', 'X-linked intellectual disability - hypotonic face', 'Renier-Gabreels-Jasper syndrome', 'Smith-Fineman-Myers syndrome', 'Chudley-Lowry-Hoar syndrome'], ['Spinocerebellar ataxia type 2', 'hemoglobin measurement', 'reticulocyte count', 'hematocrit', 'lymphocyte count'], ['Spinocerebellar ataxia type 3', 'Machado-Joseph disease', 'Machado-Joseph disease type 3', 'Machado-Joseph disease type 1', 'Machado-Joseph disease type 2'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'multiple myeloma', 'mean corpuscular hemoglobin', 'unspecified peripheral T-cell lymphoma'], ['NK-cell enteropathy', 'acute myeloid leukemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer'], ['Male infertility due to large-headed multiflagellar polyploid spermatozoa', 'Male infertility with teratozoospermia due to single gene mutation', 'cancer', 'Non-syndromic male infertility due to sperm motility disorder', 'leukemia'], ['Hereditary central diabetes insipidus', 'Central diabetes insipidus', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma'], ['Nephrogenic diabetes insipidus', 'Nephrogenic syndrome of inappropriate antidiuresis', 'Hyponatremia', 'hemorrhage', 'heart failure'], ['head and neck squamous cell carcinoma', 'adolescent idiopathic scoliosis', 'rosacea', 'obesity', 'psoriasis'], ['hepatocellular carcinoma', 'cancer', 'liver cancer', 'hepatocellular adenoma', 'Caudal duplication'], ['acute myeloid leukemia', 'neoplasm', 'erythrocyte count', 'electrocardiography', 'mean corpuscular volume'], ['gut microbiome measurement', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia', 'colorectal carcinoma', 'Hyperinsulinism due to glucokinase deficiency'], ['Immunodeficiency by defective expression of HLA class 1', 'Familial renal amyloidosis', 'HIV infection', 'non-Hodgkins lymphoma', 'AL amyloidosis'], ['prostate carcinoma', 'insomnia measurement', 'N-glycan measurement', 'alkaline phosphatase measurement', 'body fat percentage'], ['testosterone measurement', 'erythrocyte count', 'mean corpuscular volume', 'red blood cell distribution width', 'ankylosing spondylitis'], ['eosinophil count', 'lymphocyte count', 'monocyte count', 'neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes'], ['B4GALT1-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type II', 'urate measurement', 'serum IgG glycosylation measurement'], ["Alzheimer's disease", 'mean platelet volume', 'apolipoprotein A 1 measurement', 'diastolic blood pressure', 'separation anxiety disorder'], ['C-peptide measurement', 'body mass index', 'self reported educational attainment', 'mathematical ability', 'cognitive function measurement'], ['platelet count', 'platelet crit', 'eosinophil count', 'chronic lymphocytic leukemia', 'mean platelet volume'], ['heel bone mineral density', 'appendicular lean mass', 'sitting height measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['Nestor-Guillermo progeria syndrome', 'HIV infection', 'body height', 'cerebrospinal fluid biomarker measurement', 'lymphocyte count'], ['colorectal adenocarcinoma', 'colonic neoplasm', 'T-cell acute lymphoblastic leukemia', 'colon adenocarcinoma', 'bile duct carcinoma'], ['Bardet-Biedl syndrome 9', 'Bardet-Biedl syndrome', 'polydactyly (disease)', 'obesity', 'Retinal dystrophy'], ['Severe motor and intellectual disabilities-sensorineural deafness-dystonia syndrome', 'CADDS', 'genetic disorder', 'breast adenocarcinoma', 'endometrial cancer'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'cutaneous melanoma', 'drug use measurement'], ['Butyrylcholinesterase deficiency', "Alzheimer's disease", 'Dementia', "Parkinson's disease", 'delirium'], ['Autism-epilepsy syndrome due to branched chain ketoacid dehydrogenase kinase deficiency', 'Intellectual disability', 'mental retardation', 'body mass index', "Parkinson's disease"], ['chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'acute myeloid leukemia', 'lymphocyte count'], ['eosinophil count', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'small cell lung carcinoma'], ['platelet count', 'small cell lung carcinoma', 'eosinophil count', 'primary myelofibrosis', 'platelet crit'], ['cancer', 'prostate carcinoma', 'monocyte count', 'asthma', 'red blood cell density measurement'], ['diffuse large B-cell lymphoma', 'chronic lymphocytic leukemia', 'lymphoid neoplasm', 'lung carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['colorectal adenocarcinoma', 'melanoma', 'breast ductal adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'colon adenocarcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'MALT lymphoma', 'neoplasm of mature B-cells', 'colorectal adenocarcinoma', 'mesothelioma'], ['Dias-Logan syndrome', 'Intellectual disability', 'fetal hemoglobin measurement', 'self reported educational attainment', 'mean corpuscular volume'], ['Oculofaciocardiodental syndrome', 'Microphthalmia, Lenz type', 'chronic lymphocytic leukemia', 'gastric adenocarcinoma', 'colorectal adenocarcinoma'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cancer', 'myelodysplastic syndrome', 'leukemia'], ['Björnstad syndrome', 'Isolated CoQ-cytochrome C reductase deficiency', 'GRACILE syndrome', 'mitochondrial complex III deficiency nuclear type 1', 'Renal tubulopathy - encephalopathy - liver failure'], ['reticulocyte count', 'visual perception measurement', 'event death', 'neoplasm', 'benign prostatic hyperplasia'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma'], ['Hereditary angioedema', 'angioedema', 'heart failure', 'subcutaneous adipose tissue measurement', 'ovarian cancer'], ['obesity', 'body mass index', 'lean body mass', 'body fat percentage', 'fat body mass'], ['Best vitelliform macular dystrophy', 'Autosomal recessive bestrophinopathy', 'Retinitis pigmentosa', 'MRCS syndrome', 'Autosomal dominant vitreoretinochoroidopathy'], ['Ehlers-Danlos syndrome, progeroid type', 'hepatocellular carcinoma', 'type II diabetes mellitus', 'carotid atherosclerosis', 'diabetes mellitus'], ['intellectual developmental disorder and retinitis pigmentosa; IDDRP', 'Down syndrome', 'brain volume measurement', 'age at menopause', 'systolic blood pressure'], ['Multicystic dysplastic kidney', 'urate measurement', 'refractive error measurement', 'hematocrit', 'hemoglobin measurement'], ['prostate carcinoma', 'red blood cell density measurement', 'erythrocyte count', 'mean corpuscular hemoglobin', 'mean platelet volume'], ['sex hormone-binding globulin measurement', 'vital capacity', 'FEV/FEC ratio', 'lymphocyte count', 'chronic lymphocytic leukemia'], ['Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta', 'High bone mass osteogenesis imperfecta', 'breast adenocarcinoma', 'cutaneous melanoma'], ['Brachydactyly type A2', 'colorectal cancer', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'sitting height measurement'], ['Microphthalmia with brain and digit anomalies', 'Orofacial clefting syndrome', 'heel bone mineral density', 'colorectal cancer', 'refractive error measurement'], ['body height', 'lean body mass', 'appendicular lean mass', 'vital capacity', 'forced expiratory volume'], ['refractive error measurement', 'corneal topography', 'genetic disorder', 'metopic craniosynostosis', 'nose morphology measurement'], ['heel bone mineral density', 'hemorrhoid', 'rectal disease', 'anus disease', 'blood protein measurement'], ['hematocrit', 'hemoglobin measurement', 'colorectal neoplasm', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['Brachydactyly type A2', 'acromesomelic dysplasia, Demirhan type', 'Brachydactyly type A1', 'Acromesomelic dysplasia', 'Fibular aplasia - complex brachydactyly'], ['Idiopathic and/or familial pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'Pulmonary venoocclusive disease', 'Heritable pulmonary arterial hypertension', 'Pulmonary arterial hypertension associated with congenital heart disease'], ['Circumscribed cutaneous aplasia of the vertex', 'aplasia cutis congenita (disease)', 'proto-oncogene tyrosine-protein kinase receptor Ret measurement', 'hip fracture', 'femur fracture'], ['alopecia areata', 'breast adenocarcinoma', 'ovarian cancer', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['FEV/FEC ratio', 'body height', 'sleep duration', 'low density lipoprotein cholesterol measurement', 'adolescent idiopathic scoliosis'], ['Cardiofaciocutaneous syndrome', 'LEOPARD syndrome', 'Noonan syndrome', 'cutaneous melanoma', 'melanoma'], ['Hereditary breast and ovarian cancer syndrome', 'breast carcinoma', 'Hereditary breast cancer', 'Fanconi anemia, complementation group S', 'breast-ovarian cancer, familial, susceptibility to, 1'], ['breast carcinoma', 'Hereditary breast and ovarian cancer syndrome', 'Hereditary breast cancer', 'Fanconi anemia complementation group D1', 'Fanconi anemia'], ['coronary artery disease', 'multiple sclerosis', 'schizophrenia', 'neoplasm', 'lymphocyte percentage of leukocytes'], ['colorectal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'lung carcinoma', 'colon adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['Intellectual disability', 'non-small cell lung carcinoma', 'NUT midline carcinoma', 'lymphoid neoplasm', 'squamous cell lung carcinoma'], ['liver cancer', 'cutaneous melanoma', 'inflammatory bowel disease', "Crohn's disease", 'granular cell cancer'], ['neoplasm', 'cancer', 'acute myeloid leukemia', 'ovarian cancer', 'hepatocellular carcinoma'], ['Male infertility with teratozoospermia due to single gene mutation', 'primary ovarian insufficiency', 'monocyte count', 'coronary artery disease', 'smoking status measurement'], ['Inborn errors of metabolism', 'breast adenocarcinoma', 'low grade glioma', 'neoplasm', 'hepatocellular carcinoma'], ['cancer', 'chronic obstructive pulmonary disease', 'FEV/FEC ratio', 'smoking status measurement', 'forced expiratory volume'], ['X-linked agammaglobulinemia', 'Isolated agammaglobulinemia', 'Non-acquired isolated growth hormone deficiency', 'chronic lymphocytic leukemia', 'Mantle cell lymphoma'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'coronary artery disease', 'neoplasm', 'rheumatoid arthritis'], ['cutaneous melanoma', 'cognitive function measurement', 'intelligence', 'self reported educational attainment', 'mathematical ability'], ['blood protein measurement', "Barrett's esophagus", 'esophageal adenocarcinoma', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['Mosaic variegated aneuploidy syndrome', 'colonic neoplasm', 'colorectal adenocarcinoma', 'acute myeloid leukemia', 'multiple myeloma'], ['hepatocellular carcinoma', 'neoplasm', 'alcohol consumption measurement', 'platelet count', 'alcohol use disorder measurement'], ['chronotype measurement', 'self reported educational attainment', 'cancer', 'anxiety', 'cardiovascular disease'], ['Ehlers-Danlos syndrome, periodontitis type', 'complement component C1s deficiency', 'Immunodeficiency due to an early component of complement deficiency', 'Hereditary angioedema', 'systemic lupus erythematosus'], ['Immunodeficiency due to an early component of complement deficiency', 'age-related macular degeneration', 'lupus erythematosus', 'Immunodeficiency due to a complement cascade protein anomaly', 'atherosclerosis'], ['Atypical hemolytic-uremic syndrome', 'Complement component 3 deficiency', 'Atypical hemolytic-uremic syndrome with C3 anomaly', 'age-related macular degeneration', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis'], ['platelet count', 'chronotype measurement', 'body mass index', 'eosinophil count', 'BMI-adjusted waist circumference'], ['Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder', 'thioredoxin domain-containing protein 12 measurement', 'Isochromosomy Yp', 'Isolated follicle stimulating hormone deficiency'], ['Immunodeficiency due to a late component of complements deficiency', 'Paroxysmal nocturnal hemoglobinuria', 'Atypical hemolytic-uremic syndrome', 'immune system disease', 'Erythroderma desquamativum'], ['anti-neutrophil antibody associated vasculitis', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['ovarian cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', "Alzheimer's disease"], ['Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'Huntington disease-like syndrome due to C9ORF72 expansions', 'cathepsin L1 measurement', 'sporadic amyotrophic lateral sclerosis'], ['decreased susceptibility to bacterial infection', 'sitting height ratio', 'breast cancer', 'neoplasm', 'Hereditary renal hypouricemia'], ['influenza A (H1N1)', 'colorectal adenocarcinoma', 'colon carcinoma', 'malignant colon neoplasm', 'colonic neoplasm'], ['Ataxia-telangiectasia', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'Ataxia-telangiectasia variant'], ['Temtamy syndrome', 'Global developmental delay', 'Intellectual disability', 'mental retardation', 'attention deficit hyperactivity disorder'], ['glaucoma', 'altitude sickness', 'angle-closure glaucoma', 'Seizure', 'chronic obstructive pulmonary disease'], ['Osteopetrosis with renal tubular acidosis', 'epilepsy', 'glaucoma', 'ocular hypertension', 'altitude sickness'], ['mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'blood protein measurement', 'vital capacity', 'grip strength measurement'], ['carbonic anhydrase 6 measurement', 'blood protein measurement', 'obesity', 'protein measurement', 'synovial sarcoma'], ['Dysequilibrium syndrome', 'Rare hereditary ataxia', 'erythrocyte count', 'thyroxine measurement', 'body height'], ['low grade glioma', 'breast adenocarcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'gastric adenocarcinoma'], ['BMI-adjusted waist circumference', 'body height', 'lean body mass', 'physical activity measurement', 'platelet count'], ['blood protein measurement', 'forced expiratory volume', 'peak expiratory flow', 'platelet endothelial cell adhesion molecule measurement', 'vital capacity'], ['Seizure', 'neurodevelopmental disorder with seizures and non-epileptic hyperkinetic movements', 'epilepsy', 'Bilateral tonic-clonic seizure', 'Intellectual disability'], ['Childhood absence epilepsy', 'Familial hyperaldosteronism', 'epilepsy', 'cardiovascular disease', 'Polyuria'], ['Autosomal dominant non-syndromic intellectual disability', 'cardiovascular disease', 'Urinary incontinence', 'Polyuria', 'generalized anxiety disorder'], ['lean body mass', 'body weight', 'appendicular lean mass', 'body height', 'fat body mass'], ['body weight', 'body mass index', 'lean body mass', 'fasting blood glucose measurement', 'base metabolic rate measurement'], ['protein measurement', 'smoking status measurement', 'clear cell renal carcinoma', 'papillary renal cell carcinoma', 'Dravet syndrome'], ['migraine disorder', 'AL amyloidosis', 'trigeminal neuralgia', 'pain', 'glioblastoma multiforme'], ['type II diabetes mellitus', 'type I diabetes mellitus', 'postmenopausal osteoporosis', 'bone fracture', 'diabetes mellitus'], ['migraine disorder', 'BMI-adjusted waist-hip ratio', 'lymphatic malformation 8', 'physical activity measurement', 'waist-hip ratio'], ['FEV/FEC ratio', 'alkaline phosphatase measurement', 'acute myeloid leukemia', 'sitting height measurement', 'body height'], ['self reported educational attainment', 'FEV/FEC ratio', 'atrial fibrillation', 'mastitis', 'rheumatoid arthritis'], ['essential thrombocythemia', 'myeloproliferative disorder', 'influenza', 'Myelofibrosis', 'acute myeloid leukemia'], ['electrocardiography', 'atrial fibrillation', 'PR interval', 'chronic lymphocytic leukemia', "Parkinson's disease"], ['intellectual developmental disorder 62', 'Intellectual disability, severe', 'Autosomal dominant non-syndromic intellectual disability', 'autism', 'Global developmental delay'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'age at menarche', 'mathematical ability', 'hypothyroidism'], ['lung adenocarcinoma', 'endometrial cancer', 'neoplasm', 'systemic lupus erythematosus', 'infection'], ['platelet count', 'breast adenocarcinoma', 'cutaneous melanoma', 'leukocyte count', 'neoplasm'], ['mean platelet volume', 'platelet component distribution width', 'platelet count', 'urate measurement', 'adolescent idiopathic scoliosis'], ['Autosomal recessive limb-girdle muscular dystrophy type 2A', 'muscular dystrophy, limb-girdle, autosomal dominant 4', 'Muscular dystrophy', 'Muscle weakness', 'Shoulder girdle muscle weakness'], ['forced expiratory volume', 'vital capacity', 'balding measurement', 'systemic lupus erythematosus', 'colorectal carcinoma'], ['inflammatory bowel disease', 'gout', 'non-small cell lung carcinoma', 'bacteriemia', 'cardiovascular disease'], ['psoriasis', 'Pityriasis rubra pilaris', 'acneiform dermatitis', 'familial pityriasis rubra pilaris', 'Generalized pustular psoriasis'], ['interleukin 18 measurement', 'infect', 'infection', 'visual perception measurement', 'neuroblastoma'], ['microcephaly, developmental delay, and brittle hair syndrome', 'colorectal adenocarcinoma', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'Endometrial Endometrioid Adenocarcinoma'], ['mucocutaneous lymph node syndrome', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'lung adenocarcinoma', 'hepatocellular carcinoma'], ['ovarian cancer', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma'], ['Autoimmune lymphoproliferative syndrome with recurrent viral infections', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'basal cell carcinoma', 'viral disease'], ['lung adenocarcinoma', 'glioblastoma multiforme', 'ovarian cancer', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['Familial hypocalciuric hypercalcemia type 1', 'Autosomal dominant hypocalcemia', 'Familial hypocalciuric hypercalcemia', 'Bartter syndrome with hypocalcemia', 'Neonatal severe primary hyperparathyroidism'], ['body mass index', 'lean body mass', "Alzheimer's disease", 'Birt-Hogg-Dubé syndrome', 'appendicular lean mass'], ['acatalasia', 'Acatalasemia', 'Peroxisomal disease', 'catalase measurement', 'blood protein measurement'], ['body height', 'acute myeloid leukemia', 'intraocular pressure measurement', 'lymphocyte percentage of leukocytes', 'lymphocyte count'], ['Noonan syndrome-like disorder with juvenile myelomonocytic leukemia', 'Juvenile Myelomonocytic Leukemia', 'Noonan syndrome', 'cancer', 'listeriosis'], ['Congenital intrinsic factor deficiency', 'Gräsbeck-Imerslund disease', 'neoplasm', 'diffuse large B-cell lymphoma', 'infect'], ['colorectal adenocarcinoma', 'pancreatic adenocarcinoma', 'non-small cell lung carcinoma', 'cancer', 'lung cancer'], ['Classical homocystinuria', 'Methylmalonic acidemia with homocystinuria', 'Abnormality of the cardiovascular system', 'homocystinuria (disease)', 'homocysteine measurement'], ['Familial exudative vitreoretinopathy', 'Berardinelli-Seip congenital lipodystrophy', 'Autosomal dominant neovascular inflammatory vitreoretinopathy', 'X-linked retinoschisis', 'X-linked retinal dysplasia'], ['viral disease', 'endometrial carcinoma', 'triglyceride measurement', 'lean body mass', 'aspartate aminotransferase measurement'], ['lymphocyte count', 'erythrocyte count', 'red blood cell density measurement', 'leukocyte count', 'hematocrit'], ['lean body mass', 'fat body mass', 'monocyte count', 'risk-taking behaviour', 'testosterone measurement'], ['PEHO-like syndrome', 'self reported educational attainment', 'body height', 'forced expiratory volume', 'appendicular lean mass'], ['Eczema', 'allergic rhinitis', 'response to reverse transcriptase inhibitor', 'prostate carcinoma', 'type II diabetes mellitus'], ['body mass index', 'physical activity measurement', 'fat body mass', 'lean body mass', 'visceral adipose tissue measurement'], ["Crohn's disease", 'inflammatory bowel disease', 'monocyte percentage of leukocytes', 'ulcerative colitis', 'psoriasis'], ['C-C motif chemokine 3 measurement', 'blood protein measurement', 'CCL4 measurement', 'visual perception measurement', 'neoplasm'], ['breast adenocarcinoma', 'neoplasm', 'breast cancer', 'infect', 'infection'], ['monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'inflammatory bowel disease', "Crohn's disease"], ['monocyte chemotactic protein-2 measurement', 'C-C motif chemokine 7 measurement', 'C-C motif chemokine 8 measurement', 'blood protein measurement', 'protein measurement'], ['psoriasis', 'acute myeloid leukemia', 'breast adenocarcinoma', 'monocyte count', 'gastric adenocarcinoma'], ['blood protein measurement', 'C-C motif chemokine 16 measurement', 'protein measurement', 'obesity', 'endometriosis'], ['eosinophil count', 'blood protein measurement', 'C-C motif chemokine 17 measurement', 'neoplasm', 'mucocutaneous lymph node syndrome'], ['blood protein measurement', 'C-C motif chemokine 18 measurement', 'obesity', 'protein measurement', 'C-C motif chemokine 3-like 1 measurement'], ['ulcerative colitis', 'asthma', 'allergy', 'childhood onset asthma', 'age at onset'], ['rheumatoid arthritis', 'lymphocyte count', 'ankylosing spondylitis', 'ulcerative colitis', 'sclerosing cholangitis'], ['asthma', 'systemic lupus erythematosus', 'neoplasm', 'red blood cell density measurement', 'hematocrit'], ["Alzheimer's disease", 'atopic eczema', 'basophil count', 'basophil percentage of leukocytes', 'eosinophilic esophagitis'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'lymphocyte count', 'prostate adenocarcinoma'], ['Hereditary cerebral cavernous malformation', 'Cavernous Hemangioma', 'Capillary malformation - arteriovenous malformation', 'diastolic blood pressure', 'neutrophil count'], ['idiopathic pulmonary fibrosis', 'Duchenne muscular dystrophy', 'blood protein measurement', 'pancreatic carcinoma', 'neoplasm'], ['diastolic blood pressure', 'body height', 'lean body mass', 'pulse pressure measurement', 'systolic blood pressure'], ['cancer', 'T-cell leukemia', 'esophageal adenocarcinoma', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['multiple myeloma', 'AL amyloidosis', 'endometrial cancer', 'gastric carcinoma', 'cutaneous melanoma'], ['diffuse large B-cell lymphoma', 'mean corpuscular volume', 'T-cell acute lymphoblastic leukemia', 'breast ductal adenocarcinoma', 'mean corpuscular hemoglobin'], ['gastric carcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'basal cell carcinoma', 'bladder carcinoma', 'lung adenocarcinoma'], ['head and neck squamous cell carcinoma', 'esophageal cancer', 'glioblastoma multiforme', 'endometrial cancer', 'low grade glioma'], ['basophil count', 'celiac disease', 'glioblastoma multiforme', 'basophil percentage of leukocytes', 'mean corpuscular hemoglobin'], ['HIV-1 infection', 'monocyte count', 'non-alcoholic steatohepatitis', 'COVID-19', 'monocyte chemotactic protein 1 measurement'], ['celiac disease', 'glioblastoma multiforme', 'ovarian cancer', "Behcet's syndrome", 'daytime rest measurement'], ['neoplasm', 'mycosis fungoides', "Sezary's disease", 'Cutaneous T-cell lymphoma', 'tropical spastic paraparesis'], ['HIV infection', 'HIV-1 infection', 'AIDS', 'infection', 'type I diabetes mellitus'], ['rheumatoid arthritis', 'ACPA-positive rheumatoid arthritis', 'eosinophil count', 'serum non-albumin protein measurement', 'albumin:globulin ratio measurement'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'monocyte count', 'head and neck squamous cell carcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'Thyroid preparation use measurement', 'myxedema', 'head and neck squamous cell carcinoma'], ["Crohn's disease", 'CCL4 measurement', 'ovarian cancer', 'Hirschsprung disease', 'C-X-C motif chemokine 16 measurement'], ['HIV-1 infection', 'CCL4 measurement', 'monocyte percentage of leukocytes', 'macrophage inflammatory protein 1b measurement', 'viral load'], ['breast adenocarcinoma', 'Neurodegeneration', 'amyotrophic lateral sclerosis', 'breast cancer', 'forced expiratory volume'], ['ovarian cancer', 'gastric adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['neurofilament light chain measurement', 'pancreatic adenocarcinoma', 'neoplasm', 'psoriasis', 'Langerhans Cell Histiocytosis'], ['Townes-Brocks syndrome', 'chronic obstructive pulmonary disease', 'QRS duration', 'multiple sclerosis', 'neoplasm'], ['rheumatoid arthritis', 'refractory anemia', 'infection', 'infect', 'systemic lupus erythematosus'], ['psoriasis', 'atopic eczema', 'immune system disease', 'Pityriasis rubra pilaris', 'rheumatoid arthritis'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'focal segmental glomerulosclerosis', "Alzheimer's disease", 'reticulocyte count', 'red blood cell distribution width'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'T-B+ severe combined immunodeficiency due to CD3delta/CD3epsilon/CD3zeta', 'acute lymphoblastic leukemia', 'neoplasm', 'B-cell acute lymphoblastic leukemia'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'immunodeficiency 18', 'T-B+ severe combined immunodeficiency due to CD3delta/CD3epsilon/CD3zeta', 'acute lymphoblastic leukemia', 'neoplasm'], ['Combined immunodeficiency due to CD3gamma deficiency', 'cutaneous Leishmaniasis', 'acute lymphoblastic leukemia', 'Primary immunodeficiency due to a defect in adaptive immunity', 'neoplasm'], ['HIV infection', 'HIV-1 infection', 'Okt4 epitope deficiency', 'viral disease', 'mycosis fungoides'], ['ulcerative colitis', "Crohn's disease", 'psoriasis', 'inflammatory bowel disease', 'ankylosing spondylitis'], ['Susceptibility to respiratory infections associated with CD8alpha chain mutation', 'blood protein measurement', 'inflammatory bowel disease', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume'], ['diphtheria', 'mean platelet volume', 'platelet count', 'platelet component distribution width', 'breast adenocarcinoma'], ['Primary immunodeficiency', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['Common variable immunodeficiency', 'neoplasm', 'acute lymphoblastic leukemia', 'diffuse large B-cell lymphoma', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency'], ['acute lymphoblastic leukemia', 'neoplasm', 'hairy cell leukemia', 'B-cell acute lymphoblastic leukemia', 'systemic lupus erythematosus'], ['multiple sclerosis', 'neoplasm', 'hepatocellular carcinoma', 'breast cancer', 'cancer'], ['Autosomal recessive lymphoproliferative disease', 'Lymphoproliferative syndrome', 'Combined T and B cell immunodeficiency', 'X-Linked Combined Immunodeficiency Diseases', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['HIV infection', 'eosinophil count', 'rheumatoid arthritis', 'cancer', 'celiac disease'], ['acute myeloid leukemia', 'neoplasm', "Alzheimer's disease", 'platelet count', 'neutrophil count'], ['risk-taking behaviour', 'heart rate', 'lean body mass', 'diet measurement', 'glomerular filtration rate'], ['coronary artery disease', 'platelet glycoprotein VI measurement', 'Primary immunodeficiency', 'inherited bleeding disorder, platelet-type', 'Cognitive impairment'], ['inflammatory bowel disease', 'acute myeloid leukemia', 'chronic lymphocytic leukemia', 'Hyper-IgM syndrome type 3', 'Hepatic veno-occlusive disease - immunodeficiency'], ['multiple myeloma', 'neoplasm', 'ADP-ribosyl cyclase/cyclic ADP-ribose hydrolase 2 measurement', 'amyloidosis', 'asymptomatic myeloma'], ['Hyper-IgM syndrome type 3', 'Hyper-IgM syndrome with susceptibility to opportunistic infections', 'CD40 measurement', 'rheumatoid arthritis', "Crohn's disease"], ['X-linked hyper-IgM syndrome', 'Hyper-IgM syndrome with susceptibility to opportunistic infections', 'hyper-IgM syndrome', 'genetic disorder', 'thyroid gland adenocarcinoma'], ['calcium oxalate urolithiasis', 'systemic lupus erythematosus', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with MCP/CD46 anomaly', 'response to vaccine', 'leukocyte count', 'resting heart rate'], ['bipolar disorder', 'mathematical ability', 'Abnormality of refraction', 'smoking behaviour measurement', 'Tourette syndrome'], ['breast adenocarcinoma', 'monocyte count', 'protein measurement', 'head and neck squamous cell carcinoma', 'tropical spastic paraparesis'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'acute graft vs. host disease', 'chronic lymphocytic leukemia', 'immune system disease'], ['lymphocyte count', 'eosinophil count', 'monocyte count', 'non-Hodgkins lymphoma', 'Beta blocking agent use measurement'], ['Protein-losing enteropathy', 'Primary intestinal lymphangiectasia', 'complement decay-accelerating factor measurement', 'refractive error measurement', 'blood protein measurement'], ['multiple sclerosis', 'primary biliary cirrhosis', 'systemic lupus erythematosus', 'non-Hodgkins lymphoma', 'diastolic blood pressure'], ['Primary CD59 deficiency', 'Paroxysmal nocturnal hemoglobinuria', 'hemolytic anemia', 'hemolysis', 'blood protein measurement'], ['glioblastoma multiforme', 'childhood onset asthma', 'neoplasm', 'melanoma', 'breast cancer'], ['protein measurement', 'platelet component distribution width', 'red blood cell distribution width', 'metabolic syndrome', 'neoplasm'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'myeloid white cell count', 'monocyte count', 'neutrophil percentage of leukocytes'], ['Autosomal recessive lymphoproliferative disease', 'Lymphoproliferative syndrome', 'Combined T and B cell immunodeficiency', 'lymphoma', 'asthma'], ['coronary artery disease', 'lung adenocarcinoma', 'mean platelet volume', 'Activated PIK3-delta syndrome', 'Combined immunodeficiency with skin granulomas'], ['non-small cell lung carcinoma', 'lung adenocarcinoma', 'lymphoid neoplasm', 'squamous cell lung carcinoma', 'acute myeloid leukemia'], ['Autosomal agammaglobulinemia', 'Isolated agammaglobulinemia', 'Agammaglobulinemia', 'diffuse large B-cell lymphoma', 'colorectal adenocarcinoma'], ['Autosomal agammaglobulinemia', 'diffuse large B-cell lymphoma', 'Agammaglobulinemia', 'Isolated agammaglobulinemia', 'Central Nervous System Lymphoma'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'immune system disease', 'dermatomyositis', 'chronic kidney disease'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'glomerulonephritis (disease)', 'type II diabetes mellitus', 'infection'], ['white matter microstructure measurement', 'red blood cell distribution width', 'reticulocyte count', 'reticulocyte measurement', 'lymphocyte count'], ['rheumatoid arthritis', 'caffeine metabolite measurement', 'self reported educational attainment', 'reticulocyte count', 'waist-hip ratio'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'immune system disease', 'dermatomyositis', 'chronic kidney disease'], ['urate measurement', 'uric acid measurement', 'gout', 'drug use measurement', 'irritable bowel syndrome'], ['high density lipoprotein cholesterol measurement', 'sex hormone-binding globulin measurement', 'creatine kinase measurement', 'cutaneous melanoma', 'squamous cell lung carcinoma'], ['eosinophil count', 'blood protein measurement', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'basal cell carcinoma'], ['eosinophil count', 'blood protein measurement', 'asthma', 'reticulocyte count', 'sum of eosinophil and basophil counts'], ['cortisol measurement', 'Eczema', 'allergic rhinitis', 'eosinophil percentage of leukocytes', 'seasonal allergic rhinitis'], ['lung adenocarcinoma', 'cutaneous melanoma', 'thyroid gland adenocarcinoma', 'colorectal adenocarcinoma', 'nasopharyngeal squamous cell carcinoma'], ['mean platelet volume', 'type I diabetes mellitus', 'reticulocyte count', 'eosinophil count', 'platelet component distribution width'], ['vitamin D measurement', 'triglyceride measurement', 'self reported educational attainment', 'physical activity measurement', 'neoplasm'], ['non-small cell lung carcinoma', 'neoplasm', 'cancer', 'Merkel cell skin cancer', 'bladder carcinoma'], ['serum gamma-glutamyl transferase measurement', 'sex hormone-binding globulin measurement', 'serum alanine aminotransferase measurement', 'aspartate aminotransferase measurement', 'neoplasm'], ['myelodysplastic syndrome', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count', 'lymphocyte count'], ['vital capacity', 'drug use measurement', 'PR interval', "Parkinson's disease", 'alcohol consumption measurement'], ['Congenital dyserythropoietic anemia type I', 'Congenital dyserythropoietic anemia', 'bipolar I disorder', 'Melorheostosis', 'postmenopausal osteoporosis'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'endometrial cancer', 'ovarian cancer', 'lung adenocarcinoma'], ['ovarian cancer', 'breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'clear cell renal carcinoma'], ["Alzheimer's disease", 'systolic blood pressure', 'lung adenocarcinoma', 'diastolic blood pressure', 'endometrial cancer'], ['FEV/FEC ratio', 'type II diabetes mellitus', 'chronic obstructive pulmonary disease', 'forced expiratory volume', 'electrocardiography'], ['endometrial cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Congenital hypothyroidism', 'atrophy of thyroid'], ['macula measurement', 'body mass index', 'blood protein measurement', 'neoplasm', 'cancer'], ['Familial gastric cancer', 'Blepharo-cheilo-odontic syndrome', 'hereditary diffuse gastric adenocarcinoma', 'gastric adenocarcinoma', 'Hereditary breast cancer'], ['agenesis of corpus callosum, cardiac, ocular, and genital syndrome', 'Agenesis of corpus callosum', 'self reported educational attainment', 'mathematical ability', 'Familial isolated arrhythmogenic ventricular dysplasia, right dominant form'], ['EEM syndrome', 'Hypotrichosis with juvenile macular degeneration', 'Retinal dystrophy', 'Macular dystrophy', 'macular degeneration'], ['reticulocyte count', 'breast adenocarcinoma', 'endometrial cancer', 'FEV/FEC ratio', 'glioblastoma multiforme'], ['heel bone mineral density', 'protein measurement', 'blood protein measurement', 'serum gamma-glutamyl transferase measurement', 'breast adenocarcinoma'], ['Autosomal dominant non-syndromic intellectual disability', 'Rare genetic intellectual disability with developmental anomaly', 'endometrial cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['pulse pressure measurement', 'asthma', 'response to diisocyanate', 'electrocardiography', 'systolic blood pressure'], ['chronotype measurement', 'diet measurement', 'acute myeloid leukemia', 'self reported educational attainment', 'Ovarian Endometriosis'], ['chronic lymphocytic leukemia', 'ovarian cancer', 'small cell lung carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['chronic lymphocytic leukemia', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma'], ['Familial melanoma', 'cutaneous melanoma', 'melanoma, cutaneous malignant, susceptibility to, 3', 'breast carcinoma', 'breast cancer'], ["Alzheimer's disease", 'Lissencephaly with cerebellar hypoplasia', 'chronic lymphocytic leukemia', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['Autosomal recessive primary microcephaly', 'Isolated congenital microcephaly', 'Microcephaly', 'amyloidosis', 'rheumatoid arthritis'], ['vital capacity', 'generalised epilepsy', 'hepatocellular carcinoma', 'neoplasm', 'gastric cancer'], ['breast carcinoma', 'neoplasm', 'Autosomal recessive primary microcephaly', 'breast cancer', 'breast neoplasm'], ['HIV infection', 'glioblastoma multiforme', 'endometrial cancer', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['intellectual developmental disorder with hypotonia and behavioral abnormalities', 'syndromic intellectual disability', 'cancer', 'Global developmental delay', 'Behavioral abnormality'], ['HIV infection', 'X-linked osteoporosis with fractures', 'chronic lymphocytic leukemia', 'endometrial cancer', 'lung adenocarcinoma'], ['undetermined early-onset epileptic encephalopathy', 'developmental and epileptic encephalopathy, 87', 'risk-taking behaviour', 'Microcephaly', 'breast adenocarcinoma'], ['urinary bladder cancer', 'electrocardiography', 'QRS duration', 'atrial fibrillation', 'sex hormone-binding globulin measurement'], ['Familial melanoma', 'cutaneous melanoma', 'Familial atypical multiple mole melanoma syndrome', 'head and neck squamous cell carcinoma', 'pancreatic adenocarcinoma'], ['endometrial cancer', 'ovarian cancer', 'lung adenocarcinoma', 'red blood cell distribution width', 'mean corpuscular hemoglobin'], ['red blood cell distribution width', 'refractive error measurement', 'cortical surface area measurement', 'reticulocyte measurement', 'hepatocellular carcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'melanoma', 'cancer', 'neoplasm'], ['body height', 'neoplasm', 'ovarian cancer', 'Ehlers-Danlos syndrome, progeroid type', "Alzheimer's disease"], ['lymphocyte percentage of leukocytes', 'small vessel stroke', "Alzheimer's disease", 'neutrophil percentage of leukocytes', 'paraneoplastic neurologic syndrome'], ['breast adenocarcinoma', 'white matter microstructure measurement', 'Blackfan-Diamond anemia', 'Autosomal dominant severe congenital neutropenia', 'peak expiratory flow'], ['Hypotrichosis simplex of the scalp', 'Generalized peeling skin syndrome', 'Peeling skin syndrome type B', 'Hypotrichosis simplex', 'Birt-Hogg-Dubé syndrome'], ['breast adenocarcinoma', 'pancreatic adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm', 'melanoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Female infertility due to zona pellucida defect', 'Non-syndromic male infertility due to sperm motility disorder'], ['Ringed hair disease', 'Griscelli disease type 3', 'Uncombable hair syndrome', 'Microcephaly - albinism - digital anomalies', 'platelet storage pool deficiency'], ['serum carcinoembryonic antigen measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'head and neck squamous cell carcinoma'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm', 'gastric cancer', 'acute lymphoblastic leukemia'], ['serum carcinoembryonic antigen measurement', 'non-small cell lung carcinoma', 'neoplasm', 'colorectal carcinoma', 'gastric cancer'], ['breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'head and neck squamous cell carcinoma', 'colorectal carcinoma', 'Female infertility due to zona pellucida defect'], ['Recurrent infection due to specific granule deficiency', 'eosinophil count', 'basophil count', 'eosinophil percentage of leukocytes', 'basophil percentage of leukocytes'], ['colorectal carcinoma', 'neutrophil percentage of leukocytes', 'hypertension', 'lymphocyte percentage of leukocytes', 'blood pressure'], ['MODY', 'monogenic diabetes', 'reticulocyte count', 'diabetes mellitus', 'chronic kidney disease'], ['Autosomal recessive primary microcephaly', 'Seckel syndrome', 'puberty onset measurement', 'testosterone measurement', 'Testicular Germ Cell Tumor'], ['risk-taking behaviour', 'smoking status measurement', 'monocyte percentage of leukocytes', 'self reported educational attainment', 'diastolic blood pressure'], ['glomerular filtration rate', 'creatinine measurement', 'reticulocyte count', 'prostate carcinoma', 'apolipoprotein A 1 measurement'], ['Cholesterol-ester transfer protein deficiency', 'coronary artery disease', 'Hyperalphalipoproteinemia', 'high density lipoprotein cholesterol measurement', 'total cholesterol measurement'], ['Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with B factor anomaly', 'Immunodeficiency due to a complement cascade protein anomaly', 'age-related macular degeneration', 'chronic hepatitis B virus infection'], ['Recurrent Neisseria infections due to factor D deficiency', 'age-related macular degeneration', 'reticulocyte count', 'reticulocyte measurement', 'Immature Reticulocyte Fraction Measurement'], ['Atypical hemolytic-uremic syndrome', 'Dense deposit disease', 'Immunodeficiency with factor H anomaly', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with H factor anomaly'], ['Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'age-related macular degeneration', 'Atypical hemolytic-uremic syndrome', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with H factor anomaly'], ['Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'age-related macular degeneration', 'Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with H factor anomaly', 'IGA glomerulonephritis'], ['Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'C3 glomerulonephritis', 'Primary membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'diabetic nephropathy'], ['Atypical hemolytic-uremic syndrome', 'Immunodeficiency with factor I anomaly', 'Atypical hemolytic-uremic syndrome with I factor anomaly', 'age-related macular degeneration', 'Familial drusen'], ['Properdin deficiency', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'esophageal cancer', 'breast adenocarcinoma'], ['Cystic fibrosis', 'Congenital bilateral absence of vas deferens', 'congenital bilateral aplasia of vas deferens from CFTR mutation', 'Hereditary chronic pancreatitis', 'response to ivacaftor - efficacy'], ['eosinophil count', 'sitting height measurement', 'ovarian cancer', 'lung adenocarcinoma', 'glioblastoma multiforme'], ['gastric cancer', 'Ovarian hyperstimulation syndrome', 'gonorrhea', 'ovarian cancer', 'Bronchopulmonary dysplasia'], ['red blood cell distribution width', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'acute myeloid leukemia', 'vitamin supplement exposure measurement'], ['Intellectual disability', 'mental retardation', 'Global developmental delay', 'attention deficit hyperactivity disorder', 'eosinophil count'], ['body height', 'Short stature due to GHSR deficiency', 'appendicular lean mass', 'breast adenocarcinoma', 'Congenital hydronephrosis'], ['Lennox-Gastaut syndrome', 'Epileptic encephalopathy', 'complex neurodevelopmental disorder', 'Childhood-onset epilepsy syndrome', 'Intellectual disability'], ['total cholesterol measurement', 'systolic blood pressure', 'low density lipoprotein cholesterol measurement', 'PR interval', 'appendicular lean mass'], ['CHARGE syndrome', 'Kallmann syndrome', 'hypogonadotropic hypogonadism 5 with or without anosmia', 'Normosmic congenital hypogonadotropic hypogonadism', 'hearing loss'], ['Abnormality of refraction', 'refractive error measurement', 'interleukin 17 receptor B measurement', 'cardiovascular disease', 'smoking status measurement'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'squamous cell lung carcinoma'], ['Hereditary breast cancer', 'prostate carcinoma', 'Li-Fraumeni syndrome', 'breast carcinoma', 'osteosarcoma'], ['blood protein measurement', 'acute myeloid leukemia', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'body height'], ['neoplasm', 'amyotrophic lateral sclerosis', 'axial length measurement', 'breast cancer', 'Hand-foot syndrome'], ['Congenital muscular dystrophy due to phosphatidylcholine biosynthesis defect', 'Congenital muscular dystrophy', 'breast adenocarcinoma', 'Seizure', 'Severe global developmental delay'], ['Choroideremia', 'Retinitis pigmentosa', 'Retinal dystrophy', 'night blindness', 'Abnormality of the eye'], ['N-acylethanolamine-hydrolyzing acid amidase measurement', 'breast adenocarcinoma', 'acute myeloid leukemia', 'prostate carcinoma', 'Hematochezia'], ['chronic obstructive pulmonary disease', 'Prune belly syndrome', 'asthma', 'overactive bladder', 'Urinary incontinence'], ['Urinary incontinence', 'Polyuria', 'Hyperhidrosis', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['CHST3-related skeletal dysplasia', 'Autosomal dominant Larsen syndrome', 'Spondyloepiphyseal dysplasia and spondyloepimetaphyseal dysplasia', 'Back pain', 'body height'], ['lean body mass', 'appendicular lean mass', 'body height', 'fat body mass', 'base metabolic rate measurement'], ['intellectual disability, autosomal dominant 45', 'Autosomal dominant non-syndromic intellectual disability', 'mental retardation', 'low grade glioma', 'genetic disorder'], ['Immunodeficiency by defective expression of HLA class 2', 'lung carcinoma', 'gastric carcinoma', 'colon adenocarcinoma', 'colorectal adenocarcinoma'], ['Juvenile myoclonic epilepsy', 'Endocrine-cerebro-osteodysplasia syndrome', 'Short rib-polydactyly syndrome', 'smoking status measurement', 'smoking behaviour measurement'], ['grip strength measurement', 'triglyceride measurement', 'body height', 'body fat percentage', 'self reported educational attainment'], ['appendicular lean mass', 'acute myeloid leukemia', 'BMI-adjusted waist circumference', 'eosinophil count', 'neoplasm'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'Congenital factor II deficiency', 'prostate carcinoma'], ['eosinophil count', 'basophil percentage of leukocytes', 'eosinophil percentage of leukocytes', 'basophil count', 'sum of eosinophil and basophil counts'], ['Cold-induced sweating syndrome', 'Crisponi syndrome', 'acute lymphoblastic leukemia', 'cutaneous melanoma', 'breast adenocarcinoma'], ['Dent disease', 'Dent disease type 1', 'Hypophosphatemic rickets', 'genetic disorder', 'nephrolithiasis'], ['Ichthyosis - hypotrichosis - sclerosing cholangitis', 'optic disc size measurement', 'endometrial cancer', 'neoplasm', 'lung adenocarcinoma'], ['glioblastoma multiforme', 'mean platelet volume', 'asthma', 'gastric adenocarcinoma', 'neoplasm'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast cancer', 'non-small cell lung carcinoma'], ['waist-hip ratio', 'birth weight', 'platelet count', 'eosinophil count', 'head and neck squamous cell carcinoma'], ['hearing loss', 'endometrial cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['FEV/FEC ratio', 'endometrial cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma'], ['blood protein measurement', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'neoplasm', 'eosinophil count'], ['blood protein measurement', 'hepatocellular carcinoma', 'neoplasm', 'gastric cancer', 'lung cancer'], ['blood protein measurement', 'acute myeloid leukemia', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'colitis', 'multiple sclerosis', 'Autosomal agammaglobulinemia'], ['squamous cell lung carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'multiple sclerosis'], ['Chronic mucocutaneous candidosis', 'aspergillosis, susceptibility to', 'inflammatory bowel disease', 'infection', 'infect'], ['blood protein measurement', 'breast adenocarcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'breast cancer'], ['neutrophil count', 'eosinophil count', 'leukocyte count', 'myeloid white cell count', 'acute myeloid leukemia'], ['Autosomal recessive non-syndromic intellectual disability', 'melanocytic neoplasm', 'Short stature due to GHSR deficiency', 'body mass index', 'cervical squamous cell carcinoma'], ['schistosomiasis', 'red blood cell distribution width', 'influenza', 'cancer', 'glioma'], ['glioblastoma multiforme', 'red blood cell distribution width', 'Brugada syndrome', 'Familial short QT syndrome', 'Red cell distribution width'], ['multiple myeloma', 'myeloid neoplasm', 'Miyoshi myopathy', 'neoplasm', "Alzheimer's disease"], ['influenza', 'cutaneous melanoma', 'esophageal cancer', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['intellectual disability, autosomal dominant 56', 'Autosomal dominant non-syndromic intellectual disability', 'genetic disorder', 'lung adenocarcinoma', 'inflammatory myofibroblastic tumor'], ['Nail-patella-like renal disease', 'Balkan nephropathy', 'Fibronectin glomerulopathy', 'Cystinuria type B', 'Cystinuria type A'], ["Alzheimer's disease", 'family history of Alzheimer’s disease', 'fasting blood insulin measurement', 'age at onset', 'refractive error measurement'], ['heel bone mineral density', 'adult onset asthma', 'childhood onset asthma', 'vitamin B12 measurement', 'smoking status measurement'], ['thyroid gland adenocarcinoma', 'cutaneous melanoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme'], ['pancreatic adenocarcinoma', 'neoplasm', 'body height', 'breast cancer', 'Fragile X syndrome'], ['Proximal myotonic myopathy', 'carcinoma of liver and intrahepatic biliary tract', 'breast ductal adenocarcinoma', 'Uterine Carcinosarcoma', 'Hepatobiliary Neoplasm'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Posterior column ataxia - retinitis pigmentosa', 'protein measurement', 'ovarian cancer'], ['endometrial cancer', 'acute myeloid leukemia', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['squamous cell lung carcinoma', 'gastric adenocarcinoma', 'lung adenocarcinoma', 'neoplasm', 'Non-syndromic male infertility due to sperm motility disorder'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'urinary bladder cancer'], ['Myopia', 'blood protein measurement', 'Blackfan-Diamond anemia', 'experimental autoimmune encephalomyelitis', 'multiple sclerosis'], ['obesity', 'Lennox-Gastaut syndrome', 'pain', 'Dravet syndrome', 'Chronic pain'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'monocyte count', 'eosinophil count', 'reticulocyte count'], ['thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'experimental autoimmune encephalomyelitis'], ['cancer', 'myelodysplastic syndrome', 'leukemia', 'precursor lymphoblastic lymphoma/leukemia', 'chronic myelogenous leukemia'], ['fatty acid measurement', 'breast milk measurement', 'parental genotype effect measurement', 'breast adenocarcinoma', 'Total congenital cataract'], ['Osteogenesis imperfecta type 2', 'Osteogenesis imperfecta type 1', 'Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta type 4'], ['Spondyloepiphyseal dysplasia congenita', 'Stickler syndrome type 1', 'Kniest dysplasia', 'Achondrogenesis type 2', 'Spondyloepimetaphyseal dysplasia congenita, Strudwick type'], ['Alport syndrome', 'X-linked Alport syndrome', 'Proteinuria', 'Hematuria', 'Dupuytren Contracture'], ['Stickler syndrome type 3', 'Otospondylomegaepiphyseal dysplasia', 'otospondylomegaepiphyseal dysplasia, autosomal recessive', 'Fibrochondrogenesis', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB'], ['Punctate palmoplantar keratoderma type 1', 'protein measurement', 'body height', 'heel bone mineral density', 'eye measurement'], ['Knobloch syndrome', 'glaucoma, primary closed-angle', 'Dupuytren Contracture', 'Abnormality of connective tissue', 'diabetic foot'], ['Steel syndrome', 'Dupuytren Contracture', 'Abnormality of connective tissue', 'diabetic foot', 'skin wound'], ['Pseudoachondroplasia', 'Multiple epiphyseal dysplasia type 1', 'Multiple epiphyseal dysplasia', 'carpal tunnel syndrome', 'breast adenocarcinoma'], ["Parkinson's disease", 'schizophrenia', 'drug dependence', 'chronic kidney disease', 'urinary metabolite measurement'], ['preeclampsia', 'breast adenocarcinoma', 'prostate carcinoma', 'glioblastoma multiforme', 'stroke'], ['neoplasm', 'heart failure', 'breast cancer', 'pancreatic carcinoma', 'pancreatic neoplasm'], ['leigh syndrome due to mitochondrial complex iv deficiency', 'Isolated cytochrome C oxidase deficiency', 'mitochondrial complex 4 deficiency, nuclear type 15', 'hepatic failure, early-onset, and neurologic disorder due to cytochrome c oxidase deficiency', 'head and neck squamous cell carcinoma'], ['genetic disorder', 'BMI-adjusted waist circumference', 'waist-hip ratio', 'physical activity measurement', 'FEV/FEC ratio'], ['protein measurement', 'blood protein measurement', 'endometrial cancer', 'pancreatic adenocarcinoma', 'cutaneous melanoma'], ['blood protein measurement', 'MAP kinase-activated protein kinase 3 measurement', 'carboxypeptidase B2 measurement', 'protein measurement', 'thrombin activatable fibrinolysis inhibitor activation peptide measurement'], ['mathematical ability', 'high density lipoprotein cholesterol measurement', 'cognitive function measurement', 'glioblastoma multiforme', 'low grade glioma'], ['vital capacity', 'body height', 'sitting height measurement', 'forced expiratory volume', 'Disorder of lipid metabolism'], ['traffic air pollution measurement', 'pneumoconiosis', 'response to silica exposure', 'silicosis', 'mean corpuscular volume'], ['Hereditary coproporphyria', 'harderoporphyria', 'angiopoietin-1 receptor measurement', 'Nuclear cataract', 'Total congenital cataract'], ['blood protein measurement', 'age at first sexual intercourse measurement', 'Abnormality of refraction', 'appendicular lean mass', 'migraine disorder'], ['carbamoyl phosphate synthetase I deficiency disease', 'Carbamoyl-phosphate synthase deficiency', 'pulmonary arterial hypertension', 'Congenital dyserythropoietic anemia type I', 'glycine measurement'], ['Carnitine palmitoyltransferase II deficiency', 'Carnitine palmitoyl transferase II deficiency, myopathic form', 'Carnitine palmitoyl transferase II deficiency, severe infantile form', 'Carnitine palmitoyl transferase II deficiency, neonatal form', 'Familial acute necrotizing encephalopathy'], ['basal cell carcinoma', 'skin cancer', 'skin carcinoma', 'keratinocyte carcinoma', 'non-melanoma skin carcinoma'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['Common variable immunodeficiency', 'systemic lupus erythematosus', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'lupus erythematosus', 'Recurrent infections'], ['Autosomal recessive non-syndromic intellectual disability', 'multiple myeloma', 'acute myeloid leukemia', 'immune system disease', 'Mantle cell lymphoma'], ['endometrial cancer', 'neutrophil count', 'myeloid white cell count', 'body height', "Parkinson's disease"], ['Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'acute myeloid leukemia', 'undifferentiated pleomorphic sarcoma', 'synovial sarcoma'], ['Rubinstein-Taybi syndrome', 'Rubinstein-Taybi syndrome due to CREBBP mutations', 'Menke-Hennekam syndrome 1', 'urinary bladder cancer', 'cancer'], ['Autosomal dominant nocturnal frontal lobe epilepsy', 'body mass index', 'body weight', 'waist circumference', 'body height'], ['type II diabetes mellitus', 'glioblastoma multiforme', 'neoplasm', 'colorectal carcinoma', 'irritable bowel syndrome'], ['lean body mass', 'heel bone mineral density', 'monocyte percentage of leukocytes', 'fat body mass', 'body height'], ['body height', 'diverticulitis', 'appendicular lean mass', 'blood protein measurement', 'grip strength measurement'], ['Cold-induced sweating syndrome', 'Crisponi syndrome', 'blood protein measurement', 'lung adenocarcinoma', 'Oral ulcer'], ['Endometrial Endometrioid Adenocarcinoma', 'hemangioblastoma', 'lung carcinoma', 'cutaneous melanoma', 'basal cell carcinoma'], ['esophageal squamous cell carcinoma', 'esophageal cancer', 'atopic eczema', 'vitamin D measurement', 'childhood onset asthma'], ['pneumonia', 'C-reactive protein measurement', 'bacterial disease', 'chronic obstructive pulmonary disease', 'asthma'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Macular dystrophy'], ['Alpha-crystallinopathy', 'Posterior polar cataract', 'Alpha-B crystallin-related late-onset distal myopathy', 'Familial isolated dilated cardiomyopathy', 'cataract 16 multiple types'], ['red blood cell distribution width', 'visceral adipose tissue measurement', 'rheumatoid arthritis', 'systemic lupus erythematosus', 'eosinophil count'], ['Cerulean cataract', 'Non-syndromic congenital cataract', 'Coralliform cataract', 'Nuclear cataract', 'Coppock-like cataract'], ['mathematical ability', 'self reported educational attainment', 'type I diabetes mellitus', 'Familial primary pulmonary hypoplasia', 'Pyknoachondrogenesis'], ['autism', 'Intellectual disability', 'Global developmental delay', 'body mass index', 'body height'], ['intelligence', 'self reported educational attainment', 'cognitive function measurement', 'wellbeing measurement', 'body fat percentage'], ['monocyte count', 'platelet count', 'leukocyte count', 'lymphocyte count', 'aspartate aminotransferase measurement'], ['Hereditary diffuse leukoencephalopathy with axonal spheroids and pigmented glia', 'brain abnormalities, neurodegeneration, and dysosteosclerosis', 'neoplasm', 'renal cell carcinoma', 'Gastrointestinal stromal tumor'], ['eosinophil percentage of leukocytes', 'rheumatoid arthritis', 'eosinophil count', 'eosinophil percentage of granulocytes', 'neutrophil percentage of granulocytes'], ['Congenital pulmonary alveolar proteinosis', 'Intraalveolar phospholipid accumulation', 'pulmonary alveolar proteinosis', 'melanoma', 'Sepsis'], ['neutrophil count', 'myeloid white cell count', 'granulocyte count', 'sum of neutrophil and eosinophil counts', 'sum of basophil and neutrophil counts'], ['acute lymphoblastic leukemia', 'Normosmic congenital hypogonadotropic hypogonadism', 'appendicular lean mass', 'Prolactin-Producing Pituitary Gland Adenoma', 'Short stature-delayed bone age due to thyroid hormone metabolism deficiency'], ['Non-acquired isolated growth hormone deficiency', 'body height', 'Isolated growth hormone deficiency type IB', 'diet measurement', 'appendicular lean mass'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast carcinoma'], ['Familial advanced sleep-phase syndrome', 'myeloid white cell count', 'granulocyte percentage of myeloid white cells', 'monocyte percentage of leukocytes', 'cancer'], ['erythrocyte count', 'ovarian cancer', 'thyroid gland adenocarcinoma', 'cutaneous melanoma', 'gastric adenocarcinoma'], ['infection', 'infect', 'hair shape measurement', 'atypical teratoid rhabdoid tumor', 'FEV/FEC ratio'], ['hypertrophic cardiomyopathy 12', 'Familial isolated dilated cardiomyopathy', 'familial hypertrophic cardiomyopathy', 'dilated cardiomyopathy', 'Familial dilated cardiomyopathy'], ['Hereditary cerebral hemorrhage with amyloidosis, Icelandic type', 'age-related macular degeneration', 'Hereditary cerebral hemorrhage with amyloidosis', 'AL amyloidosis', 'cystatin C measurement'], ['blood protein measurement', 'cystatin-F measurement', 'alcohol consumption measurement', 'lean body mass', 'Creutzfeldt Jacob Disease'], ['neoplasm', 'glioma', 'melanoma', 'lung cancer', 'non-small cell lung carcinoma'], ['neoplasm', 'glioma', 'lung cancer', 'renal cell carcinoma', 'melanoma'], ['hypotonia, ataxia, developmental delay, and tooth enamel defect syndrome', 'genetic disorder', 'type II diabetes mellitus', 'Intellectual disability', 'mental retardation'], ['autoimmune lymphoproliferative syndrome due to CTLA4 haploinsuffiency', 'type I diabetes mellitus', 'melanoma', 'celiac disease', 'Autoimmune lymphoproliferative syndrome'], ['Pilomatrixoma', 'medulloblastoma', 'Familial exudative vitreoretinopathy', 'hepatocellular carcinoma', 'pilomatrixoma'], ['Blepharo-cheilo-odontic syndrome', 'listeriosis', 'cleft lip', 'cleft palate', 'hepatocellular carcinoma'], ['Hereditary chronic pancreatitis', 'chronic pancreatitis', 'genetic disorder', 'acute pancreatitis', 'alcoholic pancreatitis'], ['red blood cell density measurement', 'erythrocyte count', 'mean corpuscular volume', 'refractive error measurement', 'high density lipoprotein cholesterol measurement'], ['Galactosialidosis', 'Lysosomal disease', 'Non-immune hydrops fetalis', 'mental retardation', 'Intellectual disability'], ['Keratolytic winter erythema', 'blood protein measurement', 'basophil count', 'risk-taking behaviour', 'basophil percentage of leukocytes'], ['Papillon-Lefèvre syndrome', 'Haim-Munk syndrome', 'periodontitis', 'aggressive periodontitis', 'chronic periodontitis'], ['CLN10 disease', 'Neuronal ceroid lipofuscinosis', 'Congenital neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Adult neuronal ceroid lipofuscinosis'], ['Tuberculosis', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'endometrial cancer'], ['Pycnodysostosis', 'Skeletal dysplasia', 'Macrocephaly', 'Short stature due to GHSR deficiency', 'Nail dysplasia'], ['severe acute respiratory syndrome', 'COVID-19', 'cathepsin L1 measurement', 'clear cell renal carcinoma', 'neoplasm'], ['sex hormone-binding globulin measurement', 'cathepsin S measurement', 'basal cell carcinoma', 'breast adenocarcinoma', 'blood protein measurement'], ['Gräsbeck-Imerslund disease', 'megaloblastic anemia (disease)', 'vitamin B12 measurement', 'gut microbiome measurement', 'genetic disorder'], ['Pseudohypoaldosteronism type 2E', 'Pseudohypoaldosteronism type 2A', 'Pseudohypoaldosteronism type 2', 'papillary renal cell carcinoma', 'genetic disorder'], ['multiple myeloma', 'immune system disease', 'Mantle cell lymphoma', 'AL amyloidosis', 'cutaneous lupus erythematosus'], ['global developmental delay with or without impaired intellectual development', 'myelodysplastic syndrome', 'cancer', 'skin carcinoma', 'basal cell carcinoma'], ['adolescent idiopathic scoliosis', 'electrocardiography', 'progesterone measurement', 'body height', 'systemic juvenile idiopathic arthritis'], ['blood protein measurement', 'CX3CL1 measurement', 'glioblastoma multiforme', 'eosinophil count', 'head and neck squamous cell carcinoma'], ['age-related macular degeneration', 'monocyte count', 'lymphocyte count', 'monocyte percentage of leukocytes', 'leukocyte count'], ['growth-regulated alpha protein measurement', 'blood protein measurement', 'glioblastoma multiforme', 'leukocyte count', 'neutrophil count'], ['head and neck squamous cell carcinoma', 'attention deficit hyperactivity disorder', 'breast carcinoma', 'neoplasm', 'midbrain'], ['blood protein measurement', 'CXCL5 measurement', 'C-X-C motif chemokine 5 measurement', 'neoplasm', 'colorectal carcinoma'], ['blood protein measurement', 'ovarian cancer', 'CXCL6 measurement', 'interleukin-8 measurement', 'gastric adenocarcinoma'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'low grade glioma', 'inflammatory bowel disease', 'neoplasm'], ['head and neck squamous cell carcinoma', 'rheumatoid arthritis', 'inflammatory bowel disease', 'Townes-Brocks syndrome', 'neoplasm'], ['inflammatory bowel disease', 'C-X-C motif chemokine 11 measurement', 'neoplasm', 'blood protein measurement', 'protein measurement'], ['HIV-1 infection', 'monocyte count', 'hematocrit', 'hemoglobin measurement', 'erythrocyte count'], ['tonsillectomy risk measurement', 'lymphatic system disease', 'ovarian cancer', 'drug use measurement', 'thyroid gland adenocarcinoma'], ['neoplasm', 'hepatocellular carcinoma', 'cancer', 'glioblastoma multiforme', 'lip and oral cavity carcinoma'], ['neoplasm', 'colon carcinoma', 'colonic neoplasm', 'colorectal adenocarcinoma', 'malignant colon neoplasm'], ['type I diabetes mellitus', 'neutrophil count', 'leukocyte count', 'neutrophil percentage of leukocytes', 'myeloid white cell count'], ['type I diabetes mellitus', 'ulcerative colitis', 'glioblastoma multiforme', 'psoriasis', "Crohn's disease"], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'neoplasm', 'breast cancer'], ['WHIM syndrome', 'multiple myeloma', 'HIV infection', 'non-Hodgkins lymphoma', 'acute lymphoblastic leukemia'], ['allergic rhinitis', 'atopic eczema', 'albumin:globulin ratio measurement', 'multiple sclerosis', 'anti-human herpes virus 7 antibody measurement'], ['Hereditary methemoglobinemia', 'Recessive hereditary methemoglobinemia type 1', 'Recessive hereditary methemoglobinemia type 2', 'methemoglobinemia', 'Autosomal dominant methemoglobinemia'], ['Chronic granulomatous disease', 'X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'X-linked mendelian susceptibility to mycobacterial diseases', 'congenital heart disease', 'Mendelian susceptibility to mycobacterial diseases'], ['Brooke-Spiegler syndrome', 'Familial cylindromatosis', 'Familial multiple trichoepithelioma', 'basal cell carcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['erythrocyte count', 'cutaneous melanoma', 'coffee consumption measurement', 'drug use measurement', 'glioblastoma multiforme'], ['response to efavirenz', 'HIV-1 infection', 'virologic response measurement', 'DDT metabolite measurement', 'gestational serum measurement'], ['blood metabolite measurement', 'chronic kidney disease', 'testosterone measurement', 'urinary metabolite measurement', 'cutaneous melanoma'], ['cholesterol embolism', 'response to phenytoin', 'response to anticoagulant', 'blood metabolite measurement', 'appendicular lean mass'], ['response to clopidogrel', 'systolic blood pressure', 'lymphocyte count', 'serum metabolite measurement', 'cutaneous melanoma'], ['debrisoquine, poor metabolism of', 'cutaneous melanoma', 'glioblastoma multiforme', 'chronic kidney disease', 'endometrial cancer'], ['Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'esophageal squamous cell carcinoma', 'intrahepatic cholangiocarcinoma', 'nan', 'nan'], ['HIV infection', 'HIV-1 infection', 'infection', 'AIDS', 'COVID-19'], ['HIV infection', 'HIV-1 infection', 'infection', 'AIDS', 'pneumocystosis'], ['cutaneous melanoma', 'glioblastoma multiforme', 'lung adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['46,XY disorder of sex development - adrenal insufficiency due to CYP11A1 deficiency', 'Congenital adrenal hyperplasia', 'neoplasm', 'Inherited isolated adrenal insufficiency due to CYP11A1 deficiency', 'adrenal cortex carcinoma'], ['Congenital adrenal hyperplasia due to 11-beta-hydroxylase deficiency', 'Congenital adrenal hyperplasia', 'adrenal gland disease', "pituitary-dependent Cushing's disease", 'Familial hyperaldosteronism type I'], ['Familial hypoaldosteronism', 'Familial hyperreninemic hypoaldosteronism type 1', 'corticosterone methyloxidase type 2 deficiency', 'steroid inherited metabolic disorder', 'hypertension'], ['Congenital adrenal hyperplasia due to 17-alpha-hydroxylase deficiency', 'Congenital adrenal hyperplasia', '46,XY disorder of sex development due to isolated 17,20 lyase deficiency', 'prostate cancer', 'adrenal gland disease'], ['Aromatase deficiency', 'breast cancer', 'breast carcinoma', 'breast neoplasm', 'neoplasm'], ['Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency', 'Congenital adrenal hyperplasia', 'genetic disorder', 'Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency, simple virilizing form', 'Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency, salt wasting form'], ['triglyceride measurement', 'low density lipoprotein cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'alcohol drinking', 'platelet count'], ['coronary artery disease', 'coronary atherosclerosis', 'Myocardial Ischemia', 'glioblastoma multiforme', 'angina pectoris'], ['Non-syndromic congenital cataract', 'hematocrit', 'hemoglobin measurement', 'handedness', 'eosinophil count'], ['mean platelet volume', 'response to vaccine', 'cytokine measurement', 'monocyte count', 'high density lipoprotein cholesterol measurement'], ['asthma', 'seasonal allergic rhinitis', 'allergic rhinitis', 'Rhinitis, Allergic, Perennial', 'obstructive sleep apnea'], ['body height', 'peak expiratory flow', 'BMI-adjusted waist-hip ratio', 'BMI-adjusted waist circumference', 'waist-hip ratio'], ['Walker-Warburg syndrome', 'Autosomal recessive limb-girdle muscular dystrophy type 2P', 'Limb-girdle muscular dystrophy', 'Congenital muscular dystrophy', 'Congenital muscular dystrophy, Fukuyama type'], ['amyotrophic lateral sclerosis', 'schizophrenia', 'mean platelet volume', 'hepatocellular carcinoma', 'cirrhosis of liver'], ['breast adenocarcinoma', 'sitting height measurement', 'ovarian cancer', 'Glycogen storage disease due to phosphoglycerate mutase deficiency', 'head and neck squamous cell carcinoma'], ['neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes', 'neutrophil count', 'lymphocyte count', 'sum of basophil and neutrophil counts'], ['Familial congenital mirror movements', 'Horizontal gaze palsy with progressive scoliosis', 'Agenesis of corpus callosum', 'smoking status measurement', 'neuroticism measurement'], ['Cerebro-facio-articular syndrome', 'Mitral valve prolapse', 'hemoglobin measurement', 'platelet crit', 'hematocrit'], ['cup-to-disc ratio measurement', 'optic cup area measurement', 'protein measurement', 'mathematical ability', 'self reported educational attainment'], ['Hoyeraal-Hreidarsson syndrome', 'breast carcinoma', 'coronary artery disease', 'type I diabetes mellitus', 'autoimmune thyroid disease'], ['Autosomal recessive non-syndromic intellectual disability', 'eosinophil count', 'neutrophil percentage of leukocytes', 'body height', 'low density lipoprotein cholesterol measurement'], ['Piebaldism', 'kidney disease', 'hair colour measurement', 'skin pigmentation', 'hair color'], ['Perry syndrome', 'amyotrophic lateral sclerosis', 'sporadic amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Distal hereditary motor neuropathy type 7'], ['multiple myeloma', 'immune system disease', 'Mantle cell lymphoma', 'AL amyloidosis', 'cutaneous lupus erythematosus'], ['Xeroderma pigmentosum complementation group E', 'Xeroderma pigmentosum', 'basal cell carcinoma', 'prostate carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['Aromatic L-amino acid decarboxylase deficiency', "Parkinson's disease", 'depressive disorder', 'stroke', 'impulse control disorder'], ['liposarcoma', 'dedifferentiated liposarcoma', 'pleomorphic liposarcoma', 'lipoma', 'well-differentiated liposarcoma'], ['DDOST-CDG', 'COVID-19', 'osteosarcoma', 'Ewing sarcoma', 'Congenital disorder of glycosylation'], ['breast adenocarcinoma', 'schizophrenia', 'alkaline phosphatase measurement', 'susceptibility to shingles measurement', 'sleep duration'], ['Spondyloepimetaphyseal dysplasia - short limb - abnormal calcification', 'Warburg-Cinotti syndrome', 'metastatic colorectal cancer', 'neoplasm', 'squamous cell lung carcinoma'], ['insulin like growth factor measurement', 'protein measurement', 'sex hormone-binding globulin measurement', 'multiple sclerosis', 'triglyceride measurement'], ['protein measurement', 'triglyceride measurement', 'insulin like growth factor measurement', 'serum alanine aminotransferase measurement', 'sex hormone-binding globulin measurement'], ['Intellectual disability', 'medulloblastoma', 'mental retardation', 'X-linked syndromic intellectual disability', 'X-linked non-syndromic intellectual disability'], ['DDX41-related hematologic malignancy predisposition syndrome', 'myeloproliferative disorder', 'acute myeloid leukemia', 'chronic myelogenous leukemia', 'Deafness - lymphedema - leukemia'], ['benign neoplasm of lip', 'Intellectual disability', 'cancer', 'neoplasm', 'hepatocellular carcinoma'], ['Singleton-Merten dysplasia', 'hemoglobin measurement', 'psoriasis', 'red blood cell density measurement', 'hematocrit'], ['progressive encephalopathy with leukodystrophy due to DECR deficiency', 'protein measurement', 'smoking status measurement', 'Glycogen storage disease due to liver phosphorylase kinase deficiency', 'Carnitine palmitoyl transferase 1A deficiency'], ['basophil percentage of leukocytes', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'basophil count'], ['blood protein measurement', 'cutaneous melanoma', 'HIV-1 infection', 'viral load', 'squamous cell lung carcinoma'], ['glaucoma', 'cognitive function measurement', 'mathematical ability', 'self reported educational attainment', 'arterial stiffness measurement'], ['Desminopathy', 'Scapuloperoneal amyotrophy', 'dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy', 'Familial dilated cardiomyopathy'], ['Diarrhea', 'Congenital chronic diarrhea with protein-losing enteropathy', 'genetic disorder', 'familial chylomicronemia syndrome', 'glioblastoma multiforme'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'hemoglobin measurement', 'hematocrit'], ['Smith-Lemli-Opitz syndrome', 'genetic disorder', 'Intellectual disability', 'Genetic disorder of sex development', 'cataract'], ['Desmosterolosis', 'hepatocellular carcinoma', 'breast cancer', 'white matter microstructure measurement', 'visceral adipose tissue measurement'], ['Retinitis pigmentosa', 'developmental delay and seizures with or without movement abnormalities', 'undetermined early-onset epileptic encephalopathy', 'Posterior column ataxia - retinitis pigmentosa', 'developmental and epileptic encephalopathy'], ['Constitutional megaloblastic anemia with severe neurologic disease', 'acute lymphoblastic leukemia', 'rheumatoid arthritis', 'non-small cell lung carcinoma', 'psoriasis'], ['46,XY complete gonadal dysgenesis', '46,XY gonadal dysgenesis - motor and sensory neuropathy', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Partial chromosome Y deletion'], ['Postaxial acrofacial dysostosis', 'rheumatoid arthritis', 'multiple sclerosis', 'psoriatic arthritis', 'immune system disease'], ['neoplasm', 'esophageal squamous cell carcinoma', 'Niemann-Pick disease type C', 'Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['Arnold-Chiari malformation type I', 'blood protein measurement', 'electrocardiography', 'dickkopf‐related protein 1 measurement', 'QRS complex'], ['hair shape measurement', 'balding measurement', 'blood protein measurement', 'androgenetic alopecia', 'Alopecia'], ['refractive error measurement', 'body height', 'cortical surface area measurement', 'neuroimaging measurement', 'brain volume measurement'], ['multiple sclerosis', 'mean platelet volume', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect', 'female infertility due to oocyte meiotic arrest'], ['Pyruvate dehydrogenase E2 deficiency', 'Pyruvate dehydrogenase deficiency', 'dystonic disorder', 'genetic disorder', 'waist-hip ratio'], ['Pyruvate dehydrogenase E3 deficiency', 'Pyruvate dehydrogenase deficiency', 'Maple syrup urine disease', 'Leigh syndrome', 'genetic disorder'], ['body height', 'mean platelet volume', 'lean body mass', 'vital capacity', 'forced expiratory volume'], ['neurodevelopmental disorder with non-specific brain abnormalities and with or without seizures', 'cancer', 'T-cell leukemia', 'genetic disorder', 'Alobar holoprosencephaly'], ['Autosomal recessive spondylocostal dysostosis', 'Rib fusion', 'Mesoaxial synostotic syndactyly with phalangeal reduction', 'Syndactyly', 'spondylocostal dysostosis'], ['Adams-Oliver syndrome', 'cancer', 'T-cell leukemia', 'Circumscribed cutaneous aplasia of the vertex', 'cognitive function measurement'], ['Tricho-dento-osseous syndrome', 'Amelogenesis imperfecta', 'Hypomaturation-hypoplastic amelogenesis imperfecta with taurodontism', 'Taurodontism', 'Peripheral pulmonary artery stenosis'], ['Orofacial clefting syndrome', 'balding measurement', 'blood urea nitrogen measurement', 'hair colour measurement', 'hair color'], ['Duchenne muscular dystrophy', 'Becker muscular dystrophy', 'dilated cardiomyopathy 3B', 'Muscular dystrophy', 'Familial isolated dilated cardiomyopathy'], ['Autosomal recessive hypophosphatemic rickets', 'Hypophosphatemic rickets', 'Acromesomelic dysplasia, Grebe type', 'Fibular aplasia - complex brachydactyly', 'osteoarthritis'], ['Steinert myotonic dystrophy', 'Myotonic dystrophy', 'breast adenocarcinoma', 'Catecholaminergic polymorphic ventricular tachycardia', 'Noonan syndrome'], ['sex hormone-binding globulin measurement', 'serum alanine aminotransferase measurement', 'serum gamma-glutamyl transferase measurement', 'apolipoprotein B measurement', 'low density lipoprotein cholesterol measurement'], ['Early infantile epileptic encephalopathy', 'polyendocrine-polyneuropathy syndrome', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'X-linked mixed deafness with perilymphatic gusher', 'deafness'], ['Non-syndromic male infertility due to sperm motility disorder', 'Primary ciliary dyskinesia', 'Kartagener Syndrome', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement'], ['Primary ciliary dyskinesia', 'genetic disorder', 'Kartagener Syndrome', 'prostate carcinoma', 'response to radiation'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'smoking status measurement', 'ciliopathy', 'Recurrent infections'], ['Primary ciliary dyskinesia', 'ciliopathy', 'Kartagener Syndrome', 'genetic disorder', 'bronchiectasis'], ['CLN4B disease', 'Adult neuronal ceroid lipofuscinosis', 'Neuronal ceroid lipofuscinosis', 'mean platelet volume', 'Immature Reticulocyte Fraction Measurement'], ['systemic lupus erythematosus', 'Autosomal recessive systemic lupus erythematosus', 'neutrophil count', 'vital capacity', 'myeloid white cell count'], ['Autosomal recessive systemic lupus erythematosus', 'systemic lupus erythematosus', 'systemic scleroderma', 'limited scleroderma', 'lupus nephritis'], ['systemic lupus erythematosus', 'cirrhosis of liver', 'Primary membranoproliferative glomerulonephritis', 'rheumatoid arthritis', 'reticulocyte count'], ['FEV/FEC ratio', 'blood protein measurement', 'ulcerative colitis', 'ovarian cancer', 'pulmonary function measurement'], ['Autosomal dominant cerebellar ataxia, deafness and narcolepsy', 'Hereditary sensory and autonomic neuropathy type 1', 'acute myeloid leukemia', 'myelodysplastic syndrome', 'chronic myelomonocytic leukemia'], ['acute myeloid leukemia', 'Tall stature-intellectual disability-facial dysmorphism syndrome', 'myelodysplastic syndrome', 'chronic myelomonocytic leukemia', 'diffuse large B-cell lymphoma'], ['risk-taking behaviour', 'intelligence', 'waist-hip ratio', 'mean corpuscular hemoglobin concentration', 'insomnia'], ['acute lymphoblastic leukemia', 'Okt4 epitope deficiency', 'neoplasm', 'seminoma', 'Alymphoid cystic thymic dysgenesis'], ['ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'inflammatory bowel disease', 'Brugada syndrome'], ['lean body mass', 'heel bone mineral density', 'hemoglobin measurement', 'appendicular lean mass', 'erythrocyte count'], ['DPAGT1-CDG', 'Congenital myasthenic syndromes', 'Congenital myasthenic syndromes with glycosylation defect', 'Congenital disorder of glycosylation', 'genetic disorder'], ['infection', 'Sepsis', 'urinary tract infection', 'pneumonia', 'endocarditis'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'glioblastoma multiforme', 'cutaneous melanoma'], ['Global developmental delay', 'Cerebellar vermis hypoplasia', 'Dandy-Walker syndrome', 'Hydrocephalus', 'pancreatic adenocarcinoma'], ['Intellectual disability', 'Macrocephaly', 'Global developmental delay', 'mental retardation', 'schizophrenia'], ['lymphocyte count', 'monocyte count', 'monocyte percentage of leukocytes', 'myeloid white cell count', 'multiple sclerosis'], ['type II diabetes mellitus', 'diabetes mellitus', 'type I diabetes mellitus', 'polycystic ovary syndrome', 'prediabetes syndrome'], ['electrocardiography', 'dermatopontin measurement', 'Progressive sensorineural hearing impairment', 'PR interval', 'TPE interval measurement'], ['schizophrenia', 'unipolar depression', 'bipolar disorder', 'attention deficit hyperactivity disorder', 'Tourette syndrome'], ['schizophrenia', 'psychosis', "Parkinson's disease", 'bipolar disorder', 'schizoaffective disorder'], ['kidney Wilms tumor', 'colorectal adenocarcinoma', 'gastric carcinoma', 'kidney neoplasm', 'lung carcinoma'], ['Familial isolated arrhythmogenic right ventricular dysplasia', 'Arrhythmogenic right ventricular dysplasia', 'Abnormality of the cardiovascular system', 'Familial isolated arrhythmogenic ventricular dysplasia, left dominant form', 'Familial isolated arrhythmogenic ventricular dysplasia, biventricular form'], ['Hereditary hypotrichosis with recurrent skin vesicles', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'ovarian cancer'], ['genetic disorder', 'reticulocyte count', 'fasting blood glucose measurement', 'self reported educational attainment', 'brain aneurysm'], ['Keratosis palmoplantaris striata', 'Severe dermatitis-multiple allergies-metabolic wasting syndrome', 'Diffuse palmoplantar keratoderma with painful fissures', 'Focal palmoplantar keratoderma with joint keratoses', 'palmoplantar keratoderma i, striate, focal, or diffuse'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer', 'ovarian cancer', 'low grade glioma'], ['Hypotrichosis simplex', 'Monilethrix', 'glioblastoma multiforme', 'breast adenocarcinoma', 'clear cell renal carcinoma'], ['Woolly hair-palmoplantar keratoderma-dilated cardiomyopathy syndrome', 'Skin fragility-woolly hair-palmoplantar keratoderma syndrome', 'Keratosis palmoplantaris striata', 'Lethal acantholytic epidermolysis bullosa', 'arrhythmogenic cardiomyopathy with woolly hair and keratoderma'], ['Dentinogenesis imperfecta type 2', 'deafness, autosomal dominant 39, with dentinogenesis imperfecta 1', 'Dentinogenesis imperfecta type 3', 'Dentin dysplasia type II', 'Dentin dysplasia'], ['Hereditary sensory and autonomic neuropathy type 6', 'Epidermolysis bullosa simplex', 'KRT14-related epidermolysis bullosa simplex', 'autism spectrum disorder', 'Charcot-Marie-Tooth disease'], ['hematocrit', 'total iron binding capacity', 'Fuchs endothelial corneal dystrophy', 'Lisch epithelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy'], ['cutaneous melanoma', 'nevus count', 'insulin like growth factor measurement', 'reticulocyte count', 'type II diabetes mellitus'], ['cup-to-disc ratio measurement', 'birth weight', 'C-reactive protein measurement', 'optic cup area measurement', 'body height'], ['pancreatic adenocarcinoma', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'obesity'], ['protein measurement', 'heel bone mineral density', 'acute myeloid leukemia', 'high density lipoprotein cholesterol measurement', 'response to statin'], ['heel bone mineral density', "Crohn's disease", 'serum non-albumin protein measurement', 'osteoarthritis', 'bone density'], ['glioblastoma multiforme', 'thyroid gland adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'cutaneous melanoma'], ['eosinophil count', 'hair color', 'eosinophil percentage of leukocytes', 'lymphocyte count', 'systemic lupus erythematosus'], ['mean corpuscular hemoglobin', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm', 'pancreatic carcinoma', 'neoplasm'], ['Dyggve-Melchior-Clausen disease', 'Smith-McCort dysplasia', 'waist-hip ratio', 'lean body mass', 'grip strength measurement'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2O', 'Autosomal dominant childhood-onset proximal spinal muscular atrophy', 'Autosomal dominant non-syndromic intellectual disability', 'Autosomal dominant childhood-onset proximal spinal muscular atrophy without contractures', 'Abnormality of neuronal migration'], ['complex neurodevelopmental disorder', 'Rare genetic intellectual disability with developmental anomaly', 'Intellectual disability', 'mental retardation', 'genetic disorder'], ['metabolic syndrome', 'macula measurement', 'ovarian cancer', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm'], ['small cell lung carcinoma', 'urinary bladder cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['asthma', 'multiple sclerosis', 'Senior-Loken syndrome', 'age-related hearing impairment', 'diet measurement'], ['breast carcinoma', 'hypertension', 'balding measurement', 'vital capacity', 'diastolic blood pressure'], ['interleukin-27 measurement', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'blood protein measurement', 'cutaneous melanoma'], ['MEND syndrome', 'X-linked dominant chondrodysplasia punctata', 'polydactyly (disease)', 'Focal palmoplantar and gingival keratoderma', 'Trichodysplasia - xeroderma'], ['forced expiratory volume', 'mean corpuscular volume', 'vital capacity', 'lymphocyte count', 'leukocyte count'], ['breast cancer', 'breast ductal carcinoma in situ', 'glomerular filtration rate', 'neoplasm', 'serum alanine aminotransferase measurement'], ['Hirschsprung disease', 'body height', 'appendicular lean mass', 'lean body mass', 'worry measurement'], ['Distal arthrogryposis type 5D', 'Distal arthrogryposis', 'genetic disorder', 'protein measurement', 'refractive error measurement'], ['body weights and measures', 'alkaline phosphatase measurement', 'body height', 'Red cell distribution width', 'acute myeloid leukemia'], ['X-linked hypohidrotic ectodermal dysplasia', 'Oligodontia', 'Hypohidrotic ectodermal dysplasia', 'tooth agenesis', 'Ectodermal dysplasia syndrome'], ['COVID-19', 'protein C measurement', 'alkaline phosphatase measurement', 'birth weight', 'hematological measurement'], ['Auriculocondylar syndrome', 'hematocrit', 'erythrocyte count', 'hemoglobin measurement', 'red blood cell density measurement'], ['Waardenburg-Shah syndrome', 'Hirschsprung disease', 'Ondine syndrome', 'Haddad syndrome', 'Waardenburg syndrome'], ['mandibulofacial dysostosis with alopecia', 'pulmonary arterial hypertension', 'hypertension', 'systemic scleroderma', 'pulmonary hypertension'], ['Waardenburg-Shah syndrome', 'ABCD syndrome', 'Hirschsprung disease', 'pulmonary arterial hypertension', 'hypertension'], ['Cohen-Gibson syndrome', 'Tall stature-intellectual disability-facial dysmorphism syndrome', 'viral disease', 'melanoma', 'breast carcinoma'], ['Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'mental retardation', 'Autosomal dominant non-syndromic intellectual disability', 'developmental and epileptic encephalopathy'], ['Familial drusen', 'Inguinal hernia', 'body height', 'BMI-adjusted waist circumference', 'physical activity measurement'], ['cutaneous melanoma', 'Fuchs endothelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy', 'smoking status measurement', 'X-linked endothelial corneal dystrophy'], ['self reported educational attainment', 'insomnia measurement', 'mathematical ability', 'smoking status measurement', 'smoking initiation'], ['Severe combined immunodeficiency due to CARD11 deficiency', 'Autosomal recessive early-onset inflammatory bowel disease', 'Autosomal agammaglobulinemia', 'IgE responsiveness, atopic', 'X-Linked Combined Immunodeficiency Diseases'], ['cancer', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Autosomal dominant primary hypomagnesemia with hypocalciuria', 'platelet count', 'reticulocyte count'], ['mean reticulocyte volume', 'diastolic blood pressure', 'mean corpuscular hemoglobin', 'systolic blood pressure', 'visual perception measurement'], ['non-small cell lung carcinoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cancer', 'colorectal adenocarcinoma'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'hepatocellular carcinoma', 'neoplasm', 'glioma'], ['prostate adenocarcinoma', 'ovarian cancer', 'breast adenocarcinoma', 'esophageal cancer', 'drug use measurement'], ['leukocyte count', 'neutrophil count', 'lymphocyte percentage of leukocytes', 'monocyte count', 'myeloid white cell count'], ['serum gamma-glutamyl transferase measurement', 'hemoglobin measurement', 'CC16 measurement', 'chronic obstructive pulmonary disease', 'lung disease associated with cystic fibrosis'], ['Kleefstra syndrome', 'Intellectual disability', 'genetic disorder', 'mental retardation', 'Global developmental delay'], ['hepatocellular carcinoma', 'colorectal carcinoma', 'calf circumference measurement', 'cholangiocarcinoma', 'cancer'], ['leukoencephalopathy, motor delay, spasticity, and dysarthria syndrome', 'reticulocyte count', 'reticulocyte measurement', 'mean corpuscular volume', 'Immature Reticulocyte Fraction Measurement'], ['leukoencephalopathy, developmental delay, and episodic neurologic regression syndrome', 'influenza', 'platelet count', 'adolescent idiopathic scoliosis', 'Cognitive impairment'], ['Pulmonary capillary hemangiomatosis', 'Pulmonary venoocclusive disease', 'Heritable pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'Non-syndromic pontocerebellar hypoplasia'], ['Ovarioleukodystrophy', 'CACH syndrome', 'Cree leukoencephalopathy', 'Juvenile or adult CACH syndrome', 'Late infantile CACH syndrome'], ['head and neck squamous cell carcinoma', 'colon adenocarcinoma', 'pancreatic carcinoma', 'esophageal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['age at menopause', 'menopause', 'breast cancer', 'renal cell carcinoma', 'neoplasm'], ['Autosomal dominant severe congenital neutropenia', 'Cyclic neutropenia', 'Severe congenital neutropenia', 'Constitutional neutropenia', 'neutropenia'], ['HIV-1 infection', 'viral load', 'neoplasm', 'breast cancer', 'melanoma'], ['urinary bladder cancer', 'matrix metalloproteinase 7 measurement', 'hepatocellular carcinoma', 'cholangiocarcinoma', 'lung adenocarcinoma'], ['B-cell acute lymphoblastic leukemia', 'lymphocyte count', 'small intestine neuroendocrine tumor', 'arterial stiffness measurement', 'FEV/FEC ratio'], ['Supravalvular aortic stenosis', 'Autosomal dominant cutis laxa', 'genetic disorder', 'Williams syndrome', 'Cutis laxa'], ['intelligence', 'chronic kidney disease', 'urinary metabolite measurement', 'leukemia', 'infect'], ['schizophrenia', 'type II diabetes mellitus', 'leukocyte count', 'hair colour measurement', 'myeloid white cell count'], ['X-linked Emery-Dreifuss muscular dystrophy', 'Emery-Dreifuss muscular dystrophy', 'neuromuscular disease', 'Abnormality of the cardiovascular system', 'myopathy'], ['non-small cell lung carcinoma', 'neoplasm', 'lung adenocarcinoma', 'forced expiratory volume', 'lung carcinoma'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'self reported educational attainment', 'lifestyle measurement', 'C-reactive protein measurement'], ['leukocyte count', 'body mass index', 'neuroticism measurement', 'breast adenocarcinoma', 'fat body mass'], ['blood protein measurement', 'brain injury', 'optic disc size measurement', 'age at menarche', 'unipolar depression'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'Cerebral arteriovenous malformation', 'Heritable pulmonary arterial hypertension', 'genetic disorder'], ['neutrophil count', 'Tuberculosis', 'platelet count', 'myeloid white cell count', 'neutrophil percentage of leukocytes'], ['protein measurement', 'uterine fibroid', 'chronic kidney disease', 'urinary metabolite measurement', 'erythrocyte count'], ['hypertension', 'systolic blood pressure', 'blood pressure', 'diastolic blood pressure', 'Diuretic use measurement'], ['Generalized arterial calcification of infancy', 'Autosomal recessive hypophosphatemic rickets', 'Hypopigmentation-punctate palmoplantar keratoderma syndrome', 'type II diabetes mellitus', 'ossification of the posterior longitudinal ligament of the spine'], ['lean body mass', 'appendicular lean mass', 'vital capacity', 'brain volume measurement', 'forced expiratory volume'], ['alkaline phosphatase measurement', 'type II diabetes mellitus', 'body mass index', 'vitamin D measurement', 'testosterone measurement'], ['Autosomal recessive spastic paraplegia type 64', 'protein measurement', 'platelet count', 'electrocardiography', 'DNA methylation'], ['experimental autoimmune encephalomyelitis', 'intrahepatic cholangiocarcinoma', 'chronic obstructive pulmonary disease', 'monocyte percentage of leukocytes', 'hepatocellular carcinoma'], ['eosinophil percentage of leukocytes', 'prostate carcinoma', 'renal cell carcinoma', 'colorectal cancer', 'eosinophil count'], ['Microcephaly - polymicrogyria - corpus callosum agenesis', 'rheumatoid arthritis', 'eosinophil count', 'multiple sclerosis', 'myeloid white cell count'], ['Rubinstein-Taybi syndrome', 'Rubinstein-Taybi syndrome due to EP300 haploinsufficiency', 'head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Menke-Hennekam syndrome 2'], ['Autosomal dominant secondary polycythemia', 'Paraganglioma', 'Primary familial polycythemia', 'renal cell carcinoma', 'electrocardiography'], ['Intestinal epithelial dysplasia', 'Lynch syndrome', 'colorectal cancer, hereditary nonpolyposis, type 8', 'neoplasm', 'hereditary nonpolyposis colorectal carcinoma'], ['cancer', 'cytokine measurement', 'response to vaccine', 'hair color', 'balding measurement'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'sex hormone-binding globulin measurement', 'non-small cell lung carcinoma'], ['Non-syndromic congenital cataract', 'Total congenital cataract', 'Posterior polar cataract', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'neuroimaging measurement'], ['thyroid cancer', 'neoplasm', 'heel bone mineral density', 'thyroid carcinoma', 'hair colour measurement'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'mathematical ability', 'non-small cell lung carcinoma'], ['Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'Familial prostate cancer', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma'], ['Capillary malformation - arteriovenous malformation', 'hydrops fetalis, nonimmune, and/or atrial septal defect, susceptibility to', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma'], ['Primary familial polycythemia', 'Blackfan-Diamond anemia', 'Autosomal dominant secondary polycythemia', 'hematocrit', 'hemoglobin measurement'], ['leukodystrophy, hypomyelinating, 15', 'heel bone mineral density', 'glomerular filtration rate', 'bone quantitative ultrasound measurement', 'reticulocyte count'], ['eosinophil peroxidase deficiency', 'eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'Neutropenia - monocytopenia - deafness'], ['psoriasis', 'ankylosing spondylitis', 'systolic blood pressure', 'endoplasmic reticulum aminopeptidase 1 measurement', 'platelet count'], ['non-small cell lung carcinoma', 'breast carcinoma', 'cancer', 'neoplasm', 'breast neoplasm'], ['Lethal congenital contracture syndrome type 2', 'gastric adenocarcinoma', 'cancer', 'neoplasm', 'bladder transitional cell carcinoma'], ['non-small cell lung carcinoma', 'breast carcinoma', 'neoplasm', 'amyotrophic lateral sclerosis', 'cancer'], ['papillary thyroid carcinoma', 'melanocytic neoplasm', 'breast ductal adenocarcinoma', 'Thyroid Gland Oncocytic Follicular Carcinoma', 'vital capacity'], ['Cockayne syndrome', 'Cockayne syndrome type 1', 'Cockayne syndrome type 2', 'Cockayne syndrome type 3', 'UV-sensitive syndrome'], ['cancer', 'endometrial cancer', 'lung adenocarcinoma', 'multiple myeloma', 'cutaneous melanoma'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'hemoglobin measurement', 'red blood cell distribution width'], ['type II diabetes mellitus', 'sex hormone-binding globulin measurement', "Crohn's disease", 'ulcerative colitis', 'ankylosing spondylitis'], ['heel bone mineral density', 'bone density', 'ulcer disease', 'bone quantitative ultrasound measurement', 'mean corpuscular hemoglobin'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'low grade glioma', 'blood protein measurement', 'endothelial cell-selective adhesion molecule measurement'], ['Roberts syndrome', 'Roberts-SC phocomelia syndrome', 'acute myeloid leukemia', 'endometrial cancer', 'body height'], ['S-formylglutathione hydrolase measurement', 'protein measurement', 'cerebrospinal fluid biomarker measurement', 'systolic blood pressure', 'platelet component distribution width'], ['endothelial cell-specific molecule 1 measurement', 'FEV/FEC ratio', 'neoplasm', 'prostate cancer', 'urinary bladder cancer'], ['breast carcinoma', 'breast cancer', 'breast neoplasm', 'neoplasm', 'breast adenocarcinoma'], ['schizophrenia', 'breast cancer', 'neoplasm', 'postmenopausal osteoporosis', 'osteoporosis'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Elevated circulating glutaric acid concentration', 'Glutaric aciduria', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'Multiple acyl-CoA dehydrogenation deficiency, mild type'], ['type II diabetes mellitus', 'lymphocyte count', 'platelet count', 'hair color', 'esophageal adenocarcinoma'], ['body mass index', 'high density lipoprotein cholesterol measurement', 'physical activity measurement', 'glomerular filtration rate', 'lean body mass'], ['body height', 'blood protein measurement', 'rheumatoid arthritis', 'refractory anemia', 'dilated cardiomyopathy'], ['mean corpuscular volume', 'eosinophil count', 'mean corpuscular hemoglobin', 'eosinophil percentage of leukocytes', 'benign neoplasm'], ['heart disease', 'mean corpuscular hemoglobin concentration', 'Cutaneous T-cell lymphoma', 'heel bone mineral density', 'bone density'], ['Pontocerebellar hypoplasia type 1', 'Non-syndromic pontocerebellar hypoplasia', 'Microcephaly', 'genetic disorder', 'Severe intrauterine growth retardation'], ['acute myeloid leukemia', 'multiple myeloma', 'B-cell non-Hodgkins lymphoma', 'unipolar depression', 'depressive disorder'], ['exostoses, multiple, type 1', 'Multiple osteochondromas', 'hereditary multiple exostoses', 'chondrosarcoma', 'Langer-Giedion syndrome'], ['immunoskeletal dysplasia with neurodevelopmental abnormalities', 'Skeletal dysplasia - intellectual disability', 'body height', 'white matter microstructure measurement', 'acute myeloid leukemia'], ['body height', 'waist-hip ratio', 'Mantle cell lymphoma', 'type II diabetes mellitus', 'cirrhosis of liver'], ['Weaver syndrome', 'lymphoma', 'non-Hodgkins lymphoma', 'melanoma', 'breast carcinoma'], ['Congenital factor II deficiency', 'stroke', 'venous thromboembolism', 'deep vein thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency'], ['myocardial infarction', 'peripheral arterial disease', 'stroke', 'coronary artery disease', 'reticulocyte count'], ['mean platelet volume', 'glioblastoma multiforme', 'proteinase-activated receptor 1 measurement', 'monocyte count', 'head and neck squamous cell carcinoma'], ['tissue factor measurement', 'D dimer measurement', 'coagulation factor measurement', 'cervical cancer', 'neutrophil count'], ['Congenital factor V deficiency', 'thrombophilia due to activated protein C resistance', 'Venous thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency', 'Sepsis'], ['Congenital factor VII deficiency', 'Abnormality of coagulation', 'factor VII deficiency', 'factor VII measurement', 'prothrombin time measurement'], ['Hemophilia A', 'Hemophilia B', 'Sepsis', 'Mild hemophilia A', 'Abnormality of coagulation'], ['Hemophilia B', 'Hemophilia A', 'Venous thrombosis', 'thrombophilia, X-linked, due to factor 9 defect', 'warfarin sensitivity, X-linked'], ['Congenital factor X deficiency', 'atrial fibrillation', 'venous thromboembolism', 'deep vein thrombosis', 'pulmonary embolism'], ['Congenital factor XI deficiency', 'Hereditary angioedema', 'factor XI measurement', 'Abnormal bleeding', 'venous thromboembolism'], ['Congenital factor XII deficiency', 'Hereditary angioedema', 'Hereditary angioedema type 3', 'Hereditary angioedema type 1', 'angioedema'], ['Autosomal recessive spastic paraplegia type 35', 'Hereditary spastic paraplegia', 'Spastic paraplegia', 'genetic disorder', 'Fatty acid hydroxylase-associated neurodegeneration'], ['migraine disorder', 'pain', 'Fever', 'osteoarthritis', 'Low back pain'], ['C-reactive protein measurement', 'serum alanine aminotransferase measurement', 'blood protein measurement', 'alkaline phosphatase measurement', 'hair color'], ['protein measurement', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract'], ['breast adenocarcinoma', 'gestational diabetes', 'neoplasm', 'hydrops fetalis (disease)', 'Hand-foot syndrome'], ['pathological myopia', 'hepatocellular carcinoma', 'neoplasm', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis'], ['FADD-related immunodeficiency', 'Abnormality of cardiovascular system morphology', 'genetic disorder', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['red blood cell distribution width', 'mean platelet volume', 'platelet count', 'chronic kidney disease', 'serum metabolite measurement'], ['red blood cell distribution width', 'blood metabolite measurement', 'mean platelet volume', 'chronic kidney disease', 'serum metabolite measurement'], ['hemoglobin measurement', 'mean corpuscular volume', 'hematocrit', 'Pentosuria', 'HbA1c measurement'], ['Lethal osteosclerotic bone dysplasia', 'Hypoplastic amelogenesis imperfecta', 'Isolated focal cortical dysplasia type II', 'Midline cerebral malformation', 'Neonatal death'], ['lymphocyte count', 'body height', 'basophil count', 'neutrophil count', 'balding measurement'], ['chronotype measurement', 'age at menarche', 'potassium measurement', 'adolescent idiopathic scoliosis', 'sleep duration'], ['neuroimaging measurement', 'brain volume measurement', 'brain measurement', 'cortical thickness', 'platelet count'], ['Hypomyelination - congenital cataract', 'Leukodystrophy', 'adolescent idiopathic scoliosis', 'Thrombocytopenia', 'attention deficit hyperactivity disorder'], ['Fanconi anemia', 'VACTERL with hydrocephalus', 'Fanconi anemia complementation group A', 'VACTERL association, X-linked, with or without hydrocephalus', 'acute myeloid leukemia'], ['Fanconi anemia', 'acute myeloid leukemia', 'Fanconi anemia complementation group A', 'leukemia', 'colorectal adenocarcinoma'], ['Fanconi anemia', 'Fanconi anemia complementation group L', 'Fanconi anemia complementation group A', 'acute myeloid leukemia', 'Abnormality of radial ray'], ['blind loop syndrome', 'leukocyte count', 'eosinophil count', 'autoimmune disease', 'glomerular filtration rate'], ['Autoimmune lymphoproliferative syndrome', 'cutaneous melanoma', 'adult lymphoma', "Hashimoto's thyroiditis", 'chronic lymphocytic leukemia'], ['Autoimmune lymphoproliferative syndrome', 'celiac disease', 'allergy', 'Eczema', 'neutrophil count'], ['autism spectrum disorder', 'esophageal squamous cell carcinoma', 'colorectal carcinoma', 'esophageal adenocarcinoma', 'Epileptic encephalopathy'], ['influenza', 'clear cell renal carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'gastric carcinoma'], ['Autosomal recessive cutis laxa type 1', 'Autosomal dominant cutis laxa', 'Hereditary sensorimotor neuropathy with hyperelastic skin', 'age-related macular degeneration', 'Charcot-Marie-Tooth disease'], ['Marfan syndrome', 'Isolated ectopia lentis', 'Geleophysic dysplasia', 'Acromicric dysplasia', 'Marfan syndrome type 1'], ['Short stature due to GHSR deficiency', 'cancer', 'pulse pressure measurement', 'breast cancer', 'schistosomiasis'], ['autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'ABCD syndrome', 'Waardenburg-Shah syndrome'], ['chronotype measurement', 'diet measurement', 'blood protein measurement', 'refractory anemia', 'rheumatoid arthritis'], ['chin morphology measurement', 'electrocardiography', 'hepatocellular carcinoma', 'bone mineral content measurement', 'cancer'], ['intellectual developmental disorder with dysmorphic facies and behavioral abnormalities', 'gastric adenocarcinoma', 'non-Hodgkins lymphoma', 'basal cell carcinoma', 'endometrial carcinoma'], ['Split hand-split foot malformation', 'Distal limb deficiencies - micrognathia syndrome', 'lung adenocarcinoma', 'body height', 'head and neck squamous cell carcinoma'], ['serum IgE measurement', 'CCL2 measurement', 'seasonal allergic rhinitis', 'growth-regulated alpha protein measurement', 'self reported educational attainment'], ['blood protein measurement', 'low affinity immunoglobulin epsilon Fc receptor measurement', 'cerebrospinal fluid biomarker measurement', 'neutrophil count', 'thyroid gland adenocarcinoma'], ['breast carcinoma', 'Autosomal agammaglobulinemia', 'Hyper-IgM syndrome type 4', 'Hyper-IgM syndrome type 2', 'Hyper-IgM syndrome type 5'], ['inflammatory bowel disease', 'cutaneous Leishmaniasis', 'ulcerative colitis', 'systemic lupus erythematosus', 'mucocutaneous lymph node syndrome'], ['systemic lupus erythematosus', 'breast ductal adenocarcinoma', 'head and neck squamous cell carcinoma', 'thyroid carcinoma', 'low affinity immunoglobulin gamma Fc region receptor II-a/b measurement'], ['inflammatory bowel disease', 'Hyper-IgM syndrome type 3', 'Autosomal agammaglobulinemia', 'Primary systemic amyloidosis', 'Neutrophil immunodeficiency syndrome'], ['autosomal recessive primary immunodeficiency with defective spontaneous natural killer cell cytotoxicity', 'cutaneous Leishmaniasis', 'Primary immunodeficiency', 'basophil count', 'hematocrit'], ['monocyte count', 'neutrophil count', 'basophil percentage of leukocytes', 'serum IgG measurement', 'hematocrit'], ['sex hormone-binding globulin measurement', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'red blood cell density measurement'], ['coronary artery disease', 'blood protein measurement', 'Graves disease', 'rheumatoid arthritis', 'multiple sclerosis'], ['rheumatoid arthritis', 'Graves disease', 'blood protein measurement', 'Fc receptor-like protein 3 measurement', 'protein measurement'], ['systemic lupus erythematosus', 'myeloid white cell count', 'adolescent idiopathic scoliosis', 'multiple myeloma', 'myeloid neoplasm'], ['lymphocyte count', 'erythrocyte count', 'blood protein measurement', 'monocyte count', 'lymphocyte percentage of leukocytes'], ['Disseminated superficial actinic porokeratosis', 'osteoporosis', 'postmenopausal osteoporosis', 'multiple myeloma', 'osteitis deformans'], ['Congenital adrenal hyperplasia', 'coronary artery disease', 'lean body mass', 'fat body mass', 'body height'], ['auditory neuropathy-optic atrophy syndrome', 'genetic disorder', 'Congenital adrenal hyperplasia', 'pancreatic carcinoma', 'low grade glioma'], ['Erythropoietic protoporphyria', 'protoporphyria, erythropoietic, 2', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume'], ['HIV infection', 'multiple myeloma', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'B-cell non-Hodgkins lymphoma'], ['body height', 'intraocular pressure measurement', 'vital capacity', 'forced expiratory volume', 'BMI-adjusted waist circumference'], ['head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'ovarian carcinoma', 'hemangioblastoma'], ['self reported educational attainment', 'fat body mass', 'body mass index', 'body fat percentage', 'lean body mass'], ['type II diabetes mellitus', 'diabetes mellitus', 'Abnormal blood glucose concentration', 'endometrial cancer', 'glioblastoma multiforme'], ['monocyte percentage of leukocytes', 'neutrophil count', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia'], ['Aarskog-Scott syndrome', 'Aarskog-Scott syndrome, X-linked', 'genetic disorder', 'X-linked intellectual disability, Siderius type', 'thyroid gland adenocarcinoma'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'waist-hip ratio'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Acrocephalosyndactyly'], ['Deafness with labyrinthine aplasia, microtia, and microdontia', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma'], ['cancer', 'bone development disease', 'Craniosynostosis', 'ovarian carcinoma', 'female reproductive endometrioid cancer'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Thyrotoxicosis'], ['hypogonadotropic hypogonadism 6 with or without anosmia', 'Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'Holoprosencephaly', 'cancer'], ['Multiple synostoses syndrome', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma'], ['Lacrimoauriculodentodigital syndrome', 'Aplasia of lacrimal and salivary glands', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer'], ['breast adenocarcinoma', 'endometrial cancer', 'glioblastoma multiforme', 'gastric adenocarcinoma', 'neoplasm'], ['cancer', 'bone development disease', 'lean body mass', 'body height', 'appendicular lean mass'], ['cancer', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'hair colour measurement'], ['diet measurement', 'energy intake', 'triglyceride measurement', 'Abnormal urine sodium concentration', 'protein intake measurement'], ['Autosomal dominant hypophosphatemic rickets', 'tumoral calcinosis, hyperphosphatemic, familial, 2', 'familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'cancer', 'bone development disease'], ['Hartsfield-Bixler-Demyer syndrome', 'hypogonadotropic hypogonadism 2 with or without anosmia', 'Kallmann syndrome', 'Osteoglophonic dwarfism', 'Pfeiffer syndrome'], ['Crouzon disease', 'Pfeiffer syndrome', 'Cutis gyrata - acanthosis nigricans - craniosynostosis', 'Apert syndrome', 'Jackson-Weiss syndrome'], ['Achondroplasia', 'Thanatophoric dysplasia type 1', 'Muenke syndrome', 'Severe achondroplasia - developmental delay - acanthosis nigricans', 'Crouzon syndrome - acanthosis nigricans'], ['rhabdomyosarcoma', 'breast carcinoma', 'cancer', 'bone development disease', 'idiopathic pulmonary fibrosis'], ['blood protein measurement', 'FEV/FEC ratio', 'self reported educational attainment', 'rheumatoid arthritis', 'refractory anemia'], ['Hereditary leiomyomatosis and renal cell cancer', 'Fumaric aciduria', 'Multiple cutaneous leiomyomas', 'Inherited cancer-predisposing syndrome', 'Endometrial Endometrioid Adenocarcinoma'], ['Reducing body myopathy', 'X-linked myopathy with postural muscle atrophy', 'Emery-Dreifuss muscular dystrophy', 'X-linked Emery-Dreifuss muscular dystrophy', 'Uruguay Faciocardiomusculoskeletal syndrome'], ['atopic eczema', 'chronic kidney disease', 'acute graft vs. host disease', 'kidney disease', 'renal cell carcinoma'], ['Familial atrial fibrillation', 'lifestyle measurement', 'Brugada syndrome', 'Familial short QT syndrome', 'Pallister-Hall syndrome'], ['cognitive function measurement', 'diet measurement', 'eosinophil count', 'lean body mass', 'body mass index'], ['testosterone measurement', 'sex hormone-binding globulin measurement', 'BMI-adjusted waist-hip ratio', 'vital capacity', 'hematocrit'], ['atopic eczema', 'Inherited ichthyosis', 'Eczema', 'ichthyosis vulgaris', 'genetic disorder'], ['Ewing sarcoma', 'Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'osteosarcoma', 'body height', 'platelet count'], ['response to statin', 'hair colour measurement', 'Flexion contracture', 'pathological myopia', 'neoplasm'], ['Periventricular nodular heterotopia', 'Osteodysplasty, Melnick-Needles type', 'Frontometaphyseal dysplasia', 'Otopalatodigital syndrome type 2', 'Otopalatodigital syndrome type 1'], ['Autosomal dominant Larsen syndrome', 'Spondylocarpotarsal synostosis', 'Atelosteogenesis type III', 'Atelosteogenesis type I', 'Boomerang dysplasia'], ['Muscle filaminopathy', 'hypertrophic cardiomyopathy', 'Familial isolated restrictive cardiomyopathy', 'Distal myopathy with posterior leg and anterior hand involvement', 'dilated cardiomyopathy'], ['lung adenocarcinoma', 'mathematical ability', 'cognitive function measurement', 'Niemann-Pick disease type C', 'neoplasm'], ['renal cell carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'Gastrointestinal stromal tumor', 'idiopathic pulmonary fibrosis'], ['acute myeloid leukemia', 'hepatocellular carcinoma', 'acute lymphoblastic leukemia', 'renal cell carcinoma', 'cancer'], ['cancer', 'monocyte count', 'monocyte percentage of leukocytes', 'thyroid gland adenocarcinoma', 'neoplasm'], ['Milroy disease', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'Gastrointestinal stromal tumor', 'thyroid carcinoma'], ['Posterior column ataxia - retinitis pigmentosa', 'Retinitis pigmentosa', 'genetic disorder', 'Retinal dystrophy', 'Hereditary sensory and autonomic neuropathy'], ['Fowler syndrome', 'Posterior column ataxia - retinitis pigmentosa', 'multiple sclerosis', 'marginal zone B-cell lymphoma', 'small cell lung carcinoma'], ['hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'hair shape measurement', 'red blood cell density measurement'], ['Fragile X syndrome', 'Fragile X-associated tremor/ataxia syndrome', 'primary ovarian insufficiency', 'Intellectual disability', 'mental retardation'], ["Spondylometaphyseal dysplasia, 'corner fracture' type", 'Fibronectin glomerulopathy', 'cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy'], ['intraocular pressure measurement', 'appendicular lean mass', 'red blood cell distribution width', 'body height', 'lean body mass'], ['triglyceride measurement', 'intraocular pressure measurement', 'blood protein measurement', 'chronic kidney disease', 'urinary metabolite measurement'], ['Neurodegenerative syndrome due to cerebral folate transport deficiency', 'Seizure', 'ovarian cancer', 'neoplasm', 'Epileptic encephalopathy'], ['body height', "Barrett's esophagus", 'neoplasm', 'rheumatoid arthritis', 'Acute hepatic failure'], ['Berardinelli-Seip congenital lipodystrophy', "Crohn's disease", 'heel bone mineral density', 'inflammatory bowel disease', 'glioblastoma multiforme'], ['Bamforth-Lazarus syndrome', 'Familial papillary or follicular thyroid carcinoma', 'follicular thyroid carcinoma', 'Congenital hypothyroidism', 'choanal atresia'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'appendicular lean mass', 'mean corpuscular hemoglobin concentration'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'low grade glioma', 'mean corpuscular hemoglobin concentration', 'chronic lymphocytic leukemia'], ['sex hormone-binding globulin measurement', 'appendicular lean mass', 'mathematical ability', 'vital capacity', 'body height'], ['mental retardation with language impairment', 'Intellectual disability-severe speech delay-mild dysmorphism syndrome', 'basal cell carcinoma', 'skin carcinoma', 'esophageal adenocarcinoma'], ['Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'type I diabetes mellitus', 'hydrops fetalis (disease)', 'azoospermia', 'Fanconi anemia'], ['periodontitis', 'ovarian cancer', 'pancreatic adenocarcinoma', 'eosinophil count', 'lymphocyte count'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'prostate adenocarcinoma'], ['self reported educational attainment', 'appendicular lean mass', 'body height', 'glomerular filtration rate', 'follistatin measurement'], ['body height', 'blood protein measurement', 'appendicular lean mass', 'refractive error measurement', 'neoplasm'], ['FTH1-related iron overload', 'Autosomal recessive bestrophinopathy', 'Hemochromatosis type 3', 'squamous cell lung carcinoma', 'red blood cell distribution width'], ['Hereditary hyperferritinemia with congenital cataracts', 'Neuroferritinopathy', 'L-ferritin deficiency', 'movement disorder', 'Genetic hyperferritinemia without iron overload'], ['beta-amyloid 1-42 measurement', 'cerebrospinal fluid biomarker measurement', 'left inferior lateral ventricle volume measurement', 'systolic blood pressure', 'Young adult-onset Parkinsonism'], ['Lethal polymalformative syndrome, Boissel type', 'obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'Genetic obesity', 'type II diabetes mellitus'], ['X-linked non-syndromic intellectual disability', 'mental retardation', 'genetic disorder', 'Intellectual disability', 'non-small cell lung carcinoma'], ['hematocrit', 'hemoglobin measurement', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'myeloid white cell count'], ['chronic lymphocytic leukemia', 'glioma', 'anaplastic oligodendroglioma', 'gastric carcinoma', 'hemangioblastoma'], ['anthrax infection', 'COVID-19', 'HIV infection', 'ovarian cancer', 'lung adenocarcinoma'], ['Frontotemporal dementia with motor neuron disease', 'sporadic amyotrophic lateral sclerosis', 'amyotrophic lateral sclerosis', 'essential tremor', 'Juvenile amyotrophic lateral sclerosis'], ['urinary albumin to creatinine ratio', 'albuminuria', 'alkaline phosphatase measurement', 'breakfast skipping measurement', 'diet measurement'], ['galactoside 34-L-fucosyltransferase measurement', 'e-selectin measurement', 'red blood cell distribution width', 'galectin-3 measurement', 'cancer antigen 19.9 measurement'], ['congenital disorder of glycosylation with defective fucosylation', 'Intellectual disability', 'Seizure', 'COVID-19', 'genetic disorder'], ['Friedreich ataxia', 'hereditary ataxia', 'Abnormality of the cardiovascular system', 'Headache', 'hypertrophic cardiomyopathy'], ['hematocrit', 'hemoglobin measurement', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'hypertrophic cardiomyopathy 20'], ['Hereditary thrombocytopenia with normal platelets', 'thrombocytopenia 3', 'Autosomal thrombocytopenia with normal platelets', 'mean corpuscular volume', 'blood platelet disease'], ['cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cutaneous Leishmaniasis', 'neoplasm'], ['Familial exudative vitreoretinopathy', 'Coats disease', 'Persistent hyperplastic primary vitreous', 'Retinal dystrophy', 'Norrie disease'], ['Glycogen storage disease due to glucose-6-phosphatase deficiency type a', 'Glycogen storage disease due to glucose-6-phosphatase deficiency', 'Glycogen storage disease', 'genetic disorder', 'Short stature due to GHSR deficiency'], ['anemia, nonspherocytic hemolytic, due to G6PD deficiency', 'anemia (phenotype)', 'class I glucose-6-phosphate dehydrogenase deficiency', 'congenital nonspherocytic hemolytic anemia', 'glucosephosphate dehydrogenase deficiency'], ['Glycogen storage disease due to acid maltase deficiency', 'Glycogen storage disease due to acid maltase deficiency, adult onset', 'Glycogen storage disease due to acid maltase deficiency, infantile onset', 'type II diabetes mellitus', 'Glycogen storage disease'], ['neoplasm', 'cancer', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'chronic kidney disease'], ['Inherited congenital spastic tetraplegia', 'developmental and epileptic encephalopathy', 'sex hormone-binding globulin measurement', 'cognitive function measurement', 'mathematical ability'], ['arterial stiffness measurement', 'adolescent idiopathic scoliosis', 'thyroid gland adenocarcinoma', 'cutaneous melanoma', 'obesity'], ['body mass index', 'body fat percentage', 'vital capacity', 'cleft lip', 'forced expiratory volume'], ['glomerular filtration rate', 'blood metabolite measurement', 'appendicular lean mass', 'body height', 'acute myeloid leukemia'], ["Parkinson's disease", 'triglyceride measurement', 'glioblastoma multiforme', 'high density lipoprotein cholesterol measurement', 'Ehlers-Danlos syndrome, progeroid type'], ['temporal lobe epilepsy', 'body height', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'Seizure'], ['white matter microstructure measurement', 'neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'sporadic Creutzfeld Jacob disease'], ['Krabbe disease', 'Seizure', 'body mass index', 'genetic disorder', 'Adult Krabbe disease'], ['head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'mononeuropathy', 'colitis', 'Severe combined immunodeficiency due to CORO1A deficiency'], ['heel bone mineral density', 'colitis', 'Homozygous familial hypercholesterolemia', 'sitting height measurement', 'bone quantitative ultrasound measurement'], ['endometrial cancer', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'neoplasm', 'non-small cell lung carcinoma', 'amyotrophic lateral sclerosis'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2D', 'Distal hereditary motor neuropathy type 5', 'Charcot-Marie-Tooth disease', 'Distal hereditary motor neuropathy type 2', 'spinal muscular atrophy, infantile, James type'], ['triglyceride measurement', 'red blood cell distribution width', 'low density lipoprotein cholesterol measurement', 'erythrocyte count', 'hemoglobin measurement'], ['Hypoparathyroidism - deafness - renal disease', 'breast adenocarcinoma', 'breast carcinoma', 'asthma', 'eosinophil count'], ['Interauricular communication', 'Ventricular septal defect', 'Single ventricular septal defect', 'Tetralogy of Fallot', 'Genetic cardiac anomaly'], ['Gaucher disease', 'Gaucher disease type 1', 'Fetal Gaucher disease', 'Gaucher disease type 2', 'Gaucher disease type 3'], ['neoplasm', 'infection', 'infect', 'hepatocellular carcinoma', 'glioma'], ['type II diabetes mellitus', 'hypothyroidism', 'psoriasis', 'lymphocyte count', 'leukocyte count'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'medulloblastoma', 'myeloid white cell count', 'hepatocellular carcinoma'], ['type I diabetes mellitus', 'infertility', 'diabetes mellitus', 'Hypoglycemia', 'obesity'], ['triglyceride measurement', 'neoplasm', 'nasopharyngeal squamous cell carcinoma', 'nasopharyngeal neoplasm', 'lung cancer'], ['MODY', 'Permanent neonatal diabetes mellitus', 'Hyperinsulinism due to glucokinase deficiency', 'type II diabetes mellitus', 'DEND syndrome'], ['type II diabetes mellitus', 'triglyceride measurement', 'lean body mass', 'total cholesterol measurement', 'platelet count'], ['Gamma-glutamylcysteine synthetase deficiency', 'hemolytic anemia', 'monocyte count', 'monocyte percentage of leukocytes', 'low density lipoprotein cholesterol measurement'], ['glioblastoma multiforme', 'colorectal carcinoma', 'gut microbiome measurement', 'taxonomic microbiome measurement', 'polymyalgia rheumatica'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'colorectal neoplasm', 'erythrocyte count'], ['Colobomatous microphthalmia', 'Isolated anophthalmia - microphthalmia', 'Isolated Klippel-Feil syndrome', 'microphthalmia', 'neoplasm'], ['prostate carcinoma', 'hemorrhoid', 'pulse pressure measurement', 'esophageal adenocarcinoma', "Barrett's esophagus"], ['Orofacial clefting syndrome', 'orofacial cleft', 'chronic obstructive pulmonary disease', 'stroke', 'Hepatic fibrosis'], ['growth differentiation factor 15 measurement', 'coffee consumption measurement', 'hyperemesis gravidarum', 'nausea and vomiting of pregnancy severity measurement', 'obesity'], ['Hirschsprung disease', 'Ondine syndrome', 'Haddad syndrome', 'prostate carcinoma', 'blood protein measurement'], ['systolic blood pressure', 'coronary artery disease', 'pulse pressure measurement', 'alcohol drinking', 'systemic lupus erythematosus'], ['Alexander disease', 'Autosomal dominant cerebellar ataxia', 'Seizure', 'Alexander disease type I', 'Alexander disease type II'], ['Hepatoencephalopathy due to combined oxidative phosphorylation deficiency type 1', 'combined oxidative phosphorylation deficiency', 'systolic blood pressure', 'blood protein measurement', 'PR interval'], ['Congenital myasthenic syndromes', 'Congenital myasthenic syndromes with glycosylation defect', 'congenital disorder of glycosylation type I', 'vertebral disease', 'musculoskeletal system disease'], ['airway imaging measurement', 'BMI-adjusted waist circumference', 'body height', 'appendicular lean mass', 'lean body mass'], ['heel bone mineral density', 'body mass index', 'chronotype measurement', 'blood protein measurement', 'arterial stiffness measurement'], ['Non-acquired isolated growth hormone deficiency', 'Short stature due to growth hormone qualitative anomaly', 'Isolated growth hormone deficiency type IB', 'Isolated growth hormone deficiency type II', 'Isolated growth hormone deficiency type IA'], ['Laron syndrome', 'Short stature due to GHSR deficiency', 'pituitary dwarfism', 'Short stature due to partial GHR deficiency', 'acromegaly'], ['obesity', 'ovarian cancer', 'breast adenocarcinoma', 'self reported educational attainment', 'head and neck squamous cell carcinoma'], ['Short stature due to GHSR deficiency', 'lean body mass', 'body height', 'refractive error measurement', 'appendicular lean mass'], ['appendicular lean mass', 'sitting height measurement', 'type II diabetes mellitus', 'systolic blood pressure', 'thyroid gland adenocarcinoma'], ['body mass index', 'lean body mass', 'body fat percentage', 'fat body mass', 'waist circumference'], ['Cataract-microcornea syndrome', 'Non-syndromic congenital cataract', 'Pulverulent cataract', 'cataract', 'Nuclear cataract'], ['Autosomal recessive spastic paraplegia type 44', 'Pelizaeus-Merzbacher-like disease due to GJC2 mutation', 'Milroy disease', 'Pelizaeus-Merzbacher-like disease', 'Spastic paraplegia'], ['Muscular dystrophy', 'gastric cancer', 'gastric carcinoma', 'neoplasm', 'body height'], ['Fabry disease', 'cardiomyopathy', 'hypertrophic cardiomyopathy', 'response to deoxygalactonojirimycin', 'Abnormality of the cardiovascular system'], ['GM1 gangliosidosis type 2', 'GM1 gangliosidosis type 1', 'GM1 gangliosidosis type 3', 'Mucopolysaccharidosis type 4B', 'Mucopolysaccharidosis type 4'], ['Glycine encephalopathy', 'genetic disorder', 'Smith-Magenis syndrome', 'eosinophil count', 'Atypical glycine encephalopathy'], ['Holoprosencephaly', 'postaxial polydactyly-anterior pituitary anomalies-facial dysmorphism syndrome', 'Pallister-Hall syndrome', 'Microform holoprosencephaly', 'genetic disorder'], ['Glomuvenous malformation', 'Blue rubber bleb nevus', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells'], ['type II diabetes mellitus', 'diabetes mellitus', 'obesity', 'type I diabetes mellitus', 'diabetic nephropathy'], ['Hereditary hyperekplexia', 'pain', 'coronary artery disease', 'ischemia reperfusion injury', 'kidney failure'], ['global developmental delay, progressive ataxia, and elevated glutamine', 'Early infantile epileptic encephalopathy', 'reticulocyte count', 'mean platelet volume', 'BMI-adjusted waist-hip ratio'], ['amino acid measurement', 'glutamine measurement', 'blood metabolite measurement', 'serum metabolite measurement', 'head and neck squamous cell carcinoma'], ['body mass index', 'pulse pressure measurement', 'cardiovascular disease', 'eosinophil count', 'platelet count'], ['Hyperinsulinism-hyperammonemia syndrome', 'epilepsy', 'gout', 'uric acid measurement', 'hemoglobin measurement'], ['Congenital brain dysgenesis due to glutamine synthetase deficiency', 'Seizure', 'Red cell distribution width', 'lymphocyte count', 'high density lipoprotein cholesterol measurement'], ['body height', 'hematocrit', 'hemoglobin measurement', 'intraocular pressure measurement', 'lean body mass'], ['Sturge-Weber syndrome', 'Familial multiple nevi flammei', 'cutaneous melanoma', 'Uveal Melanoma', 'melanoma'], ['mean platelet volume', 'erosive tooth wear measurement', 'childhood onset asthma', 'eosinophil count', 'DNA methylation'], ['Mucolipidosis type II', 'Mucolipidosis type III', 'Mucolipidosis', 'Sanfilippo syndrome type A', 'Joubert syndrome and related disorders'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Isolated congenital hypogonadotropic hypogonadism', 'age at menopause', 'Isolated thyroid-stimulating hormone deficiency', 'lean body mass'], ['Normosmic congenital hypogonadotropic hypogonadism', 'hypogonadotropic hypogonadism', 'prostate adenocarcinoma', 'prostate cancer', 'infertility'], ['self reported educational attainment', 'neoplasm', 'Cognitive impairment', 'cognitive decline measurement', 'Pallister-Hall syndrome'], ['Early infantile epileptic encephalopathy', 'Failure to thrive', 'Seizure', 'Feeding difficulties', 'microcephaly (disease)'], ['Bernard-Soulier syndrome', 'Pseudo-von Willebrand disease', 'Macrothrombocytopenia', 'Optic neuropathy', 'Autosomal dominant macrothrombocytopenia'], ['body mass index', 'type II diabetes mellitus', 'lean body mass', 'fat body mass', 'insulin sensitivity measurement'], ['platelet component distribution width', 'platelet count', 'mean platelet volume', 'lung adenocarcinoma', 'cutaneous melanoma'], ['prostate carcinoma', "Barrett's esophagus", 'lactose intolerance adult type', 'Autosomal recessive early-onset inflammatory bowel disease', 'Microvillus inclusion disease'], ['red blood cell distribution width', 'glioblastoma multiforme', 'monocyte count', 'neutrophil count', 'leukocyte count'], ['protein measurement', 'body mass index', 'balding measurement', 'cutaneous melanoma', 'lung adenocarcinoma'], ['Simpson-Golabi-Behmel syndrome', 'Nephroblastoma', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma', 'kidney neoplasm'], ['reticulocyte count', 'reticulocyte measurement', 'ovarian cancer', 'prostate carcinoma', 'head and neck squamous cell carcinoma'], ['Hemolytic anemia due to glucophosphate isomerase deficiency', 'hemolytic anemia', 'endometrial cancer', 'breast adenocarcinoma', 'ovarian cancer'], ['alkaline phosphatase measurement', 'mean corpuscular hemoglobin', 'liver enzyme measurement', 'scrapie', 'Abnormal sperm morphology'], ['amyloidosis cutis dyschromia', 'Familial primary localized cutaneous amyloidosis', 'blood protein measurement', "Parkinson's disease", 'cerebrospinal fluid biomarker measurement'], ['ovarian cancer', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'Fibronectin glomerulopathy'], ['refractory anemia', 'rheumatoid arthritis', 'cancer', 'Autosomal agammaglobulinemia', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'myeloid white cell count', 'cutaneous melanoma', 'prostate adenocarcinoma'], ['systolic blood pressure', 'FEV/FEC ratio', 'body height', 'neutrophil count', 'platelet count'], ['neuroblastoma', 'acne', 'periodontitis', 'gastric carcinoma', 'hepatocellular carcinoma'], ['diet measurement', 'reticulocyte count', 'acute myeloid leukemia', 'prostate adenocarcinoma', 'breast cancer'], ['platelet count', 'blood protein measurement', 'mean corpuscular volume', 'Sepsis', "Crohn's disease"], ['endometrial cancer', 'prostate adenocarcinoma', 'attention deficit hyperactivity disorder', 'Hyperinsulinism due to INSR deficiency', 'Autosomal recessive hyperinsulinism due to Kir6.2 deficiency'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'low grade glioma', 'prostate adenocarcinoma'], ['Hyperinsulinism due to INSR deficiency', 'Hyperinsulinism due to glucokinase deficiency', 'Exercise-induced hyperinsulinism', 'Hyperinsulinism due to short chain 3-hydroxylacyl-CoA dehydrogenase deficiency', 'Autosomal recessive hyperinsulinism due to Kir6.2 deficiency'], ['Headache', 'migraine disorder', 'fat body mass', 'hypertrophic cardiomyopathy', 'lean body mass'], ['monocyte count', 'base metabolic rate measurement', 'Benign familial chorea', 'refractive error measurement', 'body weight'], ['sex hormone-binding globulin measurement', 'hip bone mineral density', 'blood protein measurement', 'Sepsis', 'smoking status measurement'], ['triglyceride measurement', 'alkaline phosphatase measurement', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement', 'hair color'], ['heel bone mineral density', 'high density lipoprotein cholesterol measurement', 'hepatocellular carcinoma', 'mean platelet volume', 'eosinophil percentage of leukocytes'], ['sleep duration', 'low density lipoprotein cholesterol measurement', 'blood protein measurement', 'Adult-onset autosomal recessive sideroblastic anemia', 'Hemoglobin E - beta-thalassemia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'cutaneous melanoma', 'erythrocyte count'], ['Chudley-McCullough syndrome', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Rare genetic deafness', 'deafness', 'nonsyndromic deafness'], ['mean corpuscular hemoglobin concentration', 'neoplasm', 'periodontitis', 'self reported educational attainment', 'intelligence'], ['Spondylometaphyseal dysplasia, Sedaghatian type', 'eosinophil count', 'eosinophil percentage of leukocytes', "Crohn's disease", 'neutrophil percentage of granulocytes'], ['thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'lung adenocarcinoma'], ['Hereditary mixed polyposis syndrome', 'colorectal cancer', 'mean corpuscular volume', 'colorectal adenoma', 'mean corpuscular hemoglobin'], ['tooth agenesis', 'FEV/FEC ratio', 'heel bone mineral density', 'bone density', 'bone quantitative ultrasound measurement'], ['Van der Woude syndrome', 'Cleft palate', 'cleft lip', 'genetic disorder', 'head and neck squamous cell carcinoma'], ['neurodevelopmental disorder with language impairment and behavioral abnormalities', 'epilepsy', 'migraine disorder', 'obesity', 'alcohol dependence'], ['neurodevelopmental disorder with or without seizures and gait abnormalities', 'obesity', 'epilepsy', 'migraine disorder', 'alcohol dependence'], ['epilepsy', 'migraine disorder', 'obesity', 'alcohol dependence', 'post-traumatic stress disorder'], ['neurodevelopmental disorder with or without hyperkinetic movements and seizures, autosomal dominant', 'Autosomal dominant non-syndromic intellectual disability', 'Epileptic encephalopathy', 'Intellectual disability', "Parkinson's disease"], ['West syndrome', 'Autosomal dominant non-syndromic intellectual disability', 'Epileptic encephalopathy', 'autism', 'complex neurodevelopmental disorder'], ['protein measurement', 'rheumatic fever', 'osteosarcoma', 'reticulocyte count', 'pulse pressure measurement'], ['unipolar depression', 'hair colour measurement', 'neuroticism measurement', 'depressive symptom measurement', 'wellbeing measurement'], ['Frontotemporal dementia', 'Grn-related frontotemporal lobar degeneration with Tdp43 inclusions', 'Neuronal ceroid lipofuscinosis', 'CLN11 disease', 'Progressive non-fluent aphasia'], ['Short stature-auditory canal atresia-mandibular hypoplasia-skeletal anomalies syndrome', 'acute myeloid leukemia', 'nose morphology measurement', 'Cleft palate', 'cleft lip'], ['bipolar disorder', 'depressive disorder', 'bipolar I disorder', 'cancer', 'major depressive disorder'], ['Familial amyloidosis, Finnish type', 'AL amyloidosis', 'amyloidosis', 'psoriasis', 'hemoglobin measurement'], ['Hemolytic anemia due to glutathione reductase deficiency', 'multiple myeloma', 'glioblastoma multiforme', 'neoplasm', 'medulloblastoma'], ['glioblastoma multiforme', 'blood protein measurement', 'Dent disease type 1', 'Immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with DGKE deficiency'], ['HIV infection', 'endometrial cancer', 'glioblastoma multiforme', 'ovarian cancer', 'squamous cell lung carcinoma'], ['lean body mass', 'body fat percentage', 'fat body mass', 'body mass index', 'red blood cell distribution width'], ['testosterone measurement', 'prostate carcinoma', 'creatinine measurement', 'glomerular filtration rate', 'Ringed hair disease'], ['cutaneous melanoma', 'apolipoprotein A 1 measurement', 'Fibronectin glomerulopathy', 'inflammatory bowel disease', 'irritable bowel syndrome'], ['Intestinal obstruction in the newborn due to guanylate cyclase 2C deficiency', 'Chronic diarrhea due to guanylate cyclase 2C overactivity', 'Diarrhea', 'irritable bowel syndrome', 'Constipation'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Central areolar choroidal dystrophy', 'Congenital stationary night blindness', 'Retinal dystrophy'], ['mean corpuscular hemoglobin concentration', 'malaria', 'Hereditary elliptocytosis', 'atrial fibrillation', 'erythrocyte count'], ['Vitiligo', 'blood protein measurement', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['infectious disease', 'viral disease', 'breast carcinoma', 'melanoma', 'cancer'], ['infectious disease', 'viral disease', 'breast carcinoma', 'melanoma', 'cancer'], ['cancer', 'melanoma', 'breast carcinoma', 'neoplasm', 'hepatocellular carcinoma'], ['infectious disease', 'viral disease', 'breast carcinoma', 'cancer', 'melanoma'], ['Hyperandrogenism due to cortisone reductase deficiency', 'heel bone mineral density', 'body height', 'anthropometric measurement', 'blood protein measurement'], ['Histidinemia', 'vitamin D measurement', 'sunburn', 'urinary metabolite measurement', 'chronic kidney disease'], ['Hemochromatosis type 2', 'hemochromatosis type 2B', 'Rare hereditary hemochromatosis', 'hereditary hemochromatosis', 'erythrocyte count'], ['calcium oxalate urolithiasis', 'colorectal cancer', 'protein measurement', 'traffic air pollution measurement', 'Primary hyperoxaluria type 1'], ['folding of antihelix', 'outer ear morphology trait', 'antitragus size', 'balding measurement', 'lobe attachment'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Usher syndrome', 'Usher syndrome type 3', 'Dysarthria', 'mental retardation'], ['balding measurement', 'hematocrit', 'body height', 'red blood cell density measurement', 'erythrocyte count'], ['low density lipoprotein cholesterol measurement', 'blood protein measurement', 'kidney injury molecule 1 measurement', 'neutrophil count', 'lymphocyte count'], ['Subcutaneous Panniculitis-Like T-Cell Lymphoma', 'hepatitis A virus cellular receptor 2 measurement', 'blood protein measurement', 'inflammatory bowel disease', 'neoplasm'], ['Sickle cell anemia', 'Beta-thalassemia intermedia', 'Beta-thalassemia major', 'Beta-thalassemia', 'Heinz body anemia'], ['cancer', 'diphtheria', 'white matter microstructure measurement', 'creatine kinase measurement', 'lung adenocarcinoma'], ['Hemoglobinopathy Toms River', 'cyanosis, transient neonatal', 'fetal hemoglobin measurement', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hereditary persistence of fetal hemoglobin - sickle cell disease'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width', 'mean corpuscular volume'], ['glioblastoma multiforme', 'high density lipoprotein cholesterol measurement', 'waist-hip ratio', 'Red cell distribution width', 'red blood cell distribution width'], ['Methylmalonic acidemia with homocystinuria, type cblX', 'mental retardation', 'genetic disorder', 'X-linked intellectual disability', 'X-linked non-syndromic intellectual disability'], ['PR interval', 'central corneal thickness', 'reticulocyte count', 'appendicular lean mass', 'corneal topography'], ['inflammatory bowel disease', 'mean corpuscular hemoglobin concentration', 'mean reticulocyte volume', 'mean corpuscular volume', 'neoplasm'], ['Early infantile epileptic encephalopathy', 'Generalized epilepsy with febrile seizures-plus', 'undetermined early-onset epileptic encephalopathy', 'developmental and epileptic encephalopathy', 'breast carcinoma'], ['Familial sick sinus syndrome', 'atrial fibrillation', 'Brugada syndrome', 'heart failure', 'cardiac arrhythmia'], ['narcolepsy with cataplexy', 'breast adenocarcinoma', 'lung adenocarcinoma', "Alzheimer's disease", 'unipolar depression'], ['insomnia', 'Sleep Disorder', 'panic disorder', 'post-traumatic stress disorder', 'intelligence'], ['insomnia', 'sleep duration', 'Sleep Disorder', 'circadian rhythm', 'panic disorder'], ['pancreatic adenocarcinoma', 'Intermediate osteopetrosis', 'Wiskott-Aldrich syndrome', 'Combined immunodeficiency with skin granulomas', 'Craniodiaphyseal dysplasia'], ['neoplasm', 'multiple myeloma', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['2q37 microdeletion syndrome', 'multiple myeloma', 'neoplasm', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma'], ['neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['X-linked dominant chondrodysplasia, Chassaing-Lacombe type', 'neoplasm', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'unspecified peripheral T-cell lymphoma'], ['neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['Cornelia de Lange syndrome', 'Wilson-Turner syndrome', 'neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma'], ['neoplasm', 'multiple myeloma', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['blood protein measurement', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement', 'platelet count', 'platelet crit'], ['hemoglobin measurement', 'red blood cell density measurement', 'visceral adipose tissue measurement', 'hematocrit', 'Helicobacter pylori infectious disease'], ['infantile epileptic encephalopathy', 'Blackfan-Diamond anemia', 'breast cancer', 'Alpha-thalassemia - myelodysplastic syndrome', 'infect'], ['Septo-optic dysplasia', 'Combined pituitary hormone deficiencies, genetic forms', 'pituitary hormone deficiency, combined, 1', 'Pituitary stalk interruption syndrome', 'Kallmann syndrome'], ['lean body mass', 'fat body mass', 'systolic blood pressure', 'insomnia', 'appendicular lean mass'], ['mesenchymal chondrosarcoma', 'dedifferentiated chondrosarcoma', 'head and neck squamous cell carcinoma', 'hepatocellular carcinoma', 'esophageal squamous cell carcinoma'], ['hereditary hemochromatosis type 1', 'hereditary hemochromatosis', 'serum iron measurement', 'transferrin measurement', 'Familial porphyria cutanea tarda'], ['primary ovarian insufficiency', 'red blood cell distribution width', 'premature menopause', 'eosinophil count', 'monocyte count'], ['Alkaptonuria', 'Fuchs endothelial corneal dystrophy', 'Methylmalonic aciduria due to transcobalamin receptor defect', 'Posterior polymorphous corneal dystrophy', 'Pentosuria'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'hepatocyte growth factor measurement', 'cancer', 'deafness', 'platelet count'], ['platelet count', 'low density lipoprotein cholesterol measurement', 'testosterone measurement', 'triglyceride measurement', 'alcohol consumption measurement'], ['Tuberculosis', 'listeriosis', 'breast adenocarcinoma', 'low grade glioma', 'endometrial cancer'], ['Non-spherocytic hemolytic anemia due to hexokinase deficiency', 'Retinitis pigmentosa', 'Charcot-Marie-Tooth disease type 4G', 'reticulocyte count', 'hematocrit'], ['HIV infection', 'squamous cell lung carcinoma', 'nasopharyngeal neoplasm', 'melanocytic neoplasm', 'cutaneous melanoma'], ['Stevens-Johnson syndrome', 'spondyloarthropathy', 'head and neck squamous cell carcinoma', 'asthma', 'sex hormone-binding globulin measurement'], ['psoriasis', 'mean corpuscular volume', 'basophil count', 'BMI-adjusted waist-hip ratio', 'smoking behavior'], ['type I diabetes mellitus', 'diabetic retinopathy', 'diabetic maculopathy', 'diabetic ketoacidosis', 'complication'], ['reactive arthritis', 'Sarcoidosis', 'membranous glomerulonephritis', 'susceptibility to Mycobacterium tuberculosis infection measurement', 'wellbeing measurement'], ['anti-neutrophil antibody associated vasculitis', 'hepatitis B virus infection', 'chronic hepatitis B virus infection', 'cervical carcinoma', 'keratinocyte carcinoma'], ['hepatitis B virus infection', 'chronic hepatitis B virus infection', 'allergic rhinitis', 'Eczema', 'dermatomyositis'], ['testosterone measurement', 'autoimmune thyroid disease', 'hemoglobin measurement', 'asthma', 'rheumatoid arthritis'], ['rheumatoid arthritis', 'multiple sclerosis', 'Sarcoidosis', 'systemic lupus erythematosus', "Crohn's disease"], ['platelet crit', 'hemoglobin measurement', 'thyroid gland adenocarcinoma', 'self reported educational attainment', 'mucocutaneous lymph node syndrome'], ['eosinophil percentage of leukocytes', 'smoking status measurement', 'schizophrenia', 'pemphigus foliaceus', 'bipolar disorder'], ['urate measurement', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'esophageal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['diastolic blood pressure', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement', 'triglyceride measurement', 'neoplasm'], ['Acute intermittent porphyria', 'Abdominal pain', 'Vomiting', 'Visual loss', 'Acute episodes of neuropathic symptoms'], ['mean corpuscular volume', 'eosinophil count', 'Primary immunodeficiency', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['heel bone mineral density', 'self reported educational attainment', 'body height', 'systemic lupus erythematosus', 'peak expiratory flow'], ['BMI-adjusted waist circumference', 'monocyte percentage of leukocytes', 'mean reticulocyte volume', 'neutrophil percentage of leukocytes', 'monocyte count'], ['heme oxygenase 1 deficiency', 'amyloidosis', 'glioblastoma multiforme', 'gastric adenocarcinoma', 'chronotype measurement'], ['MODY', 'type II diabetes mellitus', 'hepatocellular adenoma', 'type I diabetes mellitus', 'monogenic diabetes'], ['Renal cysts and diabetes syndrome', 'type II diabetes mellitus', 'renal cell adenocarcinoma', 'Abnormal renal morphology', 'permanent neonatal diabetes mellitus'], ['chronotype measurement', 'mathematical ability', 'sex hormone-binding globulin measurement', 'body fat percentage', 'self reported educational attainment'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'Non-syndromic genetic deafness', 'FEV/FEC ratio', 'autosomal dominant nonsyndromic deafness'], ['lung adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'lung carcinoma', 'acute myeloid leukemia'], ['Synpolydactyly type 1', 'Syndactyly type 5', 'Brachydactyly', 'Brachydactyly type E', 'Brachydactyly-syndactyly, Zhao type'], ['anhaptoglobinemia', 'cutaneous melanoma', 'endometrial cancer', 'blood protein measurement', 'head and neck squamous cell carcinoma'], ['Tyrosinemia type 3', 'Hawkinsinuria', 'tyrosinemia', 'Hypertyrosinemia', 'Alkaptonuria'], ['age at menarche', 'mean platelet volume', 'diastolic blood pressure', 'heel bone mineral density', 'bone quantitative ultrasound measurement'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'Hypercholesterolemia', 'haptoglobin measurement', 'type II diabetes mellitus'], ['Lesch-Nyhan syndrome', 'Hypoxanthine guanine phosphoribosyltransferase partial deficiency', 'gout', 'genetic disorder', 'Microcephaly'], ['Hermansky-Pudlak syndrome', 'Hermansky-Pudlak syndrome with pulmonary fibrosis', 'pulmonary fibrosis', 'Oculocutaneous albinism', 'inflammatory bowel disease'], ['hepatocellular carcinoma', 'blood protein measurement', 'lung cancer', 'neoplasm', 'breast cancer'], ['squamous cell lung carcinoma', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'pes'], ['Costello syndrome', 'Noonan syndrome', 'bladder carcinoma', 'follicular thyroid carcinoma', 'urinary bladder cancer'], ['cirrhosis of liver', 'Hepatic fibrosis', 'Familial progressive cardiac conduction defect', 'Brugada syndrome', 'sudden cardiac arrest'], ['Hereditary thrombophilia due to congenital histidine-rich (poly-L) glycoprotein deficiency', 'dual specificity mitogen-activated protein kinase kinase 4 measurement', 'CD27 antigen measurement', 'histidine-rich glycoprotein measurement', 'blood protein measurement'], ['narcolepsy with cataplexy', 'hypersomnia', 'Meniere disease', 'Vertigo', 'narcolepsy'], ['self reported educational attainment', 'body mass index', 'body fat percentage', 'lean body mass', 'vital capacity'], ['Hyperandrogenism due to cortisone reductase deficiency', 'gastroesophageal reflux disease', 'Peptic ulcer', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['Apparent mineralocorticoid excess', 'Peptic ulcer', 'gastroesophageal reflux disease', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['Perrault syndrome', 'Bifunctional enzyme deficiency', 'hearing loss', 'Rare genetic deafness', 'reticulocyte count'], ['erythrocyte count', 'lean body mass', 'mean corpuscular volume', 'appendicular lean mass', 'age at menopause'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'serum alanine aminotransferase measurement', 'platelet count', 'alcohol abuse'], ['breast adenocarcinoma', 'urinary bladder cancer', 'neoplasm', 'cancer', 'breast cancer'], ['lymphocyte count', 'monocyte count', 'platelet crit', 'myeloid white cell count', 'leukocyte count'], ['cataract 5 multiple types', 'Zonular cataract', 'Non-syndromic congenital cataract', 'cataract', 'Total congenital cataract'], ['mean corpuscular volume', 'myeloperoxidase measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width'], ['cancer', 'influenza', 'diphtheria', 'lung adenocarcinoma', 'Gastrointestinal stromal tumor'], ['diphtheria', 'lung adenocarcinoma', 'Gastrointestinal stromal tumor', 'multiple myeloma', 'COVID-19'], ['reticulocyte count', 'triglyceride measurement', 'heel bone mineral density', 'chronic obstructive pulmonary disease', 'BMI-adjusted waist-hip ratio'], ['breast adenocarcinoma', 'neuroticism measurement', 'smoking behavior', 'neoplasm', 'cancer'], ['even-plus syndrome', 'Autosomal recessive sideroblastic anemia', 'Constitutional sideroblastic anemia', 'breast adenocarcinoma', 'mean corpuscular hemoglobin concentration'], ['breast adenocarcinoma', 'protein measurement', 'blood protein measurement', 'neoplasm', 'hepatocellular carcinoma'], ['Distal hereditary motor neuropathy type 2', 'red blood cell distribution width', 'neoplasm', 'cancer', 'hepatocellular carcinoma'], ['Autosomal dominant spastic paraplegia type 13', 'Pelizaeus-Merzbacher-like disease', 'Pelizaeus-Merzbacher-like disease due to HSPD1 mutation', 'Leukodystrophy', 'Hereditary spastic paraplegia'], ['Schwartz-Jampel syndrome', 'Dyssegmental dysplasia, Silverman-Handmaker type', 'AL amyloidosis', 'osteosarcoma', 'colorectal adenocarcinoma'], ['endometrial cancer', 'Pyoderma', 'dental caries', 'tooth disease', 'hypertension'], ['schizophrenia', 'unipolar depression', 'autism', 'anxiety disorder', 'major depressive disorder'], ['schizophrenia', 'migraine disorder', 'cardiovascular disease', 'pain', 'Headache'], ['migraine disorder', 'schizophrenia', 'Dravet syndrome', 'pain', 'Headache'], ['migraine disorder', 'schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'pain'], ['schizophrenia', 'unipolar depression', 'bipolar disorder', 'psychosis', 'depressive disorder'], ['hemorrhage', 'schizophrenia', 'schizoaffective disorder', 'psychosis', 'post operative nausea and vomiting'], ['schizophrenia', 'bipolar disorder', 'unipolar depression', 'autism', 'depressive disorder'], ['schizophrenia', 'Constipation', 'post operative nausea and vomiting', 'migraine disorder', 'Chronic constipation'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'obesity', 'unipolar depression'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'obesity', 'unipolar depression'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'unipolar depression', 'psychosis'], ['CARASIL', 'cerebral arteriopathy, autosomal dominant, with subcortical infarcts and leukoencephalopathy, type 2', 'age-related macular degeneration', 'CADASIL', 'dementia (disease)'], ['3-methylglutaconic aciduria', 'Young adult-onset Parkinsonism', 'neutropenia', 'intestinal cancer', 'Early-onset progressive neurodegeneration - blindness - ataxia - spasticity'], ['carpal tunnel syndrome', 'osteoarthritis', 'infect', 'infection', 'low density lipoprotein cholesterol measurement'], ['Huntington disease', 'Lopes-Maciel-Rodan syndrome', 'Juvenile Huntington disease', 'Rett syndrome', 'self reported educational attainment'], ['ACPA-negative rheumatoid arthritis', 'joint damage measurement', 'protein measurement', 'drug use measurement', 'rheumatoid arthritis'], ['AL amyloidosis', 'blood protein measurement', 'lean body mass', 'fat body mass', 'breast adenocarcinoma'], ['growth retardation, intellectual developmental disorder, hypotonia, and hepatopathy', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'urate measurement'], ['myeloid white cell count', 'leukocyte count', 'ICAM-1 measurement', 'lymphocyte count', 'ovarian cancer'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'Combined T and B cell immunodeficiency', 'cancer', 'breast adenocarcinoma'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'rheumatoid arthritis', 'genetic disorder'], ['acute myeloid leukemia', 'glioblastoma multiforme', 'cutaneous melanoma', 'cholangiocarcinoma', 'astrocytoma'], ['acute myeloid leukemia', 'D-2-hydroxyglutaric aciduria', '2-hydroxyglutaric aciduria', 'myelodysplastic syndrome', 'osteosarcoma'], ['head and neck malignant neoplasia', 'urogenital neoplasm', 'lung cancer', 'neoplasm', 'melanoma'], ['Mucopolysaccharidosis type 2', 'Mucopolysaccharidosis', 'Mucopolysaccharidosis type 2, severe form', 'Mucopolysaccharidosis type 2, attenuated form', 'genetic disorder'], ['Mucopolysaccharidosis type 1', 'Hurler syndrome', 'Scheie syndrome', 'Hurler-Scheie syndrome', 'genetic disorder'], ['infect', 'infection', 'breast cancer', 'systolic blood pressure', 'melanoma'], ['lymphocyte percentage of leukocytes', 'periodontal measurement', 'neutrophil percentage of leukocytes', 'lymphocyte count', 'leukocyte count'], ['eosinophil count', 'neutrophil count', 'leukocyte count', 'neutrophil percentage of leukocytes', 'myeloid white cell count'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'leukocyte count'], ['ovarian cancer', 'breast adenocarcinoma', 'systemic lupus erythematosus', 'hypothyroidism', "Hashimoto's thyroiditis"], ['multiple sclerosis', 'chronic hepatitis C virus infection', 'chronic hepatitis B virus infection', 'hepatitis C virus infection', 'melanoma'], ['COVID-19', 'multiple sclerosis', 'chronic hepatitis B virus infection', 'chronic hepatitis C virus infection', 'hepatitis C virus infection'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['Aplastic anemia', 'Primary hemophagocytic lymphohistiocytosis', 'immune system disease', 'idiopathic aplastic anemia', 'lymphocyte count'], ['Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR1 deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IFNgammaR1 deficiency', 'immunodeficiency 27A', 'Autosomal dominant mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR1 deficiency', 'disseminated atypical mycobacterial infection'], ['immunodeficiency 28', 'Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR2 deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IFNgammaR2 deficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Osteopetrosis'], ['chronic hepatitis C virus infection', 'ovarian mucinous adenocarcinoma', 'hepatitis C virus infection', 'hepatocellular carcinoma', 'low density lipoprotein cholesterol measurement'], ['Cranioectodermal dysplasia', 'Microcephaly', 'low density lipoprotein cholesterol measurement', 'BMI-adjusted waist-hip ratio', 'Brachydactyly type C'], ['Agenesis of the corpus callosum - intellectual disability - coloboma - micrognathia', 'systemic lupus erythematosus', 'refractory anemia', 'rheumatoid arthritis', 'esophageal squamous cell carcinoma'], ['Growth delay due to insulin-like growth factor type 1 deficiency', 'body height', 'sex hormone-binding globulin measurement', 'vital capacity', 'peak expiratory flow'], ['Growth delay due to insulin-like growth factor I resistance', 'lean body mass', 'Graves ophthalmopathy', 'body height', 'platelet count'], ['Silver-Russell syndrome', 'Beckwith-Wiedemann syndrome', 'Hemihypertrophy', 'Silver-Russell syndrome due to a point mutation', 'fetal growth restriction'], ['hepatocellular carcinoma', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'cation-independent mannose-6-phosphate receptor measurement', 'low density lipoprotein cholesterol measurement'], ['Short stature due to primary acid-labile subunit deficiency', 'Reduced insulin like growth factor binding protein acid labile subunit level', 'Delayed puberty', 'Short stature due to GHSR deficiency', 'cutaneous melanoma'], ['body height', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'sitting height measurement', 'puberty onset measurement'], ['pulse pressure measurement', 'systolic blood pressure', 'diastolic blood pressure', 'body height', 'appendicular lean mass'], ['refractive error measurement', 'thyroid stimulating hormone measurement', 'appendicular lean mass', 'FEV/FEC ratio', "Hashimoto's thyroiditis"], ['Familial retinal arterial macroaneurysm', 'low density lipoprotein cholesterol measurement', 'insulin-like growth factor-binding protein 7 measurement', 'blood protein measurement', 'infant grey matter volume measurement'], ['Spinal muscular atrophy with respiratory distress type 1', 'Charcot-Marie-Tooth disease', 'Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Distal spinal muscular atrophy', 'genetic disorder'], ['Autosomal agammaglobulinemia', 'Agammaglobulinemia', 'Isolated agammaglobulinemia', 'arterial stiffness measurement', 'blood protein measurement'], ['body mass index', 'myeloid cell surface antigen CD33 measurement', 'Apolipoprotein A-I deficiency', 'Tyrosinemia type 1', 'Hepatic veno-occlusive disease - immunodeficiency'], ['lung adenocarcinoma', 'eosinophil count', 'eosinophil percentage of leukocytes', 'systemic lupus erythematosus', 'psoriasis'], ['Incontinentia pigmenti', 'Hypohidrotic ectodermal dysplasia with immunodeficiency', 'X-linked mendelian susceptibility to mycobacterial diseases due to IKBKG deficiency', 'ectodermal dysplasia and immunodeficiency 1', 'X-linked mendelian susceptibility to mycobacterial diseases'], ['Pancytopenia due to IKZF1 mutations', 'acute lymphoblastic leukemia', 'B-cell acute lymphoblastic leukemia', 'Common variable immunodeficiency', 'acute myeloid leukemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'systemic lupus erythematosus', 'basophil count', 'neutrophil percentage of granulocytes'], ['leukocyte count', 'lymphocyte percentage of leukocytes', 'chronic lymphocytic leukemia', 'eosinophil count', 'eosinophil percentage of leukocytes'], ['asthma', 'type I diabetes mellitus', 'eosinophil count', 'Isolated sulfite oxidase deficiency', 'Vitiligo'], ['Cryopyrin-associated periodic syndrome', 'immune system disease', 'Muckle-Wells syndrome', 'Familial cold urticaria', 'Antisynthetase syndrome'], ['rheumatoid arthritis', 'COVID-19', 'fibrinogen measurement', 'pericarditis', 'immune system disease'], ['interleukin 1 Receptor accessory protein measurement', 'protein measurement', 'blood protein measurement', 'obesity', 'visual perception measurement'], ['eosinophil count', 'asthma', 'allergy', 'allergic rhinitis', 'childhood onset asthma'], ['interleukin 1 receptor-like 2 measurement', 'eosinophil count', 'childhood onset asthma', 'acute myeloid leukemia', 'interleukin 18 receptor 1 measurement'], ['Sterile multifocal osteomyelitis with periostitis and pustulosis', 'interleukin 1 receptor antagonist measurement', 'diabetic nephropathy', 'C-reactive protein measurement', 'neutrophil count'], ['seasonal allergic rhinitis', 'psoriasis', 'eosinophil count', 'breast adenocarcinoma', 'allergy'], ['Immunodeficiency due to CD25 deficiency', 'type I diabetes mellitus', 'diabetes mellitus', 'acute graft vs. host disease', 'multiple sclerosis'], ['Immunodeficiency syndrome with autoimmunity', 'metastatic melanoma', 'kidney cancer', 'mycosis fungoides', "Sezary's disease"], ['X-Linked Combined Immunodeficiency Diseases', 'T-B+ severe combined immunodeficiency due to gamma chain deficiency', 'gamma chain deficiency', 'Omenn syndrome', 'Severe combined immunodeficiency'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'sum of eosinophil and basophil counts', 'adult onset asthma', 'gastric adenocarcinoma'], ['acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer'], ['stroke', 'adult onset asthma', 'childhood onset asthma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['mean corpuscular hemoglobin concentration', 'Monoamine oxidase A deficiency', 'pontine autosomal dominant microangiopathy with leukoencephalopathy', 'neoplasm', 'colorectal adenocarcinoma'], ['asthma', 'atopic eczema', 'Eczema', 'chronic rhinosinusitis with nasal polyps', 'eosinophil count'], ['asthma', 'Airway obstruction', 'Hypereosinophilic syndrome', 'chronic obstructive pulmonary disease', 'Nasal Cavity Polyp'], ['asthma', 'eosinophil count', 'basophil count', 'basophil percentage of leukocytes', 'eosinophil percentage of leukocytes'], ['rheumatoid arthritis', 'immune system disease', 'systemic juvenile idiopathic arthritis', "Kaposi's sarcoma", 'juvenile idiopathic arthritis'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'COVID-19', 'temporal arteritis', 'immune system disease'], ['Autosomal recessive hyper-IgE syndrome', 'neuromyelitis optica', 'Stüve-Wiedemann syndrome', 'hepatocellular adenoma', 'carcinoma of liver and intrahepatic biliary tract'], ['Epidermodysplasia verruciformis', 'lymphocyte count', 'monocyte count', 'eosinophil count', 'blood protein measurement'], ['T-B+ severe combined immunodeficiency due to IL-7Ralpha deficiency', 'Omenn syndrome', 'multiple sclerosis', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'T-B+ severe combined immunodeficiency due to CD45 deficiency'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'pancreatic adenocarcinoma', 'asthma', 'neoplasm'], ['Autosomal recessive early-onset inflammatory bowel disease', "Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome'], ['Autosomal recessive early-onset inflammatory bowel disease', 'inflammatory bowel disease', 'ulcerative colitis', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'infection'], ['lean body mass', 'appendicular lean mass', 'osteoarthritis', 'osteoarthritis, hip', 'fat body mass'], ['primary biliary cirrhosis', 'multiple sclerosis', 'biliary liver cirrhosis', "Behcet's syndrome", 'Sjogren syndrome'], ['psoriasis', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12B deficiency', "Crohn's disease", 'psoriatic arthritis', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['asthma', 'atopic eczema', 'Eczema', 'psoriasis', 'allergic rhinitis'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'low grade glioma', 'breast adenocarcinoma'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'gastric adenocarcinoma'], ['protein measurement', 'post-traumatic stress disorder symptom measurement', 'response to trauma exposure', 'rheumatoid arthritis', 'neoplasm'], ['hair color', 'interleukin 16 measurement', 'hair colour measurement', 'appendicular lean mass', 'lymphocyte count'], ['psoriasis', 'ankylosing spondylitis', 'psoriatic arthritis', 'psoriasis vulgaris', 'immune system disease'], ['heel bone mineral density', 'bilirubin measurement', 'diet measurement', 'metabolic syndrome', 'bone quantitative ultrasound measurement'], ['chronic obstructive pulmonary disease', 'acute lymphoblastic leukemia', 'aspartate aminotransferase measurement', 'neoplasm', 'candidiasis'], ['Chronic mucocutaneous candidosis', 'pancreatic carcinoma', 'response to gemcitabine', 'overall survival', 'mean platelet volume'], ['immunodeficiency 51', 'Chronic mucocutaneous candidosis', 'psoriasis', 'psoriasis vulgaris', 'immune system disease'], ['X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'candidiasis', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12RB1 deficiency', 'Recurrent Neisseria infections due to factor D deficiency', 'Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR2 deficiency'], ['interleukin 18 measurement', 'breast adenocarcinoma', 'bone quantitative ultrasound measurement', 'heel bone mineral density', 'neoplasm'], ['urinary albumin to creatinine ratio', 'body height', 'Romano-Ward syndrome', 'Brugada syndrome', "adult-onset Still's disease"], ['asthma', 'eosinophil count', "Behcet's syndrome", 'childhood onset asthma', 'allergy'], ['eosinophil count', 'inflammatory bowel disease', "Crohn's disease", 'ulcerative colitis', 'mosquito bite reaction itch intensity measurement'], ['Common variable immunodeficiency', 'Autosomal recessive early-onset inflammatory bowel disease', 'primary biliary cirrhosis', 'psoriasis', 'childhood onset asthma'], ['Cryptosporidiosis - chronic cholangitis - liver disease', 'Omenn syndrome', 'Combined T and B cell immunodeficiency', 'gamma chain deficiency', 'eosinophil count'], ['atopic eczema', 'vitamin D measurement', 'dermatitis', 'DNA methylation', 'Eczema'], ['psoriasis', 'psoriasis vulgaris', 'immune system disease', "Crohn's disease", 'psoriatic arthritis'], ['inflammatory bowel disease', 'ulcerative colitis', "Crohn's disease", 'psoriasis', 'ankylosing spondylitis'], ['thyroid gland adenocarcinoma', 'neoplasm', 'infection', 'infect', 'colitis'], ['red blood cell distribution width', 'mean corpuscular volume', "Crohn's disease", 'psoriasis', 'ulcerative colitis'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'platelet count', 'monocyte count'], ['allergic rhinitis', 'asthma', 'Eczema', 'psoriasis', "Alzheimer's disease"], ['Familial primary localized cutaneous amyloidosis', 'Primary localized amyloidosis', 'leukocyte count', 'basophil count', 'Autosomal agammaglobulinemia'], ['asthma', 'eosinophil count', 'childhood onset asthma', 'drug use measurement', 'allergy'], ['neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'cortical surface area measurement', 'type II diabetes mellitus'], ['psoriasis', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma', 'interleukin 1 receptor antagonist measurement', 'Malignant epithelial tumor of ovary'], ['interleukin 1 receptor antagonist measurement', 'drug use measurement', 'quality of life during menstruation measurement', 'pathological myopia', 'low density lipoprotein cholesterol measurement'], ['Generalized pustular psoriasis', 'psoriasis 14, pustular', 'deficiency of the interleukin-36 receptor antagonist', 'Pustulosis palmaris et plantaris', 'Acrodermatitis continua suppurativa of Hallopeau'], ['interleukin-1 beta measurement', 'Eczema', 'psoriasis', 'neoplasm', 'rheumatoid arthritis'], ['brain measurement', 'neuroimaging measurement', 'neutrophil count', 'sex hormone-binding globulin measurement', 'myeloid white cell count'], ['pulse pressure measurement', 'respiratory system disease', 'asthma', 'Tuberculosis', 'Townes-Brocks syndrome'], ['FEV/FEC ratio', 'systolic blood pressure', 'platelet count', 'mean platelet volume', 'Hand-foot syndrome'], ['Abnormality of brain morphology', 'cortical surface area measurement', 'glomerular filtration rate', 'brain volume measurement', 'pathological myopia'], ['Nephronophthisis', 'prostate carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['ovarian adenocarcinoma', 'pulmonary arterial hypertension', 'pulse pressure measurement', 'self reported educational attainment', 'sex hormone-binding globulin measurement'], ['muscular dystrophy, congenital, with cataracts and intellectual disability', 'Congenital muscular dystrophy with intellectual disability', 'Marinesco-Sjögren syndrome', 'neoplasm', 'acute myeloid leukemia'], ['MODY', 'Permanent neonatal diabetes mellitus', 'type I diabetes mellitus', 'diabetes mellitus, permanent neonatal 4', 'hyperproinsulinemia'], ['Leprechaunism', 'Rabson-Mendenhall syndrome', 'type II diabetes mellitus', 'Hyperinsulinism due to INSR deficiency', 'Insulin-resistance syndrome type A'], ['Familial medullary thyroid carcinoma', 'Hereditary sensory and autonomic neuropathy type 4', 'hypothyroidism', 'visual perception measurement', 'platelet count'], ['Orofaciodigital syndrome', 'Short rib dysplasia', 'Short rib-polydactyly syndrome, Verma-Naumoff type', 'mean corpuscular volume', 'Short rib-polydactyly syndrome'], ['multiple myeloma', 'neoplasm', 'hepatocellular carcinoma', 'Immunodeficiency due to an early component of complement deficiency', 'acute lymphoblastic leukemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts', 'neutrophil percentage of granulocytes'], ['asthma', 'treatment resistant depression', 'response to ketamine', 'thyroid gland adenocarcinoma', 'protein measurement'], ['Immunodeficiency due to interleukin-1 receptor-associated kinase-4 deficiency', 'Primary immunodeficiency', 'X-linked mendelian susceptibility to mycobacterial diseases due to IKBKG deficiency', 'gram-negative bacterial infections', 'lung adenocarcinoma'], ['Herpetic encephalitis', 'schizophrenia', 'endometrial cancer', 'ovarian cancer', 'autoimmune thyroid disease'], ['multiple myeloma', 'basal cell carcinoma', 'chronic lymphocytic leukemia', 'diffuse large B-cell lymphoma', 'lymphoid neoplasm'], ['systemic lupus erythematosus', 'rheumatoid arthritis', 'inflammatory bowel disease', 'ulcerative colitis', 'eosinophil count'], ['Van der Woude syndrome', 'Autosomal dominant popliteal pterygium syndrome', 'Popliteal pterygium syndrome', 'cleft lip', 'Cleft palate'], ['Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'monocyte count', 'monocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'neutrophil count'], ['immunodeficiency disease', 'pancreatic adenocarcinoma', 'apolipoprotein A 1 measurement', 'neoplasm', 'breast cancer'], ['Craniofacial dysplasia-osteopenia syndrome', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'body height', 'hypospadias'], ['Mendelian susceptibility to mycobacterial diseases due to complete ISG15 deficiency', 'Bilateral striopallidodentate calcinosis', 'influenza', 'Mendelian susceptibility to mycobacterial diseases', 'blood protein measurement'], ['mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'sitting height measurement', 'age-related hearing impairment'], ['heel bone mineral density', 'smoking status measurement', 'body mass index', 'body fat percentage', 'lean body mass'], ['heel bone mineral density', 'breast carcinoma', 'smoking status measurement', 'body height', 'hematocrit'], ['reticulocyte count', 'type II diabetes mellitus', 'Immature Reticulocyte Fraction Measurement', 'FEV/FEC ratio', 'reticulocyte measurement'], ['Glanzmann thrombasthenia', 'Autosomal dominant macrothrombocytopenia', 'cancer', 'coronary artery disease', 'acute myocardial infarction'], ["Crohn's disease", 'relapsing-remitting multiple sclerosis', 'multiple sclerosis', 'ulcerative colitis', 'monocyte count'], ['Congenital muscular dystrophy with integrin alpha-7 deficiency', 'Congenital muscular dystrophy', 'Congenital fiber-type disproportion myopathy', 'Abnormality of brain morphology', 'breast adenocarcinoma'], ['psoriasis', 'dry eye syndrome', 'immune system disease', 'eye disease', 'lymphocyte count'], ['systemic lupus erythematosus', 'glioblastoma multiforme', 'systemic scleroderma', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['glioblastoma multiforme', 'lung adenocarcinoma', 'multiple myeloma', 'asthma', 'breast adenocarcinoma'], ['body mass index', 'IGA glomerulonephritis', 'acute myeloid leukemia', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['Leukocyte adhesion deficiency', 'Leukocyte adhesion deficiency type I', 'dry eye syndrome', 'genetic disorder', 'eye disease'], ['Glanzmann thrombasthenia', 'myocardial infarction', 'cancer', 'coronary artery disease', 'acute myocardial infarction'], ['blood protein measurement', 'coronary artery disease', 'eosinophil count', 'heel bone mineral density', 'glomerular filtration rate'], ['lymphoproliferative syndrome 1', 'renal cell carcinoma', 'sarcoma', 'neoplasm', 'soft tissue sarcoma'], ["Crohn's disease", 'psoriasis', 'inflammatory bowel disease', 'ulcerative colitis', 'sclerosing cholangitis'], ['body mass index', 'multiple myeloma', 'myeloid neoplasm', 'Miyoshi myopathy', 'neoplasm'], ['Isovaleric acidemia', 'genetic disorder', 'blood metabolite measurement', 'testosterone measurement', 'idiopathic pulmonary fibrosis'], ['platelet count', 'monocyte percentage of leukocytes', 'monocyte count', 'lymphocyte percentage of leukocytes', 'platelet crit'], ['Alagille syndrome', 'Alagille syndrome due to a JAG1 point mutation', 'Tetralogy of Fallot', 'Alagille syndrome due to 20p12 microdeletion', 'deafness, congenital heart defects, and posterior embryotoxon'], ['cancer', 'T-cell leukemia', 'erythrocyte count', 'hemoglobin measurement', 'hematocrit'], ['polycythemia vera', 'rheumatoid arthritis', 'primary myelofibrosis', 'myelofibrosis', 'immune system disease'], ['polycythemia vera', 'essential thrombocythemia', 'acute myeloid leukemia', 'primary myelofibrosis', 'ulcerative colitis'], ['T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'ulcerative colitis', 'rheumatoid arthritis', 'ankylosing spondylitis', 'immune system disease'], ['Huntington disease-like 2', 'hair colour measurement', 'migraine disorder', 'small vessel stroke', 'stroke'], ["Alzheimer's disease", 'colon adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['eosinophil count', 'monocyte count', 'eosinophil percentage of leukocytes', 'grip strength measurement', 'monocyte percentage of leukocytes'], ['cancer', 'T-cell leukemia', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma'], ['mental retardation', 'genetic disorder', 'Intellectual disability', 'Autosomal recessive non-syndromic intellectual disability', 'colorectal adenocarcinoma'], ['Genitopatellar syndrome', 'Blepharophimosis-intellectual disability syndrome, SBBYS type', 'prostate adenocarcinoma', 'genetic disorder', 'Intellectual disability'], ['Intellectual disability', 'autism', 'Global developmental delay', 'Seizure', 'Feeding difficulties'], ['Early infantile epileptic encephalopathy', 'multiple sclerosis', 'undetermined early-onset epileptic encephalopathy', 'Epileptic encephalopathy', 'secondary progressive multiple sclerosis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'transverse myelitis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['Familial atrial fibrillation', 'atrial fibrillation', 'multiple sclerosis', 'secondary progressive multiple sclerosis', 'cardiac arrhythmia'], ['Progressive myoclonic epilepsy', 'multiple sclerosis', 'secondary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis', 'transverse myelitis'], ['Spinocerebellar ataxia type 13', 'multiple sclerosis', 'Spinocerebellar ataxia type 15/16', 'secondary progressive multiple sclerosis', 'transverse myelitis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'self reported educational attainment', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['Familial long QT syndrome', 'Jervell and Lange-Nielsen syndrome', 'long QT syndrome 5', 'Romano-Ward syndrome', 'Prolonged QT interval'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'primary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'transverse myelitis'], ['Romano-Ward syndrome', 'Familial long QT syndrome', 'Familial short QT syndrome', 'Prolonged QT interval', 'atrial fibrillation'], ['atrial fibrillation', 'self reported educational attainment', 'intelligence', 'cardiac arrhythmia', 'mathematical ability'], ['Familial hyperaldosteronism type III', 'atrial fibrillation', 'Romano-Ward syndrome', 'aldosterone-producing adenoma', 'Cardiodysrhythmic potassium-sensitive periodic paralysis'], ['type II diabetes mellitus', 'diabetes mellitus', 'Transient neonatal diabetes mellitus', 'Familial hyperinsulinism', 'hyperinsulinemic hypoglycemia, familial, 2'], ['pulmonary arterial hypertension', 'coronary artery disease', 'Idiopathic and/or familial pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'ischemia reperfusion injury'], ['coronary artery disease', 'migraine disorder', 'appendicular lean mass', 'body height', 'intraocular pressure measurement'], ['Intellectual disability, Birk-Barel type', 'coronary artery disease', 'ischemia reperfusion injury', 'kidney failure', 'Agitation'], ['Dehydrated hereditary stomatocytosis', 'pain', 'alcohol abuse', 'osteoarthritis, knee', 'osteoarthritis, hip'], ['Jervell and Lange-Nielsen syndrome', 'Familial long QT syndrome', 'Romano-Ward syndrome', 'Familial short QT syndrome', 'Familial atrial fibrillation'], ['Early infantile epileptic encephalopathy', 'Benign familial neonatal seizures', 'Seizure', 'Benign familial neonatal-infantile seizures', 'multiple sclerosis'], ['Benign familial neonatal seizures', 'Seizure', 'Intellectual disability', 'multiple sclerosis', 'epilepsy'], ['Autosomal dominant non-syndromic intellectual disability', 'multiple sclerosis', 'Seizure', 'epilepsy', 'mental retardation'], ['developmental disability', 'breast adenocarcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'endometrial cancer'], ['breast adenocarcinoma', 'clear cell renal carcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['age at menarche', 'body height', 'mean corpuscular volume', 'body mass index', 'reticulocyte count'], ['intellectual disability, autosomal recessive 65', 'Autosomal recessive non-syndromic intellectual disability', 'genetic disorder', 'autism', 'breast adenocarcinoma'], ['Kabuki syndrome', 'urinary bladder cancer', 'squamous cell lung carcinoma', 'genetic disorder', 'astrocytoma'], ['cancer', 'hepatocellular carcinoma', 'Familial capillary hemangioma', 'non-small cell lung carcinoma', 'neoplasm'], ['lung adenocarcinoma', 'multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'non-small cell lung carcinoma', 'squamous cell lung carcinoma'], ['Essential fructosuria', 'Disorder of carbohydrate metabolism', 'risk-taking behaviour', 'neutrophil count', 'brain measurement'], ['neoplasm', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'lung cancer', 'non-small cell lung carcinoma'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2A1', 'adrenal gland pheochromocytoma', 'Hereditary pheochromocytoma-paraganglioma', 'Charcot-Marie-Tooth disease', 'neuroblastoma'], ['non-small cell lung carcinoma', 'thyroid cancer', 'lung adenocarcinoma', 'undifferentiated pleomorphic sarcoma', 'adenosquamous lung carcinoma'], ['Microcephaly - lymphedema - chorioretinopathy', 'genetic disorder', 'Syndromic retinitis pigmentosa', 'Retinal dystrophy', 'type II diabetes mellitus'], ['ovarian cancer', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'gastric carcinoma', 'head and neck carcinoma'], ['Goldberg-Shprintzen megacolon syndrome', 'Shprintzen-Goldberg syndrome', 'peripheral neuropathy', 'eosinophil percentage of leukocytes', 'eosinophil count'], ['pancreatic adenocarcinoma', 'prostate carcinoma', '3-hydroxypropylmercapturic acid measurement', 'smoking behavior', 'aortic stenosis'], ['Gastrointestinal stromal tumor', 'Piebaldism', 'Mastocytosis', 'maculopapular cutaneous mastocytosis', 'acute myeloid leukemia'], ['Familial progressive hyperpigmentation', 'Familial progressive hyper- and hypopigmentation', 'hyperpigmentation with or without hypopigmentation, familial progressive', 'testicular carcinoma', 'Waardenburg syndrome type 2'], ['tumoral calcinosis, hyperphosphatemic, familial, 3', 'familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'cancer', 'type II diabetes mellitus', 'birth weight'], ['cancer', 'alcohol consumption measurement', 'alcohol use disorder measurement', 'alcohol drinking', 'longitudinal alcohol consumption measurement'], ['breast carcinoma', 'Secretory Meningioma', 'Chordoid Meningioma', 'Endometrial Endometrioid Adenocarcinoma', 'Fibrous Meningioma'], ['urinary bladder cancer', 'squamous cell lung carcinoma', 'cervical squamous cell carcinoma', 'dilated cardiomyopathy', 'colorectal cancer'], ['prostate carcinoma', 'gastric carcinoma', 'prostate adenocarcinoma', 'heel bone mineral density', 'gastric cancer'], ['Familial progressive cardiac conduction defect', 'X-linked endothelial corneal dystrophy', 'Fuchs endothelial corneal dystrophy', 'Primary dystonia, DYT13 type', 'Primary dystonia, DYT2 type'], ['body height', 'acute quadriplegic myopathy', 'chronic kidney disease', 'dermatomyositis', 'astrocytoma'], ['Hodgkins lymphoma', 'classic Hodgkin lymphoma', 'irritability measurement', 'Irritability', 'insomnia measurement'], ['kallikrein, decreased urinary activity of', 'cutaneous melanoma', 'lung adenocarcinoma', 'acute kidney failure', 'Townes-Brocks syndrome'], ['prostate carcinoma', 'prostate specific antigen measurement', 'prostate cancer', 'male reproductive organ cancer', 'cutaneous melanoma'], ['Hypomaturation amelogenesis imperfecta', 'Amelogenesis imperfecta', 'amelogenesis imperfecta, type ij', 'prostate specific antigen measurement', 'genetic disorder'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'neuroblastoma', 'urinary bladder cancer'], ['kallikrein‐6 measurement', 'blood protein measurement', 'neoplasm', "Alzheimer's disease", 'non-small cell lung carcinoma'], ['blood protein measurement', 'kallikrein-7 measurement', 'breast adenocarcinoma', 'kallikrein-11 measurement', 'ovarian cancer'], ['blood protein measurement', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'gastric adenocarcinoma'], ['prostate specific antigen measurement', 'prostate carcinoma', 'prostate cancer', 'ovarian cancer', 'breast cancer'], ['Congenital prekallikrein deficiency', 'Hereditary angioedema', 'hemorrhage', 'Bietti crystalline dystrophy', 'serum metabolite measurement'], ['blood protein measurement', 'rheumatoid arthritis', 'refractory anemia', 'Townes-Brocks syndrome', 'juvenile idiopathic arthritis'], ['platelet count', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'monocyte count', 'granulocyte percentage of myeloid white cells'], ['lean body mass', 'red blood cell density measurement', 'total cholesterol measurement', 'appendicular lean mass', 'fat body mass'], ['acute myeloid leukemia', 'neoplasm', 'hepatocellular carcinoma', 'cancer', 'cervical cancer'], ['chronic kidney disease', 'urinary metabolite measurement', 'coronary artery disease', 'lean body mass', 'reticulocyte count'], ['Wiedemann-Steiner syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'cutaneous melanoma'], ['Kabuki syndrome', 'head and neck squamous cell carcinoma', 'medulloblastoma', 'prostate adenocarcinoma', 'gastric adenocarcinoma'], ['intellectual disability, autosomal dominant 51', 'genetic disorder', 'serum IgG glycosylation measurement', 'autism spectrum disorder', 'Rare genetic intellectual disability'], ['Malignant Pancreatic Neoplasm', 'pancreatic carcinoma', 'pancreatic neoplasm', 'breast cancer', 'cancer'], ['Congenital high-molecular-weight kininogen deficiency', 'blood protein measurement', 'kininogen-1 measurement', 'partial thromboplastin time', 'factor XI measurement'], ['Cardiofaciocutaneous syndrome', 'Noonan syndrome', 'Juvenile Myelomonocytic Leukemia', 'acute myeloid leukemia', 'multiple myeloma'], ['Localized epidermolysis bullosa simplex', 'Generalized epidermolysis bullosa simplex, non-Dowling-Meara type', 'Epidermolysis bullosa simplex, Dowling-Meara type', 'Dowling-Degos disease', 'Epidermolysis bullosa simplex with mottled pigmentation'], ['Pachyonychia congenita', 'ovarian cancer', 'prostate adenocarcinoma', 'head and neck squamous cell carcinoma', 'clear cell renal carcinoma'], ['prostate adenocarcinoma', 'clear cell renal carcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'neoplasm'], ['Epidermolytic ichthyosis', 'Annular epidermolytic ichthyosis', 'Congenital reticular ichthyosiform erythroderma', 'Pachyonychia congenita', 'palmoplantar keratosis'], ['Meesmann corneal dystrophy', 'genetic disorder', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'clear cell renal carcinoma'], ['White sponge nevus', 'head and neck squamous cell carcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'clear cell renal carcinoma'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'esophageal cancer'], ['Pachyonychia congenita', 'Sebocystomatosis', 'Anonychia', 'clear cell renal carcinoma', 'glioblastoma multiforme'], ['cirrhosis of liver', 'cirrhosis, familial', 'glioblastoma multiforme', 'cleft lip', 'Cleft palate'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'esophageal cancer'], ['pancreatic adenocarcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'colorectal carcinoma'], ['Woolly hair', 'Hypotrichosis simplex', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma'], ['head and neck squamous cell carcinoma', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'Obesity due to melanocortin 4 receptor deficiency', 'Dysplasia epiphysealis hemimelica'], ['ovarian cancer', 'clear cell renal carcinoma', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma', 'neuroblastoma'], ['hair color', 'head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'clear cell renal carcinoma', 'prostate adenocarcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'reticulocyte count'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'low grade glioma'], ['head and neck squamous cell carcinoma', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'hair color', 'low grade glioma', 'psoriasis'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'Hypotrichosis simplex'], ['Encephalopathy due to hydroxykynureninuria', 'Congenital vertebral-cardiac-renal anomalies syndrome', 'vertebral, cardiac, renal, and limb defects syndrome 2', 'Catel-Manzke syndrome', 'Ulbright-Hodes syndrome'], ['genetic disorder', 'monocyte count', 'low grade glioma', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['triglyceride measurement', 'cardiac troponin T measurement', 'experimental autoimmune encephalomyelitis', 'electrocardiography', 'left ventricular structural measurement'], ['sex hormone-binding globulin measurement', 'blood protein measurement', "Hashimoto's thyroiditis", 'neoplasm', 'melanoma'], ['inflammatory bowel disease', 'systemic lupus erythematosus', 'malaria', 'acute lymphoblastic leukemia', 'severe malarial anemia'], ['mean corpuscular hemoglobin concentration', 'breast carcinoma', 'pancreatic adenocarcinoma', 'breast cancer', 'neoplasm'], ['Congenital muscular dystrophy type 1A', 'muscular dystrophy, limb-girdle, autosomal recessive 23', 'Congenital muscular dystrophy', 'LAMA2-related muscular dystrophy', 'laminin alpha 2-related dystrophy'], ['Junctional epidermolysis bullosa, Herlitz type', 'Junctional epidermolysis bullosa, non-Herlitz type', 'Junctional epidermolysis bullosa', 'eye disease', 'Generalized junctional epidermolysis bullosa, non-Herlitz type'], ['neutrophil count', 'leukocyte count', 'myeloid white cell count', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['Glycogen storage disease due to LAMP-2 deficiency', 'Glycogen Storage Disease Type 2b', 'hypertrophic cardiomyopathy', 'dilated cardiomyopathy', 'cardiomyopathy'], ['inflammatory bowel disease', "Parkinson's disease", 'esophageal squamous cell carcinoma', 'pancreatitis', 'lung carcinoma'], ['neoplasm', 'acute myeloid leukemia', 'hepatocellular carcinoma', 'colorectal carcinoma', 'acute chest syndrome'], ['Acute infantile liver failure-multisystemic involvement syndrome', 'self reported educational attainment', 'mathematical ability', 'Global developmental delay', 'sensorineural hearing loss'], ['Perrault syndrome', 'Mitochondrial myopathy and sideroblastic anemia', 'Rare genetic deafness', 'Mitochondrial myopathy', 'body mass index'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Severe combined immunodeficiency', 'breast adenocarcinoma', 'self reported educational attainment', 'lymphocyte count'], ['basal cell carcinoma', 'cervical squamous cell carcinoma', 'colorectal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'Hepatobiliary Neoplasm'], ['blood protein measurement', 'basophil percentage of leukocytes', 'ovarian cancer', 'glioblastoma multiforme', 'thyroid gland adenocarcinoma'], ['Greenberg dysplasia', 'regressive spondylometaphyseal dysplasia', 'Pelger-Huet anomaly', 'Reynolds syndrome', 'Jeune syndrome'], ['Familial LCAT deficiency', 'Fish-eye disease', 'LCAT deficiency', 'high density lipoprotein cholesterol measurement', 'esophageal cancer'], ['cancer', 'Severe combined immunodeficiency due to LCK deficiency', 'acute lymphoblastic leukemia', 'renal cell carcinoma', 'chronic myelogenous leukemia'], ['cutaneous melanoma', 'neoplasm', 'cancer', 'hepatocellular carcinoma', 'coronary artery disease'], ['endometrial cancer', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'hepatocellular carcinoma'], ['lymphocyte count', 'autoimmune disease', 'MAP kinase-activated protein kinase 3 measurement', 'blood protein measurement', 'lung carcinoma'], ['eosinophil count', 'mean platelet volume', 'eosinophil percentage of leukocytes', 'breast adenocarcinoma', 'inflammatory bowel disease'], ['Glycogen storage disease due to lactate dehydrogenase M-subunit deficiency', 'Glycogen storage disease', 'Glycogen storage disease due to lactate dehydrogenase deficiency', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'serum amyloid A protein measurement'], ['familial hypercholesterolemia', 'Homozygous familial hypercholesterolemia', 'hypercholesterolemia, familial, 1', 'Hypercholesterolemia', 'Hyperlipoproteinemia type 3'], ['Obesity due to congenital leptin deficiency', 'BMI-adjusted leptin measurement', 'leptin measurement', 'type II diabetes mellitus', 'Obesity due to leptin receptor gene deficiency'], ['Obesity due to leptin receptor gene deficiency', 'obesity', 'body mass index', 'leptin receptor measurement', 'lipodystrophy'], ['longitudinal BMI measurement', 'non-alcoholic steatohepatitis', 'neutrophil count', 'leukocyte count', 'age at menarche'], ['white matter microstructure measurement', 'phospholipid measurement', 'visceral adipose tissue measurement', 'Reticular dysgenesis', 'Combined immunodeficiency with skin granulomas'], ['mean platelet volume', 'protein measurement', 'leukocyte count', 'blood protein measurement', 'reticulocyte count'], ['blood protein measurement', 'acute myeloid leukemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'eosinophil count'], ['serum metabolite measurement', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'blood protein measurement', 'endometrial cancer'], ['lean body mass', 'hair shape measurement', 'osteoporosis', 'postmenopausal osteoporosis', 'balding measurement'], ['arterial stiffness measurement', 'reticulocyte count', 'osteoarthritis, spine', 'type II diabetes mellitus', 'neoplasm'], ['DNA methylation', 'non-small cell lung carcinoma', 'pancreatic adenocarcinoma', 'lung cancer', 'clear cell renal carcinoma'], ['heel bone mineral density', 'lung adenocarcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'urate measurement'], ['leukocyte immunoglobulin-like receptor subfamily B member 1 measurement', 'inflammatory bowel disease', 'systemic lupus erythematosus', 'blood protein measurement', 'rheumatoid arthritis'], ['ovarian cancer', 'breast adenocarcinoma', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'myeloid white cell count'], ['Wolman disease', 'Cholesteryl ester storage disease', 'Lysosomal acid lipase deficiency', 'familial hypercholesterolemia', 'coronary artery disease'], ['Hyperlipidemia due to hepatic triglyceride lipase deficiency', 'high density lipoprotein cholesterol measurement', 'serum metabolite measurement', 'triglyceride measurement', 'total cholesterol measurement'], ['Familial partial lipodystrophy', 'Familial partial lipodystrophy, Dunnigan type', 'Familial partial lipodystrophy, Köbberling type', 'esophageal cancer', 'Partial chromosome Y deletion'], ['high density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'vitamin D measurement', 'low density lipoprotein cholesterol measurement', 'red blood cell distribution width'], ['Familial partial lipodystrophy, Dunnigan type', 'Hutchinson-Gilford progeria syndrome', 'dilated cardiomyopathy', 'Emery-Dreifuss muscular dystrophy', 'Congenital muscular dystrophy due to LMNA mutation'], ['prostate carcinoma', 'smoking status measurement', 'serum gamma-glutamyl transferase measurement', 'urate measurement', 'neuroticism measurement'], ['neutrophil count', "Crohn's disease", 'neutrophil percentage of leukocytes', 'inflammatory bowel disease', 'blood protein measurement'], ['neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum', 'wellbeing measurement', 'depressive symptom measurement', 'neuroticism measurement', 'carpal tunnel syndrome'], ['CODAS syndrome', 'Pyruvate dehydrogenase E1-alpha deficiency', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration'], ['Familial thoracic aortic aneurysm and aortic dissection', 'marfan syndrome/loeys-dietz syndrome/familial thoracic aortic aneurysms and dissections', 'Aortic dissection', 'aortic aneurysm', 'FEV/FEC ratio'], ['exfoliation syndrome', 'open-angle glaucoma', 'glaucoma', 'vital capacity', 'body fat distribution'], ['body height', 'blood protein measurement', 'appendicular lean mass', 'sitting height measurement', 'forced expiratory volume'], ['Stickler syndrome', 'Autosomal recessive Stickler syndrome', 'breast adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma'], ['coronary artery disease', 'myocardial infarction', 'cardiovascular disease', 'low density lipoprotein cholesterol measurement', 'angina pectoris'], ['eosinophil count', 'monocyte count', 'central corneal thickness', 'body height', 'eosinophil percentage of leukocytes'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['mean corpuscular volume', 'glioblastoma multiforme', 'ovarian cancer', 'mean corpuscular hemoglobin', 'vital capacity'], ['Familial lipoprotein lipase deficiency', 'Combined hyperlipidemia', 'Hyperlipoproteinemia type 1', 'coronary artery disease', 'high density lipoprotein cholesterol measurement'], ['basophil percentage of leukocytes', 'Meckel syndrome', 'eosinophil count', 'Bardet-Biedl syndrome', 'blood protein measurement'], ['skin carcinoma', 'Uterine Carcinosarcoma', 'lipoma', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma'], ['Common variable immunodeficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'neutrophil count', 'lobe attachment'], ['Ulerythema ophryogenesis', 'Keratosis pilaris atrophicans', 'Atrophoderma vermiculata', 'Keratosis follicularis spinulosa decalvans', 'migraine disorder'], ['Donnai-Barrow syndrome', 'Intellectual disability', 'urate measurement', 'glomerular filtration rate', 'uric acid measurement'], ['Familial exudative vitreoretinopathy', 'Osteoporosis - pseudoglioma', 'polycystic liver disease 4 with or without kidney cysts', 'Autosomal dominant osteopetrosis type 1', 'Autosomal dominant osteosclerosis, Worth type'], ['coronary artery disease, autosomal dominant 2', 'Oligodontia', 'tooth agenesis', 'orofacial cleft', 'Orofacial clefting syndrome'], ['myocardial infarction', 'self reported educational attainment', 'brain measurement', 'neuroimaging measurement', 'blood protein measurement'], ['Congenital lactic acidosis, Saguenay-Lac-Saint-Jean type', 'Leigh syndrome', 'Mitochondrial disease', 'Isolated cytochrome C oxidase deficiency', 'grip strength measurement'], ['blood protein measurement', 'sinusitis', 'Brugada syndrome', 'Familial progressive cardiac conduction defect', 'Familial short QT syndrome'], ['Herpes Zoster', 'eosinophil count', 'atopic eczema', 'allergy', 'eosinophil percentage of leukocytes'], ['Hereditary late-onset Parkinson disease', 'Young adult-onset Parkinsonism', "Parkinson's disease", 'mean platelet volume', 'inflammatory bowel disease'], ['systolic blood pressure', 'diastolic blood pressure', 'alcohol drinking', 'hypertension', 'mean arterial pressure'], ['cutaneous melanoma', 'lung adenocarcinoma', 'red blood cell density measurement', 'lymphocyte count', 'erythrocyte count'], ['psoriatic arthritis', 'lymphocyte count', 'tumor necrosis factor beta measurement', 'lymphocyte percentage of leukocytes', 'monocyte count'], ['small intestine neuroendocrine tumor', 'monocyte count', 'FEV/FEC ratio', 'amino acid measurement', 'blood metabolite measurement'], ['thyroid gland adenocarcinoma', 'low grade glioma', 'chronic obstructive pulmonary disease', 'asthma', 'rheumatoid arthritis'], ['breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'language impairment', 'breast cancer', 'Neutrophil immunodeficiency syndrome'], ['Tuberculosis', 'AL amyloidosis', 'blood protein measurement', 'cerebrospinal fluid biomarker measurement', 'alkaline phosphatase measurement'], ['type II diabetes mellitus', 'albuminuria', 'childhood onset asthma', 'Drugs used in diabetes use measurement', 'high density lipoprotein cholesterol measurement'], ['colorectal carcinoma', 'neoplasm', 'pes', 'acute myeloid leukemia', 'heel bone mineral density'], ['body height', 'fat body mass', 'body fat percentage', 'chronic obstructive pulmonary disease', 'central corneal thickness'], ['cancer', 'neoplasm', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'Sensorineural deafness with dilated cardiomyopathy'], ['cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'cutaneous Leishmaniasis'], ['drug use measurement', 'HIV-1 infection', 'viral load', 'melanoma', 'acute myeloid leukemia'], ['blood protein measurement', 'platelet count', 'systolic blood pressure', 'migraine disorder', 'diastolic blood pressure'], ['Familial renal amyloidosis', 'AL amyloidosis', 'Familial renal amyloidosis due to lysozyme variant', 'monocyte count', 'monocyte percentage of leukocytes'], ['Noonan syndrome', 'Neurofibromatosis type 3', 'genetic disorder', 'rasopathy', 'Endometrial Endometrioid Adenocarcinoma'], ['total cholesterol measurement', 'monocyte count', 'low density lipoprotein cholesterol measurement', 'appendicular lean mass', 'eosinophil count'], ['adolescent idiopathic scoliosis', 'childhood onset asthma', 'alkaline phosphatase measurement', 'heel bone mineral density', 'hepatocellular carcinoma'], ['coronary artery disease', 'refractive error measurement', 'Abnormality of refraction', 'systolic blood pressure', 'lung adenocarcinoma'], ['Fanconi anemia', 'acute myeloid leukemia', 'colorectal carcinoma', 'breast adenocarcinoma', 'Ewing sarcoma'], ["Crohn's disease", 'ulcerative colitis', 'hepatocellular carcinoma', 'Combined immunodeficiency with skin granulomas', 'Hyper-IgM syndrome type 3'], ['Aymé-Gripp syndrome', 'Non-syndromic congenital cataract', 'Pulverulent cataract', 'Cerulean cataract', 'Early-onset non-syndromic cataract'], ['Autosomal recessive spastic paraplegia type 75', 'Autosomal recessive complex spastic paraplegia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'Hereditary spastic paraplegia'], ['hepatocellular carcinoma', 'neoplasm', 'non-small cell lung carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm'], ['X-linked immunodeficiency with magnesium defect, Epstein-Barr virus infection and neoplasia', 'congenital disorder of glycosylation type I', 'Combined T and B cell immunodeficiency', 'Congenital disorder of glycosylation', 'COVID-19'], ['Retinitis pigmentosa', 'Posterior column ataxia - retinitis pigmentosa', 'Retinal dystrophy', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect'], ['Combined immunodeficiency due to MALT1 deficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm'], ['sex hormone-binding globulin measurement', 'blood protein measurement', 'aspartate aminotransferase measurement', 'arterial stiffness measurement', 'social anxiety disorder'], ["Parkinson's disease", 'unipolar depression', 'depressive disorder', 'bipolar disorder', 'major depressive disorder'], ['Cardiofaciocutaneous syndrome', 'Noonan syndrome', 'cutaneous melanoma', 'melanoma', 'neoplasm'], ['Cardiofaciocutaneous syndrome', 'Heart-hand syndrome type 3', 'Noonan syndrome', 'cancer', 'cutaneous melanoma'], ['body mass index', 'body fat percentage', 'lean body mass', 'waist circumference', 'fat body mass'], ['gastric adenocarcinoma', 'anthrax infection', 'appendicular lean mass', 'lean body mass', 'type II diabetes mellitus'], ['protein C measurement', 'body height', 'self reported educational attainment', 'blood protein measurement', 'neoplasm'], ['eosinophil count', 'neutrophil count', 'myeloid white cell count', 'monocyte count', 'high density lipoprotein cholesterol measurement'], ['Familial gastric cancer', 'gastric cancer', 'neoplasm', 'cancer', 'myeloid white cell count'], ["Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'C-reactive protein measurement', 'ankylosing spondylitis'], ['birth weight', 'C-reactive protein measurement', 'resting heart rate', "Parkinson's disease", 'Homozygous familial hypercholesterolemia'], ['NIK deficiency', 'monocyte count', 'Combined immunodeficiency', 'monocyte percentage of leukocytes', 'eosinophil count'], ['myopathy, centronuclear, 6, with fiber-type disproportion', 'Autosomal recessive centronuclear myopathy', 'Congenital fiber-type disproportion myopathy', 'mathematical ability', 'FEV/FEC ratio'], ['neoplasm', 'Immunodeficiency due to an early component of complement deficiency', "Hashimoto's thyroiditis", 'Autosomal recessive systemic lupus erythematosus', 'complement component C1s deficiency'], ['leukocyte count', "Parkinson's disease", 'childhood onset asthma', 'body height', 'neutrophil count'], ['mean platelet volume', 'hemoglobin measurement', 'body fat distribution', 'erythrocyte count', 'lean body mass'], ['Noonan syndrome', 'cancer', 'head and neck squamous cell carcinoma', 'cervical squamous cell carcinoma', 'hypertrophic cardiomyopathy'], ['cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy', 'Costello syndrome', 'Cardiofaciocutaneous syndrome'], ['low grade glioma', 'glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'diastolic blood pressure'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['ovarian cancer', 'grip strength measurement', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['sex hormone-binding globulin measurement', 'acute myeloid leukemia', 'testosterone measurement', 'breast adenocarcinoma', 'Epileptic encephalopathy'], ['cutaneous melanoma', 'drug use measurement', 'osteosarcoma', 'Hernia', 'unipolar depression'], ['dilated cardiomyopathy', 'acute coronary syndrome', 'COVID-19', 'ovarian cancer', 'endometrial cancer'], ['MAP kinase-activated protein kinase 2 measurement', 'mean corpuscular hemoglobin concentration', 'glioblastoma multiforme', 'lung adenocarcinoma', 'mean corpuscular hemoglobin'], ['Frontotemporal dementia', 'Pick disease', 'Classical progressive supranuclear palsy', 'Progressive supranuclear palsy', 'Semantic dementia'], ['infect', 'infection', 'anxiety', 'dilated cardiomyopathy', 'Familial congenital mirror movements'], ['pancreatic adenocarcinoma', 'neoplasm', 'melanoma', 'gastric cancer', 'Malignant Pancreatic Neoplasm'], ['severe early-onset pulmonary alveolar proteinosis due to MARS deficiency', 'Autosomal dominant Charcot-Marie-Tooth disease type 2 due to MARS mutation', 'Acute infantile liver failure-multisystemic involvement syndrome', 'Charcot-Marie-Tooth disease', 'Autosomal recessive spastic paraplegia type 70'], ['risky sexual behaviour measurement', 'alcohol dependence', 'Microcephalic primordial dwarfism due to ZNF335 deficiency', 'Polymicrogyria due to TUBB2B mutation', 'Spinocerebellar ataxia type 17'], ['Brain demyelination due to methionine adenosyltransferase deficiency', 'hepatic methionine adenosyltransferase deficiency', 'Psychomotor retardation due to S-adenosylhomocysteine hydrolase deficiency', 'vitamin D measurement', 'hematocrit'], ['platelet count', 'prostate carcinoma', 'platelet crit', 'coronary artery disease', 'Myocardial Ischemia'], ['breast adenocarcinoma', 'smoking status measurement', 'reticulocyte count', 'chronotype measurement', 'gonorrhea'], ['body height', 'monocyte count', 'high density lipoprotein cholesterol measurement', 'myeloid white cell count', 'breast adenocarcinoma'], ['Hereditary pheochromocytoma-paraganglioma', 'adrenal gland pheochromocytoma', 'multiple myeloma', 'Inherited cancer-predisposing syndrome', 'Gastrointestinal stromal tumor'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement', 'breast cancer', 'neoplasm'], ['mean corpuscular hemoglobin', 'leukocyte count', 'mean corpuscular volume', 'neutrophil count', 'mean corpuscular hemoglobin concentration'], ['Pyruvate dehydrogenase E3-binding protein deficiency', 'mannose-binding protein C measurement', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['Fuchs endothelial corneal dystrophy', 'neutrophil count', 'testosterone measurement', 'myeloid white cell count', 'type II diabetes mellitus'], ['endometrial cancer', 'balding measurement', 'hair shape measurement', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy'], ['lymphocyte count', 'poisoning', 'brain volume measurement', 'neuroimaging measurement', 'facial asymmetry measurement'], ['spondyloepiphyseal dysplasia, Kondo-Fu type', 'cutaneous melanoma', 'colorectal carcinoma', 'lung adenocarcinoma', 'drug use measurement'], ['Oculocutaneous albinism type 2', 'Familial melanoma', 'Erythropoietic protoporphyria', 'skin pigmentation', 'Large congenital melanocytic nevus'], ['Familial glucocorticoid deficiency', 'multiple sclerosis', 'systemic lupus erythematosus', 'rheumatoid arthritis', 'Sarcoidosis'], ['age at menarche', 'age at first facial hair', 'body height', 'lean body mass', 'appendicular lean mass'], ['obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'sexual dysfunction', 'body mass index', 'type II diabetes mellitus'], ['DNA methylation', 'ovarian cancer', 'glioblastoma multiforme', 'fat body mass', 'hip bone mineral density'], ['platelet crit', 'platelet count', 'neoplasm', 'melanoma', 'cancer'], ['Autosomal recessive isolated optic atrophy', 'neoplasm', 'prostate carcinoma', 'colorectal carcinoma', 'cancer'], ['colorectal adenocarcinoma', 'colonic neoplasm', 'protein measurement', 'waist-hip ratio', 'triglyceride measurement'], ['Methylmalonic acidemia due to methylmalonyl-CoA epimerase deficiency', 'N-acetyl-D-glucosamine kinase measurement', 'emotional symptom measurement', 'Methylmalonic acidemia with homocystinuria', 'eosinophil count'], ['coronary artery disease', 'systolic blood pressure', 'pulse pressure measurement', 'mean platelet volume', 'cardiovascular disease'], ['type II diabetes mellitus', 'liver dysplastic nodule', 'skin aging', 'skin aging measurement', 'heel bone mineral density'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'neuroticism measurement', 'Normosmic congenital hypogonadotropic hypogonadism'], ['alcohol consumption measurement', 'smoking status measurement', 'low density lipoprotein cholesterol measurement', 'lean body mass', 'alcohol drinking'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'eosinophil percentage of leukocytes', 'eosinophil count', 'type II diabetes mellitus'], ['small cell lung carcinoma', 'colorectal carcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['primary ovarian insufficiency', 'age at menopause', 'uterine fibroid', 'anti-Mullerian hormone measurement', 'Menorrhagia'], ['Autosomal recessive primary microcephaly', 'Premature chromosome condensation with microcephaly and intellectual disability', 'Isolated congenital microcephaly', 'Primary microcephaly', 'intraocular pressure measurement'], ['neoplasm', 'colorectal carcinoma', 'breast cancer', 'melanoma', 'cancer'], ['lymphocyte count', 'emotional symptom measurement', 'total cholesterol measurement', 'red blood cell distribution width', 'breast cancer'], ['Early infantile epileptic encephalopathy', 'Hereditary pheochromocytoma-paraganglioma', 'mean platelet volume', 'eosinophil count', 'neutrophil count'], ['hepatocellular carcinoma', 'glioma', 'non-small cell lung carcinoma', 'pancreatic carcinoma', 'pancreatic neoplasm'], ['Lessel-Kubisch syndrome', 'Li-Fraumeni syndrome', 'basal cell carcinoma', 'lymphoid neoplasm', 'prostate carcinoma'], ['kidney neoplasm', 'basal cell carcinoma', 'hemangioblastoma', 'carcinoma of liver and intrahepatic biliary tract', 'lymphoid neoplasm'], ['platelet-derived growth factor BB measurement', 'neoplasm', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'cancer', 'breast cancer'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'neoplasm', 'blood pressure', 'childhood onset asthma'], ['hemoglobin measurement', 'body height', 'hematocrit', 'systemic juvenile idiopathic arthritis', 'Abnormal sperm morphology'], ['Rett syndrome', 'Atypical Rett syndrome', 'X-linked intellectual disability - psychosis - macroorchidism', 'Severe neonatal-onset encephalopathy with microcephaly', 'syndromic X-linked intellectual disability Lubs type'], ['X-linked intellectual disability with marfanoid habitus', 'Blepharophimosis-intellectual disability syndrome, MKB type', 'fg syndrome', 'Intellectual disability', 'mental retardation'], ['intellectual developmental disorder 62', 'genetic disorder', 'intellectual developmental disorder 61', 'optic cup area measurement', 'mathematical ability'], ['Familial Mediterranean fever', 'sweet syndrome', "Behcet's syndrome", 'genetic disorder', 'Abnormal cerebral white matter morphology'], ['neoplasm', 'cancer', 'breast cancer', 'hepatocellular carcinoma', 'gastric cancer'], ["Alzheimer's disease", 'thyroid cancer', 'mathematical ability', 'self reported educational attainment', 'body fat percentage'], ['Multiple endocrine neoplasia type 1', 'parathyroid adenoma', 'lung carcinoid tumor', 'Familial isolated hyperparathyroidism', 'Multiple endocrine neoplasia'], ['platelet component distribution width', 'platelet count', 'platelet crit', 'self reported educational attainment', 'heel bone mineral density'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'otosclerosis', 'lean body mass'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Leber congenital amaurosis', 'appendicular lean mass', 'systolic blood pressure'], ['papillary renal cell carcinoma', 'Familial papillary renal cell carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'non-small cell lung carcinoma'], ['reticulocyte count', 'type II diabetes mellitus', 'reticulocyte measurement', 'C-reactive protein measurement', 'sex hormone-binding globulin measurement'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'infection'], ['AL amyloidosis', 'myocardial infarction', 'coronary atherosclerosis', 'Myocardial Ischemia', 'angina pectoris'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2A2', 'Hereditary motor and sensory neuropathy type 6', 'Autosomal recessive axonal Charcot-Marie-Tooth disease type 2', 'Charcot-Marie-Tooth disease', 'genetic disorder'], ['type II diabetes mellitus', 'diabetes mellitus', 'abnormal glucose tolerance', 'chronic hepatitis C virus infection', 'hypertension'], ['COVID-19', 'severe acute respiratory syndrome', 'apolipoprotein A 1 measurement', 'type II diabetes mellitus', 'sex hormone-binding globulin measurement'], ['MGAT2-CDG', 'congenital disorder of glycosylation type II', 'COVID-19', 'Abnormal facial shape', 'Global developmental delay'], ['serum IgG glycosylation measurement', 'cognitive function measurement', 'reticulocyte count', 'reticulocyte measurement', 'leukocyte count'], ['N-glycan measurement', 'alkaline phosphatase measurement', 'COVID-19', 'eosinophil count', 'schizophrenia'], ['breast adenocarcinoma', 'musculoskeletal system disease', 'Myotonic syndrome', 'neoplasm', 'hepatocellular carcinoma'], ['Keutel syndrome', 'grip strength measurement', 'blood protein measurement', 'osteoarthritis, hand', 'heel bone mineral density'], ['melanoma-derived growth regulatory protein measurement', 'obesity', 'blood protein measurement', 'protein measurement', 'erythrocyte count'], ['combined oxidative phosphorylation deficiency', '3-methylglutaconic aciduria type 3', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'liver disease'], ['Proximal myopathy with extrapyramidal signs', 'genetic disorder', 'C-reactive protein measurement', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'systemic juvenile idiopathic arthritis', 'protein measurement', 'leukocyte count'], ['Non-syndromic congenital cataract', 'Nuclear cataract', 'Total congenital cataract', 'Cerulean cataract', 'Cataract with Y-shaped suture opacities'], ['Combined oxidative phosphorylation defect type 11', 'Mitochondrial disorder due to a defect in mitochondrial protein synthesis', 'cardiomyopathy', 'Left ventricular noncompaction', 'Infantile muscular hypotonia'], ['psoriasis', 'vulvar intraepithelial neoplasia', 'colorectal carcinoma', 'neoplasm', 'obesity'], ['Waardenburg syndrome', 'Tietz syndrome', 'Waardenburg syndrome type 2', 'coloboma, osteopetrosis, microphthalmia, macrocephaly, albinism, and deafness', 'Ocular albinism with congenital sensorineural deafness'], ['mean platelet volume', 'sex hormone-binding globulin measurement', 'red blood cell distribution width', 'mean corpuscular volume', 'myeloid white cell count'], ['cutaneous melanoma', 'lung adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'Niemann-Pick disease type C'], ['cleft lip', 'Cleft palate', 'mathematical ability', 'reticulocyte count', 'neoplasm'], ['Meckel syndrome', 'Joubert syndrome with ocular defect', 'Bardet-Biedl syndrome', 'Joubert syndrome', 'polydactyly (disease)'], ['serum gamma-glutamyl transferase measurement', 'lean body mass', 'pulse pressure measurement', 'serum alanine aminotransferase measurement', 'acute myeloid leukemia'], ['B-cell acute lymphoblastic leukemia', 'hemoglobin measurement', 'platelet component distribution width', 'hematocrit', 'glomerular filtration rate'], ['glioblastoma multiforme', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'appendicular lean mass'], ['Malonic aciduria', 'hypertrophic cardiomyopathy', 'mental retardation', 'Intellectual disability', 'adolescent idiopathic scoliosis'], ['Vitamin B12-responsive methylmalonic acidemia type cblB', 'methylmalonic aciduria cblb type', 'Vitamin B12-responsive methylmalonic acidemia', 'Methylmalonic acidemia with homocystinuria', 'Methylmalonic acidemia without homocystinuria'], ['heel bone mineral density', 'Jaundice', 'age at menarche', 'hip circumference', 'BMI-adjusted waist-hip ratio'], ['charcot-marie-tooth disease, axonal, type 2t', 'Spinocerebellar ataxia type 43', 'Autosomal dominant cerebellar ataxia', 'heart failure', 'Reduced ejection fraction'], ['celiac disease', 'multiple sclerosis', 'rheumatoid arthritis', 'autoimmune disease', 'sclerosing cholangitis'], ['acne', 'rosacea', 'COVID-19', 'Helicobacter pylori infectious disease', 'chronic obstructive pulmonary disease'], ['Torg-Winchester syndrome', 'Multicentric osteolysis-nodulosis-arthropathy spectrum', 'Nodulosis-arthropathy-osteolysis syndrome', 'lip cancer', 'heel bone mineral density'], ['coronary artery disease', 'matrix metalloproteinase measurement', 'blood protein measurement', 'lung cancer', "Alzheimer's disease"], ['acne', 'chronic obstructive pulmonary disease', 'rosacea', 'periodontitis', 'COVID-19'], ['Metaphyseal anadysplasia', 'lumbar disc degeneration', 'lung cancer', 'gastric adenocarcinoma', 'breast cancer'], ['ulcerative colitis', 'pterygium', 'lung adenocarcinoma', 'breast adenocarcinoma', 'serum IgG glycosylation measurement'], ['lung cancer', 'breast cancer', 'matrix metalloproteinase 12 measurement', 'blood protein measurement', 'matrix metalloproteinase 10 measurement'], ['Spondyloepimetaphyseal dysplasia, Missouri type', 'Metaphyseal anadysplasia', 'Metaphyseal chondrodysplasia, Spahr type', 'acne', 'rosacea'], ['schizophrenia', 'appendicular lean mass', 'heel bone mineral density', 'anorexia nervosa', 'BMI-adjusted waist-hip ratio'], ['insomnia measurement', 'arterial stiffness measurement', 'appendicular lean mass', 'breast adenocarcinoma', 'drug use measurement'], ['self reported educational attainment', 'mathematical ability', 'diet measurement', 'alcohol consumption measurement', 'appendicular lean mass'], ['Xanthinuria type II', 'Hereditary xanthinuria', 'Primary hyperoxaluria type 1', 'Homozygous familial hypercholesterolemia', 'Interauricular communication'], ['hematocrit', 'hemoglobin measurement', 'high density lipoprotein cholesterol measurement', 'erythrocyte count', 'red blood cell density measurement'], ['triglyceride measurement', 'balding measurement', 'body height', 'sex hormone-binding globulin measurement', 'rheumatoid arthritis'], ['reticulocyte count', 'refractive error measurement', 'appendicular lean mass', 'body height', 'monocyte count'], ['body height', 'sitting height measurement', 'isolated hyperchlorhidrosis', 'glioma', 'waist-hip ratio'], ['infect', 'infection', 'protein measurement', 'body height', 'Acute hepatic failure'], ['breast adenocarcinoma', 'blood protein measurement', 'glioblastoma multiforme', 'adolescent idiopathic scoliosis', 'mean corpuscular volume'], ['lean body mass', 'body height', 'fat body mass', 'cutaneous melanoma', 'telomere length'], ['thrombocytopenia, anemia, and myelofibrosis', 'Thrombocytopenia', 'blood platelet disease', 'blood protein measurement', 'Abnormal bleeding'], ['Congenital amegakaryocytic thrombocytopenia', 'Familial thrombocytosis', 'thrombocythemia 2', 'essential thrombocythemia', 'Thrombocytopenia'], ['Myeloperoxidase deficiency', 'neutrophil count', 'basophil count', 'myeloid white cell count', 'neutrophil percentage of leukocytes'], ['cancer', 'hemoglobin measurement', 'hematocrit', 'lung adenocarcinoma', 'mean corpuscular hemoglobin concentration'], ['red blood cell distribution width', 'protein measurement', 'eye colour measurement', 'mean corpuscular hemoglobin concentration', 'schizophrenia'], ['Navajo neurohepatopathy', 'Autosomal recessive axonal Charcot-Marie-Tooth disease type 2', 'Mitochondrial DNA depletion syndrome', 'Mitochondrial DNA depletion syndrome, hepatocerebral form', 'Hepatic failure'], ['Charcot-Marie-Tooth disease type 1B', 'Autosomal dominant Charcot-Marie-Tooth disease type 2I', 'Dejerine-Sottas syndrome', 'Autosomal dominant Charcot-Marie-Tooth disease type 2J', 'Autosomal dominant intermediate Charcot-Marie-Tooth disease type D'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness', 'Non-syndromic genetic deafness', 'erythrocyte count', 'non-small cell lung carcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'brain volume measurement', 'eosinophil count', 'neuroimaging measurement', 'cortical thickness'], ['Familial glucocorticoid deficiency', 'Cytomegalic congenital adrenal hypoplasia', 'hypertension', 'blood pressure', 'aortic stenosis'], ['Tuberculosis', 'myeloid white cell count', 'neutrophil count', 'monocyte count', 'heel bone mineral density'], ['reticulocyte count', 'protein measurement', 'body mass index', 'body height', 'hip circumference'], ['risk-taking behaviour', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'Nuclear cataract', 'Total congenital cataract'], ['Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract', 'Coralliform cataract'], ['pulse pressure measurement', 'systolic blood pressure', 'Cholesterol-ester transfer protein deficiency', 'high density lipoprotein cholesterol measurement', 'childhood onset asthma'], ['chronic obstructive pulmonary disease', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'colorectal carcinoma', 'neoplasm'], ['alkaline phosphatase measurement', 'liver enzyme measurement', 'non-alcoholic steatohepatitis', 'glomerular filtration rate', 'miotic rate'], ['schizophrenia', 'melanoma', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'Hepatobiliary Neoplasm'], ['Common variable immunodeficiency', 'rheumatoid arthritis', 'non-Hodgkins lymphoma', 'multiple sclerosis', 'chronic lymphocytic leukemia'], ['cutaneous melanoma', 'adolescent idiopathic scoliosis', 'nevus count', 'body height', 'fatty acid measurement'], ['cancer antigen 125 measurement', 'erythrocyte count', 'pancreatic carcinoma', 'neoplasm', 'mesothelioma'], ['prostate carcinoma', 'prostate specific antigen measurement', 'male reproductive organ cancer', 'blood protein measurement', 'cancer biomarker measurement'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'Primary immunodeficiency', 'esophageal cancer', 'anaplastic large cell lymphoma'], ["Barrett's esophagus", 'prostate carcinoma', 'prostate adenocarcinoma', 'Familial prostate cancer', 'esophageal adenocarcinoma'], ['ulcerative colitis', "Crohn's disease", 'inflammatory bowel disease', 'sclerosing cholangitis', 'psoriasis'], ['nasopharyngeal neoplasm', 'body mass index', 'multiple myeloma', 'body fat percentage', 'age at first birth measurement'], ['Myostatin-related muscle hypertrophy', 'Duchenne muscular dystrophy', 'cancer', 'self reported educational attainment', 'muscular disease'], ['mitochondrial myopathy-cerebellar ataxia-pigmentary retinopathy syndrome', 'Mitochondrial myopathy', 'Mitochondrial myopathy with reversible cytochrome C oxidase deficiency', 'genetic disorder', 'gastric carcinoma'], ['Craniosynostosis, Boston type', 'Parietal foramina', 'Parietal foramina with cleidocranial dysplasia', 'hair color', 'Craniosynostosis'], ['Isolated cytochrome C oxidase deficiency', 'Mitochondrial non-syndromic sensorineural deafness', 'Leber hereditary optic neuropathy', 'MELAS', 'Mitochondrial non-syndromic sensorineural deafness with susceptibility to aminoglycoside exposure'], ['Isolated cytochrome C oxidase deficiency', 'MELAS', 'Mitochondrial disease', 'Kearns-Sayre syndrome', 'leigh syndrome due to mitochondrial complex iv deficiency'], ['Leber hereditary optic neuropathy', 'leigh syndrome due to mitochondrial complex iv deficiency', 'Isolated cytochrome C oxidase deficiency', 'MELAS', 'Leigh syndrome'], ['adolescent idiopathic scoliosis', 'hepatocellular carcinoma', 'medulloblastoma', 'Kidney Oncocytoma', 'ulcerative colitis'], ['endometrial cancer', 'breast adenocarcinoma', 'acute myeloid leukemia', 'low grade glioma', 'glioblastoma multiforme'], ['Diaphyseal medullary stenosis - bone malignancy', 'cutaneous melanoma', 'nevus count', 'skin cancer', 'protein measurement'], ['Left ventricular noncompaction', 'DNA methylation', 'hypertrophic cardiomyopathy 4', 'Glycogen storage disease due to LAMP-2 deficiency', 'Glycogen Storage Disease Type 2b'], ['combined immunodeficiency and megaloblastic anemia with or without hyperhomocysteinemia', 'Neural tube closure defect', 'Isolated spina bifida', 'colorectal adenocarcinoma', 'colorectal neoplasm'], ['mean platelet volume', 'cancer', 'neoplasm', 'acute myeloid leukemia', 'non-small cell lung carcinoma'], ['reticulocyte count', 'cervical cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'Tibial muscular dystrophy'], ['hemoglobin measurement', 'macula measurement', 'breast adenocarcinoma', 'glioblastoma multiforme', 'type II diabetes mellitus'], ['insomnia', 'delirium', 'schizophrenia', 'major depressive disorder', 'bipolar disorder'], ['insomnia', 'delirium', 'schizophrenia', 'major depressive disorder', 'bipolar disorder'], ['Isolated focal cortical dysplasia type II', 'Macrocephaly-intellectual disability-neurodevelopmental disorder-small thorax syndrome', 'Isolated focal cortical dysplasia type IIa', 'Isolated focal cortical dysplasia type IIb', 'Isolated focal cortical dysplasia'], ['platelet component distribution width', 'platelet count', 'mean platelet volume', 'left ventricular hypertrophy', 'primary hypertension'], ['neoplasm', 'appendicular lean mass', 'electrocardiography', 'mean platelet volume', 'hemorrhoid'], ['Abetalipoproteinemia', 'familial hypercholesterolemia', 'cardiovascular disease', 'Homozygous familial hypercholesterolemia', 'vital capacity'], ['erythrocyte count', 'hemoglobin measurement', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'gastric carcinoma'], ['Autosomal dominant medullary cystic kidney disease with or without hyperuricemia', 'Autosomal dominant medullary cystic kidney disease without hyperuricemia', 'lung adenocarcinoma', 'ovarian cancer', 'peritoneum cancer'], ['adult onset asthma', 'interstitial lung disease', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['glomerular filtration rate', 'mean corpuscular hemoglobin', 'head and neck squamous cell carcinoma', 'pancreatic adenocarcinoma', 'red blood cell distribution width'], ['asthma', 'pancreatic carcinoma', 'eosinophil count', 'breast adenocarcinoma', 'cholelithiasis'], ['asthma', 'inherited susceptibility to asthma', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['cancer antigen 125 measurement', 'ovarian cancer', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma'], ['glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma', 'breast adenocarcinoma'], ['glioblastoma multiforme', 'body height', 'protein measurement', 'esophageal cancer', 'Blackfan-Diamond anemia'], ['Congenital myasthenic syndromes', 'Fetal akinesia deformation sequence', 'Postsynaptic congenital myasthenic syndromes', 'heel bone mineral density', 'Respiratory insufficiency'], ['Disseminated superficial actinic porokeratosis', 'mean corpuscular hemoglobin', 'neuroblastoma', 'esophageal squamous cell carcinoma', 'breast carcinoma'], ['cancer', 'diabetes mellitus', 'heart failure', 'mortality', 'stroke'], ['melanoma', 'cutaneous melanoma', 'nevus count', 'thyroid gland adenocarcinoma', 'blood protein measurement'], ['basophil count', 'neutrophil count', 'mean platelet volume', 'leukocyte count', 'monocyte count'], ['hypertrophic cardiomyopathy', 'hypertrophic cardiomyopathy 4', 'Left ventricular noncompaction', 'Familial dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy'], ['drug use measurement', 'blood protein measurement', 'systolic blood pressure', 'YKL40 measurement', 'protein measurement'], ['PR interval', 'total cholesterol measurement', 'jaw fracture', 'apolipoprotein B measurement', 'glomerular filtration rate'], ['Burkitts lymphoma', 'prostate carcinoma', 'bladder carcinoma', 'diffuse large B-cell lymphoma', 'lung adenocarcinoma'], ['Other immunodeficiency syndrome with predominantly antibody defects', 'prostate carcinoma', 'acute lymphoblastic leukemia', 'colonic neoplasm', 'colon carcinoma'], ['Feingold syndrome type 1', 'Feingold syndrome', 'basal cell carcinoma', 'genetic disorder', 'kidney Wilms tumor'], ['Pyogenic bacterial infections due to MyD88 deficiency', 'chronic lymphocytic leukemia', 'Primary immunodeficiency', 'non-Hodgkins lymphoma', 'diffuse large B-cell lymphoma'], ['hepatocellular carcinoma', 'neoplasm', 'rheumatoid arthritis', 'optic cup area measurement', 'refractory anemia'], ['myopathy, proximal, and ophthalmoplegia', 'Hereditary inclusion body myopathy - joint contractures - ophthalmoplegia', 'inclusion body myositis', 'Childhood-onset autosomal recessive myopathy with external ophthalmoplegia', 'cutaneous Leishmaniasis'], ['Familial atrial fibrillation', 'heart failure', 'Reduced ejection fraction', 'breast adenocarcinoma', 'body fat percentage'], ['lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'Familial visceral myopathy', 'endometrial cancer'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'Hemoglobin E - beta-thalassemia'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Rare genetic vascular disease', 'Megacystis-microcolon-intestinal hypoperistalsis syndrome', 'Familial visceral myopathy', 'Rare disease with thoracic aortic aneurysm and aortic dissection'], ['hypertrophic cardiomyopathy', 'cardiomyopathy', 'platelet count', 'Rare familial disorder with hypertrophic cardiomyopathy', 'ventricular tachycardia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'cancer'], ['heel bone mineral density', 'Familial primary pulmonary hypoplasia', 'Postsynaptic congenital myasthenic syndromes', 'Hereditary proximal myopathy with early respiratory failure', 'Distal spinal muscular atrophy type 3'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'cutaneous Leishmaniasis', 'endometrial cancer', 'breast adenocarcinoma', 'mean platelet volume'], ['lymphocyte count', 'leukocyte count', 'neutrophil count', 'myeloid white cell count', 'monocyte count'], ['experimental autoimmune encephalomyelitis', 'stroke', 'systemic lupus erythematosus', 'infection', 'infect'], ['systolic blood pressure', 'hypertension', 'blood pressure', 'cardiovascular disease', 'diet measurement'], ['N-acylethanolamine-hydrolyzing acid amidase measurement', 'blood protein measurement', 'cerebrospinal fluid biomarker measurement', 'protein measurement', 'neoplasm'], ['Sanfilippo syndrome type B', 'Mucopolysaccharidosis type 3', 'Charcot-Marie-Tooth disease axonal type 2V', 'Mucopolysaccharidosis', 'genetic disorder'], ['Hyperammonemia due to N-acetylglutamate synthetase deficiency', 'Hyperammonemia', 'selective IgA deficiency disease', 'apolipoprotein B measurement', 'squamous cell lung carcinoma'], ['age at menopause', 'protein measurement', 'mean platelet volume', 'lung adenocarcinoma', 'neutrophil count'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'cutaneous melanoma', 'lung adenocarcinoma', 'neoplasm'], ['smoking behavior', '3-hydroxy-1-methylpropylmercapturic acid measurement', 'hepatocellular carcinoma', 'obsessive-compulsive disorder', 'Hereditary geniospasm'], ['protein measurement', 'red blood cell distribution width', 'schizophrenia', 'interleukin 18 measurement', 'neoplasm'], ['Neurodevelopmental disorder', 'Isolated NADH-CoQ reductase deficiency', 'hemoglobin measurement', 'erythrocyte count', 'hematocrit'], ['glomerular filtration rate', 'melanoma', 'platelet count', 'vitamin D measurement', 'protein measurement'], ['total cholesterol measurement', 'triglyceride measurement', 'urinary metabolite measurement', 'chronic kidney disease', 'mean corpuscular hemoglobin concentration'], ['chronic kidney disease', 'urinary metabolite measurement', 'blood metabolite measurement', 'glomerular filtration rate', 'hydroxy-leucine measurement'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'optic disc size measurement', 'pulse pressure measurement'], ['Gray platelet syndrome', 'Thrombocytopenia', 'Abnormal bleeding', 'granulocyte percentage of myeloid white cells', 'neutrophil percentage of leukocytes'], ['body mass index', 'neuroimaging measurement', 'refractive error measurement', 'waist-hip ratio', 'brain measurement'], ['Nijmegen breakage syndrome', 'idiopathic aplastic anemia', 'acute lymphoblastic leukemia', 'Aplastic anemia', 'Inherited cancer-predisposing syndrome'], ['neoplasm', 'atrial fibrillation', 'endometriosis', 'cancer', 'Intermediate osteopetrosis'], ['bone quantitative ultrasound measurement', 'smoking behavior', 'smoking status measurement', 'heel bone mineral density', 'body mass index'], ['platelet crit', 'mosaic loss of chromosome Y measurement', 'glioblastoma multiforme', 'endometrial cancer', 'small cell lung carcinoma'], ['neuroimaging measurement', 'refractive error measurement', 'PR interval', 'brain volume measurement', 'brain measurement'], ['head and neck squamous cell carcinoma', 'melanoma', 'colorectal adenocarcinoma', 'colon adenocarcinoma', 'basal cell carcinoma'], ['appendicular lean mass', 'body height', 'lean body mass', 'vital capacity', 'C-reactive protein measurement'], ['breast carcinoma', 'alveolar rhabdomyosarcoma', 'embryonal rhabdomyosarcoma', 'diffuse large B-cell lymphoma', 'breast adenocarcinoma'], ['platelet count', 'osteoarthritis', 'high density lipoprotein cholesterol measurement', 'endometrial cancer', 'breast adenocarcinoma'], ['neuroticism measurement', 'wellbeing measurement', 'unipolar depression', 'erythrocyte count', 'mood disorder'], ['endometrial cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Irritability', 'glomerular filtration rate'], ['breast adenocarcinoma', 'cutaneous melanoma', 'thyroid neoplasm', 'colorectal adenocarcinoma', 'viral disease'], ['prostate adenocarcinoma', 'colorectal adenocarcinoma', 'body height', 'lean body mass', 'cutaneous melanoma'], ['viral disease', 'complement C4 measurement', 'visual perception measurement', 'mathematical ability', 'FEV/FEC ratio'], ['body mass index', 'type II diabetes mellitus', 'systolic blood pressure', 'blood pressure', 'alcohol drinking'], ['acne inversa, familial, 1', "Alzheimer's disease", 'hidradenitis suppurativa', 'Dementia', 'head and neck squamous cell carcinoma'], ['gut microbiome measurement', 'hepatocellular carcinoma', 'cancer', 'colorectal carcinoma', 'primary cutaneous amyloidosis'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['Norrie disease', 'Familial exudative vitreoretinopathy', 'Persistent hyperplastic primary vitreous', 'retinal detachment', 'Short lingual frenulum'], ['Charcot-Marie-Tooth disease type 4D', 'Charcot-Marie-Tooth disease', 'Charcot-Marie-Tooth disease type 4', 'colorectal adenocarcinoma', 'gastric carcinoma'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus', 'gestational diabetes'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus', 'gestational diabetes'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['Zlotogora-Ogur syndrome', 'EEC syndrome', 'cleft lip', 'Orofacial clefting syndrome', 'mathematical ability'], ["Alzheimer's disease", 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement', 'mean platelet volume', 'family history of Alzheimer’s disease'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Charcot-Marie-Tooth disease axonal type 2CC', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'thyroid gland adenocarcinoma'], ['Charcot-Marie-Tooth disease type 1F', 'Autosomal dominant Charcot-Marie-Tooth disease type 2E', 'Charcot-Marie-Tooth disease, dominant intermediate G', 'autosomal dominant intermediate Charcot-Marie-Tooth disease type G', 'Charcot-Marie-Tooth disease'], ['cancer', 'breast adenocarcinoma', 'platelet count', 'mean platelet volume', 'vital capacity'], ['HIV infection', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['HIV infection', 'breast adenocarcinoma', 'visual perception measurement', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['blood protein measurement', 'self reported educational attainment', 'adolescent idiopathic scoliosis', 'type II diabetes mellitus', 'acute myeloid leukemia'], ['fibula fracture', 'neoplasm', 'lymphocyte count', 'glioma', 'breast cancer'], ['sialidosis type II', 'sialidosis type I', 'sialidosis', 'Non-immune hydrops fetalis', 'Lysosomal disease'], ['atrial fibrillation', 'atrial flutter', 'white matter hyperintensity measurement', 'cardiac edema', 'cardiac arrhythmia'], ['MODY', 'maturity-onset diabetes of the young type 6', 'type II diabetes mellitus', 'monogenic diabetes', 'body mass index'], ['Congenital malabsorptive diarrhea due to paucity of enteroendocrine cells', 'type II diabetes mellitus', 'body height', 'drug use measurement', 'diabetes mellitus'], ['Neurofibromatosis type 1', 'Neurofibromatosis-Noonan syndrome', 'Neurofibromatosis type 1 due to NF1mutation or intragenic deletion', 'Juvenile Myelomonocytic Leukemia', 'plexiform neurofibroma'], ['Neurofibromatosis type 2', 'Neurofibromatosis type 3', 'schwannoma', 'mesothelioma', 'Meningioma'], ['platelet count', 'platelet crit', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell distribution width'], ['immunodeficiency, developmental delay, and hypohomocysteinemia', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'liver cancer', 'papillary renal cell carcinoma'], ['sex hormone-binding globulin measurement', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'skin aging measurement', 'high density lipoprotein cholesterol measurement'], ['Marshall-Smith syndrome', 'Sotos syndrome', 'Malan overgrowth syndrome', 'genetic disorder', 'lymphocyte count'], ['Common variable immunodeficiency', 'infectious disease', 'Primary immunodeficiency', 'eosinophil count', 'ulcerative colitis'], ['Common variable immunodeficiency', 'immunodeficiency, common variable, 10', 'Deficiency in anterior pituitary function-variable immunodeficiency syndrome', 'Primary immunodeficiency', 'colon adenocarcinoma'], ['protein measurement', 'stroke', 'mathematical ability', 'neoplasm', 'cancer'], ['Hereditary sensory and autonomic neuropathy type 5', 'erythrocyte count', 'diastolic blood pressure', 'hypertension', 'osteoarthritis'], ['Nance-Horan syndrome', 'Nuclear cataract', 'Non-syndromic congenital cataract', 'genetic disorder', 'Total congenital cataract'], ['mean platelet volume', 'Isolated Dandy-Walker malformation', 'blood protein measurement', 'waist-hip ratio', 'Hydrocephalus'], ['hypertension', 'schizophrenia', 'obesity', 'atrial fibrillation', 'nervous system disease'], ['acute myeloid leukemia', 'body mass index', 'hip circumference', 'Triple A syndrome', 'systemic lupus erythematosus'], ['Abnormality of refraction', 'Ewing sarcoma', 'chronotype measurement', 'type II diabetes mellitus', 'head and neck squamous cell carcinoma'], ['peak expiratory flow', 'type II diabetes mellitus', 'adolescent idiopathic scoliosis', 'cutaneous melanoma', 'colorectal carcinoma'], ['spastic ataxia 8, autosomal recessive, with hypomyelinating leukodystrophy', 'Autosomal recessive spastic ataxia with leukoencephalopathy', 'Hypomyelination - hypogonadotropic hypogonadism - hypodontia', 'visual perception measurement', 'migraine disorder'], ['body fat percentage', 'body mass index', 'lean body mass', 'smoking status measurement', 'type II diabetes mellitus'], ['blood protein measurement', 'anxiety', 'schizophrenia', 'Huntington disease-like 2', 'Huntington disease-like syndrome due to C9ORF72 expansions'], ['Corneal intraepithelial dyskeratosis with palmoplantar hyperkeratosis and laryngeal dyskeratosis', 'autoinflammation with arthritis and dyskeratosis', 'respiratory papillomatosis, juvenile recurrent, congenital', 'insulin like growth factor measurement', 'autoinflammatory syndrome'], ['venous thromboembolism', 'hepatocellular carcinoma', 'neoplasm', 'intrahepatic cholangiocarcinoma', 'laryngeal carcinoma'], ['CINCA syndrome', 'Familial cold urticaria', 'Muckle-Wells syndrome', 'keratitis fugax hereditaria', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['dilated cardiomyopathy', 'anorexia nervosa', 'bipolar disorder', 'schizophrenia', 'autism spectrum disorder'], ['blood protein measurement', 'melanoma', 'neoplasm', 'hepatocellular carcinoma', 'acute myeloid leukemia'], ['body height', 'sitting height measurement', 'white matter microstructure measurement', 'blood protein measurement', 'body weight'], ['type II diabetes mellitus', 'blood protein measurement', 'tumor necrosis factor-inducible gene 6 protein measurement', 'protein measurement', 'obesity'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Retinal dystrophy', 'Nystagmus', 'Diarrhea'], ['HIV infection', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'systolic blood pressure', 'endometrial cancer'], ['platelet count', 'central nervous system cancer', 'mean platelet volume', 'heel bone mineral density', 'monocyte percentage of leukocytes'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'heel bone mineral density'], ['smoking status measurement', 'body mass index', 'self reported educational attainment', 'neuroticism measurement', 'protein measurement'], ['mean corpuscular volume', 'reticulocyte count', 'reticulocyte measurement', 'myeloid white cell count', 'mean reticulocyte volume'], ['Blau syndrome', "Crohn's disease", 'ulcerative colitis', "Behcet's syndrome", 'inflammatory bowel disease'], ['Familial cortical myoclonus', 'non-small cell lung carcinoma', 'cancer', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'Familial hypofibrinogenemia'], ['protein measurement', 'schizophrenia', 'bladder carcinoma', 'colorectal cancer', 'type II diabetes mellitus'], ['psoriasis', 'psoriasis vulgaris', 'nitric oxide exhalation measurement', 'inflammatory bowel disease', 'eosinophil count'], ['Adams-Oliver syndrome', 'Familial bicuspid aortic valve', 'head and neck squamous cell carcinoma', 'aortic stenosis', 'cancer'], ['Acroosteolysis dominant type', 'Alagille syndrome', 'Alagille syndrome due to a NOTCH2 point mutation', 'basal cell carcinoma', 'prostate adenocarcinoma'], ['CADASIL', 'Lateral meningocele syndrome', 'Infantile myofibromatosis', 'migraine disorder', 'dementia (disease)'], ['colorectal cancer', 'chronic hepatitis B virus infection', 'head and neck squamous cell carcinoma', 'asthma', 'systemic scleroderma'], ['neurodevelopmental disorder with or without autistic features and/or structural brain abnormalities', 'Intellectual disability, severe', 'chronotype measurement', 'electrocardiography', 'non-small cell lung carcinoma'], ['lung adenocarcinoma', 'colon carcinoma', 'colonic neoplasm', 'colorectal adenocarcinoma', 'malignant colon neoplasm'], ['Ataxia-telangiectasia', 'eosinophil count', 'mean corpuscular volume', 'platelet count', 'eosinophil percentage of leukocytes'], ['Niemann-Pick disease type C', 'Niemann-Pick disease', 'genetic disorder', 'Aceruloplasminemia', 'Cognitive impairment'], ['Niemann-Pick disease type C', 'Niemann-Pick disease type C, severe early infantile neurologic onset', 'dystonic disorder', 'Niemann-Pick disease type C, late infantile neurologic onset', 'Niemann-Pick disease type C, severe perinatal form'], ['acute myeloid leukemia', 'mean platelet volume', 'low density lipoprotein cholesterol measurement', 'body height', 'total cholesterol measurement'], ['cutaneous melanoma', '3-hydroxy-3-methylglutaryl-CoA synthase deficiency', 'hyperproinsulinemia', 'Familial glucocorticoid deficiency', 'Exercise-induced hyperinsulinism'], ['Juvenile nephronophthisis', 'Joubert syndrome with renal defect', 'Nephronophthisis', 'Senior-Loken syndrome', 'Joubert syndrome'], ['Congenital nephrotic syndrome, Finnish type', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'familial nephrotic syndrome', 'focal segmental glomerulosclerosis', 'nephrotic syndrome'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'idiopathic nephrotic syndrome', 'response to prednisolone', 'Congenital nephrotic syndrome, Finnish type', 'Proteinuria'], ['Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency', 'Sitosterolemia', 'hypercholesterolemia, autosomal dominant, 3', 'Ataxia with vitamin E deficiency'], ['refractive error measurement', 'low grade glioma', 'breast adenocarcinoma', 'Astigmatism', 'red blood cell distribution width'], ['acute myeloid leukemia', 'HIV infection', 'Dyskeratosis congenita', 'anaplastic large cell lymphoma', 'Hodgkins lymphoma'], ['Familial atrial fibrillation', 'Atrial stand still', 'AL amyloidosis', 'systolic blood pressure', 'breast adenocarcinoma'], ['BNP measurement', 'diastolic blood pressure', 'smoking status measurement', 'alcohol drinking', 'pulse pressure measurement'], ['body height', 'lean body mass', 'appendicular lean mass', 'fat body mass', 'forced expiratory volume'], ['congestive heart failure', 'hypertension', 'heart failure', 'pulmonary hypertension', 'obesity'], ['Acromesomelic dysplasia, Maroteaux type', 'Tall stature - scoliosis - macrodactyly of the great toes', 'Metaphyseal chondrodysplasia, Schmid type', 'Familial focal epilepsy with variable foci', 'Shprintzen-Goldberg syndrome'], ['body height', 'lean body mass', 'systolic blood pressure', 'hypertension', 'appendicular lean mass'], ['chronotype measurement', 'bladder disease', 'asthma', 'protein measurement', "Alzheimer's disease"], ['body mass index', 'cognitive function measurement', 'body height', 'self reported educational attainment', 'forced expiratory volume'], ['mathematical ability', 'self reported educational attainment', 'wellbeing measurement', 'daytime rest measurement', 'triglyceride measurement'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'low grade glioma'], ['Severe early-onset obesity-insulin resistance syndrome due to SH2B1 deficiency', 'Obesity due to melanocortin 4 receptor deficiency', 'Obesity due to prohormone convertase I deficiency', 'Obesity due to pro-opiomelanocortin deficiency', 'Obesity due to leptin receptor gene deficiency'], ['type II diabetes mellitus', 'diabetes mellitus', 'body mass index', 'appendicular lean mass', 'fat body mass'], ['Progressive familial intrahepatic cholestasis', 'primary biliary cirrhosis', 'morbid obesity', 'Fatigue', 'cholelithiasis'], ['high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement', 'total cholesterol measurement', 'lymphocyte percentage of leukocytes', 'adolescent idiopathic scoliosis'], ['red blood cell distribution width', 'Kleefstra syndrome', 'neoplasm', 'myocarditis', 'blood metabolite measurement'], ['alcohol consumption measurement', 'acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'prostate cancer'], ['Goldmann-Favre syndrome', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Joubert syndrome and related disorders', 'Cone rod dystrophy'], ['Glucocorticoid resistance', 'atrial fibrillation', 'multiple myeloma', 'asthma', 'acute lymphoblastic leukemia'], ['Renal pseudohypoaldosteronism type 1', 'Pseudohypoaldosteronism type 1', 'Pseudohyperaldosteronism type 2', 'hypertension', 'chronic kidney disease'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'sitting height measurement'], ['lung carcinoma', 'colorectal adenocarcinoma', 'chondrosarcoma', 'carcinoma of liver and intrahepatic biliary tract', 'colon adenocarcinoma'], ['46,XY partial gonadal dysgenesis', 'primary ovarian insufficiency', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation', '46,XX testicular disorder of sex development', '46,XY complete gonadal dysgenesis'], ['pancreatic carcinoma', 'vital capacity', 'ulcerative colitis', 'hematocrit', 'hemoglobin measurement'], ['age at menarche', 'sex hormone-binding globulin measurement', 'brain aneurysm', 'alcohol consumption measurement', 'cutaneous melanoma'], ['Noonan syndrome', 'cutaneous melanoma', 'multiple myeloma', 'Juvenile Myelomonocytic Leukemia', 'acute myeloid leukemia'], ['Cytomegalic congenital adrenal hypoplasia', '46,XY complete gonadal dysgenesis', '46,XX testicular disorder of sex development', '46,XY partial gonadal dysgenesis', 'Alternating hemiplegia of childhood'], ['obesity', 'Genetic obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'renal cell carcinoma', "Barrett's esophagus"], ['platelet count', 'platelet component distribution width', 'red blood cell distribution width', 'mean platelet volume', 'platelet crit'], ['cancer', 'lung adenocarcinoma', 'pancreatic carcinoma', 'pancreatic ductal adenocarcinoma', 'breast ductal adenocarcinoma'], ['cancer', 'hair colour measurement', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['cancer', 'kidney cancer', 'kidney neoplasm', 'acute myeloid leukemia', 'adolescent idiopathic scoliosis'], ['cancer', 'hemoglobin measurement', 'hematocrit', 'glomerular filtration rate', 'urate measurement'], ['schizophrenia', 'autism spectrum disorder', 'unipolar depression', 'bipolar disorder', 'attention deficit hyperactivity disorder'], ['Retinitis pigmentosa', 'Goldmann-Favre syndrome', 'Retinal dystrophy', 'self reported educational attainment', 'smoking status measurement'], ['body mass index', 'self reported educational attainment', 'protein measurement', "Alzheimer's disease", 'non-small cell lung carcinoma'], ['waist-hip ratio', 'acute myeloid leukemia', 'blood protein measurement', 'neuroimaging measurement', 'high density lipoprotein cholesterol measurement'], ['balding measurement', 'FEV/FEC ratio', 'genetic disorder', 'adolescent idiopathic scoliosis', 'breast adenocarcinoma'], ['connective tissue disease', 'adolescent idiopathic scoliosis', 'infection', 'infect', 'Dementia'], ['acute lymphoblastic leukemia', 'Wolf-Hirschhorn syndrome', 'head and neck squamous cell carcinoma', 'Intellectual disability', 'kidney neoplasm'], ['balding measurement', "Parkinson's disease", 'cortical surface area measurement', 'total cortical area measurement', 'smoking status measurement'], ['hair colour measurement', 'adolescent idiopathic scoliosis', 'hair color', 'DNA methylation', 'neoplasm'], ['Autosomal recessive non-syndromic intellectual disability', 'Dubowitz syndrome', 'genetic disorder', 'DNA methylation', 'heel bone mineral density'], ['sex hormone-binding globulin measurement', 'serum gamma-glutamyl transferase measurement', 'neoplasm', 'cancer', 'lupus erythematosus'], ['cutaneous melanoma', 'systemic lupus erythematosus', 'neoplasm', 'inclusion body myositis', 'pancreatic neoplasm'], ['Autosomal recessive spastic paraplegia type 45', 'Autosomal recessive spastic paraplegia type 48', 'Intellectual disability', 'lymphoid neoplasm', 'T-cell acute lymphoblastic leukemia'], ['appendicular lean mass', 'reticulocyte count', 'blood protein measurement', 'reticulocyte measurement', 'mean corpuscular volume'], ['mean platelet volume', 'platelet count', 'birth measurement', 'gestational age', 'osteosarcoma'], ['Hereditary arterial and articular multiple calcification syndrome', 'metabolite measurement', 'blood metabolite measurement', 'heel bone mineral density', 'cutaneous melanoma'], ['insomnia', 'BMI-adjusted waist circumference', 'diastolic blood pressure', 'body height', 'glaucoma'], ['open-angle glaucoma', 'genetic disorder', 'colorectal carcinoma', 'proliferative diabetic retinopathy', 'X-linked reticulate pigmentary disorder with systemic manifestations'], ['self reported educational attainment', 'body mass index', 'smoking status measurement', 'health study participation', 'lifestyle measurement'], ['Hereditary sensory and autonomic neuropathy type 4', 'Familial medullary thyroid carcinoma', 'neoplasm', 'non-small cell lung carcinoma', 'cancer'], ['obesity, hyperphagia, and developmental delay', 'Early infantile epileptic encephalopathy', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['neoplasm', 'non-small cell lung carcinoma', 'cancer', 'metastasis', 'cutaneous melanoma'], ['mathematical ability', 'breast ductal adenocarcinoma', 'hair colour measurement', 'hepatocellular carcinoma', 'cancer'], ['low density lipoprotein cholesterol measurement', 'C-reactive protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['visual perception measurement', 'breast cancer', 'neoplasm', 'diffuse large B-cell lymphoma', 'Autosomal dominant severe congenital neutropenia'], ['refractive error measurement', 'mean platelet volume', 'eosinophil count', 'BMI-adjusted waist circumference', 'physical activity measurement'], ['mean corpuscular volume', 'chronic kidney disease', 'urinary metabolite measurement', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'Autosomal recessive primary microcephaly', 'HIV infection', 'influenza', 'viral disease'], ['influenza', 'HIV infection', 'viral disease', 'Histiocytosis', 'blood protein measurement'], ['encephalopathy, acute, infection-induced', 'HIV infection', 'influenza', 'viral disease', 'T-cell acute lymphoblastic leukemia'], ['mean platelet volume', 'appendicular lean mass', 'hepatocellular carcinoma', 'prostate cancer', 'urinary bladder cancer'], ['NUT midline carcinoma', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'non-small cell lung carcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['Autosomal recessive Robinow syndrome', 'Robinow syndrome', 'colorectal cancer', 'colorectal adenoma', 'refractive error measurement'], ['Gyrate atrophy of choroid and retina', 'Retinal dystrophy', 'Ornithine transcarbamylase deficiency', 'Leber hereditary optic neuropathy', 'Abnormal choroid morphology'], ['Congenital intrauterine infection-like syndrome', 'Autosomal recessive primary microcephaly', 'Cerebral calcification', 'Global developmental delay', 'glioblastoma multiforme'], ['trypanosomiasis', 'Alopecia', 'Global developmental delay', 'Macrocephaly', 'Intellectual disability'], ['mean platelet volume', 'platelet component distribution width', 'serum IgM measurement', 'IGA glomerulonephritis', 'chronic lymphocytic leukemia'], ['cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'intelligence', 'pulse pressure measurement'], ['body height', 'creatinine measurement', 'waist-hip ratio', 'BMI-adjusted waist circumference', 'prostate cancer'], ['cancer', "Alzheimer's disease", 'clear cell renal carcinoma', 'breast adenocarcinoma', 'lung cancer'], ['self reported educational attainment', 'myocardial infarction', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'adolescent idiopathic scoliosis', 'esophageal cancer', 'hepatocellular carcinoma'], ['childhood onset asthma', 'protein measurement', 'infection', 'infect', "Alzheimer's disease"], ['Autosomal dominant optic atrophy, classic type', 'Autosomal dominant optic atrophy plus syndrome', 'Behr syndrome', '3-methylglutaconic aciduria type 3', 'mitochondrial DNA depletion syndrome 14 (cardioencephalomyopathic type)'], ['Blue cone monochromatism', 'red color blindness', 'Progressive cone dystrophy', 'red-green color blindness', 'Cone rod dystrophy'], ['red-green color blindness', 'Blue cone monochromatism', 'Progressive cone dystrophy', 'Cone rod dystrophy', 'Bradyopsia'], ['Tritanopia', 'breast adenocarcinoma', 'atrial fibrillation', 'cutaneous melanoma', 'mathematical ability'], ['alcohol dependence', 'opioid dependence', 'pain', 'migraine disorder', 'Chronic pain'], ['mean corpuscular volume', 'lymphocyte count', 'insulin like growth factor measurement', 'glioblastoma multiforme', 'monocyte count'], ['open-angle glaucoma', 'familial amyotrophic lateral sclerosis', 'amyotrophic lateral sclerosis', 'motor neuron disease', 'osteitis deformans'], ['neuroblastoma', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'multiple myeloma', 'acute myeloid leukemia'], ['medulloblastoma', 'ovarian cancer', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['rheumatoid arthritis', 'refractory anemia', 'Neutropenia - monocytopenia - deafness', 'blood protein measurement', "Crohn's disease"], ['neutrophil count', 'myeloid white cell count', 'Niemann-Pick disease type C', 'cancer', 'neoplasm'], ['reticulocyte measurement', 'reticulocyte count', 'lung adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer'], ['Familial primary localized cutaneous amyloidosis', 'cutaneous nodular amyloidosis', 'ulcerative colitis', 'hepatocellular carcinoma', 'overall survival'], ['cup-to-disc ratio measurement', 'appendicitis', 'ventral hernia', 'sitting height measurement', 'body height'], ['Ornithine transcarbamylase deficiency', 'Hyperammonemia', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'genetic disorder'], ['white matter hyperintensity measurement', 'systolic blood pressure', 'mean corpuscular volume', 'smoking status measurement', 'diastolic blood pressure'], ['blood protein measurement', 'medulloblastoma', 'Neurodevelopmental disorder', 'autism', 'post-traumatic stress disorder'], ['postpartum hemorrhage', 'pregnancy', 'hemorrhage', 'schizophrenia', 'premature birth'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'Non-syndromic genetic deafness', 'mean corpuscular hemoglobin concentration', 'cutaneous melanoma'], ['Cough', 'triglyceride measurement', 'cutaneous melanoma', 'idiopathic pulmonary fibrosis', 'mean corpuscular hemoglobin concentration'], ['body mass index', 'type II diabetes mellitus', 'cutaneous melanoma', 'schizophrenia', 'fat body mass'], ['schizophrenia', 'serum alanine aminotransferase measurement', 'lung adenocarcinoma', 'lean body mass', 'late-onset Alzheimers disease'], ['dry eye syndrome', 'type II diabetes mellitus', 'oophoritis', 'salpingitis', 'mathematical ability'], ['dry eye syndrome', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'Pyknoachondrogenesis'], ['P2Y12 defect', 'myocardial infarction', 'coronary artery disease', 'acute coronary syndrome', 'ST Elevation Myocardial Infarction'], ['Cole-Carpenter syndrome', 'Osteogenesis imperfecta', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['multiple myeloma', 'thyroid gland adenocarcinoma', 'squamous cell lung carcinoma', 'endometrial cancer', 'self reported educational attainment'], ['vitamin D measurement', 'breast adenocarcinoma', 'infect', 'infection', 'glioma'], ['rheumatoid arthritis', 'interstitial lung disease', 'basophil percentage of leukocytes', 'basophil count', 'neutrophil count'], ['monocyte count', 'monocyte percentage of leukocytes', 'diet measurement', 'breast adenocarcinoma', 'protein measurement'], ['Cryopyrin-associated periodic syndrome', 'prostate cancer', 'neoplasm', 'testicular carcinoma', 'ovarian neoplasm'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'infection', 'infect', 'obstructive sleep apnea'], ['Phenylketonuria', 'Maternal hyperphenylalaninemia', 'Hyperphenylalaninemia', 'Mild hyperphenylalaninemia', 'genetic disorder'], ['intellectual developmental disorder with macrocephaly, seizures, and speech delay', 'body fat percentage', 'fat body mass', 'ulcerative colitis', 'endometrial cancer'], ['electrocardiography', 'Non-syndromic male infertility due to sperm motility disorder', 'Partial chromosome Y deletion', 'Normosmic congenital hypogonadotropic hypogonadism', 'Isochromosomy Yp'], ['BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'hematocrit', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration'], ['spina bifida', 'Neural tube closure defect', 'Isolated spina bifida', 'Neural tube defect', 'refractive error measurement'], ['glomerular filtration rate', 'cancer', 'lung cancer', 'ovarian cancer', 'prostate cancer'], ['Young adult-onset Parkinsonism', "Parkinson's disease", 'ulcerative colitis', "Crohn's disease", 'body weight'], ['ovarian cancer', 'neoplasm', 'ovarian carcinoma', 'breast neoplasm', 'peritoneal neoplasm'], ['ovarian cancer', 'neoplasm', 'ovarian carcinoma', 'breast neoplasm', 'peritoneal neoplasm'], ['neoplasm', 'ovarian cancer', 'ovarian carcinoma', 'breast cancer', 'non-small cell lung carcinoma'], ['balding measurement', 'breast adenocarcinoma', 'acute myeloid leukemia', 'melanoma', 'body fat percentage'], ['neuroimaging measurement', 'myeloid white cell count', 'cortical surface area measurement', 'leukocyte count', 'brain measurement'], ['Peters anomaly', 'Isolated aniridia', 'Aniridia', 'Foveal hypoplasia - presenile cataract', 'Autosomal dominant keratitis'], ['body height', 'glioblastoma multiforme', 'neoplasm', 'hepatocellular carcinoma', 'glioma'], ['clear cell renal carcinoma', 'cutaneous melanoma', 'renal cell carcinoma', 'renal carcinoma', 'papillary renal cell carcinoma'], ['congenital anomalies of kidney and urinary tract syndrome with or without hearing loss, abnormal ears, or developmental delay', 'acute lymphoblastic leukemia', 'genetic disorder', 'B-cell acute lymphoblastic leukemia', 'myoepithelial tumor'], ['pyruvate carboxylase deficiency disease', 'Pyruvate carboxylase deficiency', 'Congenital lactic acidosis', 'Pyruvate carboxylase deficiency, benign type', 'Pyruvate carboxylase deficiency, severe neonatal type'], ['Hyperphenylalaninemia', 'Dehydratase deficiency', 'type II diabetes mellitus', 'smoking status measurement', 'red blood cell distribution width'], ['lung adenocarcinoma', 'endometrial cancer', 'low grade glioma', 'gastric adenocarcinoma', 'breast adenocarcinoma'], ['Propionic acidemia', 'self reported educational attainment', 'smoking behavior', 'Abnormality of refraction', 'hair shape measurement'], ['Propionic acidemia', 'dilated cardiomyopathy', 'fibrinogen measurement', 'triglyceride measurement', 'head and neck squamous cell carcinoma'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'hepatocellular carcinoma', 'non-small cell lung carcinoma'], ['Ataxia-telangiectasia-like disorder', 'lung adenocarcinoma', 'gastric adenocarcinoma', 'body height', 'clear cell renal carcinoma'], ['Obesity due to prohormone convertase I deficiency', 'lean body mass', 'body mass index', 'fat body mass', 'insulin measurement'], ['body height', 'refractive error measurement', 'low density lipoprotein cholesterol measurement', 'intraocular pressure measurement', 'vital capacity'], ['appendicular lean mass', 'body height', 'soluble transferrin receptor measurement', 'iron biomarker measurement', 'triglyceride measurement'], ['familial hypercholesterolemia', 'hypercholesterolemia, autosomal dominant, 3', 'coronary artery disease', 'Homozygous familial hypercholesterolemia', 'Hypercholesterolemia'], ['Spondylometaphyseal dysplasia - cone-rod dystrophy', 'Leber congenital amaurosis', 'genetic disorder', 'mean platelet volume', 'glioblastoma multiforme'], ['glomerular filtration rate', 'rheumatoid arthritis', 'metabolic syndrome', 'neoplasm', 'pseudotumor cerebri'], ['non-small cell lung carcinoma', 'renal cell carcinoma', 'neoplasm', 'metastatic melanoma', 'head and neck squamous cell carcinoma'], ['cutaneous melanoma', 'bile duct carcinoma', 'Hepatobiliary Neoplasm', 'breast ductal adenocarcinoma', 'kidney neoplasm'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'deafness, autosomal dominant 74', 'chronic kidney disease', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['Brachydactyly - arterial hypertension', 'stroke', 'coronary artery disease', 'type II diabetes mellitus', 'cardiovascular disease'], ['asthma', 'essential thrombocythemia', 'heart failure', 'Airway obstruction', 'coronary artery disease'], ['psoriasis', 'asthma', 'chronic obstructive pulmonary disease', 'Airway obstruction', 'atopic eczema'], ['Headache', 'asthma', 'chronic obstructive pulmonary disease', 'psoriasis', 'Airway obstruction'], ['Acrodysostosis', 'Acrodysostosis with multiple hormone resistance', 'asthma', 'chronic obstructive pulmonary disease', 'psoriasis'], ['erectile dysfunction', 'pulmonary arterial hypertension', 'benign prostatic hyperplasia', 'Impotence', 'coronary artery disease'], ['Retinitis pigmentosa', 'Congenital stationary night blindness', 'stroke', 'coronary artery disease', 'Retinal dystrophy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['dyskinesia, limb and orofacial, infantile-onset', 'Autosomal dominant striatal neurodegeneration', 'stroke', 'coronary artery disease', 'bipolar disorder'], ['Congenital disorder of glycosylation', 'triglyceride measurement', 'handedness', 'telomere length', 'body height'], ['hair shape measurement', 'puberty onset measurement', 'cancer', 'Alopecia', 'age at first facial hair'], ['platelet count', 'hematocrit', 'reticulocyte measurement', 'systolic blood pressure', 'sex hormone-binding globulin measurement'], ['adrenal suppression measurement', 'response to corticosteroid', 'acute myeloid leukemia', 'visual cortical surface area measurement', 'brain volume measurement'], ['Gastrointestinal stromal tumor', 'cancer', 'GIST-plus syndrome', 'acute myeloid leukemia', 'neoplasm'], ['Infantile myofibromatosis', 'Bilateral striopallidodentate calcinosis', 'Acroosteolysis-keloid-like lesions-premature aging syndrome', 'skeletal overgrowth-craniofacial dysmorphism-hyperelastic skin-white matter lesions syndrome', 'hepatocellular carcinoma'], ['Pyruvate dehydrogenase E3-binding protein deficiency', 'Pyruvate dehydrogenase deficiency', 'Leigh syndrome', 'Pyruvate dehydrogenase E1-alpha deficiency', 'protein measurement'], ['mean corpuscular volume', 'body fat percentage', 'mean corpuscular hemoglobin concentration', 'Bicuspid aortic valve', 'body height'], ["Crohn's disease", '[Pyruvate dehydrogenase acetyl-transferring] kinase isozyme 1, mitochondrial measurement', 'lactic acidosis', 'Pyruvate dehydrogenase deficiency', 'eosinophil count'], ['dilated cardiomyopathy', 'hypertrophic cardiomyopathy', 'Abnormality of the cardiovascular system', 'type II diabetes mellitus', 'Rare familial disorder with hypertrophic cardiomyopathy'], ['breast adenocarcinoma', 'cognitive function measurement', 'forced expiratory volume', 'self reported educational attainment', 'Apolipoprotein A-I deficiency'], ['Pyruvate dehydrogenase phosphatase deficiency', 'Pyruvate dehydrogenase deficiency', 'neoplasm', 'cancer', 'infection'], ['ovarian carcinoma', 'cancer', 'colorectal adenocarcinoma', 'breast carcinoma', 'ovarian cancer'], ['Deafness - encephaloneuropathy - obesity - valvulopathy', 'Coenzyme Q10 deficiency', 'hepatocellular adenoma', 'adolescent idiopathic scoliosis', 'Osteosclerosis - ichthyosis - premature ovarian failure'], ['Partial pancreatic agenesis', 'MODY', 'Permanent neonatal diabetes mellitus', 'maturity-onset diabetes of the young type 4', 'fasting blood glucose measurement'], ['mean corpuscular volume', 'neoplasm', 'multiple sclerosis', 'cancer', 'glioblastoma multiforme'], ['Spinocerebellar ataxia type 23', 'aortic root size', 'mean platelet volume', 'platelet crit', 'glioblastoma multiforme'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'red blood cell distribution width', 'cutaneous melanoma', 'Red cell distribution width'], ['coronary artery disease', 'systolic blood pressure', 'pulse pressure measurement', 'mean platelet volume', 'neoplasm'], ['schizophrenia', 'breast carcinoma', 'hepatocellular carcinoma', 'insomnia measurement', 'diffuse large B-cell lymphoma'], ['Prolidase deficiency', 'body fat percentage', 'body mass index', 'fat body mass', 'lean body mass'], ['Familial advanced sleep-phase syndrome', 'chronotype measurement', 'Sleep Disorder', 'neoplasm', 'glioma'], ['Neonatal adrenoleukodystrophy', 'Zellweger syndrome', 'Peroxisome biogenesis disorder-Zellweger syndrome spectrum', 'Infantile Refsum disease', 'ulcerative colitis'], ['serum metabolite measurement', 'very low density lipoprotein cholesterol measurement', 'breast adenocarcinoma', 'coronary artery calcification', 'squamous cell lung carcinoma'], ['endometrial cancer', 'reaction time measurement', 'mathematical ability', 'worry measurement', 'intelligence'], ['glioblastoma multiforme', 'gastric adenocarcinoma', 'colorectal cancer', 'atherosclerosis', 'atopic eczema'], ['mean corpuscular volume', 'red blood cell distribution width', 'BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'granulocyte count'], ['gastric carcinoma', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'colorectal carcinoma', 'oral cavity carcinoma'], ['age-related macular degeneration', 'diabetic macular edema', 'wet macular degeneration', 'retinal vein occlusion', 'macular retinal edema'], ['blood protein measurement', 'ovarian cancer', 'lung adenocarcinoma', 'endometrial cancer', 'squamous cell lung carcinoma'], ['ovarian cancer', 'lung adenocarcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'non-small cell lung carcinoma'], ['lean body mass', 'fat body mass', 'body weight', 'body height', 'appendicular lean mass'], ['fat body mass', 'body mass index', 'body fat percentage', 'smoking status measurement', 'chronotype measurement'], ['polycystic ovary syndrome', 'infertility', 'endometriosis', 'uterine fibroid', 'acne'], ['primary ovarian insufficiency', 'neoplasm', 'cancer', 'renal cell carcinoma', 'renal cell adenocarcinoma'], ['endometrial cancer', 'hematocrit', 'susceptibility to breast cancer', 'cutaneous melanoma', 'systolic blood pressure'], ['lean body mass', 'fat body mass', 'appendicular lean mass', 'body weight', 'ovarian cancer'], ['X-linked hypophosphatemia', 'Hypophosphatemic rickets', 'Lower limb pain', 'Bowing of the legs', 'Dent disease'], ['hepatocellular carcinoma', 'self reported educational attainment', 'refractive error measurement', 'chronotype measurement', 'mathematical ability'], ['3-phosphoglycerate dehydrogenase deficiency, infantile/juvenile form', 'Neu-Laxova syndrome', 'Neurometabolic disorder due to serine deficiency', 'blood metabolite measurement', 'glycine measurement'], ['appendicular lean mass', 'type II diabetes mellitus', 'low density lipoprotein cholesterol measurement', 'osteosarcoma', 'bilirubin measurement'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'esophageal cancer', 'squamous cell lung carcinoma'], ['endometrial cancer', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'pancreatic adenocarcinoma'], ['Dehydrated hereditary stomatocytosis', 'lymphedema, hereditary, iii', 'genetic disorder', 'hydrops fetalis (disease)', 'Thickened nuchal skin fold'], ['Multiple congenital anomalies-hypotonia-seizures syndrome type 2', 'Paroxysmal nocturnal hemoglobinuria', 'Ferro-cerebro-cutaneous syndrome', 'West syndrome', 'Malignant migrating partial seizures of infancy'], ['short stature with microcephaly and distinctive facies', 'brain volume measurement', 'brain measurement', 'neuroimaging measurement', 'gait measurement'], ['blood protein measurement', 'neoplasm', 'Cystic fibrosis', 'lymphocyte count', 'pancreatic carcinoma'], ['glycosylphosphatidylinositol biosynthesis defect 18', 'insomnia measurement', 'glioma', 'platelet crit', 'urate measurement'], ['Short stature due to GHSR deficiency', 'platelet count', 'glioblastoma multiforme', 'breast adenocarcinoma', 'body height'], ['severe acute respiratory syndrome', 'COVID-19', 'body mass index', 'body fat percentage', 'breast adenocarcinoma'], ['Megalencephaly-capillary malformation-polymicrogyria syndrome', 'CLOVE syndrome', 'colorectal adenocarcinoma', 'seborrheic keratosis', 'CLAPO syndrome'], ['cancer', 'prostate adenocarcinoma', 'esophageal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm'], ['Activated PIK3-delta syndrome', 'chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'hypertension', 'systolic blood pressure'], ['SHORT syndrome', 'Autosomal agammaglobulinemia', 'Activated PIK3-delta syndrome', 'cancer', 'Agammaglobulinemia'], ['Spinocerebellar ataxia with axonal neuropathy type 2', 'breast cancer', 'monocyte percentage of leukocytes', 'autoimmune thyroid disease', 'hypothyroidism'], ['Fleck corneal dystrophy', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'lung adenocarcinoma'], ["Alzheimer's disease", 'family history of Alzheimer’s disease', 'red blood cell distribution width', 'heel bone mineral density', 'electrocardiography'], ['heel bone mineral density', 'electrocardiography', 'platelet crit', 'platelet count', 'monocyte count'], ['Central Nervous System Lymphoma', 'diffuse large B-cell lymphoma', 'lymphoma', 'lymphoid neoplasm', 'multiple myeloma'], ['type II diabetes mellitus', 'birth weight', 'peak expiratory flow', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement'], ['thyroxine measurement', 'hormone measurement', 'stressful life event measurement', 'C-reactive protein measurement', 'Cone rod dystrophy'], ['neoplasm', "Alzheimer's disease", 'Peeling skin syndrome', 'systemic scleroderma', 'sebaceous of Jadassohn nevus'], ['Lethal congenital contracture syndrome type 3', 'glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma'], ['Liberfarb syndrome', 'hearing loss', 'glioblastoma multiforme', 'osteochondrodysplasia', 'microcephaly (disease)'], ['breast adenocarcinoma', 'retinal degeneration', 'Autosomal recessive hypophosphatemic rickets', 'Abnormality of refraction', 'Cholesterol-ester transfer protein deficiency'], ['Axenfeld-Rieger syndrome', 'ring dermoid of cornea', 'Rieger anomaly', 'Familial ocular anterior segment mesenchymal dysgenesis', 'Peters anomaly'], ['Familial ocular anterior segment mesenchymal dysgenesis', 'Posterior polar cataract', 'Non-syndromic congenital cataract', 'cataract', 'Cataract-glaucoma'], ['Autosomal dominant polycystic kidney disease', 'polycystic kidney disease', 'Autosomal recessive polycystic kidney disease', 'kidney disease', 'Meckel syndrome'], ['Autosomal dominant polycystic kidney disease', 'polycystic kidney disease 2', 'polycystic kidney disease', 'kidney disease', 'Meckel syndrome'], ['mean platelet volume', 'blood metabolite measurement', 'heel bone mineral density', 'red blood cell distribution width', 'serum metabolite measurement'], ['Autosomal recessive polycystic kidney disease', 'polycystic kidney disease', 'Meckel syndrome', 'lean body mass', 'Renal cyst'], ['gastric carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'erythrocyte count', 'forced expiratory volume'], ['urolithiasis', 'endometrial cancer', 'prostate cancer', 'Blackfan-Diamond anemia', 'Activated PIK3-delta syndrome'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'infection', 'infect', 'esophageal adenocarcinoma'], ['rheumatoid arthritis', 'cutaneous melanoma', 'systolic blood pressure', 'diastolic blood pressure', 'appendicular lean mass'], ['colorectal cancer', 'colonic neoplasm', 'protein measurement', 'phospholipase A2, membrane associated measurement', 'blood protein measurement'], ['Infantile neuroaxonal dystrophy', 'Adult-onset dystonia-parkinsonism', 'PLA2G6-associated neurodegeneration', 'Iron accumulation in brain', "Parkinson's disease"], ['acute coronary syndrome', 'Platelet-activating factor acetylhydrolase deficiency', 'asthma', 'atopic eczema', 'inherited susceptibility to asthma'], ['age-related macular degeneration', 'complement factor I measurement', 'obesity', 'protein measurement', 'low density lipoprotein cholesterol measurement'], ['chronic kidney disease', 'hypertension', 'glomerular filtration rate', "Alzheimer's disease", 'chronic obstructive pulmonary disease'], ['visual perception measurement', 'neoplasm', 'monocyte percentage of leukocytes', 'monocyte count', 'non-small cell lung carcinoma'], ['Silver-Russell syndrome', 'silver-russell syndrome 4', 'body height', 'vital capacity', 'lean body mass'], ['hemorrhage', 'osteoarthritis', 'scoliosis', 'skin pigmentation', 'hair color'], ['Quebec platelet disorder', 'blood protein measurement', "Crohn's disease", 'inflammatory bowel disease', 'psoriasis'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'eosinophil percentage of granulocytes', 'neutrophil percentage of granulocytes', 'sum of eosinophil and basophil counts'], ['Familial multiple lipomatosis', 'body mass index', 'lean body mass', 'cerebrospinal fluid biomarker measurement', 'smoking behavior'], ['Autosomal dominant cerebellar ataxia type 1', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', "Alzheimer's disease"], ['Epidermolysis bullosa simplex with muscular dystrophy', 'Epidermolysis bullosa simplex with pyloric atresia', 'Epidermolysis bullosa simplex, Ogna type', 'Autosomal recessive limb-girdle muscular dystrophy type 2Q', 'Epidermolysis bullosa simplex'], ['cancer', 'breast cancer', 'protein measurement', 'diabetes mellitus', 'body mass index'], ['Hypoplasminogenemia', 'Ligneous conjunctivitis', 'myocardial infarction', 'coronary artery disease', 'stroke'], ['hepatocellular carcinoma', 'insulin resistance', 'optic cup area measurement', 'Hyperlipoproteinemia type 4', 'coronary artery disease, autosomal dominant 2'], ['acute myeloid leukemia', 'mean corpuscular volume', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['hypertrophic cardiomyopathy', 'Familial isolated dilated cardiomyopathy', 'Familial dilated cardiomyopathy', 'dilated cardiomyopathy', 'cardiomyopathy'], ['Spastic paraplegia type 2', 'Pelizaeus-Merzbacher disease', 'Pelizaeus-Merzbacher disease, connatal form', 'Hereditary spastic paraplegia', 'genetic disorder'], ['platelet count', 'nervous system disease', 'coronary artery disease', 'mean corpuscular volume', 'mean corpuscular hemoglobin'], ['serum metabolite measurement', 'high density lipoprotein cholesterol measurement', 'lipid measurement', 'triglyceride measurement', 'alcohol drinking'], ['Congenital chronic diarrhea with protein-losing enteropathy', 'neoplasm', 'hepatocellular carcinoma', 'low density lipoprotein cholesterol measurement', "Hb Bart's hydrops fetalis"], ['platelet count', 'Neurodevelopmental disorder', 'blood protein measurement', 'skin aging measurement', 'skin aging'], ['adolescent idiopathic scoliosis', 'risk-taking behaviour', 'brain volume measurement', 'insulin like growth factor measurement', 'protein measurement'], ['Short stature due to GHSR deficiency', 'Neurodevelopmental disorder', 'autism', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['thyroid gland adenocarcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'Mental deterioration', 'neuroblastoma'], ['head and neck squamous cell carcinoma', 'sitting height measurement', 'body height', 'appendicular lean mass', 'forced expiratory volume'], ['lean body mass', 'blood protein measurement', 'neoplasm', 'Griscelli disease type 3', 'lung cancer'], ['viral disease', 'acute myeloid leukemia', 'basal cell carcinoma', 'leukocyte count', 'waist-hip ratio'], ['Charcot-Marie-Tooth disease type 1G', 'peripheral neuropathy', 'genetic disorder', 'Autosomal dominant slowed nerve conduction velocity', 'Hereditary neuropathy with liability to pressure palsies'], ['Dejerine-Sottas syndrome', 'Charcot-Marie-Tooth disease type 1A', 'Hereditary neuropathy with liability to pressure palsies', 'Charcot-Marie-Tooth disease type 1E', 'Roussy-Lévy syndrome'], ['hepatocellular carcinoma', 'colorectal adenocarcinoma', 'esophageal squamous cell carcinoma', 'diffuse large B-cell lymphoma', 'prostate carcinoma'], ['Paroxysmal non-kinesigenic dyskinesia', 'Paroxysmal dystonia', 'Episodic hemiplegia', 'Generalized epilepsy - paroxysmal dyskinesia', 'lean body mass'], ['Early infantile epileptic encephalopathy', 'Ataxia-oculomotor apraxia type 4', 'Microcephaly - seizures - developmental delay', 'Charcot-Marie-Tooth disease type 2B2', 'genetic disorder'], ['obesity', 'Pancreatic triacylglycerol lipase deficiency', 'type II diabetes mellitus', 'Hepatic steatosis', 'Pancreatic colipase deficiency'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'chronic pancreatitis'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'esophageal squamous cell carcinoma'], ['neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'body height', 'cardiovascular disease'], ['breast carcinoma', 'colorectal carcinoma', 'self reported educational attainment', 'ovarian cancer', 'blood pressure'], ['daytime rest measurement', 'body mass index', 'optic cup area measurement', 'type II diabetes mellitus', 'thyroid gland adenocarcinoma'], ['Purine nucleoside phosphorylase deficiency', 'Combined T and B cell immunodeficiency', 'Anemia, Hemolytic, Autoimmune', 'age at menopause', 'protein measurement'], ['non-alcoholic fatty liver disease', 'cirrhosis of liver', 'serum alanine aminotransferase measurement', 'platelet count', 'type II diabetes mellitus'], ['Young adult-onset Parkinsonism', 'familial nephrotic syndrome', 'Atypical juvenile parkinsonism', 'systolic blood pressure', 'pulse pressure measurement'], ['Mandibular hypoplasia-deafness-progeroid syndrome', 'colorectal cancer', 'acute myeloid leukemia', 'non-small cell lung carcinoma', 'chronic lymphocytic leukemia'], ['acute myeloid leukemia', 'acute lymphoblastic leukemia', 'non-small cell lung carcinoma', 'pancreatic carcinoma', 'chronic lymphocytic leukemia'], ['neoplasm', 'infection', 'infect', 'breast cancer', 'non-small cell lung carcinoma'], ['endometrial cancer', 'venous thromboembolism', 'infect', 'infection', 'Hoyeraal-Hreidarsson syndrome'], ['Alpers syndrome', 'Autosomal recessive progressive external ophthalmoplegia', 'Autosomal dominant progressive external ophthalmoplegia', 'Sensory ataxic neuropathy - dysarthria - ophthalmoparesis', 'Spinocerebellar ataxia with epilepsy'], ['Hypomyelination - hypogonadotropic hypogonadism - hypodontia', 'Hypomyelinating leukodystrophy with or without oligondontia and/or hypogonadism', 'Autosomal recessive non-syndromic intellectual disability', 'Endosteal sclerosis - cerebellar hypoplasia', 'Isolated congenital hypogonadotropic hypogonadism'], ['endometrial cancer', 'Primary hypomagnesemia with secondary hypocalcemia', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Pseudohypoparathyroidism type 2', 'Autosomal dominant hypocalcemia'], ['body height', 'colorectal carcinoma', 'bacteriemia', 'neoplasm', 'colorectal cancer'], ['Autosomal recessive hypophosphatemic rickets', 'Melorheostosis', 'postmenopausal osteoporosis', 'Pyknoachondrogenesis', 'Gnathodiaphyseal dysplasia'], ['46,XX gonadal dysgenesis', 'medulloblastoma', 'alcohol consumption measurement', 'asthma', 'diet measurement'], ['erythrocyte count', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'Leukodystrophy'], ['Obesity due to pro-opiomelanocortin deficiency', 'obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'Genetic obesity', 'adrenal gland disease'], ['amyotrophic lateral sclerosis', 'aspartate aminotransferase, cytoplasmic measurement', 'arylesterase enzyme measurement', 'leucine carboxyl methyltransferase 1 measurement', 'metabolite measurement'], ['amyotrophic lateral sclerosis', 'acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'systemic lupus erythematosus'], ['amyotrophic lateral sclerosis', 'high density lipoprotein cholesterol measurement', 'head and neck squamous cell carcinoma', 'hepatocellular carcinoma', 'neoplasm'], ['erythrocyte count', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'hemoglobin measurement'], ['Congenital adrenal hyperplasia due to cytochrome P450 oxidoreductase deficiency', 'Congenital adrenal hyperplasia', 'Antley-Bixler syndrome', 'coffee consumption measurement', 'platelet count'], ['Focal dermal hypoplasia', 'cancer', 'polydactyly (disease)', 'Global developmental delay', 'endometrial cancer'], ['blood protein measurement', 'smoking status measurement', 'cerebrospinal fluid biomarker measurement', 'Varicose veins', 'varicose disease'], ['eosinophil count', 'lung carcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'cutaneous melanoma', 'lymphoid neoplasm'], ['non-Hodgkins lymphoma', 'eosinophil count', 'chronotype measurement', 'forced expiratory volume', 'sitting height measurement'], ['gastric carcinoma', 'myoepithelial tumor', 'lung carcinoma', 'cutaneous melanoma', 'basal cell carcinoma'], ['inorganic pyrophosphatase measurement', 'erythrocyte count', 'neoplasm', 'non-small cell lung carcinoma', 'melanoma'], ['narcolepsy with cataplexy', 'heel bone mineral density', 'lean body mass', 'fat body mass', 'non-small cell lung carcinoma'], ['coronary artery disease', 'cardiovascular disease', 'Disorder of lipid metabolism', 'Hypertriglyceridemia', 'type II diabetes mellitus'], ['type II diabetes mellitus', 'Familial partial lipodystrophy associated with PPARG mutations', 'diabetes mellitus', 'ulcerative colitis', 'colorectal adenocarcinoma'], ['electrocardiography', 'hematocrit', 'hemoglobin measurement', 'sex hormone-binding globulin measurement', 'erythrocyte count'], ['systemic lupus erythematosus', 'sunburn', 'cutaneous melanoma', 'facial pigmentation measurement', 'nevus count'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neuroblastoma'], ['blood metabolite measurement', 'acylcarnitine measurement', 'metabolite measurement', 'serum metabolite measurement', 'peptidyl-prolyl cis-trans isomerase D measurement'], ['lean body mass', 'reticulocyte count', 'body height', 'reticulocyte measurement', 'appendicular lean mass'], ['Neurodevelopmental disorder', 'neuroblastoma', 'ovarian carcinoma', 'Ewing sarcoma', 'osteosarcoma'], ['hepatocellular carcinoma', 'red blood cell distribution width', 'neoplasm', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia'], ['lymphocyte count', 'response to antidepressant', 'refractive error measurement', 'brain stem volume measurement', 'peak expiratory flow'], ['appendicular lean mass', 'body height', 'lean body mass', 'fat body mass', 'forced expiratory volume'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness', 'type II diabetes mellitus', 'body height', 'balding measurement'], ['Intermediate maple syrup urine disease', 'Maple syrup urine disease', 'blood metabolite measurement', 'amino acid measurement', 'serum metabolite measurement'], ['Pentosuria', 'Isolated polycystic liver disease', 'Myopathy due to calsequestrin and SERCA1 protein overload', 'Metabolic myopathy due to lactate transporter defect', 'appendicular lean mass'], ['Porphyria variegata', 'Abnormal blistering of the skin', 'migraine disorder', 'Abnormal urinary color', 'Constipation'], ['rosette-forming glioneuronal tumor of fourth ventricule', 'Ewing sarcoma', 'Eisenmenger syndrome', 'epithelioid sarcoma', 'hepatocellular carcinoma'], ['erythrocyte count', 'hematocrit', 'self reported educational attainment', 'neoplasm', 'schizophrenia'], ["Alzheimer's disease", 'obesity', 'infect', 'infection', 'neoplasm'], ['type II diabetes mellitus', 'body mass index', 'schizophrenia', 'grip strength measurement', 'gait measurement'], ['body mass index', 'insomnia measurement', 'chronic obstructive pulmonary disease', 'cancer', 'influenza A (H1N1)'], ['ovarian cancer', 'breast adenocarcinoma', 'sex ratio', 'acne', 'platelet count'], ['glioblastoma multiforme', 'breast cancer', 'cancer', 'hepatocellular carcinoma', 'neoplasm'], ['schizophrenia', 'attention deficit hyperactivity disorder', 'alcohol use disorder measurement', 'gastric adenocarcinoma', 'low grade glioma'], ['neurodevelopmental disorder and language delay with or without structural brain abnormalities', 'cancer', 'Global developmental delay', 'Feeding difficulties', 'language impairment'], ['lymphocyte count', 'hemoglobin measurement', 'eosinophil count', 'eosinophil percentage of leukocytes', "Crohn's disease"], ['CLN1 disease', 'Infantile neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Adult neuronal ceroid lipofuscinosis', 'Late infantile neuronal ceroid lipofuscinosis'], ['head and neck squamous cell carcinoma', 'blood protein measurement', 'triglyceride measurement', 'cardiovascular disease', 'body height'], ['age at onset', 'Myopia', 'neoplasm', 'acute myeloid leukemia', 'Uveal melanoma'], ['neoplasm', 'arthritis', 'gastric cancer', 'cancer', 'rheumatoid arthritis'], ['systemic scleroderma', 'psoriasis', 'myositis', 'polymyositis', 'Pyomyositis'], ['renal carcinoma', 'breast carcinoma', 'hepatocellular carcinoma', 'Renal Cell Carcinoma Associated with Xp11.2 Translocations/TFE3 Gene Fusions', 'papillary renal cell carcinoma'], ['blood protein measurement', 'adolescent idiopathic scoliosis', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cutaneous melanoma'], ['multiple myeloma', 'diffuse large B-cell lymphoma', 'ulcerative colitis', 'colorectal adenocarcinoma', 'melanoma'], ['HIV-1 infection', 'viral load', 'protein measurement', 'breast carcinoma', 'alcohol drinking'], ["Alzheimer's disease", 'lung adenocarcinoma', 'endometrial cancer', 'urinary bladder cancer', 'scrapie'], ['breast adenocarcinoma', 'urinary bladder cancer', 'multiple sclerosis', 'type II diabetes mellitus', 'body fat percentage'], ['electrocardiography', 'causalgia', 'T wave morphology measurement', 'diet measurement', 'blood protein measurement'], ['Familial hemophagocytic lymphohistiocytosis', 'adult lymphoma', 'Aplastic anemia', 'colorectal adenocarcinoma', 'cutaneous melanoma'], ['pappalysin‐1 measurement', 'eosinophil count', 'mean corpuscular hemoglobin concentration', 'sum of eosinophil and basophil counts', 'mean corpuscular volume'], ['Camptodactyly-arthropathy-coxa-vara-pericarditis syndrome', 'camptodactyly-arthropathy-coxa vara-pericarditis syndrome', 'genetic disorder', 'FEV/FEC ratio', 'Arthrogryposis syndrome'], ['sensory perception of bitter taste', 'neoplasm', 'coffee consumption', 'coffee consumption measurement', 'glioblastoma multiforme'], ['neoplasm', 'breast cancer', 'endometrial cancer', 'cancer', 'leiomyoma'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['sex hormone-binding globulin measurement', 'protein measurement', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['neutrophil count', 'basophil count', 'lung adenocarcinoma', 'breast adenocarcinoma', 'leukocyte count'], ['Acrodysostosis', 'Carney complex', 'Acrodysostosis with multiple hormone resistance', 'Primary pigmented nodular adrenocortical disease', 'Carney complex, type 1'], ['acute myeloid leukemia', 'neoplasm', 'mast-cell leukemia', 'ulcerative colitis', 'inflammatory bowel disease'], ['Spinocerebellar ataxia type 14', 'acute myeloid leukemia', 'Spinocerebellar ataxia type 15/16', 'neoplasm', 'mast-cell leukemia'], ['congenital heart defects and ectodermal dysplasia', 'acute myeloid leukemia', 'self reported educational attainment', 'body mass index', 'neoplasm'], ['multiple myeloma', 'heel bone mineral density', 'type I diabetes mellitus', 'aspartate aminotransferase measurement', 'sclerosing cholangitis'], ['Severe combined immunodeficiency due to DNA-PKcs deficiency', 'Combined T and B cell immunodeficiency', 'mean corpuscular volume', 'Cernunnos-XLF deficiency', 'mean corpuscular hemoglobin concentration'], ['Familial thoracic aortic aneurysm and aortic dissection', 'body height', 'lean body mass', 'Abnormality of the cardiovascular system', 'Rare genetic vascular disease'], ['Young adult-onset Parkinsonism', "Parkinson's disease", 'late-onset Parkinson disease', 'dystonic disorder', 'body mass index'], ['AL amyloidosis', 'neutrophil count', 'leukocyte count', 'basophil count', 'myeloid white cell count'], ['Familial hyperprolactinemia', 'hyperprolactinemia', 'breast carcinoma', 'appendicular lean mass', 'testosterone measurement'], ['breast adenocarcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'ovarian cancer', 'body height'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'low grade glioma', 'urinary bladder cancer'], ['sex hormone-binding globulin measurement', 'red blood cell distribution width', 'lean body mass', 'body fat percentage', 'testosterone measurement'], ['platelet crit', 'Autosomal recessive non-syndromic intellectual disability', 'hepatocellular carcinoma', 'neoplasm', 'hemoglobin measurement'], ['Gerstmann-Straussler-Scheinker syndrome', 'Huntington disease-like 1', 'Fatal familial insomnia', 'Inherited Creutzfeldt-Jakob disease', 'prion disease'], ['Hereditary thrombophilia due to congenital protein C deficiency', 'protein c deficiency', 'genetic disorder', 'Abnormal thrombosis', 'deep vein thrombosis'], ['protein C measurement', 'factor VII measurement', 'venous thromboembolism', 'prothrombin time measurement', 'coronary artery disease'], ['Hyperprolinemia type 1', 'schizophrenia', 'amino acid measurement', 'proline measurement', 'hyperprolinemia'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'blood protein measurement', 'serum metabolite measurement', 'breast adenocarcinoma'], ['Hirschsprung disease', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'medulloblastoma'], ['Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'Septo-optic dysplasia', 'genetic disorder', 'Isolated congenital hypogonadotropic hypogonadism'], ['Retinitis pigmentosa', 'Cone rod dystrophy', 'Stargardt disease', 'Retinal macular dystrophy type 2', 'Macular dystrophy'], ['Hereditary thrombophilia due to congenital protein S deficiency', 'thrombophilia due to protein S deficiency, autosomal dominant', 'protein S deficiency', 'Reduced protein S activity', 'Abnormal bleeding'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Varicose veins', 'tonsillectomy risk measurement', 'lean body mass'], ['Retinitis pigmentosa', 'Butterfly-shaped pigment dystrophy', 'Adult-onset foveomacular vitelliform dystrophy', 'patterned macular dystrophy 1', 'Central areolar choroidal dystrophy'], ['Lethal ataxia with deafness and optic atrophy', 'Phosphoribosylpyrophosphate synthetase superactivity', 'X-linked Charcot-Marie-Tooth disease type 5', 'X-linked non-syndromic sensorineural deafness type DFN', 'Retinal dystrophy'], ['Infantile convulsions and choreoathetosis', 'Paroxysmal kinesigenic dyskinesia', 'Benign familial neonatal-infantile seizures', 'Autosomal recessive non-syndromic intellectual disability', 'Benign familial infantile epilepsy'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'neoplasm', 'ovarian cancer'], ['esophageal squamous cell carcinoma', 'Autosomal dominant severe congenital neutropenia', 'X-linked severe congenital neutropenia', 'Pentosuria', 'Blackfan-Diamond anemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts'], ['anxiety', 'white matter microstructure measurement', 'worry measurement', 'schizophrenia', 'systolic blood pressure'], ['blood protein measurement', 'Neutrophil immunodeficiency syndrome', 'Recurrent infection due to specific granule deficiency', 'neoplasm', 'gastroenteritis'], ['basophil count', 'age at onset', 'Myopia', 'basophil percentage of leukocytes', 'monocyte percentage of leukocytes'], ['Neu-Laxova syndrome', 'Phosphoserine aminotransferase deficiency', 'Neurometabolic disorder due to serine deficiency', 'adolescent idiopathic scoliosis', 'mean corpuscular volume'], ['gastric carcinoma', 'gastric cancer', 'duodenal ulcer', 'bladder carcinoma', 'urinary bladder cancer'], ['self reported educational attainment', 'drug use measurement', 'smoking status measurement', 'triglyceride measurement', 'anxiety'], ['Early-onset autosomal dominant Alzheimer disease', 'Frontotemporal dementia', 'Pick disease', 'dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy'], ['hidradenitis suppurativa', 'acne inversa, familial, 1', 'Dowling-Degos disease', "Alzheimer's disease", 'Dementia'], ['blood protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'visual perception measurement', 'head and neck squamous cell carcinoma'], ['breast adenocarcinoma', 'multiple myeloma', 'head and neck squamous cell carcinoma', 'low grade glioma', 'neoplasm'], ['HIV infection', 'bladder carcinoma', 'colorectal adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'breast ductal adenocarcinoma'], ['HIV infection', 'glioblastoma multiforme', 'diastolic blood pressure', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['HIV infection', 'glioblastoma multiforme', 'endometrial cancer', 'lung adenocarcinoma', 'gastric adenocarcinoma'], ['proteasome-associated autoinflammatory syndrome 3', 'HIV infection', 'CANDLE syndrome', 'JMP syndrome', 'Proteasome disability syndrome'], ['CANDLE syndrome', 'Proteasome disability syndrome', 'Nakajo-Nishimura syndrome', 'JMP syndrome', 'HIV infection'], ['proteasome-associated autoinflammatory syndrome 3', 'HIV infection', 'glioblastoma multiforme', 'urinary bladder cancer', 'gastric adenocarcinoma'], ['HIV infection', 'ovarian cancer', 'chronic lymphocytic leukemia', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['HIV infection', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'esophageal cancer'], ['HIV infection', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer'], ['HIV infection', 'ovarian cancer', 'endometrial cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['ulcerative colitis', 'inflammatory bowel disease', 'neutrophil count', 'hemoglobin measurement', 'ankylosing spondylitis'], ['synovial sarcoma', 'Wolfram syndrome', 'Classical progressive supranuclear palsy', 'Progressive supranuclear palsy', 'Sepsis'], ['ovarian carcinoma', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'low density lipoprotein cholesterol measurement', 'apolipoprotein B measurement'], ['diet measurement', 'gastric cancer', 'neoplasm', 'red blood cell density measurement', 'non-small cell lung carcinoma'], ['Gorlin syndrome', 'Holoprosencephaly', 'basal cell carcinoma', 'medulloblastoma', 'Inherited cancer-predisposing syndrome'], ['mediator of RNA polymerase II transcription subunit 1 measurement', 'smoking status measurement', 'Okt4 epitope deficiency', 'Alymphoid cystic thymic dysgenesis', 'Combined immunodeficiency with skin granulomas'], ['Lenz-Majewski hyperostotic dwarfism', 'type II diabetes mellitus', 'peripheral arterial disease', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['breast adenocarcinoma', 'Immature Reticulocyte Fraction Measurement', 'Normosmic congenital hypogonadotropic hypogonadism', 'Testicular regression syndrome', 'Hypergonadotropic hypogonadism - cataract syndrome'], ['Cowden syndrome', 'Lhermitte-Duclos disease', 'PTEN hamartoma tumor syndrome', 'Proteus-like syndrome', 'Macrocephaly-autism syndrome'], ['Permanent neonatal diabetes mellitus - pancreatic and cerebellar agenesis', 'Partial pancreatic agenesis', 'Permanent neonatal diabetes mellitus', 'pancreatic adenocarcinoma', 'head and neck squamous cell carcinoma'], ['asthma', 'seasonal allergic rhinitis', 'Nasal Cavity Polyp', 'prostate adenocarcinoma', 'allergic rhinitis'], ['blood protein measurement', 'central corneal thickness', 'intraocular pressure measurement', 'renal system measurement', 'Keratoconus'], ['erectile dysfunction', 'Premature rupture of membranes', 'myocardial infarction', 'pregnancy', 'pain'], ['erectile dysfunction', 'myocardial infarction', 'Premature rupture of membranes', 'pregnancy', 'pain'], ['hemorrhage', 'osteoarthritis', 'postpartum hemorrhage', 'pregnancy', 'Premature rupture of membranes'], ['Premature rupture of membranes', 'pain', "Crohn's disease", 'pregnancy', 'red blood cell density measurement'], ['asthma', 'neoplasm', 'hepatocellular carcinoma', 'experimental autoimmune encephalomyelitis', 'neuroblastoma'], ['glaucoma', 'open-angle glaucoma', 'ocular hypertension', 'hypotrichosis', 'Vitiligo'], ['pulmonary arterial hypertension', 'pulmonary hypertension', 'Recurrent thrombophlebitis', 'cardiovascular disease', 'heart failure'], ['rheumatoid arthritis', 'ulcerative colitis', 'migraine disorder', 'osteoarthritis', 'coronary artery disease'], ['rheumatoid arthritis', 'migraine disorder', 'osteoarthritis', 'ulcerative colitis', 'coronary artery disease'], ['hypoparathyroidism, familial isolated 1', 'Familial isolated hypoparathyroidism due to impaired PTH secretion', 'Familial isolated hypoparathyroidism', 'primary hyperparathyroidism', 'chronic kidney disease'], ['Brachydactyly', 'Brachydactyly type E', 'body height', 'breast carcinoma', 'hair shape measurement'], ['appendicular lean mass', 'brain volume measurement', 'mean corpuscular hemoglobin', 'atrial fibrillation', 'mathematical ability'], ["Alzheimer's disease", 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'monocyte count', 'glioblastoma multiforme'], ['BMI-adjusted waist-hip ratio', 'age at menarche', 'waist-hip ratio', 'neoplasm', 'colorectal carcinoma'], ['eosinophil count', 'appendicular lean mass', 'lymphocyte count', 'monocyte count', 'eosinophil percentage of leukocytes'], ['rheumatoid arthritis', 'eosinophil count', "Crohn's disease", 'hemoglobin measurement', 'hematocrit'], ['cutaneous melanoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['Noonan syndrome', 'LEOPARD syndrome', 'Juvenile Myelomonocytic Leukemia', 'Metachondromatosis', 'acute myeloid leukemia'], ['T-B+ severe combined immunodeficiency due to CD45 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'Omenn syndrome', 'lymphocyte count', 'mean corpuscular hemoglobin'], ['blood protein measurement', 'appendicular lean mass', 'Short stature due to GHSR deficiency', 'reticulocyte count', 'mean corpuscular volume'], ['non-syndromic intellectual disability', 'self reported educational attainment', 'mathematical ability', 'protein measurement', 'prostate carcinoma'], ['Global developmental delay', 'Aceruloplasminemia', 'hearing loss', 'breast adenocarcinoma', 'esophageal cancer'], ['6-pyruvoyl-tetrahydropterin synthase deficiency', 'Hyperphenylalaninemia', 'Intellectual disability', 'dystonic disorder', 'heel bone mineral density'], ['8q24.3 microdeletion syndrome', 'genetic disorder', 'Intellectual disability', 'syndromic intellectual disability', 'CHARGE syndrome'], ['mean platelet volume', 'platelet count', 'platelet component distribution width', 'lymphocyte count', 'blood protein measurement'], ['mean corpuscular hemoglobin', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'body height'], ['head and neck squamous cell carcinoma', 'neoplasm', 'prostate cancer', "Alzheimer's disease", 'melanoma'], ['Autosomal recessive cutis laxa type 2B', 'De Barsy syndrome', 'Autosomal recessive cutis laxa type 2', 'PYCR1-related De Barsy syndrome', 'genetic disorder'], ['DNA methylation', 'ovarian cancer', 'blood protein measurement', 'apolipoprotein B measurement', 'pulse pressure measurement'], ['schizophrenia', 'chronic lymphocytic leukemia', 'body height', 'blood protein measurement', 'lymphocyte percentage of leukocytes'], ['blood protein measurement', 'pulse pressure measurement', 'breast adenocarcinoma', 'red blood cell density measurement', 'neoplasm'], ['body height', 'forced expiratory volume', 'urate measurement', 'BMI-adjusted waist circumference', 'vital capacity'], ['forced expiratory volume', 'waist-hip ratio', 'psoriasis', 'body height', 'chronotype measurement'], ['neoplasm', 'cancer', 'colorectal carcinoma', 'osteosarcoma', 'breast carcinoma'], ['breast adenocarcinoma', 'squamous cell lung carcinoma', 'response to vaccine', 'cytokine measurement', 'hemoglobin measurement'], ['neoplasm', 'rheumatoid arthritis', 'refractory anemia', 'breast cancer', 'acute myeloid leukemia'], ['pulse pressure measurement', 'pathological myopia', 'smoking behavior', 'appendicular lean mass', 'small cell lung carcinoma'], ['mental retardation, autosomal dominant 48', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'cutaneous Leishmaniasis', 'cancer'], ['lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'gastric adenocarcinoma', 'clear cell renal carcinoma'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['Nijmegen breakage syndrome-like disorder', 'Inherited cancer-predisposing syndrome', 'Hereditary breast and ovarian cancer syndrome', 'ovarian cancer', 'low grade glioma'], ['Fanconi anemia', 'Familial congenital mirror movements', 'Hereditary breast cancer', 'breast carcinoma', 'genetic disorder'], ['electrocardiography', 'parental longevity', 'neoplasm', 'breast cancer', 'kidney cancer'], ['Noonan syndrome', 'LEOPARD syndrome', 'dilated cardiomyopathy', 'hepatocellular carcinoma', 'rasopathy'], ['Omenn syndrome', 'Combined immunodeficiency with skin granulomas', 'Severe combined immunodeficiency due to complete RAG1/2 deficiency', 'Combined immunodeficiency T+ B+ due to partial RAG1 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['Omenn syndrome', 'Combined immunodeficiency with skin granulomas', 'Severe combined immunodeficiency due to complete RAG1/2 deficiency', 'rag2 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['Smith-Magenis syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Combined immunodeficiency due to ORAI1 deficiency'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'neoplasm', 'melanoma', 'glioblastoma multiforme'], ['skin cancer', 'non-melanoma skin carcinoma', 'body mass index', 'hair color', 'skin carcinoma'], ['adolescent idiopathic scoliosis', 'obesity', 'Familial cylindromatosis', 'pseudotumor cerebri', 'Capillary malformation - arteriovenous malformation'], ['type II diabetes mellitus', 'BMI-adjusted waist-hip ratio', 'chondrosarcoma', 'Female infertility due to zona pellucida defect', 'female infertility due to oocyte meiotic arrest'], ['glioblastoma multiforme', 'adolescent idiopathic scoliosis', 'hepatocellular carcinoma', 'ulcerative colitis', 'eosinophil percentage of leukocytes'], ['reticulocyte count', 'cutaneous melanoma', 'gut microbiome measurement', 'body height', 'optic disc size measurement'], ['albuminuria', 'body height', 'systolic blood pressure', 'diastolic blood pressure', 'cutaneous melanoma'], ['acute promyelocytic leukemia', 'acne', 'psoriasis', 'acute myeloid leukemia', 'freckles'], ['Matthew-Wood syndrome', 'acne', 'acute promyelocytic leukemia', 'psoriasis', 'Syndromic microphthalmia'], ['mean platelet volume', 'urinary metabolite measurement', 'diet measurement', 'alcohol consumption measurement', 'aspartate aminotransferase measurement'], ['FEV/FEC ratio', 'white matter hyperintensity measurement', 'endometrial carcinoma', 'non-small cell lung carcinoma', 'colorectal cancer'], ['Retinoblastoma', 'small cell lung carcinoma', 'retinoblastoma (nonhereditary)', 'bladder carcinoma', 'cutaneous melanoma'], ['viral disease', 'cancer', 'breast carcinoma', 'melanoma', 'endometrial cancer'], ['genetic disorder', 'lung adenocarcinoma', 'breast adenocarcinoma', 'red blood cell distribution width', 'cutaneous melanoma'], ['lean body mass', 'appendicular lean mass', 'body height', 'self reported educational attainment', 'fat body mass'], ['high density lipoprotein cholesterol measurement', 'neoplasm', 'lung cancer', 'Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder'], ['Thrombocytopenia - absent radius', 'Global developmental delay', 'Clinodactyly of the 5th finger', 'Abnormality of brain morphology', 'endometrial cancer'], ['TARP syndrome', 'lung adenocarcinoma', 'colorectal adenocarcinoma', 'non-small cell lung carcinoma', 'genetic disorder'], ['refractive error measurement', 'rheumatoid arthritis', 'multiple sclerosis', 'refractory anemia', 'systemic scleroderma'], ['Progressive retinal dystrophy due to retinol transport defect', 'Colobomatous microphthalmia', 'Reticular dystrophy of the retinal pigment epithelium', 'microphthalmia', 'Retinitis pigmentosa'], ['cancer', 'multiple myeloma', 'HIV infection', 'immune system disease', 'Mantle cell lymphoma'], ['Reticular dystrophy of the retinal pigment epithelium', 'Familial exudative vitreoretinopathy', 'Coats disease', 'Retinitis pigmentosa', 'neutrophil count'], ['FEV/FEC ratio', 'mean platelet volume', 'Miyoshi myopathy', 'multiple myeloma', 'myeloid neoplasm'], ['ovarian cancer', 'atrial fibrillation', 'breast carcinoma', 'age-related macular degeneration', 'Stargardt disease'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'hearing loss', 'Non-syndromic genetic deafness', 'testosterone measurement'], ['Rothmund-Thomson syndrome', 'Baller-Gerold syndrome', 'RAPADILINO syndrome', 'Rothmund-Thomson syndrome type 2', 'osteosarcoma'], ['ovarian cancer', 'heel bone mineral density', 'worry measurement', 'bone quantitative ultrasound measurement', 'high density lipoprotein cholesterol measurement'], ['neoplasm', 'Wolfram syndrome', 'glioma', 'systemic lupus erythematosus', 'synovial sarcoma'], ['ulcer disease', 'Primary immunodeficiency', 'gastric adenocarcinoma', 'Rare pervasive developmental disorder', 'breast adenocarcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Recurrent infections', 'lung adenocarcinoma', 'low density lipoprotein cholesterol measurement', 'free cholesterol measurement'], ['Lissencephaly syndrome, Norman-Roberts type', 'temporal lobe epilepsy', 'Lissencephaly', 'Rolandic epilepsy', 'Autosomal dominant epilepsy with auditory features'], ['monocyte count', 'eosinophil count', 'platelet count', 'protein measurement', 'platelet crit'], ['eosinophil count', 'red blood cell distribution width', 'eosinophil percentage of leukocytes', 'mean reticulocyte volume', 'mean corpuscular volume'], ['Multiple endocrine neoplasia type 2A', 'Hirschsprung disease', 'Familial medullary thyroid carcinoma', 'Multiple endocrine neoplasia type 2', 'Multiple endocrine neoplasia type 2B'], ['ulcerative colitis', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia', 'Glycogen storage disease due to liver phosphorylase kinase deficiency', 'adrenal gland pheochromocytoma'], ['cerebellar ataxia, neuropathy, and vestibular areflexia syndrome', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'polyneuropathy', 'mean corpuscular hemoglobin concentration'], ['endometrial cancer', 'lung adenocarcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'esophageal cancer'], ['Immunodeficiency by defective expression of HLA class 2', 'Combined T and B cell immunodeficiency', 'inflammatory bowel disease', 'hepatocellular carcinoma', 'neoplasm'], ['Immunodeficiency by defective expression of HLA class 2', 'pancreatic carcinoma', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm', 'Isolated osteopoikilosis'], ['heel bone mineral density', 'blood protein measurement', 'high density lipoprotein cholesterol measurement', 'sleep duration', 'opioid dependence'], ['Rh deficiency syndrome', 'red blood cell distribution width', 'platelet count', 'low density lipoprotein cholesterol measurement', 'reticulocyte count'], ['Rh deficiency syndrome', 'red blood cell distribution width', 'mean corpuscular volume', 'platelet count', 'platelet crit'], ['Retinitis pigmentosa', 'Congenital stationary night blindness', 'Retinitis punctata albescens', 'Retinal dystrophy', 'Saldino-Mainzer syndrome'], ['systolic blood pressure', 'breast adenocarcinoma', 'HMG CoA reductase inhibitor use measurement', 'neoplasm', 'hepatocellular carcinoma'], ['platelet count', 'mean platelet volume', 'platelet crit', 'neutrophil count', 'reticulocyte measurement'], ['Epidermodysplasia verruciformis', 'T-cell immunodeficiency with epidermodysplasia verruciformis', 'lymphoid neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'breast ductal adenocarcinoma'], ['protein measurement', 'appendicitis', 'neuroblastoma', 'body height', 'lean body mass'], ['reticulocyte count', 'reticulocyte measurement', 'red blood cell density measurement', 'Immature Reticulocyte Fraction Measurement', 'platelet count'], ['blood protein measurement', 'synovial sarcoma', 'Wolfram syndrome', 'Progressive supranuclear palsy', 'Classical progressive supranuclear palsy'], ['cone-rod synaptic disorder syndrome, congenital nonprogressive', 'self reported educational attainment', 'demyelinating disease', 'intelligence', 'particulate matter air pollution measurement'], ['immunodeficiency disease', 'autoinflammation with episodic fever and lymphadenopathy', 'severe acute respiratory syndrome', 'Primary immunodeficiency', 'Autosomal recessive early-onset inflammatory bowel disease'], ['leprosy', "Crohn's disease", 'protein measurement', 'ulcer disease', 'Oral ulcer'], ['severe acute respiratory syndrome', 'viral disease', 'mitral valve disease', 'hematocrit', 'hemoglobin measurement'], ['Bartsocas-Papas syndrome', 'CHAND syndrome', 'skin cancer', 'basal cell carcinoma', 'non-melanoma skin carcinoma'], ["Parkinson's disease", 'body mass index', 'adolescent idiopathic scoliosis', 'disease progression measurement', 'metastatic colorectal cancer'], ['Retinitis punctata albescens', 'Retinitis pigmentosa', 'Fundus albipunctatus', 'Bothnia retinal dystrophy', 'Retinal dystrophy'], ['glioblastoma multiforme', 'cutaneous melanoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['cutaneous melanoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'neoplasm', 'colorectal carcinoma'], ['myeloid white cell count', 'Niemann-Pick disease type C', 'self reported educational attainment', 'neutrophil count', 'alcohol consumption measurement'], ['eosinophil count', 'lean body mass', 'vitamin D measurement', 'diastolic blood pressure', 'fat body mass'], ['waist-hip ratio', 'red blood cell distribution width', "Alzheimer's disease", 'skin basal cell carcinoma', 'mean corpuscular volume'], ['blood protein measurement', 'visual perception measurement', 'neoplasm', 'acute kidney failure', "Barrett's esophagus"], ['blood protein measurement', 'eosinophil count', 'eosinophil cationic protein measurement', 'glioma', 'asthma'], ['nose morphology measurement', 'lung adenocarcinoma', 'ovarian cancer', 'eosinophil cationic protein measurement', 'squamous cell lung carcinoma'], ['Adult-onset chronic progressive external ophthalmoplegia with mitochondrial myopathy', 'Mitochondrial disease', 'Hyperlysinemia', 'Barth syndrome', 'progressive external ophthalmoplegia'], ['Cystic fibrosis', 'sRAGE measurement', 'waist circumference', 'cancer', 'neoplasm'], ['lymphocyte count', 'monocyte count', 'myeloid white cell count', 'hematocrit', 'testosterone measurement'], ['autoinflammatory syndrome with pyogenic bacterial infection and amylopectinosis', 'Polyglucosan body myopathy', 'Short stature due to GHSR deficiency', 'ovarian carcinoma', 'breast cancer'], ['squamous cell lung carcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'clear cell renal carcinoma', 'urinary bladder cancer'], ['Hyperplastic polyposis syndrome', 'gastric adenocarcinoma', 'pancreatic carcinoma', 'colorectal adenocarcinoma', 'pancreatic adenocarcinoma'], ['self reported educational attainment', 'lifestyle measurement', 'mathematical ability', 'glioma', 'glioblastoma multiforme'], ['mean corpuscular volume', 'ovarian cancer', 'breast adenocarcinoma', 'waist-hip ratio', 'esophageal cancer'], ['HIV infection', 'mental or behavioural disorder', 'acute myeloid leukemia', 'Cryopyrin-associated periodic syndrome', 'prostate cancer'], ['psoriasis', 'reticulocyte count', 'reticulocyte measurement', 'Immature Reticulocyte Fraction Measurement', 'Blackfan-Diamond anemia'], ['Isolated growth hormone deficiency type IB', 'isolated congenital growth hormone deficiency', 'Isolated growth hormone deficiency type IA', 'chronic lymphocytic leukemia', 'esophageal squamous cell carcinoma'], ['Pituitary stalk interruption syndrome', 'Congenital hydronephrosis', 'heel bone mineral density', 'neuroimaging measurement', 'lobe attachment'], ['Familial vesicoureteral reflux', 'chronotype measurement', 'body mass index', 'snoring measurement', 'type II diabetes mellitus'], ['Horizontal gaze palsy with progressive scoliosis', 'lung adenocarcinoma', 'cutaneous melanoma', 'genetic disorder', 'macula measurement'], ['aortic valve disease', 'Familial bicuspid aortic valve', 'aneurysm or dilatation of ascending aorta', 'Bicuspid aortic valve', 'congenital diaphragmatic hernia'], ['ocular hypertension', 'open-angle glaucoma', 'glaucoma', 'Fuchs endothelial corneal dystrophy', 'breast neoplasm'], ['ocular hypertension', 'open-angle glaucoma', 'glaucoma', 'mean corpuscular volume', 'lymphocyte count'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'vital capacity', 'Macular dystrophy', 'forced expiratory volume'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'acute myeloid leukemia', 'cortical surface area measurement', 'neuroimaging measurement'], ['intellectual developmental disorder with or without epilepsy or cerebellar ataxia', 'body height', 'protein measurement', 'body mass index', 'asthma'], ['generalised epilepsy', 'genetic disorder', 'refractive error measurement', 'testosterone measurement', 'age at menarche'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'inflammatory bowel disease', 'ulcerative colitis', 'Eczema', 'asthma'], ['non-small cell lung carcinoma', 'neoplasm', 'lung adenocarcinoma', 'lung carcinoma', 'cutaneous melanoma'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Leber congenital amaurosis', 'Macular dystrophy', 'Abnormality of the eye'], ['blood metabolite measurement', 'glycine measurement', 'serum metabolite measurement', 'body height', 'glomerular filtration rate'], ['Leber congenital amaurosis', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Severe early-childhood-onset retinal dystrophy', 'Adrenocorticotropic hormone deficiency'], ['cortical surface area measurement', 'brain volume measurement', 'Exercise-induced hyperinsulinism', 'Renal-hepatic-pancreatic dysplasia', 'Testicular regression syndrome'], ['Retinitis pigmentosa', 'Cone rod dystrophy', 'Primary ciliary dyskinesia - retinitis pigmentosa', 'Primary ciliary dyskinesia', 'Retinal dystrophy'], ['Meckel syndrome', 'Joubert syndrome with hepatic defect', 'Joubert syndrome with renal defect', 'Joubert syndrome', 'body mass index'], ['Blackfan-Diamond anemia', 'influenza', 'genetic disorder', 'non-small cell lung carcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['X-linked syndromic intellectual disability', 'X-linked intellectual disability - psychosis - macroorchidism', 'autism', 'influenza', 'multiple myeloma'], ['Blackfan-Diamond anemia', 'influenza', 'Short thumb', 'Abnormality of radial ray', 'Fanconi anemia'], ['spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'influenza', 'Spondyloepimetaphyseal dysplasia, matrilin-3 type', 'peak expiratory flow', 'hair color'], ['influenza', 'renal cell carcinoma', 'colorectal cancer', 'eosinophil count', 'colorectal adenoma'], ['Blackfan-Diamond anemia', 'influenza', 'intelligence', 'cognitive function measurement', 'glioblastoma multiforme'], ['influenza', 'endometrial cancer', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['influenza', 'clear cell renal carcinoma', 'endometrial cancer', 'neoplasm', 'colorectal carcinoma'], ['Hypotrichosis simplex', 'influenza', 'aspartate aminotransferase measurement', 'endometrial cancer', 'monocyte count'], ['influenza', 'colorectal adenocarcinoma', 'colon adenocarcinoma', 'bile duct carcinoma', 'basal cell carcinoma'], ['Blackfan-Diamond anemia', 'influenza', 'neoplasm', 'Thymoma', 'breast cancer'], ['influenza', 'Blackfan-Diamond anemia', 'myeloid white cell count', 'clear cell renal carcinoma', 'leukocyte count'], ['influenza', 'acute myeloid leukemia', 'lung cancer', 'lean body mass', 'body mass index'], ['Blackfan-Diamond anemia', 'influenza', 'neuroblastoma', 'grip strength measurement', 'Abnormal sperm morphology'], ['influenza', 'Blackfan-Diamond anemia', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['influenza', 'protein measurement', 'DNA methylation', 'Autosomal recessive spondylocostal dysostosis', 'Spondylometaphyseal dysplasia, A4 type'], ['influenza', 'endometrial cancer', 'metaplastic breast carcinoma', 'neoplasm', 'pancreatic carcinoma'], ['influenza', 'body mass index', 'body height', 'lean body mass', 'fat body mass'], ['influenza', 'rheumatoid arthritis', 'hair shape measurement', 'lymphocyte count', 'chronic lymphocytic leukemia'], ['influenza', 'breast carcinoma', 'Abnormal sperm morphology', 'Keratoconus', 'body fat percentage'], ['COVID-19', 'monocyte count', 'monocyte percentage of leukocytes', 'leukocyte count', 'basophil count'], ['breast adenocarcinoma', 'lung adenocarcinoma', "Crohn's disease", 'inflammatory bowel disease', 'red blood cell distribution width'], ['influenza', 'clear cell renal carcinoma', 'serum albumin measurement', 'total cholesterol measurement', 'blood protein measurement'], ['influenza', 'electrocardiography', "Parkinson's disease", 'heel bone mineral density', 'glioblastoma multiforme'], ['influenza', 'clear cell renal carcinoma', 'pancreatic adenocarcinoma', 'smoking behavior', 'gastric cancer'], ['influenza', 'neoplasm', 'breast cancer', 'myelodysplastic syndrome', 'Menkes disease'], ['Blackfan-Diamond anemia', 'influenza', 'Short thumb', 'Abnormality of radial ray', 'kidney failure'], ['influenza', 'Blackfan-Diamond anemia', 'monocyte count', 'systemic lupus erythematosus', 'body height'], ['Blackfan-Diamond anemia', 'influenza', 'Abnormality of radial ray', 'clear cell renal carcinoma', 'protein measurement'], ['Blackfan-Diamond anemia', 'influenza', 'osteosarcoma', 'clear cell renal carcinoma', 'late-onset Alzheimers disease'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'influenza'], ['influenza', 'blood metabolite measurement', 'serum metabolite measurement', 'metabolite measurement', 'pulse pressure measurement'], ['Blackfan-Diamond anemia', 'influenza', 'clear cell renal carcinoma', 'endometrial cancer', 'mathematical ability'], ['Familial isolated congenital asplenia', 'influenza', 'schizophrenia', 'dentate gyrus volume measurement', 'small cell lung carcinoma'], ['genetic disorder', 'Noonan syndrome', 'endometrial cancer', 'lung adenocarcinoma', 'hypertension'], ['eosinophil count', 'platelet count', 'alkaline phosphatase measurement', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['X-linked retinoschisis', 'retinoschisis', 'Retinal dystrophy', 'North Carolina macular dystrophy', 'Angelman syndrome'], ['chronotype measurement', 'systolic blood pressure', 'pulse pressure measurement', 'Obesity due to melanocortin 4 receptor deficiency', 'Obesity due to congenital leptin deficiency'], ['age at menarche', 'cervical cancer', 'melanoma', 'gastric cancer', 'handedness'], ['Testicular Germ Cell Tumor', 'heel bone mineral density', 'red blood cell density measurement', 'adolescent idiopathic scoliosis', 'age at menopause'], ['Tetraamelia - multiple malformations', 'humerofemoral hypoplasia with radiotibial ray deficiency', 'hair shape measurement', 'esophageal adenocarcinoma', 'prostate carcinoma'], ['Anonychia congenita totalis', 'Congenital anonychia', 'Anonychia', 'genetic disorder', 'Ectodermal dysplasia syndrome'], ['myeloid white cell count', 'neutrophil count', 'ovarian carcinoma', 'cancer', 'Plasmodium falciparum malaria'], ['Dyskeratosis congenita', 'idiopathic pulmonary fibrosis', 'pulmonary fibrosis and/or bone marrow failure, telomere-related, 1', 'Hoyeraal-Hreidarsson syndrome', 'acute myeloid leukemia'], ['Autosomal dominant spastic paraplegia type 12', 'Hereditary spastic paraplegia', 'Spastic paraplegia', 'drug use measurement', 'infect'], ['blood protein measurement', 'non-small cell lung carcinoma', 'breast cancer', 'urinary bladder cancer', 'neoplasm'], ['basal cell carcinoma', 'acute myeloid leukemia', 'mathematical ability', 'acute lymphoblastic leukemia', 'lymphoid neoplasm'], ['Cleidocranial dysplasia', 'Metaphyseal dysplasia - maxillary hypoplasia - brachydacty', 'Craniosynostosis', 'brain volume measurement', 'neuroimaging measurement'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'red blood cell distribution width', 'head and neck squamous cell carcinoma', 'eosinophil count'], ['breast adenocarcinoma', 'blood protein measurement', 'endometrial cancer', 'testosterone measurement', 'Glycogen storage disease due to muscle and heart glycogen synthase deficiency'], ['reticulocyte count', 'heart failure', 'cardiovascular disease', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['cryptorchidism', 'hypertension', 'cardiovascular disease', 'systolic blood pressure', 'hemoglobin measurement'], ['ovarian cancer', 'cutaneous melanoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['Congenital multicore myopathy with external ophthalmoplegia', 'malignant hyperthermia, susceptibility to, 1', 'King-Denborough syndrome', 'Central core disease', 'Malignant hyperthermia'], ['Catecholaminergic polymorphic ventricular tachycardia', 'catecholaminergic polymorphic ventricular tachycardia 1', 'Familial isolated arrhythmogenic right ventricular dysplasia', 'Arrhythmogenic right ventricular dysplasia', 'Abnormality of the cardiovascular system'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'chronic progressive multiple sclerosis', 'secondary progressive multiple sclerosis'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'deafness', 'immune system disease'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'monocyte count', 'Cognitive impairment', 'immune system disease'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'Cognitive impairment', 'monocyte count'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'Cognitive impairment', 'primary progressive multiple sclerosis'], ['Primary immunodeficiency', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'platelet crit', 'hepatocellular carcinoma'], ['pulse pressure measurement', 'prostate carcinoma', 'neoplasm', 'ovarian cancer', "Crohn's disease"], ['protein measurement', 'neoplasm', 'pancreatic carcinoma', 'hepatocellular carcinoma', 'Malignant Pancreatic Neoplasm'], ['Primary immunodeficiency', 'glioblastoma multiforme', 'cutaneous melanoma', 'endometrial cancer', 'inflammatory bowel disease'], ['Primary immunodeficiency', 'cutaneous melanoma', 'basophil percentage of leukocytes', 'matrix metalloproteinase 8 measurement', 'basophil count'], ['blood metabolite measurement', 'hemoglobin measurement', 'diverticulitis', 'diverticular disease', 'childhood onset asthma'], ['endometrial cancer', 'neutrophil count', 'monocyte percentage of leukocytes', 'basophil percentage of leukocytes', 'granulocyte percentage of myeloid white cells'], ['head and neck squamous cell carcinoma', 'cutaneous melanoma', 't-tau:beta-amyloid 1-42 ratio measurement', 'lymphocyte count', "Alzheimer's disease"], ['Autosomal recessive spastic ataxia of Charlevoix-Saguenay', 'Spastic paraplegia', 'genetic disorder', 'Autosomal recessive spastic ataxia', 'Abnormality of brain morphology'], ['Oguchi disease', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Congenital stationary night blindness', 'Progressive cone dystrophy'], ['Townes-Brocks syndrome', 'heel bone mineral density', 'erythrocyte count', 'hematocrit', 'C-reactive protein measurement'], ['drug use measurement', 'adenocarcinoma', 'neoplasm', 'pancreatic ductal adenocarcinoma', 'obesity'], ['Retinitis pigmentosa', 'Griscelli disease type 3', 'Uncombable hair syndrome', 'attention deficit hyperactivity disorder', 'Hypotrichosis simplex'], ['Sarcosinemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'Gastrointestinal stromal tumor', 'neoplasm'], ['Hereditary folate malabsorption', 'amyotrophic lateral sclerosis', 'blood protein measurement', 'semaphorin-3A measurement', 'interleukin 5 measurement'], ['neurodevelopmental disorder with microcephaly, ataxia, and seizures', 'Autosomal recessive non-syndromic intellectual disability', 'Cerebral arteriovenous malformation', 'infect', 'infection'], ['Hyperuricemia-pulmonary hypertension-renal failure-alkalosis syndrome', 'pulmonary arterial hypertension', 'multiple myeloma', 'B-cell non-Hodgkins lymphoma', 'head and neck carcinoma'], ['Disseminated superficial actinic porokeratosis', 'porokeratosis', 'neoplasm', 'hepatocellular carcinoma', 'infect'], ['testosterone measurement', 'sex hormone-binding globulin measurement', 'blood protein measurement', 'BMI-adjusted waist-hip ratio', 'waist-hip ratio'], ['mean platelet volume', 'platelet count', 'sitting height measurement', 'platelet component distribution width', 'Autosomal dominant spastic paraplegia type 3'], ['Shwachman-Diamond syndrome', 'acute myeloid leukemia', 'genetic disorder', 'Aplastic anemia', 'Deeply set eye'], ['heel bone mineral density', 'reticulocyte count', 'bone quantitative ultrasound measurement', 'lymphocyte count', 'bone density'], ['colorectal carcinoma', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma', 'gastric carcinoma', 'lung adenocarcinoma'], ['adolescent idiopathic scoliosis', 'body height', "Alzheimer's disease", 'appendicular lean mass', 'smoking status measurement'], ['von Willebrand factor measurement', 'blood protein measurement', 'factor VIII measurement', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['basophil count', 'basophil percentage of leukocytes', 'skin pigmentation', 'macula measurement', 'mean platelet volume'], ['type II diabetes mellitus', 'reticulocyte measurement', 'lymphocyte count', 'monocyte count', 'basophil count'], ['heel bone mineral density', 'neutrophil count', 'Huntington disease', "Alzheimer's disease", 'malaria'], ['protein measurement', 'mathematical ability', 'eosinophil count', 'appendicular lean mass', 'leukocyte count'], ['Generalized epilepsy with febrile seizures-plus', 'Dravet syndrome', 'migraine, familial hemiplegic, 3', 'Familial or sporadic hemiplegic migraine', 'Seizure'], ['Early infantile epileptic encephalopathy', 'Benign familial neonatal-infantile seizures', 'Seizure', 'Benign familial infantile epilepsy', 'bipolar disorder'], ['Familial focal epilepsy with variable foci', 'Early infantile epileptic encephalopathy', 'partial epilepsy', 'bipolar disorder', 'epilepsy'], ['Hyperkalemic periodic paralysis', 'Paramyotonia congenita of Von Eulenburg', 'Hypokalemic periodic paralysis', 'Potassium-aggravated myotonia', 'Acetazolamide-responsive myotonia'], ['Brugada syndrome', 'Familial atrial fibrillation', 'Familial long QT syndrome', 'Familial progressive cardiac conduction defect', 'Familial sick sinus syndrome'], ['epilepsy', 'pain', 'bipolar disorder', 'Seizure', 'unipolar depression'], ['Early infantile epileptic encephalopathy', 'epilepsy', 'cognitive impairment with or without cerebellar ataxia', 'Seizure', 'Benign familial infantile epilepsy'], ['Paroxysmal extreme pain disorder', 'Primary erythermalgia', 'Sodium channelopathy-related small fiber neuropathy', 'Generalized epilepsy with febrile seizures-plus', 'Channelopathy-associated congenital insensitivity to pain'], ['atrial fibrillation', 'Sodium channelopathy-related small fiber neuropathy', 'epilepsy', 'pain', 'bipolar disorder'], ['leukocyte count', 'myeloid white cell count', 'lymphocyte count', 'hemoglobin measurement', 'snoring measurement'], ['Fasciitis', 'central nervous system cancer', 'type II diabetes mellitus', 'Palmar Fibromatosis', 'heel bone mineral density'], ['Interauricular communication', 'atrial heart septal defect', 'diabetes mellitus', 'insulinoma', 'Hyperglycemia'], ['immature platelet fraction', 'idiopathic pulmonary fibrosis', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia', 'vital capacity'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'apolipoprotein A 1 measurement'], ['FEV/FEC ratio', 'appendicular lean mass', 'ovarian cancer', 'peak expiratory flow', 'breast adenocarcinoma'], ['lung adenocarcinoma', 'non-small cell lung carcinoma', 'gastric adenocarcinoma', 'squamous cell lung carcinoma', 'diffuse gastric adenocarcinoma'], ['neoplasm', 'esophageal squamous cell carcinoma', 'graft versus host disease', 'colorectal carcinoma', 'rheumatoid arthritis'], ['hair color', 'blood protein measurement', 'psoriasis', "Crohn's disease", 'sclerosing cholangitis'], ['adrenal gland pheochromocytoma', 'Hereditary pheochromocytoma-paraganglioma', 'Carney-Stratakis syndrome', 'Gastrointestinal stromal tumor', 'Cowden syndrome'], ['Hereditary pheochromocytoma-paraganglioma', 'adrenal gland pheochromocytoma', 'Carney-Stratakis syndrome', 'Isolated succinate-CoQ reductase deficiency', 'Cowden syndrome'], ['cerebrospinal fluid biomarker measurement', 'Gastrointestinal stromal tumor', 'neoplasm', 'body mass index', 'renal carcinoma'], ['HIV infection', 'influenza', 'viral disease', 'Histiocytosis', 'breast adenocarcinoma'], ['sex hormone-binding globulin measurement', 'neoplasm', 'cancer', 'non-small cell lung carcinoma', 'platelet crit'], ['blood protein measurement', 'monocyte percentage of leukocytes', 'neoplasm', 'melanoma', 'granulocyte percentage of myeloid white cells'], ['Sickle cell anemia', 'glioblastoma multiforme', 'e-selectin measurement', 'breast adenocarcinoma', 'age at menopause'], ['extraoral halitosis due to methanethiol oxidase deficiency', 'red blood cell distribution width', 'protein measurement', 'serum copper measurement', 'neoplasm'], ['Rigid spine syndrome', 'Congenital fiber-type disproportion myopathy', 'rigid spine muscular dystrophy 1', 'Classic multiminicore myopathy', 'Desmin-related myopathy with Mallory body-like inclusions'], ['lean body mass', 'whole body water mass', 'appendicular lean mass', 'fat body mass', 'vital capacity'], ['Sickle cell anemia', 'stroke', 'adhesion molecule measurement', 'soluble P-selectin measurement', 'blood protein measurement'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'inflammatory bowel disease', 'blood protein measurement', 'intraocular pressure measurement'], ['HIV infection', 'cortical surface area measurement', 'bone density', 'neuroimaging measurement', 'bone quantitative ultrasound measurement'], ['Kallmann syndrome', 'Brugada syndrome', 'brain volume measurement', 'Hirschsprung disease', 'PR interval'], ['Neurodevelopmental disorder', 'neoplasm', 'language impairment', 'anxiety', 'body height'], ['reticulocyte count', 'mean corpuscular volume', 'reticulocyte measurement', 'Abnormality of refraction', 'skin aging'], ['sex hormone-binding globulin measurement', 'pes', 'diastolic blood pressure', 'cancer', 'colorectal carcinoma'], ['refractive error measurement', 'glomerular filtration rate', 'body height', 'Non-syndromic male infertility due to sperm motility disorder', 'spermatogenic failure 43'], ['Alpha-1-antitrypsin deficiency', 'chronic obstructive pulmonary disease', 'Hemorrhagic disease due to alpha-1-antitrypsin Pittsburgh mutation', 'refractive error measurement', 'cholelithiasis'], ['endometrial cancer', 'Bronchopulmonary dysplasia', 'Borderline personality disorder', 'prostate cancer', 'nonpapillary renal cell carcinoma'], ['Alpha-1-antichymotrypsin deficiency', 'breast adenocarcinoma', 'prostate specific antigen measurement', 'alpha-1-antichymotrypsin measurement', 'alpha-1-antichymotrypsin complex measurement'], ['blood protein measurement', 'lung adenocarcinoma', 'glioblastoma multiforme', 'kallistatin measurement', 'cutaneous melanoma'], ['Corticosteroid-binding globulin deficiency', 'cortisol measurement', 'blood protein measurement', 'serum alpha-1-antitrypsin measurement', 'serum alanine aminotransferase measurement'], ['neutrophil count', 'childhood onset asthma', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract'], ['neoplasm', 'hepatocellular carcinoma', 'vitamin D measurement', 'psoriasis', 'protein measurement'], ['Hereditary thrombophilia due to congenital antithrombin deficiency', 'deep vein thrombosis', 'atrial fibrillation', 'pulmonary embolism', 'venous thromboembolism'], ['Venous thrombosis', 'heparin cofactor 2 deficiency', 'cutaneous melanoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['Congenital plasminogen activator inhibitor type 1 deficiency', 'Abnormal bleeding', 'erythrocyte count', 'plasminogen activator inhibitor 1 measurement', 'breast adenocarcinoma'], ['mean platelet volume', 'platelet-derived growth factor BB measurement', 'body height', 'interleukin 10 measurement', 'inflammatory biomarker measurement'], ['Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta type 4', 'blood protein measurement', 'forced expiratory volume'], ['Congenital alpha2 antiplasmin deficiency', 'Rare hemorrhagic disorder due to a constitutional coagulation factors defect', 'Abnormal bleeding', "Alzheimer's disease", 'testosterone measurement'], ['Hereditary angioedema type 1', 'Hereditary angioedema type 2', 'hereditary angioedema with C1Inh deficiency', 'Hereditary angioedema', 'Bartholin gland disease'], ['breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['intellectual disability, autosomal dominant 58', 'acute lymphoblastic leukemia', 'T-cell acute lymphoblastic leukemia', 'Intellectual disability', 'Autosomal dominant non-syndromic intellectual disability'], ['Schinzel-Giedion syndrome', 'chronic myelogenous leukemia', 'acute myeloid leukemia', 'Juvenile Myelomonocytic Leukemia', 'myelodysplastic syndrome'], ['Luscan-Lumish syndrome', 'acute lymphoblastic leukemia', 'clear cell renal carcinoma', 'acute myeloid leukemia', 'renal cell carcinoma'], ['monocyte count', 'Rare pervasive developmental disorder', 'smoking status measurement', 'cigarettes per day measurement', 'smoking initiation'], ['acute myeloid leukemia', 'prostate carcinoma', 'adolescent idiopathic scoliosis', 'Hepatitis', 'mathematical ability'], ['chronic lymphocytic leukemia', 'acute myeloid leukemia', 'Uveal Melanoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['breast carcinoma', 'breast neoplasm', 'chronic lymphocytic leukemia', 'hepatocellular carcinoma', 'pancreatic adenocarcinoma'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'squamous cell lung carcinoma'], ['electrocardiography', 'body height', 'appendicular lean mass', 'FEV/FEC ratio', 'sitting height measurement'], ['alpha-linolenic acid measurement', 'gastric cancer', 'melanoma', 'hepatocellular carcinoma', 'colorectal carcinoma'], ['cutaneous melanoma', 'intelligence', 'body height', 'mathematical ability', 'cognitive function measurement'], ['Autosomal recessive limb-girdle muscular dystrophy type 2D', 'Limb-girdle muscular dystrophy', 'Autosomal recessive limb-girdle muscular dystrophy', 'Qualitative or quantitative defects of sarcoglycan', 'Muscular dystrophy'], ['Autosomal recessive limb-girdle muscular dystrophy type 2E', 'Limb-girdle muscular dystrophy', 'Autosomal recessive limb-girdle muscular dystrophy', 'Qualitative or quantitative defects of beta-sarcoglycan', 'genetic disorder'], ['Autosomal recessive limb-girdle muscular dystrophy type 2C', 'Autosomal recessive limb-girdle muscular dystrophy', 'type II diabetes mellitus', 'acute myeloid leukemia', 'muscular dystrophy, limb-girdle, autosomal recessive 23'], ['electrocardiography', 'hair color', 'sex hormone-binding globulin measurement', 'hair colour measurement', 'protein measurement'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'prostate carcinoma', 'Uncombable hair syndrome', 'Ringed hair disease'], ['Calvarial doughnut lesions - bone fragility', 'genetic disorder', 'protein measurement', 'Osteopenia', 'breast cancer'], ['platelet count', 'cancer', 'neoplasm', 'testosterone measurement', '46,XY disorder of sex development due to 5-alpha-reductase 2 deficiency'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'sex hormone-binding globulin measurement', 'eosinophil count', 'eosinophil percentage of leukocytes'], ['X-linked lymphoproliferative disease', 'adult lymphoma', 'genetic disorder', 'Aplastic anemia', 'Lymphoproliferative syndrome'], ['infect', 'infection', 'platelet component distribution width', 'mean platelet volume', 'platelet count'], ['Cherubism', 'chronotype measurement', 'platelet count', 'mean platelet volume', 'Gorham-Stout disease'], ['Age-related cataract', 'neutrophil count', 'prostate carcinoma', 'cognitive function measurement', 'self reported educational attainment'], ['Monosomy 22q13', 'schizophrenia', 'Intellectual disability', 'mental retardation', 'genetic disorder'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'heel bone mineral density', 'blood protein measurement', 'type II diabetes mellitus'], ['cancer', 'lung adenocarcinoma', 'acute myeloid leukemia', 'thyroid gland adenocarcinoma', 'endometrial cancer'], ['Holoprosencephaly', 'holoprosencephaly 3', 'Microform holoprosencephaly', 'Solitary median maxillary central incisor', 'Polydactyly of a triphalangeal thumb'], ['eosinophilic pneumonia', 'obsessive-compulsive disorder', 'Pentosuria', 'colorectal adenocarcinoma', 'Hereditary geniospasm'], ['Congenital sucrase-isomaltase deficiency', 'type II diabetes mellitus', 'intestinal disaccharide deficiency and disaccharide malabsorption', 'genetic disorder', 'abnormal glucose tolerance'], ['autoimmune disease', 'serum metabolite measurement', 'urinary system disease', 'prostate carcinoma', 'primary biliary cirrhosis'], ['red blood cell density measurement', 'colorectal carcinoma', 'breast cancer', 'hepatocellular carcinoma', 'mean corpuscular hemoglobin concentration'], ['lymphocyte count', 'apolipoprotein A 1 measurement', 'triglyceride measurement', 'systemic lupus erythematosus', 'lupus erythematosus'], ['cerebrospinal fluid biomarker measurement', 'neoplasm', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', "Alzheimer's disease"], ['monocyte percentage of leukocytes', 'blood protein measurement', 'sialic acid-binding Ig-like lectin 9 measurement', 'monocyte count', 'cerebrospinal fluid biomarker measurement'], ['COVID-19', 'Combined immunodeficiency with skin granulomas', 'Pancytopenia due to IKZF1 mutations', 'Hyper-IgM syndrome type 3', 'Autosomal dominant severe congenital neutropenia'], ['gastric carcinoma', 'esophageal carcinoma', 'cancer', 'neoplasm', 'bladder carcinoma'], ['Distal hereditary motor neuropathy, Jerash type', 'Juvenile amyotrophic lateral sclerosis', 'pain', 'sinusitis', 'Cough'], ['Early infantile epileptic encephalopathy', 'Early myoclonic encephalopathy', 'West syndrome', 'systolic blood pressure', 'hair color'], ['Rolandic epilepsy', 'Hyperinsulinism due to INSR deficiency', 'Hyperinsulinism due to glucokinase deficiency', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'Autosomal dominant hyperinsulinism due to Kir6.2 deficiency'], ['self reported educational attainment', 'mathematical ability', 'allergy', 'asthma', 'Eczema'], ['Spondyloepiphyseal dysplasia and spondyloepimetaphyseal dysplasia', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement', 'vital capacity', 'proprotein convertase subtilisin/kexin type 7 measurement'], ['Marinesco-Sjögren syndrome', 'excessive daytime sleepiness measurement', 'lifestyle measurement', 'appendicular lean mass', 'glioma'], ['hematocrit', 'erythrocyte count', 'hemoglobin measurement', 'appendicular lean mass', 'sitting height measurement'], ['type II diabetes mellitus', 'HbA1c measurement', 'body mass index', 'Testicular regression syndrome', 'Normosmic congenital hypogonadotropic hypogonadism'], ['mean platelet volume', 'platelet crit', 'platelet component distribution width', 'reticulocyte measurement', 'platelet count'], ['breast adenocarcinoma', 'mean platelet volume', 'testosterone measurement', 'systolic blood pressure', 'neutrophil count'], ['neoplasm', 'colorectal carcinoma', 'Ehlers-Danlos syndrome, progeroid type', 'non-small cell lung carcinoma', 'melanoma'], ['mean platelet volume', 'platelet count', 'monocyte count', 'lymphocyte percentage of leukocytes', 'lymphocyte count'], ['facial morphology measurement', 'heel bone mineral density', 'hepatocellular carcinoma', 'kidney Wilms tumor', 'lung adenocarcinoma'], ['Holoprosencephaly', 'Schizencephaly', 'Microform holoprosencephaly', 'alcohol consumption measurement', 'smoking status measurement'], ['prostate carcinoma', 'glomerular filtration rate', 'male reproductive organ cancer', 'aspartate aminotransferase measurement', 'prostate cancer'], ['mean corpuscular volume', 'red blood cell distribution width', 'appendicular lean mass', 'body height', 'lean body mass'], ['hair colour measurement', 'inflammatory bowel disease', 'appendicular lean mass', 'cancer', 'gut microbiome measurement'], ['Graves disease', 'pancreatic adenocarcinoma', 'myxedema', 'rheumatoid arthritis', 'eosinophil count'], ['multiple myeloma', 'neoplasm', 'SLAM family member 7 measurement', 'blood protein measurement', 'protein measurement'], ['Dicarboxylic aminoaciduria', 'schizophrenia', 'Sensorineural hearing impairment', 'response to risperidone', 'Rare genetic intellectual disability with developmental anomaly'], ['appendicular lean mass', 'low density lipoprotein cholesterol measurement', 'sun exposure measurement', 'eye measurement', 'neoplasm'], ['Encephalopathy due to GLUT1 deficiency', 'Paroxysmal exertion-induced dyskinesia', 'generalised epilepsy', 'Paroxysmal dystonic choreathetosis with episodic ataxia and spasticity', 'Hereditary cryohydrocytosis with reduced stomatin'], ['prostate carcinoma', 'sex hormone-binding globulin measurement', 'multiple sclerosis', 'testosterone measurement', 'body height'], ['Arterial tortuosity syndrome', 'Abnormality of the cardiovascular system', 'marfan syndrome/loeys-dietz syndrome/familial thoracic aortic aneurysms and dissections', 'Rare disease with thoracic aortic aneurysm and aortic dissection', 'Moyamoya disease'], ['Sensorineural hearing impairment', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm'], ['Hereditary spherocytosis', 'Autosomal dominant distal renal tubular acidosis', 'Distal renal tubular acidosis with anemia', 'Hereditary cryohydrocytosis with normal stomatin', 'Distal renal tubular acidosis'], ['Glucose-galactose malabsorption', 'type II diabetes mellitus', 'diabetes mellitus', 'Glucose intolerance', 'type I diabetes mellitus'], ['Renal glucosuria', 'type II diabetes mellitus', 'diabetes mellitus', 'heart failure', 'type I diabetes mellitus'], ['adolescent idiopathic scoliosis', "Crohn's disease", 'attention deficit hyperactivity disorder', 'Chest pain', 'lung adenocarcinoma'], ['Familial thyroid dyshormonogenesis', 'Congenital hypothyroidism', 'goiter', 'Neonatal diabetes - congenital hypothyroidism - congenital glaucoma - hepatic fibrosis - polycystic kidneys', 'Pendred syndrome'], ['1,5 anhydroglucitol measurement', 'Berardinelli-Seip congenital lipodystrophy', 'Familial partial lipodystrophy associated with PLIN1 mutations', 'Glycogen storage disease due to liver glycogen phosphorylase deficiency', 'Glycogen storage disease due to liver phosphorylase kinase deficiency'], ['alcohol consumption measurement', 'appendicular lean mass', 'alcohol use disorder measurement', 'lean body mass', 'systolic blood pressure'], ['attention deficit hyperactivity disorder', 'major depressive disorder', 'unipolar depression', 'depressive disorder', 'schizophrenia'], ['Infantile dystonia-parkinsonism', 'nicotine dependence', 'attention deficit hyperactivity disorder', 'unipolar depression', 'depressive disorder'], ['Hereditary hyperekplexia', 'Exaggerated startle response', 'brain disease', 'DNA methylation', 'cutaneous melanoma'], ['cutaneous melanoma', 'inflammatory bowel disease', 'ulcerative colitis', 'sitting height measurement', 'emphysema'], ['X-linked creatine transporter deficiency', 'Intellectual disability', 'mental retardation', 'genetic disorder', 'cerebral creatine deficiency syndrome'], ['Glycine encephalopathy', 'Atypical glycine encephalopathy', 'Neonatal glycine encephalopathy', 'Infantile glycine encephalopathy', 'schizophrenia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'platelet count', 'hemoglobin measurement'], ['basophil count', 'basophil percentage of leukocytes', 'erythrocyte count', 'platelet count', 'allergic rhinitis'], ['protein measurement', 'electrocardiography', 'QT interval', 'heel bone mineral density', 'PR interval'], ['red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'platelet count', 'hemoglobin measurement', 'hematocrit'], ['Christianson syndrome', 'mental retardation', 'Intellectual disability', 'microcephaly (disease)', 'genetic disorder'], ['viral disease', 'hepatitis D virus infection', 'cutaneous melanoma', 'hemoglobin measurement', 'appendicular lean mass'], ['primary ovarian insufficiency', 'Chronic constipation', 'cholelithiasis', 'biliary tract disease', 'gallbladder disease'], ['hematocrit', 'hemoglobin measurement', 'mean corpuscular volume', "Alzheimer's disease", 'neoplasm'], ['Microcytic anemia with liver iron overload', 'Hypochromic microcytic anemia', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'Blackfan-Diamond anemia'], ['hearing loss', 'red blood cell distribution width', 'sex hormone-binding globulin measurement', 'body fat distribution', 'lymphocyte count'], ['Gitelman syndrome', 'hypertension', 'chronic kidney disease', 'congestive heart failure', 'nephrotic syndrome'], ['Pentosuria', 'Methylmalonic aciduria due to transcobalamin receptor defect', 'Iminoglycinuria', 'Cystathioninuria', 'Seizures - intellectual disability due to hydroxylysinuria'], ['Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'Seizure', 'Amelo-cerebro-hypohidrotic syndrome', 'Pyridoxine-dependent epilepsy'], ['mean corpuscular hemoglobin concentration', 'reticulocyte count', 'reticulocyte measurement', 'bladder carcinoma', 'hematocrit'], ['refractive error measurement', 'breast carcinoma', 'erythrocyte count', 'hematocrit', 'FEV/FEC ratio'], ['Exercise-induced hyperinsulinism', 'Metabolic myopathy due to lactate transporter defect', 'Inborn errors of metabolism', 'refractive error measurement', 'diastolic blood pressure'], ['body mass index', 'glioblastoma multiforme', 'neoplasm', 'cancer', 'inflammatory bowel disease'], ['body height', 'self reported educational attainment', 'diet measurement', 'mean corpuscular volume', 'mathematical ability'], ['age-related macular degeneration', 'breast adenocarcinoma', 'atrophic macular degeneration', 'wet macular degeneration', 'glioblastoma multiforme'], ['Salla disease', 'Free sialic acid storage disease, infantile form', 'Free sialic acid storage disease', 'Inborn errors of metabolism', 'Intermediate severe Salla disease'], ['FEV/FEC ratio', 'hypertension', 'appendicular lean mass', 'protein measurement', 'Young adult-onset Parkinsonism'], ['Bilateral striopallidodentate calcinosis', 'dystonic disorder', 'basal ganglia cerebrovascular disease', 'genetic disorder', 'sex hormone-binding globulin measurement'], ['glomerular filtration rate', 'creatinine measurement', 'prostate carcinoma', 'blood metabolite measurement', 'chronic kidney disease'], ['coronary artery disease', 'prostate carcinoma', 'type II diabetes mellitus', 'lean body mass', 'body mass index'], ['Systemic primary carnitine deficiency', 'Propionic acidemia', 'genetic disorder', 'mean platelet volume', 'leukocyte count'], ['gout', 'epilepsy', 'influenza', 'urate measurement', 'urinary tract infection'], ['systolic blood pressure', 'smoking status measurement', 'glomerular filtration rate', 'pulse pressure measurement', 'diastolic blood pressure'], ['gout', 'influenza', 'epilepsy', 'urinary tract infection', 'obesity'], ['Hereditary renal hypouricemia', 'gout', 'hyperuricemia', 'acquired metabolic disease', 'urate measurement'], ['D,L-2-hydroxyglutaric aciduria', 'Presynaptic congenital myasthenic syndromes', 'Congenital myasthenic syndromes', 'protein measurement', 'amino acid measurement'], ['lung adenocarcinoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'actinic keratosis', 'body height'], ['Cardiomyopathy - hypotonia - lactic acidosis', 'ovarian carcinoma', 'neoplasm', 'glioma', 'systemic lupus erythematosus'], ['mitochondrial DNA depletion syndrome 19', 'Isolated NADH-CoQ reductase deficiency', 'cutaneous melanoma', 'squamous cell lung carcinoma', 'endometrial cancer'], ['Carnitine-acylcarnitine translocase deficiency', 'atrial fibrillation', 'heel bone mineral density', 'alcohol consumption measurement', 'eosinophil count'], ['mitochondrial DNA depletion syndrome 18', 'vital capacity', 'protein measurement', 'mean corpuscular volume', 'forced expiratory volume'], ['mean corpuscular hemoglobin concentration', 'red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['Congenital chloride diarrhea', 'Polyhydramnios', 'hydrops fetalis (disease)', 'intestinal obstruction', 'Secretory diarrhea'], ['Pendred syndrome', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness 4', 'goiter', 'hearing loss'], ['Ichthyosis prematurity syndrome', 'genetic disorder', 'autosomal recessive disease', 'cutaneous melanoma', 'body mass index'], ['stroke', 'coronary artery disease', 'diabetes mellitus', 'ischemia reperfusion injury', 'bipolar disorder'], ['type II diabetes mellitus', 'diabetes mellitus', 'fasting blood glucose measurement', 'HbA1c measurement', 'insulin measurement'], ['Congenital cataract-hearing loss-severe developmental delay syndrome', 'Autosomal dominant spastic paraplegia type 42', 'Hereditary spastic paraplegia', 'neurodegenerative disease', 'cataract'], ['Pulmonary alveolar microlithiasis', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['SLC35A1-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type II', 'Seizure', 'Macrothrombocytopenia'], ['SLC35A2-CDG', 'Congenital disorder of glycosylation', 'Intellectual disability', 'epilepsy', 'X-linked complex neurodevelopmental disorder'], ['Primary bone dysplasia with multiple joint dislocations', 'glioblastoma multiforme', 'cutaneous melanoma', 'heel bone mineral density', 'myeloid white cell count'], ['eosinophil count', 'birth weight', 'body height', 'protein measurement', 'lean body mass'], ['Abnormality of neuronal migration', 'Leishmaniasis', 'head and neck squamous cell carcinoma', 'neoplasm', 'Miyoshi myopathy'], ['estrogen-receptor positive breast cancer', 'response to aromatase inhibitor', 'neuroimaging measurement', 'brain volume measurement', 'brain measurement'], ['Acrodermatitis enteropathica', 'genetic disorder', 'pancreatic carcinoma', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm'], ['Myopia', 'acute myeloid leukemia', "Barrett's esophagus", 'FTH1-related iron overload', 'Pentosuria'], ['esophageal carcinoma', 'survival time', 'squamous cell carcinoma', 'breast cancer', 'neoplasm'], ['Oculocutaneous albinism type 4', 'Oculocutaneous albinism', 'hair color', 'skin pigmentation', 'cutaneous melanoma'], ['platelet count', 'urinary albumin to creatinine ratio', 'serum alanine aminotransferase measurement', 'aspartate aminotransferase measurement', 'urate measurement'], ['maternal riboflavin deficiency', 'Transient neonatal multiple acyl-CoA dehydrogenase deficiency', 'body height', 'Low levels of vitamin B2', 'Keratoconus'], ['Riboflavin transporter deficiency', 'genetic disorder', 'neoplasm', 'melanoma', 'gastric cancer'], ['Progressive supranuclear palsy', 'serum metabolite measurement', 'lean body mass', 'cutaneous melanoma', 'fat body mass'], ['Rotor syndrome', 'blood metabolite measurement', 'chronic kidney disease', 'urinary metabolite measurement', 'serum metabolite measurement'], ['Rotor syndrome', 'bilirubin measurement', 'Disorder of bilirubin metabolism and excretion', 'ceramide measurement', 'testosterone measurement'], ['visual perception measurement', 'cutaneous melanoma', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'parasitic infection'], ['schizophrenia', 'body height', "Crohn's disease", 'BMI-adjusted waist circumference', 'BMI-adjusted waist-hip ratio'], ['inflammatory bowel disease', 'neoplasm', 'cancer', 'Ewing sarcoma', 'hepatocellular carcinoma'], ['leukocyte count', 'Autosomal agammaglobulinemia', 'lung adenocarcinoma', 'platelet count', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency'], ['Congenital hydronephrosis', 'type II diabetes mellitus', 'body weight', 'protein measurement', 'smoking status measurement'], ['ulcerative colitis', 'lean body mass', 'age at menarche', 'body height', 'lymphocyte count'], ['coronary artery disease', 'pulse pressure measurement', 'alcohol drinking', 'obesity', 'diabetes mellitus'], ['acute myeloid leukemia', 'cardiovascular disease', 'neoplasm', 'infect', 'infection'], ['Mal de Meleda', 'Hereditary palmoplantar keratoderma, Gamborg-Nielsen type', 'palmoplantar keratosis', 'Diffuse palmoplantar keratoderma', 'DNA methylation'], ['hemoglobin measurement', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'high density lipoprotein cholesterol measurement', 'protein measurement'], ['colorectal adenocarcinoma', 'cancer', 'genetic disorder', 'gastric carcinoma', 'colon adenocarcinoma'], ['Generalized juvenile polyposis/juvenile polyposis coli', 'Myhre syndrome', 'Juvenile polyposis syndrome', 'juvenile polyposis/hereditary hemorrhagic telangiectasia syndrome', 'Hereditary hemorrhagic telangiectasia'], ['colorectal cancer', 'eosinophil count', 'glomerular filtration rate', 'hemoglobin measurement', 'erythrocyte count'], ['pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'Idiopathic pulmonary arterial hypertension', 'Idiopathic and/or familial pulmonary arterial hypertension', 'heel bone mineral density'], ['Abnormality of brain morphology', 'prostate adenocarcinoma', 'Coffin-Siris syndrome', 'coronary artery disease', 'genetic disorder'], ['intellectual disability - sparse hair - brachydactyly', 'Intellectual disability', 'mental retardation', 'schizophrenia', 'genetic disorder'], ['Familial rhabdoid tumor', 'Coffin-Siris syndrome', 'lung adenocarcinoma', 'medulloblastoma', 'Ovarian Small Cell Carcinoma'], ['Familial rhabdoid tumor', 'Neurofibromatosis type 3', 'Coffin-Siris syndrome', 'atypical teratoid rhabdoid tumor', 'schwannoma'], ['Cornelia de Lange syndrome', 'Early infantile epileptic encephalopathy', 'Epileptic encephalopathy', 'genetic disorder', 'Wiedemann-Steiner syndrome'], ['melanoma', 'neoplasm', 'cancer', 'drug use measurement', 'lung cancer'], ['endometrial cancer', 'heel bone mineral density', 'coffee consumption measurement', 'tea consumption measurement', 'gut microbiome measurement'], ['Proximal spinal muscular atrophy', 'Proximal spinal muscular atrophy type 3', 'Proximal spinal muscular atrophy type 2', 'Proximal spinal muscular atrophy type 1', 'Proximal spinal muscular atrophy type 4'], ['Proximal spinal muscular atrophy type 3', 'Proximal spinal muscular atrophy type 2', 'Proximal spinal muscular atrophy type 1', 'Proximal spinal muscular atrophy', 'spinal muscular atrophy'], ['basal cell carcinoma', 'Curry-Jones syndrome', 'medulloblastoma', 'acute myeloid leukemia', 'neoplasm'], ['Microphthalmia with limb anomalies', 'polydactyly (disease)', 'lean body mass', 'red blood cell distribution width', 'mean corpuscular volume'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'mean corpuscular hemoglobin', 'red blood cell distribution width'], ['Niemann-Pick disease type A', 'Niemann-Pick disease type B', 'Niemann-Pick disease type E', 'Niemann-Pick disease', 'Neuronal ceroid lipofuscinosis'], ['thyroid gland adenocarcinoma', 'body height', 'forced expiratory volume', 'sitting height measurement', 'lean body mass'], ['body height', 'erythrocyte count', 'self reported educational attainment', 'intelligence', 'schizophrenia'], ['Ewing sarcoma', 'infect', 'infection', 'neoplasm', 'adrenal cortex carcinoma'], ['X-linked intellectual disability, Snyder type', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Smith-Magenis syndrome'], ['monocyte count', 'coronary artery disease', 'coronary atherosclerosis', 'prostate carcinoma', 'body mass index'], ['mean corpuscular volume', 'ulcerative colitis', 'inflammatory bowel disease', 'mean corpuscular hemoglobin', 'eosinophil percentage of leukocytes'], ['lymphocyte count', 'sex hormone-binding globulin measurement', 'smoking status measurement', 'smoking initiation', 'monocyte count'], ['electrocardiography', 'PR interval', 'acute myeloid leukemia', 'systemic lupus erythematosus', 'protein measurement'], ['body height', 'sitting height measurement', 'psoriasis', 'neoplasm', 'hepatocellular carcinoma'], ['hemoglobin measurement', 'esophageal adenocarcinoma', 'esophageal squamous cell carcinoma', 'oral cavity carcinoma', 'erythrocyte count'], ['Congenital myasthenic syndromes', 'migraine disorder', 'Facial wrinkling', 'Spasticity', 'Torticollis'], ['Hereditary late-onset Parkinson disease', 'Lewy body dementia', 'Young adult-onset Parkinsonism', "Parkinson's disease", 'AL amyloidosis'], ['Ehlers-Danlos syndrome, progeroid type', "Alzheimer's disease", 'Lewy body dementia', 'cancer', 'pervasive developmental disorder - not otherwise specified'], ['red blood cell distribution width', 'chronotype measurement', 'mean platelet volume', 'platelet count', 'atrial fibrillation'], ['Hypotrichosis simplex', 'red blood cell distribution width', 'Non-syndromic male infertility due to sperm motility disorder', 'Partial chromosome Y deletion', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa'], ['FEV/FEC ratio', 'mean platelet volume', 'peak expiratory flow', 'pulmonary function measurement', 'Abnormality of refraction'], ['Prader-Willi syndrome', 'periodontal measurement', 'autism spectrum disorder', 'vasculitis', 'Proximal spinal muscular atrophy'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'lip cancer', 'C-reactive protein measurement'], ['diabetes mellitus', 'testosterone measurement', 'chronotype measurement', 'cutaneous melanoma', 'schizophrenia'], ['lean body mass', 'body height', 'appendicular lean mass', 'forced expiratory volume', 'fat body mass'], ['amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'sporadic amyotrophic lateral sclerosis', 'spastic tetraplegia and axial hypotonia, progressive', 'motor neuron disease'], ['diabetic nephropathy', 'diabetic retinopathy', 'gastric carcinoma', 'acute myeloid leukemia', 'neuroblastoma'], ['sorbitol dehydrogenase deficiency with peripheral neuropathy', 'genetic disorder', 'Red cell distribution width', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement'], ["Alzheimer's disease", 'Early-onset autosomal dominant Alzheimer disease', 'refractive error measurement', 'alcohol consumption measurement', 'family history of Alzheimer’s disease'], ['appendicular lean mass', 'glomerular filtration rate', 'mathematical ability', 'low density lipoprotein cholesterol measurement', 'coronary artery disease'], ['Noonan syndrome', 'Hereditary gingival fibromatosis', 'rasopathy', 'cancer', 'bone development disease'], ['Sclerosteosis', 'Hyperostosis corticalis generalisata', 'Craniodiaphyseal dysplasia', 'postmenopausal osteoporosis', 'osteoporosis'], ['Anophthalmia/microphthalmia - esophageal atresia', 'genetic disorder', 'refractive error measurement', 'Septo-optic dysplasia', 'Jackson-Weiss syndrome'], ['Campomelic dysplasia', 'Pierre-Robin sequence', '46,XX testicular disorder of sex development', '46,XY complete gonadal dysgenesis', 'colorectal adenocarcinoma'], ['Familial vesicoureteral reflux', 'Heritable pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'vesicoureteral reflux', 'endometrial cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['birth weight', 'hair color', 'hepatocellular carcinoma', 'glioma', 'neoplasm'], ['appendicular lean mass', 'body height', 'red blood cell distribution width', 'multiple myeloma', 'lean body mass'], ['hair color', 'skin pigmentation', 'hair colour measurement', 'Amelogenesis imperfecta', 'body fat distribution'], ['neoplasm', 'cervical cancer', 'hepatocellular carcinoma', 'Myopia', 'cancer'], ['neoplasm', 'cervical cancer', 'hepatocellular carcinoma', 'Myopia', 'cancer'], ['Non-syndromic male infertility due to sperm motility disorder', 'BMI-adjusted waist circumference', 'body height', 'physical activity measurement', 'smoking behavior'], ['lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme'], ['SPARC-like protein 1 measurement', 'blood protein measurement', 'protein measurement', "Alzheimer's disease", 'neoplasm'], ['systemic lupus erythematosus', 'eosinophil count', 'neoplasm', 'non-small cell lung carcinoma', 'cancer'], ['epilepsy, hearing loss, and mental retardation syndrome', 'BMI-adjusted waist-hip ratio', 'appendicular lean mass', 'type II diabetes mellitus', 'body height'], ['monocyte count', 'breast carcinoma', 'red blood cell distribution width', 'total cholesterol measurement', 'neoplasm'], ['mean corpuscular volume', 'erythrocyte count', 'schizophrenia', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration'], ['esophageal carcinoma', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa', 'spermatogenic failure 43'], ['ovarian cancer', 'neutrophil count', 'gastric adenocarcinoma', 'endometrial cancer', 'reticulocyte count'], ['body height', 'heel bone mineral density', 'sitting height measurement', 'psoriasis', 'longitudinal alcohol consumption measurement'], ['Hereditary chronic pancreatitis', 'chronic pancreatitis', 'genetic disorder', 'alcoholic pancreatitis', 'blood protein measurement'], ['spermatogenic failure 29', 'azoospermia', 'reticulocyte measurement', 'reticulocyte count', 'blood protein measurement'], ['Netherton syndrome', 'Increased circulating IgE level', 'Epidermolytic ichthyosis', 'Failure to thrive', 'blood protein measurement'], ['glioblastoma multiforme', 'blood protein measurement', 'hepatocellular carcinoma', 'neoplasm', 'squamous cell carcinoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'hepatocellular carcinoma'], ['type II diabetes mellitus', 'electrocardiography', 'blood protein measurement', 'body height', 'inflammatory bowel disease'], ['bone density', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'breast adenocarcinoma', 'hepatocellular carcinoma'], ['Dopa-responsive dystonia due to sepiapterin reductase deficiency', 'Intellectual disability', 'Dystonia', 'mental retardation', 'genetic disorder'], ['ovarian cancer', 'clear cell renal carcinoma', 'neoplasm', 'gonorrhea', 'psoriasis'], ['Hereditary elliptocytosis', 'Hereditary spherocytosis', 'elliptocytosis 2', 'Pyropoikilocytosis', 'pyropoikilocytosis, hereditary'], ['Hereditary spherocytosis', 'Hereditary elliptocytosis', 'hemolytic anemia', 'Pyropoikilocytosis', 'red blood cell distribution width'], ['mean platelet volume', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement', 'neutrophil percentage of leukocytes', 'hematocrit'], ['neoplasm', 'breast cancer', 'obesity', 'nonpapillary renal cell carcinoma', 'hepatocellular carcinoma'], ['platelet count', 'electrocardiography', 'left ventricular structural measurement', 'left ventricular systolic function measurement', 'left ventricular ejection fraction measurement'], ['Kallmann syndrome', 'white matter microstructure measurement', 'chronic obstructive pulmonary disease', 'glioma', 'classic Hodgkin lymphoma'], ['neoplasm', 'cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'lymphoid neoplasm'], ['Ichthyosis follicularis - alopecia - photophobia', 'BRESEK syndrome', 'Hereditary mucoepithelial dysplasia', 'sex hormone-binding globulin measurement', 'body height'], ['birth weight', 'serum alanine aminotransferase measurement', 'breast adenocarcinoma', 'lean body mass', 'Posterior cortical atrophy'], ['cancer', 'gonorrhea', 'prostate cancer', 'calf circumference measurement', 'cholangiocarcinoma'], ['breast adenocarcinoma', 'eosinophil percentage of leukocytes', 'neoplasm', 'eosinophil count', 'glioblastoma multiforme'], ['genetic disorder', 'appendicular lean mass', 'cancer', 'Transient neonatal diabetes mellitus', 'Hyperinsulinism due to glucokinase deficiency'], ['refractive error measurement', 'lean body mass', 'heel bone mineral density', 'appendicular lean mass', 'mean corpuscular volume'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'lymphoid neoplasm', 'leukemia'], ['mean corpuscular volume', 'monocyte count', 'lymphocyte count', 'red blood cell distribution width', 'leukocyte count'], ['vein disease', 'lymphatic system disease', 'varicose disease', 'Varicose veins', 'Hernia'], ['chronic lymphocytic leukemia', 'cancer', 'non-small cell lung carcinoma', 'chronic myelogenous leukemia', 'colorectal cancer'], ['bile duct carcinoma', 'Hepatobiliary Neoplasm', 'bile duct adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'hepatocellular carcinoma'], ['erythrocyte count', 'platelet crit', 'mean corpuscular volume', 'mean reticulocyte volume', 'mean corpuscular hemoglobin'], ['lung adenocarcinoma', 'mathematical ability', 'glioblastoma multiforme', 'low grade glioma', 'oral cavity carcinoma'], ['hepatocellular carcinoma', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'lung cancer', 'neoplasm'], ['46,XY complete gonadal dysgenesis', '46,XX testicular disorder of sex development', '46,XX ovotesticular disorder of sex development', '46,XY partial gonadal dysgenesis', '46,xx sex reversal 5'], ['platelet count', 'grip strength measurement', 'cancer', 'platelet crit', 'lung cancer'], ['HIV infection', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['age at menarche', 'glioblastoma multiforme', 'breast adenocarcinoma', 'multiple sclerosis', 'temporal arteritis'], ['neoplasm', 'acromegaly', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['neoplasm', 'acromegaly', 'neuroendocrine neoplasm', 'obesity', 'carcinoid syndrome'], ['acromegaly', 'neoplasm', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['acromegaly', 'neoplasm', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['synovial sarcoma', 'rhabdomyosarcoma', 'undifferentiated pleomorphic sarcoma', 'malignant peripheral nerve sheath tumor', 'Extraskeletal Ewing Sarcoma/Peripheral Primitive Neuroectodermal Tumor'], ['neoplasm', 'synovial sarcoma', 'nan', 'nan', 'nan'], ['COVID-19', 'mean platelet volume', 'alkaline phosphatase measurement', 'low density lipoprotein cholesterol measurement', 'severe acute respiratory syndrome'], ['refractive error measurement', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'ovarian cancer', 'neoplasm'], ['COVID-19', 'severe acute respiratory syndrome', 'lung adenocarcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma'], ['vitamin D measurement', 'low density lipoprotein cholesterol measurement', 'C-reactive protein measurement', 'Nephronophthisis', 'basophil percentage of leukocytes'], ['Ichthyosis-hypotrichosis syndrome', 'HIV-1 infection', 'viral load', 'Autosomal recessive congenital ichthyosis', 'glioblastoma multiforme'], ['hemoglobin measurement', 'unipolar depression', 'response to placebo', 'red blood cell density measurement', 'adolescent idiopathic scoliosis'], ['intellectual disability, autosomal dominant 47', 'Autosomal dominant non-syndromic intellectual disability', 'mental retardation', 'stag1-related disorder', 'genetic disorder'], ['Mendelian susceptibility to mycobacterial diseases due to partial STAT1 deficiency', 'Chronic mucocutaneous candidosis', 'Autoimmune enteropathy and endocrinopathy-susceptibility to chronic infections syndrome', 'Susceptibility to viral and mycobacterial infections', 'Combined T and B cell immunodeficiency'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Congenital intrauterine infection-like syndrome', 'measles', 'lung adenocarcinoma', 'thyroid gland adenocarcinoma'], ['Autosomal dominant hyper-IgE syndrome', 'autoimmune disease', 'Hyper-IgE syndrome', 'cancer', 'Permanent neonatal diabetes mellitus'], ['cancer', 'acute myeloid leukemia', 'lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer'], ['Laron syndrome with immunodeficiency', 'cancer', 'acute myeloid leukemia', 'Combined T and B cell immunodeficiency', 'growth hormone insensitivity syndrome with immune dysregulation 2, autosomal dominant'], ['solitary fibrous tumor', 'diffuse large B-cell lymphoma', 'non-Hodgkins lymphoma', 'spindle cell tumor', 'hemangioblastoma'], ['arterial stiffness measurement', 'coffee consumption measurement', 'coffee consumption', 'diastolic blood pressure', 'type II diabetes mellitus'], ['type II diabetes mellitus', 'pulse pressure measurement', 'arterial stiffness measurement', 'coffee consumption measurement', 'coffee consumption'], ['osteoarthritis, hip', 'risk-taking behaviour', 'erythrocyte count', 'mean corpuscular volume', 'vital capacity'], ['Tubular aggregate myopathy', 'Combined immunodeficiency due to STIM1 deficiency', 'Stormorken-Sjaastad-Langslet syndrome', 'Combined immunodeficiency due to CRAC channel dysfunction', 'Combined T and B cell immunodeficiency'], ['STING-associated vasculopathy with onset in infancy', 'familial chilblain lupus', 'esophageal squamous cell carcinoma', 'anti-merkel cell virus antibody measurement', 'merkel cell virus seropositivity'], ['Combined immunodeficiency due to STK4 deficiency', 'Combined T and B cell immunodeficiency', 'self reported educational attainment', 'mean platelet volume', 'bipolar disorder'], ['cutaneous melanoma', 'cutaneous squamous cell carcinoma', 'blood protein measurement', 'gp41 C34 peptide, HIV measurement', 'psoriatic arthritis'], ["Parkinson's disease", 'lean body mass', 'body mass index', 'type I diabetes mellitus', 'prostate carcinoma'], ['BMI-adjusted waist-hip ratio', 'hemoglobin measurement', 'hematocrit', 'neoplasm', 'breast cancer'], ['protein measurement', 'ependymoma', 'astrocytoma', 'atypical teratoid rhabdoid tumor', 'colorectal cancer'], ['body mass index', 'peak expiratory flow', 'insomnia measurement', 'insomnia', 'type II diabetes mellitus'], ['appendicular lean mass', 'body fat percentage', 'visceral adipose tissue measurement', 'sleep duration', 'protein measurement'], ['Recessive X-linked ichthyosis', 'Syndromic X-linked ichthyosis', 'x-linked ichthyosis with steryl-sulfatase deficiency', 'Autosomal recessive congenital ichthyosis', 'genetic disorder'], ['STT3A-CDG', 'COVID-19', 'Intellectual disability', 'developmental disability', 'mean corpuscular hemoglobin concentration'], ['Cystic fibrosis', 'botulism', 'triglyceride measurement', 'clear cell renal carcinoma', 'cutaneous melanoma'], ['von Willebrand factor measurement', 'tissue plasminogen activator measurement', 'infect', 'infection', 'hemolytic-uremic syndrome'], ['asthma', 'white matter microstructure measurement', 'protein measurement', 'bone quantitative ultrasound measurement', 'body height'], ['breast adenocarcinoma', 'ovarian carcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['non-small cell lung carcinoma', 'neoplasm', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm'], ['breast carcinoma', 'PR interval', 'neoplasm', 'breast cancer', 'non-small cell lung carcinoma'], ['Mitochondrial DNA depletion syndrome, encephalomyopathic form with methylmalonic aciduria', 'Mitochondrial DNA depletion syndrome, encephalomyopathic form', 'Mitochondrial DNA depletion syndrome', 'Leigh syndrome', 'mean corpuscular hemoglobin'], ['childhood onset asthma', 'hepatocellular carcinoma', 'eosinophil count', 'celiac disease', 'red blood cell distribution width'], ['risk-taking behaviour', 'obese body mass index status', 'C-reactive protein measurement', 'breast cancer', 'lip and oral cavity carcinoma'], ['sex hormone-binding globulin measurement', 'heel bone mineral density', 'breast cancer', 'Uterine leiomyoma', 'uterine fibroid'], ['vitamin D measurement', 'blood metabolite measurement', 'cholelithiasis', 'triglyceride measurement', 'sleep duration'], ['Multiple sulfatase deficiency', 'heel bone mineral density', 'self reported educational attainment', 'high density lipoprotein cholesterol measurement', 'genetic disorder'], ['platelet component distribution width', 'rheumatoid arthritis', 'acute myeloid leukemia', 'Spina bifida - hypospadias', 'Craniorachischisis'], ['Imagawa-Matsumoto syndrome', 'prostate adenocarcinoma', 'Weaver syndrome', 'breast carcinoma', 'viral disease'], ['epilepsy', 'Seizure', 'Bilateral tonic-clonic seizure', 'botulism', "Parkinson's disease"], ['neurodevelopmental disorder with ataxia, hypotonia, and microcephaly', 'Intellectual disability', 'Lower limb spasticity', 'Microcephaly', 'mental retardation'], ['sprain', 'muscle strain', 'dislocation', 'ankle brachial index', 'female infertility'], ['habitual abortion', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation', 'head and neck squamous cell carcinoma', 'male infertility', 'acute myeloid leukemia'], ['cutaneous Leishmaniasis', 'autoimmune thrombocytopenic purpura', 'arthritis', 'colitis', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['Atypical juvenile parkinsonism', 'Early infantile epileptic encephalopathy', 'genetic disorder', 'undetermined early-onset epileptic encephalopathy', 'Young adult-onset Parkinsonism'], ['intraocular pressure measurement', 'platelet component distribution width', 'age-related hearing impairment', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['visual perception measurement', 'FEV/FEC ratio', 'Ehlers-Danlos syndrome, progeroid type', 'melanoma', 'neoplasm'], ['Non-steroidal anti-inflammatory and antirheumatic product use measurement', 'hematocrit', 'red blood cell density measurement', 'hemoglobin measurement', 'myeloid white cell count'], ['chronotype measurement', 'mathematical ability', 'body mass index', 'drug use measurement', 'astrocytoma'], ['X-linked non-syndromic intellectual disability', 'mental retardation', 'genetic disorder', 'neoplasm', 'breast cancer'], ['infantile hypotonia-oculomotor anomalies-hyperkinetic movements-developmental delay syndrome', 'botulism', 'Intellectual disability', 'cognitive function measurement', 'Autosomal recessive non-syndromic intellectual disability'], ['Congenital myasthenic syndromes', 'Presynaptic congenital myasthenic syndromes', 'botulism', 'acute myeloid leukemia', 'Muscle weakness'], ['Autosomal dominant secondary polycythemia', 'blood pressure', 'Primary familial polycythemia', 'Wiskott-Aldrich syndrome', 'diabetes mellitus, permanent neonatal 4'], ['glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma', 'endometrial cancer', 'esophageal cancer'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'neuroblastoma'], ['pancreatic adenocarcinoma', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'breast cancer'], ['risk-taking behaviour', 'feeling nervous measurement', 'anxiety', 'emotional symptom measurement', 'social interaction measurement'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Isolated congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'Delayed puberty', 'hypogonadotropic hypogonadism'], ['osteoarthritis, knee', 'mean reticulocyte volume', 'gait measurement', 'body height', 'joint disease'], ['Chemotherapy-induced nausea and vomiting', 'post operative nausea and vomiting', 'ovarian cancer', 'cancer', 'Vomiting'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'Isolated congenital hypogonadotropic hypogonadism', 'age at menarche', 'testosterone measurement'], ['Gelatinous drop-like corneal dystrophy', 'triple-negative breast cancer', 'breast cancer', 'metastatic malignant neoplasm', 'prostate cancer'], ['breast adenocarcinoma', 'lymphocyte count', 'monocyte count', 'hematocrit', 'platelet count'], ['Microcephaly', 'Partial agenesis of the corpus callosum', 'Severe global developmental delay', 'lymphocyte count', 'platelet count'], ['blood protein measurement', 'red blood cell distribution width', 'cerebrospinal fluid biomarker measurement', 'triglyceride measurement', 'proprotein convertase subtilisin/kexin type 7 measurement'], ['T-cell acute lymphoblastic leukemia', 'Cutaneous T-cell lymphoma', 'B-cell acute lymphoblastic leukemia', 'unspecified peripheral T-cell lymphoma', 'mean corpuscular volume'], ['Immunodeficiency by defective expression of HLA class 1', 'Immunodeficiency by defective expression of HLA class 2', 'inflammatory bowel disease', 'neoplasm', 'melanoma'], ['Immunodeficiency by defective expression of HLA class 1', 'Immunodeficiency by defective expression of HLA class 2', 'neoplasm', 'body mass index', 'Reticular dysgenesis'], ['Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Frontotemporal dementia', 'motor neuron disease'], ['sensory perception of bitter taste', 'alcohol consumption measurement', 'diet measurement', 'tea consumption measurement', 'bitter alcoholic beverage consumption measurement'], ['type II diabetes mellitus', 'mean platelet volume', 'mean corpuscular volume', 'erythrocyte count', 'mean corpuscular hemoglobin'], ['neutrophil count', 'myeloid white cell count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'leukocyte count'], ['platelet count', 'platelet crit', 'attention deficit hyperactivity disorder', 'neoplasm', 'hepatocellular carcinoma'], ['Micro syndrome', 'mental retardation', 'developmental disability', 'Hepatitis', 'Seizure'], ['DOORS syndrome', 'Familial infantile myoclonic epilepsy', "Rolandic epilepsy - paroxysmal exercise-induced dystonia - writer's cramp", 'Malignant migrating partial seizures of infancy', 'Progressive myoclonic epilepsy with dystonia'], ['mean platelet volume', 'vital capacity', 'platelet component distribution width', 'electrocardiography', 'sitting height measurement'], ['head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'chronic lymphocytic leukemia'], ['Sanjad-Sakati syndrome', 'Kenny-Caffey syndrome', 'Autosomal recessive Kenny-Caffey syndrome', 'encephalopathy, progressive, with amyotrophy and optic atrophy', 'genetic disorder'], ['Frontotemporal dementia with motor neuron disease', 'Herpetic encephalitis', 'amyotrophic lateral sclerosis', 'Frontotemporal dementia', 'Pick disease'], ['hypothyroidism, congenital, nongoitrous', 'viral disease', 'Central congenital hypothyroidism', 'breast adenocarcinoma', 'eosinophilic pneumonia'], ['mean corpuscular hemoglobin', 'C-reactive protein measurement', 'arterial stiffness measurement', 'neoplasm', 'colorectal carcinoma'], ['22q11.2 deletion syndrome', 'Conotruncal heart malformations', 'Tetralogy of Fallot', 'Shprintzen-Goldberg syndrome', 'Double outlet right ventricle'], ['Coxopodopatellar syndrome', 'pulmonary arterial hypertension', 'Autosomal recessive amelia', 'Heritable pulmonary arterial hypertension', 'Idiopathic pulmonary arterial hypertension'], ['Sacral agenesis-abnormal ossification of the vertebral bodies-persistent notochordal canal syndrome', 'Chordoma', 'Neural tube closure defect', 'Isolated spina bifida', 'spina bifida'], ['neoplasm', 'hepatocellular carcinoma', 'cancer', 'cervical cancer', 'esophageal squamous cell carcinoma'], ['Autosomal agammaglobulinemia', 'Isolated agammaglobulinemia', 'Agammaglobulinemia', 'primary immunodeficiency disease', 'acute lymphoblastic leukemia'], ['Pitt-Hopkins syndrome', 'Fuchs endothelial corneal dystrophy', 'schizophrenia', 'genetic disorder', 'risk-taking behaviour'], ['colorectal adenocarcinoma', 'type II diabetes mellitus', 'diabetes mellitus', 'breast carcinoma', 'body mass index'], ['birth weight', 'infect', 'infection', 'Autosomal recessive spondylocostal dysostosis', 'Poland syndrome'], ['type II diabetes mellitus', 'basal cell carcinoma', 'chronic hepatitis B virus infection', 'toxic epidermal necrolysis', 'Stevens-Johnson syndrome'], ['Developmental delay with variable intellectual impairment and behavioural abnormalities', 'Intellectual disability', 'developmental delay with variable intellectual impairment and behavioral abnormalities', 'Neurodevelopmental abnormality', 'genetic disorder'], ['pulse pressure measurement', 'systolic blood pressure', 'platelet count', 'eosinophil count', 'cardiovascular disease'], ['chronic lymphocytic leukemia', 'lymphoid neoplasm', 'mosaic loss of chromosome Y measurement', 'platelet crit', 'platelet count'], ['Protein R deficiency', 'vitamin B12 measurement', 'pernicious anemia', 'head and neck squamous cell carcinoma', 'hypertrophic cardiomyopathy'], ['Treacher-Collins syndrome', 'Crouzon disease', 'Microcephaly', 'lumbar disc degeneration', 'Isolated anencephaly/exencephaly'], ['Hypertryptophanemia', 'blood metabolite measurement', 'hypospadias', 'hepatocellular carcinoma', 'myeloid white cell count'], ['Mucocutaneous venous malformations', 'Congenital glaucoma', 'neoplasm', 'metastatic colorectal cancer', 'thyroid cancer'], ['TELO2-related intellectual disability-neurodevelopmental disorder', 'genetic disorder', 'rheumatoid arthritis', 'Microcephaly', 'Niemann-Pick disease type C'], ['Agammaglobulinemia', 'glioblastoma multiforme', 'neoplasm', 'Acute hepatic failure', 'papillary thyroid carcinoma'], ['cancer', 'age at menopause', 'body height', 'multiple sclerosis', 'sphingolipid measurement'], ['vital capacity', 'lymphocyte percentage of leukocytes', 'forced expiratory volume', 'neutrophil count', 'leukocyte count'], ['eosinophil count', 'chronic obstructive pulmonary disease', 'infect', 'infection', 'Hematuria'], ['head and neck squamous cell carcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'esophageal cancer', 'squamous cell lung carcinoma'], ['Dyskeratosis congenita', 'pulmonary fibrosis and/or bone marrow failure, telomere-related, 1', 'idiopathic pulmonary fibrosis', 'Aplastic anemia', 'idiopathic aplastic anemia'], ['intraocular pressure measurement', 'glaucoma', 'body mass index', 'neoplasm', 'chronotype measurement'], ['mean platelet volume', 'self reported educational attainment', 'PR interval', 'lifestyle measurement', 'platelet crit'], ['myelodysplastic syndrome', 'prostate carcinoma', 'acute myeloid leukemia', 'breast carcinoma', 'lung carcinoma'], ['Congenital atransferrinemia', 'iron biomarker measurement', 'red blood cell distribution width', 'total iron binding capacity', 'alcohol drinking'], ['glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'ovarian cancer'], ['high density lipoprotein cholesterol measurement', 'neoplasm', 'hepatocellular carcinoma', 'chronic kidney disease', 'breast carcinoma'], ['Autosomal recessive spastic paraplegia type 57', 'Hereditary motor and sensory neuropathy, Okinawa type', 'Autosomal recessive complex spastic paraplegia', 'Charcot-Marie-Tooth disease type 2', 'anaplastic large cell lymphoma'], ['intracranial hemorrhage', 'postmenopausal atrophic vaginitis', 'lymphocyte count', 'lung adenocarcinoma', 'leukocyte count'], ['neoplasm', 'hepatocellular carcinoma', 'breast cancer', 'colorectal carcinoma', 'urinary bladder cancer'], ['low grade glioma', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'glioblastoma multiforme', 'esophageal squamous cell carcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'Primary immunodeficiency', 'mean corpuscular hemoglobin'], ['Familial thyroid dyshormonogenesis', "Hashimoto's thyroiditis", 'hypothyroidism', 'Neonatal diabetes - congenital hypothyroidism - congenital glaucoma - hepatic fibrosis - polycystic kidneys', 'Congenital hypothyroidism'], ['cancer', 'grip strength measurement', 'osteoarthritis', 'tooth agenesis', 'Oligodontia'], ['Camurati-Engelmann disease', 'inflammatory bowel disease, immunodeficiency, and encephalopathy', 'cancer', 'Cystic fibrosis', 'Autosomal recessive early-onset inflammatory bowel disease'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Loeys-Dietz syndrome', 'Holt-Oram syndrome', 'Abnormality of the cardiovascular system', 'BMI-adjusted waist circumference'], ['Lattice corneal dystrophy type I', 'Granular corneal dystrophy type I', 'Microcystic corneal dystrophy', 'Granular corneal dystrophy type II', 'Reis-Bücklers corneal dystrophy'], ['Loeys-Dietz syndrome', 'Familial thoracic aortic aneurysm and aortic dissection', 'Multiple keratoacanthoma, Ferguson-Smith type', 'cancer', 'Rare disease with thoracic aortic aneurysm and aortic dissection'], ['Loeys-Dietz syndrome', 'Familial thoracic aortic aneurysm and aortic dissection', 'Marfan syndrome', 'Marfan syndrome type 2', 'Lynch syndrome'], ['obesity', 'myeloid white cell count', 'protein measurement', 'platelet count', 'electrocardiography'], ['Uncombable hair syndrome', 'basal cell carcinoma', 'skin cancer', 'non-melanoma skin carcinoma', 'keratinocyte carcinoma'], ['self reported educational attainment', 'mathematical ability', 'eosinophil count', 'adolescent idiopathic scoliosis', 'lean body mass'], ['Autosomal recessive dopa-responsive dystonia', 'tyrosine hydroxylase deficiency', 'Dopa-responsive dystonia', 'Dystonia', 'Intellectual disability'], ['thrombomodulin-related bleeding disorder', 'Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with thrombomodulin anomaly', 'Venous thrombosis', 'thrombomodulin measurement'], ['autism spectrum disorder', 'cortical surface area measurement', 'brain volume measurement', 'neuroimaging measurement', 'cortical thickness'], ['essential thrombocythemia', 'Congenital amegakaryocytic thrombocytopenia', 'Familial thrombocytosis', 'Hereditary isolated aplastic anemia', 'Hereditary thrombocytosis with transverse limb defect'], ['neoplasm', 'Androgen insensitivity syndrome', 'Åland Islands eye disease', 'breast cancer', 'hepatocellular carcinoma'], ['mathematical ability', 'glomerular filtration rate', 'neoplasm', 'hepatocellular carcinoma', 'liver cancer'], ['hair shape measurement', 'balding measurement', 'smoking status measurement', 'risk-taking behaviour', 'skin aging measurement'], ['breast adenocarcinoma', 'low grade glioma', 'ovarian cancer', 'late-onset Alzheimers disease', 'glioblastoma multiforme'], ['head and neck squamous cell carcinoma', 'endometrial cancer', 'cutaneous melanoma', 'Sepsis', 'neoplasm'], ['heel bone mineral density', 'body height', 'gut microbiome measurement', 'blood protein measurement', 'hip bone size'], ["Sorsby's fundus dystrophy", 'Retinal dystrophy', 'metalloproteinase inhibitor 3 measurement', 'macular degeneration', 'Cerebral arteriovenous malformation'], ['red blood cell distribution width', 'eosinophil count', 'triglyceride measurement', 'wellbeing measurement', 'type II diabetes mellitus'], ['body mass index', 'childhood onset asthma', 'body height', 'ovarian carcinoma', 'age at menopause'], ['Primary immunodeficiency', 'bacteremia, susceptibility', 'appendicular lean mass', 'neutrophil percentage of granulocytes', 'cutaneous melanoma'], ['Progressive familial intrahepatic cholestasis', 'Familial hypercholanemia', 'Progressive familial intrahepatic cholestasis type 2', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'genetic disorder'], ['Red cell distribution width', 'mean corpuscular volume', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['neoplasm', 'melanoma', 'cancer', 'glioma', 'Autosomal recessive early-onset inflammatory bowel disease'], ['self reported educational attainment', 'lymphocyte count', 'lymphocyte percentage of leukocytes', 'body mass index', 'balding measurement'], ['age at menopause', 'mean platelet volume', 'vital capacity', 'body height', 'appendicular lean mass'], ['Interauricular communication', 'Genetic cardiac anomaly', 'refractive error measurement', 'Atrial septal defect, ostium primum type', 'Atrial septal defect, ostium secundum type'], ['adolescent idiopathic scoliosis', 'breast adenocarcinoma', 'cutaneous melanoma', 'Proximal amyotrophy', 'Kennedy disease'], ['Primary immunodeficiency', 'childhood onset asthma', 'asthma', 'allergy', 'seasonal allergic rhinitis'], ['Tuberculosis', 'Primary immunodeficiency', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cutaneous melanoma'], ['Herpetic encephalitis', "Hashimoto's thyroiditis", 'Primary immunodeficiency', 'HIV-1 infection', 'drug use measurement'], ['pneumonia', 'influenza', 'COVID-19', 'Primary immunodeficiency', 'toll-like receptor 4:Lymphocyte antigen 96 complex measurement'], ['Primary immunodeficiency', 'monocyte percentage of leukocytes', 'monocyte count', 'legionellosis', 'chronic lymphocytic leukemia'], ['Primary immunodeficiency', 'childhood onset asthma', 'asthma', 'allergy', 'lung adenocarcinoma'], ['rheumatoid arthritis', 'COVID-19', 'actinic keratosis', 'systemic lupus erythematosus', 'basal cell carcinoma'], ['osteoarthritis', 'chronic lymphocytic leukemia', 'ovarian cancer', 'neoplasm', 'cancer'], ['rheumatoid arthritis', 'COVID-19', 'systemic lupus erythematosus', 'obesity', 'IGA glomerulonephritis'], ['glioblastoma multiforme', 'Hydrolethalus', 'Hodgkins lymphoma', 'lactose intolerance adult type', 'Hirschsprung disease'], ['low density lipoprotein cholesterol measurement', 'type II diabetes mellitus', 'triglyceride measurement', 'vitamin D measurement', 'hematocrit'], ['protein measurement', 'vital capacity', 'intelligence', 'forced expiratory volume', 'smoking behaviour measurement'], ['breast adenocarcinoma', 'pancreatic adenocarcinoma', 'squamous cell lung carcinoma', 'cutaneous melanoma', 'infection'], ['central corneal thickness', 'testosterone measurement', 'breast cancer', 'neoplasm', 'cyst'], ['body mass index', 'visual perception measurement', 'platelet count', 'risk-taking behaviour', 'Helicobacter pylori infectious disease'], ['mean corpuscular hemoglobin', 'mean corpuscular volume', 'Blackfan-Diamond anemia', 'Microcytic anemia with liver iron overload', 'Alpha-thalassemia - myelodysplastic syndrome'], ['neoplasm', 'Hereditary renal hypouricemia', 'protein measurement', 'cerebellar ataxia', 'Ataxia'], ['neoplasm', 'breast cancer', 'endometrial cancer', 'cancer', 'leiomyoma'], ['HbA1c measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'hemoglobin measurement', 'Uncombable hair syndrome'], ['cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'age at menarche', 'lean body mass'], ['prostate carcinoma', 'COVID-19', 'benign prostatic hyperplasia', 'prostate adenocarcinoma', 'severe acute respiratory syndrome'], ['Autosomal recessive cerebral atrophy', 'non-small cell lung carcinoma', 'neoplasm', 'cancer', 'breast cancer'], ['IRIDA syndrome', 'Hypochromic microcytic anemia', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'red blood cell distribution width'], ['reticulocyte count', 'idiopathic pulmonary fibrosis', 'non-small cell lung carcinoma', 'infect', 'infection'], ['total cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'triglyceride measurement', 'chronic kidney disease', 'urinary metabolite measurement'], ['DNA methylation', 'neoplasm', 'pes', 'prostate cancer', 'colorectal carcinoma'], ['Congenital enteropathy due to enteropeptidase deficiency', 'self reported educational attainment', 'acute myeloid leukemia', 'anemia (disease)', 'bipolar disorder'], ['neoplasm', 'breast cancer', 'nonpapillary renal cell carcinoma', 'colorectal carcinoma', 'lung adenocarcinoma'], ['hepatocellular carcinoma', 'Severe combined immunodeficiency due to CORO1A deficiency', 'attention deficit hyperactivity disorder', 'lean body mass', 'vital capacity'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'blood protein measurement', 'tenascin measurement', 'protein measurement'], ['psoriatic arthritis', 'rheumatoid arthritis', 'psoriasis', "Crohn's disease", 'ankylosing spondylitis'], ['breast cancer', 'cancer', 'folate deficiency', 'sporadic amyotrophic lateral sclerosis', 'serine/threonine-protein kinase 17B measurement'], ['cathepsin L1 measurement', 'inflammatory bowel disease', 'glioma', 'gonorrhea', 'esophageal squamous cell carcinoma'], ['Autoimmune lymphoproliferative syndrome', 'Hodgkins lymphoma', 'diffuse large B-cell lymphoma', 'systemic lupus erythematosus', 'rheumatoid arthritis'], ['Tumor necrosis factor receptor 1 associated periodic syndrome', 'multiple sclerosis', "Behcet's syndrome", 'lymphocyte count', 'CHARGE syndrome'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'tumor necrosis factor receptor II measurement', 'glioblastoma multiforme'], ['Primary immunodeficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'neoplasm', 'systemic lupus erythematosus'], ['ulcerative colitis', "Crohn's disease", 'inflammatory bowel disease', 'psoriasis', 'ankylosing spondylitis'], ['Hodgkins lymphoma', 'anaplastic large cell lymphoma', 'lymphoma', 'neoplasm', 'eosinophil count'], ['rheumatoid arthritis', 'hair color', 'diffuse large B-cell lymphoma', 'ulcerative colitis', 'pancreatic adenocarcinoma'], ['age-related macular degeneration', 'monocyte count', 'central serous retinopathy', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['mean corpuscular volume', 'TNF-related apoptosis-inducing ligand receptor 2 measurement', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'esophageal cancer'], ['Juvenile Paget disease', 'eosinophil count', 'bone density', 'alkaline phosphatase measurement', 'eosinophil percentage of leukocytes'], ['breast cancer', 'hepatocellular carcinoma', 'rheumatoid arthritis', 'neoplasm', 'melanoma'], ['Common variable immunodeficiency', 'immunodeficiency, common variable, 2', 'Primary immunodeficiency', 'selective IgA deficiency disease', 'multiple myeloma'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'coronary artery disease', 'immunodeficiency, common variable, 2', 'atherosclerosis'], ['diffuse large B-cell lymphoma', 'neoplasm of mature B-cells', 'basal cell carcinoma', 'lymphoid neoplasm', 'lung carcinoma'], ['multiple myeloma', 'neoplasm', 'pancreatic carcinoma', 'cutaneous melanoma', 'gastric carcinoma'], ['drug use measurement', 'neoplasm', 'Miyoshi myopathy', 'systemic lupus erythematosus', 'infect'], ['mean platelet volume', 'breast adenocarcinoma', 'platelet count', 'head and neck squamous cell carcinoma', 'neoplasm'], ['Autosomal recessive lower motor neuron disease with childhood onset', 'Distal spinal muscular atrophy', 'lung cancer', 'colitis', 'prostate neoplasm'], ['systemic lupus erythematosus', 'asthma', 'rheumatoid arthritis', 'allergy', 'Eczema'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'inflammatory bowel disease', "Crohn's disease", 'sum of eosinophil and basophil counts'], ['neoplasm', 'colorectal adenocarcinoma', 'colon carcinoma', 'colonic neoplasm', 'malignant colon neoplasm'], ['TNF-related apoptosis-inducing ligand measurement', 'sex hormone-binding globulin measurement', 'monocyte percentage of leukocytes', 'platelet count', 'monocyte count'], ['Autosomal recessive malignant osteopetrosis', 'autosomal recessive osteopetrosis 2', 'osteoporosis', 'postmenopausal osteoporosis', 'giant cell tumor'], ['serum non-albumin protein measurement', 'alcohol consumption measurement', 'IGA glomerulonephritis', 'blood protein measurement', 'albumin:globulin ratio measurement'], ['systemic lupus erythematosus', 'monocyte count', 'platelet count', 'myeloid white cell count', 'neutrophil count'], ['multiple sclerosis', 'tumor necrosis factor ligand superfamily member 14 measurement', 'monocyte count', 'blood protein measurement', 'monocyte percentage of leukocytes'], ["Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'psoriasis', 'ankylosing spondylitis'], ["Crohn's disease", 'celiac disease', 'ulcerative colitis', 'ankylosing spondylitis', 'psoriasis'], ['Autosomal recessive non-syndromic intellectual disability', 'body mass index', 'lymphocyte count', 'lymphocyte percentage of leukocytes', 'arterial stiffness measurement'], ['Frontotemporal dementia', 'Infantile myofibromatosis', 'reticulum cell sarcoma', 'Familial atypical multiple mole melanoma syndrome', 'Epidermodysplasia verruciformis'], ['Infantile-onset mesial temporal lobe epilepsy with severe cognitive regression', "Parkinson's disease", 'hematocrit', 'mean corpuscular hemoglobin', 'breast cancer'], ['cancer', 'alcohol drinking', 'systolic blood pressure', 'ovarian cancer', 'mathematical ability'], ['sex hormone-binding globulin measurement', 'cancer', 'testosterone measurement', 'neutrophil count', 'alkaline phosphatase measurement'], ['Arthrogryposis - hyperkeratosis, lethal form', 'Pelvic dysplasia - arthrogryposis of lower limbs', 'age-related macular degeneration', 'obesity', 'diabetes mellitus'], ['mean corpuscular hemoglobin concentration', 'mean platelet volume', 'red blood cell distribution width', 'platelet count', 'hemoglobin measurement'], ['reticulocyte count', 'chronic obstructive pulmonary disease', 'forced expiratory volume', 'breast carcinoma', 'vital capacity'], ['reticulocyte count', 'prostate carcinoma', 'reticulocyte measurement', 'basal cell carcinoma', 'myeloid white cell count'], ['acute myeloid leukemia', 'neoplasm', 'breast cancer', 'acute lymphoblastic leukemia', 'adult acute lymphoblastic leukemia'], ['Autosomal recessive limb-girdle muscular dystrophy', 'Abnormality of refraction', 'prostate carcinoma', 'hepatocellular carcinoma', 'mean corpuscular volume'], ['Li-Fraumeni syndrome', 'head and neck squamous cell carcinoma', 'osteosarcoma', 'colorectal adenocarcinoma', 'basal cell carcinoma'], ['EEC syndrome', 'Ankyloblepharon - ectodermal defects - cleft lip/palate', 'ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3', 'ADULT syndrome', 'Split hand-split foot malformation'], ['acute myeloid leukemia', 'ovarian cancer', 'lung adenocarcinoma', 'urinary bladder cancer', 'gastric adenocarcinoma'], ['lean body mass', 'fat body mass', 'body mass index', 'body weight', 'base metabolic rate measurement'], ['hemoglobin measurement', 'hematocrit', 'red blood cell density measurement', 'drug use measurement', 'asthma'], ['reaction time measurement', 'risk-taking behaviour', 'hypertrophic cardiomyopathy', 'waist-hip ratio', 'smoking status measurement'], ['Triose phosphate-isomerase deficiency', 'multiple myeloma', 'acute myeloid leukemia', 'B-cell non-Hodgkins lymphoma', 'endometrial cancer'], ['Digitotalar dysmorphism', 'Nemaline myopathy', 'Cap myopathy', 'Distal arthrogryposis', 'Sheldon-Hall syndrome'], ['Congenital fiber-type disproportion myopathy', 'Nemaline myopathy', 'Cap myopathy', 'anaplastic large cell lymphoma', 'papillary thyroid carcinoma'], ['Familial thyroid dyshormonogenesis', 'Abnormality of the thyroid gland', 'Graves disease', 'Graves ophthalmopathy', 'nodular goiter'], ['CLN2 disease', 'Childhood-onset autosomal recessive slowly progressive spinocerebellar ataxia', 'Neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Late infantile neuronal ceroid lipofuscinosis'], ['glomerular filtration rate', 'appendicular lean mass', 'diabetes mellitus', 'Abnormal urine potassium concentration', 'peptic esophagitis'], ['HIV infection', 'influenza', 'viral disease', 'papillary thyroid carcinoma', 'Benign Thyroid Gland Neoplasm'], ['blood protein measurement', 'systemic mastocytosis', 'appendicular lean mass', 'tryptase beta-2 measurement', 'DNA methylation'], ['eosinophil count', 'response to drug', 'hypertension', 'appendicular lean mass', 'myeloid white cell count'], ['red blood cell density measurement', 'low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['platelet count', 'platelet crit', 'eosinophil count', 'mean platelet volume', 'alcohol abuse'], ['rheumatoid arthritis', 'lymphocyte count', 'myeloid white cell count', 'asthma', 'leukocyte count'], ['genetic disorder', 'ovarian cancer', 'breast adenocarcinoma', 'lung adenocarcinoma', 'ovarian carcinoma'], ['Herpetic encephalitis', 'herpes simplex encephalitis, susceptibility to, 3', 'Primary immunodeficiency', 'multiple myeloma', 'lean body mass'], ['Chronic mucocutaneous candidosis', 'rheumatoid arthritis', 'psoriasis', 'discoid lupus erythematosus', 'psoriatic arthritis'], ['Autosomal dominant hypohidrotic ectodermal dysplasia', 'glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'esophageal cancer'], ['Symbol', 'nan', 'nan', 'nan', 'nan'], ['blood protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Sepsis'], ['Nasu-Hakola disease', 'dementia (disease)', 'dystonic disorder', 'C-reactive protein measurement', 'lung adenocarcinoma'], ['mean corpuscular volume', 'acute myeloid leukemia', 'blood protein measurement', 'monocyte count', 'leukocyte count'], ['monocyte count', 'myeloid white cell count', 'leukocyte count', 'blood protein measurement', 'reticulocyte count'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil count', 'myeloid white cell count', 'neutrophil percentage of granulocytes'], ['Aicardi-Goutières syndrome', 'systemic lupus erythematosus', 'Chilblain lupus', 'Retinal vasculopathy and cerebral leukodystrophy', 'Hereditary vascular retinopathy'], ['acute myeloid leukemia', 'neoplasm', 'brucellosis', 'hepatocellular carcinoma', 'T-cell acute lymphoblastic leukemia'], ['sex hormone-binding globulin measurement', 'body height', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['neoplasm', 'X-linked retinal dysplasia', 'Ehlers-Danlos syndrome, progeroid type', 'rheumatoid arthritis', 'refractory anemia'], ['lung adenocarcinoma', 'ovarian cancer', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma'], ['schizophrenia', 'thyroid gland adenocarcinoma', 'autism spectrum disorder', 'bipolar disorder', 'unipolar depression'], ['Autosomal recessive limb-girdle muscular dystrophy type 2H', 'Bardet-Biedl syndrome', 'Autosomal recessive limb-girdle muscular dystrophy', 'Limb-girdle muscular dystrophy', 'myopathy'], ['clear cell renal carcinoma', 'risk-taking behaviour', 'mosquito bite reaction itch intensity measurement', 'colorectal carcinoma', 'glioma'], ['spinal muscular atrophy with congenital bone fractures 1', 'prenatal-onset spinal muscular atrophy with congenital bone fractures', 'amyotrophic lateral sclerosis', 'axial length measurement', "Alzheimer's disease"], ['glioblastoma multiforme', 'neoplasm', 'Hyper-IgM syndrome type 3', 'Hepatic veno-occlusive disease - immunodeficiency', 'Hyper-IgM syndrome type 5'], ['combined oxidative phosphorylation deficiency 35', 'developmental disability', 'genetic disorder', 'Macrocephaly', 'Epileptic encephalopathy'], ['mean platelet volume', 'platelet count', 'thyroid cancer', 'thyroid carcinoma', 'platelet component distribution width'], ['Familial episodic pain syndrome with predominantly upper body involvement', 'arthritis', 'pain', 'Familial episodic pain syndrome', 'Myalgia'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'glioblastoma multiforme', 'ovarian cancer'], ['Spinocerebellar ataxia type 41', 'Autosomal dominant cerebellar ataxia', 'childhood onset asthma', 'acute myeloid leukemia', 'breast adenocarcinoma'], ['smoking status measurement', 'smoking initiation', 'protein measurement', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma', 'cutaneous melanoma', 'mental retardation'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'response to prednisolone', 'myeloid white cell count', 'kidney failure'], ['Familial progressive cardiac conduction defect', 'Erythrokeratoderma variabilis progressiva', 'Progressive symmetric erythrokeratodermia', 'Erythrokeratodermia variabilis', 'sudden cardiac arrest'], ['pain', 'arthritis', 'Myalgia', 'frozen shoulder', 'Muscle spasm'], ['migraine disorder', 'pain', 'Fever', 'osteoarthritis', 'arthritis'], ['Mutilating palmoplantar keratoderma with periorificial keratotic plaques', 'Non-epidermolytic palmoplantar keratoderma', 'breast adenocarcinoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['Metatropic dysplasia', 'Spondylometaphyseal dysplasia, Kozlowski type', 'Autosomal dominant Charcot-Marie-Tooth disease type 2C', 'Autosomal dominant congenital benign spinal muscular atrophy', 'Familial digital arthropathy-brachydactyly'], ['hyperparathyroidism, transient neonatal', 'Neonatal severe primary hyperparathyroidism', 'hyperparathyroidism', 'anxiety measurement', 'glioblastoma multiforme'], ['developmental delay with or without dysmorphic facies and autism', 'deafness, autosomal dominant 75', 'Global developmental delay', 'autism', 'Intellectual disability'], ['Tuberous sclerosis', 'Lymphangioleiomyomatosis', 'Isolated focal cortical dysplasia type II', 'urinary bladder cancer', 'Inherited cancer-predisposing syndrome'], ['HIV infection', 'viral disease', 'infect', 'infection', 'hepatocellular carcinoma'], ['Hypothyroidism due to TSH receptor mutations', 'Familial hyperthyroidism due to mutations in TSH receptor', 'Familial gestational hyperthyroidism', 'thyroid neoplasm', 'Thyroid hypoplasia'], ['Abnormality of the ear', 'protein measurement', 'endometrial neoplasm', 'colorectal cancer', 'lymphocyte count'], ['vitamin D measurement', 'factor VII measurement', 'self reported educational attainment', 'osteoarthritis', 'body mass index'], ['asthma', 'eosinophil count', 'Nasal Cavity Polyp', 'childhood onset asthma', 'Eczema'], ['heart failure', 'smoking status measurement', 'response to drug', 'hypertension', 'overweight body mass index status'], ['intellectual disability, X-linked 58', 'X-linked non-syndromic intellectual disability', 'mental retardation', 'breast adenocarcinoma', 'Intellectual disability'], ['Familial exudative vitreoretinopathy', 'bone density', 'Norrie disease', 'Persistent hyperplastic primary vitreous', 'Vitreoretinopathy'], ['hematocrit', 'hemoglobin measurement', 'urate measurement', 'systemic lupus erythematosus', 'glomerular filtration rate'], ['monocyte count', 'myeloid white cell count', 'leukocyte count', 'monocyte percentage of leukocytes', 'neutrophil count'], ['attention deficit hyperactivity disorder', 'intellectual disability, autosomal dominant 45', 'X-linked dominant intellectual disability - epilepsy syndrome', 'X-linked retinal dysplasia', 'Severe early-childhood-onset retinal dystrophy'], ['gastric carcinoma', 'lung adenocarcinoma', 'colorectal carcinoma', 'risky sexual behaviour measurement', 'appendicular lean mass'], ['macula measurement', 'pancreatic neoplasm', 'hepatocellular carcinoma', 'neoplasm', 'glioblastoma multiforme'], ['Tibial muscular dystrophy', 'Early-onset myopathy with fatal cardiomyopathy', 'Hereditary proximal myopathy with early respiratory failure', 'Familial dilated cardiomyopathy', 'dilated cardiomyopathy'], ['Ataxia with vitamin E deficiency', 'hereditary ataxia', 'diet measurement', 'blood protein measurement', 'C-reactive protein measurement'], ['ATTRV122I amyloidosis', 'Familial amyloid polyneuropathy', 'Familial transthyretin-related amyloidosis', 'familial amyloid neuropathy', 'cardiomyopathy'], ['Retinitis pigmentosa', 'autism spectrum disorder', 'diet measurement', 'hearing loss', 'age-related hearing impairment'], ['Congenital fibrosis of extraocular muscles', 'Cortical dysgenesis with pontocerebellar hypoplasia due to TUBB3 mutation', 'head and neck squamous cell carcinoma', 'ovarian cancer', 'pancreatic adenocarcinoma'], ['Autosomal recessive chorioretinopathy-microcephaly', 'ovarian cancer', 'breast adenocarcinoma', 'glycine measurement', 'eosinophil count'], ['Autosomal recessive chorioretinopathy-microcephaly', 'Microcephaly - lymphedema - chorioretinopathy', 'Autosomal recessive primary microcephaly', 'Isolated congenital microcephaly', 'ovarian cancer'], ['red blood cell distribution width', 'body height', 'FEV/FEC ratio', 'sitting height measurement', 'forced expiratory volume'], ['Saethre-Chotzen syndrome', 'Isolated oxycephaly', 'Isolated scaphocephaly', 'Isolated brachycephaly', 'Syndromic craniosynostosis'], ['Autosomal dominant progressive external ophthalmoplegia', 'Perrault syndrome', 'Infantile onset spinocerebellar ataxia', 'Mitochondrial DNA depletion syndrome, hepatocerebral form due to DGUOK deficiency', 'Mitochondrial DNA depletion syndrome'], ['pancreatic adenocarcinoma', 'Benign familial chorea', 'Cholesterol-ester transfer protein deficiency', 'Testicular regression syndrome', 'Homozygous familial hypercholesterolemia'], ['mean corpuscular volume', 'sex hormone-binding globulin measurement', 'mean corpuscular hemoglobin concentration', 'body height', 'mean corpuscular hemoglobin'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'haptoglobin measurement', 'hemoglobin measurement', 'hematocrit'], ['acute promyelocytic leukemia', 'neoplasm', 'diphtheria', 'rheumatic disease', 'leukemia'], ['Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'rheumatoid arthritis', 'ulcerative colitis', 'ankylosing spondylitis', 'immune system disease'], ['colorectal carcinoma', 'breast carcinoma', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'neoplasm'], ['Oculocutaneous albinism type 1', 'Oculocutaneous albinism type 1A', 'Oculocutaneous albinism type 1B', 'Temperature-sensitive oculocutaneous albinism type 1', 'Ocular albinism with congenital sensorineural deafness'], ['urinary albumin to creatinine ratio', 'Moderate albuminuria', 'metabolic syndrome', 'blood protein measurement', 'neoplasm'], ['Nasu-Hakola disease', 'dementia (disease)', 'late-onset Alzheimers disease', "Parkinson's disease", 'memory impairment'], ['Oculocutaneous albinism type 3', 'Oculocutaneous albinism', 'hair color', 'Piebaldism', 'cutaneous melanoma'], ['acute myeloid leukemia', 'lung adenocarcinoma', 'myelodysplastic syndrome', 'Uterine Carcinosarcoma', 'chronic myelomonocytic leukemia'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'influenza'], ['total cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'monocyte count', 'lymphocyte count', 'sex hormone-binding globulin measurement'], ['HIV infection', 'cancer', 'listeriosis', 'T-cell leukemia', 'Tuberculosis'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'Cystic fibrosis'], ['platelet count', 'self reported educational attainment', 'sex hormone-binding globulin measurement', 'insulin like growth factor measurement', 'platelet crit'], ['red blood cell distribution width', 'mean reticulocyte volume', 'platelet crit', 'reticulocyte measurement', 'platelet count'], ['colorectal carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'breast carcinoma', 'hepatocellular carcinoma'], ['Angelman syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Global developmental delay'], ['Short stature due to GHSR deficiency', 'oral cavity carcinoma', 'head and neck carcinoma', 'esophageal adenocarcinoma', 'esophageal squamous cell carcinoma'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'alcohol drinking', 'triglyceride measurement', 'alcohol consumption measurement'], ['Early-onset progressive neurodegeneration - blindness - ataxia - spasticity', 'Young adult-onset Parkinsonism', 'Spastic paraplegia', 'glioblastoma multiforme', 'non-small cell lung carcinoma'], ['low grade glioma', 'heart failure', 'colorectal carcinoma', 'Hand-foot syndrome', 'hydrops fetalis (disease)'], ['triglyceride measurement', 'sleep duration', 'diabetes mellitus', 'Partial pancreatic agenesis', 'Asperger syndrome'], ['obesity', 'coronary artery disease', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'Duane retraction syndrome'], ['Gaucher disease', 'Niemann-Pick disease type C', 'red blood cell distribution width', 'neutrophil count', 'Gaucher disease type 1'], ['developmental and epileptic encephalopathy, 84', 'Epileptic encephalopathy', 'hemorrhoid', 'peripheral arterial disease', 'traffic air pollution measurement'], ['Crigler-Najjar syndrome type 1', 'Crigler-Najjar syndrome type 2', 'Gilbert syndrome', 'Crigler-Najjar syndrome', 'Transient familial neonatal hyperbilirubinemia'], ['cutaneous melanoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'testosterone measurement'], ['erythrocyte count', 'hemoglobin measurement', 'red blood cell density measurement', 'hematocrit', 'glomerular filtration rate'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'asthma', 'seasonal allergic rhinitis', 'eosinophil percentage of granulocytes'], ['Pachyonychia congenita', 'blood protein measurement', 'midbrain', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm'], ['breast adenocarcinoma', 'cancer', 'glioma', 'acute myeloid leukemia', 'nonpapillary renal cell carcinoma'], ['Familial juvenile hyperuricemic nephropathy type 1', 'Autosomal dominant medullary cystic kidney disease with or without hyperuricemia', 'Medullary Cystic Kidney Disease Type II', 'Autosomal dominant medullary cystic kidney disease with hyperuricemia', 'hypertension'], ['chronotype measurement', 'vital capacity', 'acute myeloid leukemia', 'breast adenocarcinoma', 'ovarian cancer'], ["Alzheimer's disease", 'Early-onset autosomal dominant Alzheimer disease', 'fat body mass', 'refractive error measurement', 'type II diabetes mellitus'], ['Hyper-IgM syndrome type 5', 'Hyper-IgM syndrome without susceptibility to opportunistic infections', 'Hyper-IgM syndrome type 2', 'infect', 'infection'], ['protein measurement', 'Hemoglobin E - beta-thalassemia', 'self reported educational attainment', 'Adult-onset autosomal recessive sideroblastic anemia', 'Delta-beta-thalassemia'], ['urinary bladder cancer', 'Dent disease type 1', 'Ochoa syndrome', 'Hereditary renal hypouricemia', 'Primary hyperoxaluria type 2'], ['myeloid white cell count', 'leukocyte count', 'cutaneous melanoma', 'neutrophil count', 'glioma'], ['vaginal discharge', 'candidiasis', 'neoplasm', 'gastric cancer', 'cancer'], ['cardiomyopathy', 'Propionic acidemia', 'Isolated NADH-CoQ reductase deficiency', 'mitochondrial complex 3 deficiency, nuclear type 10', 'Isolated CoQ-cytochrome C reductase deficiency'], ['sex hormone-binding globulin measurement', 'reticulocyte count', 'sleep duration', 'triglyceride measurement', 'mean corpuscular volume'], ['Retinitis pigmentosa', 'Usher syndrome type 2', 'Usher syndrome', 'hearing loss', 'Retinal dystrophy'], ['mean corpuscular volume', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'triglyceride measurement'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'heel bone mineral density', 'low density lipoprotein cholesterol measurement', 'vitamin D measurement'], ['Hao-Fountain syndrome', 'platelet count', 'mean platelet volume', 'platelet component distribution width', 'lymphocyte count'], ['vital capacity', 'triglyceride measurement', 'ovarian cancer', 'age at menarche', 'low grade glioma'], ['Congenital intrauterine infection-like syndrome', 'ovarian cancer', 'lung adenocarcinoma', 'ovarian carcinoma', 'glioblastoma multiforme'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'body height', 'reticulocyte count'], ['endometrial cancer', 'diabetes mellitus', 'ovarian carcinoma', 'colorectal carcinoma', 'cancer'], ['chronotype measurement', 'sitting height measurement', 'head and neck squamous cell carcinoma', 'body fat percentage', 'migraine disorder'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'low grade glioma', 'heart failure', 'Capillary malformation - arteriovenous malformation'], ['neurodevelopmental disorder with hypotonia and autistic features with or without hyperkinetic movements', 'Intellectual disability', 'botulism', 'Global developmental delay', 'Cerebral visual impairment'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cyst'], ['urinary albumin to creatinine ratio', 'coronary artery disease', 'prostate carcinoma', 'lean body mass', 'eosinophil count'], ['spina bifida', 'Isolated spina bifida', 'Neural tube closure defect', 'body height', 'neuroimaging measurement'], ['neoplasm', 'lung cancer', 'cancer', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy'], ['lymphocyte count', 'monocyte count', 'neutrophil percentage of leukocytes', 'leukocyte count', 'lymphocyte percentage of leukocytes'], ['Wagner disease', 'white matter microstructure measurement', 'appendicular lean mass', 'body fat percentage', 'lean body mass'], ['Inclusion body myopathy with Paget disease of bone and frontotemporal dementia', 'Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Autosomal dominant Charcot-Marie-Tooth disease type 2'], ['cup-to-disc ratio measurement', 'glioblastoma multiforme', 'acute myeloid leukemia', 'gastric carcinoma', 'multiple myeloma'], ['diabetic retinopathy', 'age-related macular degeneration', 'wet macular degeneration', 'diabetic macular edema', 'glioblastoma multiforme'], ['age-related macular degeneration', 'proliferative diabetic retinopathy', 'retinal detachment', 'diabetic macular edema', 'pathological myopia'], ['Milroy disease', 'age-related macular degeneration', 'proliferative diabetic retinopathy', 'retinal detachment', 'testosterone measurement'], ['neoplasm', 'Lymphangioleiomyomatosis', 'breast carcinoma', 'endometrial cancer', 'hemophagocytic syndrome'], ['vitamin D measurement', 'amyotrophic lateral sclerosis', 'axial length measurement', "Alzheimer's disease", 'Lewy body dementia'], ['Von Hippel-Lindau disease', 'Chuvash erythrocytosis', 'adrenal gland pheochromocytoma', 'nonpapillary renal cell carcinoma', 'papillary renal cell carcinoma'], ['Non-syndromic male infertility due to sperm motility disorder', 'neoplasm', 'Primary familial polycythemia', 'HbA1c measurement', 'mean corpuscular hemoglobin concentration'], ['Pulverulent cataract', 'Non-syndromic congenital cataract', 'cataract', 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement'], ['chronotype measurement', 'circadian rhythm', 'sum of eosinophil and basophil counts', 'lymphocyte count', 'monocyte count'], ['medulloblastoma', 'experimental autoimmune encephalomyelitis', 'lean body mass', 'Spinocerebellar ataxia type 4', 'body fat percentage'], ['Hereditary combined deficiency of vitamin K-dependent clotting factors', 'atrial fibrillation', 'vitamin K-dependent clotting factors, combined deficiency of, type 2', 'deep vein thrombosis', 'cholesterol embolism'], ['hypertension', 'colitis', "Barrett's esophagus", 'esophageal adenocarcinoma', 'Bartsocas-Papas syndrome'], ['inflammatory bowel disease', 'prostate carcinoma', 'heel bone mineral density', 'bone density', 'electrocardiography'], ['psoriasis', 'nonpapillary renal cell carcinoma', 'heel bone mineral density', 'electrocardiography', 'bone density'], ['age at onset', 'Myopia', 'visual perception measurement', 'glioblastoma multiforme', 'testosterone measurement'], ['Cohen syndrome', 'genetic disorder', 'Microcephaly', 'Intellectual disability', 'mental retardation'], ['pontocerebellar hypoplasia type 13', 'visual perception measurement', 'neoplasm', 'Sepsis', 'malaria'], ['Keratoconus', 'craniofacial anomalies and anterior segment dysgenesis syndrome', 'Posterior polymorphous corneal dystrophy', 'mean corpuscular hemoglobin', 'alcohol consumption measurement'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'blood protein measurement', 'low density lipoprotein cholesterol measurement', 'heel bone mineral density'], ['acute myeloid leukemia', 'intelligence', 'Posterior polymorphous corneal dystrophy', 'X-linked endothelial corneal dystrophy', 'Fuchs endothelial corneal dystrophy'], ['Von Willebrand disease', 'Von Willebrand disease type 2B', 'Von Willebrand disease type 2A', 'Von Willebrand disease type 2N', 'Von Willebrand disease type 2M'], ['Wiskott-Aldrich syndrome', 'X-linked thrombocytopenia with normal platelets', 'Hereditary thrombocytopenia with normal platelets', 'X-linked severe congenital neutropenia', 'Thrombocytopenia'], ['endometrial cancer', 'clear cell renal carcinoma', 'neuroblastoma', 'osteosarcoma', 'Ewing sarcoma'], ['red blood cell distribution width', 'platelet component distribution width', 'nan', 'nan', 'nan'], ['Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'Short stature due to primary acid-labile subunit deficiency', 'Transient neonatal diabetes mellitus', 'Hyperinsulinism due to INSR deficiency', 'Obesity due to melanocortin 4 receptor deficiency'], ['cutaneous squamous cell carcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['blood protein measurement', 'ovarian cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'Herpes Zoster'], ['neoplasm', 'ovarian cancer', 'renal fibrosis', 'chronic kidney disease', 'endometrial cancer'], ['Wolfram syndrome', 'Wolfram-like syndrome', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'type II diabetes mellitus', 'Non-syndromic congenital cataract'], ['heel bone mineral density', 'urinary albumin to creatinine ratio', 'Moderate albuminuria', 'albuminuria', 'bone quantitative ultrasound measurement'], ['bone density', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'hematocrit', 'erythrocyte count'], ['Hereditary sensory and autonomic neuropathy type 2', 'Pseudohypoaldosteronism type 2C', 'body mass index', 'body fat percentage', 'Pseudohypoaldosteronism type 2'], ['mean platelet volume', 'neuroimaging measurement', 'mathematical ability', 'brain volume measurement', 'brain measurement'], ['Atypical Mayer-Rokitansky-Küster-Hauser syndrome', 'Mayer-Rokitansky-Küster-Hauser syndrome', 'SERKAL syndrome', 'body height', 'lean body mass'], ['Werner syndrome', 'colorectal adenocarcinoma', 'gastric carcinoma', 'colorectal cancer', 'kidney neoplasm'], ['Denys-Drash syndrome', 'Frasier syndrome', 'Nephroblastoma', 'Meacham syndrome', 'Wilms tumor 1'], ['Autosomal recessive cerebellar ataxia-epilepsy-intellectual disability syndrome due to WWOX deficiency', 'Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'esophageal squamous cell carcinoma', 'West syndrome'], ['neoplasm', 'cancer', 'prostate cancer', 'gastric cancer', 'neuroblastoma'], ['hematocrit', 'hemoglobin measurement', 'hair shape measurement', 'mean corpuscular hemoglobin', 'pancreatic carcinoma'], ['adolescent idiopathic scoliosis', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Hepatitis'], ['ulcer disease', 'glioblastoma multiforme', 'C-X-C motif chemokine 16 measurement', 'breast adenocarcinoma', 'autoimmune disease'], ['Xanthinuria type I', 'Hereditary xanthinuria', 'Xanthinuria type II', 'gout', 'leukemia'], ['X-linked lymphoproliferative disease', 'inflammatory bowel disease', 'Splenomegaly', 'Lymphoproliferative syndrome', 'Sepsis'], ['monocyte count', 'monocyte percentage of leukocytes', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement', 'Left ventricular noncompaction'], ['McLeod neuroacanthocytosis syndrome', 'genetic disorder', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma'], ['chronic lymphocytic leukemia', 'multiple myeloma', 'HIV infection', 'influenza', 'gastric carcinoma'], ['Intellectual disability-coarse face-macrocephaly-cerebellar hypotrophy syndrome', 'Adult-onset autosomal recessive cerebellar ataxia', 'breast adenocarcinoma', 'ovarian cancer', 'body height'], ['Fanconi anemia', 'Inherited cancer-predisposing syndrome', 'acute myeloid leukemia', 'mean platelet volume', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation'], ['HIV infection', 'lung adenocarcinoma', 'breast adenocarcinoma', 'childhood onset asthma', 'endometrial cancer'], ['Spondylo-ocular syndrome', 'Pseudoxanthoma elasticum', 'genetic disorder', 'skin pigmentation', 'Autosomal recessive polycystic kidney disease'], ['Uveal coloboma - cleft lip and palate - intellectual disability', 'sex hormone-binding globulin measurement', 'polycystic ovary syndrome', 'pulse pressure measurement', 'birth weight'], ['Autosomal dominant intermediate Charcot-Marie-Tooth disease type C', 'Charcot-Marie-Tooth disease', 'Primary amenorrhea', 'retinal degeneration', 'Hepatic steatosis'], ['birth weight', 'HbA1c measurement', 'fasting blood glucose measurement', 'type II diabetes mellitus', 'serum metabolite measurement'], ['Leber hereditary optic neuropathy', 'Autosomal recessive isolated optic atrophy', 'Autosomal recessive syndromic optic atrophy', 'platelet count', 'platelet crit'], ['cancer', 'Noonan syndrome', 'Costello syndrome', 'hypertrophic cardiomyopathy', 'lean body mass'], ['endometrial stromal sarcoma', 'clear cell sarcoma', 'synovial sarcoma', 'bile duct carcinoma', 'undifferentiated pleomorphic sarcoma'], ['lean body mass', 'fat body mass', 'gut microbiome measurement', 'taxonomic microbiome measurement', 'body weight'], ['reticulocyte measurement', 'body mass index', 'reticulocyte count', 'chronotype measurement', 'risk-taking behaviour'], ['Combined immunodeficiency due to ZAP70 deficiency', 'autoimmune disease', 'Combined T and B cell immunodeficiency', 'Severe combined immunodeficiency', 'X-Linked Combined Immunodeficiency Diseases'], ['mean corpuscular volume', 'body fat percentage', 'erythrocyte count', 'mean corpuscular hemoglobin', 'body mass index'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'asthma', 'body mass index', 'hematocrit'], ['response to vaccine', 'lean body mass', 'visual perception measurement', 'chronic hepatitis B virus infection', 'cardiovascular disease'], ['body height', 'body mass index', 'self reported educational attainment', 'neoplasm', 'gait measurement'], ['lymphocyte count', 'neuroticism measurement', 'mathematical ability', 'feeling tense measurement', 'eosinophil percentage of leukocytes'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'asthma', 'Dupuytren Contracture', 'lymphocyte count'], ['Intellectual disability-developmental delay-contractures syndrome', 'genetic disorder', 'X-linked syndromic intellectual disability', 'Autosomal recessive spastic paraplegia type 25', 'Juvenile amyotrophic lateral sclerosis'], ['schizophrenia', 'self reported educational attainment', 'cutaneous melanoma', 'pancreatic adenocarcinoma', 'visual perception measurement'], ['Mowat-Wilson syndrome', 'genetic disorder', 'mean corpuscular hemoglobin concentration', 'renal cell carcinoma', 'self reported educational attainment'], ['neoplasm', 'prostate cancer', 'cancer', 'hepatocellular carcinoma', 'pancreatic carcinoma'], ['eosinophil count', 'ulcerative colitis', 'psoriasis', 'eosinophil percentage of leukocytes', 'Hepatic fibrosis'], ['systolic blood pressure', 'chronotype measurement', 'pulse pressure measurement', 'Agents acting on the renin-angiotensin system use measurement', 'appendicular lean mass'], ['red blood cell distribution width', 'erythrocyte count', 'red blood cell density measurement', 'mean corpuscular hemoglobin', 'age at menopause'], ['cancer', 'breast adenocarcinoma', 'infect', 'infection', 'atrial fibrillation'], ['Hand-foot syndrome', 'hydrops fetalis (disease)', 'rheumatoid arthritis', 'obesity', 'Hypoglycemia'], ['mean corpuscular volume', 'body height', 'lean body mass', 'forced expiratory volume', 'infectious diarrheal disease'], ['particulate matter air pollution measurement', 'unipolar depression', 'C-reactive protein measurement', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'depressive symptom measurement'], ['platelet count', 'mean platelet volume', 'platelet crit', 'mean corpuscular volume', 'Abnormality of refraction'], ['cancer', 'leukemia', 'myelodysplastic syndrome', 'precursor lymphoblastic lymphoma/leukemia', 'body mass index'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'neoplasm', 'breast cancer', 'mathematical ability'], ['cancer', 'neoplasm', 'systemic juvenile idiopathic arthritis', 'chronic kidney disease', 'Zika virus infectious disease'], ['particulate matter air pollution measurement', 'neoplasm', 'breast cancer', 'unipolar depression', 'aortic stenosis'], ['body height', 'platelet component distribution width', 'pancreatic adenocarcinoma', 'lung cancer', 'nan'], ['Brittle cornea syndrome', 'Keratoconus', 'lean body mass', 'central corneal thickness', 'appendicular lean mass'], ['hematocrit', 'sex hormone-binding globulin measurement', 'smoking status measurement', 'body height', 'grip strength measurement'], ['head and neck squamous cell carcinoma', 'Zinc deficiency', 'hematocrit', 'neoplasm', 'sex hormone-binding globulin measurement'], ['Myoclonus-dystonia syndrome', 'Huntington disease-like 2', 'X-linked dominant intellectual disability - epilepsy syndrome', 'Spinocerebellar ataxia type 14', 'FRAXE intellectual disability'], ['dehydroepiandrosterone sulphate measurement', 'blood metabolite measurement', 'body fat percentage', 'platelet count', 'smoking behavior'], ['Blackfan-Diamond anemia', 'appendicular lean mass', 'neutrophil count', 'alcohol dependence', 'amyotrophic lateral sclerosis'], ['appendicular lean mass', 'fat body mass', 'lean body mass', 'Congenital contractural arachnodactyly', 'cholangiocarcinoma'], ['prostate carcinoma', 'visual perception measurement', 'Ataxia-telangiectasia', 'schizophrenia', 'self reported educational attainment'], ['mean corpuscular hemoglobin concentration', 'Down syndrome', 'mean corpuscular hemoglobin', 'body height', 'appendicular lean mass'], ['balding measurement', 'type II diabetes mellitus', 'hemoglobin measurement', 'hair shape measurement', 'hematocrit'], ['sitting height measurement', 'rheumatoid arthritis', 'neutrophil count', 'lymphocyte count', 'diet measurement'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'rheumatoid arthritis', 'refractory anemia', 'non-small cell lung carcinoma'], ['Triple A syndrome', 'influenza', 'HIV infection', 'viral disease', 'Histiocytosis'], ['glioblastoma multiforme', 'androgenetic alopecia', 'hair shape measurement', 'vital capacity', 'forced expiratory volume'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'mean reticulocyte volume', 'mean corpuscular hemoglobin'], ["Hashimoto's thyroiditis", 'thyroid volume', 'high density lipoprotein cholesterol measurement', 'physical activity', 'macrophage inflammatory protein 1b measurement'], ['Tangier disease', 'Apolipoprotein A-I deficiency', 'hyperlipidemia', 'Hypercholesterolemia', 'high density lipoprotein cholesterol measurement'], ['Stargardt disease', 'Cone rod dystrophy', 'age-related macular degeneration', 'Severe early-childhood-onset retinal dystrophy', 'Retinitis pigmentosa'], ['non-small cell lung carcinoma', 'blood protein measurement', 'body height', "Alzheimer's disease", 'cognitive decline measurement'], ['Dyschromatosis universalis', 'Familial pseudohyperkalemia', 'Colobomatous microphthalmia', 'microphthalmia', 'Coloboma of iris'], ['X-linked sideroblastic anemia with ataxia', 'sideroblastic anemia', 'Spinocerebellar ataxia type 1', 'genetic disorder', 'vaginal discharge'], ['Dubin-Johnson syndrome', 'intrahepatic cholestasis', 'genetic disorder', 'erythrocyte count', 'Pseudoxanthoma elasticum'], ['Pseudoxanthoma elasticum', 'Generalized arterial calcification of infancy', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count'], ['hyperinsulinemic hypoglycemia, familial, 1', 'type II diabetes mellitus', 'diabetes mellitus, permanent neonatal 3', 'Familial hyperinsulinism', 'Transient neonatal diabetes mellitus'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['influenza A (H1N1)', 'Graves disease', 'serum IgM measurement', 'QRS duration', 'hemoglobin measurement'], ['neuritic plaque measurement', 'birth weight', 'endometrial cancer', 'high density lipoprotein cholesterol measurement', 'cutaneous melanoma'], ['gout', 'urate measurement', 'uric acid measurement', 'body mass index', 'underweight body mass index status'], ['mean platelet volume', 'gut microbiome measurement', 'salivary gland disease', 'Atrophy', 'coronary artery disease'], ['breast adenocarcinoma', 'periodontal measurement', 'Palmar Fibromatosis', 'monocyte count', 'narcolepsy with cataplexy'], ['breast adenocarcinoma', 'ovarian cancer', 'platelet component distribution width', 'platelet crit', 'non-small cell lung carcinoma'], ['chronic myelogenous leukemia', 'congenital heart defects and skeletal malformations syndrome', 'acute lymphoblastic leukemia', 'neoplasm', 'Gastrointestinal stromal tumor'], ['kidney neoplasm', 'lymphoid neoplasm', 'bile duct carcinoma', 'hemangioblastoma', 'basal cell carcinoma'], ['thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['lymphocyte count', 'hematocrit', 'low grade glioma', 'thyroid gland adenocarcinoma', 'Skull Fractures'], ['autism spectrum disorder', 'triglyceride measurement', 'breast cancer', 'cancer', 'Obesity due to melanocortin 4 receptor deficiency'], ['Isobutyryl-CoA dehydrogenase deficiency', 'body fat percentage', 'body height', 'fat body mass', 'lean body mass'], ['Spondyloepimetaphyseal dysplasia, aggrecan type', 'Spondyloepiphyseal dysplasia, Kimberley type', 'Familial osteochondritis dissecans', 'Osteochondritis Dissecans', 'body height'], ['Ketoacidosis due to beta-ketothiolase deficiency', 'beta-ketothiolase deficiency', 'genetic disorder', 'rheumatoid arthritis', 'nonpapillary renal cell carcinoma'], ['IGA glomerulonephritis', 'pulse pressure measurement', 'systolic blood pressure', 'hair shape measurement', 'dry eye syndrome'], ['Dyskeratosis congenita', 'Hoyeraal-Hreidarsson syndrome', 'Hereditary isolated aplastic anemia', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['diabetic nephropathy', 'hypertension', 'Renal tubular dysgenesis of genetic origin', 'cardiovascular disease', 'type II diabetes mellitus'], ['COVID-19', 'severe acute respiratory syndrome', 'endometrial cancer', 'cutaneous melanoma', 'squamous cell lung carcinoma'], ["Alzheimer's disease", 'Dementia', 'Myasthenia gravis', "Parkinson's disease", 'Cognitive impairment'], ['breast adenocarcinoma', 'endometrial cancer', 'ovarian cancer', 'low grade glioma', 'glioblastoma multiforme'], ['eosinophil count', 'monocyte count', 'monocyte percentage of leukocytes', 'leukocyte count', 'myeloid white cell count'], ['cardiovascular disease', 'coronary artery disease', 'familial hypercholesterolemia', 'hyperlipidemia', 'reticulocyte count'], ['erythrocyte count', 'red blood cell density measurement', 'grip strength measurement', 'body fat percentage', "Parkinson's disease"], ['Peroxisomal acyl-CoA oxidase deficiency', 'Mitchell syndrome', 'Muscle weakness', 'chronic kidney disease', 'urinary metabolite measurement'], ['low molecular weight phosphotyrosine protein phosphatase measurement', 'blood protein measurement', 'appendicular lean mass', 'high density lipoprotein cholesterol measurement', 'protein measurement'], ['prostate cancer', 'self reported educational attainment', 'prostate adenocarcinoma', 'hematocrit', 'hemoglobin measurement'], ['cutaneous melanoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'infect'], ['platelet count', 'red blood cell distribution width', 'prostate carcinoma', 'vital capacity', 'heel bone mineral density'], ['mean corpuscular hemoglobin concentration', 'reticulocyte count', 'hematocrit', 'hemoglobin measurement', 'erythrocyte count'], ['type II diabetes mellitus', 'age at menopause', 'mean platelet volume', 'platelet crit', 'menopause'], ['Oral cleft', 'cutaneous melanoma', 'glomerular filtration rate', 'coronary artery disease', 'forced expiratory volume'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Moyamoya disease', 'Multisystemic smooth muscle dysfunction syndrome', 'Rare disease with thoracic aortic aneurysm and aortic dissection', 'Rare genetic vascular disease'], ['Baraitser-Winter syndrome', 'Developmental malformations - deafness - dystonia', 'cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'focal segmental glomerulosclerosis', 'neuroimaging measurement', 'insulin like growth factor measurement', 'brain measurement'], ['Fibrodysplasia ossificans progressiva', 'genetic disorder', 'kidney neoplasm', 'endometrial cancer', 'carcinoma of liver and intrahepatic biliary tract'], ['pancreatic carcinoma', 'FEV/FEC ratio', 'wellbeing measurement', 'CC16 measurement', 'lung adenocarcinoma'], ['colorectal adenocarcinoma', 'pancreatic adenocarcinoma', 'ulcerative colitis', 'Hepatobiliary Neoplasm', 'gastric carcinoma'], ['visceral heterotaxy', 'Heterotaxia', 'glomerular filtration rate', 'Familial isolated congenital asplenia', 'Double outlet right ventricle'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'epistaxis', 'pulmonary arteriovenous malformation (disease)', 'Heritable pulmonary arterial hypertension'], ['Neurological conditions associated with aminoacylase 1 deficiency', 'Aminoacylase 1 deficiency', 'Intellectual disability', 'toxic encephalopathy', 'body height'], ['Severe combined immunodeficiency due to adenosine deaminase deficiency', 'Severe combined immunodeficiency', 'partial adenosine deaminase deficiency', 'Omenn syndrome', 'chronic lymphocytic leukemia'], ['polyarteritis nodosa, childhoood-onset', 'Sneddon syndrome', 'vasculitis', 'Blackfan-Diamond anemia', "Behcet's syndrome"], ['neoplasm', 'mucous membrane pemphigoid', 'pancreatic carcinoma', 'hepatocellular carcinoma', 'Malignant Pancreatic Neoplasm'], ['Reticulate acropigmentation of Kitamura', "Alzheimer's disease", 'body mass index', 'family history of Alzheimer’s disease', 'schizophrenia'], ['pancreatic adenocarcinoma', 'refractive error', 'body height', 'lean body mass', 'fat body mass'], ['Neonatal inflammatory skin and bowel disease', 'Recurrent infections', 'inflammatory bowel disease', 'inflammatory skin disease', 'genetic disorder'], ['primary ovarian insufficiency', 'vital capacity', 'breast adenocarcinoma', 'hair colour measurement', "Alzheimer's disease"], ['eosinophil count', 'eosinophil percentage of leukocytes', 'head and neck squamous cell carcinoma', 'asthma', "Alzheimer's disease"], ['body height', 'a disintegrin and metalloproteinase with thrombospondin motifs 5 measurement', 'blood protein measurement', 'BMI-adjusted waist circumference', 'appendicular lean mass'], ['Congenital thrombotic thrombocytopenic purpura due to ADAMTS-13 deficiency', 'congenital thrombotic thrombocytopenic purpura', 'thrombotic thrombocytopenic purpura', 'Thrombocytopenia', 'Abnormal bleeding'], ['Dyschromatosis symmetrica hereditaria', 'Aicardi-Goutières syndrome', 'genetic disorder', 'dystonic disorder', 'Familial infantile bilateral striatal necrosis'], ['ulcerative colitis', 'autoimmune disease', 'eosinophil percentage of leukocytes', 'inflammatory bowel disease', "Crohn's disease"], ['body mass index', 'monocyte count', 'lymphocyte count', 'monocyte percentage of leukocytes', 'body height'], ['insulin like growth factor measurement', 'protein measurement', 'fetal genotype effect measurement', 'soluble interleukin-2 receptor subunit alpha measurement', 'gestational serum measurement'], ['DNA methylation', 'gut microbiome measurement', 'refractory celiac disease', 'thyroid gland adenocarcinoma', 'ovarian cancer'], ['Inherited congenital spastic tetraplegia', 'self reported educational attainment', 'cerebral palsy', 'eosinophil count', 'adolescent idiopathic scoliosis'], ['vibratory urticaria', 'ovarian cancer', 'glioblastoma multiforme', 'low grade glioma', 'Hiatus hernia'], ['self reported educational attainment', 'chronotype measurement', 'PR interval', 'risk-taking behaviour', 'Abnormality of refraction'], ['eosinophil count', 'neutrophil count', 'red blood cell distribution width', 'leukocyte count', 'lymphocyte percentage of leukocytes'], ['neoplasm', 'malaria', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm', 'prostate cancer'], ['adiponectin measurement', 'BMI-adjusted adiponectin measurement', 'blood protein measurement', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['Hypermethioninemia encephalopathy due to adenosine kinase deficiency', 'body mass index', 'tooth eruption', 'lean body mass', 'odontogenesis'], ['thyroid gland adenocarcinoma', 'diastolic blood pressure', 'lean body mass', 'adrenomedullin measurement', 'apolipoprotein A 1 measurement'], ['cardiovascular disease', 'hypertension', 'benign prostatic hyperplasia', 'myocardial infarction', 'orthostatic hypotension'], ['unipolar depression', 'pain', 'attention deficit hyperactivity disorder', 'hypertension', 'delirium'], ['chronic obstructive pulmonary disease', 'asthma', 'myocardial infarction', 'hypertension', 'open-angle glaucoma'], ['acute lymphoblastic leukemia', 'B-cell acute lymphoblastic leukemia', 'chronic myelogenous leukemia', 'triglyceride measurement', 'heel bone mineral density'], ['Cognitive impairment-coarse facies-heart defects-obesity-pulmonary involvement-short stature-skeletal dysplasia syndrome', '3q26 microduplication syndrome', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm', 'HER2 Positive Breast Carcinoma'], ['prostate carcinoma', 'blood protein measurement', 'afamin measurement', 'peripheral nerve injury', 'cancer biomarker measurement'], ['Congenital deficiency in alpha-fetoprotein', 'Hereditary persistence of alpha-fetoprotein', 'primary ovarian insufficiency', 'cancer biomarker measurement', 'blood protein measurement'], ['influenza A (H1N1)', 'myeloid white cell count', 'glioblastoma multiforme', 'eosinophil count', 'neutrophil count'], ['Glycogen storage disease due to glycogen debranching enzyme deficiency', 'Glycogen storage disease', 'genetic disorder', 'hypertrophic cardiomyopathy', 'endometrial cancer'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'mood instability measurement', 'hypothyroidism'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'waist circumference', 'risk-taking behaviour', 'phospholipid measurement'], ['protein measurement', 'acute myeloid leukemia', 'Tinnitus', 'pancreatic carcinoma', 'neoplasm'], ['Renal tubular dysgenesis of genetic origin', 'hypertension', 'Bilateral renal dysplasia', 'diastolic blood pressure', 'systolic blood pressure'], ['hypertension', 'Renal tubular dysgenesis of genetic origin', 'primary hypertension', 'kidney disease', 'congestive heart failure'], ['X-linked non-syndromic intellectual disability', 'Inguinal hernia', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['Primary hyperoxaluria type 1', 'Primary hyperoxaluria', 'nephrolithiasis', 'Nephrocalcinosis', 'body height'], ['Joubert syndrome with ocular defect', 'Joubert syndrome', 'Retinitis pigmentosa', 'Retinal dystrophy', 'multiple sclerosis'], ['Retinitis pigmentosa', 'eosinophil count', 'skin pigmentation', 'cutaneous melanoma', 'eosinophil percentage of leukocytes'], ['leukocyte count', 'apolipoprotein A 1 measurement', 'neoplasm', 'infect', 'infection'], ['Severe X-linked mitochondrial encephalomyopathy', 'X-linked hereditary sensory and autonomic neuropathy with deafness', 'X-linked Charcot-Marie-Tooth disease type 4', 'Spondyloepimetaphyseal dysplasia, Bieganski type', 'Leukoencephalopathy - metaphyseal chondrodysplasia'], ['Reticular dysgenesis', 'Intellectual disability', 'gut microbiome measurement', 'neoplasm', 'cancer'], ['glomerular filtration rate', 'neoplasm', 'cancer', 'acute myeloid leukemia', 'colorectal carcinoma'], ['glioblastoma multiforme', 'blood protein measurement', 'pulse pressure measurement', 'reticulocyte count', 'FEV/FEC ratio'], ['diabetes mellitus', 'ovarian cancer', 'squamous cell lung carcinoma', 'blood pressure', 'white matter growth measurement'], ['46,XY disorder of sex development due to isolated 17,20 lyase deficiency', '46,XY complete gonadal dysgenesis', 'testosterone measurement', 'lean body mass', 'appendicular lean mass'], ['lung adenocarcinoma', 'heel bone mineral density', 'metabolite measurement', 'cutaneous melanoma', 'head and neck squamous cell carcinoma'], ['triglyceride measurement', 'hemoglobin measurement', 'sex hormone-binding globulin measurement', 'lean body mass', 'testosterone measurement'], ['Proteus syndrome', 'breast carcinoma', 'Cowden syndrome', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['neoplasm', 'hepatocellular carcinoma', 'gastric cancer', 'head and neck squamous cell carcinoma', 'kidney neoplasm'], ['type II diabetes mellitus', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'cancer', 'prostate carcinoma', 'Familial partial lipodystrophy due to AKT2 mutations'], ['Megalencephaly - polymicrogyria - postaxial polydactyly - hydrocephalus', 'Megalencephaly-capillary malformation-polymicrogyria syndrome', 'cancer', 'glioma', 'schizophrenia'], ['hyperthyroxinemia', 'Congenital analbuminemia', 'liver cancer', 'Ehlers-Danlos syndrome, arthrochalasic type', 'Ehlers-Danlos syndrome type 7A'], ['glioblastoma multiforme', 'cutaneous melanoma', 'urinary bladder cancer', 'age at menarche', 'neoplasm'], ['Isolated anophthalmia - microphthalmia', 'Anophthalmia - microphthalmia', 'microphthalmia', 'autism', 'Breast hypertrophy'], ['alcohol dependence', 'parasitic infection', 'cocaine dependence', 'alcohol drinking', 'alcohol consumption measurement'], ['Sjögren-Larsson syndrome', 'Intellectual disability', 'mean platelet volume', 'mean corpuscular hemoglobin', 'gonorrhea'], ['Pyridoxine-dependent epilepsy', 'Seizure', 'Ventriculomegaly', 'genetic disorder', 'osteoporosis'], ['ALG3-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type I', 'genetic disorder', 'Intellectual disability'], ['neuroblastoma', 'non-small cell lung carcinoma', 'neuroblastoma, susceptibility to, 3', 'neoplasm', 'anaplastic large cell lymphoma'], ['dental caries', 'protein measurement', 'concussion', 'HIV-1 infection', 'viral load'], ['rheumatoid arthritis', 'ulcerative colitis', 'asthma', "Crohn's disease", 'juvenile idiopathic arthritis'], ['bipolar disorder', 'risk-taking behaviour', 'cutaneous melanoma', 'asthma', 'inflammatory bowel disease'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'Nasal Cavity Polyp', 'Eczema', 'sum of eosinophil and basophil counts'], ['inflammatory bowel disease', 'glioblastoma multiforme', 'cutaneous melanoma', 'breast adenocarcinoma', 'blood protein measurement'], ['hypertrophic cardiomyopathy', 'cardiomyopathy', 'PR interval', 'P wave terminal force measurement', 'Left ventricular mass to end-diastolic volume ratio'], ['Hypophosphatasia', 'Adult hypophosphatasia', 'Odontohypophosphatasia', 'Infantile hypophosphatasia', 'Childhood-onset hypophosphatasia'], ['blood protein measurement', 'body height', 'lean body mass', 'fat body mass', 'DNA methylation'], ['lifestyle measurement', 'cutaneous melanoma', 'squamous cell lung carcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['self reported educational attainment', 'neoplasm', 'risk-taking behaviour', 'body fat percentage', 'prostate cancer'], ['Hypomaturation amelogenesis imperfecta', 'Amelogenesis imperfecta', 'Hypoplastic amelogenesis imperfecta', 'amelogenesis imperfecta, type ij', 'Hypomaturation-hypoplastic amelogenesis imperfecta with taurodontism'], ['Gräsbeck-Imerslund disease', 'Imerslund-Grasbeck syndrome type 2', 'megaloblastic anemia (disease)', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['amyotrophic lateral sclerosis', 'type I diabetes mellitus', 'angiogenin measurement', 'familial amyotrophic lateral sclerosis', 'blood protein measurement'], ['intraocular pressure measurement', 'protein measurement', 'open-angle glaucoma', 'glaucoma', 'mean corpuscular volume'], ['heel bone mineral density', 'intraocular pressure measurement', 'glomerular filtration rate', 'ovarian cancer', 'fallopian tube cancer'], ['food allergy measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['vital capacity', 'forced expiratory volume', 'brain measurement', 'neuroimaging measurement', 'blood protein measurement'], ['Hypobetalipoproteinemia', 'familial hypobetalipoproteinemia 1', 'Homozygous familial hypercholesterolemia', 'cutaneous melanoma', 'triglyceride measurement'], ['triglyceride measurement', 'coronary artery disease', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['intraocular pressure measurement', 'refractive error measurement', 'open-angle glaucoma', 'glaucoma', 'corneal topography'], ['high density lipoprotein cholesterol measurement', 'type II diabetes mellitus', 'type I diabetes mellitus', 'triglyceride measurement', 'total cholesterol measurement'], ['Familial long QT syndrome', 'Romano-Ward syndrome', 'genetic disorder', 'complex neurodevelopmental disorder', 'Death in infancy'], ['chondrocalcinosis 2', 'Craniometaphyseal dysplasia', 'Chondrocalcinosis', 'type II diabetes mellitus', 'glaucoma'], ['Autosomal recessive primary microcephaly', 'Microcephaly', 'mental retardation', 'Hypotonia', 'Intellectual disability'], ['chronotype measurement', 'diet measurement', 'brain volume measurement', 'reticulocyte count', 'putamen volume'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration'], ['Diarrhea', 'eosinophil count', 'appendicular lean mass', 'Recurrent infection of the gastrointestinal tract', 'Chronic diarrhea'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'eosinophil count'], ['intraocular pressure measurement', 'serum alanine aminotransferase measurement', 'blood metabolite measurement', 'gallstones', 'body height'], ['GAPO syndrome', 'anthrax infection', 'Familial capillary hemangioma', 'capillary hemangioma', 'body height'], ['Juvenile hyaline fibromatosis', 'hyaline fibromatosis syndrome', 'anthrax infection', 'lymphocyte count', 'lymphocyte percentage of leukocytes'], ['age at menarche', 'brain volume measurement', 'annexin A1 measurement', 'body height', 'breast adenocarcinoma'], ['lymphocyte count', 'monocyte count', 'lymphocyte percentage of leukocytes', 'breast adenocarcinoma', 'inflammatory bowel disease'], ['type II diabetes mellitus', 'lean body mass', 'appendicular lean mass', 'diabetes mellitus', 'body fat percentage'], ['glioblastoma multiforme', 'atherosclerosis', 'Hypercholesterolemia', 'pulse pressure measurement', "Alzheimer's disease"], ['urinary bladder cancer', 'pulse pressure measurement', 'chronic kidney disease', 'atherosclerosis', 'colorectal carcinoma'], ['atrial fibrillation', 'appendicular lean mass', 'body height', 'polycystic ovary syndrome', 'snoring measurement'], ['KID syndrome', 'ichthyosiform erythroderma, corneal involvement, and deafness', 'HIV infection', 'MEDNIK syndrome', 'Failure to thrive'], ['X-linked intellectual disability - Dandy-Walker malformation - basal ganglia disease - Seizures', 'X-linked syndromic intellectual disability', 'Fried syndrome', 'HIV infection', 'X-linked intellectual disability - hypotonia - facial dysmorphism - aggressive behavior'], ['Generalized pustular psoriasis', 'HIV infection', 'Pustulosis palmaris et plantaris', 'Acrodermatitis continua suppurativa of Hallopeau', 'asthma'], ['intelligence', 'self reported educational attainment', 'adolescent idiopathic scoliosis', 'schizophrenia', 'mathematical ability'], ['white matter microstructure measurement', 'body height', 'sitting height measurement', 'vital capacity', 'monocyte count'], ['Familial adenomatous polyposis', 'colorectal adenocarcinoma', 'Gardner syndrome', 'APC-related attenuated familial adenomatous polyposis', 'desmoid tumor'], ['Hypotrichosis simplex', 'hair shape measurement', 'balding measurement', 'heel bone mineral density', 'Alopecia'], ["Crohn's disease", 'ulcerative colitis', 'inflammatory bowel disease', 'blood protein measurement', 'hepatocyte growth factor-like protein measurement'], ['attention deficit hyperactivity disorder', 'glioma', 'pulmonary arterial hypertension', 'glioblastoma multiforme', 'neoplasm'], ['breast adenocarcinoma', 'endometrial cancer', 'neoplasm', 'cancer', 'hepatocellular carcinoma'], ['neoplasm', 'coronary artery disease', 'hepatocellular carcinoma', 'proliferative diabetic retinopathy', 'glioblastoma multiforme'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'hypertension', 'coronary artery disease'], ['blood protein measurement', 'breast adenocarcinoma', 'pancreatic adenocarcinoma', 'glioblastoma multiforme', 'esophageal cancer'], ['Familial renal amyloidosis', 'Apolipoprotein A-I deficiency', 'Tangier disease', 'AL amyloidosis', 'Hypoalphalipoproteinemia'], ['AL amyloidosis', 'triglyceride measurement', 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement', 'apolipoprotein A-IV measurement'], ['familial hypercholesterolemia', 'Hypobetalipoproteinemia', 'hypercholesterolemia, autosomal dominant, type B', 'familial hypobetalipoproteinemia 1', 'cardiovascular disease'], ['colon adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'cutaneous melanoma', 'hepatocellular carcinoma', 'bladder carcinoma'], ['cutaneous melanoma', 'cancer', 'breast cancer', 'multiple myeloma', 'myeloid white cell count'], ['Familial apolipoprotein C-II deficiency', 'familial chylomicronemia syndrome', 'amyloidosis', 'Hyperlipoproteinemia type 1', 'Hyperlipoproteinemia type 4'], ['Cholesterol-ester transfer protein deficiency', 'coronary artery disease', 'Hyperalphalipoproteinemia', 'platelet crit', 'phosphatidylinositol measurement'], ['Hyperlipoproteinemia type 3', 'Lipoprotein glomerulopathy', 'coronary artery disease', 'Sea-blue histiocytosis', 'Early-onset autosomal dominant Alzheimer disease'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'glomerulonephritis (disease)', 'Proteinuria', 'chronic kidney disease'], ['cutaneous melanoma', 'breast adenocarcinoma', 'coronary artery disease', 'systemic lupus erythematosus', 'Sepsis'], ['Early-onset autosomal dominant Alzheimer disease', "Alzheimer's disease", 'Hereditary cerebral hemorrhage with amyloidosis, Dutch type', 'Hereditary cerebral hemorrhage with amyloidosis, Iowa type', 'Hereditary cerebral hemorrhage with amyloidosis, Italian type'], ['Adenine phosphoribosyltransferase deficiency', 'Purine nucleoside phosphorylase deficiency', 'chronic kidney disease', 'interstitial nephritis', 'red blood cell distribution width'], ['Nephrogenic diabetes insipidus', 'allergy', 'allergic rhinitis', 'asthma', 'cyst'], ['glomerular filtration rate', 'hematocrit', 'hemoglobin measurement', 'chronic kidney disease', 'blood urea nitrogen measurement'], ['Androgen insensitivity syndrome', 'Partial androgen insensitivity syndrome', 'prostate carcinoma', 'Kennedy disease', 'prostate adenocarcinoma'], ['short stature, rhizomelic, with microcephaly, micrognathia, and developmental delay', 'myeloid white cell count', 'total cholesterol measurement', 'Microcephaly', 'breast adenocarcinoma'], ['Argininemia', 'glomerular filtration rate', 'type II diabetes mellitus', 'diabetes mellitus', 'creatinine measurement'], ['glioblastoma multiforme', 'cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'chronic myelogenous leukemia'], ['prostate carcinoma', 'varicose disease', 'vein disease', 'lymphatic system disease', 'Varicose veins'], ['basophil percentage of leukocytes', 'inflammatory bowel disease', 'hepatocellular carcinoma', 'acute myeloid leukemia', 'platelet crit'], ['Juvenile Myelomonocytic Leukemia', 'eosinophil count', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma', 'Hepatobiliary Neoplasm'], ['immunodeficiency 62', 'immunodeficiency disease', 'breast adenocarcinoma', 'Combined immunodeficiency with skin granulomas', 'Neutrophil immunodeficiency syndrome'], ['neurodevelopmental disorder with midbrain and hindbrain malformations', 'thyroid gland adenocarcinoma', 'testosterone measurement', 'breast adenocarcinoma', 'Microcephaly'], ['thyroid gland adenocarcinoma', 'pancreatic adenocarcinoma', 'glioblastoma multiforme', 'body mass index', 'lung adenocarcinoma'], ['breast adenocarcinoma', 'cardiovascular disease', 'glioblastoma multiforme', 'systolic blood pressure', 'hematocrit'], ['sex hormone-binding globulin measurement', 'reticulocyte count', 'diet measurement', 'hearing loss', 'reticulocyte measurement'], ['Coffin-Siris syndrome', 'mental retardation', 'gastric adenocarcinoma', 'colorectal adenocarcinoma', 'urinary bladder cancer'], ['age at menopause', 'glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'gastric adenocarcinoma'], ['acute myeloid leukemia', 'breast adenocarcinoma', 'monocyte count', 'lean body mass', 'endometrial cancer'], ['Joubert syndrome', 'Retinitis pigmentosa', 'neutrophil count', 'prostate carcinoma', 'BMI-adjusted waist-hip ratio'], ['age-related macular degeneration', 'wet macular degeneration', 'atrophic macular degeneration', 'retinopathy', 'diabetic retinopathy'], ['neuroticism measurement', 'worry measurement', 'body mass index', 'waist-hip ratio', 'body fat percentage'], ['Metachromatic leukodystrophy', 'Metachromatic leukodystrophy, juvenile form', 'Metachromatic leukodystrophy, adult form', 'Metachromatic leukodystrophy, late infantile form', 'genetic disorder'], ['Mucopolysaccharidosis type 6', 'Mucopolysaccharidosis', 'Mucopolysaccharidosis type 6, slowly progressing', 'Metachromatic leukodystrophy', 'genetic disorder'], ['attention deficit hyperactivity disorder', 'obesity', 'atrial fibrillation', 'breast cancer', 'Interauricular communication'], ['chronic myelogenous leukemia', "Barrett's esophagus", 'hepatocellular carcinoma', 'psoriasis', 'neoplasm'], ['hepatocellular carcinoma', 'neuroblastoma', 'cancer', 'non-small cell lung carcinoma', 'neoplasm'], ['intelligence', 'self reported educational attainment', 'neoplasm', 'pancreatic carcinoma', 'hepatocellular carcinoma'], ['Farber lipogranulomatosis', 'Hereditary myoclonus - progressive distal muscular atrophy', 'Rolandic epilepsy', 'Keloid', 'Fetal akinesia deformation sequence'], ['blood protein measurement', 'neutral ceramidase measurement', 'cerebrospinal fluid biomarker measurement', 'red blood cell distribution width', 'pulse pressure measurement'], ['mathematical ability', 'self reported educational attainment', 'body mass index', 'acute myeloid leukemia', 'body fat percentage'], ['clear cell renal carcinoma', 'hematocrit', 'red blood cell density measurement', 'bipolar disorder', 'body fat distribution'], ['spinal muscular atrophy with congenital bone fractures 2', 'Fetal akinesia deformation sequence', 'spinal muscular atrophy', 'arthrogryposis', 'contracture'], ['Haddad syndrome', 'Ondine syndrome', 'amino acid measurement', 'phenylalanine measurement', 'sex hormone-binding globulin measurement'], ['mean platelet volume', 'erythrocyte count', 'testosterone measurement', 'platelet count', 'self reported educational attainment'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'apolipoprotein B measurement', 'interleukin-27 measurement', 'HMG CoA reductase inhibitor use measurement'], ['blood protein measurement', 'Hypercholesterolemia', 'red blood cell distribution width', 'X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12RB1 deficiency'], ['breast adenocarcinoma', 'Seizure', 'head and neck squamous cell carcinoma', 'breast cancer', 'nonpapillary renal cell carcinoma'], ['body mass index', 'lean body mass', 'puberty onset measurement', 'fat body mass', 'hypertension'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'mean reticulocyte volume', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['Argininosuccinic aciduria', 'blood protein measurement', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'neutrophil count'], ['amino acid measurement', 'asparagine measurement', 'chronic kidney disease', 'serum metabolite measurement', 'hair colour measurement'], ['vital capacity', 'prostate carcinoma', 'mean platelet volume', 'neoplasm', 'platelet count'], ['alveolar rhabdomyosarcoma', 'alveolar soft part sarcoma', 'renal carcinoma', 'Paraganglioma', 'Chordoma'], ['esophageal adenocarcinoma', "Barrett's esophagus", 'sex interaction measurement', 'waist-hip ratio', 'BMI-adjusted waist-hip ratio'], ['appendicular lean mass', 'body height', 'glioblastoma multiforme', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'lung adenocarcinoma', 'mean corpuscular hemoglobin', 'neurotic disorder'], ['Achromatopsia', 'Cone rod dystrophy', 'Macular dystrophy', 'basophil percentage of leukocytes', 'drug use measurement'], ['complement C4 measurement', 'self reported educational attainment', 'mean corpuscular volume', 'Peeling skin syndrome type A', 'Acral peeling skin syndrome'], ['AICA-ribosiduria', 'Intellectual disability', 'anaplastic large cell lymphoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['Ataxia-telangiectasia', 'Hereditary breast cancer', 'colorectal adenocarcinoma', 'breast carcinoma', 'prostate adenocarcinoma'], ['facial morphology measurement', 'colorectal cancer', 'colorectal adenoma', 'hearing loss', 'intelligence'], ['gastroesophageal reflux disease', 'gastric ulcer', 'Helicobacter pylori infectious disease', 'duodenal ulcer', 'Peptic ulcer'], ['hematocrit', 'hair color', 'hemoglobin measurement', 'aspartate aminotransferase measurement', 'eosinophil count'], ['mitochondrial complex V deficiency, nuclear type 6', 'Isolated ATP synthase deficiency', 'Leigh syndrome', 'malaria', 'Sepsis'], ['immunodeficiency 47', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Granular Cell Tumor', 'cancer'], ['X-linked intellectual disability, Hedera type', 'X-linked parkinsonism-spasticity syndrome', 'X-linked intellectual disability - epilepsy', 'congenital disorder of glycosylation type II', 'ATP6AP2-related disorder'], ['Wilson disease', 'dystonic disorder', 'genetic disorder', 'mean corpuscular volume', 'Epileptic encephalopathy'], ['Dysequilibrium syndrome', 'Rare hereditary ataxia', 'Intellectual disability', 'genetic disorder', 'fatty acid measurement'], ['Progressive familial intrahepatic cholestasis', 'Benign recurrent intrahepatic cholestasis', 'Progressive familial intrahepatic cholestasis type 1', 'Benign recurrent intrahepatic cholestasis type 1', 'Intrahepatic cholestasis of pregnancy'], ['systolic blood pressure', 'cystic fibrosis associated meconium ileus', 'diastolic blood pressure', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['Seckel syndrome', 'Seckel syndrome 1', 'Familial cutaneous telangiectasia and oropharyngeal predisposition cancer syndrome', 'cutaneous melanoma', 'lung carcinoma'], ['Alpha-thalassemia - X-linked intellectual disability syndrome', 'X-linked intellectual disability - hypotonic face', 'Renier-Gabreels-Jasper syndrome', 'Smith-Fineman-Myers syndrome', 'Chudley-Lowry-Hoar syndrome'], ['Spinocerebellar ataxia type 2', 'hemoglobin measurement', 'reticulocyte count', 'hematocrit', 'lymphocyte count'], ['Spinocerebellar ataxia type 3', 'Machado-Joseph disease', 'Machado-Joseph disease type 3', 'Machado-Joseph disease type 1', 'Machado-Joseph disease type 2'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'multiple myeloma', 'mean corpuscular hemoglobin', 'unspecified peripheral T-cell lymphoma'], ['NK-cell enteropathy', 'acute myeloid leukemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer'], ['Male infertility due to large-headed multiflagellar polyploid spermatozoa', 'Male infertility with teratozoospermia due to single gene mutation', 'cancer', 'Non-syndromic male infertility due to sperm motility disorder', 'leukemia'], ['Hereditary central diabetes insipidus', 'Central diabetes insipidus', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma'], ['Nephrogenic diabetes insipidus', 'Nephrogenic syndrome of inappropriate antidiuresis', 'Hyponatremia', 'hemorrhage', 'heart failure'], ['head and neck squamous cell carcinoma', 'adolescent idiopathic scoliosis', 'rosacea', 'obesity', 'psoriasis'], ['hepatocellular carcinoma', 'cancer', 'liver cancer', 'hepatocellular adenoma', 'Caudal duplication'], ['acute myeloid leukemia', 'neoplasm', 'erythrocyte count', 'electrocardiography', 'mean corpuscular volume'], ['gut microbiome measurement', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia', 'colorectal carcinoma', 'Hyperinsulinism due to glucokinase deficiency'], ['Immunodeficiency by defective expression of HLA class 1', 'Familial renal amyloidosis', 'HIV infection', 'non-Hodgkins lymphoma', 'AL amyloidosis'], ['prostate carcinoma', 'insomnia measurement', 'N-glycan measurement', 'alkaline phosphatase measurement', 'body fat percentage'], ['testosterone measurement', 'erythrocyte count', 'mean corpuscular volume', 'red blood cell distribution width', 'ankylosing spondylitis'], ['eosinophil count', 'lymphocyte count', 'monocyte count', 'neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes'], ['B4GALT1-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type II', 'urate measurement', 'serum IgG glycosylation measurement'], ["Alzheimer's disease", 'mean platelet volume', 'apolipoprotein A 1 measurement', 'diastolic blood pressure', 'separation anxiety disorder'], ['C-peptide measurement', 'body mass index', 'self reported educational attainment', 'mathematical ability', 'cognitive function measurement'], ['platelet count', 'platelet crit', 'eosinophil count', 'chronic lymphocytic leukemia', 'mean platelet volume'], ['heel bone mineral density', 'appendicular lean mass', 'sitting height measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['Nestor-Guillermo progeria syndrome', 'HIV infection', 'body height', 'cerebrospinal fluid biomarker measurement', 'lymphocyte count'], ['colorectal adenocarcinoma', 'colonic neoplasm', 'T-cell acute lymphoblastic leukemia', 'colon adenocarcinoma', 'bile duct carcinoma'], ['Bardet-Biedl syndrome 9', 'Bardet-Biedl syndrome', 'polydactyly (disease)', 'obesity', 'Retinal dystrophy'], ['Severe motor and intellectual disabilities-sensorineural deafness-dystonia syndrome', 'CADDS', 'genetic disorder', 'breast adenocarcinoma', 'endometrial cancer'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'cutaneous melanoma', 'drug use measurement'], ['Butyrylcholinesterase deficiency', "Alzheimer's disease", 'Dementia', "Parkinson's disease", 'delirium'], ['Autism-epilepsy syndrome due to branched chain ketoacid dehydrogenase kinase deficiency', 'Intellectual disability', 'mental retardation', 'body mass index', "Parkinson's disease"], ['chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'acute myeloid leukemia', 'lymphocyte count'], ['eosinophil count', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'small cell lung carcinoma'], ['platelet count', 'small cell lung carcinoma', 'eosinophil count', 'primary myelofibrosis', 'platelet crit'], ['cancer', 'prostate carcinoma', 'monocyte count', 'asthma', 'red blood cell density measurement'], ['diffuse large B-cell lymphoma', 'chronic lymphocytic leukemia', 'lymphoid neoplasm', 'lung carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['colorectal adenocarcinoma', 'melanoma', 'breast ductal adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'colon adenocarcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'MALT lymphoma', 'neoplasm of mature B-cells', 'colorectal adenocarcinoma', 'mesothelioma'], ['Dias-Logan syndrome', 'Intellectual disability', 'fetal hemoglobin measurement', 'self reported educational attainment', 'mean corpuscular volume'], ['Oculofaciocardiodental syndrome', 'Microphthalmia, Lenz type', 'chronic lymphocytic leukemia', 'gastric adenocarcinoma', 'colorectal adenocarcinoma'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cancer', 'myelodysplastic syndrome', 'leukemia'], ['Björnstad syndrome', 'Isolated CoQ-cytochrome C reductase deficiency', 'GRACILE syndrome', 'mitochondrial complex III deficiency nuclear type 1', 'Renal tubulopathy - encephalopathy - liver failure'], ['reticulocyte count', 'visual perception measurement', 'event death', 'neoplasm', 'benign prostatic hyperplasia'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma'], ['Hereditary angioedema', 'angioedema', 'heart failure', 'subcutaneous adipose tissue measurement', 'ovarian cancer'], ['obesity', 'body mass index', 'lean body mass', 'body fat percentage', 'fat body mass'], ['Best vitelliform macular dystrophy', 'Autosomal recessive bestrophinopathy', 'Retinitis pigmentosa', 'MRCS syndrome', 'Autosomal dominant vitreoretinochoroidopathy'], ['Ehlers-Danlos syndrome, progeroid type', 'hepatocellular carcinoma', 'type II diabetes mellitus', 'carotid atherosclerosis', 'diabetes mellitus'], ['intellectual developmental disorder and retinitis pigmentosa; IDDRP', 'Down syndrome', 'brain volume measurement', 'age at menopause', 'systolic blood pressure'], ['Multicystic dysplastic kidney', 'urate measurement', 'refractive error measurement', 'hematocrit', 'hemoglobin measurement'], ['prostate carcinoma', 'red blood cell density measurement', 'erythrocyte count', 'mean corpuscular hemoglobin', 'mean platelet volume'], ['sex hormone-binding globulin measurement', 'vital capacity', 'FEV/FEC ratio', 'lymphocyte count', 'chronic lymphocytic leukemia'], ['Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta', 'High bone mass osteogenesis imperfecta', 'breast adenocarcinoma', 'cutaneous melanoma'], ['Brachydactyly type A2', 'colorectal cancer', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'sitting height measurement'], ['Microphthalmia with brain and digit anomalies', 'Orofacial clefting syndrome', 'heel bone mineral density', 'colorectal cancer', 'refractive error measurement'], ['body height', 'lean body mass', 'appendicular lean mass', 'vital capacity', 'forced expiratory volume'], ['refractive error measurement', 'corneal topography', 'genetic disorder', 'metopic craniosynostosis', 'nose morphology measurement'], ['heel bone mineral density', 'hemorrhoid', 'rectal disease', 'anus disease', 'blood protein measurement'], ['hematocrit', 'hemoglobin measurement', 'colorectal neoplasm', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['Brachydactyly type A2', 'acromesomelic dysplasia, Demirhan type', 'Brachydactyly type A1', 'Acromesomelic dysplasia', 'Fibular aplasia - complex brachydactyly'], ['Idiopathic and/or familial pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'Pulmonary venoocclusive disease', 'Heritable pulmonary arterial hypertension', 'Pulmonary arterial hypertension associated with congenital heart disease'], ['Circumscribed cutaneous aplasia of the vertex', 'aplasia cutis congenita (disease)', 'proto-oncogene tyrosine-protein kinase receptor Ret measurement', 'hip fracture', 'femur fracture'], ['alopecia areata', 'breast adenocarcinoma', 'ovarian cancer', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['FEV/FEC ratio', 'body height', 'sleep duration', 'low density lipoprotein cholesterol measurement', 'adolescent idiopathic scoliosis'], ['Cardiofaciocutaneous syndrome', 'LEOPARD syndrome', 'Noonan syndrome', 'cutaneous melanoma', 'melanoma'], ['Hereditary breast and ovarian cancer syndrome', 'breast carcinoma', 'Hereditary breast cancer', 'Fanconi anemia, complementation group S', 'breast-ovarian cancer, familial, susceptibility to, 1'], ['breast carcinoma', 'Hereditary breast and ovarian cancer syndrome', 'Hereditary breast cancer', 'Fanconi anemia complementation group D1', 'Fanconi anemia'], ['coronary artery disease', 'multiple sclerosis', 'schizophrenia', 'neoplasm', 'lymphocyte percentage of leukocytes'], ['colorectal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'lung carcinoma', 'colon adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['Intellectual disability', 'non-small cell lung carcinoma', 'NUT midline carcinoma', 'lymphoid neoplasm', 'squamous cell lung carcinoma'], ['liver cancer', 'cutaneous melanoma', 'inflammatory bowel disease', "Crohn's disease", 'granular cell cancer'], ['neoplasm', 'cancer', 'acute myeloid leukemia', 'ovarian cancer', 'hepatocellular carcinoma'], ['Male infertility with teratozoospermia due to single gene mutation', 'primary ovarian insufficiency', 'monocyte count', 'coronary artery disease', 'smoking status measurement'], ['Inborn errors of metabolism', 'breast adenocarcinoma', 'low grade glioma', 'neoplasm', 'hepatocellular carcinoma'], ['cancer', 'chronic obstructive pulmonary disease', 'FEV/FEC ratio', 'smoking status measurement', 'forced expiratory volume'], ['X-linked agammaglobulinemia', 'Isolated agammaglobulinemia', 'Non-acquired isolated growth hormone deficiency', 'chronic lymphocytic leukemia', 'Mantle cell lymphoma'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'coronary artery disease', 'neoplasm', 'rheumatoid arthritis'], ['cutaneous melanoma', 'cognitive function measurement', 'intelligence', 'self reported educational attainment', 'mathematical ability'], ['blood protein measurement', "Barrett's esophagus", 'esophageal adenocarcinoma', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['Mosaic variegated aneuploidy syndrome', 'colonic neoplasm', 'colorectal adenocarcinoma', 'acute myeloid leukemia', 'multiple myeloma'], ['hepatocellular carcinoma', 'neoplasm', 'alcohol consumption measurement', 'platelet count', 'alcohol use disorder measurement'], ['chronotype measurement', 'self reported educational attainment', 'cancer', 'anxiety', 'cardiovascular disease'], ['Ehlers-Danlos syndrome, periodontitis type', 'complement component C1s deficiency', 'Immunodeficiency due to an early component of complement deficiency', 'Hereditary angioedema', 'systemic lupus erythematosus'], ['Immunodeficiency due to an early component of complement deficiency', 'age-related macular degeneration', 'lupus erythematosus', 'Immunodeficiency due to a complement cascade protein anomaly', 'atherosclerosis'], ['Atypical hemolytic-uremic syndrome', 'Complement component 3 deficiency', 'Atypical hemolytic-uremic syndrome with C3 anomaly', 'age-related macular degeneration', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis'], ['platelet count', 'chronotype measurement', 'body mass index', 'eosinophil count', 'BMI-adjusted waist circumference'], ['Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder', 'thioredoxin domain-containing protein 12 measurement', 'Isochromosomy Yp', 'Isolated follicle stimulating hormone deficiency'], ['Immunodeficiency due to a late component of complements deficiency', 'Paroxysmal nocturnal hemoglobinuria', 'Atypical hemolytic-uremic syndrome', 'immune system disease', 'Erythroderma desquamativum'], ['anti-neutrophil antibody associated vasculitis', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['ovarian cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', "Alzheimer's disease"], ['Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'Huntington disease-like syndrome due to C9ORF72 expansions', 'cathepsin L1 measurement', 'sporadic amyotrophic lateral sclerosis'], ['decreased susceptibility to bacterial infection', 'sitting height ratio', 'breast cancer', 'neoplasm', 'Hereditary renal hypouricemia'], ['influenza A (H1N1)', 'colorectal adenocarcinoma', 'colon carcinoma', 'malignant colon neoplasm', 'colonic neoplasm'], ['Ataxia-telangiectasia', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'Ataxia-telangiectasia variant'], ['Temtamy syndrome', 'Global developmental delay', 'Intellectual disability', 'mental retardation', 'attention deficit hyperactivity disorder'], ['glaucoma', 'altitude sickness', 'angle-closure glaucoma', 'Seizure', 'chronic obstructive pulmonary disease'], ['Osteopetrosis with renal tubular acidosis', 'epilepsy', 'glaucoma', 'ocular hypertension', 'altitude sickness'], ['mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'blood protein measurement', 'vital capacity', 'grip strength measurement'], ['carbonic anhydrase 6 measurement', 'blood protein measurement', 'obesity', 'protein measurement', 'synovial sarcoma'], ['Dysequilibrium syndrome', 'Rare hereditary ataxia', 'erythrocyte count', 'thyroxine measurement', 'body height'], ['low grade glioma', 'breast adenocarcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'gastric adenocarcinoma'], ['BMI-adjusted waist circumference', 'body height', 'lean body mass', 'physical activity measurement', 'platelet count'], ['blood protein measurement', 'forced expiratory volume', 'peak expiratory flow', 'platelet endothelial cell adhesion molecule measurement', 'vital capacity'], ['Seizure', 'neurodevelopmental disorder with seizures and non-epileptic hyperkinetic movements', 'epilepsy', 'Bilateral tonic-clonic seizure', 'Intellectual disability'], ['Childhood absence epilepsy', 'Familial hyperaldosteronism', 'epilepsy', 'cardiovascular disease', 'Polyuria'], ['Autosomal dominant non-syndromic intellectual disability', 'cardiovascular disease', 'Urinary incontinence', 'Polyuria', 'generalized anxiety disorder'], ['lean body mass', 'body weight', 'appendicular lean mass', 'body height', 'fat body mass'], ['body weight', 'body mass index', 'lean body mass', 'fasting blood glucose measurement', 'base metabolic rate measurement'], ['protein measurement', 'smoking status measurement', 'clear cell renal carcinoma', 'papillary renal cell carcinoma', 'Dravet syndrome'], ['migraine disorder', 'AL amyloidosis', 'trigeminal neuralgia', 'pain', 'glioblastoma multiforme'], ['type II diabetes mellitus', 'type I diabetes mellitus', 'postmenopausal osteoporosis', 'bone fracture', 'diabetes mellitus'], ['migraine disorder', 'BMI-adjusted waist-hip ratio', 'lymphatic malformation 8', 'physical activity measurement', 'waist-hip ratio'], ['FEV/FEC ratio', 'alkaline phosphatase measurement', 'acute myeloid leukemia', 'sitting height measurement', 'body height'], ['self reported educational attainment', 'FEV/FEC ratio', 'atrial fibrillation', 'mastitis', 'rheumatoid arthritis'], ['essential thrombocythemia', 'myeloproliferative disorder', 'influenza', 'Myelofibrosis', 'acute myeloid leukemia'], ['electrocardiography', 'atrial fibrillation', 'PR interval', 'chronic lymphocytic leukemia', "Parkinson's disease"], ['intellectual developmental disorder 62', 'Intellectual disability, severe', 'Autosomal dominant non-syndromic intellectual disability', 'autism', 'Global developmental delay'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'age at menarche', 'mathematical ability', 'hypothyroidism'], ['lung adenocarcinoma', 'endometrial cancer', 'neoplasm', 'systemic lupus erythematosus', 'infection'], ['platelet count', 'breast adenocarcinoma', 'cutaneous melanoma', 'leukocyte count', 'neoplasm'], ['mean platelet volume', 'platelet component distribution width', 'platelet count', 'urate measurement', 'adolescent idiopathic scoliosis'], ['Autosomal recessive limb-girdle muscular dystrophy type 2A', 'muscular dystrophy, limb-girdle, autosomal dominant 4', 'Muscular dystrophy', 'Muscle weakness', 'Shoulder girdle muscle weakness'], ['forced expiratory volume', 'vital capacity', 'balding measurement', 'systemic lupus erythematosus', 'colorectal carcinoma'], ['inflammatory bowel disease', 'gout', 'non-small cell lung carcinoma', 'bacteriemia', 'cardiovascular disease'], ['psoriasis', 'Pityriasis rubra pilaris', 'acneiform dermatitis', 'familial pityriasis rubra pilaris', 'Generalized pustular psoriasis'], ['interleukin 18 measurement', 'infect', 'infection', 'visual perception measurement', 'neuroblastoma'], ['microcephaly, developmental delay, and brittle hair syndrome', 'colorectal adenocarcinoma', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'Endometrial Endometrioid Adenocarcinoma'], ['mucocutaneous lymph node syndrome', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'lung adenocarcinoma', 'hepatocellular carcinoma'], ['ovarian cancer', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma'], ['Autoimmune lymphoproliferative syndrome with recurrent viral infections', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'basal cell carcinoma', 'viral disease'], ['lung adenocarcinoma', 'glioblastoma multiforme', 'ovarian cancer', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['Familial hypocalciuric hypercalcemia type 1', 'Autosomal dominant hypocalcemia', 'Familial hypocalciuric hypercalcemia', 'Bartter syndrome with hypocalcemia', 'Neonatal severe primary hyperparathyroidism'], ['body mass index', 'lean body mass', "Alzheimer's disease", 'Birt-Hogg-Dubé syndrome', 'appendicular lean mass'], ['acatalasia', 'Acatalasemia', 'Peroxisomal disease', 'catalase measurement', 'blood protein measurement'], ['body height', 'acute myeloid leukemia', 'intraocular pressure measurement', 'lymphocyte percentage of leukocytes', 'lymphocyte count'], ['Noonan syndrome-like disorder with juvenile myelomonocytic leukemia', 'Juvenile Myelomonocytic Leukemia', 'Noonan syndrome', 'cancer', 'listeriosis'], ['Congenital intrinsic factor deficiency', 'Gräsbeck-Imerslund disease', 'neoplasm', 'diffuse large B-cell lymphoma', 'infect'], ['colorectal adenocarcinoma', 'pancreatic adenocarcinoma', 'non-small cell lung carcinoma', 'cancer', 'lung cancer'], ['Classical homocystinuria', 'Methylmalonic acidemia with homocystinuria', 'Abnormality of the cardiovascular system', 'homocystinuria (disease)', 'homocysteine measurement'], ['Familial exudative vitreoretinopathy', 'Berardinelli-Seip congenital lipodystrophy', 'Autosomal dominant neovascular inflammatory vitreoretinopathy', 'X-linked retinoschisis', 'X-linked retinal dysplasia'], ['viral disease', 'endometrial carcinoma', 'triglyceride measurement', 'lean body mass', 'aspartate aminotransferase measurement'], ['lymphocyte count', 'erythrocyte count', 'red blood cell density measurement', 'leukocyte count', 'hematocrit'], ['lean body mass', 'fat body mass', 'monocyte count', 'risk-taking behaviour', 'testosterone measurement'], ['PEHO-like syndrome', 'self reported educational attainment', 'body height', 'forced expiratory volume', 'appendicular lean mass'], ['Eczema', 'allergic rhinitis', 'response to reverse transcriptase inhibitor', 'prostate carcinoma', 'type II diabetes mellitus'], ['body mass index', 'physical activity measurement', 'fat body mass', 'lean body mass', 'visceral adipose tissue measurement'], ["Crohn's disease", 'inflammatory bowel disease', 'monocyte percentage of leukocytes', 'ulcerative colitis', 'psoriasis'], ['C-C motif chemokine 3 measurement', 'blood protein measurement', 'CCL4 measurement', 'visual perception measurement', 'neoplasm'], ['breast adenocarcinoma', 'neoplasm', 'breast cancer', 'infect', 'infection'], ['monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'inflammatory bowel disease', "Crohn's disease"], ['monocyte chemotactic protein-2 measurement', 'C-C motif chemokine 7 measurement', 'C-C motif chemokine 8 measurement', 'blood protein measurement', 'protein measurement'], ['psoriasis', 'acute myeloid leukemia', 'breast adenocarcinoma', 'monocyte count', 'gastric adenocarcinoma'], ['blood protein measurement', 'C-C motif chemokine 16 measurement', 'protein measurement', 'obesity', 'endometriosis'], ['eosinophil count', 'blood protein measurement', 'C-C motif chemokine 17 measurement', 'neoplasm', 'mucocutaneous lymph node syndrome'], ['blood protein measurement', 'C-C motif chemokine 18 measurement', 'obesity', 'protein measurement', 'C-C motif chemokine 3-like 1 measurement'], ['ulcerative colitis', 'asthma', 'allergy', 'childhood onset asthma', 'age at onset'], ['rheumatoid arthritis', 'lymphocyte count', 'ankylosing spondylitis', 'ulcerative colitis', 'sclerosing cholangitis'], ['asthma', 'systemic lupus erythematosus', 'neoplasm', 'red blood cell density measurement', 'hematocrit'], ["Alzheimer's disease", 'atopic eczema', 'basophil count', 'basophil percentage of leukocytes', 'eosinophilic esophagitis'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'lymphocyte count', 'prostate adenocarcinoma'], ['Hereditary cerebral cavernous malformation', 'Cavernous Hemangioma', 'Capillary malformation - arteriovenous malformation', 'diastolic blood pressure', 'neutrophil count'], ['idiopathic pulmonary fibrosis', 'Duchenne muscular dystrophy', 'blood protein measurement', 'pancreatic carcinoma', 'neoplasm'], ['diastolic blood pressure', 'body height', 'lean body mass', 'pulse pressure measurement', 'systolic blood pressure'], ['cancer', 'T-cell leukemia', 'esophageal adenocarcinoma', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['multiple myeloma', 'AL amyloidosis', 'endometrial cancer', 'gastric carcinoma', 'cutaneous melanoma'], ['diffuse large B-cell lymphoma', 'mean corpuscular volume', 'T-cell acute lymphoblastic leukemia', 'breast ductal adenocarcinoma', 'mean corpuscular hemoglobin'], ['gastric carcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'basal cell carcinoma', 'bladder carcinoma', 'lung adenocarcinoma'], ['head and neck squamous cell carcinoma', 'esophageal cancer', 'glioblastoma multiforme', 'endometrial cancer', 'low grade glioma'], ['basophil count', 'celiac disease', 'glioblastoma multiforme', 'basophil percentage of leukocytes', 'mean corpuscular hemoglobin'], ['HIV-1 infection', 'monocyte count', 'non-alcoholic steatohepatitis', 'COVID-19', 'monocyte chemotactic protein 1 measurement'], ['celiac disease', 'glioblastoma multiforme', 'ovarian cancer', "Behcet's syndrome", 'daytime rest measurement'], ['neoplasm', 'mycosis fungoides', "Sezary's disease", 'Cutaneous T-cell lymphoma', 'tropical spastic paraparesis'], ['HIV infection', 'HIV-1 infection', 'AIDS', 'infection', 'type I diabetes mellitus'], ['rheumatoid arthritis', 'ACPA-positive rheumatoid arthritis', 'eosinophil count', 'serum non-albumin protein measurement', 'albumin:globulin ratio measurement'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'monocyte count', 'head and neck squamous cell carcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'Thyroid preparation use measurement', 'myxedema', 'head and neck squamous cell carcinoma'], ["Crohn's disease", 'CCL4 measurement', 'ovarian cancer', 'Hirschsprung disease', 'C-X-C motif chemokine 16 measurement'], ['HIV-1 infection', 'CCL4 measurement', 'monocyte percentage of leukocytes', 'macrophage inflammatory protein 1b measurement', 'viral load'], ['breast adenocarcinoma', 'Neurodegeneration', 'amyotrophic lateral sclerosis', 'breast cancer', 'forced expiratory volume'], ['ovarian cancer', 'gastric adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['neurofilament light chain measurement', 'pancreatic adenocarcinoma', 'neoplasm', 'psoriasis', 'Langerhans Cell Histiocytosis'], ['Townes-Brocks syndrome', 'chronic obstructive pulmonary disease', 'QRS duration', 'multiple sclerosis', 'neoplasm'], ['rheumatoid arthritis', 'refractory anemia', 'infection', 'infect', 'systemic lupus erythematosus'], ['psoriasis', 'atopic eczema', 'immune system disease', 'Pityriasis rubra pilaris', 'rheumatoid arthritis'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'focal segmental glomerulosclerosis', "Alzheimer's disease", 'reticulocyte count', 'red blood cell distribution width'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'T-B+ severe combined immunodeficiency due to CD3delta/CD3epsilon/CD3zeta', 'acute lymphoblastic leukemia', 'neoplasm', 'B-cell acute lymphoblastic leukemia'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'immunodeficiency 18', 'T-B+ severe combined immunodeficiency due to CD3delta/CD3epsilon/CD3zeta', 'acute lymphoblastic leukemia', 'neoplasm'], ['Combined immunodeficiency due to CD3gamma deficiency', 'cutaneous Leishmaniasis', 'acute lymphoblastic leukemia', 'Primary immunodeficiency due to a defect in adaptive immunity', 'neoplasm'], ['HIV infection', 'HIV-1 infection', 'Okt4 epitope deficiency', 'viral disease', 'mycosis fungoides'], ['ulcerative colitis', "Crohn's disease", 'psoriasis', 'inflammatory bowel disease', 'ankylosing spondylitis'], ['Susceptibility to respiratory infections associated with CD8alpha chain mutation', 'blood protein measurement', 'inflammatory bowel disease', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume'], ['diphtheria', 'mean platelet volume', 'platelet count', 'platelet component distribution width', 'breast adenocarcinoma'], ['Primary immunodeficiency', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['Common variable immunodeficiency', 'neoplasm', 'acute lymphoblastic leukemia', 'diffuse large B-cell lymphoma', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency'], ['acute lymphoblastic leukemia', 'neoplasm', 'hairy cell leukemia', 'B-cell acute lymphoblastic leukemia', 'systemic lupus erythematosus'], ['multiple sclerosis', 'neoplasm', 'hepatocellular carcinoma', 'breast cancer', 'cancer'], ['Autosomal recessive lymphoproliferative disease', 'Lymphoproliferative syndrome', 'Combined T and B cell immunodeficiency', 'X-Linked Combined Immunodeficiency Diseases', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['HIV infection', 'eosinophil count', 'rheumatoid arthritis', 'cancer', 'celiac disease'], ['acute myeloid leukemia', 'neoplasm', "Alzheimer's disease", 'platelet count', 'neutrophil count'], ['risk-taking behaviour', 'heart rate', 'lean body mass', 'diet measurement', 'glomerular filtration rate'], ['coronary artery disease', 'platelet glycoprotein VI measurement', 'Primary immunodeficiency', 'inherited bleeding disorder, platelet-type', 'Cognitive impairment'], ['inflammatory bowel disease', 'acute myeloid leukemia', 'chronic lymphocytic leukemia', 'Hyper-IgM syndrome type 3', 'Hepatic veno-occlusive disease - immunodeficiency'], ['multiple myeloma', 'neoplasm', 'ADP-ribosyl cyclase/cyclic ADP-ribose hydrolase 2 measurement', 'amyloidosis', 'asymptomatic myeloma'], ['Hyper-IgM syndrome type 3', 'Hyper-IgM syndrome with susceptibility to opportunistic infections', 'CD40 measurement', 'rheumatoid arthritis', "Crohn's disease"], ['X-linked hyper-IgM syndrome', 'Hyper-IgM syndrome with susceptibility to opportunistic infections', 'hyper-IgM syndrome', 'genetic disorder', 'thyroid gland adenocarcinoma'], ['calcium oxalate urolithiasis', 'systemic lupus erythematosus', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with MCP/CD46 anomaly', 'response to vaccine', 'leukocyte count', 'resting heart rate'], ['bipolar disorder', 'mathematical ability', 'Abnormality of refraction', 'smoking behaviour measurement', 'Tourette syndrome'], ['breast adenocarcinoma', 'monocyte count', 'protein measurement', 'head and neck squamous cell carcinoma', 'tropical spastic paraparesis'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'acute graft vs. host disease', 'chronic lymphocytic leukemia', 'immune system disease'], ['lymphocyte count', 'eosinophil count', 'monocyte count', 'non-Hodgkins lymphoma', 'Beta blocking agent use measurement'], ['Protein-losing enteropathy', 'Primary intestinal lymphangiectasia', 'complement decay-accelerating factor measurement', 'refractive error measurement', 'blood protein measurement'], ['multiple sclerosis', 'primary biliary cirrhosis', 'systemic lupus erythematosus', 'non-Hodgkins lymphoma', 'diastolic blood pressure'], ['Primary CD59 deficiency', 'Paroxysmal nocturnal hemoglobinuria', 'hemolytic anemia', 'hemolysis', 'blood protein measurement'], ['glioblastoma multiforme', 'childhood onset asthma', 'neoplasm', 'melanoma', 'breast cancer'], ['protein measurement', 'platelet component distribution width', 'red blood cell distribution width', 'metabolic syndrome', 'neoplasm'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'myeloid white cell count', 'monocyte count', 'neutrophil percentage of leukocytes'], ['Autosomal recessive lymphoproliferative disease', 'Lymphoproliferative syndrome', 'Combined T and B cell immunodeficiency', 'lymphoma', 'asthma'], ['coronary artery disease', 'lung adenocarcinoma', 'mean platelet volume', 'Activated PIK3-delta syndrome', 'Combined immunodeficiency with skin granulomas'], ['non-small cell lung carcinoma', 'lung adenocarcinoma', 'lymphoid neoplasm', 'squamous cell lung carcinoma', 'acute myeloid leukemia'], ['Autosomal agammaglobulinemia', 'Isolated agammaglobulinemia', 'Agammaglobulinemia', 'diffuse large B-cell lymphoma', 'colorectal adenocarcinoma'], ['Autosomal agammaglobulinemia', 'diffuse large B-cell lymphoma', 'Agammaglobulinemia', 'Isolated agammaglobulinemia', 'Central Nervous System Lymphoma'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'immune system disease', 'dermatomyositis', 'chronic kidney disease'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'glomerulonephritis (disease)', 'type II diabetes mellitus', 'infection'], ['white matter microstructure measurement', 'red blood cell distribution width', 'reticulocyte count', 'reticulocyte measurement', 'lymphocyte count'], ['rheumatoid arthritis', 'caffeine metabolite measurement', 'self reported educational attainment', 'reticulocyte count', 'waist-hip ratio'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'immune system disease', 'dermatomyositis', 'chronic kidney disease'], ['urate measurement', 'uric acid measurement', 'gout', 'drug use measurement', 'irritable bowel syndrome'], ['high density lipoprotein cholesterol measurement', 'sex hormone-binding globulin measurement', 'creatine kinase measurement', 'cutaneous melanoma', 'squamous cell lung carcinoma'], ['eosinophil count', 'blood protein measurement', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'basal cell carcinoma'], ['eosinophil count', 'blood protein measurement', 'asthma', 'reticulocyte count', 'sum of eosinophil and basophil counts'], ['cortisol measurement', 'Eczema', 'allergic rhinitis', 'eosinophil percentage of leukocytes', 'seasonal allergic rhinitis'], ['lung adenocarcinoma', 'cutaneous melanoma', 'thyroid gland adenocarcinoma', 'colorectal adenocarcinoma', 'nasopharyngeal squamous cell carcinoma'], ['mean platelet volume', 'type I diabetes mellitus', 'reticulocyte count', 'eosinophil count', 'platelet component distribution width'], ['vitamin D measurement', 'triglyceride measurement', 'self reported educational attainment', 'physical activity measurement', 'neoplasm'], ['non-small cell lung carcinoma', 'neoplasm', 'cancer', 'Merkel cell skin cancer', 'bladder carcinoma'], ['serum gamma-glutamyl transferase measurement', 'sex hormone-binding globulin measurement', 'serum alanine aminotransferase measurement', 'aspartate aminotransferase measurement', 'neoplasm'], ['myelodysplastic syndrome', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count', 'lymphocyte count'], ['vital capacity', 'drug use measurement', 'PR interval', "Parkinson's disease", 'alcohol consumption measurement'], ['Congenital dyserythropoietic anemia type I', 'Congenital dyserythropoietic anemia', 'bipolar I disorder', 'Melorheostosis', 'postmenopausal osteoporosis'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'endometrial cancer', 'ovarian cancer', 'lung adenocarcinoma'], ['ovarian cancer', 'breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'clear cell renal carcinoma'], ["Alzheimer's disease", 'systolic blood pressure', 'lung adenocarcinoma', 'diastolic blood pressure', 'endometrial cancer'], ['FEV/FEC ratio', 'type II diabetes mellitus', 'chronic obstructive pulmonary disease', 'forced expiratory volume', 'electrocardiography'], ['endometrial cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Congenital hypothyroidism', 'atrophy of thyroid'], ['macula measurement', 'body mass index', 'blood protein measurement', 'neoplasm', 'cancer'], ['Familial gastric cancer', 'Blepharo-cheilo-odontic syndrome', 'hereditary diffuse gastric adenocarcinoma', 'gastric adenocarcinoma', 'Hereditary breast cancer'], ['agenesis of corpus callosum, cardiac, ocular, and genital syndrome', 'Agenesis of corpus callosum', 'self reported educational attainment', 'mathematical ability', 'Familial isolated arrhythmogenic ventricular dysplasia, right dominant form'], ['EEM syndrome', 'Hypotrichosis with juvenile macular degeneration', 'Retinal dystrophy', 'Macular dystrophy', 'macular degeneration'], ['reticulocyte count', 'breast adenocarcinoma', 'endometrial cancer', 'FEV/FEC ratio', 'glioblastoma multiforme'], ['heel bone mineral density', 'protein measurement', 'blood protein measurement', 'serum gamma-glutamyl transferase measurement', 'breast adenocarcinoma'], ['Autosomal dominant non-syndromic intellectual disability', 'Rare genetic intellectual disability with developmental anomaly', 'endometrial cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['pulse pressure measurement', 'asthma', 'response to diisocyanate', 'electrocardiography', 'systolic blood pressure'], ['chronotype measurement', 'diet measurement', 'acute myeloid leukemia', 'self reported educational attainment', 'Ovarian Endometriosis'], ['chronic lymphocytic leukemia', 'ovarian cancer', 'small cell lung carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['chronic lymphocytic leukemia', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma'], ['Familial melanoma', 'cutaneous melanoma', 'melanoma, cutaneous malignant, susceptibility to, 3', 'breast carcinoma', 'breast cancer'], ["Alzheimer's disease", 'Lissencephaly with cerebellar hypoplasia', 'chronic lymphocytic leukemia', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['Autosomal recessive primary microcephaly', 'Isolated congenital microcephaly', 'Microcephaly', 'amyloidosis', 'rheumatoid arthritis'], ['vital capacity', 'generalised epilepsy', 'hepatocellular carcinoma', 'neoplasm', 'gastric cancer'], ['breast carcinoma', 'neoplasm', 'Autosomal recessive primary microcephaly', 'breast cancer', 'breast neoplasm'], ['HIV infection', 'glioblastoma multiforme', 'endometrial cancer', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['intellectual developmental disorder with hypotonia and behavioral abnormalities', 'syndromic intellectual disability', 'cancer', 'Global developmental delay', 'Behavioral abnormality'], ['HIV infection', 'X-linked osteoporosis with fractures', 'chronic lymphocytic leukemia', 'endometrial cancer', 'lung adenocarcinoma'], ['undetermined early-onset epileptic encephalopathy', 'developmental and epileptic encephalopathy, 87', 'risk-taking behaviour', 'Microcephaly', 'breast adenocarcinoma'], ['urinary bladder cancer', 'electrocardiography', 'QRS duration', 'atrial fibrillation', 'sex hormone-binding globulin measurement'], ['Familial melanoma', 'cutaneous melanoma', 'Familial atypical multiple mole melanoma syndrome', 'head and neck squamous cell carcinoma', 'pancreatic adenocarcinoma'], ['endometrial cancer', 'ovarian cancer', 'lung adenocarcinoma', 'red blood cell distribution width', 'mean corpuscular hemoglobin'], ['red blood cell distribution width', 'refractive error measurement', 'cortical surface area measurement', 'reticulocyte measurement', 'hepatocellular carcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'melanoma', 'cancer', 'neoplasm'], ['body height', 'neoplasm', 'ovarian cancer', 'Ehlers-Danlos syndrome, progeroid type', "Alzheimer's disease"], ['lymphocyte percentage of leukocytes', 'small vessel stroke', "Alzheimer's disease", 'neutrophil percentage of leukocytes', 'paraneoplastic neurologic syndrome'], ['breast adenocarcinoma', 'white matter microstructure measurement', 'Blackfan-Diamond anemia', 'Autosomal dominant severe congenital neutropenia', 'peak expiratory flow'], ['Hypotrichosis simplex of the scalp', 'Generalized peeling skin syndrome', 'Peeling skin syndrome type B', 'Hypotrichosis simplex', 'Birt-Hogg-Dubé syndrome'], ['breast adenocarcinoma', 'pancreatic adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm', 'melanoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Female infertility due to zona pellucida defect', 'Non-syndromic male infertility due to sperm motility disorder'], ['Ringed hair disease', 'Griscelli disease type 3', 'Uncombable hair syndrome', 'Microcephaly - albinism - digital anomalies', 'platelet storage pool deficiency'], ['serum carcinoembryonic antigen measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'head and neck squamous cell carcinoma'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm', 'gastric cancer', 'acute lymphoblastic leukemia'], ['serum carcinoembryonic antigen measurement', 'non-small cell lung carcinoma', 'neoplasm', 'colorectal carcinoma', 'gastric cancer'], ['breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'head and neck squamous cell carcinoma', 'colorectal carcinoma', 'Female infertility due to zona pellucida defect'], ['Recurrent infection due to specific granule deficiency', 'eosinophil count', 'basophil count', 'eosinophil percentage of leukocytes', 'basophil percentage of leukocytes'], ['colorectal carcinoma', 'neutrophil percentage of leukocytes', 'hypertension', 'lymphocyte percentage of leukocytes', 'blood pressure'], ['MODY', 'monogenic diabetes', 'reticulocyte count', 'diabetes mellitus', 'chronic kidney disease'], ['Autosomal recessive primary microcephaly', 'Seckel syndrome', 'puberty onset measurement', 'testosterone measurement', 'Testicular Germ Cell Tumor'], ['risk-taking behaviour', 'smoking status measurement', 'monocyte percentage of leukocytes', 'self reported educational attainment', 'diastolic blood pressure'], ['glomerular filtration rate', 'creatinine measurement', 'reticulocyte count', 'prostate carcinoma', 'apolipoprotein A 1 measurement'], ['Cholesterol-ester transfer protein deficiency', 'coronary artery disease', 'Hyperalphalipoproteinemia', 'high density lipoprotein cholesterol measurement', 'total cholesterol measurement'], ['Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with B factor anomaly', 'Immunodeficiency due to a complement cascade protein anomaly', 'age-related macular degeneration', 'chronic hepatitis B virus infection'], ['Recurrent Neisseria infections due to factor D deficiency', 'age-related macular degeneration', 'reticulocyte count', 'reticulocyte measurement', 'Immature Reticulocyte Fraction Measurement'], ['Atypical hemolytic-uremic syndrome', 'Dense deposit disease', 'Immunodeficiency with factor H anomaly', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with H factor anomaly'], ['Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'age-related macular degeneration', 'Atypical hemolytic-uremic syndrome', 'Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with H factor anomaly'], ['Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'age-related macular degeneration', 'Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with H factor anomaly', 'IGA glomerulonephritis'], ['Non-immunoglobulin-mediated membranoproliferative glomerulonephritis', 'C3 glomerulonephritis', 'Primary membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with anti-factor H antibodies', 'diabetic nephropathy'], ['Atypical hemolytic-uremic syndrome', 'Immunodeficiency with factor I anomaly', 'Atypical hemolytic-uremic syndrome with I factor anomaly', 'age-related macular degeneration', 'Familial drusen'], ['Properdin deficiency', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'esophageal cancer', 'breast adenocarcinoma'], ['Cystic fibrosis', 'Congenital bilateral absence of vas deferens', 'congenital bilateral aplasia of vas deferens from CFTR mutation', 'Hereditary chronic pancreatitis', 'response to ivacaftor - efficacy'], ['eosinophil count', 'sitting height measurement', 'ovarian cancer', 'lung adenocarcinoma', 'glioblastoma multiforme'], ['gastric cancer', 'Ovarian hyperstimulation syndrome', 'gonorrhea', 'ovarian cancer', 'Bronchopulmonary dysplasia'], ['red blood cell distribution width', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'acute myeloid leukemia', 'vitamin supplement exposure measurement'], ['Intellectual disability', 'mental retardation', 'Global developmental delay', 'attention deficit hyperactivity disorder', 'eosinophil count'], ['body height', 'Short stature due to GHSR deficiency', 'appendicular lean mass', 'breast adenocarcinoma', 'Congenital hydronephrosis'], ['Lennox-Gastaut syndrome', 'Epileptic encephalopathy', 'complex neurodevelopmental disorder', 'Childhood-onset epilepsy syndrome', 'Intellectual disability'], ['total cholesterol measurement', 'systolic blood pressure', 'low density lipoprotein cholesterol measurement', 'PR interval', 'appendicular lean mass'], ['CHARGE syndrome', 'Kallmann syndrome', 'hypogonadotropic hypogonadism 5 with or without anosmia', 'Normosmic congenital hypogonadotropic hypogonadism', 'hearing loss'], ['Abnormality of refraction', 'refractive error measurement', 'interleukin 17 receptor B measurement', 'cardiovascular disease', 'smoking status measurement'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'squamous cell lung carcinoma'], ['Hereditary breast cancer', 'prostate carcinoma', 'Li-Fraumeni syndrome', 'breast carcinoma', 'osteosarcoma'], ['blood protein measurement', 'acute myeloid leukemia', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'body height'], ['neoplasm', 'amyotrophic lateral sclerosis', 'axial length measurement', 'breast cancer', 'Hand-foot syndrome'], ['Congenital muscular dystrophy due to phosphatidylcholine biosynthesis defect', 'Congenital muscular dystrophy', 'breast adenocarcinoma', 'Seizure', 'Severe global developmental delay'], ['Choroideremia', 'Retinitis pigmentosa', 'Retinal dystrophy', 'night blindness', 'Abnormality of the eye'], ['N-acylethanolamine-hydrolyzing acid amidase measurement', 'breast adenocarcinoma', 'acute myeloid leukemia', 'prostate carcinoma', 'Hematochezia'], ['chronic obstructive pulmonary disease', 'Prune belly syndrome', 'asthma', 'overactive bladder', 'Urinary incontinence'], ['Urinary incontinence', 'Polyuria', 'Hyperhidrosis', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['CHST3-related skeletal dysplasia', 'Autosomal dominant Larsen syndrome', 'Spondyloepiphyseal dysplasia and spondyloepimetaphyseal dysplasia', 'Back pain', 'body height'], ['lean body mass', 'appendicular lean mass', 'body height', 'fat body mass', 'base metabolic rate measurement'], ['intellectual disability, autosomal dominant 45', 'Autosomal dominant non-syndromic intellectual disability', 'mental retardation', 'low grade glioma', 'genetic disorder'], ['Immunodeficiency by defective expression of HLA class 2', 'lung carcinoma', 'gastric carcinoma', 'colon adenocarcinoma', 'colorectal adenocarcinoma'], ['Juvenile myoclonic epilepsy', 'Endocrine-cerebro-osteodysplasia syndrome', 'Short rib-polydactyly syndrome', 'smoking status measurement', 'smoking behaviour measurement'], ['grip strength measurement', 'triglyceride measurement', 'body height', 'body fat percentage', 'self reported educational attainment'], ['appendicular lean mass', 'acute myeloid leukemia', 'BMI-adjusted waist circumference', 'eosinophil count', 'neoplasm'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'Congenital factor II deficiency', 'prostate carcinoma'], ['eosinophil count', 'basophil percentage of leukocytes', 'eosinophil percentage of leukocytes', 'basophil count', 'sum of eosinophil and basophil counts'], ['Cold-induced sweating syndrome', 'Crisponi syndrome', 'acute lymphoblastic leukemia', 'cutaneous melanoma', 'breast adenocarcinoma'], ['Dent disease', 'Dent disease type 1', 'Hypophosphatemic rickets', 'genetic disorder', 'nephrolithiasis'], ['Ichthyosis - hypotrichosis - sclerosing cholangitis', 'optic disc size measurement', 'endometrial cancer', 'neoplasm', 'lung adenocarcinoma'], ['glioblastoma multiforme', 'mean platelet volume', 'asthma', 'gastric adenocarcinoma', 'neoplasm'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast cancer', 'non-small cell lung carcinoma'], ['waist-hip ratio', 'birth weight', 'platelet count', 'eosinophil count', 'head and neck squamous cell carcinoma'], ['hearing loss', 'endometrial cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['FEV/FEC ratio', 'endometrial cancer', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma'], ['blood protein measurement', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'neoplasm', 'eosinophil count'], ['blood protein measurement', 'hepatocellular carcinoma', 'neoplasm', 'gastric cancer', 'lung cancer'], ['blood protein measurement', 'acute myeloid leukemia', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'colitis', 'multiple sclerosis', 'Autosomal agammaglobulinemia'], ['squamous cell lung carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'multiple sclerosis'], ['Chronic mucocutaneous candidosis', 'aspergillosis, susceptibility to', 'inflammatory bowel disease', 'infection', 'infect'], ['blood protein measurement', 'breast adenocarcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'breast cancer'], ['neutrophil count', 'eosinophil count', 'leukocyte count', 'myeloid white cell count', 'acute myeloid leukemia'], ['Autosomal recessive non-syndromic intellectual disability', 'melanocytic neoplasm', 'Short stature due to GHSR deficiency', 'body mass index', 'cervical squamous cell carcinoma'], ['schistosomiasis', 'red blood cell distribution width', 'influenza', 'cancer', 'glioma'], ['glioblastoma multiforme', 'red blood cell distribution width', 'Brugada syndrome', 'Familial short QT syndrome', 'Red cell distribution width'], ['multiple myeloma', 'myeloid neoplasm', 'Miyoshi myopathy', 'neoplasm', "Alzheimer's disease"], ['influenza', 'cutaneous melanoma', 'esophageal cancer', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['intellectual disability, autosomal dominant 56', 'Autosomal dominant non-syndromic intellectual disability', 'genetic disorder', 'lung adenocarcinoma', 'inflammatory myofibroblastic tumor'], ['Nail-patella-like renal disease', 'Balkan nephropathy', 'Fibronectin glomerulopathy', 'Cystinuria type B', 'Cystinuria type A'], ["Alzheimer's disease", 'family history of Alzheimer’s disease', 'fasting blood insulin measurement', 'age at onset', 'refractive error measurement'], ['heel bone mineral density', 'adult onset asthma', 'childhood onset asthma', 'vitamin B12 measurement', 'smoking status measurement'], ['thyroid gland adenocarcinoma', 'cutaneous melanoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme'], ['pancreatic adenocarcinoma', 'neoplasm', 'body height', 'breast cancer', 'Fragile X syndrome'], ['Proximal myotonic myopathy', 'carcinoma of liver and intrahepatic biliary tract', 'breast ductal adenocarcinoma', 'Uterine Carcinosarcoma', 'Hepatobiliary Neoplasm'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Posterior column ataxia - retinitis pigmentosa', 'protein measurement', 'ovarian cancer'], ['endometrial cancer', 'acute myeloid leukemia', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['squamous cell lung carcinoma', 'gastric adenocarcinoma', 'lung adenocarcinoma', 'neoplasm', 'Non-syndromic male infertility due to sperm motility disorder'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'urinary bladder cancer'], ['Myopia', 'blood protein measurement', 'Blackfan-Diamond anemia', 'experimental autoimmune encephalomyelitis', 'multiple sclerosis'], ['obesity', 'Lennox-Gastaut syndrome', 'pain', 'Dravet syndrome', 'Chronic pain'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'monocyte count', 'eosinophil count', 'reticulocyte count'], ['thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'experimental autoimmune encephalomyelitis'], ['cancer', 'myelodysplastic syndrome', 'leukemia', 'precursor lymphoblastic lymphoma/leukemia', 'chronic myelogenous leukemia'], ['fatty acid measurement', 'breast milk measurement', 'parental genotype effect measurement', 'breast adenocarcinoma', 'Total congenital cataract'], ['Osteogenesis imperfecta type 2', 'Osteogenesis imperfecta type 1', 'Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta type 4'], ['Spondyloepiphyseal dysplasia congenita', 'Stickler syndrome type 1', 'Kniest dysplasia', 'Achondrogenesis type 2', 'Spondyloepimetaphyseal dysplasia congenita, Strudwick type'], ['Alport syndrome', 'X-linked Alport syndrome', 'Proteinuria', 'Hematuria', 'Dupuytren Contracture'], ['Stickler syndrome type 3', 'Otospondylomegaepiphyseal dysplasia', 'otospondylomegaepiphyseal dysplasia, autosomal recessive', 'Fibrochondrogenesis', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB'], ['Punctate palmoplantar keratoderma type 1', 'protein measurement', 'body height', 'heel bone mineral density', 'eye measurement'], ['Knobloch syndrome', 'glaucoma, primary closed-angle', 'Dupuytren Contracture', 'Abnormality of connective tissue', 'diabetic foot'], ['Steel syndrome', 'Dupuytren Contracture', 'Abnormality of connective tissue', 'diabetic foot', 'skin wound'], ['Pseudoachondroplasia', 'Multiple epiphyseal dysplasia type 1', 'Multiple epiphyseal dysplasia', 'carpal tunnel syndrome', 'breast adenocarcinoma'], ["Parkinson's disease", 'schizophrenia', 'drug dependence', 'chronic kidney disease', 'urinary metabolite measurement'], ['preeclampsia', 'breast adenocarcinoma', 'prostate carcinoma', 'glioblastoma multiforme', 'stroke'], ['neoplasm', 'heart failure', 'breast cancer', 'pancreatic carcinoma', 'pancreatic neoplasm'], ['leigh syndrome due to mitochondrial complex iv deficiency', 'Isolated cytochrome C oxidase deficiency', 'mitochondrial complex 4 deficiency, nuclear type 15', 'hepatic failure, early-onset, and neurologic disorder due to cytochrome c oxidase deficiency', 'head and neck squamous cell carcinoma'], ['genetic disorder', 'BMI-adjusted waist circumference', 'waist-hip ratio', 'physical activity measurement', 'FEV/FEC ratio'], ['protein measurement', 'blood protein measurement', 'endometrial cancer', 'pancreatic adenocarcinoma', 'cutaneous melanoma'], ['blood protein measurement', 'MAP kinase-activated protein kinase 3 measurement', 'carboxypeptidase B2 measurement', 'protein measurement', 'thrombin activatable fibrinolysis inhibitor activation peptide measurement'], ['mathematical ability', 'high density lipoprotein cholesterol measurement', 'cognitive function measurement', 'glioblastoma multiforme', 'low grade glioma'], ['vital capacity', 'body height', 'sitting height measurement', 'forced expiratory volume', 'Disorder of lipid metabolism'], ['traffic air pollution measurement', 'pneumoconiosis', 'response to silica exposure', 'silicosis', 'mean corpuscular volume'], ['Hereditary coproporphyria', 'harderoporphyria', 'angiopoietin-1 receptor measurement', 'Nuclear cataract', 'Total congenital cataract'], ['blood protein measurement', 'age at first sexual intercourse measurement', 'Abnormality of refraction', 'appendicular lean mass', 'migraine disorder'], ['carbamoyl phosphate synthetase I deficiency disease', 'Carbamoyl-phosphate synthase deficiency', 'pulmonary arterial hypertension', 'Congenital dyserythropoietic anemia type I', 'glycine measurement'], ['Carnitine palmitoyltransferase II deficiency', 'Carnitine palmitoyl transferase II deficiency, myopathic form', 'Carnitine palmitoyl transferase II deficiency, severe infantile form', 'Carnitine palmitoyl transferase II deficiency, neonatal form', 'Familial acute necrotizing encephalopathy'], ['basal cell carcinoma', 'skin cancer', 'skin carcinoma', 'keratinocyte carcinoma', 'non-melanoma skin carcinoma'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['Common variable immunodeficiency', 'systemic lupus erythematosus', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'lupus erythematosus', 'Recurrent infections'], ['Autosomal recessive non-syndromic intellectual disability', 'multiple myeloma', 'acute myeloid leukemia', 'immune system disease', 'Mantle cell lymphoma'], ['endometrial cancer', 'neutrophil count', 'myeloid white cell count', 'body height', "Parkinson's disease"], ['Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'acute myeloid leukemia', 'undifferentiated pleomorphic sarcoma', 'synovial sarcoma'], ['Rubinstein-Taybi syndrome', 'Rubinstein-Taybi syndrome due to CREBBP mutations', 'Menke-Hennekam syndrome 1', 'urinary bladder cancer', 'cancer'], ['Autosomal dominant nocturnal frontal lobe epilepsy', 'body mass index', 'body weight', 'waist circumference', 'body height'], ['type II diabetes mellitus', 'glioblastoma multiforme', 'neoplasm', 'colorectal carcinoma', 'irritable bowel syndrome'], ['lean body mass', 'heel bone mineral density', 'monocyte percentage of leukocytes', 'fat body mass', 'body height'], ['body height', 'diverticulitis', 'appendicular lean mass', 'blood protein measurement', 'grip strength measurement'], ['Cold-induced sweating syndrome', 'Crisponi syndrome', 'blood protein measurement', 'lung adenocarcinoma', 'Oral ulcer'], ['Endometrial Endometrioid Adenocarcinoma', 'hemangioblastoma', 'lung carcinoma', 'cutaneous melanoma', 'basal cell carcinoma'], ['esophageal squamous cell carcinoma', 'esophageal cancer', 'atopic eczema', 'vitamin D measurement', 'childhood onset asthma'], ['pneumonia', 'C-reactive protein measurement', 'bacterial disease', 'chronic obstructive pulmonary disease', 'asthma'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Macular dystrophy'], ['Alpha-crystallinopathy', 'Posterior polar cataract', 'Alpha-B crystallin-related late-onset distal myopathy', 'Familial isolated dilated cardiomyopathy', 'cataract 16 multiple types'], ['red blood cell distribution width', 'visceral adipose tissue measurement', 'rheumatoid arthritis', 'systemic lupus erythematosus', 'eosinophil count'], ['Cerulean cataract', 'Non-syndromic congenital cataract', 'Coralliform cataract', 'Nuclear cataract', 'Coppock-like cataract'], ['mathematical ability', 'self reported educational attainment', 'type I diabetes mellitus', 'Familial primary pulmonary hypoplasia', 'Pyknoachondrogenesis'], ['autism', 'Intellectual disability', 'Global developmental delay', 'body mass index', 'body height'], ['intelligence', 'self reported educational attainment', 'cognitive function measurement', 'wellbeing measurement', 'body fat percentage'], ['monocyte count', 'platelet count', 'leukocyte count', 'lymphocyte count', 'aspartate aminotransferase measurement'], ['Hereditary diffuse leukoencephalopathy with axonal spheroids and pigmented glia', 'brain abnormalities, neurodegeneration, and dysosteosclerosis', 'neoplasm', 'renal cell carcinoma', 'Gastrointestinal stromal tumor'], ['eosinophil percentage of leukocytes', 'rheumatoid arthritis', 'eosinophil count', 'eosinophil percentage of granulocytes', 'neutrophil percentage of granulocytes'], ['Congenital pulmonary alveolar proteinosis', 'Intraalveolar phospholipid accumulation', 'pulmonary alveolar proteinosis', 'melanoma', 'Sepsis'], ['neutrophil count', 'myeloid white cell count', 'granulocyte count', 'sum of neutrophil and eosinophil counts', 'sum of basophil and neutrophil counts'], ['acute lymphoblastic leukemia', 'Normosmic congenital hypogonadotropic hypogonadism', 'appendicular lean mass', 'Prolactin-Producing Pituitary Gland Adenoma', 'Short stature-delayed bone age due to thyroid hormone metabolism deficiency'], ['Non-acquired isolated growth hormone deficiency', 'body height', 'Isolated growth hormone deficiency type IB', 'diet measurement', 'appendicular lean mass'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast carcinoma'], ['Familial advanced sleep-phase syndrome', 'myeloid white cell count', 'granulocyte percentage of myeloid white cells', 'monocyte percentage of leukocytes', 'cancer'], ['erythrocyte count', 'ovarian cancer', 'thyroid gland adenocarcinoma', 'cutaneous melanoma', 'gastric adenocarcinoma'], ['infection', 'infect', 'hair shape measurement', 'atypical teratoid rhabdoid tumor', 'FEV/FEC ratio'], ['hypertrophic cardiomyopathy 12', 'Familial isolated dilated cardiomyopathy', 'familial hypertrophic cardiomyopathy', 'dilated cardiomyopathy', 'Familial dilated cardiomyopathy'], ['Hereditary cerebral hemorrhage with amyloidosis, Icelandic type', 'age-related macular degeneration', 'Hereditary cerebral hemorrhage with amyloidosis', 'AL amyloidosis', 'cystatin C measurement'], ['blood protein measurement', 'cystatin-F measurement', 'alcohol consumption measurement', 'lean body mass', 'Creutzfeldt Jacob Disease'], ['neoplasm', 'glioma', 'melanoma', 'lung cancer', 'non-small cell lung carcinoma'], ['neoplasm', 'glioma', 'lung cancer', 'renal cell carcinoma', 'melanoma'], ['hypotonia, ataxia, developmental delay, and tooth enamel defect syndrome', 'genetic disorder', 'type II diabetes mellitus', 'Intellectual disability', 'mental retardation'], ['autoimmune lymphoproliferative syndrome due to CTLA4 haploinsuffiency', 'type I diabetes mellitus', 'melanoma', 'celiac disease', 'Autoimmune lymphoproliferative syndrome'], ['Pilomatrixoma', 'medulloblastoma', 'Familial exudative vitreoretinopathy', 'hepatocellular carcinoma', 'pilomatrixoma'], ['Blepharo-cheilo-odontic syndrome', 'listeriosis', 'cleft lip', 'cleft palate', 'hepatocellular carcinoma'], ['Hereditary chronic pancreatitis', 'chronic pancreatitis', 'genetic disorder', 'acute pancreatitis', 'alcoholic pancreatitis'], ['red blood cell density measurement', 'erythrocyte count', 'mean corpuscular volume', 'refractive error measurement', 'high density lipoprotein cholesterol measurement'], ['Galactosialidosis', 'Lysosomal disease', 'Non-immune hydrops fetalis', 'mental retardation', 'Intellectual disability'], ['Keratolytic winter erythema', 'blood protein measurement', 'basophil count', 'risk-taking behaviour', 'basophil percentage of leukocytes'], ['Papillon-Lefèvre syndrome', 'Haim-Munk syndrome', 'periodontitis', 'aggressive periodontitis', 'chronic periodontitis'], ['CLN10 disease', 'Neuronal ceroid lipofuscinosis', 'Congenital neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Adult neuronal ceroid lipofuscinosis'], ['Tuberculosis', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'endometrial cancer'], ['Pycnodysostosis', 'Skeletal dysplasia', 'Macrocephaly', 'Short stature due to GHSR deficiency', 'Nail dysplasia'], ['severe acute respiratory syndrome', 'COVID-19', 'cathepsin L1 measurement', 'clear cell renal carcinoma', 'neoplasm'], ['sex hormone-binding globulin measurement', 'cathepsin S measurement', 'basal cell carcinoma', 'breast adenocarcinoma', 'blood protein measurement'], ['Gräsbeck-Imerslund disease', 'megaloblastic anemia (disease)', 'vitamin B12 measurement', 'gut microbiome measurement', 'genetic disorder'], ['Pseudohypoaldosteronism type 2E', 'Pseudohypoaldosteronism type 2A', 'Pseudohypoaldosteronism type 2', 'papillary renal cell carcinoma', 'genetic disorder'], ['multiple myeloma', 'immune system disease', 'Mantle cell lymphoma', 'AL amyloidosis', 'cutaneous lupus erythematosus'], ['global developmental delay with or without impaired intellectual development', 'myelodysplastic syndrome', 'cancer', 'skin carcinoma', 'basal cell carcinoma'], ['adolescent idiopathic scoliosis', 'electrocardiography', 'progesterone measurement', 'body height', 'systemic juvenile idiopathic arthritis'], ['blood protein measurement', 'CX3CL1 measurement', 'glioblastoma multiforme', 'eosinophil count', 'head and neck squamous cell carcinoma'], ['age-related macular degeneration', 'monocyte count', 'lymphocyte count', 'monocyte percentage of leukocytes', 'leukocyte count'], ['growth-regulated alpha protein measurement', 'blood protein measurement', 'glioblastoma multiforme', 'leukocyte count', 'neutrophil count'], ['head and neck squamous cell carcinoma', 'attention deficit hyperactivity disorder', 'breast carcinoma', 'neoplasm', 'midbrain'], ['blood protein measurement', 'CXCL5 measurement', 'C-X-C motif chemokine 5 measurement', 'neoplasm', 'colorectal carcinoma'], ['blood protein measurement', 'ovarian cancer', 'CXCL6 measurement', 'interleukin-8 measurement', 'gastric adenocarcinoma'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'low grade glioma', 'inflammatory bowel disease', 'neoplasm'], ['head and neck squamous cell carcinoma', 'rheumatoid arthritis', 'inflammatory bowel disease', 'Townes-Brocks syndrome', 'neoplasm'], ['inflammatory bowel disease', 'C-X-C motif chemokine 11 measurement', 'neoplasm', 'blood protein measurement', 'protein measurement'], ['HIV-1 infection', 'monocyte count', 'hematocrit', 'hemoglobin measurement', 'erythrocyte count'], ['tonsillectomy risk measurement', 'lymphatic system disease', 'ovarian cancer', 'drug use measurement', 'thyroid gland adenocarcinoma'], ['neoplasm', 'hepatocellular carcinoma', 'cancer', 'glioblastoma multiforme', 'lip and oral cavity carcinoma'], ['neoplasm', 'colon carcinoma', 'colonic neoplasm', 'colorectal adenocarcinoma', 'malignant colon neoplasm'], ['type I diabetes mellitus', 'neutrophil count', 'leukocyte count', 'neutrophil percentage of leukocytes', 'myeloid white cell count'], ['type I diabetes mellitus', 'ulcerative colitis', 'glioblastoma multiforme', 'psoriasis', "Crohn's disease"], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'neoplasm', 'breast cancer'], ['WHIM syndrome', 'multiple myeloma', 'HIV infection', 'non-Hodgkins lymphoma', 'acute lymphoblastic leukemia'], ['allergic rhinitis', 'atopic eczema', 'albumin:globulin ratio measurement', 'multiple sclerosis', 'anti-human herpes virus 7 antibody measurement'], ['Hereditary methemoglobinemia', 'Recessive hereditary methemoglobinemia type 1', 'Recessive hereditary methemoglobinemia type 2', 'methemoglobinemia', 'Autosomal dominant methemoglobinemia'], ['Chronic granulomatous disease', 'X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'X-linked mendelian susceptibility to mycobacterial diseases', 'congenital heart disease', 'Mendelian susceptibility to mycobacterial diseases'], ['Brooke-Spiegler syndrome', 'Familial cylindromatosis', 'Familial multiple trichoepithelioma', 'basal cell carcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['erythrocyte count', 'cutaneous melanoma', 'coffee consumption measurement', 'drug use measurement', 'glioblastoma multiforme'], ['response to efavirenz', 'HIV-1 infection', 'virologic response measurement', 'DDT metabolite measurement', 'gestational serum measurement'], ['blood metabolite measurement', 'chronic kidney disease', 'testosterone measurement', 'urinary metabolite measurement', 'cutaneous melanoma'], ['cholesterol embolism', 'response to phenytoin', 'response to anticoagulant', 'blood metabolite measurement', 'appendicular lean mass'], ['response to clopidogrel', 'systolic blood pressure', 'lymphocyte count', 'serum metabolite measurement', 'cutaneous melanoma'], ['debrisoquine, poor metabolism of', 'cutaneous melanoma', 'glioblastoma multiforme', 'chronic kidney disease', 'endometrial cancer'], ['Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'esophageal squamous cell carcinoma', 'intrahepatic cholangiocarcinoma', 'nan', 'nan'], ['HIV infection', 'HIV-1 infection', 'infection', 'AIDS', 'COVID-19'], ['HIV infection', 'HIV-1 infection', 'infection', 'AIDS', 'pneumocystosis'], ['cutaneous melanoma', 'glioblastoma multiforme', 'lung adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['46,XY disorder of sex development - adrenal insufficiency due to CYP11A1 deficiency', 'Congenital adrenal hyperplasia', 'neoplasm', 'Inherited isolated adrenal insufficiency due to CYP11A1 deficiency', 'adrenal cortex carcinoma'], ['Congenital adrenal hyperplasia due to 11-beta-hydroxylase deficiency', 'Congenital adrenal hyperplasia', 'adrenal gland disease', "pituitary-dependent Cushing's disease", 'Familial hyperaldosteronism type I'], ['Familial hypoaldosteronism', 'Familial hyperreninemic hypoaldosteronism type 1', 'corticosterone methyloxidase type 2 deficiency', 'steroid inherited metabolic disorder', 'hypertension'], ['Congenital adrenal hyperplasia due to 17-alpha-hydroxylase deficiency', 'Congenital adrenal hyperplasia', '46,XY disorder of sex development due to isolated 17,20 lyase deficiency', 'prostate cancer', 'adrenal gland disease'], ['Aromatase deficiency', 'breast cancer', 'breast carcinoma', 'breast neoplasm', 'neoplasm'], ['Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency', 'Congenital adrenal hyperplasia', 'genetic disorder', 'Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency, simple virilizing form', 'Classic congenital adrenal hyperplasia due to 21-hydroxylase deficiency, salt wasting form'], ['triglyceride measurement', 'low density lipoprotein cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'alcohol drinking', 'platelet count'], ['coronary artery disease', 'coronary atherosclerosis', 'Myocardial Ischemia', 'glioblastoma multiforme', 'angina pectoris'], ['Non-syndromic congenital cataract', 'hematocrit', 'hemoglobin measurement', 'handedness', 'eosinophil count'], ['mean platelet volume', 'response to vaccine', 'cytokine measurement', 'monocyte count', 'high density lipoprotein cholesterol measurement'], ['asthma', 'seasonal allergic rhinitis', 'allergic rhinitis', 'Rhinitis, Allergic, Perennial', 'obstructive sleep apnea'], ['body height', 'peak expiratory flow', 'BMI-adjusted waist-hip ratio', 'BMI-adjusted waist circumference', 'waist-hip ratio'], ['Walker-Warburg syndrome', 'Autosomal recessive limb-girdle muscular dystrophy type 2P', 'Limb-girdle muscular dystrophy', 'Congenital muscular dystrophy', 'Congenital muscular dystrophy, Fukuyama type'], ['amyotrophic lateral sclerosis', 'schizophrenia', 'mean platelet volume', 'hepatocellular carcinoma', 'cirrhosis of liver'], ['breast adenocarcinoma', 'sitting height measurement', 'ovarian cancer', 'Glycogen storage disease due to phosphoglycerate mutase deficiency', 'head and neck squamous cell carcinoma'], ['neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes', 'neutrophil count', 'lymphocyte count', 'sum of basophil and neutrophil counts'], ['Familial congenital mirror movements', 'Horizontal gaze palsy with progressive scoliosis', 'Agenesis of corpus callosum', 'smoking status measurement', 'neuroticism measurement'], ['Cerebro-facio-articular syndrome', 'Mitral valve prolapse', 'hemoglobin measurement', 'platelet crit', 'hematocrit'], ['cup-to-disc ratio measurement', 'optic cup area measurement', 'protein measurement', 'mathematical ability', 'self reported educational attainment'], ['Hoyeraal-Hreidarsson syndrome', 'breast carcinoma', 'coronary artery disease', 'type I diabetes mellitus', 'autoimmune thyroid disease'], ['Autosomal recessive non-syndromic intellectual disability', 'eosinophil count', 'neutrophil percentage of leukocytes', 'body height', 'low density lipoprotein cholesterol measurement'], ['Piebaldism', 'kidney disease', 'hair colour measurement', 'skin pigmentation', 'hair color'], ['Perry syndrome', 'amyotrophic lateral sclerosis', 'sporadic amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Distal hereditary motor neuropathy type 7'], ['multiple myeloma', 'immune system disease', 'Mantle cell lymphoma', 'AL amyloidosis', 'cutaneous lupus erythematosus'], ['Xeroderma pigmentosum complementation group E', 'Xeroderma pigmentosum', 'basal cell carcinoma', 'prostate carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation'], ['Aromatic L-amino acid decarboxylase deficiency', "Parkinson's disease", 'depressive disorder', 'stroke', 'impulse control disorder'], ['liposarcoma', 'dedifferentiated liposarcoma', 'pleomorphic liposarcoma', 'lipoma', 'well-differentiated liposarcoma'], ['DDOST-CDG', 'COVID-19', 'osteosarcoma', 'Ewing sarcoma', 'Congenital disorder of glycosylation'], ['breast adenocarcinoma', 'schizophrenia', 'alkaline phosphatase measurement', 'susceptibility to shingles measurement', 'sleep duration'], ['Spondyloepimetaphyseal dysplasia - short limb - abnormal calcification', 'Warburg-Cinotti syndrome', 'metastatic colorectal cancer', 'neoplasm', 'squamous cell lung carcinoma'], ['insulin like growth factor measurement', 'protein measurement', 'sex hormone-binding globulin measurement', 'multiple sclerosis', 'triglyceride measurement'], ['protein measurement', 'triglyceride measurement', 'insulin like growth factor measurement', 'serum alanine aminotransferase measurement', 'sex hormone-binding globulin measurement'], ['Intellectual disability', 'medulloblastoma', 'mental retardation', 'X-linked syndromic intellectual disability', 'X-linked non-syndromic intellectual disability'], ['DDX41-related hematologic malignancy predisposition syndrome', 'myeloproliferative disorder', 'acute myeloid leukemia', 'chronic myelogenous leukemia', 'Deafness - lymphedema - leukemia'], ['benign neoplasm of lip', 'Intellectual disability', 'cancer', 'neoplasm', 'hepatocellular carcinoma'], ['Singleton-Merten dysplasia', 'hemoglobin measurement', 'psoriasis', 'red blood cell density measurement', 'hematocrit'], ['progressive encephalopathy with leukodystrophy due to DECR deficiency', 'protein measurement', 'smoking status measurement', 'Glycogen storage disease due to liver phosphorylase kinase deficiency', 'Carnitine palmitoyl transferase 1A deficiency'], ['basophil percentage of leukocytes', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'basophil count'], ['blood protein measurement', 'cutaneous melanoma', 'HIV-1 infection', 'viral load', 'squamous cell lung carcinoma'], ['glaucoma', 'cognitive function measurement', 'mathematical ability', 'self reported educational attainment', 'arterial stiffness measurement'], ['Desminopathy', 'Scapuloperoneal amyotrophy', 'dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy', 'Familial dilated cardiomyopathy'], ['Diarrhea', 'Congenital chronic diarrhea with protein-losing enteropathy', 'genetic disorder', 'familial chylomicronemia syndrome', 'glioblastoma multiforme'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'hemoglobin measurement', 'hematocrit'], ['Smith-Lemli-Opitz syndrome', 'genetic disorder', 'Intellectual disability', 'Genetic disorder of sex development', 'cataract'], ['Desmosterolosis', 'hepatocellular carcinoma', 'breast cancer', 'white matter microstructure measurement', 'visceral adipose tissue measurement'], ['Retinitis pigmentosa', 'developmental delay and seizures with or without movement abnormalities', 'undetermined early-onset epileptic encephalopathy', 'Posterior column ataxia - retinitis pigmentosa', 'developmental and epileptic encephalopathy'], ['Constitutional megaloblastic anemia with severe neurologic disease', 'acute lymphoblastic leukemia', 'rheumatoid arthritis', 'non-small cell lung carcinoma', 'psoriasis'], ['46,XY complete gonadal dysgenesis', '46,XY gonadal dysgenesis - motor and sensory neuropathy', 'breast adenocarcinoma', 'glioblastoma multiforme', 'Partial chromosome Y deletion'], ['Postaxial acrofacial dysostosis', 'rheumatoid arthritis', 'multiple sclerosis', 'psoriatic arthritis', 'immune system disease'], ['neoplasm', 'esophageal squamous cell carcinoma', 'Niemann-Pick disease type C', 'Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['Arnold-Chiari malformation type I', 'blood protein measurement', 'electrocardiography', 'dickkopf‐related protein 1 measurement', 'QRS complex'], ['hair shape measurement', 'balding measurement', 'blood protein measurement', 'androgenetic alopecia', 'Alopecia'], ['refractive error measurement', 'body height', 'cortical surface area measurement', 'neuroimaging measurement', 'brain volume measurement'], ['multiple sclerosis', 'mean platelet volume', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect', 'female infertility due to oocyte meiotic arrest'], ['Pyruvate dehydrogenase E2 deficiency', 'Pyruvate dehydrogenase deficiency', 'dystonic disorder', 'genetic disorder', 'waist-hip ratio'], ['Pyruvate dehydrogenase E3 deficiency', 'Pyruvate dehydrogenase deficiency', 'Maple syrup urine disease', 'Leigh syndrome', 'genetic disorder'], ['body height', 'mean platelet volume', 'lean body mass', 'vital capacity', 'forced expiratory volume'], ['neurodevelopmental disorder with non-specific brain abnormalities and with or without seizures', 'cancer', 'T-cell leukemia', 'genetic disorder', 'Alobar holoprosencephaly'], ['Autosomal recessive spondylocostal dysostosis', 'Rib fusion', 'Mesoaxial synostotic syndactyly with phalangeal reduction', 'Syndactyly', 'spondylocostal dysostosis'], ['Adams-Oliver syndrome', 'cancer', 'T-cell leukemia', 'Circumscribed cutaneous aplasia of the vertex', 'cognitive function measurement'], ['Tricho-dento-osseous syndrome', 'Amelogenesis imperfecta', 'Hypomaturation-hypoplastic amelogenesis imperfecta with taurodontism', 'Taurodontism', 'Peripheral pulmonary artery stenosis'], ['Orofacial clefting syndrome', 'balding measurement', 'blood urea nitrogen measurement', 'hair colour measurement', 'hair color'], ['Duchenne muscular dystrophy', 'Becker muscular dystrophy', 'dilated cardiomyopathy 3B', 'Muscular dystrophy', 'Familial isolated dilated cardiomyopathy'], ['Autosomal recessive hypophosphatemic rickets', 'Hypophosphatemic rickets', 'Acromesomelic dysplasia, Grebe type', 'Fibular aplasia - complex brachydactyly', 'osteoarthritis'], ['Steinert myotonic dystrophy', 'Myotonic dystrophy', 'breast adenocarcinoma', 'Catecholaminergic polymorphic ventricular tachycardia', 'Noonan syndrome'], ['sex hormone-binding globulin measurement', 'serum alanine aminotransferase measurement', 'serum gamma-glutamyl transferase measurement', 'apolipoprotein B measurement', 'low density lipoprotein cholesterol measurement'], ['Early infantile epileptic encephalopathy', 'polyendocrine-polyneuropathy syndrome', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'X-linked mixed deafness with perilymphatic gusher', 'deafness'], ['Non-syndromic male infertility due to sperm motility disorder', 'Primary ciliary dyskinesia', 'Kartagener Syndrome', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement'], ['Primary ciliary dyskinesia', 'genetic disorder', 'Kartagener Syndrome', 'prostate carcinoma', 'response to radiation'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'smoking status measurement', 'ciliopathy', 'Recurrent infections'], ['Primary ciliary dyskinesia', 'ciliopathy', 'Kartagener Syndrome', 'genetic disorder', 'bronchiectasis'], ['CLN4B disease', 'Adult neuronal ceroid lipofuscinosis', 'Neuronal ceroid lipofuscinosis', 'mean platelet volume', 'Immature Reticulocyte Fraction Measurement'], ['systemic lupus erythematosus', 'Autosomal recessive systemic lupus erythematosus', 'neutrophil count', 'vital capacity', 'myeloid white cell count'], ['Autosomal recessive systemic lupus erythematosus', 'systemic lupus erythematosus', 'systemic scleroderma', 'limited scleroderma', 'lupus nephritis'], ['systemic lupus erythematosus', 'cirrhosis of liver', 'Primary membranoproliferative glomerulonephritis', 'rheumatoid arthritis', 'reticulocyte count'], ['FEV/FEC ratio', 'blood protein measurement', 'ulcerative colitis', 'ovarian cancer', 'pulmonary function measurement'], ['Autosomal dominant cerebellar ataxia, deafness and narcolepsy', 'Hereditary sensory and autonomic neuropathy type 1', 'acute myeloid leukemia', 'myelodysplastic syndrome', 'chronic myelomonocytic leukemia'], ['acute myeloid leukemia', 'Tall stature-intellectual disability-facial dysmorphism syndrome', 'myelodysplastic syndrome', 'chronic myelomonocytic leukemia', 'diffuse large B-cell lymphoma'], ['risk-taking behaviour', 'intelligence', 'waist-hip ratio', 'mean corpuscular hemoglobin concentration', 'insomnia'], ['acute lymphoblastic leukemia', 'Okt4 epitope deficiency', 'neoplasm', 'seminoma', 'Alymphoid cystic thymic dysgenesis'], ['ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'inflammatory bowel disease', 'Brugada syndrome'], ['lean body mass', 'heel bone mineral density', 'hemoglobin measurement', 'appendicular lean mass', 'erythrocyte count'], ['DPAGT1-CDG', 'Congenital myasthenic syndromes', 'Congenital myasthenic syndromes with glycosylation defect', 'Congenital disorder of glycosylation', 'genetic disorder'], ['infection', 'Sepsis', 'urinary tract infection', 'pneumonia', 'endocarditis'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'glioblastoma multiforme', 'cutaneous melanoma'], ['Global developmental delay', 'Cerebellar vermis hypoplasia', 'Dandy-Walker syndrome', 'Hydrocephalus', 'pancreatic adenocarcinoma'], ['Intellectual disability', 'Macrocephaly', 'Global developmental delay', 'mental retardation', 'schizophrenia'], ['lymphocyte count', 'monocyte count', 'monocyte percentage of leukocytes', 'myeloid white cell count', 'multiple sclerosis'], ['type II diabetes mellitus', 'diabetes mellitus', 'type I diabetes mellitus', 'polycystic ovary syndrome', 'prediabetes syndrome'], ['electrocardiography', 'dermatopontin measurement', 'Progressive sensorineural hearing impairment', 'PR interval', 'TPE interval measurement'], ['schizophrenia', 'unipolar depression', 'bipolar disorder', 'attention deficit hyperactivity disorder', 'Tourette syndrome'], ['schizophrenia', 'psychosis', "Parkinson's disease", 'bipolar disorder', 'schizoaffective disorder'], ['kidney Wilms tumor', 'colorectal adenocarcinoma', 'gastric carcinoma', 'kidney neoplasm', 'lung carcinoma'], ['Familial isolated arrhythmogenic right ventricular dysplasia', 'Arrhythmogenic right ventricular dysplasia', 'Abnormality of the cardiovascular system', 'Familial isolated arrhythmogenic ventricular dysplasia, left dominant form', 'Familial isolated arrhythmogenic ventricular dysplasia, biventricular form'], ['Hereditary hypotrichosis with recurrent skin vesicles', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'ovarian cancer'], ['genetic disorder', 'reticulocyte count', 'fasting blood glucose measurement', 'self reported educational attainment', 'brain aneurysm'], ['Keratosis palmoplantaris striata', 'Severe dermatitis-multiple allergies-metabolic wasting syndrome', 'Diffuse palmoplantar keratoderma with painful fissures', 'Focal palmoplantar keratoderma with joint keratoses', 'palmoplantar keratoderma i, striate, focal, or diffuse'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer', 'ovarian cancer', 'low grade glioma'], ['Hypotrichosis simplex', 'Monilethrix', 'glioblastoma multiforme', 'breast adenocarcinoma', 'clear cell renal carcinoma'], ['Woolly hair-palmoplantar keratoderma-dilated cardiomyopathy syndrome', 'Skin fragility-woolly hair-palmoplantar keratoderma syndrome', 'Keratosis palmoplantaris striata', 'Lethal acantholytic epidermolysis bullosa', 'arrhythmogenic cardiomyopathy with woolly hair and keratoderma'], ['Dentinogenesis imperfecta type 2', 'deafness, autosomal dominant 39, with dentinogenesis imperfecta 1', 'Dentinogenesis imperfecta type 3', 'Dentin dysplasia type II', 'Dentin dysplasia'], ['Hereditary sensory and autonomic neuropathy type 6', 'Epidermolysis bullosa simplex', 'KRT14-related epidermolysis bullosa simplex', 'autism spectrum disorder', 'Charcot-Marie-Tooth disease'], ['hematocrit', 'total iron binding capacity', 'Fuchs endothelial corneal dystrophy', 'Lisch epithelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy'], ['cutaneous melanoma', 'nevus count', 'insulin like growth factor measurement', 'reticulocyte count', 'type II diabetes mellitus'], ['cup-to-disc ratio measurement', 'birth weight', 'C-reactive protein measurement', 'optic cup area measurement', 'body height'], ['pancreatic adenocarcinoma', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'obesity'], ['protein measurement', 'heel bone mineral density', 'acute myeloid leukemia', 'high density lipoprotein cholesterol measurement', 'response to statin'], ['heel bone mineral density', "Crohn's disease", 'serum non-albumin protein measurement', 'osteoarthritis', 'bone density'], ['glioblastoma multiforme', 'thyroid gland adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'cutaneous melanoma'], ['eosinophil count', 'hair color', 'eosinophil percentage of leukocytes', 'lymphocyte count', 'systemic lupus erythematosus'], ['mean corpuscular hemoglobin', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm', 'pancreatic carcinoma', 'neoplasm'], ['Dyggve-Melchior-Clausen disease', 'Smith-McCort dysplasia', 'waist-hip ratio', 'lean body mass', 'grip strength measurement'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2O', 'Autosomal dominant childhood-onset proximal spinal muscular atrophy', 'Autosomal dominant non-syndromic intellectual disability', 'Autosomal dominant childhood-onset proximal spinal muscular atrophy without contractures', 'Abnormality of neuronal migration'], ['complex neurodevelopmental disorder', 'Rare genetic intellectual disability with developmental anomaly', 'Intellectual disability', 'mental retardation', 'genetic disorder'], ['metabolic syndrome', 'macula measurement', 'ovarian cancer', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm'], ['small cell lung carcinoma', 'urinary bladder cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['asthma', 'multiple sclerosis', 'Senior-Loken syndrome', 'age-related hearing impairment', 'diet measurement'], ['breast carcinoma', 'hypertension', 'balding measurement', 'vital capacity', 'diastolic blood pressure'], ['interleukin-27 measurement', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'blood protein measurement', 'cutaneous melanoma'], ['MEND syndrome', 'X-linked dominant chondrodysplasia punctata', 'polydactyly (disease)', 'Focal palmoplantar and gingival keratoderma', 'Trichodysplasia - xeroderma'], ['forced expiratory volume', 'mean corpuscular volume', 'vital capacity', 'lymphocyte count', 'leukocyte count'], ['breast cancer', 'breast ductal carcinoma in situ', 'glomerular filtration rate', 'neoplasm', 'serum alanine aminotransferase measurement'], ['Hirschsprung disease', 'body height', 'appendicular lean mass', 'lean body mass', 'worry measurement'], ['Distal arthrogryposis type 5D', 'Distal arthrogryposis', 'genetic disorder', 'protein measurement', 'refractive error measurement'], ['body weights and measures', 'alkaline phosphatase measurement', 'body height', 'Red cell distribution width', 'acute myeloid leukemia'], ['X-linked hypohidrotic ectodermal dysplasia', 'Oligodontia', 'Hypohidrotic ectodermal dysplasia', 'tooth agenesis', 'Ectodermal dysplasia syndrome'], ['COVID-19', 'protein C measurement', 'alkaline phosphatase measurement', 'birth weight', 'hematological measurement'], ['Auriculocondylar syndrome', 'hematocrit', 'erythrocyte count', 'hemoglobin measurement', 'red blood cell density measurement'], ['Waardenburg-Shah syndrome', 'Hirschsprung disease', 'Ondine syndrome', 'Haddad syndrome', 'Waardenburg syndrome'], ['mandibulofacial dysostosis with alopecia', 'pulmonary arterial hypertension', 'hypertension', 'systemic scleroderma', 'pulmonary hypertension'], ['Waardenburg-Shah syndrome', 'ABCD syndrome', 'Hirschsprung disease', 'pulmonary arterial hypertension', 'hypertension'], ['Cohen-Gibson syndrome', 'Tall stature-intellectual disability-facial dysmorphism syndrome', 'viral disease', 'melanoma', 'breast carcinoma'], ['Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'mental retardation', 'Autosomal dominant non-syndromic intellectual disability', 'developmental and epileptic encephalopathy'], ['Familial drusen', 'Inguinal hernia', 'body height', 'BMI-adjusted waist circumference', 'physical activity measurement'], ['cutaneous melanoma', 'Fuchs endothelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy', 'smoking status measurement', 'X-linked endothelial corneal dystrophy'], ['self reported educational attainment', 'insomnia measurement', 'mathematical ability', 'smoking status measurement', 'smoking initiation'], ['Severe combined immunodeficiency due to CARD11 deficiency', 'Autosomal recessive early-onset inflammatory bowel disease', 'Autosomal agammaglobulinemia', 'IgE responsiveness, atopic', 'X-Linked Combined Immunodeficiency Diseases'], ['cancer', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Autosomal dominant primary hypomagnesemia with hypocalciuria', 'platelet count', 'reticulocyte count'], ['mean reticulocyte volume', 'diastolic blood pressure', 'mean corpuscular hemoglobin', 'systolic blood pressure', 'visual perception measurement'], ['non-small cell lung carcinoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cancer', 'colorectal adenocarcinoma'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'hepatocellular carcinoma', 'neoplasm', 'glioma'], ['prostate adenocarcinoma', 'ovarian cancer', 'breast adenocarcinoma', 'esophageal cancer', 'drug use measurement'], ['leukocyte count', 'neutrophil count', 'lymphocyte percentage of leukocytes', 'monocyte count', 'myeloid white cell count'], ['serum gamma-glutamyl transferase measurement', 'hemoglobin measurement', 'CC16 measurement', 'chronic obstructive pulmonary disease', 'lung disease associated with cystic fibrosis'], ['Kleefstra syndrome', 'Intellectual disability', 'genetic disorder', 'mental retardation', 'Global developmental delay'], ['hepatocellular carcinoma', 'colorectal carcinoma', 'calf circumference measurement', 'cholangiocarcinoma', 'cancer'], ['leukoencephalopathy, motor delay, spasticity, and dysarthria syndrome', 'reticulocyte count', 'reticulocyte measurement', 'mean corpuscular volume', 'Immature Reticulocyte Fraction Measurement'], ['leukoencephalopathy, developmental delay, and episodic neurologic regression syndrome', 'influenza', 'platelet count', 'adolescent idiopathic scoliosis', 'Cognitive impairment'], ['Pulmonary capillary hemangiomatosis', 'Pulmonary venoocclusive disease', 'Heritable pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'Non-syndromic pontocerebellar hypoplasia'], ['Ovarioleukodystrophy', 'CACH syndrome', 'Cree leukoencephalopathy', 'Juvenile or adult CACH syndrome', 'Late infantile CACH syndrome'], ['head and neck squamous cell carcinoma', 'colon adenocarcinoma', 'pancreatic carcinoma', 'esophageal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['age at menopause', 'menopause', 'breast cancer', 'renal cell carcinoma', 'neoplasm'], ['Autosomal dominant severe congenital neutropenia', 'Cyclic neutropenia', 'Severe congenital neutropenia', 'Constitutional neutropenia', 'neutropenia'], ['HIV-1 infection', 'viral load', 'neoplasm', 'breast cancer', 'melanoma'], ['urinary bladder cancer', 'matrix metalloproteinase 7 measurement', 'hepatocellular carcinoma', 'cholangiocarcinoma', 'lung adenocarcinoma'], ['B-cell acute lymphoblastic leukemia', 'lymphocyte count', 'small intestine neuroendocrine tumor', 'arterial stiffness measurement', 'FEV/FEC ratio'], ['Supravalvular aortic stenosis', 'Autosomal dominant cutis laxa', 'genetic disorder', 'Williams syndrome', 'Cutis laxa'], ['intelligence', 'chronic kidney disease', 'urinary metabolite measurement', 'leukemia', 'infect'], ['schizophrenia', 'type II diabetes mellitus', 'leukocyte count', 'hair colour measurement', 'myeloid white cell count'], ['X-linked Emery-Dreifuss muscular dystrophy', 'Emery-Dreifuss muscular dystrophy', 'neuromuscular disease', 'Abnormality of the cardiovascular system', 'myopathy'], ['non-small cell lung carcinoma', 'neoplasm', 'lung adenocarcinoma', 'forced expiratory volume', 'lung carcinoma'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'self reported educational attainment', 'lifestyle measurement', 'C-reactive protein measurement'], ['leukocyte count', 'body mass index', 'neuroticism measurement', 'breast adenocarcinoma', 'fat body mass'], ['blood protein measurement', 'brain injury', 'optic disc size measurement', 'age at menarche', 'unipolar depression'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'Cerebral arteriovenous malformation', 'Heritable pulmonary arterial hypertension', 'genetic disorder'], ['neutrophil count', 'Tuberculosis', 'platelet count', 'myeloid white cell count', 'neutrophil percentage of leukocytes'], ['protein measurement', 'uterine fibroid', 'chronic kidney disease', 'urinary metabolite measurement', 'erythrocyte count'], ['hypertension', 'systolic blood pressure', 'blood pressure', 'diastolic blood pressure', 'Diuretic use measurement'], ['Generalized arterial calcification of infancy', 'Autosomal recessive hypophosphatemic rickets', 'Hypopigmentation-punctate palmoplantar keratoderma syndrome', 'type II diabetes mellitus', 'ossification of the posterior longitudinal ligament of the spine'], ['lean body mass', 'appendicular lean mass', 'vital capacity', 'brain volume measurement', 'forced expiratory volume'], ['alkaline phosphatase measurement', 'type II diabetes mellitus', 'body mass index', 'vitamin D measurement', 'testosterone measurement'], ['Autosomal recessive spastic paraplegia type 64', 'protein measurement', 'platelet count', 'electrocardiography', 'DNA methylation'], ['experimental autoimmune encephalomyelitis', 'intrahepatic cholangiocarcinoma', 'chronic obstructive pulmonary disease', 'monocyte percentage of leukocytes', 'hepatocellular carcinoma'], ['eosinophil percentage of leukocytes', 'prostate carcinoma', 'renal cell carcinoma', 'colorectal cancer', 'eosinophil count'], ['Microcephaly - polymicrogyria - corpus callosum agenesis', 'rheumatoid arthritis', 'eosinophil count', 'multiple sclerosis', 'myeloid white cell count'], ['Rubinstein-Taybi syndrome', 'Rubinstein-Taybi syndrome due to EP300 haploinsufficiency', 'head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Menke-Hennekam syndrome 2'], ['Autosomal dominant secondary polycythemia', 'Paraganglioma', 'Primary familial polycythemia', 'renal cell carcinoma', 'electrocardiography'], ['Intestinal epithelial dysplasia', 'Lynch syndrome', 'colorectal cancer, hereditary nonpolyposis, type 8', 'neoplasm', 'hereditary nonpolyposis colorectal carcinoma'], ['cancer', 'cytokine measurement', 'response to vaccine', 'hair color', 'balding measurement'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'sex hormone-binding globulin measurement', 'non-small cell lung carcinoma'], ['Non-syndromic congenital cataract', 'Total congenital cataract', 'Posterior polar cataract', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'neuroimaging measurement'], ['thyroid cancer', 'neoplasm', 'heel bone mineral density', 'thyroid carcinoma', 'hair colour measurement'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'mathematical ability', 'non-small cell lung carcinoma'], ['Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'Familial prostate cancer', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma'], ['Capillary malformation - arteriovenous malformation', 'hydrops fetalis, nonimmune, and/or atrial septal defect, susceptibility to', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma'], ['Primary familial polycythemia', 'Blackfan-Diamond anemia', 'Autosomal dominant secondary polycythemia', 'hematocrit', 'hemoglobin measurement'], ['leukodystrophy, hypomyelinating, 15', 'heel bone mineral density', 'glomerular filtration rate', 'bone quantitative ultrasound measurement', 'reticulocyte count'], ['eosinophil peroxidase deficiency', 'eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'Neutropenia - monocytopenia - deafness'], ['psoriasis', 'ankylosing spondylitis', 'systolic blood pressure', 'endoplasmic reticulum aminopeptidase 1 measurement', 'platelet count'], ['non-small cell lung carcinoma', 'breast carcinoma', 'cancer', 'neoplasm', 'breast neoplasm'], ['Lethal congenital contracture syndrome type 2', 'gastric adenocarcinoma', 'cancer', 'neoplasm', 'bladder transitional cell carcinoma'], ['non-small cell lung carcinoma', 'breast carcinoma', 'neoplasm', 'amyotrophic lateral sclerosis', 'cancer'], ['papillary thyroid carcinoma', 'melanocytic neoplasm', 'breast ductal adenocarcinoma', 'Thyroid Gland Oncocytic Follicular Carcinoma', 'vital capacity'], ['Cockayne syndrome', 'Cockayne syndrome type 1', 'Cockayne syndrome type 2', 'Cockayne syndrome type 3', 'UV-sensitive syndrome'], ['cancer', 'endometrial cancer', 'lung adenocarcinoma', 'multiple myeloma', 'cutaneous melanoma'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'hemoglobin measurement', 'red blood cell distribution width'], ['type II diabetes mellitus', 'sex hormone-binding globulin measurement', "Crohn's disease", 'ulcerative colitis', 'ankylosing spondylitis'], ['heel bone mineral density', 'bone density', 'ulcer disease', 'bone quantitative ultrasound measurement', 'mean corpuscular hemoglobin'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'low grade glioma', 'blood protein measurement', 'endothelial cell-selective adhesion molecule measurement'], ['Roberts syndrome', 'Roberts-SC phocomelia syndrome', 'acute myeloid leukemia', 'endometrial cancer', 'body height'], ['S-formylglutathione hydrolase measurement', 'protein measurement', 'cerebrospinal fluid biomarker measurement', 'systolic blood pressure', 'platelet component distribution width'], ['endothelial cell-specific molecule 1 measurement', 'FEV/FEC ratio', 'neoplasm', 'prostate cancer', 'urinary bladder cancer'], ['breast carcinoma', 'breast cancer', 'breast neoplasm', 'neoplasm', 'breast adenocarcinoma'], ['schizophrenia', 'breast cancer', 'neoplasm', 'postmenopausal osteoporosis', 'osteoporosis'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Elevated circulating glutaric acid concentration', 'Glutaric aciduria', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'Multiple acyl-CoA dehydrogenation deficiency, mild type'], ['type II diabetes mellitus', 'lymphocyte count', 'platelet count', 'hair color', 'esophageal adenocarcinoma'], ['body mass index', 'high density lipoprotein cholesterol measurement', 'physical activity measurement', 'glomerular filtration rate', 'lean body mass'], ['body height', 'blood protein measurement', 'rheumatoid arthritis', 'refractory anemia', 'dilated cardiomyopathy'], ['mean corpuscular volume', 'eosinophil count', 'mean corpuscular hemoglobin', 'eosinophil percentage of leukocytes', 'benign neoplasm'], ['heart disease', 'mean corpuscular hemoglobin concentration', 'Cutaneous T-cell lymphoma', 'heel bone mineral density', 'bone density'], ['Pontocerebellar hypoplasia type 1', 'Non-syndromic pontocerebellar hypoplasia', 'Microcephaly', 'genetic disorder', 'Severe intrauterine growth retardation'], ['acute myeloid leukemia', 'multiple myeloma', 'B-cell non-Hodgkins lymphoma', 'unipolar depression', 'depressive disorder'], ['exostoses, multiple, type 1', 'Multiple osteochondromas', 'hereditary multiple exostoses', 'chondrosarcoma', 'Langer-Giedion syndrome'], ['immunoskeletal dysplasia with neurodevelopmental abnormalities', 'Skeletal dysplasia - intellectual disability', 'body height', 'white matter microstructure measurement', 'acute myeloid leukemia'], ['body height', 'waist-hip ratio', 'Mantle cell lymphoma', 'type II diabetes mellitus', 'cirrhosis of liver'], ['Weaver syndrome', 'lymphoma', 'non-Hodgkins lymphoma', 'melanoma', 'breast carcinoma'], ['Congenital factor II deficiency', 'stroke', 'venous thromboembolism', 'deep vein thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency'], ['myocardial infarction', 'peripheral arterial disease', 'stroke', 'coronary artery disease', 'reticulocyte count'], ['mean platelet volume', 'glioblastoma multiforme', 'proteinase-activated receptor 1 measurement', 'monocyte count', 'head and neck squamous cell carcinoma'], ['tissue factor measurement', 'D dimer measurement', 'coagulation factor measurement', 'cervical cancer', 'neutrophil count'], ['Congenital factor V deficiency', 'thrombophilia due to activated protein C resistance', 'Venous thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency', 'Sepsis'], ['Congenital factor VII deficiency', 'Abnormality of coagulation', 'factor VII deficiency', 'factor VII measurement', 'prothrombin time measurement'], ['Hemophilia A', 'Hemophilia B', 'Sepsis', 'Mild hemophilia A', 'Abnormality of coagulation'], ['Hemophilia B', 'Hemophilia A', 'Venous thrombosis', 'thrombophilia, X-linked, due to factor 9 defect', 'warfarin sensitivity, X-linked'], ['Congenital factor X deficiency', 'atrial fibrillation', 'venous thromboembolism', 'deep vein thrombosis', 'pulmonary embolism'], ['Congenital factor XI deficiency', 'Hereditary angioedema', 'factor XI measurement', 'Abnormal bleeding', 'venous thromboembolism'], ['Congenital factor XII deficiency', 'Hereditary angioedema', 'Hereditary angioedema type 3', 'Hereditary angioedema type 1', 'angioedema'], ['Autosomal recessive spastic paraplegia type 35', 'Hereditary spastic paraplegia', 'Spastic paraplegia', 'genetic disorder', 'Fatty acid hydroxylase-associated neurodegeneration'], ['migraine disorder', 'pain', 'Fever', 'osteoarthritis', 'Low back pain'], ['C-reactive protein measurement', 'serum alanine aminotransferase measurement', 'blood protein measurement', 'alkaline phosphatase measurement', 'hair color'], ['protein measurement', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract'], ['breast adenocarcinoma', 'gestational diabetes', 'neoplasm', 'hydrops fetalis (disease)', 'Hand-foot syndrome'], ['pathological myopia', 'hepatocellular carcinoma', 'neoplasm', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis'], ['FADD-related immunodeficiency', 'Abnormality of cardiovascular system morphology', 'genetic disorder', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['red blood cell distribution width', 'mean platelet volume', 'platelet count', 'chronic kidney disease', 'serum metabolite measurement'], ['red blood cell distribution width', 'blood metabolite measurement', 'mean platelet volume', 'chronic kidney disease', 'serum metabolite measurement'], ['hemoglobin measurement', 'mean corpuscular volume', 'hematocrit', 'Pentosuria', 'HbA1c measurement'], ['Lethal osteosclerotic bone dysplasia', 'Hypoplastic amelogenesis imperfecta', 'Isolated focal cortical dysplasia type II', 'Midline cerebral malformation', 'Neonatal death'], ['lymphocyte count', 'body height', 'basophil count', 'neutrophil count', 'balding measurement'], ['chronotype measurement', 'age at menarche', 'potassium measurement', 'adolescent idiopathic scoliosis', 'sleep duration'], ['neuroimaging measurement', 'brain volume measurement', 'brain measurement', 'cortical thickness', 'platelet count'], ['Hypomyelination - congenital cataract', 'Leukodystrophy', 'adolescent idiopathic scoliosis', 'Thrombocytopenia', 'attention deficit hyperactivity disorder'], ['Fanconi anemia', 'VACTERL with hydrocephalus', 'Fanconi anemia complementation group A', 'VACTERL association, X-linked, with or without hydrocephalus', 'acute myeloid leukemia'], ['Fanconi anemia', 'acute myeloid leukemia', 'Fanconi anemia complementation group A', 'leukemia', 'colorectal adenocarcinoma'], ['Fanconi anemia', 'Fanconi anemia complementation group L', 'Fanconi anemia complementation group A', 'acute myeloid leukemia', 'Abnormality of radial ray'], ['blind loop syndrome', 'leukocyte count', 'eosinophil count', 'autoimmune disease', 'glomerular filtration rate'], ['Autoimmune lymphoproliferative syndrome', 'cutaneous melanoma', 'adult lymphoma', "Hashimoto's thyroiditis", 'chronic lymphocytic leukemia'], ['Autoimmune lymphoproliferative syndrome', 'celiac disease', 'allergy', 'Eczema', 'neutrophil count'], ['autism spectrum disorder', 'esophageal squamous cell carcinoma', 'colorectal carcinoma', 'esophageal adenocarcinoma', 'Epileptic encephalopathy'], ['influenza', 'clear cell renal carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'gastric carcinoma'], ['Autosomal recessive cutis laxa type 1', 'Autosomal dominant cutis laxa', 'Hereditary sensorimotor neuropathy with hyperelastic skin', 'age-related macular degeneration', 'Charcot-Marie-Tooth disease'], ['Marfan syndrome', 'Isolated ectopia lentis', 'Geleophysic dysplasia', 'Acromicric dysplasia', 'Marfan syndrome type 1'], ['Short stature due to GHSR deficiency', 'cancer', 'pulse pressure measurement', 'breast cancer', 'schistosomiasis'], ['autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'ABCD syndrome', 'Waardenburg-Shah syndrome'], ['chronotype measurement', 'diet measurement', 'blood protein measurement', 'refractory anemia', 'rheumatoid arthritis'], ['chin morphology measurement', 'electrocardiography', 'hepatocellular carcinoma', 'bone mineral content measurement', 'cancer'], ['intellectual developmental disorder with dysmorphic facies and behavioral abnormalities', 'gastric adenocarcinoma', 'non-Hodgkins lymphoma', 'basal cell carcinoma', 'endometrial carcinoma'], ['Split hand-split foot malformation', 'Distal limb deficiencies - micrognathia syndrome', 'lung adenocarcinoma', 'body height', 'head and neck squamous cell carcinoma'], ['serum IgE measurement', 'CCL2 measurement', 'seasonal allergic rhinitis', 'growth-regulated alpha protein measurement', 'self reported educational attainment'], ['blood protein measurement', 'low affinity immunoglobulin epsilon Fc receptor measurement', 'cerebrospinal fluid biomarker measurement', 'neutrophil count', 'thyroid gland adenocarcinoma'], ['breast carcinoma', 'Autosomal agammaglobulinemia', 'Hyper-IgM syndrome type 4', 'Hyper-IgM syndrome type 2', 'Hyper-IgM syndrome type 5'], ['inflammatory bowel disease', 'cutaneous Leishmaniasis', 'ulcerative colitis', 'systemic lupus erythematosus', 'mucocutaneous lymph node syndrome'], ['systemic lupus erythematosus', 'breast ductal adenocarcinoma', 'head and neck squamous cell carcinoma', 'thyroid carcinoma', 'low affinity immunoglobulin gamma Fc region receptor II-a/b measurement'], ['inflammatory bowel disease', 'Hyper-IgM syndrome type 3', 'Autosomal agammaglobulinemia', 'Primary systemic amyloidosis', 'Neutrophil immunodeficiency syndrome'], ['autosomal recessive primary immunodeficiency with defective spontaneous natural killer cell cytotoxicity', 'cutaneous Leishmaniasis', 'Primary immunodeficiency', 'basophil count', 'hematocrit'], ['monocyte count', 'neutrophil count', 'basophil percentage of leukocytes', 'serum IgG measurement', 'hematocrit'], ['sex hormone-binding globulin measurement', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'red blood cell density measurement'], ['coronary artery disease', 'blood protein measurement', 'Graves disease', 'rheumatoid arthritis', 'multiple sclerosis'], ['rheumatoid arthritis', 'Graves disease', 'blood protein measurement', 'Fc receptor-like protein 3 measurement', 'protein measurement'], ['systemic lupus erythematosus', 'myeloid white cell count', 'adolescent idiopathic scoliosis', 'multiple myeloma', 'myeloid neoplasm'], ['lymphocyte count', 'erythrocyte count', 'blood protein measurement', 'monocyte count', 'lymphocyte percentage of leukocytes'], ['Disseminated superficial actinic porokeratosis', 'osteoporosis', 'postmenopausal osteoporosis', 'multiple myeloma', 'osteitis deformans'], ['Congenital adrenal hyperplasia', 'coronary artery disease', 'lean body mass', 'fat body mass', 'body height'], ['auditory neuropathy-optic atrophy syndrome', 'genetic disorder', 'Congenital adrenal hyperplasia', 'pancreatic carcinoma', 'low grade glioma'], ['Erythropoietic protoporphyria', 'protoporphyria, erythropoietic, 2', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume'], ['HIV infection', 'multiple myeloma', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'B-cell non-Hodgkins lymphoma'], ['body height', 'intraocular pressure measurement', 'vital capacity', 'forced expiratory volume', 'BMI-adjusted waist circumference'], ['head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'ovarian carcinoma', 'hemangioblastoma'], ['self reported educational attainment', 'fat body mass', 'body mass index', 'body fat percentage', 'lean body mass'], ['type II diabetes mellitus', 'diabetes mellitus', 'Abnormal blood glucose concentration', 'endometrial cancer', 'glioblastoma multiforme'], ['monocyte percentage of leukocytes', 'neutrophil count', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia'], ['Aarskog-Scott syndrome', 'Aarskog-Scott syndrome, X-linked', 'genetic disorder', 'X-linked intellectual disability, Siderius type', 'thyroid gland adenocarcinoma'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'waist-hip ratio'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Acrocephalosyndactyly'], ['Deafness with labyrinthine aplasia, microtia, and microdontia', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma'], ['cancer', 'bone development disease', 'Craniosynostosis', 'ovarian carcinoma', 'female reproductive endometrioid cancer'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Thyrotoxicosis'], ['hypogonadotropic hypogonadism 6 with or without anosmia', 'Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'Holoprosencephaly', 'cancer'], ['Multiple synostoses syndrome', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma'], ['Lacrimoauriculodentodigital syndrome', 'Aplasia of lacrimal and salivary glands', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer'], ['breast adenocarcinoma', 'endometrial cancer', 'glioblastoma multiforme', 'gastric adenocarcinoma', 'neoplasm'], ['cancer', 'bone development disease', 'lean body mass', 'body height', 'appendicular lean mass'], ['cancer', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'hair colour measurement'], ['diet measurement', 'energy intake', 'triglyceride measurement', 'Abnormal urine sodium concentration', 'protein intake measurement'], ['Autosomal dominant hypophosphatemic rickets', 'tumoral calcinosis, hyperphosphatemic, familial, 2', 'familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'cancer', 'bone development disease'], ['Hartsfield-Bixler-Demyer syndrome', 'hypogonadotropic hypogonadism 2 with or without anosmia', 'Kallmann syndrome', 'Osteoglophonic dwarfism', 'Pfeiffer syndrome'], ['Crouzon disease', 'Pfeiffer syndrome', 'Cutis gyrata - acanthosis nigricans - craniosynostosis', 'Apert syndrome', 'Jackson-Weiss syndrome'], ['Achondroplasia', 'Thanatophoric dysplasia type 1', 'Muenke syndrome', 'Severe achondroplasia - developmental delay - acanthosis nigricans', 'Crouzon syndrome - acanthosis nigricans'], ['rhabdomyosarcoma', 'breast carcinoma', 'cancer', 'bone development disease', 'idiopathic pulmonary fibrosis'], ['blood protein measurement', 'FEV/FEC ratio', 'self reported educational attainment', 'rheumatoid arthritis', 'refractory anemia'], ['Hereditary leiomyomatosis and renal cell cancer', 'Fumaric aciduria', 'Multiple cutaneous leiomyomas', 'Inherited cancer-predisposing syndrome', 'Endometrial Endometrioid Adenocarcinoma'], ['Reducing body myopathy', 'X-linked myopathy with postural muscle atrophy', 'Emery-Dreifuss muscular dystrophy', 'X-linked Emery-Dreifuss muscular dystrophy', 'Uruguay Faciocardiomusculoskeletal syndrome'], ['atopic eczema', 'chronic kidney disease', 'acute graft vs. host disease', 'kidney disease', 'renal cell carcinoma'], ['Familial atrial fibrillation', 'lifestyle measurement', 'Brugada syndrome', 'Familial short QT syndrome', 'Pallister-Hall syndrome'], ['cognitive function measurement', 'diet measurement', 'eosinophil count', 'lean body mass', 'body mass index'], ['testosterone measurement', 'sex hormone-binding globulin measurement', 'BMI-adjusted waist-hip ratio', 'vital capacity', 'hematocrit'], ['atopic eczema', 'Inherited ichthyosis', 'Eczema', 'ichthyosis vulgaris', 'genetic disorder'], ['Ewing sarcoma', 'Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'osteosarcoma', 'body height', 'platelet count'], ['response to statin', 'hair colour measurement', 'Flexion contracture', 'pathological myopia', 'neoplasm'], ['Periventricular nodular heterotopia', 'Osteodysplasty, Melnick-Needles type', 'Frontometaphyseal dysplasia', 'Otopalatodigital syndrome type 2', 'Otopalatodigital syndrome type 1'], ['Autosomal dominant Larsen syndrome', 'Spondylocarpotarsal synostosis', 'Atelosteogenesis type III', 'Atelosteogenesis type I', 'Boomerang dysplasia'], ['Muscle filaminopathy', 'hypertrophic cardiomyopathy', 'Familial isolated restrictive cardiomyopathy', 'Distal myopathy with posterior leg and anterior hand involvement', 'dilated cardiomyopathy'], ['lung adenocarcinoma', 'mathematical ability', 'cognitive function measurement', 'Niemann-Pick disease type C', 'neoplasm'], ['renal cell carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'Gastrointestinal stromal tumor', 'idiopathic pulmonary fibrosis'], ['acute myeloid leukemia', 'hepatocellular carcinoma', 'acute lymphoblastic leukemia', 'renal cell carcinoma', 'cancer'], ['cancer', 'monocyte count', 'monocyte percentage of leukocytes', 'thyroid gland adenocarcinoma', 'neoplasm'], ['Milroy disease', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'Gastrointestinal stromal tumor', 'thyroid carcinoma'], ['Posterior column ataxia - retinitis pigmentosa', 'Retinitis pigmentosa', 'genetic disorder', 'Retinal dystrophy', 'Hereditary sensory and autonomic neuropathy'], ['Fowler syndrome', 'Posterior column ataxia - retinitis pigmentosa', 'multiple sclerosis', 'marginal zone B-cell lymphoma', 'small cell lung carcinoma'], ['hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'hair shape measurement', 'red blood cell density measurement'], ['Fragile X syndrome', 'Fragile X-associated tremor/ataxia syndrome', 'primary ovarian insufficiency', 'Intellectual disability', 'mental retardation'], ["Spondylometaphyseal dysplasia, 'corner fracture' type", 'Fibronectin glomerulopathy', 'cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy'], ['intraocular pressure measurement', 'appendicular lean mass', 'red blood cell distribution width', 'body height', 'lean body mass'], ['triglyceride measurement', 'intraocular pressure measurement', 'blood protein measurement', 'chronic kidney disease', 'urinary metabolite measurement'], ['Neurodegenerative syndrome due to cerebral folate transport deficiency', 'Seizure', 'ovarian cancer', 'neoplasm', 'Epileptic encephalopathy'], ['body height', "Barrett's esophagus", 'neoplasm', 'rheumatoid arthritis', 'Acute hepatic failure'], ['Berardinelli-Seip congenital lipodystrophy', "Crohn's disease", 'heel bone mineral density', 'inflammatory bowel disease', 'glioblastoma multiforme'], ['Bamforth-Lazarus syndrome', 'Familial papillary or follicular thyroid carcinoma', 'follicular thyroid carcinoma', 'Congenital hypothyroidism', 'choanal atresia'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'appendicular lean mass', 'mean corpuscular hemoglobin concentration'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'low grade glioma', 'mean corpuscular hemoglobin concentration', 'chronic lymphocytic leukemia'], ['sex hormone-binding globulin measurement', 'appendicular lean mass', 'mathematical ability', 'vital capacity', 'body height'], ['mental retardation with language impairment', 'Intellectual disability-severe speech delay-mild dysmorphism syndrome', 'basal cell carcinoma', 'skin carcinoma', 'esophageal adenocarcinoma'], ['Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'type I diabetes mellitus', 'hydrops fetalis (disease)', 'azoospermia', 'Fanconi anemia'], ['periodontitis', 'ovarian cancer', 'pancreatic adenocarcinoma', 'eosinophil count', 'lymphocyte count'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'prostate adenocarcinoma'], ['self reported educational attainment', 'appendicular lean mass', 'body height', 'glomerular filtration rate', 'follistatin measurement'], ['body height', 'blood protein measurement', 'appendicular lean mass', 'refractive error measurement', 'neoplasm'], ['FTH1-related iron overload', 'Autosomal recessive bestrophinopathy', 'Hemochromatosis type 3', 'squamous cell lung carcinoma', 'red blood cell distribution width'], ['Hereditary hyperferritinemia with congenital cataracts', 'Neuroferritinopathy', 'L-ferritin deficiency', 'movement disorder', 'Genetic hyperferritinemia without iron overload'], ['beta-amyloid 1-42 measurement', 'cerebrospinal fluid biomarker measurement', 'left inferior lateral ventricle volume measurement', 'systolic blood pressure', 'Young adult-onset Parkinsonism'], ['Lethal polymalformative syndrome, Boissel type', 'obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'Genetic obesity', 'type II diabetes mellitus'], ['X-linked non-syndromic intellectual disability', 'mental retardation', 'genetic disorder', 'Intellectual disability', 'non-small cell lung carcinoma'], ['hematocrit', 'hemoglobin measurement', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'myeloid white cell count'], ['chronic lymphocytic leukemia', 'glioma', 'anaplastic oligodendroglioma', 'gastric carcinoma', 'hemangioblastoma'], ['anthrax infection', 'COVID-19', 'HIV infection', 'ovarian cancer', 'lung adenocarcinoma'], ['Frontotemporal dementia with motor neuron disease', 'sporadic amyotrophic lateral sclerosis', 'amyotrophic lateral sclerosis', 'essential tremor', 'Juvenile amyotrophic lateral sclerosis'], ['urinary albumin to creatinine ratio', 'albuminuria', 'alkaline phosphatase measurement', 'breakfast skipping measurement', 'diet measurement'], ['galactoside 34-L-fucosyltransferase measurement', 'e-selectin measurement', 'red blood cell distribution width', 'galectin-3 measurement', 'cancer antigen 19.9 measurement'], ['congenital disorder of glycosylation with defective fucosylation', 'Intellectual disability', 'Seizure', 'COVID-19', 'genetic disorder'], ['Friedreich ataxia', 'hereditary ataxia', 'Abnormality of the cardiovascular system', 'Headache', 'hypertrophic cardiomyopathy'], ['hematocrit', 'hemoglobin measurement', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'hypertrophic cardiomyopathy 20'], ['Hereditary thrombocytopenia with normal platelets', 'thrombocytopenia 3', 'Autosomal thrombocytopenia with normal platelets', 'mean corpuscular volume', 'blood platelet disease'], ['cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cutaneous Leishmaniasis', 'neoplasm'], ['Familial exudative vitreoretinopathy', 'Coats disease', 'Persistent hyperplastic primary vitreous', 'Retinal dystrophy', 'Norrie disease'], ['Glycogen storage disease due to glucose-6-phosphatase deficiency type a', 'Glycogen storage disease due to glucose-6-phosphatase deficiency', 'Glycogen storage disease', 'genetic disorder', 'Short stature due to GHSR deficiency'], ['anemia, nonspherocytic hemolytic, due to G6PD deficiency', 'anemia (phenotype)', 'class I glucose-6-phosphate dehydrogenase deficiency', 'congenital nonspherocytic hemolytic anemia', 'glucosephosphate dehydrogenase deficiency'], ['Glycogen storage disease due to acid maltase deficiency', 'Glycogen storage disease due to acid maltase deficiency, adult onset', 'Glycogen storage disease due to acid maltase deficiency, infantile onset', 'type II diabetes mellitus', 'Glycogen storage disease'], ['neoplasm', 'cancer', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'chronic kidney disease'], ['Inherited congenital spastic tetraplegia', 'developmental and epileptic encephalopathy', 'sex hormone-binding globulin measurement', 'cognitive function measurement', 'mathematical ability'], ['arterial stiffness measurement', 'adolescent idiopathic scoliosis', 'thyroid gland adenocarcinoma', 'cutaneous melanoma', 'obesity'], ['body mass index', 'body fat percentage', 'vital capacity', 'cleft lip', 'forced expiratory volume'], ['glomerular filtration rate', 'blood metabolite measurement', 'appendicular lean mass', 'body height', 'acute myeloid leukemia'], ["Parkinson's disease", 'triglyceride measurement', 'glioblastoma multiforme', 'high density lipoprotein cholesterol measurement', 'Ehlers-Danlos syndrome, progeroid type'], ['temporal lobe epilepsy', 'body height', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'Seizure'], ['white matter microstructure measurement', 'neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'sporadic Creutzfeld Jacob disease'], ['Krabbe disease', 'Seizure', 'body mass index', 'genetic disorder', 'Adult Krabbe disease'], ['head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'mononeuropathy', 'colitis', 'Severe combined immunodeficiency due to CORO1A deficiency'], ['heel bone mineral density', 'colitis', 'Homozygous familial hypercholesterolemia', 'sitting height measurement', 'bone quantitative ultrasound measurement'], ['endometrial cancer', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'neoplasm', 'non-small cell lung carcinoma', 'amyotrophic lateral sclerosis'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2D', 'Distal hereditary motor neuropathy type 5', 'Charcot-Marie-Tooth disease', 'Distal hereditary motor neuropathy type 2', 'spinal muscular atrophy, infantile, James type'], ['triglyceride measurement', 'red blood cell distribution width', 'low density lipoprotein cholesterol measurement', 'erythrocyte count', 'hemoglobin measurement'], ['Hypoparathyroidism - deafness - renal disease', 'breast adenocarcinoma', 'breast carcinoma', 'asthma', 'eosinophil count'], ['Interauricular communication', 'Ventricular septal defect', 'Single ventricular septal defect', 'Tetralogy of Fallot', 'Genetic cardiac anomaly'], ['Gaucher disease', 'Gaucher disease type 1', 'Fetal Gaucher disease', 'Gaucher disease type 2', 'Gaucher disease type 3'], ['neoplasm', 'infection', 'infect', 'hepatocellular carcinoma', 'glioma'], ['type II diabetes mellitus', 'hypothyroidism', 'psoriasis', 'lymphocyte count', 'leukocyte count'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'medulloblastoma', 'myeloid white cell count', 'hepatocellular carcinoma'], ['type I diabetes mellitus', 'infertility', 'diabetes mellitus', 'Hypoglycemia', 'obesity'], ['triglyceride measurement', 'neoplasm', 'nasopharyngeal squamous cell carcinoma', 'nasopharyngeal neoplasm', 'lung cancer'], ['MODY', 'Permanent neonatal diabetes mellitus', 'Hyperinsulinism due to glucokinase deficiency', 'type II diabetes mellitus', 'DEND syndrome'], ['type II diabetes mellitus', 'triglyceride measurement', 'lean body mass', 'total cholesterol measurement', 'platelet count'], ['Gamma-glutamylcysteine synthetase deficiency', 'hemolytic anemia', 'monocyte count', 'monocyte percentage of leukocytes', 'low density lipoprotein cholesterol measurement'], ['glioblastoma multiforme', 'colorectal carcinoma', 'gut microbiome measurement', 'taxonomic microbiome measurement', 'polymyalgia rheumatica'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'colorectal neoplasm', 'erythrocyte count'], ['Colobomatous microphthalmia', 'Isolated anophthalmia - microphthalmia', 'Isolated Klippel-Feil syndrome', 'microphthalmia', 'neoplasm'], ['prostate carcinoma', 'hemorrhoid', 'pulse pressure measurement', 'esophageal adenocarcinoma', "Barrett's esophagus"], ['Orofacial clefting syndrome', 'orofacial cleft', 'chronic obstructive pulmonary disease', 'stroke', 'Hepatic fibrosis'], ['growth differentiation factor 15 measurement', 'coffee consumption measurement', 'hyperemesis gravidarum', 'nausea and vomiting of pregnancy severity measurement', 'obesity'], ['Hirschsprung disease', 'Ondine syndrome', 'Haddad syndrome', 'prostate carcinoma', 'blood protein measurement'], ['systolic blood pressure', 'coronary artery disease', 'pulse pressure measurement', 'alcohol drinking', 'systemic lupus erythematosus'], ['Alexander disease', 'Autosomal dominant cerebellar ataxia', 'Seizure', 'Alexander disease type I', 'Alexander disease type II'], ['Hepatoencephalopathy due to combined oxidative phosphorylation deficiency type 1', 'combined oxidative phosphorylation deficiency', 'systolic blood pressure', 'blood protein measurement', 'PR interval'], ['Congenital myasthenic syndromes', 'Congenital myasthenic syndromes with glycosylation defect', 'congenital disorder of glycosylation type I', 'vertebral disease', 'musculoskeletal system disease'], ['airway imaging measurement', 'BMI-adjusted waist circumference', 'body height', 'appendicular lean mass', 'lean body mass'], ['heel bone mineral density', 'body mass index', 'chronotype measurement', 'blood protein measurement', 'arterial stiffness measurement'], ['Non-acquired isolated growth hormone deficiency', 'Short stature due to growth hormone qualitative anomaly', 'Isolated growth hormone deficiency type IB', 'Isolated growth hormone deficiency type II', 'Isolated growth hormone deficiency type IA'], ['Laron syndrome', 'Short stature due to GHSR deficiency', 'pituitary dwarfism', 'Short stature due to partial GHR deficiency', 'acromegaly'], ['obesity', 'ovarian cancer', 'breast adenocarcinoma', 'self reported educational attainment', 'head and neck squamous cell carcinoma'], ['Short stature due to GHSR deficiency', 'lean body mass', 'body height', 'refractive error measurement', 'appendicular lean mass'], ['appendicular lean mass', 'sitting height measurement', 'type II diabetes mellitus', 'systolic blood pressure', 'thyroid gland adenocarcinoma'], ['body mass index', 'lean body mass', 'body fat percentage', 'fat body mass', 'waist circumference'], ['Cataract-microcornea syndrome', 'Non-syndromic congenital cataract', 'Pulverulent cataract', 'cataract', 'Nuclear cataract'], ['Autosomal recessive spastic paraplegia type 44', 'Pelizaeus-Merzbacher-like disease due to GJC2 mutation', 'Milroy disease', 'Pelizaeus-Merzbacher-like disease', 'Spastic paraplegia'], ['Muscular dystrophy', 'gastric cancer', 'gastric carcinoma', 'neoplasm', 'body height'], ['Fabry disease', 'cardiomyopathy', 'hypertrophic cardiomyopathy', 'response to deoxygalactonojirimycin', 'Abnormality of the cardiovascular system'], ['GM1 gangliosidosis type 2', 'GM1 gangliosidosis type 1', 'GM1 gangliosidosis type 3', 'Mucopolysaccharidosis type 4B', 'Mucopolysaccharidosis type 4'], ['Glycine encephalopathy', 'genetic disorder', 'Smith-Magenis syndrome', 'eosinophil count', 'Atypical glycine encephalopathy'], ['Holoprosencephaly', 'postaxial polydactyly-anterior pituitary anomalies-facial dysmorphism syndrome', 'Pallister-Hall syndrome', 'Microform holoprosencephaly', 'genetic disorder'], ['Glomuvenous malformation', 'Blue rubber bleb nevus', 'monocyte count', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells'], ['type II diabetes mellitus', 'diabetes mellitus', 'obesity', 'type I diabetes mellitus', 'diabetic nephropathy'], ['Hereditary hyperekplexia', 'pain', 'coronary artery disease', 'ischemia reperfusion injury', 'kidney failure'], ['global developmental delay, progressive ataxia, and elevated glutamine', 'Early infantile epileptic encephalopathy', 'reticulocyte count', 'mean platelet volume', 'BMI-adjusted waist-hip ratio'], ['amino acid measurement', 'glutamine measurement', 'blood metabolite measurement', 'serum metabolite measurement', 'head and neck squamous cell carcinoma'], ['body mass index', 'pulse pressure measurement', 'cardiovascular disease', 'eosinophil count', 'platelet count'], ['Hyperinsulinism-hyperammonemia syndrome', 'epilepsy', 'gout', 'uric acid measurement', 'hemoglobin measurement'], ['Congenital brain dysgenesis due to glutamine synthetase deficiency', 'Seizure', 'Red cell distribution width', 'lymphocyte count', 'high density lipoprotein cholesterol measurement'], ['body height', 'hematocrit', 'hemoglobin measurement', 'intraocular pressure measurement', 'lean body mass'], ['Sturge-Weber syndrome', 'Familial multiple nevi flammei', 'cutaneous melanoma', 'Uveal Melanoma', 'melanoma'], ['mean platelet volume', 'erosive tooth wear measurement', 'childhood onset asthma', 'eosinophil count', 'DNA methylation'], ['Mucolipidosis type II', 'Mucolipidosis type III', 'Mucolipidosis', 'Sanfilippo syndrome type A', 'Joubert syndrome and related disorders'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Isolated congenital hypogonadotropic hypogonadism', 'age at menopause', 'Isolated thyroid-stimulating hormone deficiency', 'lean body mass'], ['Normosmic congenital hypogonadotropic hypogonadism', 'hypogonadotropic hypogonadism', 'prostate adenocarcinoma', 'prostate cancer', 'infertility'], ['self reported educational attainment', 'neoplasm', 'Cognitive impairment', 'cognitive decline measurement', 'Pallister-Hall syndrome'], ['Early infantile epileptic encephalopathy', 'Failure to thrive', 'Seizure', 'Feeding difficulties', 'microcephaly (disease)'], ['Bernard-Soulier syndrome', 'Pseudo-von Willebrand disease', 'Macrothrombocytopenia', 'Optic neuropathy', 'Autosomal dominant macrothrombocytopenia'], ['body mass index', 'type II diabetes mellitus', 'lean body mass', 'fat body mass', 'insulin sensitivity measurement'], ['platelet component distribution width', 'platelet count', 'mean platelet volume', 'lung adenocarcinoma', 'cutaneous melanoma'], ['prostate carcinoma', "Barrett's esophagus", 'lactose intolerance adult type', 'Autosomal recessive early-onset inflammatory bowel disease', 'Microvillus inclusion disease'], ['red blood cell distribution width', 'glioblastoma multiforme', 'monocyte count', 'neutrophil count', 'leukocyte count'], ['protein measurement', 'body mass index', 'balding measurement', 'cutaneous melanoma', 'lung adenocarcinoma'], ['Simpson-Golabi-Behmel syndrome', 'Nephroblastoma', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma', 'kidney neoplasm'], ['reticulocyte count', 'reticulocyte measurement', 'ovarian cancer', 'prostate carcinoma', 'head and neck squamous cell carcinoma'], ['Hemolytic anemia due to glucophosphate isomerase deficiency', 'hemolytic anemia', 'endometrial cancer', 'breast adenocarcinoma', 'ovarian cancer'], ['alkaline phosphatase measurement', 'mean corpuscular hemoglobin', 'liver enzyme measurement', 'scrapie', 'Abnormal sperm morphology'], ['amyloidosis cutis dyschromia', 'Familial primary localized cutaneous amyloidosis', 'blood protein measurement', "Parkinson's disease", 'cerebrospinal fluid biomarker measurement'], ['ovarian cancer', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'Fibronectin glomerulopathy'], ['refractory anemia', 'rheumatoid arthritis', 'cancer', 'Autosomal agammaglobulinemia', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'myeloid white cell count', 'cutaneous melanoma', 'prostate adenocarcinoma'], ['systolic blood pressure', 'FEV/FEC ratio', 'body height', 'neutrophil count', 'platelet count'], ['neuroblastoma', 'acne', 'periodontitis', 'gastric carcinoma', 'hepatocellular carcinoma'], ['diet measurement', 'reticulocyte count', 'acute myeloid leukemia', 'prostate adenocarcinoma', 'breast cancer'], ['platelet count', 'blood protein measurement', 'mean corpuscular volume', 'Sepsis', "Crohn's disease"], ['endometrial cancer', 'prostate adenocarcinoma', 'attention deficit hyperactivity disorder', 'Hyperinsulinism due to INSR deficiency', 'Autosomal recessive hyperinsulinism due to Kir6.2 deficiency'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'low grade glioma', 'prostate adenocarcinoma'], ['Hyperinsulinism due to INSR deficiency', 'Hyperinsulinism due to glucokinase deficiency', 'Exercise-induced hyperinsulinism', 'Hyperinsulinism due to short chain 3-hydroxylacyl-CoA dehydrogenase deficiency', 'Autosomal recessive hyperinsulinism due to Kir6.2 deficiency'], ['Headache', 'migraine disorder', 'fat body mass', 'hypertrophic cardiomyopathy', 'lean body mass'], ['monocyte count', 'base metabolic rate measurement', 'Benign familial chorea', 'refractive error measurement', 'body weight'], ['sex hormone-binding globulin measurement', 'hip bone mineral density', 'blood protein measurement', 'Sepsis', 'smoking status measurement'], ['triglyceride measurement', 'alkaline phosphatase measurement', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement', 'hair color'], ['heel bone mineral density', 'high density lipoprotein cholesterol measurement', 'hepatocellular carcinoma', 'mean platelet volume', 'eosinophil percentage of leukocytes'], ['sleep duration', 'low density lipoprotein cholesterol measurement', 'blood protein measurement', 'Adult-onset autosomal recessive sideroblastic anemia', 'Hemoglobin E - beta-thalassemia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'cutaneous melanoma', 'erythrocyte count'], ['Chudley-McCullough syndrome', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Rare genetic deafness', 'deafness', 'nonsyndromic deafness'], ['mean corpuscular hemoglobin concentration', 'neoplasm', 'periodontitis', 'self reported educational attainment', 'intelligence'], ['Spondylometaphyseal dysplasia, Sedaghatian type', 'eosinophil count', 'eosinophil percentage of leukocytes', "Crohn's disease", 'neutrophil percentage of granulocytes'], ['thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'lung adenocarcinoma'], ['Hereditary mixed polyposis syndrome', 'colorectal cancer', 'mean corpuscular volume', 'colorectal adenoma', 'mean corpuscular hemoglobin'], ['tooth agenesis', 'FEV/FEC ratio', 'heel bone mineral density', 'bone density', 'bone quantitative ultrasound measurement'], ['Van der Woude syndrome', 'Cleft palate', 'cleft lip', 'genetic disorder', 'head and neck squamous cell carcinoma'], ['neurodevelopmental disorder with language impairment and behavioral abnormalities', 'epilepsy', 'migraine disorder', 'obesity', 'alcohol dependence'], ['neurodevelopmental disorder with or without seizures and gait abnormalities', 'obesity', 'epilepsy', 'migraine disorder', 'alcohol dependence'], ['epilepsy', 'migraine disorder', 'obesity', 'alcohol dependence', 'post-traumatic stress disorder'], ['neurodevelopmental disorder with or without hyperkinetic movements and seizures, autosomal dominant', 'Autosomal dominant non-syndromic intellectual disability', 'Epileptic encephalopathy', 'Intellectual disability', "Parkinson's disease"], ['West syndrome', 'Autosomal dominant non-syndromic intellectual disability', 'Epileptic encephalopathy', 'autism', 'complex neurodevelopmental disorder'], ['protein measurement', 'rheumatic fever', 'osteosarcoma', 'reticulocyte count', 'pulse pressure measurement'], ['unipolar depression', 'hair colour measurement', 'neuroticism measurement', 'depressive symptom measurement', 'wellbeing measurement'], ['Frontotemporal dementia', 'Grn-related frontotemporal lobar degeneration with Tdp43 inclusions', 'Neuronal ceroid lipofuscinosis', 'CLN11 disease', 'Progressive non-fluent aphasia'], ['Short stature-auditory canal atresia-mandibular hypoplasia-skeletal anomalies syndrome', 'acute myeloid leukemia', 'nose morphology measurement', 'Cleft palate', 'cleft lip'], ['bipolar disorder', 'depressive disorder', 'bipolar I disorder', 'cancer', 'major depressive disorder'], ['Familial amyloidosis, Finnish type', 'AL amyloidosis', 'amyloidosis', 'psoriasis', 'hemoglobin measurement'], ['Hemolytic anemia due to glutathione reductase deficiency', 'multiple myeloma', 'glioblastoma multiforme', 'neoplasm', 'medulloblastoma'], ['glioblastoma multiforme', 'blood protein measurement', 'Dent disease type 1', 'Immunoglobulin-mediated membranoproliferative glomerulonephritis', 'Atypical hemolytic-uremic syndrome with DGKE deficiency'], ['HIV infection', 'endometrial cancer', 'glioblastoma multiforme', 'ovarian cancer', 'squamous cell lung carcinoma'], ['lean body mass', 'body fat percentage', 'fat body mass', 'body mass index', 'red blood cell distribution width'], ['testosterone measurement', 'prostate carcinoma', 'creatinine measurement', 'glomerular filtration rate', 'Ringed hair disease'], ['cutaneous melanoma', 'apolipoprotein A 1 measurement', 'Fibronectin glomerulopathy', 'inflammatory bowel disease', 'irritable bowel syndrome'], ['Intestinal obstruction in the newborn due to guanylate cyclase 2C deficiency', 'Chronic diarrhea due to guanylate cyclase 2C overactivity', 'Diarrhea', 'irritable bowel syndrome', 'Constipation'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Central areolar choroidal dystrophy', 'Congenital stationary night blindness', 'Retinal dystrophy'], ['mean corpuscular hemoglobin concentration', 'malaria', 'Hereditary elliptocytosis', 'atrial fibrillation', 'erythrocyte count'], ['Vitiligo', 'blood protein measurement', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['infectious disease', 'viral disease', 'breast carcinoma', 'melanoma', 'cancer'], ['infectious disease', 'viral disease', 'breast carcinoma', 'melanoma', 'cancer'], ['cancer', 'melanoma', 'breast carcinoma', 'neoplasm', 'hepatocellular carcinoma'], ['infectious disease', 'viral disease', 'breast carcinoma', 'cancer', 'melanoma'], ['Hyperandrogenism due to cortisone reductase deficiency', 'heel bone mineral density', 'body height', 'anthropometric measurement', 'blood protein measurement'], ['Histidinemia', 'vitamin D measurement', 'sunburn', 'urinary metabolite measurement', 'chronic kidney disease'], ['Hemochromatosis type 2', 'hemochromatosis type 2B', 'Rare hereditary hemochromatosis', 'hereditary hemochromatosis', 'erythrocyte count'], ['calcium oxalate urolithiasis', 'colorectal cancer', 'protein measurement', 'traffic air pollution measurement', 'Primary hyperoxaluria type 1'], ['folding of antihelix', 'outer ear morphology trait', 'antitragus size', 'balding measurement', 'lobe attachment'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Usher syndrome', 'Usher syndrome type 3', 'Dysarthria', 'mental retardation'], ['balding measurement', 'hematocrit', 'body height', 'red blood cell density measurement', 'erythrocyte count'], ['low density lipoprotein cholesterol measurement', 'blood protein measurement', 'kidney injury molecule 1 measurement', 'neutrophil count', 'lymphocyte count'], ['Subcutaneous Panniculitis-Like T-Cell Lymphoma', 'hepatitis A virus cellular receptor 2 measurement', 'blood protein measurement', 'inflammatory bowel disease', 'neoplasm'], ['Sickle cell anemia', 'Beta-thalassemia intermedia', 'Beta-thalassemia major', 'Beta-thalassemia', 'Heinz body anemia'], ['cancer', 'diphtheria', 'white matter microstructure measurement', 'creatine kinase measurement', 'lung adenocarcinoma'], ['Hemoglobinopathy Toms River', 'cyanosis, transient neonatal', 'fetal hemoglobin measurement', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hereditary persistence of fetal hemoglobin - sickle cell disease'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width', 'mean corpuscular volume'], ['glioblastoma multiforme', 'high density lipoprotein cholesterol measurement', 'waist-hip ratio', 'Red cell distribution width', 'red blood cell distribution width'], ['Methylmalonic acidemia with homocystinuria, type cblX', 'mental retardation', 'genetic disorder', 'X-linked intellectual disability', 'X-linked non-syndromic intellectual disability'], ['PR interval', 'central corneal thickness', 'reticulocyte count', 'appendicular lean mass', 'corneal topography'], ['inflammatory bowel disease', 'mean corpuscular hemoglobin concentration', 'mean reticulocyte volume', 'mean corpuscular volume', 'neoplasm'], ['Early infantile epileptic encephalopathy', 'Generalized epilepsy with febrile seizures-plus', 'undetermined early-onset epileptic encephalopathy', 'developmental and epileptic encephalopathy', 'breast carcinoma'], ['Familial sick sinus syndrome', 'atrial fibrillation', 'Brugada syndrome', 'heart failure', 'cardiac arrhythmia'], ['narcolepsy with cataplexy', 'breast adenocarcinoma', 'lung adenocarcinoma', "Alzheimer's disease", 'unipolar depression'], ['insomnia', 'Sleep Disorder', 'panic disorder', 'post-traumatic stress disorder', 'intelligence'], ['insomnia', 'sleep duration', 'Sleep Disorder', 'circadian rhythm', 'panic disorder'], ['pancreatic adenocarcinoma', 'Intermediate osteopetrosis', 'Wiskott-Aldrich syndrome', 'Combined immunodeficiency with skin granulomas', 'Craniodiaphyseal dysplasia'], ['neoplasm', 'multiple myeloma', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['2q37 microdeletion syndrome', 'multiple myeloma', 'neoplasm', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma'], ['neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['X-linked dominant chondrodysplasia, Chassaing-Lacombe type', 'neoplasm', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'unspecified peripheral T-cell lymphoma'], ['neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['Cornelia de Lange syndrome', 'Wilson-Turner syndrome', 'neoplasm', 'unspecified peripheral T-cell lymphoma', 'multiple myeloma'], ['neoplasm', 'multiple myeloma', 'unspecified peripheral T-cell lymphoma', 'Cutaneous T-cell lymphoma', 'graft versus host disease'], ['blood protein measurement', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement', 'platelet count', 'platelet crit'], ['hemoglobin measurement', 'red blood cell density measurement', 'visceral adipose tissue measurement', 'hematocrit', 'Helicobacter pylori infectious disease'], ['infantile epileptic encephalopathy', 'Blackfan-Diamond anemia', 'breast cancer', 'Alpha-thalassemia - myelodysplastic syndrome', 'infect'], ['Septo-optic dysplasia', 'Combined pituitary hormone deficiencies, genetic forms', 'pituitary hormone deficiency, combined, 1', 'Pituitary stalk interruption syndrome', 'Kallmann syndrome'], ['lean body mass', 'fat body mass', 'systolic blood pressure', 'insomnia', 'appendicular lean mass'], ['mesenchymal chondrosarcoma', 'dedifferentiated chondrosarcoma', 'head and neck squamous cell carcinoma', 'hepatocellular carcinoma', 'esophageal squamous cell carcinoma'], ['hereditary hemochromatosis type 1', 'hereditary hemochromatosis', 'serum iron measurement', 'transferrin measurement', 'Familial porphyria cutanea tarda'], ['primary ovarian insufficiency', 'red blood cell distribution width', 'premature menopause', 'eosinophil count', 'monocyte count'], ['Alkaptonuria', 'Fuchs endothelial corneal dystrophy', 'Methylmalonic aciduria due to transcobalamin receptor defect', 'Posterior polymorphous corneal dystrophy', 'Pentosuria'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'hepatocyte growth factor measurement', 'cancer', 'deafness', 'platelet count'], ['platelet count', 'low density lipoprotein cholesterol measurement', 'testosterone measurement', 'triglyceride measurement', 'alcohol consumption measurement'], ['Tuberculosis', 'listeriosis', 'breast adenocarcinoma', 'low grade glioma', 'endometrial cancer'], ['Non-spherocytic hemolytic anemia due to hexokinase deficiency', 'Retinitis pigmentosa', 'Charcot-Marie-Tooth disease type 4G', 'reticulocyte count', 'hematocrit'], ['HIV infection', 'squamous cell lung carcinoma', 'nasopharyngeal neoplasm', 'melanocytic neoplasm', 'cutaneous melanoma'], ['Stevens-Johnson syndrome', 'spondyloarthropathy', 'head and neck squamous cell carcinoma', 'asthma', 'sex hormone-binding globulin measurement'], ['psoriasis', 'mean corpuscular volume', 'basophil count', 'BMI-adjusted waist-hip ratio', 'smoking behavior'], ['type I diabetes mellitus', 'diabetic retinopathy', 'diabetic maculopathy', 'diabetic ketoacidosis', 'complication'], ['reactive arthritis', 'Sarcoidosis', 'membranous glomerulonephritis', 'susceptibility to Mycobacterium tuberculosis infection measurement', 'wellbeing measurement'], ['anti-neutrophil antibody associated vasculitis', 'hepatitis B virus infection', 'chronic hepatitis B virus infection', 'cervical carcinoma', 'keratinocyte carcinoma'], ['hepatitis B virus infection', 'chronic hepatitis B virus infection', 'allergic rhinitis', 'Eczema', 'dermatomyositis'], ['testosterone measurement', 'autoimmune thyroid disease', 'hemoglobin measurement', 'asthma', 'rheumatoid arthritis'], ['rheumatoid arthritis', 'multiple sclerosis', 'Sarcoidosis', 'systemic lupus erythematosus', "Crohn's disease"], ['platelet crit', 'hemoglobin measurement', 'thyroid gland adenocarcinoma', 'self reported educational attainment', 'mucocutaneous lymph node syndrome'], ['eosinophil percentage of leukocytes', 'smoking status measurement', 'schizophrenia', 'pemphigus foliaceus', 'bipolar disorder'], ['urate measurement', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'esophageal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['diastolic blood pressure', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement', 'triglyceride measurement', 'neoplasm'], ['Acute intermittent porphyria', 'Abdominal pain', 'Vomiting', 'Visual loss', 'Acute episodes of neuropathic symptoms'], ['mean corpuscular volume', 'eosinophil count', 'Primary immunodeficiency', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['heel bone mineral density', 'self reported educational attainment', 'body height', 'systemic lupus erythematosus', 'peak expiratory flow'], ['BMI-adjusted waist circumference', 'monocyte percentage of leukocytes', 'mean reticulocyte volume', 'neutrophil percentage of leukocytes', 'monocyte count'], ['heme oxygenase 1 deficiency', 'amyloidosis', 'glioblastoma multiforme', 'gastric adenocarcinoma', 'chronotype measurement'], ['MODY', 'type II diabetes mellitus', 'hepatocellular adenoma', 'type I diabetes mellitus', 'monogenic diabetes'], ['Renal cysts and diabetes syndrome', 'type II diabetes mellitus', 'renal cell adenocarcinoma', 'Abnormal renal morphology', 'permanent neonatal diabetes mellitus'], ['chronotype measurement', 'mathematical ability', 'sex hormone-binding globulin measurement', 'body fat percentage', 'self reported educational attainment'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'Non-syndromic genetic deafness', 'FEV/FEC ratio', 'autosomal dominant nonsyndromic deafness'], ['lung adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'lung carcinoma', 'acute myeloid leukemia'], ['Synpolydactyly type 1', 'Syndactyly type 5', 'Brachydactyly', 'Brachydactyly type E', 'Brachydactyly-syndactyly, Zhao type'], ['anhaptoglobinemia', 'cutaneous melanoma', 'endometrial cancer', 'blood protein measurement', 'head and neck squamous cell carcinoma'], ['Tyrosinemia type 3', 'Hawkinsinuria', 'tyrosinemia', 'Hypertyrosinemia', 'Alkaptonuria'], ['age at menarche', 'mean platelet volume', 'diastolic blood pressure', 'heel bone mineral density', 'bone quantitative ultrasound measurement'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'Hypercholesterolemia', 'haptoglobin measurement', 'type II diabetes mellitus'], ['Lesch-Nyhan syndrome', 'Hypoxanthine guanine phosphoribosyltransferase partial deficiency', 'gout', 'genetic disorder', 'Microcephaly'], ['Hermansky-Pudlak syndrome', 'Hermansky-Pudlak syndrome with pulmonary fibrosis', 'pulmonary fibrosis', 'Oculocutaneous albinism', 'inflammatory bowel disease'], ['hepatocellular carcinoma', 'blood protein measurement', 'lung cancer', 'neoplasm', 'breast cancer'], ['squamous cell lung carcinoma', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'pes'], ['Costello syndrome', 'Noonan syndrome', 'bladder carcinoma', 'follicular thyroid carcinoma', 'urinary bladder cancer'], ['cirrhosis of liver', 'Hepatic fibrosis', 'Familial progressive cardiac conduction defect', 'Brugada syndrome', 'sudden cardiac arrest'], ['Hereditary thrombophilia due to congenital histidine-rich (poly-L) glycoprotein deficiency', 'dual specificity mitogen-activated protein kinase kinase 4 measurement', 'CD27 antigen measurement', 'histidine-rich glycoprotein measurement', 'blood protein measurement'], ['narcolepsy with cataplexy', 'hypersomnia', 'Meniere disease', 'Vertigo', 'narcolepsy'], ['self reported educational attainment', 'body mass index', 'body fat percentage', 'lean body mass', 'vital capacity'], ['Hyperandrogenism due to cortisone reductase deficiency', 'gastroesophageal reflux disease', 'Peptic ulcer', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['Apparent mineralocorticoid excess', 'Peptic ulcer', 'gastroesophageal reflux disease', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['Perrault syndrome', 'Bifunctional enzyme deficiency', 'hearing loss', 'Rare genetic deafness', 'reticulocyte count'], ['erythrocyte count', 'lean body mass', 'mean corpuscular volume', 'appendicular lean mass', 'age at menopause'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'serum alanine aminotransferase measurement', 'platelet count', 'alcohol abuse'], ['breast adenocarcinoma', 'urinary bladder cancer', 'neoplasm', 'cancer', 'breast cancer'], ['lymphocyte count', 'monocyte count', 'platelet crit', 'myeloid white cell count', 'leukocyte count'], ['cataract 5 multiple types', 'Zonular cataract', 'Non-syndromic congenital cataract', 'cataract', 'Total congenital cataract'], ['mean corpuscular volume', 'myeloperoxidase measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width'], ['cancer', 'influenza', 'diphtheria', 'lung adenocarcinoma', 'Gastrointestinal stromal tumor'], ['diphtheria', 'lung adenocarcinoma', 'Gastrointestinal stromal tumor', 'multiple myeloma', 'COVID-19'], ['reticulocyte count', 'triglyceride measurement', 'heel bone mineral density', 'chronic obstructive pulmonary disease', 'BMI-adjusted waist-hip ratio'], ['breast adenocarcinoma', 'neuroticism measurement', 'smoking behavior', 'neoplasm', 'cancer'], ['even-plus syndrome', 'Autosomal recessive sideroblastic anemia', 'Constitutional sideroblastic anemia', 'breast adenocarcinoma', 'mean corpuscular hemoglobin concentration'], ['breast adenocarcinoma', 'protein measurement', 'blood protein measurement', 'neoplasm', 'hepatocellular carcinoma'], ['Distal hereditary motor neuropathy type 2', 'red blood cell distribution width', 'neoplasm', 'cancer', 'hepatocellular carcinoma'], ['Autosomal dominant spastic paraplegia type 13', 'Pelizaeus-Merzbacher-like disease', 'Pelizaeus-Merzbacher-like disease due to HSPD1 mutation', 'Leukodystrophy', 'Hereditary spastic paraplegia'], ['Schwartz-Jampel syndrome', 'Dyssegmental dysplasia, Silverman-Handmaker type', 'AL amyloidosis', 'osteosarcoma', 'colorectal adenocarcinoma'], ['endometrial cancer', 'Pyoderma', 'dental caries', 'tooth disease', 'hypertension'], ['schizophrenia', 'unipolar depression', 'autism', 'anxiety disorder', 'major depressive disorder'], ['schizophrenia', 'migraine disorder', 'cardiovascular disease', 'pain', 'Headache'], ['migraine disorder', 'schizophrenia', 'Dravet syndrome', 'pain', 'Headache'], ['migraine disorder', 'schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'pain'], ['schizophrenia', 'unipolar depression', 'bipolar disorder', 'psychosis', 'depressive disorder'], ['hemorrhage', 'schizophrenia', 'schizoaffective disorder', 'psychosis', 'post operative nausea and vomiting'], ['schizophrenia', 'bipolar disorder', 'unipolar depression', 'autism', 'depressive disorder'], ['schizophrenia', 'Constipation', 'post operative nausea and vomiting', 'migraine disorder', 'Chronic constipation'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'obesity', 'unipolar depression'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'obesity', 'unipolar depression'], ['schizophrenia', 'schizoaffective disorder', 'post operative nausea and vomiting', 'unipolar depression', 'psychosis'], ['CARASIL', 'cerebral arteriopathy, autosomal dominant, with subcortical infarcts and leukoencephalopathy, type 2', 'age-related macular degeneration', 'CADASIL', 'dementia (disease)'], ['3-methylglutaconic aciduria', 'Young adult-onset Parkinsonism', 'neutropenia', 'intestinal cancer', 'Early-onset progressive neurodegeneration - blindness - ataxia - spasticity'], ['carpal tunnel syndrome', 'osteoarthritis', 'infect', 'infection', 'low density lipoprotein cholesterol measurement'], ['Huntington disease', 'Lopes-Maciel-Rodan syndrome', 'Juvenile Huntington disease', 'Rett syndrome', 'self reported educational attainment'], ['ACPA-negative rheumatoid arthritis', 'joint damage measurement', 'protein measurement', 'drug use measurement', 'rheumatoid arthritis'], ['AL amyloidosis', 'blood protein measurement', 'lean body mass', 'fat body mass', 'breast adenocarcinoma'], ['growth retardation, intellectual developmental disorder, hypotonia, and hepatopathy', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'urate measurement'], ['myeloid white cell count', 'leukocyte count', 'ICAM-1 measurement', 'lymphocyte count', 'ovarian cancer'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'Combined T and B cell immunodeficiency', 'cancer', 'breast adenocarcinoma'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'rheumatoid arthritis', 'genetic disorder'], ['acute myeloid leukemia', 'glioblastoma multiforme', 'cutaneous melanoma', 'cholangiocarcinoma', 'astrocytoma'], ['acute myeloid leukemia', 'D-2-hydroxyglutaric aciduria', '2-hydroxyglutaric aciduria', 'myelodysplastic syndrome', 'osteosarcoma'], ['head and neck malignant neoplasia', 'urogenital neoplasm', 'lung cancer', 'neoplasm', 'melanoma'], ['Mucopolysaccharidosis type 2', 'Mucopolysaccharidosis', 'Mucopolysaccharidosis type 2, severe form', 'Mucopolysaccharidosis type 2, attenuated form', 'genetic disorder'], ['Mucopolysaccharidosis type 1', 'Hurler syndrome', 'Scheie syndrome', 'Hurler-Scheie syndrome', 'genetic disorder'], ['infect', 'infection', 'breast cancer', 'systolic blood pressure', 'melanoma'], ['lymphocyte percentage of leukocytes', 'periodontal measurement', 'neutrophil percentage of leukocytes', 'lymphocyte count', 'leukocyte count'], ['eosinophil count', 'neutrophil count', 'leukocyte count', 'neutrophil percentage of leukocytes', 'myeloid white cell count'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'leukocyte count'], ['ovarian cancer', 'breast adenocarcinoma', 'systemic lupus erythematosus', 'hypothyroidism', "Hashimoto's thyroiditis"], ['multiple sclerosis', 'chronic hepatitis C virus infection', 'chronic hepatitis B virus infection', 'hepatitis C virus infection', 'melanoma'], ['COVID-19', 'multiple sclerosis', 'chronic hepatitis B virus infection', 'chronic hepatitis C virus infection', 'hepatitis C virus infection'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['Aplastic anemia', 'Primary hemophagocytic lymphohistiocytosis', 'immune system disease', 'idiopathic aplastic anemia', 'lymphocyte count'], ['Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR1 deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IFNgammaR1 deficiency', 'immunodeficiency 27A', 'Autosomal dominant mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR1 deficiency', 'disseminated atypical mycobacterial infection'], ['immunodeficiency 28', 'Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR2 deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IFNgammaR2 deficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Osteopetrosis'], ['chronic hepatitis C virus infection', 'ovarian mucinous adenocarcinoma', 'hepatitis C virus infection', 'hepatocellular carcinoma', 'low density lipoprotein cholesterol measurement'], ['Cranioectodermal dysplasia', 'Microcephaly', 'low density lipoprotein cholesterol measurement', 'BMI-adjusted waist-hip ratio', 'Brachydactyly type C'], ['Agenesis of the corpus callosum - intellectual disability - coloboma - micrognathia', 'systemic lupus erythematosus', 'refractory anemia', 'rheumatoid arthritis', 'esophageal squamous cell carcinoma'], ['Growth delay due to insulin-like growth factor type 1 deficiency', 'body height', 'sex hormone-binding globulin measurement', 'vital capacity', 'peak expiratory flow'], ['Growth delay due to insulin-like growth factor I resistance', 'lean body mass', 'Graves ophthalmopathy', 'body height', 'platelet count'], ['Silver-Russell syndrome', 'Beckwith-Wiedemann syndrome', 'Hemihypertrophy', 'Silver-Russell syndrome due to a point mutation', 'fetal growth restriction'], ['hepatocellular carcinoma', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'cation-independent mannose-6-phosphate receptor measurement', 'low density lipoprotein cholesterol measurement'], ['Short stature due to primary acid-labile subunit deficiency', 'Reduced insulin like growth factor binding protein acid labile subunit level', 'Delayed puberty', 'Short stature due to GHSR deficiency', 'cutaneous melanoma'], ['body height', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'sitting height measurement', 'puberty onset measurement'], ['pulse pressure measurement', 'systolic blood pressure', 'diastolic blood pressure', 'body height', 'appendicular lean mass'], ['refractive error measurement', 'thyroid stimulating hormone measurement', 'appendicular lean mass', 'FEV/FEC ratio', "Hashimoto's thyroiditis"], ['Familial retinal arterial macroaneurysm', 'low density lipoprotein cholesterol measurement', 'insulin-like growth factor-binding protein 7 measurement', 'blood protein measurement', 'infant grey matter volume measurement'], ['Spinal muscular atrophy with respiratory distress type 1', 'Charcot-Marie-Tooth disease', 'Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Distal spinal muscular atrophy', 'genetic disorder'], ['Autosomal agammaglobulinemia', 'Agammaglobulinemia', 'Isolated agammaglobulinemia', 'arterial stiffness measurement', 'blood protein measurement'], ['body mass index', 'myeloid cell surface antigen CD33 measurement', 'Apolipoprotein A-I deficiency', 'Tyrosinemia type 1', 'Hepatic veno-occlusive disease - immunodeficiency'], ['lung adenocarcinoma', 'eosinophil count', 'eosinophil percentage of leukocytes', 'systemic lupus erythematosus', 'psoriasis'], ['Incontinentia pigmenti', 'Hypohidrotic ectodermal dysplasia with immunodeficiency', 'X-linked mendelian susceptibility to mycobacterial diseases due to IKBKG deficiency', 'ectodermal dysplasia and immunodeficiency 1', 'X-linked mendelian susceptibility to mycobacterial diseases'], ['Pancytopenia due to IKZF1 mutations', 'acute lymphoblastic leukemia', 'B-cell acute lymphoblastic leukemia', 'Common variable immunodeficiency', 'acute myeloid leukemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'systemic lupus erythematosus', 'basophil count', 'neutrophil percentage of granulocytes'], ['leukocyte count', 'lymphocyte percentage of leukocytes', 'chronic lymphocytic leukemia', 'eosinophil count', 'eosinophil percentage of leukocytes'], ['asthma', 'type I diabetes mellitus', 'eosinophil count', 'Isolated sulfite oxidase deficiency', 'Vitiligo'], ['Cryopyrin-associated periodic syndrome', 'immune system disease', 'Muckle-Wells syndrome', 'Familial cold urticaria', 'Antisynthetase syndrome'], ['rheumatoid arthritis', 'COVID-19', 'fibrinogen measurement', 'pericarditis', 'immune system disease'], ['interleukin 1 Receptor accessory protein measurement', 'protein measurement', 'blood protein measurement', 'obesity', 'visual perception measurement'], ['eosinophil count', 'asthma', 'allergy', 'allergic rhinitis', 'childhood onset asthma'], ['interleukin 1 receptor-like 2 measurement', 'eosinophil count', 'childhood onset asthma', 'acute myeloid leukemia', 'interleukin 18 receptor 1 measurement'], ['Sterile multifocal osteomyelitis with periostitis and pustulosis', 'interleukin 1 receptor antagonist measurement', 'diabetic nephropathy', 'C-reactive protein measurement', 'neutrophil count'], ['seasonal allergic rhinitis', 'psoriasis', 'eosinophil count', 'breast adenocarcinoma', 'allergy'], ['Immunodeficiency due to CD25 deficiency', 'type I diabetes mellitus', 'diabetes mellitus', 'acute graft vs. host disease', 'multiple sclerosis'], ['Immunodeficiency syndrome with autoimmunity', 'metastatic melanoma', 'kidney cancer', 'mycosis fungoides', "Sezary's disease"], ['X-Linked Combined Immunodeficiency Diseases', 'T-B+ severe combined immunodeficiency due to gamma chain deficiency', 'gamma chain deficiency', 'Omenn syndrome', 'Severe combined immunodeficiency'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'sum of eosinophil and basophil counts', 'adult onset asthma', 'gastric adenocarcinoma'], ['acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer'], ['stroke', 'adult onset asthma', 'childhood onset asthma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['mean corpuscular hemoglobin concentration', 'Monoamine oxidase A deficiency', 'pontine autosomal dominant microangiopathy with leukoencephalopathy', 'neoplasm', 'colorectal adenocarcinoma'], ['asthma', 'atopic eczema', 'Eczema', 'chronic rhinosinusitis with nasal polyps', 'eosinophil count'], ['asthma', 'Airway obstruction', 'Hypereosinophilic syndrome', 'chronic obstructive pulmonary disease', 'Nasal Cavity Polyp'], ['asthma', 'eosinophil count', 'basophil count', 'basophil percentage of leukocytes', 'eosinophil percentage of leukocytes'], ['rheumatoid arthritis', 'immune system disease', 'systemic juvenile idiopathic arthritis', "Kaposi's sarcoma", 'juvenile idiopathic arthritis'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'COVID-19', 'temporal arteritis', 'immune system disease'], ['Autosomal recessive hyper-IgE syndrome', 'neuromyelitis optica', 'Stüve-Wiedemann syndrome', 'hepatocellular adenoma', 'carcinoma of liver and intrahepatic biliary tract'], ['Epidermodysplasia verruciformis', 'lymphocyte count', 'monocyte count', 'eosinophil count', 'blood protein measurement'], ['T-B+ severe combined immunodeficiency due to IL-7Ralpha deficiency', 'Omenn syndrome', 'multiple sclerosis', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'T-B+ severe combined immunodeficiency due to CD45 deficiency'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'pancreatic adenocarcinoma', 'asthma', 'neoplasm'], ['Autosomal recessive early-onset inflammatory bowel disease', "Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome'], ['Autosomal recessive early-onset inflammatory bowel disease', 'inflammatory bowel disease', 'ulcerative colitis', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'infection'], ['lean body mass', 'appendicular lean mass', 'osteoarthritis', 'osteoarthritis, hip', 'fat body mass'], ['primary biliary cirrhosis', 'multiple sclerosis', 'biliary liver cirrhosis', "Behcet's syndrome", 'Sjogren syndrome'], ['psoriasis', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12B deficiency', "Crohn's disease", 'psoriatic arthritis', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['asthma', 'atopic eczema', 'Eczema', 'psoriasis', 'allergic rhinitis'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'low grade glioma', 'breast adenocarcinoma'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'gastric adenocarcinoma'], ['protein measurement', 'post-traumatic stress disorder symptom measurement', 'response to trauma exposure', 'rheumatoid arthritis', 'neoplasm'], ['hair color', 'interleukin 16 measurement', 'hair colour measurement', 'appendicular lean mass', 'lymphocyte count'], ['psoriasis', 'ankylosing spondylitis', 'psoriatic arthritis', 'psoriasis vulgaris', 'immune system disease'], ['heel bone mineral density', 'bilirubin measurement', 'diet measurement', 'metabolic syndrome', 'bone quantitative ultrasound measurement'], ['chronic obstructive pulmonary disease', 'acute lymphoblastic leukemia', 'aspartate aminotransferase measurement', 'neoplasm', 'candidiasis'], ['Chronic mucocutaneous candidosis', 'pancreatic carcinoma', 'response to gemcitabine', 'overall survival', 'mean platelet volume'], ['immunodeficiency 51', 'Chronic mucocutaneous candidosis', 'psoriasis', 'psoriasis vulgaris', 'immune system disease'], ['X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'candidiasis', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12RB1 deficiency', 'Recurrent Neisseria infections due to factor D deficiency', 'Autosomal recessive mendelian susceptibility to mycobacterial diseases due to partial IFNgammaR2 deficiency'], ['interleukin 18 measurement', 'breast adenocarcinoma', 'bone quantitative ultrasound measurement', 'heel bone mineral density', 'neoplasm'], ['urinary albumin to creatinine ratio', 'body height', 'Romano-Ward syndrome', 'Brugada syndrome', "adult-onset Still's disease"], ['asthma', 'eosinophil count', "Behcet's syndrome", 'childhood onset asthma', 'allergy'], ['eosinophil count', 'inflammatory bowel disease', "Crohn's disease", 'ulcerative colitis', 'mosquito bite reaction itch intensity measurement'], ['Common variable immunodeficiency', 'Autosomal recessive early-onset inflammatory bowel disease', 'primary biliary cirrhosis', 'psoriasis', 'childhood onset asthma'], ['Cryptosporidiosis - chronic cholangitis - liver disease', 'Omenn syndrome', 'Combined T and B cell immunodeficiency', 'gamma chain deficiency', 'eosinophil count'], ['atopic eczema', 'vitamin D measurement', 'dermatitis', 'DNA methylation', 'Eczema'], ['psoriasis', 'psoriasis vulgaris', 'immune system disease', "Crohn's disease", 'psoriatic arthritis'], ['inflammatory bowel disease', 'ulcerative colitis', "Crohn's disease", 'psoriasis', 'ankylosing spondylitis'], ['thyroid gland adenocarcinoma', 'neoplasm', 'infection', 'infect', 'colitis'], ['red blood cell distribution width', 'mean corpuscular volume', "Crohn's disease", 'psoriasis', 'ulcerative colitis'], ['lymphocyte count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'platelet count', 'monocyte count'], ['allergic rhinitis', 'asthma', 'Eczema', 'psoriasis', "Alzheimer's disease"], ['Familial primary localized cutaneous amyloidosis', 'Primary localized amyloidosis', 'leukocyte count', 'basophil count', 'Autosomal agammaglobulinemia'], ['asthma', 'eosinophil count', 'childhood onset asthma', 'drug use measurement', 'allergy'], ['neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'cortical surface area measurement', 'type II diabetes mellitus'], ['psoriasis', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma', 'interleukin 1 receptor antagonist measurement', 'Malignant epithelial tumor of ovary'], ['interleukin 1 receptor antagonist measurement', 'drug use measurement', 'quality of life during menstruation measurement', 'pathological myopia', 'low density lipoprotein cholesterol measurement'], ['Generalized pustular psoriasis', 'psoriasis 14, pustular', 'deficiency of the interleukin-36 receptor antagonist', 'Pustulosis palmaris et plantaris', 'Acrodermatitis continua suppurativa of Hallopeau'], ['interleukin-1 beta measurement', 'Eczema', 'psoriasis', 'neoplasm', 'rheumatoid arthritis'], ['brain measurement', 'neuroimaging measurement', 'neutrophil count', 'sex hormone-binding globulin measurement', 'myeloid white cell count'], ['pulse pressure measurement', 'respiratory system disease', 'asthma', 'Tuberculosis', 'Townes-Brocks syndrome'], ['FEV/FEC ratio', 'systolic blood pressure', 'platelet count', 'mean platelet volume', 'Hand-foot syndrome'], ['Abnormality of brain morphology', 'cortical surface area measurement', 'glomerular filtration rate', 'brain volume measurement', 'pathological myopia'], ['Nephronophthisis', 'prostate carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['ovarian adenocarcinoma', 'pulmonary arterial hypertension', 'pulse pressure measurement', 'self reported educational attainment', 'sex hormone-binding globulin measurement'], ['muscular dystrophy, congenital, with cataracts and intellectual disability', 'Congenital muscular dystrophy with intellectual disability', 'Marinesco-Sjögren syndrome', 'neoplasm', 'acute myeloid leukemia'], ['MODY', 'Permanent neonatal diabetes mellitus', 'type I diabetes mellitus', 'diabetes mellitus, permanent neonatal 4', 'hyperproinsulinemia'], ['Leprechaunism', 'Rabson-Mendenhall syndrome', 'type II diabetes mellitus', 'Hyperinsulinism due to INSR deficiency', 'Insulin-resistance syndrome type A'], ['Familial medullary thyroid carcinoma', 'Hereditary sensory and autonomic neuropathy type 4', 'hypothyroidism', 'visual perception measurement', 'platelet count'], ['Orofaciodigital syndrome', 'Short rib dysplasia', 'Short rib-polydactyly syndrome, Verma-Naumoff type', 'mean corpuscular volume', 'Short rib-polydactyly syndrome'], ['multiple myeloma', 'neoplasm', 'hepatocellular carcinoma', 'Immunodeficiency due to an early component of complement deficiency', 'acute lymphoblastic leukemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts', 'neutrophil percentage of granulocytes'], ['asthma', 'treatment resistant depression', 'response to ketamine', 'thyroid gland adenocarcinoma', 'protein measurement'], ['Immunodeficiency due to interleukin-1 receptor-associated kinase-4 deficiency', 'Primary immunodeficiency', 'X-linked mendelian susceptibility to mycobacterial diseases due to IKBKG deficiency', 'gram-negative bacterial infections', 'lung adenocarcinoma'], ['Herpetic encephalitis', 'schizophrenia', 'endometrial cancer', 'ovarian cancer', 'autoimmune thyroid disease'], ['multiple myeloma', 'basal cell carcinoma', 'chronic lymphocytic leukemia', 'diffuse large B-cell lymphoma', 'lymphoid neoplasm'], ['systemic lupus erythematosus', 'rheumatoid arthritis', 'inflammatory bowel disease', 'ulcerative colitis', 'eosinophil count'], ['Van der Woude syndrome', 'Autosomal dominant popliteal pterygium syndrome', 'Popliteal pterygium syndrome', 'cleft lip', 'Cleft palate'], ['Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'monocyte count', 'monocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'neutrophil count'], ['immunodeficiency disease', 'pancreatic adenocarcinoma', 'apolipoprotein A 1 measurement', 'neoplasm', 'breast cancer'], ['Craniofacial dysplasia-osteopenia syndrome', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'body height', 'hypospadias'], ['Mendelian susceptibility to mycobacterial diseases due to complete ISG15 deficiency', 'Bilateral striopallidodentate calcinosis', 'influenza', 'Mendelian susceptibility to mycobacterial diseases', 'blood protein measurement'], ['mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'sitting height measurement', 'age-related hearing impairment'], ['heel bone mineral density', 'smoking status measurement', 'body mass index', 'body fat percentage', 'lean body mass'], ['heel bone mineral density', 'breast carcinoma', 'smoking status measurement', 'body height', 'hematocrit'], ['reticulocyte count', 'type II diabetes mellitus', 'Immature Reticulocyte Fraction Measurement', 'FEV/FEC ratio', 'reticulocyte measurement'], ['Glanzmann thrombasthenia', 'Autosomal dominant macrothrombocytopenia', 'cancer', 'coronary artery disease', 'acute myocardial infarction'], ["Crohn's disease", 'relapsing-remitting multiple sclerosis', 'multiple sclerosis', 'ulcerative colitis', 'monocyte count'], ['Congenital muscular dystrophy with integrin alpha-7 deficiency', 'Congenital muscular dystrophy', 'Congenital fiber-type disproportion myopathy', 'Abnormality of brain morphology', 'breast adenocarcinoma'], ['psoriasis', 'dry eye syndrome', 'immune system disease', 'eye disease', 'lymphocyte count'], ['systemic lupus erythematosus', 'glioblastoma multiforme', 'systemic scleroderma', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['glioblastoma multiforme', 'lung adenocarcinoma', 'multiple myeloma', 'asthma', 'breast adenocarcinoma'], ['body mass index', 'IGA glomerulonephritis', 'acute myeloid leukemia', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['Leukocyte adhesion deficiency', 'Leukocyte adhesion deficiency type I', 'dry eye syndrome', 'genetic disorder', 'eye disease'], ['Glanzmann thrombasthenia', 'myocardial infarction', 'cancer', 'coronary artery disease', 'acute myocardial infarction'], ['blood protein measurement', 'coronary artery disease', 'eosinophil count', 'heel bone mineral density', 'glomerular filtration rate'], ['lymphoproliferative syndrome 1', 'renal cell carcinoma', 'sarcoma', 'neoplasm', 'soft tissue sarcoma'], ["Crohn's disease", 'psoriasis', 'inflammatory bowel disease', 'ulcerative colitis', 'sclerosing cholangitis'], ['body mass index', 'multiple myeloma', 'myeloid neoplasm', 'Miyoshi myopathy', 'neoplasm'], ['Isovaleric acidemia', 'genetic disorder', 'blood metabolite measurement', 'testosterone measurement', 'idiopathic pulmonary fibrosis'], ['platelet count', 'monocyte percentage of leukocytes', 'monocyte count', 'lymphocyte percentage of leukocytes', 'platelet crit'], ['Alagille syndrome', 'Alagille syndrome due to a JAG1 point mutation', 'Tetralogy of Fallot', 'Alagille syndrome due to 20p12 microdeletion', 'deafness, congenital heart defects, and posterior embryotoxon'], ['cancer', 'T-cell leukemia', 'erythrocyte count', 'hemoglobin measurement', 'hematocrit'], ['polycythemia vera', 'rheumatoid arthritis', 'primary myelofibrosis', 'myelofibrosis', 'immune system disease'], ['polycythemia vera', 'essential thrombocythemia', 'acute myeloid leukemia', 'primary myelofibrosis', 'ulcerative colitis'], ['T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'ulcerative colitis', 'rheumatoid arthritis', 'ankylosing spondylitis', 'immune system disease'], ['Huntington disease-like 2', 'hair colour measurement', 'migraine disorder', 'small vessel stroke', 'stroke'], ["Alzheimer's disease", 'colon adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['eosinophil count', 'monocyte count', 'eosinophil percentage of leukocytes', 'grip strength measurement', 'monocyte percentage of leukocytes'], ['cancer', 'T-cell leukemia', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma'], ['mental retardation', 'genetic disorder', 'Intellectual disability', 'Autosomal recessive non-syndromic intellectual disability', 'colorectal adenocarcinoma'], ['Genitopatellar syndrome', 'Blepharophimosis-intellectual disability syndrome, SBBYS type', 'prostate adenocarcinoma', 'genetic disorder', 'Intellectual disability'], ['Intellectual disability', 'autism', 'Global developmental delay', 'Seizure', 'Feeding difficulties'], ['Early infantile epileptic encephalopathy', 'multiple sclerosis', 'undetermined early-onset epileptic encephalopathy', 'Epileptic encephalopathy', 'secondary progressive multiple sclerosis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'transverse myelitis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['Familial atrial fibrillation', 'atrial fibrillation', 'multiple sclerosis', 'secondary progressive multiple sclerosis', 'cardiac arrhythmia'], ['Progressive myoclonic epilepsy', 'multiple sclerosis', 'secondary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis', 'transverse myelitis'], ['Spinocerebellar ataxia type 13', 'multiple sclerosis', 'Spinocerebellar ataxia type 15/16', 'secondary progressive multiple sclerosis', 'transverse myelitis'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'self reported educational attainment', 'anterior ischemic optic neuropathy', 'primary progressive multiple sclerosis'], ['Familial long QT syndrome', 'Jervell and Lange-Nielsen syndrome', 'long QT syndrome 5', 'Romano-Ward syndrome', 'Prolonged QT interval'], ['multiple sclerosis', 'secondary progressive multiple sclerosis', 'primary progressive multiple sclerosis', 'anterior ischemic optic neuropathy', 'transverse myelitis'], ['Romano-Ward syndrome', 'Familial long QT syndrome', 'Familial short QT syndrome', 'Prolonged QT interval', 'atrial fibrillation'], ['atrial fibrillation', 'self reported educational attainment', 'intelligence', 'cardiac arrhythmia', 'mathematical ability'], ['Familial hyperaldosteronism type III', 'atrial fibrillation', 'Romano-Ward syndrome', 'aldosterone-producing adenoma', 'Cardiodysrhythmic potassium-sensitive periodic paralysis'], ['type II diabetes mellitus', 'diabetes mellitus', 'Transient neonatal diabetes mellitus', 'Familial hyperinsulinism', 'hyperinsulinemic hypoglycemia, familial, 2'], ['pulmonary arterial hypertension', 'coronary artery disease', 'Idiopathic and/or familial pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'ischemia reperfusion injury'], ['coronary artery disease', 'migraine disorder', 'appendicular lean mass', 'body height', 'intraocular pressure measurement'], ['Intellectual disability, Birk-Barel type', 'coronary artery disease', 'ischemia reperfusion injury', 'kidney failure', 'Agitation'], ['Dehydrated hereditary stomatocytosis', 'pain', 'alcohol abuse', 'osteoarthritis, knee', 'osteoarthritis, hip'], ['Jervell and Lange-Nielsen syndrome', 'Familial long QT syndrome', 'Romano-Ward syndrome', 'Familial short QT syndrome', 'Familial atrial fibrillation'], ['Early infantile epileptic encephalopathy', 'Benign familial neonatal seizures', 'Seizure', 'Benign familial neonatal-infantile seizures', 'multiple sclerosis'], ['Benign familial neonatal seizures', 'Seizure', 'Intellectual disability', 'multiple sclerosis', 'epilepsy'], ['Autosomal dominant non-syndromic intellectual disability', 'multiple sclerosis', 'Seizure', 'epilepsy', 'mental retardation'], ['developmental disability', 'breast adenocarcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'endometrial cancer'], ['breast adenocarcinoma', 'clear cell renal carcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma'], ['age at menarche', 'body height', 'mean corpuscular volume', 'body mass index', 'reticulocyte count'], ['intellectual disability, autosomal recessive 65', 'Autosomal recessive non-syndromic intellectual disability', 'genetic disorder', 'autism', 'breast adenocarcinoma'], ['Kabuki syndrome', 'urinary bladder cancer', 'squamous cell lung carcinoma', 'genetic disorder', 'astrocytoma'], ['cancer', 'hepatocellular carcinoma', 'Familial capillary hemangioma', 'non-small cell lung carcinoma', 'neoplasm'], ['lung adenocarcinoma', 'multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'non-small cell lung carcinoma', 'squamous cell lung carcinoma'], ['Essential fructosuria', 'Disorder of carbohydrate metabolism', 'risk-taking behaviour', 'neutrophil count', 'brain measurement'], ['neoplasm', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'lung cancer', 'non-small cell lung carcinoma'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2A1', 'adrenal gland pheochromocytoma', 'Hereditary pheochromocytoma-paraganglioma', 'Charcot-Marie-Tooth disease', 'neuroblastoma'], ['non-small cell lung carcinoma', 'thyroid cancer', 'lung adenocarcinoma', 'undifferentiated pleomorphic sarcoma', 'adenosquamous lung carcinoma'], ['Microcephaly - lymphedema - chorioretinopathy', 'genetic disorder', 'Syndromic retinitis pigmentosa', 'Retinal dystrophy', 'type II diabetes mellitus'], ['ovarian cancer', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'gastric carcinoma', 'head and neck carcinoma'], ['Goldberg-Shprintzen megacolon syndrome', 'Shprintzen-Goldberg syndrome', 'peripheral neuropathy', 'eosinophil percentage of leukocytes', 'eosinophil count'], ['pancreatic adenocarcinoma', 'prostate carcinoma', '3-hydroxypropylmercapturic acid measurement', 'smoking behavior', 'aortic stenosis'], ['Gastrointestinal stromal tumor', 'Piebaldism', 'Mastocytosis', 'maculopapular cutaneous mastocytosis', 'acute myeloid leukemia'], ['Familial progressive hyperpigmentation', 'Familial progressive hyper- and hypopigmentation', 'hyperpigmentation with or without hypopigmentation, familial progressive', 'testicular carcinoma', 'Waardenburg syndrome type 2'], ['tumoral calcinosis, hyperphosphatemic, familial, 3', 'familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'cancer', 'type II diabetes mellitus', 'birth weight'], ['cancer', 'alcohol consumption measurement', 'alcohol use disorder measurement', 'alcohol drinking', 'longitudinal alcohol consumption measurement'], ['breast carcinoma', 'Secretory Meningioma', 'Chordoid Meningioma', 'Endometrial Endometrioid Adenocarcinoma', 'Fibrous Meningioma'], ['urinary bladder cancer', 'squamous cell lung carcinoma', 'cervical squamous cell carcinoma', 'dilated cardiomyopathy', 'colorectal cancer'], ['prostate carcinoma', 'gastric carcinoma', 'prostate adenocarcinoma', 'heel bone mineral density', 'gastric cancer'], ['Familial progressive cardiac conduction defect', 'X-linked endothelial corneal dystrophy', 'Fuchs endothelial corneal dystrophy', 'Primary dystonia, DYT13 type', 'Primary dystonia, DYT2 type'], ['body height', 'acute quadriplegic myopathy', 'chronic kidney disease', 'dermatomyositis', 'astrocytoma'], ['Hodgkins lymphoma', 'classic Hodgkin lymphoma', 'irritability measurement', 'Irritability', 'insomnia measurement'], ['kallikrein, decreased urinary activity of', 'cutaneous melanoma', 'lung adenocarcinoma', 'acute kidney failure', 'Townes-Brocks syndrome'], ['prostate carcinoma', 'prostate specific antigen measurement', 'prostate cancer', 'male reproductive organ cancer', 'cutaneous melanoma'], ['Hypomaturation amelogenesis imperfecta', 'Amelogenesis imperfecta', 'amelogenesis imperfecta, type ij', 'prostate specific antigen measurement', 'genetic disorder'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'neuroblastoma', 'urinary bladder cancer'], ['kallikrein‐6 measurement', 'blood protein measurement', 'neoplasm', "Alzheimer's disease", 'non-small cell lung carcinoma'], ['blood protein measurement', 'kallikrein-7 measurement', 'breast adenocarcinoma', 'kallikrein-11 measurement', 'ovarian cancer'], ['blood protein measurement', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'gastric adenocarcinoma'], ['prostate specific antigen measurement', 'prostate carcinoma', 'prostate cancer', 'ovarian cancer', 'breast cancer'], ['Congenital prekallikrein deficiency', 'Hereditary angioedema', 'hemorrhage', 'Bietti crystalline dystrophy', 'serum metabolite measurement'], ['blood protein measurement', 'rheumatoid arthritis', 'refractory anemia', 'Townes-Brocks syndrome', 'juvenile idiopathic arthritis'], ['platelet count', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'monocyte count', 'granulocyte percentage of myeloid white cells'], ['lean body mass', 'red blood cell density measurement', 'total cholesterol measurement', 'appendicular lean mass', 'fat body mass'], ['acute myeloid leukemia', 'neoplasm', 'hepatocellular carcinoma', 'cancer', 'cervical cancer'], ['chronic kidney disease', 'urinary metabolite measurement', 'coronary artery disease', 'lean body mass', 'reticulocyte count'], ['Wiedemann-Steiner syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'cutaneous melanoma'], ['Kabuki syndrome', 'head and neck squamous cell carcinoma', 'medulloblastoma', 'prostate adenocarcinoma', 'gastric adenocarcinoma'], ['intellectual disability, autosomal dominant 51', 'genetic disorder', 'serum IgG glycosylation measurement', 'autism spectrum disorder', 'Rare genetic intellectual disability'], ['Malignant Pancreatic Neoplasm', 'pancreatic carcinoma', 'pancreatic neoplasm', 'breast cancer', 'cancer'], ['Congenital high-molecular-weight kininogen deficiency', 'blood protein measurement', 'kininogen-1 measurement', 'partial thromboplastin time', 'factor XI measurement'], ['Cardiofaciocutaneous syndrome', 'Noonan syndrome', 'Juvenile Myelomonocytic Leukemia', 'acute myeloid leukemia', 'multiple myeloma'], ['Localized epidermolysis bullosa simplex', 'Generalized epidermolysis bullosa simplex, non-Dowling-Meara type', 'Epidermolysis bullosa simplex, Dowling-Meara type', 'Dowling-Degos disease', 'Epidermolysis bullosa simplex with mottled pigmentation'], ['Pachyonychia congenita', 'ovarian cancer', 'prostate adenocarcinoma', 'head and neck squamous cell carcinoma', 'clear cell renal carcinoma'], ['prostate adenocarcinoma', 'clear cell renal carcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'neoplasm'], ['Epidermolytic ichthyosis', 'Annular epidermolytic ichthyosis', 'Congenital reticular ichthyosiform erythroderma', 'Pachyonychia congenita', 'palmoplantar keratosis'], ['Meesmann corneal dystrophy', 'genetic disorder', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'clear cell renal carcinoma'], ['White sponge nevus', 'head and neck squamous cell carcinoma', 'ovarian cancer', 'glioblastoma multiforme', 'clear cell renal carcinoma'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'esophageal cancer'], ['Pachyonychia congenita', 'Sebocystomatosis', 'Anonychia', 'clear cell renal carcinoma', 'glioblastoma multiforme'], ['cirrhosis of liver', 'cirrhosis, familial', 'glioblastoma multiforme', 'cleft lip', 'Cleft palate'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'esophageal cancer'], ['pancreatic adenocarcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'colorectal carcinoma'], ['Woolly hair', 'Hypotrichosis simplex', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma'], ['head and neck squamous cell carcinoma', 'prostate adenocarcinoma', 'clear cell renal carcinoma', 'Obesity due to melanocortin 4 receptor deficiency', 'Dysplasia epiphysealis hemimelica'], ['ovarian cancer', 'clear cell renal carcinoma', 'head and neck squamous cell carcinoma', 'prostate adenocarcinoma', 'neuroblastoma'], ['hair color', 'head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'clear cell renal carcinoma', 'prostate adenocarcinoma'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'reticulocyte count'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'low grade glioma'], ['head and neck squamous cell carcinoma', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'hair color', 'low grade glioma', 'psoriasis'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'Hypotrichosis simplex'], ['Encephalopathy due to hydroxykynureninuria', 'Congenital vertebral-cardiac-renal anomalies syndrome', 'vertebral, cardiac, renal, and limb defects syndrome 2', 'Catel-Manzke syndrome', 'Ulbright-Hodes syndrome'], ['genetic disorder', 'monocyte count', 'low grade glioma', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['triglyceride measurement', 'cardiac troponin T measurement', 'experimental autoimmune encephalomyelitis', 'electrocardiography', 'left ventricular structural measurement'], ['sex hormone-binding globulin measurement', 'blood protein measurement', "Hashimoto's thyroiditis", 'neoplasm', 'melanoma'], ['inflammatory bowel disease', 'systemic lupus erythematosus', 'malaria', 'acute lymphoblastic leukemia', 'severe malarial anemia'], ['mean corpuscular hemoglobin concentration', 'breast carcinoma', 'pancreatic adenocarcinoma', 'breast cancer', 'neoplasm'], ['Congenital muscular dystrophy type 1A', 'muscular dystrophy, limb-girdle, autosomal recessive 23', 'Congenital muscular dystrophy', 'LAMA2-related muscular dystrophy', 'laminin alpha 2-related dystrophy'], ['Junctional epidermolysis bullosa, Herlitz type', 'Junctional epidermolysis bullosa, non-Herlitz type', 'Junctional epidermolysis bullosa', 'eye disease', 'Generalized junctional epidermolysis bullosa, non-Herlitz type'], ['neutrophil count', 'leukocyte count', 'myeloid white cell count', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['Glycogen storage disease due to LAMP-2 deficiency', 'Glycogen Storage Disease Type 2b', 'hypertrophic cardiomyopathy', 'dilated cardiomyopathy', 'cardiomyopathy'], ['inflammatory bowel disease', "Parkinson's disease", 'esophageal squamous cell carcinoma', 'pancreatitis', 'lung carcinoma'], ['neoplasm', 'acute myeloid leukemia', 'hepatocellular carcinoma', 'colorectal carcinoma', 'acute chest syndrome'], ['Acute infantile liver failure-multisystemic involvement syndrome', 'self reported educational attainment', 'mathematical ability', 'Global developmental delay', 'sensorineural hearing loss'], ['Perrault syndrome', 'Mitochondrial myopathy and sideroblastic anemia', 'Rare genetic deafness', 'Mitochondrial myopathy', 'body mass index'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Severe combined immunodeficiency', 'breast adenocarcinoma', 'self reported educational attainment', 'lymphocyte count'], ['basal cell carcinoma', 'cervical squamous cell carcinoma', 'colorectal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'Hepatobiliary Neoplasm'], ['blood protein measurement', 'basophil percentage of leukocytes', 'ovarian cancer', 'glioblastoma multiforme', 'thyroid gland adenocarcinoma'], ['Greenberg dysplasia', 'regressive spondylometaphyseal dysplasia', 'Pelger-Huet anomaly', 'Reynolds syndrome', 'Jeune syndrome'], ['Familial LCAT deficiency', 'Fish-eye disease', 'LCAT deficiency', 'high density lipoprotein cholesterol measurement', 'esophageal cancer'], ['cancer', 'Severe combined immunodeficiency due to LCK deficiency', 'acute lymphoblastic leukemia', 'renal cell carcinoma', 'chronic myelogenous leukemia'], ['cutaneous melanoma', 'neoplasm', 'cancer', 'hepatocellular carcinoma', 'coronary artery disease'], ['endometrial cancer', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'hepatocellular carcinoma'], ['lymphocyte count', 'autoimmune disease', 'MAP kinase-activated protein kinase 3 measurement', 'blood protein measurement', 'lung carcinoma'], ['eosinophil count', 'mean platelet volume', 'eosinophil percentage of leukocytes', 'breast adenocarcinoma', 'inflammatory bowel disease'], ['Glycogen storage disease due to lactate dehydrogenase M-subunit deficiency', 'Glycogen storage disease', 'Glycogen storage disease due to lactate dehydrogenase deficiency', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'serum amyloid A protein measurement'], ['familial hypercholesterolemia', 'Homozygous familial hypercholesterolemia', 'hypercholesterolemia, familial, 1', 'Hypercholesterolemia', 'Hyperlipoproteinemia type 3'], ['Obesity due to congenital leptin deficiency', 'BMI-adjusted leptin measurement', 'leptin measurement', 'type II diabetes mellitus', 'Obesity due to leptin receptor gene deficiency'], ['Obesity due to leptin receptor gene deficiency', 'obesity', 'body mass index', 'leptin receptor measurement', 'lipodystrophy'], ['longitudinal BMI measurement', 'non-alcoholic steatohepatitis', 'neutrophil count', 'leukocyte count', 'age at menarche'], ['white matter microstructure measurement', 'phospholipid measurement', 'visceral adipose tissue measurement', 'Reticular dysgenesis', 'Combined immunodeficiency with skin granulomas'], ['mean platelet volume', 'protein measurement', 'leukocyte count', 'blood protein measurement', 'reticulocyte count'], ['blood protein measurement', 'acute myeloid leukemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'eosinophil count'], ['serum metabolite measurement', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'blood protein measurement', 'endometrial cancer'], ['lean body mass', 'hair shape measurement', 'osteoporosis', 'postmenopausal osteoporosis', 'balding measurement'], ['arterial stiffness measurement', 'reticulocyte count', 'osteoarthritis, spine', 'type II diabetes mellitus', 'neoplasm'], ['DNA methylation', 'non-small cell lung carcinoma', 'pancreatic adenocarcinoma', 'lung cancer', 'clear cell renal carcinoma'], ['heel bone mineral density', 'lung adenocarcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'urate measurement'], ['leukocyte immunoglobulin-like receptor subfamily B member 1 measurement', 'inflammatory bowel disease', 'systemic lupus erythematosus', 'blood protein measurement', 'rheumatoid arthritis'], ['ovarian cancer', 'breast adenocarcinoma', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'myeloid white cell count'], ['Wolman disease', 'Cholesteryl ester storage disease', 'Lysosomal acid lipase deficiency', 'familial hypercholesterolemia', 'coronary artery disease'], ['Hyperlipidemia due to hepatic triglyceride lipase deficiency', 'high density lipoprotein cholesterol measurement', 'serum metabolite measurement', 'triglyceride measurement', 'total cholesterol measurement'], ['Familial partial lipodystrophy', 'Familial partial lipodystrophy, Dunnigan type', 'Familial partial lipodystrophy, Köbberling type', 'esophageal cancer', 'Partial chromosome Y deletion'], ['high density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'vitamin D measurement', 'low density lipoprotein cholesterol measurement', 'red blood cell distribution width'], ['Familial partial lipodystrophy, Dunnigan type', 'Hutchinson-Gilford progeria syndrome', 'dilated cardiomyopathy', 'Emery-Dreifuss muscular dystrophy', 'Congenital muscular dystrophy due to LMNA mutation'], ['prostate carcinoma', 'smoking status measurement', 'serum gamma-glutamyl transferase measurement', 'urate measurement', 'neuroticism measurement'], ['neutrophil count', "Crohn's disease", 'neutrophil percentage of leukocytes', 'inflammatory bowel disease', 'blood protein measurement'], ['neurodevelopmental disorder with epilepsy and hypoplasia of the corpus callosum', 'wellbeing measurement', 'depressive symptom measurement', 'neuroticism measurement', 'carpal tunnel syndrome'], ['CODAS syndrome', 'Pyruvate dehydrogenase E1-alpha deficiency', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration'], ['Familial thoracic aortic aneurysm and aortic dissection', 'marfan syndrome/loeys-dietz syndrome/familial thoracic aortic aneurysms and dissections', 'Aortic dissection', 'aortic aneurysm', 'FEV/FEC ratio'], ['exfoliation syndrome', 'open-angle glaucoma', 'glaucoma', 'vital capacity', 'body fat distribution'], ['body height', 'blood protein measurement', 'appendicular lean mass', 'sitting height measurement', 'forced expiratory volume'], ['Stickler syndrome', 'Autosomal recessive Stickler syndrome', 'breast adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma'], ['coronary artery disease', 'myocardial infarction', 'cardiovascular disease', 'low density lipoprotein cholesterol measurement', 'angina pectoris'], ['eosinophil count', 'monocyte count', 'central corneal thickness', 'body height', 'eosinophil percentage of leukocytes'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma'], ['mean corpuscular volume', 'glioblastoma multiforme', 'ovarian cancer', 'mean corpuscular hemoglobin', 'vital capacity'], ['Familial lipoprotein lipase deficiency', 'Combined hyperlipidemia', 'Hyperlipoproteinemia type 1', 'coronary artery disease', 'high density lipoprotein cholesterol measurement'], ['basophil percentage of leukocytes', 'Meckel syndrome', 'eosinophil count', 'Bardet-Biedl syndrome', 'blood protein measurement'], ['skin carcinoma', 'Uterine Carcinosarcoma', 'lipoma', 'pancreatic ductal adenocarcinoma', 'pancreatic carcinoma'], ['Common variable immunodeficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'neutrophil count', 'lobe attachment'], ['Ulerythema ophryogenesis', 'Keratosis pilaris atrophicans', 'Atrophoderma vermiculata', 'Keratosis follicularis spinulosa decalvans', 'migraine disorder'], ['Donnai-Barrow syndrome', 'Intellectual disability', 'urate measurement', 'glomerular filtration rate', 'uric acid measurement'], ['Familial exudative vitreoretinopathy', 'Osteoporosis - pseudoglioma', 'polycystic liver disease 4 with or without kidney cysts', 'Autosomal dominant osteopetrosis type 1', 'Autosomal dominant osteosclerosis, Worth type'], ['coronary artery disease, autosomal dominant 2', 'Oligodontia', 'tooth agenesis', 'orofacial cleft', 'Orofacial clefting syndrome'], ['myocardial infarction', 'self reported educational attainment', 'brain measurement', 'neuroimaging measurement', 'blood protein measurement'], ['Congenital lactic acidosis, Saguenay-Lac-Saint-Jean type', 'Leigh syndrome', 'Mitochondrial disease', 'Isolated cytochrome C oxidase deficiency', 'grip strength measurement'], ['blood protein measurement', 'sinusitis', 'Brugada syndrome', 'Familial progressive cardiac conduction defect', 'Familial short QT syndrome'], ['Herpes Zoster', 'eosinophil count', 'atopic eczema', 'allergy', 'eosinophil percentage of leukocytes'], ['Hereditary late-onset Parkinson disease', 'Young adult-onset Parkinsonism', "Parkinson's disease", 'mean platelet volume', 'inflammatory bowel disease'], ['systolic blood pressure', 'diastolic blood pressure', 'alcohol drinking', 'hypertension', 'mean arterial pressure'], ['cutaneous melanoma', 'lung adenocarcinoma', 'red blood cell density measurement', 'lymphocyte count', 'erythrocyte count'], ['psoriatic arthritis', 'lymphocyte count', 'tumor necrosis factor beta measurement', 'lymphocyte percentage of leukocytes', 'monocyte count'], ['small intestine neuroendocrine tumor', 'monocyte count', 'FEV/FEC ratio', 'amino acid measurement', 'blood metabolite measurement'], ['thyroid gland adenocarcinoma', 'low grade glioma', 'chronic obstructive pulmonary disease', 'asthma', 'rheumatoid arthritis'], ['breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'language impairment', 'breast cancer', 'Neutrophil immunodeficiency syndrome'], ['Tuberculosis', 'AL amyloidosis', 'blood protein measurement', 'cerebrospinal fluid biomarker measurement', 'alkaline phosphatase measurement'], ['type II diabetes mellitus', 'albuminuria', 'childhood onset asthma', 'Drugs used in diabetes use measurement', 'high density lipoprotein cholesterol measurement'], ['colorectal carcinoma', 'neoplasm', 'pes', 'acute myeloid leukemia', 'heel bone mineral density'], ['body height', 'fat body mass', 'body fat percentage', 'chronic obstructive pulmonary disease', 'central corneal thickness'], ['cancer', 'neoplasm', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'Sensorineural deafness with dilated cardiomyopathy'], ['cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'cutaneous Leishmaniasis'], ['drug use measurement', 'HIV-1 infection', 'viral load', 'melanoma', 'acute myeloid leukemia'], ['blood protein measurement', 'platelet count', 'systolic blood pressure', 'migraine disorder', 'diastolic blood pressure'], ['Familial renal amyloidosis', 'AL amyloidosis', 'Familial renal amyloidosis due to lysozyme variant', 'monocyte count', 'monocyte percentage of leukocytes'], ['Noonan syndrome', 'Neurofibromatosis type 3', 'genetic disorder', 'rasopathy', 'Endometrial Endometrioid Adenocarcinoma'], ['total cholesterol measurement', 'monocyte count', 'low density lipoprotein cholesterol measurement', 'appendicular lean mass', 'eosinophil count'], ['adolescent idiopathic scoliosis', 'childhood onset asthma', 'alkaline phosphatase measurement', 'heel bone mineral density', 'hepatocellular carcinoma'], ['coronary artery disease', 'refractive error measurement', 'Abnormality of refraction', 'systolic blood pressure', 'lung adenocarcinoma'], ['Fanconi anemia', 'acute myeloid leukemia', 'colorectal carcinoma', 'breast adenocarcinoma', 'Ewing sarcoma'], ["Crohn's disease", 'ulcerative colitis', 'hepatocellular carcinoma', 'Combined immunodeficiency with skin granulomas', 'Hyper-IgM syndrome type 3'], ['Aymé-Gripp syndrome', 'Non-syndromic congenital cataract', 'Pulverulent cataract', 'Cerulean cataract', 'Early-onset non-syndromic cataract'], ['Autosomal recessive spastic paraplegia type 75', 'Autosomal recessive complex spastic paraplegia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'Hereditary spastic paraplegia'], ['hepatocellular carcinoma', 'neoplasm', 'non-small cell lung carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm'], ['X-linked immunodeficiency with magnesium defect, Epstein-Barr virus infection and neoplasia', 'congenital disorder of glycosylation type I', 'Combined T and B cell immunodeficiency', 'Congenital disorder of glycosylation', 'COVID-19'], ['Retinitis pigmentosa', 'Posterior column ataxia - retinitis pigmentosa', 'Retinal dystrophy', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect'], ['Combined immunodeficiency due to MALT1 deficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm'], ['sex hormone-binding globulin measurement', 'blood protein measurement', 'aspartate aminotransferase measurement', 'arterial stiffness measurement', 'social anxiety disorder'], ["Parkinson's disease", 'unipolar depression', 'depressive disorder', 'bipolar disorder', 'major depressive disorder'], ['Cardiofaciocutaneous syndrome', 'Noonan syndrome', 'cutaneous melanoma', 'melanoma', 'neoplasm'], ['Cardiofaciocutaneous syndrome', 'Heart-hand syndrome type 3', 'Noonan syndrome', 'cancer', 'cutaneous melanoma'], ['body mass index', 'body fat percentage', 'lean body mass', 'waist circumference', 'fat body mass'], ['gastric adenocarcinoma', 'anthrax infection', 'appendicular lean mass', 'lean body mass', 'type II diabetes mellitus'], ['protein C measurement', 'body height', 'self reported educational attainment', 'blood protein measurement', 'neoplasm'], ['eosinophil count', 'neutrophil count', 'myeloid white cell count', 'monocyte count', 'high density lipoprotein cholesterol measurement'], ['Familial gastric cancer', 'gastric cancer', 'neoplasm', 'cancer', 'myeloid white cell count'], ["Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'C-reactive protein measurement', 'ankylosing spondylitis'], ['birth weight', 'C-reactive protein measurement', 'resting heart rate', "Parkinson's disease", 'Homozygous familial hypercholesterolemia'], ['NIK deficiency', 'monocyte count', 'Combined immunodeficiency', 'monocyte percentage of leukocytes', 'eosinophil count'], ['myopathy, centronuclear, 6, with fiber-type disproportion', 'Autosomal recessive centronuclear myopathy', 'Congenital fiber-type disproportion myopathy', 'mathematical ability', 'FEV/FEC ratio'], ['neoplasm', 'Immunodeficiency due to an early component of complement deficiency', "Hashimoto's thyroiditis", 'Autosomal recessive systemic lupus erythematosus', 'complement component C1s deficiency'], ['leukocyte count', "Parkinson's disease", 'childhood onset asthma', 'body height', 'neutrophil count'], ['mean platelet volume', 'hemoglobin measurement', 'body fat distribution', 'erythrocyte count', 'lean body mass'], ['Noonan syndrome', 'cancer', 'head and neck squamous cell carcinoma', 'cervical squamous cell carcinoma', 'hypertrophic cardiomyopathy'], ['cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy', 'Costello syndrome', 'Cardiofaciocutaneous syndrome'], ['low grade glioma', 'glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'diastolic blood pressure'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['ovarian cancer', 'grip strength measurement', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['sex hormone-binding globulin measurement', 'acute myeloid leukemia', 'testosterone measurement', 'breast adenocarcinoma', 'Epileptic encephalopathy'], ['cutaneous melanoma', 'drug use measurement', 'osteosarcoma', 'Hernia', 'unipolar depression'], ['dilated cardiomyopathy', 'acute coronary syndrome', 'COVID-19', 'ovarian cancer', 'endometrial cancer'], ['MAP kinase-activated protein kinase 2 measurement', 'mean corpuscular hemoglobin concentration', 'glioblastoma multiforme', 'lung adenocarcinoma', 'mean corpuscular hemoglobin'], ['Frontotemporal dementia', 'Pick disease', 'Classical progressive supranuclear palsy', 'Progressive supranuclear palsy', 'Semantic dementia'], ['infect', 'infection', 'anxiety', 'dilated cardiomyopathy', 'Familial congenital mirror movements'], ['pancreatic adenocarcinoma', 'neoplasm', 'melanoma', 'gastric cancer', 'Malignant Pancreatic Neoplasm'], ['severe early-onset pulmonary alveolar proteinosis due to MARS deficiency', 'Autosomal dominant Charcot-Marie-Tooth disease type 2 due to MARS mutation', 'Acute infantile liver failure-multisystemic involvement syndrome', 'Charcot-Marie-Tooth disease', 'Autosomal recessive spastic paraplegia type 70'], ['risky sexual behaviour measurement', 'alcohol dependence', 'Microcephalic primordial dwarfism due to ZNF335 deficiency', 'Polymicrogyria due to TUBB2B mutation', 'Spinocerebellar ataxia type 17'], ['Brain demyelination due to methionine adenosyltransferase deficiency', 'hepatic methionine adenosyltransferase deficiency', 'Psychomotor retardation due to S-adenosylhomocysteine hydrolase deficiency', 'vitamin D measurement', 'hematocrit'], ['platelet count', 'prostate carcinoma', 'platelet crit', 'coronary artery disease', 'Myocardial Ischemia'], ['breast adenocarcinoma', 'smoking status measurement', 'reticulocyte count', 'chronotype measurement', 'gonorrhea'], ['body height', 'monocyte count', 'high density lipoprotein cholesterol measurement', 'myeloid white cell count', 'breast adenocarcinoma'], ['Hereditary pheochromocytoma-paraganglioma', 'adrenal gland pheochromocytoma', 'multiple myeloma', 'Inherited cancer-predisposing syndrome', 'Gastrointestinal stromal tumor'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement', 'breast cancer', 'neoplasm'], ['mean corpuscular hemoglobin', 'leukocyte count', 'mean corpuscular volume', 'neutrophil count', 'mean corpuscular hemoglobin concentration'], ['Pyruvate dehydrogenase E3-binding protein deficiency', 'mannose-binding protein C measurement', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density'], ['Fuchs endothelial corneal dystrophy', 'neutrophil count', 'testosterone measurement', 'myeloid white cell count', 'type II diabetes mellitus'], ['endometrial cancer', 'balding measurement', 'hair shape measurement', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy'], ['lymphocyte count', 'poisoning', 'brain volume measurement', 'neuroimaging measurement', 'facial asymmetry measurement'], ['spondyloepiphyseal dysplasia, Kondo-Fu type', 'cutaneous melanoma', 'colorectal carcinoma', 'lung adenocarcinoma', 'drug use measurement'], ['Oculocutaneous albinism type 2', 'Familial melanoma', 'Erythropoietic protoporphyria', 'skin pigmentation', 'Large congenital melanocytic nevus'], ['Familial glucocorticoid deficiency', 'multiple sclerosis', 'systemic lupus erythematosus', 'rheumatoid arthritis', 'Sarcoidosis'], ['age at menarche', 'age at first facial hair', 'body height', 'lean body mass', 'appendicular lean mass'], ['obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'sexual dysfunction', 'body mass index', 'type II diabetes mellitus'], ['DNA methylation', 'ovarian cancer', 'glioblastoma multiforme', 'fat body mass', 'hip bone mineral density'], ['platelet crit', 'platelet count', 'neoplasm', 'melanoma', 'cancer'], ['Autosomal recessive isolated optic atrophy', 'neoplasm', 'prostate carcinoma', 'colorectal carcinoma', 'cancer'], ['colorectal adenocarcinoma', 'colonic neoplasm', 'protein measurement', 'waist-hip ratio', 'triglyceride measurement'], ['Methylmalonic acidemia due to methylmalonyl-CoA epimerase deficiency', 'N-acetyl-D-glucosamine kinase measurement', 'emotional symptom measurement', 'Methylmalonic acidemia with homocystinuria', 'eosinophil count'], ['coronary artery disease', 'systolic blood pressure', 'pulse pressure measurement', 'mean platelet volume', 'cardiovascular disease'], ['type II diabetes mellitus', 'liver dysplastic nodule', 'skin aging', 'skin aging measurement', 'heel bone mineral density'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'neuroticism measurement', 'Normosmic congenital hypogonadotropic hypogonadism'], ['alcohol consumption measurement', 'smoking status measurement', 'low density lipoprotein cholesterol measurement', 'lean body mass', 'alcohol drinking'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'eosinophil percentage of leukocytes', 'eosinophil count', 'type II diabetes mellitus'], ['small cell lung carcinoma', 'colorectal carcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['primary ovarian insufficiency', 'age at menopause', 'uterine fibroid', 'anti-Mullerian hormone measurement', 'Menorrhagia'], ['Autosomal recessive primary microcephaly', 'Premature chromosome condensation with microcephaly and intellectual disability', 'Isolated congenital microcephaly', 'Primary microcephaly', 'intraocular pressure measurement'], ['neoplasm', 'colorectal carcinoma', 'breast cancer', 'melanoma', 'cancer'], ['lymphocyte count', 'emotional symptom measurement', 'total cholesterol measurement', 'red blood cell distribution width', 'breast cancer'], ['Early infantile epileptic encephalopathy', 'Hereditary pheochromocytoma-paraganglioma', 'mean platelet volume', 'eosinophil count', 'neutrophil count'], ['hepatocellular carcinoma', 'glioma', 'non-small cell lung carcinoma', 'pancreatic carcinoma', 'pancreatic neoplasm'], ['Lessel-Kubisch syndrome', 'Li-Fraumeni syndrome', 'basal cell carcinoma', 'lymphoid neoplasm', 'prostate carcinoma'], ['kidney neoplasm', 'basal cell carcinoma', 'hemangioblastoma', 'carcinoma of liver and intrahepatic biliary tract', 'lymphoid neoplasm'], ['platelet-derived growth factor BB measurement', 'neoplasm', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'cancer', 'breast cancer'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'neoplasm', 'blood pressure', 'childhood onset asthma'], ['hemoglobin measurement', 'body height', 'hematocrit', 'systemic juvenile idiopathic arthritis', 'Abnormal sperm morphology'], ['Rett syndrome', 'Atypical Rett syndrome', 'X-linked intellectual disability - psychosis - macroorchidism', 'Severe neonatal-onset encephalopathy with microcephaly', 'syndromic X-linked intellectual disability Lubs type'], ['X-linked intellectual disability with marfanoid habitus', 'Blepharophimosis-intellectual disability syndrome, MKB type', 'fg syndrome', 'Intellectual disability', 'mental retardation'], ['intellectual developmental disorder 62', 'genetic disorder', 'intellectual developmental disorder 61', 'optic cup area measurement', 'mathematical ability'], ['Familial Mediterranean fever', 'sweet syndrome', "Behcet's syndrome", 'genetic disorder', 'Abnormal cerebral white matter morphology'], ['neoplasm', 'cancer', 'breast cancer', 'hepatocellular carcinoma', 'gastric cancer'], ["Alzheimer's disease", 'thyroid cancer', 'mathematical ability', 'self reported educational attainment', 'body fat percentage'], ['Multiple endocrine neoplasia type 1', 'parathyroid adenoma', 'lung carcinoid tumor', 'Familial isolated hyperparathyroidism', 'Multiple endocrine neoplasia'], ['platelet component distribution width', 'platelet count', 'platelet crit', 'self reported educational attainment', 'heel bone mineral density'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'otosclerosis', 'lean body mass'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Leber congenital amaurosis', 'appendicular lean mass', 'systolic blood pressure'], ['papillary renal cell carcinoma', 'Familial papillary renal cell carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'non-small cell lung carcinoma'], ['reticulocyte count', 'type II diabetes mellitus', 'reticulocyte measurement', 'C-reactive protein measurement', 'sex hormone-binding globulin measurement'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'infection'], ['AL amyloidosis', 'myocardial infarction', 'coronary atherosclerosis', 'Myocardial Ischemia', 'angina pectoris'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2A2', 'Hereditary motor and sensory neuropathy type 6', 'Autosomal recessive axonal Charcot-Marie-Tooth disease type 2', 'Charcot-Marie-Tooth disease', 'genetic disorder'], ['type II diabetes mellitus', 'diabetes mellitus', 'abnormal glucose tolerance', 'chronic hepatitis C virus infection', 'hypertension'], ['COVID-19', 'severe acute respiratory syndrome', 'apolipoprotein A 1 measurement', 'type II diabetes mellitus', 'sex hormone-binding globulin measurement'], ['MGAT2-CDG', 'congenital disorder of glycosylation type II', 'COVID-19', 'Abnormal facial shape', 'Global developmental delay'], ['serum IgG glycosylation measurement', 'cognitive function measurement', 'reticulocyte count', 'reticulocyte measurement', 'leukocyte count'], ['N-glycan measurement', 'alkaline phosphatase measurement', 'COVID-19', 'eosinophil count', 'schizophrenia'], ['breast adenocarcinoma', 'musculoskeletal system disease', 'Myotonic syndrome', 'neoplasm', 'hepatocellular carcinoma'], ['Keutel syndrome', 'grip strength measurement', 'blood protein measurement', 'osteoarthritis, hand', 'heel bone mineral density'], ['melanoma-derived growth regulatory protein measurement', 'obesity', 'blood protein measurement', 'protein measurement', 'erythrocyte count'], ['combined oxidative phosphorylation deficiency', '3-methylglutaconic aciduria type 3', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'liver disease'], ['Proximal myopathy with extrapyramidal signs', 'genetic disorder', 'C-reactive protein measurement', 'red blood cell distribution width', 'high density lipoprotein cholesterol measurement'], ['rheumatoid arthritis', 'juvenile idiopathic arthritis', 'systemic juvenile idiopathic arthritis', 'protein measurement', 'leukocyte count'], ['Non-syndromic congenital cataract', 'Nuclear cataract', 'Total congenital cataract', 'Cerulean cataract', 'Cataract with Y-shaped suture opacities'], ['Combined oxidative phosphorylation defect type 11', 'Mitochondrial disorder due to a defect in mitochondrial protein synthesis', 'cardiomyopathy', 'Left ventricular noncompaction', 'Infantile muscular hypotonia'], ['psoriasis', 'vulvar intraepithelial neoplasia', 'colorectal carcinoma', 'neoplasm', 'obesity'], ['Waardenburg syndrome', 'Tietz syndrome', 'Waardenburg syndrome type 2', 'coloboma, osteopetrosis, microphthalmia, macrocephaly, albinism, and deafness', 'Ocular albinism with congenital sensorineural deafness'], ['mean platelet volume', 'sex hormone-binding globulin measurement', 'red blood cell distribution width', 'mean corpuscular volume', 'myeloid white cell count'], ['cutaneous melanoma', 'lung adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'Niemann-Pick disease type C'], ['cleft lip', 'Cleft palate', 'mathematical ability', 'reticulocyte count', 'neoplasm'], ['Meckel syndrome', 'Joubert syndrome with ocular defect', 'Bardet-Biedl syndrome', 'Joubert syndrome', 'polydactyly (disease)'], ['serum gamma-glutamyl transferase measurement', 'lean body mass', 'pulse pressure measurement', 'serum alanine aminotransferase measurement', 'acute myeloid leukemia'], ['B-cell acute lymphoblastic leukemia', 'hemoglobin measurement', 'platelet component distribution width', 'hematocrit', 'glomerular filtration rate'], ['glioblastoma multiforme', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'appendicular lean mass'], ['Malonic aciduria', 'hypertrophic cardiomyopathy', 'mental retardation', 'Intellectual disability', 'adolescent idiopathic scoliosis'], ['Vitamin B12-responsive methylmalonic acidemia type cblB', 'methylmalonic aciduria cblb type', 'Vitamin B12-responsive methylmalonic acidemia', 'Methylmalonic acidemia with homocystinuria', 'Methylmalonic acidemia without homocystinuria'], ['heel bone mineral density', 'Jaundice', 'age at menarche', 'hip circumference', 'BMI-adjusted waist-hip ratio'], ['charcot-marie-tooth disease, axonal, type 2t', 'Spinocerebellar ataxia type 43', 'Autosomal dominant cerebellar ataxia', 'heart failure', 'Reduced ejection fraction'], ['celiac disease', 'multiple sclerosis', 'rheumatoid arthritis', 'autoimmune disease', 'sclerosing cholangitis'], ['acne', 'rosacea', 'COVID-19', 'Helicobacter pylori infectious disease', 'chronic obstructive pulmonary disease'], ['Torg-Winchester syndrome', 'Multicentric osteolysis-nodulosis-arthropathy spectrum', 'Nodulosis-arthropathy-osteolysis syndrome', 'lip cancer', 'heel bone mineral density'], ['coronary artery disease', 'matrix metalloproteinase measurement', 'blood protein measurement', 'lung cancer', "Alzheimer's disease"], ['acne', 'chronic obstructive pulmonary disease', 'rosacea', 'periodontitis', 'COVID-19'], ['Metaphyseal anadysplasia', 'lumbar disc degeneration', 'lung cancer', 'gastric adenocarcinoma', 'breast cancer'], ['ulcerative colitis', 'pterygium', 'lung adenocarcinoma', 'breast adenocarcinoma', 'serum IgG glycosylation measurement'], ['lung cancer', 'breast cancer', 'matrix metalloproteinase 12 measurement', 'blood protein measurement', 'matrix metalloproteinase 10 measurement'], ['Spondyloepimetaphyseal dysplasia, Missouri type', 'Metaphyseal anadysplasia', 'Metaphyseal chondrodysplasia, Spahr type', 'acne', 'rosacea'], ['schizophrenia', 'appendicular lean mass', 'heel bone mineral density', 'anorexia nervosa', 'BMI-adjusted waist-hip ratio'], ['insomnia measurement', 'arterial stiffness measurement', 'appendicular lean mass', 'breast adenocarcinoma', 'drug use measurement'], ['self reported educational attainment', 'mathematical ability', 'diet measurement', 'alcohol consumption measurement', 'appendicular lean mass'], ['Xanthinuria type II', 'Hereditary xanthinuria', 'Primary hyperoxaluria type 1', 'Homozygous familial hypercholesterolemia', 'Interauricular communication'], ['hematocrit', 'hemoglobin measurement', 'high density lipoprotein cholesterol measurement', 'erythrocyte count', 'red blood cell density measurement'], ['triglyceride measurement', 'balding measurement', 'body height', 'sex hormone-binding globulin measurement', 'rheumatoid arthritis'], ['reticulocyte count', 'refractive error measurement', 'appendicular lean mass', 'body height', 'monocyte count'], ['body height', 'sitting height measurement', 'isolated hyperchlorhidrosis', 'glioma', 'waist-hip ratio'], ['infect', 'infection', 'protein measurement', 'body height', 'Acute hepatic failure'], ['breast adenocarcinoma', 'blood protein measurement', 'glioblastoma multiforme', 'adolescent idiopathic scoliosis', 'mean corpuscular volume'], ['lean body mass', 'body height', 'fat body mass', 'cutaneous melanoma', 'telomere length'], ['thrombocytopenia, anemia, and myelofibrosis', 'Thrombocytopenia', 'blood platelet disease', 'blood protein measurement', 'Abnormal bleeding'], ['Congenital amegakaryocytic thrombocytopenia', 'Familial thrombocytosis', 'thrombocythemia 2', 'essential thrombocythemia', 'Thrombocytopenia'], ['Myeloperoxidase deficiency', 'neutrophil count', 'basophil count', 'myeloid white cell count', 'neutrophil percentage of leukocytes'], ['cancer', 'hemoglobin measurement', 'hematocrit', 'lung adenocarcinoma', 'mean corpuscular hemoglobin concentration'], ['red blood cell distribution width', 'protein measurement', 'eye colour measurement', 'mean corpuscular hemoglobin concentration', 'schizophrenia'], ['Navajo neurohepatopathy', 'Autosomal recessive axonal Charcot-Marie-Tooth disease type 2', 'Mitochondrial DNA depletion syndrome', 'Mitochondrial DNA depletion syndrome, hepatocerebral form', 'Hepatic failure'], ['Charcot-Marie-Tooth disease type 1B', 'Autosomal dominant Charcot-Marie-Tooth disease type 2I', 'Dejerine-Sottas syndrome', 'Autosomal dominant Charcot-Marie-Tooth disease type 2J', 'Autosomal dominant intermediate Charcot-Marie-Tooth disease type D'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness', 'Non-syndromic genetic deafness', 'erythrocyte count', 'non-small cell lung carcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'brain volume measurement', 'eosinophil count', 'neuroimaging measurement', 'cortical thickness'], ['Familial glucocorticoid deficiency', 'Cytomegalic congenital adrenal hypoplasia', 'hypertension', 'blood pressure', 'aortic stenosis'], ['Tuberculosis', 'myeloid white cell count', 'neutrophil count', 'monocyte count', 'heel bone mineral density'], ['reticulocyte count', 'protein measurement', 'body mass index', 'body height', 'hip circumference'], ['risk-taking behaviour', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'Nuclear cataract', 'Total congenital cataract'], ['Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract', 'Coralliform cataract'], ['pulse pressure measurement', 'systolic blood pressure', 'Cholesterol-ester transfer protein deficiency', 'high density lipoprotein cholesterol measurement', 'childhood onset asthma'], ['chronic obstructive pulmonary disease', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'colorectal carcinoma', 'neoplasm'], ['alkaline phosphatase measurement', 'liver enzyme measurement', 'non-alcoholic steatohepatitis', 'glomerular filtration rate', 'miotic rate'], ['schizophrenia', 'melanoma', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'Hepatobiliary Neoplasm'], ['Common variable immunodeficiency', 'rheumatoid arthritis', 'non-Hodgkins lymphoma', 'multiple sclerosis', 'chronic lymphocytic leukemia'], ['cutaneous melanoma', 'adolescent idiopathic scoliosis', 'nevus count', 'body height', 'fatty acid measurement'], ['cancer antigen 125 measurement', 'erythrocyte count', 'pancreatic carcinoma', 'neoplasm', 'mesothelioma'], ['prostate carcinoma', 'prostate specific antigen measurement', 'male reproductive organ cancer', 'blood protein measurement', 'cancer biomarker measurement'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'Primary immunodeficiency', 'esophageal cancer', 'anaplastic large cell lymphoma'], ["Barrett's esophagus", 'prostate carcinoma', 'prostate adenocarcinoma', 'Familial prostate cancer', 'esophageal adenocarcinoma'], ['ulcerative colitis', "Crohn's disease", 'inflammatory bowel disease', 'sclerosing cholangitis', 'psoriasis'], ['nasopharyngeal neoplasm', 'body mass index', 'multiple myeloma', 'body fat percentage', 'age at first birth measurement'], ['Myostatin-related muscle hypertrophy', 'Duchenne muscular dystrophy', 'cancer', 'self reported educational attainment', 'muscular disease'], ['mitochondrial myopathy-cerebellar ataxia-pigmentary retinopathy syndrome', 'Mitochondrial myopathy', 'Mitochondrial myopathy with reversible cytochrome C oxidase deficiency', 'genetic disorder', 'gastric carcinoma'], ['Craniosynostosis, Boston type', 'Parietal foramina', 'Parietal foramina with cleidocranial dysplasia', 'hair color', 'Craniosynostosis'], ['Isolated cytochrome C oxidase deficiency', 'Mitochondrial non-syndromic sensorineural deafness', 'Leber hereditary optic neuropathy', 'MELAS', 'Mitochondrial non-syndromic sensorineural deafness with susceptibility to aminoglycoside exposure'], ['Isolated cytochrome C oxidase deficiency', 'MELAS', 'Mitochondrial disease', 'Kearns-Sayre syndrome', 'leigh syndrome due to mitochondrial complex iv deficiency'], ['Leber hereditary optic neuropathy', 'leigh syndrome due to mitochondrial complex iv deficiency', 'Isolated cytochrome C oxidase deficiency', 'MELAS', 'Leigh syndrome'], ['adolescent idiopathic scoliosis', 'hepatocellular carcinoma', 'medulloblastoma', 'Kidney Oncocytoma', 'ulcerative colitis'], ['endometrial cancer', 'breast adenocarcinoma', 'acute myeloid leukemia', 'low grade glioma', 'glioblastoma multiforme'], ['Diaphyseal medullary stenosis - bone malignancy', 'cutaneous melanoma', 'nevus count', 'skin cancer', 'protein measurement'], ['Left ventricular noncompaction', 'DNA methylation', 'hypertrophic cardiomyopathy 4', 'Glycogen storage disease due to LAMP-2 deficiency', 'Glycogen Storage Disease Type 2b'], ['combined immunodeficiency and megaloblastic anemia with or without hyperhomocysteinemia', 'Neural tube closure defect', 'Isolated spina bifida', 'colorectal adenocarcinoma', 'colorectal neoplasm'], ['mean platelet volume', 'cancer', 'neoplasm', 'acute myeloid leukemia', 'non-small cell lung carcinoma'], ['reticulocyte count', 'cervical cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'Tibial muscular dystrophy'], ['hemoglobin measurement', 'macula measurement', 'breast adenocarcinoma', 'glioblastoma multiforme', 'type II diabetes mellitus'], ['insomnia', 'delirium', 'schizophrenia', 'major depressive disorder', 'bipolar disorder'], ['insomnia', 'delirium', 'schizophrenia', 'major depressive disorder', 'bipolar disorder'], ['Isolated focal cortical dysplasia type II', 'Macrocephaly-intellectual disability-neurodevelopmental disorder-small thorax syndrome', 'Isolated focal cortical dysplasia type IIa', 'Isolated focal cortical dysplasia type IIb', 'Isolated focal cortical dysplasia'], ['platelet component distribution width', 'platelet count', 'mean platelet volume', 'left ventricular hypertrophy', 'primary hypertension'], ['neoplasm', 'appendicular lean mass', 'electrocardiography', 'mean platelet volume', 'hemorrhoid'], ['Abetalipoproteinemia', 'familial hypercholesterolemia', 'cardiovascular disease', 'Homozygous familial hypercholesterolemia', 'vital capacity'], ['erythrocyte count', 'hemoglobin measurement', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'gastric carcinoma'], ['Autosomal dominant medullary cystic kidney disease with or without hyperuricemia', 'Autosomal dominant medullary cystic kidney disease without hyperuricemia', 'lung adenocarcinoma', 'ovarian cancer', 'peritoneum cancer'], ['adult onset asthma', 'interstitial lung disease', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['glomerular filtration rate', 'mean corpuscular hemoglobin', 'head and neck squamous cell carcinoma', 'pancreatic adenocarcinoma', 'red blood cell distribution width'], ['asthma', 'pancreatic carcinoma', 'eosinophil count', 'breast adenocarcinoma', 'cholelithiasis'], ['asthma', 'inherited susceptibility to asthma', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['cancer antigen 125 measurement', 'ovarian cancer', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma'], ['glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma', 'breast adenocarcinoma'], ['glioblastoma multiforme', 'body height', 'protein measurement', 'esophageal cancer', 'Blackfan-Diamond anemia'], ['Congenital myasthenic syndromes', 'Fetal akinesia deformation sequence', 'Postsynaptic congenital myasthenic syndromes', 'heel bone mineral density', 'Respiratory insufficiency'], ['Disseminated superficial actinic porokeratosis', 'mean corpuscular hemoglobin', 'neuroblastoma', 'esophageal squamous cell carcinoma', 'breast carcinoma'], ['cancer', 'diabetes mellitus', 'heart failure', 'mortality', 'stroke'], ['melanoma', 'cutaneous melanoma', 'nevus count', 'thyroid gland adenocarcinoma', 'blood protein measurement'], ['basophil count', 'neutrophil count', 'mean platelet volume', 'leukocyte count', 'monocyte count'], ['hypertrophic cardiomyopathy', 'hypertrophic cardiomyopathy 4', 'Left ventricular noncompaction', 'Familial dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy'], ['drug use measurement', 'blood protein measurement', 'systolic blood pressure', 'YKL40 measurement', 'protein measurement'], ['PR interval', 'total cholesterol measurement', 'jaw fracture', 'apolipoprotein B measurement', 'glomerular filtration rate'], ['Burkitts lymphoma', 'prostate carcinoma', 'bladder carcinoma', 'diffuse large B-cell lymphoma', 'lung adenocarcinoma'], ['Other immunodeficiency syndrome with predominantly antibody defects', 'prostate carcinoma', 'acute lymphoblastic leukemia', 'colonic neoplasm', 'colon carcinoma'], ['Feingold syndrome type 1', 'Feingold syndrome', 'basal cell carcinoma', 'genetic disorder', 'kidney Wilms tumor'], ['Pyogenic bacterial infections due to MyD88 deficiency', 'chronic lymphocytic leukemia', 'Primary immunodeficiency', 'non-Hodgkins lymphoma', 'diffuse large B-cell lymphoma'], ['hepatocellular carcinoma', 'neoplasm', 'rheumatoid arthritis', 'optic cup area measurement', 'refractory anemia'], ['myopathy, proximal, and ophthalmoplegia', 'Hereditary inclusion body myopathy - joint contractures - ophthalmoplegia', 'inclusion body myositis', 'Childhood-onset autosomal recessive myopathy with external ophthalmoplegia', 'cutaneous Leishmaniasis'], ['Familial atrial fibrillation', 'heart failure', 'Reduced ejection fraction', 'breast adenocarcinoma', 'body fat percentage'], ['lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'Familial visceral myopathy', 'endometrial cancer'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'Hemoglobin E - beta-thalassemia'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Rare genetic vascular disease', 'Megacystis-microcolon-intestinal hypoperistalsis syndrome', 'Familial visceral myopathy', 'Rare disease with thoracic aortic aneurysm and aortic dissection'], ['hypertrophic cardiomyopathy', 'cardiomyopathy', 'platelet count', 'Rare familial disorder with hypertrophic cardiomyopathy', 'ventricular tachycardia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'cancer'], ['heel bone mineral density', 'Familial primary pulmonary hypoplasia', 'Postsynaptic congenital myasthenic syndromes', 'Hereditary proximal myopathy with early respiratory failure', 'Distal spinal muscular atrophy type 3'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'cutaneous Leishmaniasis', 'endometrial cancer', 'breast adenocarcinoma', 'mean platelet volume'], ['lymphocyte count', 'leukocyte count', 'neutrophil count', 'myeloid white cell count', 'monocyte count'], ['experimental autoimmune encephalomyelitis', 'stroke', 'systemic lupus erythematosus', 'infection', 'infect'], ['systolic blood pressure', 'hypertension', 'blood pressure', 'cardiovascular disease', 'diet measurement'], ['N-acylethanolamine-hydrolyzing acid amidase measurement', 'blood protein measurement', 'cerebrospinal fluid biomarker measurement', 'protein measurement', 'neoplasm'], ['Sanfilippo syndrome type B', 'Mucopolysaccharidosis type 3', 'Charcot-Marie-Tooth disease axonal type 2V', 'Mucopolysaccharidosis', 'genetic disorder'], ['Hyperammonemia due to N-acetylglutamate synthetase deficiency', 'Hyperammonemia', 'selective IgA deficiency disease', 'apolipoprotein B measurement', 'squamous cell lung carcinoma'], ['age at menopause', 'protein measurement', 'mean platelet volume', 'lung adenocarcinoma', 'neutrophil count'], ['head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'cutaneous melanoma', 'lung adenocarcinoma', 'neoplasm'], ['smoking behavior', '3-hydroxy-1-methylpropylmercapturic acid measurement', 'hepatocellular carcinoma', 'obsessive-compulsive disorder', 'Hereditary geniospasm'], ['protein measurement', 'red blood cell distribution width', 'schizophrenia', 'interleukin 18 measurement', 'neoplasm'], ['Neurodevelopmental disorder', 'Isolated NADH-CoQ reductase deficiency', 'hemoglobin measurement', 'erythrocyte count', 'hematocrit'], ['glomerular filtration rate', 'melanoma', 'platelet count', 'vitamin D measurement', 'protein measurement'], ['total cholesterol measurement', 'triglyceride measurement', 'urinary metabolite measurement', 'chronic kidney disease', 'mean corpuscular hemoglobin concentration'], ['chronic kidney disease', 'urinary metabolite measurement', 'blood metabolite measurement', 'glomerular filtration rate', 'hydroxy-leucine measurement'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'optic disc size measurement', 'pulse pressure measurement'], ['Gray platelet syndrome', 'Thrombocytopenia', 'Abnormal bleeding', 'granulocyte percentage of myeloid white cells', 'neutrophil percentage of leukocytes'], ['body mass index', 'neuroimaging measurement', 'refractive error measurement', 'waist-hip ratio', 'brain measurement'], ['Nijmegen breakage syndrome', 'idiopathic aplastic anemia', 'acute lymphoblastic leukemia', 'Aplastic anemia', 'Inherited cancer-predisposing syndrome'], ['neoplasm', 'atrial fibrillation', 'endometriosis', 'cancer', 'Intermediate osteopetrosis'], ['bone quantitative ultrasound measurement', 'smoking behavior', 'smoking status measurement', 'heel bone mineral density', 'body mass index'], ['platelet crit', 'mosaic loss of chromosome Y measurement', 'glioblastoma multiforme', 'endometrial cancer', 'small cell lung carcinoma'], ['neuroimaging measurement', 'refractive error measurement', 'PR interval', 'brain volume measurement', 'brain measurement'], ['head and neck squamous cell carcinoma', 'melanoma', 'colorectal adenocarcinoma', 'colon adenocarcinoma', 'basal cell carcinoma'], ['appendicular lean mass', 'body height', 'lean body mass', 'vital capacity', 'C-reactive protein measurement'], ['breast carcinoma', 'alveolar rhabdomyosarcoma', 'embryonal rhabdomyosarcoma', 'diffuse large B-cell lymphoma', 'breast adenocarcinoma'], ['platelet count', 'osteoarthritis', 'high density lipoprotein cholesterol measurement', 'endometrial cancer', 'breast adenocarcinoma'], ['neuroticism measurement', 'wellbeing measurement', 'unipolar depression', 'erythrocyte count', 'mood disorder'], ['endometrial cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Irritability', 'glomerular filtration rate'], ['breast adenocarcinoma', 'cutaneous melanoma', 'thyroid neoplasm', 'colorectal adenocarcinoma', 'viral disease'], ['prostate adenocarcinoma', 'colorectal adenocarcinoma', 'body height', 'lean body mass', 'cutaneous melanoma'], ['viral disease', 'complement C4 measurement', 'visual perception measurement', 'mathematical ability', 'FEV/FEC ratio'], ['body mass index', 'type II diabetes mellitus', 'systolic blood pressure', 'blood pressure', 'alcohol drinking'], ['acne inversa, familial, 1', "Alzheimer's disease", 'hidradenitis suppurativa', 'Dementia', 'head and neck squamous cell carcinoma'], ['gut microbiome measurement', 'hepatocellular carcinoma', 'cancer', 'colorectal carcinoma', 'primary cutaneous amyloidosis'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['Norrie disease', 'Familial exudative vitreoretinopathy', 'Persistent hyperplastic primary vitreous', 'retinal detachment', 'Short lingual frenulum'], ['Charcot-Marie-Tooth disease type 4D', 'Charcot-Marie-Tooth disease', 'Charcot-Marie-Tooth disease type 4', 'colorectal adenocarcinoma', 'gastric carcinoma'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus', 'gestational diabetes'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus', 'gestational diabetes'], ['Isolated NADH-CoQ reductase deficiency', 'type II diabetes mellitus', 'diabetes mellitus', 'polycystic ovary syndrome', 'type I diabetes mellitus'], ['Zlotogora-Ogur syndrome', 'EEC syndrome', 'cleft lip', 'Orofacial clefting syndrome', 'mathematical ability'], ["Alzheimer's disease", 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement', 'mean platelet volume', 'family history of Alzheimer’s disease'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2', 'Charcot-Marie-Tooth disease axonal type 2CC', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'thyroid gland adenocarcinoma'], ['Charcot-Marie-Tooth disease type 1F', 'Autosomal dominant Charcot-Marie-Tooth disease type 2E', 'Charcot-Marie-Tooth disease, dominant intermediate G', 'autosomal dominant intermediate Charcot-Marie-Tooth disease type G', 'Charcot-Marie-Tooth disease'], ['cancer', 'breast adenocarcinoma', 'platelet count', 'mean platelet volume', 'vital capacity'], ['HIV infection', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['HIV infection', 'breast adenocarcinoma', 'visual perception measurement', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['blood protein measurement', 'self reported educational attainment', 'adolescent idiopathic scoliosis', 'type II diabetes mellitus', 'acute myeloid leukemia'], ['fibula fracture', 'neoplasm', 'lymphocyte count', 'glioma', 'breast cancer'], ['sialidosis type II', 'sialidosis type I', 'sialidosis', 'Non-immune hydrops fetalis', 'Lysosomal disease'], ['atrial fibrillation', 'atrial flutter', 'white matter hyperintensity measurement', 'cardiac edema', 'cardiac arrhythmia'], ['MODY', 'maturity-onset diabetes of the young type 6', 'type II diabetes mellitus', 'monogenic diabetes', 'body mass index'], ['Congenital malabsorptive diarrhea due to paucity of enteroendocrine cells', 'type II diabetes mellitus', 'body height', 'drug use measurement', 'diabetes mellitus'], ['Neurofibromatosis type 1', 'Neurofibromatosis-Noonan syndrome', 'Neurofibromatosis type 1 due to NF1mutation or intragenic deletion', 'Juvenile Myelomonocytic Leukemia', 'plexiform neurofibroma'], ['Neurofibromatosis type 2', 'Neurofibromatosis type 3', 'schwannoma', 'mesothelioma', 'Meningioma'], ['platelet count', 'platelet crit', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell distribution width'], ['immunodeficiency, developmental delay, and hypohomocysteinemia', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'liver cancer', 'papillary renal cell carcinoma'], ['sex hormone-binding globulin measurement', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'skin aging measurement', 'high density lipoprotein cholesterol measurement'], ['Marshall-Smith syndrome', 'Sotos syndrome', 'Malan overgrowth syndrome', 'genetic disorder', 'lymphocyte count'], ['Common variable immunodeficiency', 'infectious disease', 'Primary immunodeficiency', 'eosinophil count', 'ulcerative colitis'], ['Common variable immunodeficiency', 'immunodeficiency, common variable, 10', 'Deficiency in anterior pituitary function-variable immunodeficiency syndrome', 'Primary immunodeficiency', 'colon adenocarcinoma'], ['protein measurement', 'stroke', 'mathematical ability', 'neoplasm', 'cancer'], ['Hereditary sensory and autonomic neuropathy type 5', 'erythrocyte count', 'diastolic blood pressure', 'hypertension', 'osteoarthritis'], ['Nance-Horan syndrome', 'Nuclear cataract', 'Non-syndromic congenital cataract', 'genetic disorder', 'Total congenital cataract'], ['mean platelet volume', 'Isolated Dandy-Walker malformation', 'blood protein measurement', 'waist-hip ratio', 'Hydrocephalus'], ['hypertension', 'schizophrenia', 'obesity', 'atrial fibrillation', 'nervous system disease'], ['acute myeloid leukemia', 'body mass index', 'hip circumference', 'Triple A syndrome', 'systemic lupus erythematosus'], ['Abnormality of refraction', 'Ewing sarcoma', 'chronotype measurement', 'type II diabetes mellitus', 'head and neck squamous cell carcinoma'], ['peak expiratory flow', 'type II diabetes mellitus', 'adolescent idiopathic scoliosis', 'cutaneous melanoma', 'colorectal carcinoma'], ['spastic ataxia 8, autosomal recessive, with hypomyelinating leukodystrophy', 'Autosomal recessive spastic ataxia with leukoencephalopathy', 'Hypomyelination - hypogonadotropic hypogonadism - hypodontia', 'visual perception measurement', 'migraine disorder'], ['body fat percentage', 'body mass index', 'lean body mass', 'smoking status measurement', 'type II diabetes mellitus'], ['blood protein measurement', 'anxiety', 'schizophrenia', 'Huntington disease-like 2', 'Huntington disease-like syndrome due to C9ORF72 expansions'], ['Corneal intraepithelial dyskeratosis with palmoplantar hyperkeratosis and laryngeal dyskeratosis', 'autoinflammation with arthritis and dyskeratosis', 'respiratory papillomatosis, juvenile recurrent, congenital', 'insulin like growth factor measurement', 'autoinflammatory syndrome'], ['venous thromboembolism', 'hepatocellular carcinoma', 'neoplasm', 'intrahepatic cholangiocarcinoma', 'laryngeal carcinoma'], ['CINCA syndrome', 'Familial cold urticaria', 'Muckle-Wells syndrome', 'keratitis fugax hereditaria', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['dilated cardiomyopathy', 'anorexia nervosa', 'bipolar disorder', 'schizophrenia', 'autism spectrum disorder'], ['blood protein measurement', 'melanoma', 'neoplasm', 'hepatocellular carcinoma', 'acute myeloid leukemia'], ['body height', 'sitting height measurement', 'white matter microstructure measurement', 'blood protein measurement', 'body weight'], ['type II diabetes mellitus', 'blood protein measurement', 'tumor necrosis factor-inducible gene 6 protein measurement', 'protein measurement', 'obesity'], ['Leber congenital amaurosis', 'Cone rod dystrophy', 'Retinal dystrophy', 'Nystagmus', 'Diarrhea'], ['HIV infection', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'systolic blood pressure', 'endometrial cancer'], ['platelet count', 'central nervous system cancer', 'mean platelet volume', 'heel bone mineral density', 'monocyte percentage of leukocytes'], ['ovarian cancer', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'heel bone mineral density'], ['smoking status measurement', 'body mass index', 'self reported educational attainment', 'neuroticism measurement', 'protein measurement'], ['mean corpuscular volume', 'reticulocyte count', 'reticulocyte measurement', 'myeloid white cell count', 'mean reticulocyte volume'], ['Blau syndrome', "Crohn's disease", 'ulcerative colitis', "Behcet's syndrome", 'inflammatory bowel disease'], ['Familial cortical myoclonus', 'non-small cell lung carcinoma', 'cancer', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'Familial hypofibrinogenemia'], ['protein measurement', 'schizophrenia', 'bladder carcinoma', 'colorectal cancer', 'type II diabetes mellitus'], ['psoriasis', 'psoriasis vulgaris', 'nitric oxide exhalation measurement', 'inflammatory bowel disease', 'eosinophil count'], ['Adams-Oliver syndrome', 'Familial bicuspid aortic valve', 'head and neck squamous cell carcinoma', 'aortic stenosis', 'cancer'], ['Acroosteolysis dominant type', 'Alagille syndrome', 'Alagille syndrome due to a NOTCH2 point mutation', 'basal cell carcinoma', 'prostate adenocarcinoma'], ['CADASIL', 'Lateral meningocele syndrome', 'Infantile myofibromatosis', 'migraine disorder', 'dementia (disease)'], ['colorectal cancer', 'chronic hepatitis B virus infection', 'head and neck squamous cell carcinoma', 'asthma', 'systemic scleroderma'], ['neurodevelopmental disorder with or without autistic features and/or structural brain abnormalities', 'Intellectual disability, severe', 'chronotype measurement', 'electrocardiography', 'non-small cell lung carcinoma'], ['lung adenocarcinoma', 'colon carcinoma', 'colonic neoplasm', 'colorectal adenocarcinoma', 'malignant colon neoplasm'], ['Ataxia-telangiectasia', 'eosinophil count', 'mean corpuscular volume', 'platelet count', 'eosinophil percentage of leukocytes'], ['Niemann-Pick disease type C', 'Niemann-Pick disease', 'genetic disorder', 'Aceruloplasminemia', 'Cognitive impairment'], ['Niemann-Pick disease type C', 'Niemann-Pick disease type C, severe early infantile neurologic onset', 'dystonic disorder', 'Niemann-Pick disease type C, late infantile neurologic onset', 'Niemann-Pick disease type C, severe perinatal form'], ['acute myeloid leukemia', 'mean platelet volume', 'low density lipoprotein cholesterol measurement', 'body height', 'total cholesterol measurement'], ['cutaneous melanoma', '3-hydroxy-3-methylglutaryl-CoA synthase deficiency', 'hyperproinsulinemia', 'Familial glucocorticoid deficiency', 'Exercise-induced hyperinsulinism'], ['Juvenile nephronophthisis', 'Joubert syndrome with renal defect', 'Nephronophthisis', 'Senior-Loken syndrome', 'Joubert syndrome'], ['Congenital nephrotic syndrome, Finnish type', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'familial nephrotic syndrome', 'focal segmental glomerulosclerosis', 'nephrotic syndrome'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'idiopathic nephrotic syndrome', 'response to prednisolone', 'Congenital nephrotic syndrome, Finnish type', 'Proteinuria'], ['Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency', 'Sitosterolemia', 'hypercholesterolemia, autosomal dominant, 3', 'Ataxia with vitamin E deficiency'], ['refractive error measurement', 'low grade glioma', 'breast adenocarcinoma', 'Astigmatism', 'red blood cell distribution width'], ['acute myeloid leukemia', 'HIV infection', 'Dyskeratosis congenita', 'anaplastic large cell lymphoma', 'Hodgkins lymphoma'], ['Familial atrial fibrillation', 'Atrial stand still', 'AL amyloidosis', 'systolic blood pressure', 'breast adenocarcinoma'], ['BNP measurement', 'diastolic blood pressure', 'smoking status measurement', 'alcohol drinking', 'pulse pressure measurement'], ['body height', 'lean body mass', 'appendicular lean mass', 'fat body mass', 'forced expiratory volume'], ['congestive heart failure', 'hypertension', 'heart failure', 'pulmonary hypertension', 'obesity'], ['Acromesomelic dysplasia, Maroteaux type', 'Tall stature - scoliosis - macrodactyly of the great toes', 'Metaphyseal chondrodysplasia, Schmid type', 'Familial focal epilepsy with variable foci', 'Shprintzen-Goldberg syndrome'], ['body height', 'lean body mass', 'systolic blood pressure', 'hypertension', 'appendicular lean mass'], ['chronotype measurement', 'bladder disease', 'asthma', 'protein measurement', "Alzheimer's disease"], ['body mass index', 'cognitive function measurement', 'body height', 'self reported educational attainment', 'forced expiratory volume'], ['mathematical ability', 'self reported educational attainment', 'wellbeing measurement', 'daytime rest measurement', 'triglyceride measurement'], ['glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'low grade glioma'], ['Severe early-onset obesity-insulin resistance syndrome due to SH2B1 deficiency', 'Obesity due to melanocortin 4 receptor deficiency', 'Obesity due to prohormone convertase I deficiency', 'Obesity due to pro-opiomelanocortin deficiency', 'Obesity due to leptin receptor gene deficiency'], ['type II diabetes mellitus', 'diabetes mellitus', 'body mass index', 'appendicular lean mass', 'fat body mass'], ['Progressive familial intrahepatic cholestasis', 'primary biliary cirrhosis', 'morbid obesity', 'Fatigue', 'cholelithiasis'], ['high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement', 'total cholesterol measurement', 'lymphocyte percentage of leukocytes', 'adolescent idiopathic scoliosis'], ['red blood cell distribution width', 'Kleefstra syndrome', 'neoplasm', 'myocarditis', 'blood metabolite measurement'], ['alcohol consumption measurement', 'acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'prostate cancer'], ['Goldmann-Favre syndrome', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Joubert syndrome and related disorders', 'Cone rod dystrophy'], ['Glucocorticoid resistance', 'atrial fibrillation', 'multiple myeloma', 'asthma', 'acute lymphoblastic leukemia'], ['Renal pseudohypoaldosteronism type 1', 'Pseudohypoaldosteronism type 1', 'Pseudohyperaldosteronism type 2', 'hypertension', 'chronic kidney disease'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'sitting height measurement'], ['lung carcinoma', 'colorectal adenocarcinoma', 'chondrosarcoma', 'carcinoma of liver and intrahepatic biliary tract', 'colon adenocarcinoma'], ['46,XY partial gonadal dysgenesis', 'primary ovarian insufficiency', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation', '46,XX testicular disorder of sex development', '46,XY complete gonadal dysgenesis'], ['pancreatic carcinoma', 'vital capacity', 'ulcerative colitis', 'hematocrit', 'hemoglobin measurement'], ['age at menarche', 'sex hormone-binding globulin measurement', 'brain aneurysm', 'alcohol consumption measurement', 'cutaneous melanoma'], ['Noonan syndrome', 'cutaneous melanoma', 'multiple myeloma', 'Juvenile Myelomonocytic Leukemia', 'acute myeloid leukemia'], ['Cytomegalic congenital adrenal hypoplasia', '46,XY complete gonadal dysgenesis', '46,XX testicular disorder of sex development', '46,XY partial gonadal dysgenesis', 'Alternating hemiplegia of childhood'], ['obesity', 'Genetic obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'renal cell carcinoma', "Barrett's esophagus"], ['platelet count', 'platelet component distribution width', 'red blood cell distribution width', 'mean platelet volume', 'platelet crit'], ['cancer', 'lung adenocarcinoma', 'pancreatic carcinoma', 'pancreatic ductal adenocarcinoma', 'breast ductal adenocarcinoma'], ['cancer', 'hair colour measurement', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['cancer', 'kidney cancer', 'kidney neoplasm', 'acute myeloid leukemia', 'adolescent idiopathic scoliosis'], ['cancer', 'hemoglobin measurement', 'hematocrit', 'glomerular filtration rate', 'urate measurement'], ['schizophrenia', 'autism spectrum disorder', 'unipolar depression', 'bipolar disorder', 'attention deficit hyperactivity disorder'], ['Retinitis pigmentosa', 'Goldmann-Favre syndrome', 'Retinal dystrophy', 'self reported educational attainment', 'smoking status measurement'], ['body mass index', 'self reported educational attainment', 'protein measurement', "Alzheimer's disease", 'non-small cell lung carcinoma'], ['waist-hip ratio', 'acute myeloid leukemia', 'blood protein measurement', 'neuroimaging measurement', 'high density lipoprotein cholesterol measurement'], ['balding measurement', 'FEV/FEC ratio', 'genetic disorder', 'adolescent idiopathic scoliosis', 'breast adenocarcinoma'], ['connective tissue disease', 'adolescent idiopathic scoliosis', 'infection', 'infect', 'Dementia'], ['acute lymphoblastic leukemia', 'Wolf-Hirschhorn syndrome', 'head and neck squamous cell carcinoma', 'Intellectual disability', 'kidney neoplasm'], ['balding measurement', "Parkinson's disease", 'cortical surface area measurement', 'total cortical area measurement', 'smoking status measurement'], ['hair colour measurement', 'adolescent idiopathic scoliosis', 'hair color', 'DNA methylation', 'neoplasm'], ['Autosomal recessive non-syndromic intellectual disability', 'Dubowitz syndrome', 'genetic disorder', 'DNA methylation', 'heel bone mineral density'], ['sex hormone-binding globulin measurement', 'serum gamma-glutamyl transferase measurement', 'neoplasm', 'cancer', 'lupus erythematosus'], ['cutaneous melanoma', 'systemic lupus erythematosus', 'neoplasm', 'inclusion body myositis', 'pancreatic neoplasm'], ['Autosomal recessive spastic paraplegia type 45', 'Autosomal recessive spastic paraplegia type 48', 'Intellectual disability', 'lymphoid neoplasm', 'T-cell acute lymphoblastic leukemia'], ['appendicular lean mass', 'reticulocyte count', 'blood protein measurement', 'reticulocyte measurement', 'mean corpuscular volume'], ['mean platelet volume', 'platelet count', 'birth measurement', 'gestational age', 'osteosarcoma'], ['Hereditary arterial and articular multiple calcification syndrome', 'metabolite measurement', 'blood metabolite measurement', 'heel bone mineral density', 'cutaneous melanoma'], ['insomnia', 'BMI-adjusted waist circumference', 'diastolic blood pressure', 'body height', 'glaucoma'], ['open-angle glaucoma', 'genetic disorder', 'colorectal carcinoma', 'proliferative diabetic retinopathy', 'X-linked reticulate pigmentary disorder with systemic manifestations'], ['self reported educational attainment', 'body mass index', 'smoking status measurement', 'health study participation', 'lifestyle measurement'], ['Hereditary sensory and autonomic neuropathy type 4', 'Familial medullary thyroid carcinoma', 'neoplasm', 'non-small cell lung carcinoma', 'cancer'], ['obesity, hyperphagia, and developmental delay', 'Early infantile epileptic encephalopathy', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['neoplasm', 'non-small cell lung carcinoma', 'cancer', 'metastasis', 'cutaneous melanoma'], ['mathematical ability', 'breast ductal adenocarcinoma', 'hair colour measurement', 'hepatocellular carcinoma', 'cancer'], ['low density lipoprotein cholesterol measurement', 'C-reactive protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['visual perception measurement', 'breast cancer', 'neoplasm', 'diffuse large B-cell lymphoma', 'Autosomal dominant severe congenital neutropenia'], ['refractive error measurement', 'mean platelet volume', 'eosinophil count', 'BMI-adjusted waist circumference', 'physical activity measurement'], ['mean corpuscular volume', 'chronic kidney disease', 'urinary metabolite measurement', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'Autosomal recessive primary microcephaly', 'HIV infection', 'influenza', 'viral disease'], ['influenza', 'HIV infection', 'viral disease', 'Histiocytosis', 'blood protein measurement'], ['encephalopathy, acute, infection-induced', 'HIV infection', 'influenza', 'viral disease', 'T-cell acute lymphoblastic leukemia'], ['mean platelet volume', 'appendicular lean mass', 'hepatocellular carcinoma', 'prostate cancer', 'urinary bladder cancer'], ['NUT midline carcinoma', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'non-small cell lung carcinoma', 'Endometrial Endometrioid Adenocarcinoma'], ['Autosomal recessive Robinow syndrome', 'Robinow syndrome', 'colorectal cancer', 'colorectal adenoma', 'refractive error measurement'], ['Gyrate atrophy of choroid and retina', 'Retinal dystrophy', 'Ornithine transcarbamylase deficiency', 'Leber hereditary optic neuropathy', 'Abnormal choroid morphology'], ['Congenital intrauterine infection-like syndrome', 'Autosomal recessive primary microcephaly', 'Cerebral calcification', 'Global developmental delay', 'glioblastoma multiforme'], ['trypanosomiasis', 'Alopecia', 'Global developmental delay', 'Macrocephaly', 'Intellectual disability'], ['mean platelet volume', 'platelet component distribution width', 'serum IgM measurement', 'IGA glomerulonephritis', 'chronic lymphocytic leukemia'], ['cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'intelligence', 'pulse pressure measurement'], ['body height', 'creatinine measurement', 'waist-hip ratio', 'BMI-adjusted waist circumference', 'prostate cancer'], ['cancer', "Alzheimer's disease", 'clear cell renal carcinoma', 'breast adenocarcinoma', 'lung cancer'], ['self reported educational attainment', 'myocardial infarction', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'adolescent idiopathic scoliosis', 'esophageal cancer', 'hepatocellular carcinoma'], ['childhood onset asthma', 'protein measurement', 'infection', 'infect', "Alzheimer's disease"], ['Autosomal dominant optic atrophy, classic type', 'Autosomal dominant optic atrophy plus syndrome', 'Behr syndrome', '3-methylglutaconic aciduria type 3', 'mitochondrial DNA depletion syndrome 14 (cardioencephalomyopathic type)'], ['Blue cone monochromatism', 'red color blindness', 'Progressive cone dystrophy', 'red-green color blindness', 'Cone rod dystrophy'], ['red-green color blindness', 'Blue cone monochromatism', 'Progressive cone dystrophy', 'Cone rod dystrophy', 'Bradyopsia'], ['Tritanopia', 'breast adenocarcinoma', 'atrial fibrillation', 'cutaneous melanoma', 'mathematical ability'], ['alcohol dependence', 'opioid dependence', 'pain', 'migraine disorder', 'Chronic pain'], ['mean corpuscular volume', 'lymphocyte count', 'insulin like growth factor measurement', 'glioblastoma multiforme', 'monocyte count'], ['open-angle glaucoma', 'familial amyotrophic lateral sclerosis', 'amyotrophic lateral sclerosis', 'motor neuron disease', 'osteitis deformans'], ['neuroblastoma', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'multiple myeloma', 'acute myeloid leukemia'], ['medulloblastoma', 'ovarian cancer', 'clear cell renal carcinoma', 'breast adenocarcinoma', 'colorectal adenocarcinoma'], ['rheumatoid arthritis', 'refractory anemia', 'Neutropenia - monocytopenia - deafness', 'blood protein measurement', "Crohn's disease"], ['neutrophil count', 'myeloid white cell count', 'Niemann-Pick disease type C', 'cancer', 'neoplasm'], ['reticulocyte measurement', 'reticulocyte count', 'lung adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer'], ['Familial primary localized cutaneous amyloidosis', 'cutaneous nodular amyloidosis', 'ulcerative colitis', 'hepatocellular carcinoma', 'overall survival'], ['cup-to-disc ratio measurement', 'appendicitis', 'ventral hernia', 'sitting height measurement', 'body height'], ['Ornithine transcarbamylase deficiency', 'Hyperammonemia', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'genetic disorder'], ['white matter hyperintensity measurement', 'systolic blood pressure', 'mean corpuscular volume', 'smoking status measurement', 'diastolic blood pressure'], ['blood protein measurement', 'medulloblastoma', 'Neurodevelopmental disorder', 'autism', 'post-traumatic stress disorder'], ['postpartum hemorrhage', 'pregnancy', 'hemorrhage', 'schizophrenia', 'premature birth'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'Non-syndromic genetic deafness', 'mean corpuscular hemoglobin concentration', 'cutaneous melanoma'], ['Cough', 'triglyceride measurement', 'cutaneous melanoma', 'idiopathic pulmonary fibrosis', 'mean corpuscular hemoglobin concentration'], ['body mass index', 'type II diabetes mellitus', 'cutaneous melanoma', 'schizophrenia', 'fat body mass'], ['schizophrenia', 'serum alanine aminotransferase measurement', 'lung adenocarcinoma', 'lean body mass', 'late-onset Alzheimers disease'], ['dry eye syndrome', 'type II diabetes mellitus', 'oophoritis', 'salpingitis', 'mathematical ability'], ['dry eye syndrome', 'glioblastoma multiforme', 'breast adenocarcinoma', 'prostate adenocarcinoma', 'Pyknoachondrogenesis'], ['P2Y12 defect', 'myocardial infarction', 'coronary artery disease', 'acute coronary syndrome', 'ST Elevation Myocardial Infarction'], ['Cole-Carpenter syndrome', 'Osteogenesis imperfecta', 'lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['multiple myeloma', 'thyroid gland adenocarcinoma', 'squamous cell lung carcinoma', 'endometrial cancer', 'self reported educational attainment'], ['vitamin D measurement', 'breast adenocarcinoma', 'infect', 'infection', 'glioma'], ['rheumatoid arthritis', 'interstitial lung disease', 'basophil percentage of leukocytes', 'basophil count', 'neutrophil count'], ['monocyte count', 'monocyte percentage of leukocytes', 'diet measurement', 'breast adenocarcinoma', 'protein measurement'], ['Cryopyrin-associated periodic syndrome', 'prostate cancer', 'neoplasm', 'testicular carcinoma', 'ovarian neoplasm'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'infection', 'infect', 'obstructive sleep apnea'], ['Phenylketonuria', 'Maternal hyperphenylalaninemia', 'Hyperphenylalaninemia', 'Mild hyperphenylalaninemia', 'genetic disorder'], ['intellectual developmental disorder with macrocephaly, seizures, and speech delay', 'body fat percentage', 'fat body mass', 'ulcerative colitis', 'endometrial cancer'], ['electrocardiography', 'Non-syndromic male infertility due to sperm motility disorder', 'Partial chromosome Y deletion', 'Normosmic congenital hypogonadotropic hypogonadism', 'Isochromosomy Yp'], ['BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'hematocrit', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration'], ['spina bifida', 'Neural tube closure defect', 'Isolated spina bifida', 'Neural tube defect', 'refractive error measurement'], ['glomerular filtration rate', 'cancer', 'lung cancer', 'ovarian cancer', 'prostate cancer'], ['Young adult-onset Parkinsonism', "Parkinson's disease", 'ulcerative colitis', "Crohn's disease", 'body weight'], ['ovarian cancer', 'neoplasm', 'ovarian carcinoma', 'breast neoplasm', 'peritoneal neoplasm'], ['ovarian cancer', 'neoplasm', 'ovarian carcinoma', 'breast neoplasm', 'peritoneal neoplasm'], ['neoplasm', 'ovarian cancer', 'ovarian carcinoma', 'breast cancer', 'non-small cell lung carcinoma'], ['balding measurement', 'breast adenocarcinoma', 'acute myeloid leukemia', 'melanoma', 'body fat percentage'], ['neuroimaging measurement', 'myeloid white cell count', 'cortical surface area measurement', 'leukocyte count', 'brain measurement'], ['Peters anomaly', 'Isolated aniridia', 'Aniridia', 'Foveal hypoplasia - presenile cataract', 'Autosomal dominant keratitis'], ['body height', 'glioblastoma multiforme', 'neoplasm', 'hepatocellular carcinoma', 'glioma'], ['clear cell renal carcinoma', 'cutaneous melanoma', 'renal cell carcinoma', 'renal carcinoma', 'papillary renal cell carcinoma'], ['congenital anomalies of kidney and urinary tract syndrome with or without hearing loss, abnormal ears, or developmental delay', 'acute lymphoblastic leukemia', 'genetic disorder', 'B-cell acute lymphoblastic leukemia', 'myoepithelial tumor'], ['pyruvate carboxylase deficiency disease', 'Pyruvate carboxylase deficiency', 'Congenital lactic acidosis', 'Pyruvate carboxylase deficiency, benign type', 'Pyruvate carboxylase deficiency, severe neonatal type'], ['Hyperphenylalaninemia', 'Dehydratase deficiency', 'type II diabetes mellitus', 'smoking status measurement', 'red blood cell distribution width'], ['lung adenocarcinoma', 'endometrial cancer', 'low grade glioma', 'gastric adenocarcinoma', 'breast adenocarcinoma'], ['Propionic acidemia', 'self reported educational attainment', 'smoking behavior', 'Abnormality of refraction', 'hair shape measurement'], ['Propionic acidemia', 'dilated cardiomyopathy', 'fibrinogen measurement', 'triglyceride measurement', 'head and neck squamous cell carcinoma'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'hepatocellular carcinoma', 'non-small cell lung carcinoma'], ['Ataxia-telangiectasia-like disorder', 'lung adenocarcinoma', 'gastric adenocarcinoma', 'body height', 'clear cell renal carcinoma'], ['Obesity due to prohormone convertase I deficiency', 'lean body mass', 'body mass index', 'fat body mass', 'insulin measurement'], ['body height', 'refractive error measurement', 'low density lipoprotein cholesterol measurement', 'intraocular pressure measurement', 'vital capacity'], ['appendicular lean mass', 'body height', 'soluble transferrin receptor measurement', 'iron biomarker measurement', 'triglyceride measurement'], ['familial hypercholesterolemia', 'hypercholesterolemia, autosomal dominant, 3', 'coronary artery disease', 'Homozygous familial hypercholesterolemia', 'Hypercholesterolemia'], ['Spondylometaphyseal dysplasia - cone-rod dystrophy', 'Leber congenital amaurosis', 'genetic disorder', 'mean platelet volume', 'glioblastoma multiforme'], ['glomerular filtration rate', 'rheumatoid arthritis', 'metabolic syndrome', 'neoplasm', 'pseudotumor cerebri'], ['non-small cell lung carcinoma', 'renal cell carcinoma', 'neoplasm', 'metastatic melanoma', 'head and neck squamous cell carcinoma'], ['cutaneous melanoma', 'bile duct carcinoma', 'Hepatobiliary Neoplasm', 'breast ductal adenocarcinoma', 'kidney neoplasm'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'deafness, autosomal dominant 74', 'chronic kidney disease', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['Brachydactyly - arterial hypertension', 'stroke', 'coronary artery disease', 'type II diabetes mellitus', 'cardiovascular disease'], ['asthma', 'essential thrombocythemia', 'heart failure', 'Airway obstruction', 'coronary artery disease'], ['psoriasis', 'asthma', 'chronic obstructive pulmonary disease', 'Airway obstruction', 'atopic eczema'], ['Headache', 'asthma', 'chronic obstructive pulmonary disease', 'psoriasis', 'Airway obstruction'], ['Acrodysostosis', 'Acrodysostosis with multiple hormone resistance', 'asthma', 'chronic obstructive pulmonary disease', 'psoriasis'], ['erectile dysfunction', 'pulmonary arterial hypertension', 'benign prostatic hyperplasia', 'Impotence', 'coronary artery disease'], ['Retinitis pigmentosa', 'Congenital stationary night blindness', 'stroke', 'coronary artery disease', 'Retinal dystrophy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['stroke', 'coronary artery disease', 'chronic kidney disease', 'Hepatitis, Alcoholic', 'diabetic nephropathy'], ['dyskinesia, limb and orofacial, infantile-onset', 'Autosomal dominant striatal neurodegeneration', 'stroke', 'coronary artery disease', 'bipolar disorder'], ['Congenital disorder of glycosylation', 'triglyceride measurement', 'handedness', 'telomere length', 'body height'], ['hair shape measurement', 'puberty onset measurement', 'cancer', 'Alopecia', 'age at first facial hair'], ['platelet count', 'hematocrit', 'reticulocyte measurement', 'systolic blood pressure', 'sex hormone-binding globulin measurement'], ['adrenal suppression measurement', 'response to corticosteroid', 'acute myeloid leukemia', 'visual cortical surface area measurement', 'brain volume measurement'], ['Gastrointestinal stromal tumor', 'cancer', 'GIST-plus syndrome', 'acute myeloid leukemia', 'neoplasm'], ['Infantile myofibromatosis', 'Bilateral striopallidodentate calcinosis', 'Acroosteolysis-keloid-like lesions-premature aging syndrome', 'skeletal overgrowth-craniofacial dysmorphism-hyperelastic skin-white matter lesions syndrome', 'hepatocellular carcinoma'], ['Pyruvate dehydrogenase E3-binding protein deficiency', 'Pyruvate dehydrogenase deficiency', 'Leigh syndrome', 'Pyruvate dehydrogenase E1-alpha deficiency', 'protein measurement'], ['mean corpuscular volume', 'body fat percentage', 'mean corpuscular hemoglobin concentration', 'Bicuspid aortic valve', 'body height'], ["Crohn's disease", '[Pyruvate dehydrogenase acetyl-transferring] kinase isozyme 1, mitochondrial measurement', 'lactic acidosis', 'Pyruvate dehydrogenase deficiency', 'eosinophil count'], ['dilated cardiomyopathy', 'hypertrophic cardiomyopathy', 'Abnormality of the cardiovascular system', 'type II diabetes mellitus', 'Rare familial disorder with hypertrophic cardiomyopathy'], ['breast adenocarcinoma', 'cognitive function measurement', 'forced expiratory volume', 'self reported educational attainment', 'Apolipoprotein A-I deficiency'], ['Pyruvate dehydrogenase phosphatase deficiency', 'Pyruvate dehydrogenase deficiency', 'neoplasm', 'cancer', 'infection'], ['ovarian carcinoma', 'cancer', 'colorectal adenocarcinoma', 'breast carcinoma', 'ovarian cancer'], ['Deafness - encephaloneuropathy - obesity - valvulopathy', 'Coenzyme Q10 deficiency', 'hepatocellular adenoma', 'adolescent idiopathic scoliosis', 'Osteosclerosis - ichthyosis - premature ovarian failure'], ['Partial pancreatic agenesis', 'MODY', 'Permanent neonatal diabetes mellitus', 'maturity-onset diabetes of the young type 4', 'fasting blood glucose measurement'], ['mean corpuscular volume', 'neoplasm', 'multiple sclerosis', 'cancer', 'glioblastoma multiforme'], ['Spinocerebellar ataxia type 23', 'aortic root size', 'mean platelet volume', 'platelet crit', 'glioblastoma multiforme'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'red blood cell distribution width', 'cutaneous melanoma', 'Red cell distribution width'], ['coronary artery disease', 'systolic blood pressure', 'pulse pressure measurement', 'mean platelet volume', 'neoplasm'], ['schizophrenia', 'breast carcinoma', 'hepatocellular carcinoma', 'insomnia measurement', 'diffuse large B-cell lymphoma'], ['Prolidase deficiency', 'body fat percentage', 'body mass index', 'fat body mass', 'lean body mass'], ['Familial advanced sleep-phase syndrome', 'chronotype measurement', 'Sleep Disorder', 'neoplasm', 'glioma'], ['Neonatal adrenoleukodystrophy', 'Zellweger syndrome', 'Peroxisome biogenesis disorder-Zellweger syndrome spectrum', 'Infantile Refsum disease', 'ulcerative colitis'], ['serum metabolite measurement', 'very low density lipoprotein cholesterol measurement', 'breast adenocarcinoma', 'coronary artery calcification', 'squamous cell lung carcinoma'], ['endometrial cancer', 'reaction time measurement', 'mathematical ability', 'worry measurement', 'intelligence'], ['glioblastoma multiforme', 'gastric adenocarcinoma', 'colorectal cancer', 'atherosclerosis', 'atopic eczema'], ['mean corpuscular volume', 'red blood cell distribution width', 'BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'granulocyte count'], ['gastric carcinoma', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'colorectal carcinoma', 'oral cavity carcinoma'], ['age-related macular degeneration', 'diabetic macular edema', 'wet macular degeneration', 'retinal vein occlusion', 'macular retinal edema'], ['blood protein measurement', 'ovarian cancer', 'lung adenocarcinoma', 'endometrial cancer', 'squamous cell lung carcinoma'], ['ovarian cancer', 'lung adenocarcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'non-small cell lung carcinoma'], ['lean body mass', 'fat body mass', 'body weight', 'body height', 'appendicular lean mass'], ['fat body mass', 'body mass index', 'body fat percentage', 'smoking status measurement', 'chronotype measurement'], ['polycystic ovary syndrome', 'infertility', 'endometriosis', 'uterine fibroid', 'acne'], ['primary ovarian insufficiency', 'neoplasm', 'cancer', 'renal cell carcinoma', 'renal cell adenocarcinoma'], ['endometrial cancer', 'hematocrit', 'susceptibility to breast cancer', 'cutaneous melanoma', 'systolic blood pressure'], ['lean body mass', 'fat body mass', 'appendicular lean mass', 'body weight', 'ovarian cancer'], ['X-linked hypophosphatemia', 'Hypophosphatemic rickets', 'Lower limb pain', 'Bowing of the legs', 'Dent disease'], ['hepatocellular carcinoma', 'self reported educational attainment', 'refractive error measurement', 'chronotype measurement', 'mathematical ability'], ['3-phosphoglycerate dehydrogenase deficiency, infantile/juvenile form', 'Neu-Laxova syndrome', 'Neurometabolic disorder due to serine deficiency', 'blood metabolite measurement', 'glycine measurement'], ['appendicular lean mass', 'type II diabetes mellitus', 'low density lipoprotein cholesterol measurement', 'osteosarcoma', 'bilirubin measurement'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'esophageal cancer', 'squamous cell lung carcinoma'], ['endometrial cancer', 'glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'pancreatic adenocarcinoma'], ['Dehydrated hereditary stomatocytosis', 'lymphedema, hereditary, iii', 'genetic disorder', 'hydrops fetalis (disease)', 'Thickened nuchal skin fold'], ['Multiple congenital anomalies-hypotonia-seizures syndrome type 2', 'Paroxysmal nocturnal hemoglobinuria', 'Ferro-cerebro-cutaneous syndrome', 'West syndrome', 'Malignant migrating partial seizures of infancy'], ['short stature with microcephaly and distinctive facies', 'brain volume measurement', 'brain measurement', 'neuroimaging measurement', 'gait measurement'], ['blood protein measurement', 'neoplasm', 'Cystic fibrosis', 'lymphocyte count', 'pancreatic carcinoma'], ['glycosylphosphatidylinositol biosynthesis defect 18', 'insomnia measurement', 'glioma', 'platelet crit', 'urate measurement'], ['Short stature due to GHSR deficiency', 'platelet count', 'glioblastoma multiforme', 'breast adenocarcinoma', 'body height'], ['severe acute respiratory syndrome', 'COVID-19', 'body mass index', 'body fat percentage', 'breast adenocarcinoma'], ['Megalencephaly-capillary malformation-polymicrogyria syndrome', 'CLOVE syndrome', 'colorectal adenocarcinoma', 'seborrheic keratosis', 'CLAPO syndrome'], ['cancer', 'prostate adenocarcinoma', 'esophageal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'kidney neoplasm'], ['Activated PIK3-delta syndrome', 'chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['chronic lymphocytic leukemia', 'neoplasm', 'neoplasm of mature B-cells', 'hypertension', 'systolic blood pressure'], ['SHORT syndrome', 'Autosomal agammaglobulinemia', 'Activated PIK3-delta syndrome', 'cancer', 'Agammaglobulinemia'], ['Spinocerebellar ataxia with axonal neuropathy type 2', 'breast cancer', 'monocyte percentage of leukocytes', 'autoimmune thyroid disease', 'hypothyroidism'], ['Fleck corneal dystrophy', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'lung adenocarcinoma'], ["Alzheimer's disease", 'family history of Alzheimer’s disease', 'red blood cell distribution width', 'heel bone mineral density', 'electrocardiography'], ['heel bone mineral density', 'electrocardiography', 'platelet crit', 'platelet count', 'monocyte count'], ['Central Nervous System Lymphoma', 'diffuse large B-cell lymphoma', 'lymphoma', 'lymphoid neoplasm', 'multiple myeloma'], ['type II diabetes mellitus', 'birth weight', 'peak expiratory flow', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement'], ['thyroxine measurement', 'hormone measurement', 'stressful life event measurement', 'C-reactive protein measurement', 'Cone rod dystrophy'], ['neoplasm', "Alzheimer's disease", 'Peeling skin syndrome', 'systemic scleroderma', 'sebaceous of Jadassohn nevus'], ['Lethal congenital contracture syndrome type 3', 'glioblastoma multiforme', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma'], ['Liberfarb syndrome', 'hearing loss', 'glioblastoma multiforme', 'osteochondrodysplasia', 'microcephaly (disease)'], ['breast adenocarcinoma', 'retinal degeneration', 'Autosomal recessive hypophosphatemic rickets', 'Abnormality of refraction', 'Cholesterol-ester transfer protein deficiency'], ['Axenfeld-Rieger syndrome', 'ring dermoid of cornea', 'Rieger anomaly', 'Familial ocular anterior segment mesenchymal dysgenesis', 'Peters anomaly'], ['Familial ocular anterior segment mesenchymal dysgenesis', 'Posterior polar cataract', 'Non-syndromic congenital cataract', 'cataract', 'Cataract-glaucoma'], ['Autosomal dominant polycystic kidney disease', 'polycystic kidney disease', 'Autosomal recessive polycystic kidney disease', 'kidney disease', 'Meckel syndrome'], ['Autosomal dominant polycystic kidney disease', 'polycystic kidney disease 2', 'polycystic kidney disease', 'kidney disease', 'Meckel syndrome'], ['mean platelet volume', 'blood metabolite measurement', 'heel bone mineral density', 'red blood cell distribution width', 'serum metabolite measurement'], ['Autosomal recessive polycystic kidney disease', 'polycystic kidney disease', 'Meckel syndrome', 'lean body mass', 'Renal cyst'], ['gastric carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'erythrocyte count', 'forced expiratory volume'], ['urolithiasis', 'endometrial cancer', 'prostate cancer', 'Blackfan-Diamond anemia', 'Activated PIK3-delta syndrome'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'infection', 'infect', 'esophageal adenocarcinoma'], ['rheumatoid arthritis', 'cutaneous melanoma', 'systolic blood pressure', 'diastolic blood pressure', 'appendicular lean mass'], ['colorectal cancer', 'colonic neoplasm', 'protein measurement', 'phospholipase A2, membrane associated measurement', 'blood protein measurement'], ['Infantile neuroaxonal dystrophy', 'Adult-onset dystonia-parkinsonism', 'PLA2G6-associated neurodegeneration', 'Iron accumulation in brain', "Parkinson's disease"], ['acute coronary syndrome', 'Platelet-activating factor acetylhydrolase deficiency', 'asthma', 'atopic eczema', 'inherited susceptibility to asthma'], ['age-related macular degeneration', 'complement factor I measurement', 'obesity', 'protein measurement', 'low density lipoprotein cholesterol measurement'], ['chronic kidney disease', 'hypertension', 'glomerular filtration rate', "Alzheimer's disease", 'chronic obstructive pulmonary disease'], ['visual perception measurement', 'neoplasm', 'monocyte percentage of leukocytes', 'monocyte count', 'non-small cell lung carcinoma'], ['Silver-Russell syndrome', 'silver-russell syndrome 4', 'body height', 'vital capacity', 'lean body mass'], ['hemorrhage', 'osteoarthritis', 'scoliosis', 'skin pigmentation', 'hair color'], ['Quebec platelet disorder', 'blood protein measurement', "Crohn's disease", 'inflammatory bowel disease', 'psoriasis'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'eosinophil percentage of granulocytes', 'neutrophil percentage of granulocytes', 'sum of eosinophil and basophil counts'], ['Familial multiple lipomatosis', 'body mass index', 'lean body mass', 'cerebrospinal fluid biomarker measurement', 'smoking behavior'], ['Autosomal dominant cerebellar ataxia type 1', 'breast adenocarcinoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', "Alzheimer's disease"], ['Epidermolysis bullosa simplex with muscular dystrophy', 'Epidermolysis bullosa simplex with pyloric atresia', 'Epidermolysis bullosa simplex, Ogna type', 'Autosomal recessive limb-girdle muscular dystrophy type 2Q', 'Epidermolysis bullosa simplex'], ['cancer', 'breast cancer', 'protein measurement', 'diabetes mellitus', 'body mass index'], ['Hypoplasminogenemia', 'Ligneous conjunctivitis', 'myocardial infarction', 'coronary artery disease', 'stroke'], ['hepatocellular carcinoma', 'insulin resistance', 'optic cup area measurement', 'Hyperlipoproteinemia type 4', 'coronary artery disease, autosomal dominant 2'], ['acute myeloid leukemia', 'mean corpuscular volume', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['hypertrophic cardiomyopathy', 'Familial isolated dilated cardiomyopathy', 'Familial dilated cardiomyopathy', 'dilated cardiomyopathy', 'cardiomyopathy'], ['Spastic paraplegia type 2', 'Pelizaeus-Merzbacher disease', 'Pelizaeus-Merzbacher disease, connatal form', 'Hereditary spastic paraplegia', 'genetic disorder'], ['platelet count', 'nervous system disease', 'coronary artery disease', 'mean corpuscular volume', 'mean corpuscular hemoglobin'], ['serum metabolite measurement', 'high density lipoprotein cholesterol measurement', 'lipid measurement', 'triglyceride measurement', 'alcohol drinking'], ['Congenital chronic diarrhea with protein-losing enteropathy', 'neoplasm', 'hepatocellular carcinoma', 'low density lipoprotein cholesterol measurement', "Hb Bart's hydrops fetalis"], ['platelet count', 'Neurodevelopmental disorder', 'blood protein measurement', 'skin aging measurement', 'skin aging'], ['adolescent idiopathic scoliosis', 'risk-taking behaviour', 'brain volume measurement', 'insulin like growth factor measurement', 'protein measurement'], ['Short stature due to GHSR deficiency', 'Neurodevelopmental disorder', 'autism', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['thyroid gland adenocarcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'Mental deterioration', 'neuroblastoma'], ['head and neck squamous cell carcinoma', 'sitting height measurement', 'body height', 'appendicular lean mass', 'forced expiratory volume'], ['lean body mass', 'blood protein measurement', 'neoplasm', 'Griscelli disease type 3', 'lung cancer'], ['viral disease', 'acute myeloid leukemia', 'basal cell carcinoma', 'leukocyte count', 'waist-hip ratio'], ['Charcot-Marie-Tooth disease type 1G', 'peripheral neuropathy', 'genetic disorder', 'Autosomal dominant slowed nerve conduction velocity', 'Hereditary neuropathy with liability to pressure palsies'], ['Dejerine-Sottas syndrome', 'Charcot-Marie-Tooth disease type 1A', 'Hereditary neuropathy with liability to pressure palsies', 'Charcot-Marie-Tooth disease type 1E', 'Roussy-Lévy syndrome'], ['hepatocellular carcinoma', 'colorectal adenocarcinoma', 'esophageal squamous cell carcinoma', 'diffuse large B-cell lymphoma', 'prostate carcinoma'], ['Paroxysmal non-kinesigenic dyskinesia', 'Paroxysmal dystonia', 'Episodic hemiplegia', 'Generalized epilepsy - paroxysmal dyskinesia', 'lean body mass'], ['Early infantile epileptic encephalopathy', 'Ataxia-oculomotor apraxia type 4', 'Microcephaly - seizures - developmental delay', 'Charcot-Marie-Tooth disease type 2B2', 'genetic disorder'], ['obesity', 'Pancreatic triacylglycerol lipase deficiency', 'type II diabetes mellitus', 'Hepatic steatosis', 'Pancreatic colipase deficiency'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'chronic pancreatitis'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'esophageal squamous cell carcinoma'], ['neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'body height', 'cardiovascular disease'], ['breast carcinoma', 'colorectal carcinoma', 'self reported educational attainment', 'ovarian cancer', 'blood pressure'], ['daytime rest measurement', 'body mass index', 'optic cup area measurement', 'type II diabetes mellitus', 'thyroid gland adenocarcinoma'], ['Purine nucleoside phosphorylase deficiency', 'Combined T and B cell immunodeficiency', 'Anemia, Hemolytic, Autoimmune', 'age at menopause', 'protein measurement'], ['non-alcoholic fatty liver disease', 'cirrhosis of liver', 'serum alanine aminotransferase measurement', 'platelet count', 'type II diabetes mellitus'], ['Young adult-onset Parkinsonism', 'familial nephrotic syndrome', 'Atypical juvenile parkinsonism', 'systolic blood pressure', 'pulse pressure measurement'], ['Mandibular hypoplasia-deafness-progeroid syndrome', 'colorectal cancer', 'acute myeloid leukemia', 'non-small cell lung carcinoma', 'chronic lymphocytic leukemia'], ['acute myeloid leukemia', 'acute lymphoblastic leukemia', 'non-small cell lung carcinoma', 'pancreatic carcinoma', 'chronic lymphocytic leukemia'], ['neoplasm', 'infection', 'infect', 'breast cancer', 'non-small cell lung carcinoma'], ['endometrial cancer', 'venous thromboembolism', 'infect', 'infection', 'Hoyeraal-Hreidarsson syndrome'], ['Alpers syndrome', 'Autosomal recessive progressive external ophthalmoplegia', 'Autosomal dominant progressive external ophthalmoplegia', 'Sensory ataxic neuropathy - dysarthria - ophthalmoparesis', 'Spinocerebellar ataxia with epilepsy'], ['Hypomyelination - hypogonadotropic hypogonadism - hypodontia', 'Hypomyelinating leukodystrophy with or without oligondontia and/or hypogonadism', 'Autosomal recessive non-syndromic intellectual disability', 'Endosteal sclerosis - cerebellar hypoplasia', 'Isolated congenital hypogonadotropic hypogonadism'], ['endometrial cancer', 'Primary hypomagnesemia with secondary hypocalcemia', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Pseudohypoparathyroidism type 2', 'Autosomal dominant hypocalcemia'], ['body height', 'colorectal carcinoma', 'bacteriemia', 'neoplasm', 'colorectal cancer'], ['Autosomal recessive hypophosphatemic rickets', 'Melorheostosis', 'postmenopausal osteoporosis', 'Pyknoachondrogenesis', 'Gnathodiaphyseal dysplasia'], ['46,XX gonadal dysgenesis', 'medulloblastoma', 'alcohol consumption measurement', 'asthma', 'diet measurement'], ['erythrocyte count', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'Leukodystrophy'], ['Obesity due to pro-opiomelanocortin deficiency', 'obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'Genetic obesity', 'adrenal gland disease'], ['amyotrophic lateral sclerosis', 'aspartate aminotransferase, cytoplasmic measurement', 'arylesterase enzyme measurement', 'leucine carboxyl methyltransferase 1 measurement', 'metabolite measurement'], ['amyotrophic lateral sclerosis', 'acute myeloid leukemia', 'neoplasm', 'glioblastoma multiforme', 'systemic lupus erythematosus'], ['amyotrophic lateral sclerosis', 'high density lipoprotein cholesterol measurement', 'head and neck squamous cell carcinoma', 'hepatocellular carcinoma', 'neoplasm'], ['erythrocyte count', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'hemoglobin measurement'], ['Congenital adrenal hyperplasia due to cytochrome P450 oxidoreductase deficiency', 'Congenital adrenal hyperplasia', 'Antley-Bixler syndrome', 'coffee consumption measurement', 'platelet count'], ['Focal dermal hypoplasia', 'cancer', 'polydactyly (disease)', 'Global developmental delay', 'endometrial cancer'], ['blood protein measurement', 'smoking status measurement', 'cerebrospinal fluid biomarker measurement', 'Varicose veins', 'varicose disease'], ['eosinophil count', 'lung carcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'cutaneous melanoma', 'lymphoid neoplasm'], ['non-Hodgkins lymphoma', 'eosinophil count', 'chronotype measurement', 'forced expiratory volume', 'sitting height measurement'], ['gastric carcinoma', 'myoepithelial tumor', 'lung carcinoma', 'cutaneous melanoma', 'basal cell carcinoma'], ['inorganic pyrophosphatase measurement', 'erythrocyte count', 'neoplasm', 'non-small cell lung carcinoma', 'melanoma'], ['narcolepsy with cataplexy', 'heel bone mineral density', 'lean body mass', 'fat body mass', 'non-small cell lung carcinoma'], ['coronary artery disease', 'cardiovascular disease', 'Disorder of lipid metabolism', 'Hypertriglyceridemia', 'type II diabetes mellitus'], ['type II diabetes mellitus', 'Familial partial lipodystrophy associated with PPARG mutations', 'diabetes mellitus', 'ulcerative colitis', 'colorectal adenocarcinoma'], ['electrocardiography', 'hematocrit', 'hemoglobin measurement', 'sex hormone-binding globulin measurement', 'erythrocyte count'], ['systemic lupus erythematosus', 'sunburn', 'cutaneous melanoma', 'facial pigmentation measurement', 'nevus count'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neuroblastoma'], ['blood metabolite measurement', 'acylcarnitine measurement', 'metabolite measurement', 'serum metabolite measurement', 'peptidyl-prolyl cis-trans isomerase D measurement'], ['lean body mass', 'reticulocyte count', 'body height', 'reticulocyte measurement', 'appendicular lean mass'], ['Neurodevelopmental disorder', 'neuroblastoma', 'ovarian carcinoma', 'Ewing sarcoma', 'osteosarcoma'], ['hepatocellular carcinoma', 'red blood cell distribution width', 'neoplasm', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia'], ['lymphocyte count', 'response to antidepressant', 'refractive error measurement', 'brain stem volume measurement', 'peak expiratory flow'], ['appendicular lean mass', 'body height', 'lean body mass', 'fat body mass', 'forced expiratory volume'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness', 'type II diabetes mellitus', 'body height', 'balding measurement'], ['Intermediate maple syrup urine disease', 'Maple syrup urine disease', 'blood metabolite measurement', 'amino acid measurement', 'serum metabolite measurement'], ['Pentosuria', 'Isolated polycystic liver disease', 'Myopathy due to calsequestrin and SERCA1 protein overload', 'Metabolic myopathy due to lactate transporter defect', 'appendicular lean mass'], ['Porphyria variegata', 'Abnormal blistering of the skin', 'migraine disorder', 'Abnormal urinary color', 'Constipation'], ['rosette-forming glioneuronal tumor of fourth ventricule', 'Ewing sarcoma', 'Eisenmenger syndrome', 'epithelioid sarcoma', 'hepatocellular carcinoma'], ['erythrocyte count', 'hematocrit', 'self reported educational attainment', 'neoplasm', 'schizophrenia'], ["Alzheimer's disease", 'obesity', 'infect', 'infection', 'neoplasm'], ['type II diabetes mellitus', 'body mass index', 'schizophrenia', 'grip strength measurement', 'gait measurement'], ['body mass index', 'insomnia measurement', 'chronic obstructive pulmonary disease', 'cancer', 'influenza A (H1N1)'], ['ovarian cancer', 'breast adenocarcinoma', 'sex ratio', 'acne', 'platelet count'], ['glioblastoma multiforme', 'breast cancer', 'cancer', 'hepatocellular carcinoma', 'neoplasm'], ['schizophrenia', 'attention deficit hyperactivity disorder', 'alcohol use disorder measurement', 'gastric adenocarcinoma', 'low grade glioma'], ['neurodevelopmental disorder and language delay with or without structural brain abnormalities', 'cancer', 'Global developmental delay', 'Feeding difficulties', 'language impairment'], ['lymphocyte count', 'hemoglobin measurement', 'eosinophil count', 'eosinophil percentage of leukocytes', "Crohn's disease"], ['CLN1 disease', 'Infantile neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Adult neuronal ceroid lipofuscinosis', 'Late infantile neuronal ceroid lipofuscinosis'], ['head and neck squamous cell carcinoma', 'blood protein measurement', 'triglyceride measurement', 'cardiovascular disease', 'body height'], ['age at onset', 'Myopia', 'neoplasm', 'acute myeloid leukemia', 'Uveal melanoma'], ['neoplasm', 'arthritis', 'gastric cancer', 'cancer', 'rheumatoid arthritis'], ['systemic scleroderma', 'psoriasis', 'myositis', 'polymyositis', 'Pyomyositis'], ['renal carcinoma', 'breast carcinoma', 'hepatocellular carcinoma', 'Renal Cell Carcinoma Associated with Xp11.2 Translocations/TFE3 Gene Fusions', 'papillary renal cell carcinoma'], ['blood protein measurement', 'adolescent idiopathic scoliosis', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cutaneous melanoma'], ['multiple myeloma', 'diffuse large B-cell lymphoma', 'ulcerative colitis', 'colorectal adenocarcinoma', 'melanoma'], ['HIV-1 infection', 'viral load', 'protein measurement', 'breast carcinoma', 'alcohol drinking'], ["Alzheimer's disease", 'lung adenocarcinoma', 'endometrial cancer', 'urinary bladder cancer', 'scrapie'], ['breast adenocarcinoma', 'urinary bladder cancer', 'multiple sclerosis', 'type II diabetes mellitus', 'body fat percentage'], ['electrocardiography', 'causalgia', 'T wave morphology measurement', 'diet measurement', 'blood protein measurement'], ['Familial hemophagocytic lymphohistiocytosis', 'adult lymphoma', 'Aplastic anemia', 'colorectal adenocarcinoma', 'cutaneous melanoma'], ['pappalysin‐1 measurement', 'eosinophil count', 'mean corpuscular hemoglobin concentration', 'sum of eosinophil and basophil counts', 'mean corpuscular volume'], ['Camptodactyly-arthropathy-coxa-vara-pericarditis syndrome', 'camptodactyly-arthropathy-coxa vara-pericarditis syndrome', 'genetic disorder', 'FEV/FEC ratio', 'Arthrogryposis syndrome'], ['sensory perception of bitter taste', 'neoplasm', 'coffee consumption', 'coffee consumption measurement', 'glioblastoma multiforme'], ['neoplasm', 'breast cancer', 'endometrial cancer', 'cancer', 'leiomyoma'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer'], ['sex hormone-binding globulin measurement', 'protein measurement', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['neutrophil count', 'basophil count', 'lung adenocarcinoma', 'breast adenocarcinoma', 'leukocyte count'], ['Acrodysostosis', 'Carney complex', 'Acrodysostosis with multiple hormone resistance', 'Primary pigmented nodular adrenocortical disease', 'Carney complex, type 1'], ['acute myeloid leukemia', 'neoplasm', 'mast-cell leukemia', 'ulcerative colitis', 'inflammatory bowel disease'], ['Spinocerebellar ataxia type 14', 'acute myeloid leukemia', 'Spinocerebellar ataxia type 15/16', 'neoplasm', 'mast-cell leukemia'], ['congenital heart defects and ectodermal dysplasia', 'acute myeloid leukemia', 'self reported educational attainment', 'body mass index', 'neoplasm'], ['multiple myeloma', 'heel bone mineral density', 'type I diabetes mellitus', 'aspartate aminotransferase measurement', 'sclerosing cholangitis'], ['Severe combined immunodeficiency due to DNA-PKcs deficiency', 'Combined T and B cell immunodeficiency', 'mean corpuscular volume', 'Cernunnos-XLF deficiency', 'mean corpuscular hemoglobin concentration'], ['Familial thoracic aortic aneurysm and aortic dissection', 'body height', 'lean body mass', 'Abnormality of the cardiovascular system', 'Rare genetic vascular disease'], ['Young adult-onset Parkinsonism', "Parkinson's disease", 'late-onset Parkinson disease', 'dystonic disorder', 'body mass index'], ['AL amyloidosis', 'neutrophil count', 'leukocyte count', 'basophil count', 'myeloid white cell count'], ['Familial hyperprolactinemia', 'hyperprolactinemia', 'breast carcinoma', 'appendicular lean mass', 'testosterone measurement'], ['breast adenocarcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'ovarian cancer', 'body height'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'low grade glioma', 'urinary bladder cancer'], ['sex hormone-binding globulin measurement', 'red blood cell distribution width', 'lean body mass', 'body fat percentage', 'testosterone measurement'], ['platelet crit', 'Autosomal recessive non-syndromic intellectual disability', 'hepatocellular carcinoma', 'neoplasm', 'hemoglobin measurement'], ['Gerstmann-Straussler-Scheinker syndrome', 'Huntington disease-like 1', 'Fatal familial insomnia', 'Inherited Creutzfeldt-Jakob disease', 'prion disease'], ['Hereditary thrombophilia due to congenital protein C deficiency', 'protein c deficiency', 'genetic disorder', 'Abnormal thrombosis', 'deep vein thrombosis'], ['protein C measurement', 'factor VII measurement', 'venous thromboembolism', 'prothrombin time measurement', 'coronary artery disease'], ['Hyperprolinemia type 1', 'schizophrenia', 'amino acid measurement', 'proline measurement', 'hyperprolinemia'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'blood protein measurement', 'serum metabolite measurement', 'breast adenocarcinoma'], ['Hirschsprung disease', 'ovarian cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'medulloblastoma'], ['Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'Septo-optic dysplasia', 'genetic disorder', 'Isolated congenital hypogonadotropic hypogonadism'], ['Retinitis pigmentosa', 'Cone rod dystrophy', 'Stargardt disease', 'Retinal macular dystrophy type 2', 'Macular dystrophy'], ['Hereditary thrombophilia due to congenital protein S deficiency', 'thrombophilia due to protein S deficiency, autosomal dominant', 'protein S deficiency', 'Reduced protein S activity', 'Abnormal bleeding'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Varicose veins', 'tonsillectomy risk measurement', 'lean body mass'], ['Retinitis pigmentosa', 'Butterfly-shaped pigment dystrophy', 'Adult-onset foveomacular vitelliform dystrophy', 'patterned macular dystrophy 1', 'Central areolar choroidal dystrophy'], ['Lethal ataxia with deafness and optic atrophy', 'Phosphoribosylpyrophosphate synthetase superactivity', 'X-linked Charcot-Marie-Tooth disease type 5', 'X-linked non-syndromic sensorineural deafness type DFN', 'Retinal dystrophy'], ['Infantile convulsions and choreoathetosis', 'Paroxysmal kinesigenic dyskinesia', 'Benign familial neonatal-infantile seizures', 'Autosomal recessive non-syndromic intellectual disability', 'Benign familial infantile epilepsy'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'neoplasm', 'ovarian cancer'], ['esophageal squamous cell carcinoma', 'Autosomal dominant severe congenital neutropenia', 'X-linked severe congenital neutropenia', 'Pentosuria', 'Blackfan-Diamond anemia'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts'], ['anxiety', 'white matter microstructure measurement', 'worry measurement', 'schizophrenia', 'systolic blood pressure'], ['blood protein measurement', 'Neutrophil immunodeficiency syndrome', 'Recurrent infection due to specific granule deficiency', 'neoplasm', 'gastroenteritis'], ['basophil count', 'age at onset', 'Myopia', 'basophil percentage of leukocytes', 'monocyte percentage of leukocytes'], ['Neu-Laxova syndrome', 'Phosphoserine aminotransferase deficiency', 'Neurometabolic disorder due to serine deficiency', 'adolescent idiopathic scoliosis', 'mean corpuscular volume'], ['gastric carcinoma', 'gastric cancer', 'duodenal ulcer', 'bladder carcinoma', 'urinary bladder cancer'], ['self reported educational attainment', 'drug use measurement', 'smoking status measurement', 'triglyceride measurement', 'anxiety'], ['Early-onset autosomal dominant Alzheimer disease', 'Frontotemporal dementia', 'Pick disease', 'dilated cardiomyopathy', 'Familial isolated dilated cardiomyopathy'], ['hidradenitis suppurativa', 'acne inversa, familial, 1', 'Dowling-Degos disease', "Alzheimer's disease", 'Dementia'], ['blood protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'visual perception measurement', 'head and neck squamous cell carcinoma'], ['breast adenocarcinoma', 'multiple myeloma', 'head and neck squamous cell carcinoma', 'low grade glioma', 'neoplasm'], ['HIV infection', 'bladder carcinoma', 'colorectal adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'breast ductal adenocarcinoma'], ['HIV infection', 'glioblastoma multiforme', 'diastolic blood pressure', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma'], ['HIV infection', 'glioblastoma multiforme', 'endometrial cancer', 'lung adenocarcinoma', 'gastric adenocarcinoma'], ['proteasome-associated autoinflammatory syndrome 3', 'HIV infection', 'CANDLE syndrome', 'JMP syndrome', 'Proteasome disability syndrome'], ['CANDLE syndrome', 'Proteasome disability syndrome', 'Nakajo-Nishimura syndrome', 'JMP syndrome', 'HIV infection'], ['proteasome-associated autoinflammatory syndrome 3', 'HIV infection', 'glioblastoma multiforme', 'urinary bladder cancer', 'gastric adenocarcinoma'], ['HIV infection', 'ovarian cancer', 'chronic lymphocytic leukemia', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['HIV infection', 'glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'esophageal cancer'], ['HIV infection', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer'], ['HIV infection', 'ovarian cancer', 'endometrial cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['ulcerative colitis', 'inflammatory bowel disease', 'neutrophil count', 'hemoglobin measurement', 'ankylosing spondylitis'], ['synovial sarcoma', 'Wolfram syndrome', 'Classical progressive supranuclear palsy', 'Progressive supranuclear palsy', 'Sepsis'], ['ovarian carcinoma', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'low density lipoprotein cholesterol measurement', 'apolipoprotein B measurement'], ['diet measurement', 'gastric cancer', 'neoplasm', 'red blood cell density measurement', 'non-small cell lung carcinoma'], ['Gorlin syndrome', 'Holoprosencephaly', 'basal cell carcinoma', 'medulloblastoma', 'Inherited cancer-predisposing syndrome'], ['mediator of RNA polymerase II transcription subunit 1 measurement', 'smoking status measurement', 'Okt4 epitope deficiency', 'Alymphoid cystic thymic dysgenesis', 'Combined immunodeficiency with skin granulomas'], ['Lenz-Majewski hyperostotic dwarfism', 'type II diabetes mellitus', 'peripheral arterial disease', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['breast adenocarcinoma', 'Immature Reticulocyte Fraction Measurement', 'Normosmic congenital hypogonadotropic hypogonadism', 'Testicular regression syndrome', 'Hypergonadotropic hypogonadism - cataract syndrome'], ['Cowden syndrome', 'Lhermitte-Duclos disease', 'PTEN hamartoma tumor syndrome', 'Proteus-like syndrome', 'Macrocephaly-autism syndrome'], ['Permanent neonatal diabetes mellitus - pancreatic and cerebellar agenesis', 'Partial pancreatic agenesis', 'Permanent neonatal diabetes mellitus', 'pancreatic adenocarcinoma', 'head and neck squamous cell carcinoma'], ['asthma', 'seasonal allergic rhinitis', 'Nasal Cavity Polyp', 'prostate adenocarcinoma', 'allergic rhinitis'], ['blood protein measurement', 'central corneal thickness', 'intraocular pressure measurement', 'renal system measurement', 'Keratoconus'], ['erectile dysfunction', 'Premature rupture of membranes', 'myocardial infarction', 'pregnancy', 'pain'], ['erectile dysfunction', 'myocardial infarction', 'Premature rupture of membranes', 'pregnancy', 'pain'], ['hemorrhage', 'osteoarthritis', 'postpartum hemorrhage', 'pregnancy', 'Premature rupture of membranes'], ['Premature rupture of membranes', 'pain', "Crohn's disease", 'pregnancy', 'red blood cell density measurement'], ['asthma', 'neoplasm', 'hepatocellular carcinoma', 'experimental autoimmune encephalomyelitis', 'neuroblastoma'], ['glaucoma', 'open-angle glaucoma', 'ocular hypertension', 'hypotrichosis', 'Vitiligo'], ['pulmonary arterial hypertension', 'pulmonary hypertension', 'Recurrent thrombophlebitis', 'cardiovascular disease', 'heart failure'], ['rheumatoid arthritis', 'ulcerative colitis', 'migraine disorder', 'osteoarthritis', 'coronary artery disease'], ['rheumatoid arthritis', 'migraine disorder', 'osteoarthritis', 'ulcerative colitis', 'coronary artery disease'], ['hypoparathyroidism, familial isolated 1', 'Familial isolated hypoparathyroidism due to impaired PTH secretion', 'Familial isolated hypoparathyroidism', 'primary hyperparathyroidism', 'chronic kidney disease'], ['Brachydactyly', 'Brachydactyly type E', 'body height', 'breast carcinoma', 'hair shape measurement'], ['appendicular lean mass', 'brain volume measurement', 'mean corpuscular hemoglobin', 'atrial fibrillation', 'mathematical ability'], ["Alzheimer's disease", 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'monocyte count', 'glioblastoma multiforme'], ['BMI-adjusted waist-hip ratio', 'age at menarche', 'waist-hip ratio', 'neoplasm', 'colorectal carcinoma'], ['eosinophil count', 'appendicular lean mass', 'lymphocyte count', 'monocyte count', 'eosinophil percentage of leukocytes'], ['rheumatoid arthritis', 'eosinophil count', "Crohn's disease", 'hemoglobin measurement', 'hematocrit'], ['cutaneous melanoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'head and neck squamous cell carcinoma'], ['Noonan syndrome', 'LEOPARD syndrome', 'Juvenile Myelomonocytic Leukemia', 'Metachondromatosis', 'acute myeloid leukemia'], ['T-B+ severe combined immunodeficiency due to CD45 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'Omenn syndrome', 'lymphocyte count', 'mean corpuscular hemoglobin'], ['blood protein measurement', 'appendicular lean mass', 'Short stature due to GHSR deficiency', 'reticulocyte count', 'mean corpuscular volume'], ['non-syndromic intellectual disability', 'self reported educational attainment', 'mathematical ability', 'protein measurement', 'prostate carcinoma'], ['Global developmental delay', 'Aceruloplasminemia', 'hearing loss', 'breast adenocarcinoma', 'esophageal cancer'], ['6-pyruvoyl-tetrahydropterin synthase deficiency', 'Hyperphenylalaninemia', 'Intellectual disability', 'dystonic disorder', 'heel bone mineral density'], ['8q24.3 microdeletion syndrome', 'genetic disorder', 'Intellectual disability', 'syndromic intellectual disability', 'CHARGE syndrome'], ['mean platelet volume', 'platelet count', 'platelet component distribution width', 'lymphocyte count', 'blood protein measurement'], ['mean corpuscular hemoglobin', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'body height'], ['head and neck squamous cell carcinoma', 'neoplasm', 'prostate cancer', "Alzheimer's disease", 'melanoma'], ['Autosomal recessive cutis laxa type 2B', 'De Barsy syndrome', 'Autosomal recessive cutis laxa type 2', 'PYCR1-related De Barsy syndrome', 'genetic disorder'], ['DNA methylation', 'ovarian cancer', 'blood protein measurement', 'apolipoprotein B measurement', 'pulse pressure measurement'], ['schizophrenia', 'chronic lymphocytic leukemia', 'body height', 'blood protein measurement', 'lymphocyte percentage of leukocytes'], ['blood protein measurement', 'pulse pressure measurement', 'breast adenocarcinoma', 'red blood cell density measurement', 'neoplasm'], ['body height', 'forced expiratory volume', 'urate measurement', 'BMI-adjusted waist circumference', 'vital capacity'], ['forced expiratory volume', 'waist-hip ratio', 'psoriasis', 'body height', 'chronotype measurement'], ['neoplasm', 'cancer', 'colorectal carcinoma', 'osteosarcoma', 'breast carcinoma'], ['breast adenocarcinoma', 'squamous cell lung carcinoma', 'response to vaccine', 'cytokine measurement', 'hemoglobin measurement'], ['neoplasm', 'rheumatoid arthritis', 'refractory anemia', 'breast cancer', 'acute myeloid leukemia'], ['pulse pressure measurement', 'pathological myopia', 'smoking behavior', 'appendicular lean mass', 'small cell lung carcinoma'], ['mental retardation, autosomal dominant 48', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'cutaneous Leishmaniasis', 'cancer'], ['lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'gastric adenocarcinoma', 'clear cell renal carcinoma'], ['nan', 'nan', 'nan', 'nan', 'nan'], ['Nijmegen breakage syndrome-like disorder', 'Inherited cancer-predisposing syndrome', 'Hereditary breast and ovarian cancer syndrome', 'ovarian cancer', 'low grade glioma'], ['Fanconi anemia', 'Familial congenital mirror movements', 'Hereditary breast cancer', 'breast carcinoma', 'genetic disorder'], ['electrocardiography', 'parental longevity', 'neoplasm', 'breast cancer', 'kidney cancer'], ['Noonan syndrome', 'LEOPARD syndrome', 'dilated cardiomyopathy', 'hepatocellular carcinoma', 'rasopathy'], ['Omenn syndrome', 'Combined immunodeficiency with skin granulomas', 'Severe combined immunodeficiency due to complete RAG1/2 deficiency', 'Combined immunodeficiency T+ B+ due to partial RAG1 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['Omenn syndrome', 'Combined immunodeficiency with skin granulomas', 'Severe combined immunodeficiency due to complete RAG1/2 deficiency', 'rag2 deficiency', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency'], ['Smith-Magenis syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Combined immunodeficiency due to ORAI1 deficiency'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'neoplasm', 'melanoma', 'glioblastoma multiforme'], ['skin cancer', 'non-melanoma skin carcinoma', 'body mass index', 'hair color', 'skin carcinoma'], ['adolescent idiopathic scoliosis', 'obesity', 'Familial cylindromatosis', 'pseudotumor cerebri', 'Capillary malformation - arteriovenous malformation'], ['type II diabetes mellitus', 'BMI-adjusted waist-hip ratio', 'chondrosarcoma', 'Female infertility due to zona pellucida defect', 'female infertility due to oocyte meiotic arrest'], ['glioblastoma multiforme', 'adolescent idiopathic scoliosis', 'hepatocellular carcinoma', 'ulcerative colitis', 'eosinophil percentage of leukocytes'], ['reticulocyte count', 'cutaneous melanoma', 'gut microbiome measurement', 'body height', 'optic disc size measurement'], ['albuminuria', 'body height', 'systolic blood pressure', 'diastolic blood pressure', 'cutaneous melanoma'], ['acute promyelocytic leukemia', 'acne', 'psoriasis', 'acute myeloid leukemia', 'freckles'], ['Matthew-Wood syndrome', 'acne', 'acute promyelocytic leukemia', 'psoriasis', 'Syndromic microphthalmia'], ['mean platelet volume', 'urinary metabolite measurement', 'diet measurement', 'alcohol consumption measurement', 'aspartate aminotransferase measurement'], ['FEV/FEC ratio', 'white matter hyperintensity measurement', 'endometrial carcinoma', 'non-small cell lung carcinoma', 'colorectal cancer'], ['Retinoblastoma', 'small cell lung carcinoma', 'retinoblastoma (nonhereditary)', 'bladder carcinoma', 'cutaneous melanoma'], ['viral disease', 'cancer', 'breast carcinoma', 'melanoma', 'endometrial cancer'], ['genetic disorder', 'lung adenocarcinoma', 'breast adenocarcinoma', 'red blood cell distribution width', 'cutaneous melanoma'], ['lean body mass', 'appendicular lean mass', 'body height', 'self reported educational attainment', 'fat body mass'], ['high density lipoprotein cholesterol measurement', 'neoplasm', 'lung cancer', 'Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder'], ['Thrombocytopenia - absent radius', 'Global developmental delay', 'Clinodactyly of the 5th finger', 'Abnormality of brain morphology', 'endometrial cancer'], ['TARP syndrome', 'lung adenocarcinoma', 'colorectal adenocarcinoma', 'non-small cell lung carcinoma', 'genetic disorder'], ['refractive error measurement', 'rheumatoid arthritis', 'multiple sclerosis', 'refractory anemia', 'systemic scleroderma'], ['Progressive retinal dystrophy due to retinol transport defect', 'Colobomatous microphthalmia', 'Reticular dystrophy of the retinal pigment epithelium', 'microphthalmia', 'Retinitis pigmentosa'], ['cancer', 'multiple myeloma', 'HIV infection', 'immune system disease', 'Mantle cell lymphoma'], ['Reticular dystrophy of the retinal pigment epithelium', 'Familial exudative vitreoretinopathy', 'Coats disease', 'Retinitis pigmentosa', 'neutrophil count'], ['FEV/FEC ratio', 'mean platelet volume', 'Miyoshi myopathy', 'multiple myeloma', 'myeloid neoplasm'], ['ovarian cancer', 'atrial fibrillation', 'breast carcinoma', 'age-related macular degeneration', 'Stargardt disease'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'hearing loss', 'Non-syndromic genetic deafness', 'testosterone measurement'], ['Rothmund-Thomson syndrome', 'Baller-Gerold syndrome', 'RAPADILINO syndrome', 'Rothmund-Thomson syndrome type 2', 'osteosarcoma'], ['ovarian cancer', 'heel bone mineral density', 'worry measurement', 'bone quantitative ultrasound measurement', 'high density lipoprotein cholesterol measurement'], ['neoplasm', 'Wolfram syndrome', 'glioma', 'systemic lupus erythematosus', 'synovial sarcoma'], ['ulcer disease', 'Primary immunodeficiency', 'gastric adenocarcinoma', 'Rare pervasive developmental disorder', 'breast adenocarcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Recurrent infections', 'lung adenocarcinoma', 'low density lipoprotein cholesterol measurement', 'free cholesterol measurement'], ['Lissencephaly syndrome, Norman-Roberts type', 'temporal lobe epilepsy', 'Lissencephaly', 'Rolandic epilepsy', 'Autosomal dominant epilepsy with auditory features'], ['monocyte count', 'eosinophil count', 'platelet count', 'protein measurement', 'platelet crit'], ['eosinophil count', 'red blood cell distribution width', 'eosinophil percentage of leukocytes', 'mean reticulocyte volume', 'mean corpuscular volume'], ['Multiple endocrine neoplasia type 2A', 'Hirschsprung disease', 'Familial medullary thyroid carcinoma', 'Multiple endocrine neoplasia type 2', 'Multiple endocrine neoplasia type 2B'], ['ulcerative colitis', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia', 'Glycogen storage disease due to liver phosphorylase kinase deficiency', 'adrenal gland pheochromocytoma'], ['cerebellar ataxia, neuropathy, and vestibular areflexia syndrome', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'polyneuropathy', 'mean corpuscular hemoglobin concentration'], ['endometrial cancer', 'lung adenocarcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'esophageal cancer'], ['Immunodeficiency by defective expression of HLA class 2', 'Combined T and B cell immunodeficiency', 'inflammatory bowel disease', 'hepatocellular carcinoma', 'neoplasm'], ['Immunodeficiency by defective expression of HLA class 2', 'pancreatic carcinoma', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm', 'Isolated osteopoikilosis'], ['heel bone mineral density', 'blood protein measurement', 'high density lipoprotein cholesterol measurement', 'sleep duration', 'opioid dependence'], ['Rh deficiency syndrome', 'red blood cell distribution width', 'platelet count', 'low density lipoprotein cholesterol measurement', 'reticulocyte count'], ['Rh deficiency syndrome', 'red blood cell distribution width', 'mean corpuscular volume', 'platelet count', 'platelet crit'], ['Retinitis pigmentosa', 'Congenital stationary night blindness', 'Retinitis punctata albescens', 'Retinal dystrophy', 'Saldino-Mainzer syndrome'], ['systolic blood pressure', 'breast adenocarcinoma', 'HMG CoA reductase inhibitor use measurement', 'neoplasm', 'hepatocellular carcinoma'], ['platelet count', 'mean platelet volume', 'platelet crit', 'neutrophil count', 'reticulocyte measurement'], ['Epidermodysplasia verruciformis', 'T-cell immunodeficiency with epidermodysplasia verruciformis', 'lymphoid neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'breast ductal adenocarcinoma'], ['protein measurement', 'appendicitis', 'neuroblastoma', 'body height', 'lean body mass'], ['reticulocyte count', 'reticulocyte measurement', 'red blood cell density measurement', 'Immature Reticulocyte Fraction Measurement', 'platelet count'], ['blood protein measurement', 'synovial sarcoma', 'Wolfram syndrome', 'Progressive supranuclear palsy', 'Classical progressive supranuclear palsy'], ['cone-rod synaptic disorder syndrome, congenital nonprogressive', 'self reported educational attainment', 'demyelinating disease', 'intelligence', 'particulate matter air pollution measurement'], ['immunodeficiency disease', 'autoinflammation with episodic fever and lymphadenopathy', 'severe acute respiratory syndrome', 'Primary immunodeficiency', 'Autosomal recessive early-onset inflammatory bowel disease'], ['leprosy', "Crohn's disease", 'protein measurement', 'ulcer disease', 'Oral ulcer'], ['severe acute respiratory syndrome', 'viral disease', 'mitral valve disease', 'hematocrit', 'hemoglobin measurement'], ['Bartsocas-Papas syndrome', 'CHAND syndrome', 'skin cancer', 'basal cell carcinoma', 'non-melanoma skin carcinoma'], ["Parkinson's disease", 'body mass index', 'adolescent idiopathic scoliosis', 'disease progression measurement', 'metastatic colorectal cancer'], ['Retinitis punctata albescens', 'Retinitis pigmentosa', 'Fundus albipunctatus', 'Bothnia retinal dystrophy', 'Retinal dystrophy'], ['glioblastoma multiforme', 'cutaneous melanoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['cutaneous melanoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'neoplasm', 'colorectal carcinoma'], ['myeloid white cell count', 'Niemann-Pick disease type C', 'self reported educational attainment', 'neutrophil count', 'alcohol consumption measurement'], ['eosinophil count', 'lean body mass', 'vitamin D measurement', 'diastolic blood pressure', 'fat body mass'], ['waist-hip ratio', 'red blood cell distribution width', "Alzheimer's disease", 'skin basal cell carcinoma', 'mean corpuscular volume'], ['blood protein measurement', 'visual perception measurement', 'neoplasm', 'acute kidney failure', "Barrett's esophagus"], ['blood protein measurement', 'eosinophil count', 'eosinophil cationic protein measurement', 'glioma', 'asthma'], ['nose morphology measurement', 'lung adenocarcinoma', 'ovarian cancer', 'eosinophil cationic protein measurement', 'squamous cell lung carcinoma'], ['Adult-onset chronic progressive external ophthalmoplegia with mitochondrial myopathy', 'Mitochondrial disease', 'Hyperlysinemia', 'Barth syndrome', 'progressive external ophthalmoplegia'], ['Cystic fibrosis', 'sRAGE measurement', 'waist circumference', 'cancer', 'neoplasm'], ['lymphocyte count', 'monocyte count', 'myeloid white cell count', 'hematocrit', 'testosterone measurement'], ['autoinflammatory syndrome with pyogenic bacterial infection and amylopectinosis', 'Polyglucosan body myopathy', 'Short stature due to GHSR deficiency', 'ovarian carcinoma', 'breast cancer'], ['squamous cell lung carcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'clear cell renal carcinoma', 'urinary bladder cancer'], ['Hyperplastic polyposis syndrome', 'gastric adenocarcinoma', 'pancreatic carcinoma', 'colorectal adenocarcinoma', 'pancreatic adenocarcinoma'], ['self reported educational attainment', 'lifestyle measurement', 'mathematical ability', 'glioma', 'glioblastoma multiforme'], ['mean corpuscular volume', 'ovarian cancer', 'breast adenocarcinoma', 'waist-hip ratio', 'esophageal cancer'], ['HIV infection', 'mental or behavioural disorder', 'acute myeloid leukemia', 'Cryopyrin-associated periodic syndrome', 'prostate cancer'], ['psoriasis', 'reticulocyte count', 'reticulocyte measurement', 'Immature Reticulocyte Fraction Measurement', 'Blackfan-Diamond anemia'], ['Isolated growth hormone deficiency type IB', 'isolated congenital growth hormone deficiency', 'Isolated growth hormone deficiency type IA', 'chronic lymphocytic leukemia', 'esophageal squamous cell carcinoma'], ['Pituitary stalk interruption syndrome', 'Congenital hydronephrosis', 'heel bone mineral density', 'neuroimaging measurement', 'lobe attachment'], ['Familial vesicoureteral reflux', 'chronotype measurement', 'body mass index', 'snoring measurement', 'type II diabetes mellitus'], ['Horizontal gaze palsy with progressive scoliosis', 'lung adenocarcinoma', 'cutaneous melanoma', 'genetic disorder', 'macula measurement'], ['aortic valve disease', 'Familial bicuspid aortic valve', 'aneurysm or dilatation of ascending aorta', 'Bicuspid aortic valve', 'congenital diaphragmatic hernia'], ['ocular hypertension', 'open-angle glaucoma', 'glaucoma', 'Fuchs endothelial corneal dystrophy', 'breast neoplasm'], ['ocular hypertension', 'open-angle glaucoma', 'glaucoma', 'mean corpuscular volume', 'lymphocyte count'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'vital capacity', 'Macular dystrophy', 'forced expiratory volume'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'acute myeloid leukemia', 'cortical surface area measurement', 'neuroimaging measurement'], ['intellectual developmental disorder with or without epilepsy or cerebellar ataxia', 'body height', 'protein measurement', 'body mass index', 'asthma'], ['generalised epilepsy', 'genetic disorder', 'refractive error measurement', 'testosterone measurement', 'age at menarche'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'inflammatory bowel disease', 'ulcerative colitis', 'Eczema', 'asthma'], ['non-small cell lung carcinoma', 'neoplasm', 'lung adenocarcinoma', 'lung carcinoma', 'cutaneous melanoma'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Leber congenital amaurosis', 'Macular dystrophy', 'Abnormality of the eye'], ['blood metabolite measurement', 'glycine measurement', 'serum metabolite measurement', 'body height', 'glomerular filtration rate'], ['Leber congenital amaurosis', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Severe early-childhood-onset retinal dystrophy', 'Adrenocorticotropic hormone deficiency'], ['cortical surface area measurement', 'brain volume measurement', 'Exercise-induced hyperinsulinism', 'Renal-hepatic-pancreatic dysplasia', 'Testicular regression syndrome'], ['Retinitis pigmentosa', 'Cone rod dystrophy', 'Primary ciliary dyskinesia - retinitis pigmentosa', 'Primary ciliary dyskinesia', 'Retinal dystrophy'], ['Meckel syndrome', 'Joubert syndrome with hepatic defect', 'Joubert syndrome with renal defect', 'Joubert syndrome', 'body mass index'], ['Blackfan-Diamond anemia', 'influenza', 'genetic disorder', 'non-small cell lung carcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['X-linked syndromic intellectual disability', 'X-linked intellectual disability - psychosis - macroorchidism', 'autism', 'influenza', 'multiple myeloma'], ['Blackfan-Diamond anemia', 'influenza', 'Short thumb', 'Abnormality of radial ray', 'Fanconi anemia'], ['spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'influenza', 'Spondyloepimetaphyseal dysplasia, matrilin-3 type', 'peak expiratory flow', 'hair color'], ['influenza', 'renal cell carcinoma', 'colorectal cancer', 'eosinophil count', 'colorectal adenoma'], ['Blackfan-Diamond anemia', 'influenza', 'intelligence', 'cognitive function measurement', 'glioblastoma multiforme'], ['influenza', 'endometrial cancer', 'neoplasm', 'cancer', 'non-small cell lung carcinoma'], ['influenza', 'clear cell renal carcinoma', 'endometrial cancer', 'neoplasm', 'colorectal carcinoma'], ['Hypotrichosis simplex', 'influenza', 'aspartate aminotransferase measurement', 'endometrial cancer', 'monocyte count'], ['influenza', 'colorectal adenocarcinoma', 'colon adenocarcinoma', 'bile duct carcinoma', 'basal cell carcinoma'], ['Blackfan-Diamond anemia', 'influenza', 'neoplasm', 'Thymoma', 'breast cancer'], ['influenza', 'Blackfan-Diamond anemia', 'myeloid white cell count', 'clear cell renal carcinoma', 'leukocyte count'], ['influenza', 'acute myeloid leukemia', 'lung cancer', 'lean body mass', 'body mass index'], ['Blackfan-Diamond anemia', 'influenza', 'neuroblastoma', 'grip strength measurement', 'Abnormal sperm morphology'], ['influenza', 'Blackfan-Diamond anemia', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['influenza', 'protein measurement', 'DNA methylation', 'Autosomal recessive spondylocostal dysostosis', 'Spondylometaphyseal dysplasia, A4 type'], ['influenza', 'endometrial cancer', 'metaplastic breast carcinoma', 'neoplasm', 'pancreatic carcinoma'], ['influenza', 'body mass index', 'body height', 'lean body mass', 'fat body mass'], ['influenza', 'rheumatoid arthritis', 'hair shape measurement', 'lymphocyte count', 'chronic lymphocytic leukemia'], ['influenza', 'breast carcinoma', 'Abnormal sperm morphology', 'Keratoconus', 'body fat percentage'], ['COVID-19', 'monocyte count', 'monocyte percentage of leukocytes', 'leukocyte count', 'basophil count'], ['breast adenocarcinoma', 'lung adenocarcinoma', "Crohn's disease", 'inflammatory bowel disease', 'red blood cell distribution width'], ['influenza', 'clear cell renal carcinoma', 'serum albumin measurement', 'total cholesterol measurement', 'blood protein measurement'], ['influenza', 'electrocardiography', "Parkinson's disease", 'heel bone mineral density', 'glioblastoma multiforme'], ['influenza', 'clear cell renal carcinoma', 'pancreatic adenocarcinoma', 'smoking behavior', 'gastric cancer'], ['influenza', 'neoplasm', 'breast cancer', 'myelodysplastic syndrome', 'Menkes disease'], ['Blackfan-Diamond anemia', 'influenza', 'Short thumb', 'Abnormality of radial ray', 'kidney failure'], ['influenza', 'Blackfan-Diamond anemia', 'monocyte count', 'systemic lupus erythematosus', 'body height'], ['Blackfan-Diamond anemia', 'influenza', 'Abnormality of radial ray', 'clear cell renal carcinoma', 'protein measurement'], ['Blackfan-Diamond anemia', 'influenza', 'osteosarcoma', 'clear cell renal carcinoma', 'late-onset Alzheimers disease'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'influenza'], ['influenza', 'blood metabolite measurement', 'serum metabolite measurement', 'metabolite measurement', 'pulse pressure measurement'], ['Blackfan-Diamond anemia', 'influenza', 'clear cell renal carcinoma', 'endometrial cancer', 'mathematical ability'], ['Familial isolated congenital asplenia', 'influenza', 'schizophrenia', 'dentate gyrus volume measurement', 'small cell lung carcinoma'], ['genetic disorder', 'Noonan syndrome', 'endometrial cancer', 'lung adenocarcinoma', 'hypertension'], ['eosinophil count', 'platelet count', 'alkaline phosphatase measurement', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement'], ['X-linked retinoschisis', 'retinoschisis', 'Retinal dystrophy', 'North Carolina macular dystrophy', 'Angelman syndrome'], ['chronotype measurement', 'systolic blood pressure', 'pulse pressure measurement', 'Obesity due to melanocortin 4 receptor deficiency', 'Obesity due to congenital leptin deficiency'], ['age at menarche', 'cervical cancer', 'melanoma', 'gastric cancer', 'handedness'], ['Testicular Germ Cell Tumor', 'heel bone mineral density', 'red blood cell density measurement', 'adolescent idiopathic scoliosis', 'age at menopause'], ['Tetraamelia - multiple malformations', 'humerofemoral hypoplasia with radiotibial ray deficiency', 'hair shape measurement', 'esophageal adenocarcinoma', 'prostate carcinoma'], ['Anonychia congenita totalis', 'Congenital anonychia', 'Anonychia', 'genetic disorder', 'Ectodermal dysplasia syndrome'], ['myeloid white cell count', 'neutrophil count', 'ovarian carcinoma', 'cancer', 'Plasmodium falciparum malaria'], ['Dyskeratosis congenita', 'idiopathic pulmonary fibrosis', 'pulmonary fibrosis and/or bone marrow failure, telomere-related, 1', 'Hoyeraal-Hreidarsson syndrome', 'acute myeloid leukemia'], ['Autosomal dominant spastic paraplegia type 12', 'Hereditary spastic paraplegia', 'Spastic paraplegia', 'drug use measurement', 'infect'], ['blood protein measurement', 'non-small cell lung carcinoma', 'breast cancer', 'urinary bladder cancer', 'neoplasm'], ['basal cell carcinoma', 'acute myeloid leukemia', 'mathematical ability', 'acute lymphoblastic leukemia', 'lymphoid neoplasm'], ['Cleidocranial dysplasia', 'Metaphyseal dysplasia - maxillary hypoplasia - brachydacty', 'Craniosynostosis', 'brain volume measurement', 'neuroimaging measurement'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'red blood cell distribution width', 'head and neck squamous cell carcinoma', 'eosinophil count'], ['breast adenocarcinoma', 'blood protein measurement', 'endometrial cancer', 'testosterone measurement', 'Glycogen storage disease due to muscle and heart glycogen synthase deficiency'], ['reticulocyte count', 'heart failure', 'cardiovascular disease', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['cryptorchidism', 'hypertension', 'cardiovascular disease', 'systolic blood pressure', 'hemoglobin measurement'], ['ovarian cancer', 'cutaneous melanoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma'], ['Congenital multicore myopathy with external ophthalmoplegia', 'malignant hyperthermia, susceptibility to, 1', 'King-Denborough syndrome', 'Central core disease', 'Malignant hyperthermia'], ['Catecholaminergic polymorphic ventricular tachycardia', 'catecholaminergic polymorphic ventricular tachycardia 1', 'Familial isolated arrhythmogenic right ventricular dysplasia', 'Arrhythmogenic right ventricular dysplasia', 'Abnormality of the cardiovascular system'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'chronic progressive multiple sclerosis', 'secondary progressive multiple sclerosis'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'deafness', 'immune system disease'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'monocyte count', 'Cognitive impairment', 'immune system disease'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'Cognitive impairment', 'monocyte count'], ['multiple sclerosis', 'relapsing-remitting multiple sclerosis', 'immune system disease', 'Cognitive impairment', 'primary progressive multiple sclerosis'], ['Primary immunodeficiency', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'platelet crit', 'hepatocellular carcinoma'], ['pulse pressure measurement', 'prostate carcinoma', 'neoplasm', 'ovarian cancer', "Crohn's disease"], ['protein measurement', 'neoplasm', 'pancreatic carcinoma', 'hepatocellular carcinoma', 'Malignant Pancreatic Neoplasm'], ['Primary immunodeficiency', 'glioblastoma multiforme', 'cutaneous melanoma', 'endometrial cancer', 'inflammatory bowel disease'], ['Primary immunodeficiency', 'cutaneous melanoma', 'basophil percentage of leukocytes', 'matrix metalloproteinase 8 measurement', 'basophil count'], ['blood metabolite measurement', 'hemoglobin measurement', 'diverticulitis', 'diverticular disease', 'childhood onset asthma'], ['endometrial cancer', 'neutrophil count', 'monocyte percentage of leukocytes', 'basophil percentage of leukocytes', 'granulocyte percentage of myeloid white cells'], ['head and neck squamous cell carcinoma', 'cutaneous melanoma', 't-tau:beta-amyloid 1-42 ratio measurement', 'lymphocyte count', "Alzheimer's disease"], ['Autosomal recessive spastic ataxia of Charlevoix-Saguenay', 'Spastic paraplegia', 'genetic disorder', 'Autosomal recessive spastic ataxia', 'Abnormality of brain morphology'], ['Oguchi disease', 'Retinitis pigmentosa', 'Retinal dystrophy', 'Congenital stationary night blindness', 'Progressive cone dystrophy'], ['Townes-Brocks syndrome', 'heel bone mineral density', 'erythrocyte count', 'hematocrit', 'C-reactive protein measurement'], ['drug use measurement', 'adenocarcinoma', 'neoplasm', 'pancreatic ductal adenocarcinoma', 'obesity'], ['Retinitis pigmentosa', 'Griscelli disease type 3', 'Uncombable hair syndrome', 'attention deficit hyperactivity disorder', 'Hypotrichosis simplex'], ['Sarcosinemia', 'glioblastoma multiforme', 'breast adenocarcinoma', 'Gastrointestinal stromal tumor', 'neoplasm'], ['Hereditary folate malabsorption', 'amyotrophic lateral sclerosis', 'blood protein measurement', 'semaphorin-3A measurement', 'interleukin 5 measurement'], ['neurodevelopmental disorder with microcephaly, ataxia, and seizures', 'Autosomal recessive non-syndromic intellectual disability', 'Cerebral arteriovenous malformation', 'infect', 'infection'], ['Hyperuricemia-pulmonary hypertension-renal failure-alkalosis syndrome', 'pulmonary arterial hypertension', 'multiple myeloma', 'B-cell non-Hodgkins lymphoma', 'head and neck carcinoma'], ['Disseminated superficial actinic porokeratosis', 'porokeratosis', 'neoplasm', 'hepatocellular carcinoma', 'infect'], ['testosterone measurement', 'sex hormone-binding globulin measurement', 'blood protein measurement', 'BMI-adjusted waist-hip ratio', 'waist-hip ratio'], ['mean platelet volume', 'platelet count', 'sitting height measurement', 'platelet component distribution width', 'Autosomal dominant spastic paraplegia type 3'], ['Shwachman-Diamond syndrome', 'acute myeloid leukemia', 'genetic disorder', 'Aplastic anemia', 'Deeply set eye'], ['heel bone mineral density', 'reticulocyte count', 'bone quantitative ultrasound measurement', 'lymphocyte count', 'bone density'], ['colorectal carcinoma', 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma', 'gastric carcinoma', 'lung adenocarcinoma'], ['adolescent idiopathic scoliosis', 'body height', "Alzheimer's disease", 'appendicular lean mass', 'smoking status measurement'], ['von Willebrand factor measurement', 'blood protein measurement', 'factor VIII measurement', 'squamous cell lung carcinoma', 'lung adenocarcinoma'], ['basophil count', 'basophil percentage of leukocytes', 'skin pigmentation', 'macula measurement', 'mean platelet volume'], ['type II diabetes mellitus', 'reticulocyte measurement', 'lymphocyte count', 'monocyte count', 'basophil count'], ['heel bone mineral density', 'neutrophil count', 'Huntington disease', "Alzheimer's disease", 'malaria'], ['protein measurement', 'mathematical ability', 'eosinophil count', 'appendicular lean mass', 'leukocyte count'], ['Generalized epilepsy with febrile seizures-plus', 'Dravet syndrome', 'migraine, familial hemiplegic, 3', 'Familial or sporadic hemiplegic migraine', 'Seizure'], ['Early infantile epileptic encephalopathy', 'Benign familial neonatal-infantile seizures', 'Seizure', 'Benign familial infantile epilepsy', 'bipolar disorder'], ['Familial focal epilepsy with variable foci', 'Early infantile epileptic encephalopathy', 'partial epilepsy', 'bipolar disorder', 'epilepsy'], ['Hyperkalemic periodic paralysis', 'Paramyotonia congenita of Von Eulenburg', 'Hypokalemic periodic paralysis', 'Potassium-aggravated myotonia', 'Acetazolamide-responsive myotonia'], ['Brugada syndrome', 'Familial atrial fibrillation', 'Familial long QT syndrome', 'Familial progressive cardiac conduction defect', 'Familial sick sinus syndrome'], ['epilepsy', 'pain', 'bipolar disorder', 'Seizure', 'unipolar depression'], ['Early infantile epileptic encephalopathy', 'epilepsy', 'cognitive impairment with or without cerebellar ataxia', 'Seizure', 'Benign familial infantile epilepsy'], ['Paroxysmal extreme pain disorder', 'Primary erythermalgia', 'Sodium channelopathy-related small fiber neuropathy', 'Generalized epilepsy with febrile seizures-plus', 'Channelopathy-associated congenital insensitivity to pain'], ['atrial fibrillation', 'Sodium channelopathy-related small fiber neuropathy', 'epilepsy', 'pain', 'bipolar disorder'], ['leukocyte count', 'myeloid white cell count', 'lymphocyte count', 'hemoglobin measurement', 'snoring measurement'], ['Fasciitis', 'central nervous system cancer', 'type II diabetes mellitus', 'Palmar Fibromatosis', 'heel bone mineral density'], ['Interauricular communication', 'atrial heart septal defect', 'diabetes mellitus', 'insulinoma', 'Hyperglycemia'], ['immature platelet fraction', 'idiopathic pulmonary fibrosis', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia', 'vital capacity'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'apolipoprotein A 1 measurement'], ['FEV/FEC ratio', 'appendicular lean mass', 'ovarian cancer', 'peak expiratory flow', 'breast adenocarcinoma'], ['lung adenocarcinoma', 'non-small cell lung carcinoma', 'gastric adenocarcinoma', 'squamous cell lung carcinoma', 'diffuse gastric adenocarcinoma'], ['neoplasm', 'esophageal squamous cell carcinoma', 'graft versus host disease', 'colorectal carcinoma', 'rheumatoid arthritis'], ['hair color', 'blood protein measurement', 'psoriasis', "Crohn's disease", 'sclerosing cholangitis'], ['adrenal gland pheochromocytoma', 'Hereditary pheochromocytoma-paraganglioma', 'Carney-Stratakis syndrome', 'Gastrointestinal stromal tumor', 'Cowden syndrome'], ['Hereditary pheochromocytoma-paraganglioma', 'adrenal gland pheochromocytoma', 'Carney-Stratakis syndrome', 'Isolated succinate-CoQ reductase deficiency', 'Cowden syndrome'], ['cerebrospinal fluid biomarker measurement', 'Gastrointestinal stromal tumor', 'neoplasm', 'body mass index', 'renal carcinoma'], ['HIV infection', 'influenza', 'viral disease', 'Histiocytosis', 'breast adenocarcinoma'], ['sex hormone-binding globulin measurement', 'neoplasm', 'cancer', 'non-small cell lung carcinoma', 'platelet crit'], ['blood protein measurement', 'monocyte percentage of leukocytes', 'neoplasm', 'melanoma', 'granulocyte percentage of myeloid white cells'], ['Sickle cell anemia', 'glioblastoma multiforme', 'e-selectin measurement', 'breast adenocarcinoma', 'age at menopause'], ['extraoral halitosis due to methanethiol oxidase deficiency', 'red blood cell distribution width', 'protein measurement', 'serum copper measurement', 'neoplasm'], ['Rigid spine syndrome', 'Congenital fiber-type disproportion myopathy', 'rigid spine muscular dystrophy 1', 'Classic multiminicore myopathy', 'Desmin-related myopathy with Mallory body-like inclusions'], ['lean body mass', 'whole body water mass', 'appendicular lean mass', 'fat body mass', 'vital capacity'], ['Sickle cell anemia', 'stroke', 'adhesion molecule measurement', 'soluble P-selectin measurement', 'blood protein measurement'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'inflammatory bowel disease', 'blood protein measurement', 'intraocular pressure measurement'], ['HIV infection', 'cortical surface area measurement', 'bone density', 'neuroimaging measurement', 'bone quantitative ultrasound measurement'], ['Kallmann syndrome', 'Brugada syndrome', 'brain volume measurement', 'Hirschsprung disease', 'PR interval'], ['Neurodevelopmental disorder', 'neoplasm', 'language impairment', 'anxiety', 'body height'], ['reticulocyte count', 'mean corpuscular volume', 'reticulocyte measurement', 'Abnormality of refraction', 'skin aging'], ['sex hormone-binding globulin measurement', 'pes', 'diastolic blood pressure', 'cancer', 'colorectal carcinoma'], ['refractive error measurement', 'glomerular filtration rate', 'body height', 'Non-syndromic male infertility due to sperm motility disorder', 'spermatogenic failure 43'], ['Alpha-1-antitrypsin deficiency', 'chronic obstructive pulmonary disease', 'Hemorrhagic disease due to alpha-1-antitrypsin Pittsburgh mutation', 'refractive error measurement', 'cholelithiasis'], ['endometrial cancer', 'Bronchopulmonary dysplasia', 'Borderline personality disorder', 'prostate cancer', 'nonpapillary renal cell carcinoma'], ['Alpha-1-antichymotrypsin deficiency', 'breast adenocarcinoma', 'prostate specific antigen measurement', 'alpha-1-antichymotrypsin measurement', 'alpha-1-antichymotrypsin complex measurement'], ['blood protein measurement', 'lung adenocarcinoma', 'glioblastoma multiforme', 'kallistatin measurement', 'cutaneous melanoma'], ['Corticosteroid-binding globulin deficiency', 'cortisol measurement', 'blood protein measurement', 'serum alpha-1-antitrypsin measurement', 'serum alanine aminotransferase measurement'], ['neutrophil count', 'childhood onset asthma', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract'], ['neoplasm', 'hepatocellular carcinoma', 'vitamin D measurement', 'psoriasis', 'protein measurement'], ['Hereditary thrombophilia due to congenital antithrombin deficiency', 'deep vein thrombosis', 'atrial fibrillation', 'pulmonary embolism', 'venous thromboembolism'], ['Venous thrombosis', 'heparin cofactor 2 deficiency', 'cutaneous melanoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma'], ['Congenital plasminogen activator inhibitor type 1 deficiency', 'Abnormal bleeding', 'erythrocyte count', 'plasminogen activator inhibitor 1 measurement', 'breast adenocarcinoma'], ['mean platelet volume', 'platelet-derived growth factor BB measurement', 'body height', 'interleukin 10 measurement', 'inflammatory biomarker measurement'], ['Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'Osteogenesis imperfecta type 4', 'blood protein measurement', 'forced expiratory volume'], ['Congenital alpha2 antiplasmin deficiency', 'Rare hemorrhagic disorder due to a constitutional coagulation factors defect', 'Abnormal bleeding', "Alzheimer's disease", 'testosterone measurement'], ['Hereditary angioedema type 1', 'Hereditary angioedema type 2', 'hereditary angioedema with C1Inh deficiency', 'Hereditary angioedema', 'Bartholin gland disease'], ['breast adenocarcinoma', 'endometrial cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['intellectual disability, autosomal dominant 58', 'acute lymphoblastic leukemia', 'T-cell acute lymphoblastic leukemia', 'Intellectual disability', 'Autosomal dominant non-syndromic intellectual disability'], ['Schinzel-Giedion syndrome', 'chronic myelogenous leukemia', 'acute myeloid leukemia', 'Juvenile Myelomonocytic Leukemia', 'myelodysplastic syndrome'], ['Luscan-Lumish syndrome', 'acute lymphoblastic leukemia', 'clear cell renal carcinoma', 'acute myeloid leukemia', 'renal cell carcinoma'], ['monocyte count', 'Rare pervasive developmental disorder', 'smoking status measurement', 'cigarettes per day measurement', 'smoking initiation'], ['acute myeloid leukemia', 'prostate carcinoma', 'adolescent idiopathic scoliosis', 'Hepatitis', 'mathematical ability'], ['chronic lymphocytic leukemia', 'acute myeloid leukemia', 'Uveal Melanoma', 'cutaneous melanoma', 'breast adenocarcinoma'], ['breast carcinoma', 'breast neoplasm', 'chronic lymphocytic leukemia', 'hepatocellular carcinoma', 'pancreatic adenocarcinoma'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'squamous cell lung carcinoma'], ['electrocardiography', 'body height', 'appendicular lean mass', 'FEV/FEC ratio', 'sitting height measurement'], ['alpha-linolenic acid measurement', 'gastric cancer', 'melanoma', 'hepatocellular carcinoma', 'colorectal carcinoma'], ['cutaneous melanoma', 'intelligence', 'body height', 'mathematical ability', 'cognitive function measurement'], ['Autosomal recessive limb-girdle muscular dystrophy type 2D', 'Limb-girdle muscular dystrophy', 'Autosomal recessive limb-girdle muscular dystrophy', 'Qualitative or quantitative defects of sarcoglycan', 'Muscular dystrophy'], ['Autosomal recessive limb-girdle muscular dystrophy type 2E', 'Limb-girdle muscular dystrophy', 'Autosomal recessive limb-girdle muscular dystrophy', 'Qualitative or quantitative defects of beta-sarcoglycan', 'genetic disorder'], ['Autosomal recessive limb-girdle muscular dystrophy type 2C', 'Autosomal recessive limb-girdle muscular dystrophy', 'type II diabetes mellitus', 'acute myeloid leukemia', 'muscular dystrophy, limb-girdle, autosomal recessive 23'], ['electrocardiography', 'hair color', 'sex hormone-binding globulin measurement', 'hair colour measurement', 'protein measurement'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'prostate carcinoma', 'Uncombable hair syndrome', 'Ringed hair disease'], ['Calvarial doughnut lesions - bone fragility', 'genetic disorder', 'protein measurement', 'Osteopenia', 'breast cancer'], ['platelet count', 'cancer', 'neoplasm', 'testosterone measurement', '46,XY disorder of sex development due to 5-alpha-reductase 2 deficiency'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'sex hormone-binding globulin measurement', 'eosinophil count', 'eosinophil percentage of leukocytes'], ['X-linked lymphoproliferative disease', 'adult lymphoma', 'genetic disorder', 'Aplastic anemia', 'Lymphoproliferative syndrome'], ['infect', 'infection', 'platelet component distribution width', 'mean platelet volume', 'platelet count'], ['Cherubism', 'chronotype measurement', 'platelet count', 'mean platelet volume', 'Gorham-Stout disease'], ['Age-related cataract', 'neutrophil count', 'prostate carcinoma', 'cognitive function measurement', 'self reported educational attainment'], ['Monosomy 22q13', 'schizophrenia', 'Intellectual disability', 'mental retardation', 'genetic disorder'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'heel bone mineral density', 'blood protein measurement', 'type II diabetes mellitus'], ['cancer', 'lung adenocarcinoma', 'acute myeloid leukemia', 'thyroid gland adenocarcinoma', 'endometrial cancer'], ['Holoprosencephaly', 'holoprosencephaly 3', 'Microform holoprosencephaly', 'Solitary median maxillary central incisor', 'Polydactyly of a triphalangeal thumb'], ['eosinophilic pneumonia', 'obsessive-compulsive disorder', 'Pentosuria', 'colorectal adenocarcinoma', 'Hereditary geniospasm'], ['Congenital sucrase-isomaltase deficiency', 'type II diabetes mellitus', 'intestinal disaccharide deficiency and disaccharide malabsorption', 'genetic disorder', 'abnormal glucose tolerance'], ['autoimmune disease', 'serum metabolite measurement', 'urinary system disease', 'prostate carcinoma', 'primary biliary cirrhosis'], ['red blood cell density measurement', 'colorectal carcinoma', 'breast cancer', 'hepatocellular carcinoma', 'mean corpuscular hemoglobin concentration'], ['lymphocyte count', 'apolipoprotein A 1 measurement', 'triglyceride measurement', 'systemic lupus erythematosus', 'lupus erythematosus'], ['cerebrospinal fluid biomarker measurement', 'neoplasm', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', "Alzheimer's disease"], ['monocyte percentage of leukocytes', 'blood protein measurement', 'sialic acid-binding Ig-like lectin 9 measurement', 'monocyte count', 'cerebrospinal fluid biomarker measurement'], ['COVID-19', 'Combined immunodeficiency with skin granulomas', 'Pancytopenia due to IKZF1 mutations', 'Hyper-IgM syndrome type 3', 'Autosomal dominant severe congenital neutropenia'], ['gastric carcinoma', 'esophageal carcinoma', 'cancer', 'neoplasm', 'bladder carcinoma'], ['Distal hereditary motor neuropathy, Jerash type', 'Juvenile amyotrophic lateral sclerosis', 'pain', 'sinusitis', 'Cough'], ['Early infantile epileptic encephalopathy', 'Early myoclonic encephalopathy', 'West syndrome', 'systolic blood pressure', 'hair color'], ['Rolandic epilepsy', 'Hyperinsulinism due to INSR deficiency', 'Hyperinsulinism due to glucokinase deficiency', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'Autosomal dominant hyperinsulinism due to Kir6.2 deficiency'], ['self reported educational attainment', 'mathematical ability', 'allergy', 'asthma', 'Eczema'], ['Spondyloepiphyseal dysplasia and spondyloepimetaphyseal dysplasia', 'triglyceride measurement', 'high density lipoprotein cholesterol measurement', 'vital capacity', 'proprotein convertase subtilisin/kexin type 7 measurement'], ['Marinesco-Sjögren syndrome', 'excessive daytime sleepiness measurement', 'lifestyle measurement', 'appendicular lean mass', 'glioma'], ['hematocrit', 'erythrocyte count', 'hemoglobin measurement', 'appendicular lean mass', 'sitting height measurement'], ['type II diabetes mellitus', 'HbA1c measurement', 'body mass index', 'Testicular regression syndrome', 'Normosmic congenital hypogonadotropic hypogonadism'], ['mean platelet volume', 'platelet crit', 'platelet component distribution width', 'reticulocyte measurement', 'platelet count'], ['breast adenocarcinoma', 'mean platelet volume', 'testosterone measurement', 'systolic blood pressure', 'neutrophil count'], ['neoplasm', 'colorectal carcinoma', 'Ehlers-Danlos syndrome, progeroid type', 'non-small cell lung carcinoma', 'melanoma'], ['mean platelet volume', 'platelet count', 'monocyte count', 'lymphocyte percentage of leukocytes', 'lymphocyte count'], ['facial morphology measurement', 'heel bone mineral density', 'hepatocellular carcinoma', 'kidney Wilms tumor', 'lung adenocarcinoma'], ['Holoprosencephaly', 'Schizencephaly', 'Microform holoprosencephaly', 'alcohol consumption measurement', 'smoking status measurement'], ['prostate carcinoma', 'glomerular filtration rate', 'male reproductive organ cancer', 'aspartate aminotransferase measurement', 'prostate cancer'], ['mean corpuscular volume', 'red blood cell distribution width', 'appendicular lean mass', 'body height', 'lean body mass'], ['hair colour measurement', 'inflammatory bowel disease', 'appendicular lean mass', 'cancer', 'gut microbiome measurement'], ['Graves disease', 'pancreatic adenocarcinoma', 'myxedema', 'rheumatoid arthritis', 'eosinophil count'], ['multiple myeloma', 'neoplasm', 'SLAM family member 7 measurement', 'blood protein measurement', 'protein measurement'], ['Dicarboxylic aminoaciduria', 'schizophrenia', 'Sensorineural hearing impairment', 'response to risperidone', 'Rare genetic intellectual disability with developmental anomaly'], ['appendicular lean mass', 'low density lipoprotein cholesterol measurement', 'sun exposure measurement', 'eye measurement', 'neoplasm'], ['Encephalopathy due to GLUT1 deficiency', 'Paroxysmal exertion-induced dyskinesia', 'generalised epilepsy', 'Paroxysmal dystonic choreathetosis with episodic ataxia and spasticity', 'Hereditary cryohydrocytosis with reduced stomatin'], ['prostate carcinoma', 'sex hormone-binding globulin measurement', 'multiple sclerosis', 'testosterone measurement', 'body height'], ['Arterial tortuosity syndrome', 'Abnormality of the cardiovascular system', 'marfan syndrome/loeys-dietz syndrome/familial thoracic aortic aneurysms and dissections', 'Rare disease with thoracic aortic aneurysm and aortic dissection', 'Moyamoya disease'], ['Sensorineural hearing impairment', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'neoplasm'], ['Hereditary spherocytosis', 'Autosomal dominant distal renal tubular acidosis', 'Distal renal tubular acidosis with anemia', 'Hereditary cryohydrocytosis with normal stomatin', 'Distal renal tubular acidosis'], ['Glucose-galactose malabsorption', 'type II diabetes mellitus', 'diabetes mellitus', 'Glucose intolerance', 'type I diabetes mellitus'], ['Renal glucosuria', 'type II diabetes mellitus', 'diabetes mellitus', 'heart failure', 'type I diabetes mellitus'], ['adolescent idiopathic scoliosis', "Crohn's disease", 'attention deficit hyperactivity disorder', 'Chest pain', 'lung adenocarcinoma'], ['Familial thyroid dyshormonogenesis', 'Congenital hypothyroidism', 'goiter', 'Neonatal diabetes - congenital hypothyroidism - congenital glaucoma - hepatic fibrosis - polycystic kidneys', 'Pendred syndrome'], ['1,5 anhydroglucitol measurement', 'Berardinelli-Seip congenital lipodystrophy', 'Familial partial lipodystrophy associated with PLIN1 mutations', 'Glycogen storage disease due to liver glycogen phosphorylase deficiency', 'Glycogen storage disease due to liver phosphorylase kinase deficiency'], ['alcohol consumption measurement', 'appendicular lean mass', 'alcohol use disorder measurement', 'lean body mass', 'systolic blood pressure'], ['attention deficit hyperactivity disorder', 'major depressive disorder', 'unipolar depression', 'depressive disorder', 'schizophrenia'], ['Infantile dystonia-parkinsonism', 'nicotine dependence', 'attention deficit hyperactivity disorder', 'unipolar depression', 'depressive disorder'], ['Hereditary hyperekplexia', 'Exaggerated startle response', 'brain disease', 'DNA methylation', 'cutaneous melanoma'], ['cutaneous melanoma', 'inflammatory bowel disease', 'ulcerative colitis', 'sitting height measurement', 'emphysema'], ['X-linked creatine transporter deficiency', 'Intellectual disability', 'mental retardation', 'genetic disorder', 'cerebral creatine deficiency syndrome'], ['Glycine encephalopathy', 'Atypical glycine encephalopathy', 'Neonatal glycine encephalopathy', 'Infantile glycine encephalopathy', 'schizophrenia'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'platelet count', 'hemoglobin measurement'], ['basophil count', 'basophil percentage of leukocytes', 'erythrocyte count', 'platelet count', 'allergic rhinitis'], ['protein measurement', 'electrocardiography', 'QT interval', 'heel bone mineral density', 'PR interval'], ['red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'platelet count', 'hemoglobin measurement', 'hematocrit'], ['Christianson syndrome', 'mental retardation', 'Intellectual disability', 'microcephaly (disease)', 'genetic disorder'], ['viral disease', 'hepatitis D virus infection', 'cutaneous melanoma', 'hemoglobin measurement', 'appendicular lean mass'], ['primary ovarian insufficiency', 'Chronic constipation', 'cholelithiasis', 'biliary tract disease', 'gallbladder disease'], ['hematocrit', 'hemoglobin measurement', 'mean corpuscular volume', "Alzheimer's disease", 'neoplasm'], ['Microcytic anemia with liver iron overload', 'Hypochromic microcytic anemia', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'Blackfan-Diamond anemia'], ['hearing loss', 'red blood cell distribution width', 'sex hormone-binding globulin measurement', 'body fat distribution', 'lymphocyte count'], ['Gitelman syndrome', 'hypertension', 'chronic kidney disease', 'congestive heart failure', 'nephrotic syndrome'], ['Pentosuria', 'Methylmalonic aciduria due to transcobalamin receptor defect', 'Iminoglycinuria', 'Cystathioninuria', 'Seizures - intellectual disability due to hydroxylysinuria'], ['Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'Seizure', 'Amelo-cerebro-hypohidrotic syndrome', 'Pyridoxine-dependent epilepsy'], ['mean corpuscular hemoglobin concentration', 'reticulocyte count', 'reticulocyte measurement', 'bladder carcinoma', 'hematocrit'], ['refractive error measurement', 'breast carcinoma', 'erythrocyte count', 'hematocrit', 'FEV/FEC ratio'], ['Exercise-induced hyperinsulinism', 'Metabolic myopathy due to lactate transporter defect', 'Inborn errors of metabolism', 'refractive error measurement', 'diastolic blood pressure'], ['body mass index', 'glioblastoma multiforme', 'neoplasm', 'cancer', 'inflammatory bowel disease'], ['body height', 'self reported educational attainment', 'diet measurement', 'mean corpuscular volume', 'mathematical ability'], ['age-related macular degeneration', 'breast adenocarcinoma', 'atrophic macular degeneration', 'wet macular degeneration', 'glioblastoma multiforme'], ['Salla disease', 'Free sialic acid storage disease, infantile form', 'Free sialic acid storage disease', 'Inborn errors of metabolism', 'Intermediate severe Salla disease'], ['FEV/FEC ratio', 'hypertension', 'appendicular lean mass', 'protein measurement', 'Young adult-onset Parkinsonism'], ['Bilateral striopallidodentate calcinosis', 'dystonic disorder', 'basal ganglia cerebrovascular disease', 'genetic disorder', 'sex hormone-binding globulin measurement'], ['glomerular filtration rate', 'creatinine measurement', 'prostate carcinoma', 'blood metabolite measurement', 'chronic kidney disease'], ['coronary artery disease', 'prostate carcinoma', 'type II diabetes mellitus', 'lean body mass', 'body mass index'], ['Systemic primary carnitine deficiency', 'Propionic acidemia', 'genetic disorder', 'mean platelet volume', 'leukocyte count'], ['gout', 'epilepsy', 'influenza', 'urate measurement', 'urinary tract infection'], ['systolic blood pressure', 'smoking status measurement', 'glomerular filtration rate', 'pulse pressure measurement', 'diastolic blood pressure'], ['gout', 'influenza', 'epilepsy', 'urinary tract infection', 'obesity'], ['Hereditary renal hypouricemia', 'gout', 'hyperuricemia', 'acquired metabolic disease', 'urate measurement'], ['D,L-2-hydroxyglutaric aciduria', 'Presynaptic congenital myasthenic syndromes', 'Congenital myasthenic syndromes', 'protein measurement', 'amino acid measurement'], ['lung adenocarcinoma', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'actinic keratosis', 'body height'], ['Cardiomyopathy - hypotonia - lactic acidosis', 'ovarian carcinoma', 'neoplasm', 'glioma', 'systemic lupus erythematosus'], ['mitochondrial DNA depletion syndrome 19', 'Isolated NADH-CoQ reductase deficiency', 'cutaneous melanoma', 'squamous cell lung carcinoma', 'endometrial cancer'], ['Carnitine-acylcarnitine translocase deficiency', 'atrial fibrillation', 'heel bone mineral density', 'alcohol consumption measurement', 'eosinophil count'], ['mitochondrial DNA depletion syndrome 18', 'vital capacity', 'protein measurement', 'mean corpuscular volume', 'forced expiratory volume'], ['mean corpuscular hemoglobin concentration', 'red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['Congenital chloride diarrhea', 'Polyhydramnios', 'hydrops fetalis (disease)', 'intestinal obstruction', 'Secretory diarrhea'], ['Pendred syndrome', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness 4', 'goiter', 'hearing loss'], ['Ichthyosis prematurity syndrome', 'genetic disorder', 'autosomal recessive disease', 'cutaneous melanoma', 'body mass index'], ['stroke', 'coronary artery disease', 'diabetes mellitus', 'ischemia reperfusion injury', 'bipolar disorder'], ['type II diabetes mellitus', 'diabetes mellitus', 'fasting blood glucose measurement', 'HbA1c measurement', 'insulin measurement'], ['Congenital cataract-hearing loss-severe developmental delay syndrome', 'Autosomal dominant spastic paraplegia type 42', 'Hereditary spastic paraplegia', 'neurodegenerative disease', 'cataract'], ['Pulmonary alveolar microlithiasis', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'cutaneous melanoma'], ['SLC35A1-CDG', 'Congenital disorder of glycosylation', 'congenital disorder of glycosylation type II', 'Seizure', 'Macrothrombocytopenia'], ['SLC35A2-CDG', 'Congenital disorder of glycosylation', 'Intellectual disability', 'epilepsy', 'X-linked complex neurodevelopmental disorder'], ['Primary bone dysplasia with multiple joint dislocations', 'glioblastoma multiforme', 'cutaneous melanoma', 'heel bone mineral density', 'myeloid white cell count'], ['eosinophil count', 'birth weight', 'body height', 'protein measurement', 'lean body mass'], ['Abnormality of neuronal migration', 'Leishmaniasis', 'head and neck squamous cell carcinoma', 'neoplasm', 'Miyoshi myopathy'], ['estrogen-receptor positive breast cancer', 'response to aromatase inhibitor', 'neuroimaging measurement', 'brain volume measurement', 'brain measurement'], ['Acrodermatitis enteropathica', 'genetic disorder', 'pancreatic carcinoma', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm'], ['Myopia', 'acute myeloid leukemia', "Barrett's esophagus", 'FTH1-related iron overload', 'Pentosuria'], ['esophageal carcinoma', 'survival time', 'squamous cell carcinoma', 'breast cancer', 'neoplasm'], ['Oculocutaneous albinism type 4', 'Oculocutaneous albinism', 'hair color', 'skin pigmentation', 'cutaneous melanoma'], ['platelet count', 'urinary albumin to creatinine ratio', 'serum alanine aminotransferase measurement', 'aspartate aminotransferase measurement', 'urate measurement'], ['maternal riboflavin deficiency', 'Transient neonatal multiple acyl-CoA dehydrogenase deficiency', 'body height', 'Low levels of vitamin B2', 'Keratoconus'], ['Riboflavin transporter deficiency', 'genetic disorder', 'neoplasm', 'melanoma', 'gastric cancer'], ['Progressive supranuclear palsy', 'serum metabolite measurement', 'lean body mass', 'cutaneous melanoma', 'fat body mass'], ['Rotor syndrome', 'blood metabolite measurement', 'chronic kidney disease', 'urinary metabolite measurement', 'serum metabolite measurement'], ['Rotor syndrome', 'bilirubin measurement', 'Disorder of bilirubin metabolism and excretion', 'ceramide measurement', 'testosterone measurement'], ['visual perception measurement', 'cutaneous melanoma', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'parasitic infection'], ['schizophrenia', 'body height', "Crohn's disease", 'BMI-adjusted waist circumference', 'BMI-adjusted waist-hip ratio'], ['inflammatory bowel disease', 'neoplasm', 'cancer', 'Ewing sarcoma', 'hepatocellular carcinoma'], ['leukocyte count', 'Autosomal agammaglobulinemia', 'lung adenocarcinoma', 'platelet count', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency'], ['Congenital hydronephrosis', 'type II diabetes mellitus', 'body weight', 'protein measurement', 'smoking status measurement'], ['ulcerative colitis', 'lean body mass', 'age at menarche', 'body height', 'lymphocyte count'], ['coronary artery disease', 'pulse pressure measurement', 'alcohol drinking', 'obesity', 'diabetes mellitus'], ['acute myeloid leukemia', 'cardiovascular disease', 'neoplasm', 'infect', 'infection'], ['Mal de Meleda', 'Hereditary palmoplantar keratoderma, Gamborg-Nielsen type', 'palmoplantar keratosis', 'Diffuse palmoplantar keratoderma', 'DNA methylation'], ['hemoglobin measurement', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'high density lipoprotein cholesterol measurement', 'protein measurement'], ['colorectal adenocarcinoma', 'cancer', 'genetic disorder', 'gastric carcinoma', 'colon adenocarcinoma'], ['Generalized juvenile polyposis/juvenile polyposis coli', 'Myhre syndrome', 'Juvenile polyposis syndrome', 'juvenile polyposis/hereditary hemorrhagic telangiectasia syndrome', 'Hereditary hemorrhagic telangiectasia'], ['colorectal cancer', 'eosinophil count', 'glomerular filtration rate', 'hemoglobin measurement', 'erythrocyte count'], ['pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'Idiopathic pulmonary arterial hypertension', 'Idiopathic and/or familial pulmonary arterial hypertension', 'heel bone mineral density'], ['Abnormality of brain morphology', 'prostate adenocarcinoma', 'Coffin-Siris syndrome', 'coronary artery disease', 'genetic disorder'], ['intellectual disability - sparse hair - brachydactyly', 'Intellectual disability', 'mental retardation', 'schizophrenia', 'genetic disorder'], ['Familial rhabdoid tumor', 'Coffin-Siris syndrome', 'lung adenocarcinoma', 'medulloblastoma', 'Ovarian Small Cell Carcinoma'], ['Familial rhabdoid tumor', 'Neurofibromatosis type 3', 'Coffin-Siris syndrome', 'atypical teratoid rhabdoid tumor', 'schwannoma'], ['Cornelia de Lange syndrome', 'Early infantile epileptic encephalopathy', 'Epileptic encephalopathy', 'genetic disorder', 'Wiedemann-Steiner syndrome'], ['melanoma', 'neoplasm', 'cancer', 'drug use measurement', 'lung cancer'], ['endometrial cancer', 'heel bone mineral density', 'coffee consumption measurement', 'tea consumption measurement', 'gut microbiome measurement'], ['Proximal spinal muscular atrophy', 'Proximal spinal muscular atrophy type 3', 'Proximal spinal muscular atrophy type 2', 'Proximal spinal muscular atrophy type 1', 'Proximal spinal muscular atrophy type 4'], ['Proximal spinal muscular atrophy type 3', 'Proximal spinal muscular atrophy type 2', 'Proximal spinal muscular atrophy type 1', 'Proximal spinal muscular atrophy', 'spinal muscular atrophy'], ['basal cell carcinoma', 'Curry-Jones syndrome', 'medulloblastoma', 'acute myeloid leukemia', 'neoplasm'], ['Microphthalmia with limb anomalies', 'polydactyly (disease)', 'lean body mass', 'red blood cell distribution width', 'mean corpuscular volume'], ['reticulocyte count', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'mean corpuscular hemoglobin', 'red blood cell distribution width'], ['Niemann-Pick disease type A', 'Niemann-Pick disease type B', 'Niemann-Pick disease type E', 'Niemann-Pick disease', 'Neuronal ceroid lipofuscinosis'], ['thyroid gland adenocarcinoma', 'body height', 'forced expiratory volume', 'sitting height measurement', 'lean body mass'], ['body height', 'erythrocyte count', 'self reported educational attainment', 'intelligence', 'schizophrenia'], ['Ewing sarcoma', 'infect', 'infection', 'neoplasm', 'adrenal cortex carcinoma'], ['X-linked intellectual disability, Snyder type', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Smith-Magenis syndrome'], ['monocyte count', 'coronary artery disease', 'coronary atherosclerosis', 'prostate carcinoma', 'body mass index'], ['mean corpuscular volume', 'ulcerative colitis', 'inflammatory bowel disease', 'mean corpuscular hemoglobin', 'eosinophil percentage of leukocytes'], ['lymphocyte count', 'sex hormone-binding globulin measurement', 'smoking status measurement', 'smoking initiation', 'monocyte count'], ['electrocardiography', 'PR interval', 'acute myeloid leukemia', 'systemic lupus erythematosus', 'protein measurement'], ['body height', 'sitting height measurement', 'psoriasis', 'neoplasm', 'hepatocellular carcinoma'], ['hemoglobin measurement', 'esophageal adenocarcinoma', 'esophageal squamous cell carcinoma', 'oral cavity carcinoma', 'erythrocyte count'], ['Congenital myasthenic syndromes', 'migraine disorder', 'Facial wrinkling', 'Spasticity', 'Torticollis'], ['Hereditary late-onset Parkinson disease', 'Lewy body dementia', 'Young adult-onset Parkinsonism', "Parkinson's disease", 'AL amyloidosis'], ['Ehlers-Danlos syndrome, progeroid type', "Alzheimer's disease", 'Lewy body dementia', 'cancer', 'pervasive developmental disorder - not otherwise specified'], ['red blood cell distribution width', 'chronotype measurement', 'mean platelet volume', 'platelet count', 'atrial fibrillation'], ['Hypotrichosis simplex', 'red blood cell distribution width', 'Non-syndromic male infertility due to sperm motility disorder', 'Partial chromosome Y deletion', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa'], ['FEV/FEC ratio', 'mean platelet volume', 'peak expiratory flow', 'pulmonary function measurement', 'Abnormality of refraction'], ['Prader-Willi syndrome', 'periodontal measurement', 'autism spectrum disorder', 'vasculitis', 'Proximal spinal muscular atrophy'], ['high density lipoprotein cholesterol measurement', 'alcohol drinking', 'alcohol consumption measurement', 'lip cancer', 'C-reactive protein measurement'], ['diabetes mellitus', 'testosterone measurement', 'chronotype measurement', 'cutaneous melanoma', 'schizophrenia'], ['lean body mass', 'body height', 'appendicular lean mass', 'forced expiratory volume', 'fat body mass'], ['amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'sporadic amyotrophic lateral sclerosis', 'spastic tetraplegia and axial hypotonia, progressive', 'motor neuron disease'], ['diabetic nephropathy', 'diabetic retinopathy', 'gastric carcinoma', 'acute myeloid leukemia', 'neuroblastoma'], ['sorbitol dehydrogenase deficiency with peripheral neuropathy', 'genetic disorder', 'Red cell distribution width', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement'], ["Alzheimer's disease", 'Early-onset autosomal dominant Alzheimer disease', 'refractive error measurement', 'alcohol consumption measurement', 'family history of Alzheimer’s disease'], ['appendicular lean mass', 'glomerular filtration rate', 'mathematical ability', 'low density lipoprotein cholesterol measurement', 'coronary artery disease'], ['Noonan syndrome', 'Hereditary gingival fibromatosis', 'rasopathy', 'cancer', 'bone development disease'], ['Sclerosteosis', 'Hyperostosis corticalis generalisata', 'Craniodiaphyseal dysplasia', 'postmenopausal osteoporosis', 'osteoporosis'], ['Anophthalmia/microphthalmia - esophageal atresia', 'genetic disorder', 'refractive error measurement', 'Septo-optic dysplasia', 'Jackson-Weiss syndrome'], ['Campomelic dysplasia', 'Pierre-Robin sequence', '46,XX testicular disorder of sex development', '46,XY complete gonadal dysgenesis', 'colorectal adenocarcinoma'], ['Familial vesicoureteral reflux', 'Heritable pulmonary arterial hypertension', 'pulmonary arterial hypertension', 'vesicoureteral reflux', 'endometrial cancer'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'esophageal cancer', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['birth weight', 'hair color', 'hepatocellular carcinoma', 'glioma', 'neoplasm'], ['appendicular lean mass', 'body height', 'red blood cell distribution width', 'multiple myeloma', 'lean body mass'], ['hair color', 'skin pigmentation', 'hair colour measurement', 'Amelogenesis imperfecta', 'body fat distribution'], ['neoplasm', 'cervical cancer', 'hepatocellular carcinoma', 'Myopia', 'cancer'], ['neoplasm', 'cervical cancer', 'hepatocellular carcinoma', 'Myopia', 'cancer'], ['Non-syndromic male infertility due to sperm motility disorder', 'BMI-adjusted waist circumference', 'body height', 'physical activity measurement', 'smoking behavior'], ['lung adenocarcinoma', 'cutaneous melanoma', 'breast adenocarcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme'], ['SPARC-like protein 1 measurement', 'blood protein measurement', 'protein measurement', "Alzheimer's disease", 'neoplasm'], ['systemic lupus erythematosus', 'eosinophil count', 'neoplasm', 'non-small cell lung carcinoma', 'cancer'], ['epilepsy, hearing loss, and mental retardation syndrome', 'BMI-adjusted waist-hip ratio', 'appendicular lean mass', 'type II diabetes mellitus', 'body height'], ['monocyte count', 'breast carcinoma', 'red blood cell distribution width', 'total cholesterol measurement', 'neoplasm'], ['mean corpuscular volume', 'erythrocyte count', 'schizophrenia', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration'], ['esophageal carcinoma', 'Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa', 'spermatogenic failure 43'], ['ovarian cancer', 'neutrophil count', 'gastric adenocarcinoma', 'endometrial cancer', 'reticulocyte count'], ['body height', 'heel bone mineral density', 'sitting height measurement', 'psoriasis', 'longitudinal alcohol consumption measurement'], ['Hereditary chronic pancreatitis', 'chronic pancreatitis', 'genetic disorder', 'alcoholic pancreatitis', 'blood protein measurement'], ['spermatogenic failure 29', 'azoospermia', 'reticulocyte measurement', 'reticulocyte count', 'blood protein measurement'], ['Netherton syndrome', 'Increased circulating IgE level', 'Epidermolytic ichthyosis', 'Failure to thrive', 'blood protein measurement'], ['glioblastoma multiforme', 'blood protein measurement', 'hepatocellular carcinoma', 'neoplasm', 'squamous cell carcinoma'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'hepatocellular carcinoma'], ['type II diabetes mellitus', 'electrocardiography', 'blood protein measurement', 'body height', 'inflammatory bowel disease'], ['bone density', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'breast adenocarcinoma', 'hepatocellular carcinoma'], ['Dopa-responsive dystonia due to sepiapterin reductase deficiency', 'Intellectual disability', 'Dystonia', 'mental retardation', 'genetic disorder'], ['ovarian cancer', 'clear cell renal carcinoma', 'neoplasm', 'gonorrhea', 'psoriasis'], ['Hereditary elliptocytosis', 'Hereditary spherocytosis', 'elliptocytosis 2', 'Pyropoikilocytosis', 'pyropoikilocytosis, hereditary'], ['Hereditary spherocytosis', 'Hereditary elliptocytosis', 'hemolytic anemia', 'Pyropoikilocytosis', 'red blood cell distribution width'], ['mean platelet volume', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement', 'neutrophil percentage of leukocytes', 'hematocrit'], ['neoplasm', 'breast cancer', 'obesity', 'nonpapillary renal cell carcinoma', 'hepatocellular carcinoma'], ['platelet count', 'electrocardiography', 'left ventricular structural measurement', 'left ventricular systolic function measurement', 'left ventricular ejection fraction measurement'], ['Kallmann syndrome', 'white matter microstructure measurement', 'chronic obstructive pulmonary disease', 'glioma', 'classic Hodgkin lymphoma'], ['neoplasm', 'cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'lymphoid neoplasm'], ['Ichthyosis follicularis - alopecia - photophobia', 'BRESEK syndrome', 'Hereditary mucoepithelial dysplasia', 'sex hormone-binding globulin measurement', 'body height'], ['birth weight', 'serum alanine aminotransferase measurement', 'breast adenocarcinoma', 'lean body mass', 'Posterior cortical atrophy'], ['cancer', 'gonorrhea', 'prostate cancer', 'calf circumference measurement', 'cholangiocarcinoma'], ['breast adenocarcinoma', 'eosinophil percentage of leukocytes', 'neoplasm', 'eosinophil count', 'glioblastoma multiforme'], ['genetic disorder', 'appendicular lean mass', 'cancer', 'Transient neonatal diabetes mellitus', 'Hyperinsulinism due to glucokinase deficiency'], ['refractive error measurement', 'lean body mass', 'heel bone mineral density', 'appendicular lean mass', 'mean corpuscular volume'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'lymphoid neoplasm', 'leukemia'], ['mean corpuscular volume', 'monocyte count', 'lymphocyte count', 'red blood cell distribution width', 'leukocyte count'], ['vein disease', 'lymphatic system disease', 'varicose disease', 'Varicose veins', 'Hernia'], ['chronic lymphocytic leukemia', 'cancer', 'non-small cell lung carcinoma', 'chronic myelogenous leukemia', 'colorectal cancer'], ['bile duct carcinoma', 'Hepatobiliary Neoplasm', 'bile duct adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'hepatocellular carcinoma'], ['erythrocyte count', 'platelet crit', 'mean corpuscular volume', 'mean reticulocyte volume', 'mean corpuscular hemoglobin'], ['lung adenocarcinoma', 'mathematical ability', 'glioblastoma multiforme', 'low grade glioma', 'oral cavity carcinoma'], ['hepatocellular carcinoma', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'lung cancer', 'neoplasm'], ['46,XY complete gonadal dysgenesis', '46,XX testicular disorder of sex development', '46,XX ovotesticular disorder of sex development', '46,XY partial gonadal dysgenesis', '46,xx sex reversal 5'], ['platelet count', 'grip strength measurement', 'cancer', 'platelet crit', 'lung cancer'], ['HIV infection', 'breast adenocarcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['age at menarche', 'glioblastoma multiforme', 'breast adenocarcinoma', 'multiple sclerosis', 'temporal arteritis'], ['neoplasm', 'acromegaly', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['neoplasm', 'acromegaly', 'neuroendocrine neoplasm', 'obesity', 'carcinoid syndrome'], ['acromegaly', 'neoplasm', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['acromegaly', 'neoplasm', 'neuroendocrine neoplasm', 'carcinoid syndrome', 'obesity'], ['synovial sarcoma', 'rhabdomyosarcoma', 'undifferentiated pleomorphic sarcoma', 'malignant peripheral nerve sheath tumor', 'Extraskeletal Ewing Sarcoma/Peripheral Primitive Neuroectodermal Tumor'], ['neoplasm', 'synovial sarcoma', 'nan', 'nan', 'nan'], ['COVID-19', 'mean platelet volume', 'alkaline phosphatase measurement', 'low density lipoprotein cholesterol measurement', 'severe acute respiratory syndrome'], ['refractive error measurement', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'ovarian cancer', 'neoplasm'], ['COVID-19', 'severe acute respiratory syndrome', 'lung adenocarcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma'], ['vitamin D measurement', 'low density lipoprotein cholesterol measurement', 'C-reactive protein measurement', 'Nephronophthisis', 'basophil percentage of leukocytes'], ['Ichthyosis-hypotrichosis syndrome', 'HIV-1 infection', 'viral load', 'Autosomal recessive congenital ichthyosis', 'glioblastoma multiforme'], ['hemoglobin measurement', 'unipolar depression', 'response to placebo', 'red blood cell density measurement', 'adolescent idiopathic scoliosis'], ['intellectual disability, autosomal dominant 47', 'Autosomal dominant non-syndromic intellectual disability', 'mental retardation', 'stag1-related disorder', 'genetic disorder'], ['Mendelian susceptibility to mycobacterial diseases due to partial STAT1 deficiency', 'Chronic mucocutaneous candidosis', 'Autoimmune enteropathy and endocrinopathy-susceptibility to chronic infections syndrome', 'Susceptibility to viral and mycobacterial infections', 'Combined T and B cell immunodeficiency'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'Congenital intrauterine infection-like syndrome', 'measles', 'lung adenocarcinoma', 'thyroid gland adenocarcinoma'], ['Autosomal dominant hyper-IgE syndrome', 'autoimmune disease', 'Hyper-IgE syndrome', 'cancer', 'Permanent neonatal diabetes mellitus'], ['cancer', 'acute myeloid leukemia', 'lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer'], ['Laron syndrome with immunodeficiency', 'cancer', 'acute myeloid leukemia', 'Combined T and B cell immunodeficiency', 'growth hormone insensitivity syndrome with immune dysregulation 2, autosomal dominant'], ['solitary fibrous tumor', 'diffuse large B-cell lymphoma', 'non-Hodgkins lymphoma', 'spindle cell tumor', 'hemangioblastoma'], ['arterial stiffness measurement', 'coffee consumption measurement', 'coffee consumption', 'diastolic blood pressure', 'type II diabetes mellitus'], ['type II diabetes mellitus', 'pulse pressure measurement', 'arterial stiffness measurement', 'coffee consumption measurement', 'coffee consumption'], ['osteoarthritis, hip', 'risk-taking behaviour', 'erythrocyte count', 'mean corpuscular volume', 'vital capacity'], ['Tubular aggregate myopathy', 'Combined immunodeficiency due to STIM1 deficiency', 'Stormorken-Sjaastad-Langslet syndrome', 'Combined immunodeficiency due to CRAC channel dysfunction', 'Combined T and B cell immunodeficiency'], ['STING-associated vasculopathy with onset in infancy', 'familial chilblain lupus', 'esophageal squamous cell carcinoma', 'anti-merkel cell virus antibody measurement', 'merkel cell virus seropositivity'], ['Combined immunodeficiency due to STK4 deficiency', 'Combined T and B cell immunodeficiency', 'self reported educational attainment', 'mean platelet volume', 'bipolar disorder'], ['cutaneous melanoma', 'cutaneous squamous cell carcinoma', 'blood protein measurement', 'gp41 C34 peptide, HIV measurement', 'psoriatic arthritis'], ["Parkinson's disease", 'lean body mass', 'body mass index', 'type I diabetes mellitus', 'prostate carcinoma'], ['BMI-adjusted waist-hip ratio', 'hemoglobin measurement', 'hematocrit', 'neoplasm', 'breast cancer'], ['protein measurement', 'ependymoma', 'astrocytoma', 'atypical teratoid rhabdoid tumor', 'colorectal cancer'], ['body mass index', 'peak expiratory flow', 'insomnia measurement', 'insomnia', 'type II diabetes mellitus'], ['appendicular lean mass', 'body fat percentage', 'visceral adipose tissue measurement', 'sleep duration', 'protein measurement'], ['Recessive X-linked ichthyosis', 'Syndromic X-linked ichthyosis', 'x-linked ichthyosis with steryl-sulfatase deficiency', 'Autosomal recessive congenital ichthyosis', 'genetic disorder'], ['STT3A-CDG', 'COVID-19', 'Intellectual disability', 'developmental disability', 'mean corpuscular hemoglobin concentration'], ['Cystic fibrosis', 'botulism', 'triglyceride measurement', 'clear cell renal carcinoma', 'cutaneous melanoma'], ['von Willebrand factor measurement', 'tissue plasminogen activator measurement', 'infect', 'infection', 'hemolytic-uremic syndrome'], ['asthma', 'white matter microstructure measurement', 'protein measurement', 'bone quantitative ultrasound measurement', 'body height'], ['breast adenocarcinoma', 'ovarian carcinoma', 'squamous cell lung carcinoma', 'glioblastoma multiforme', 'lung adenocarcinoma'], ['non-small cell lung carcinoma', 'neoplasm', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm'], ['breast carcinoma', 'PR interval', 'neoplasm', 'breast cancer', 'non-small cell lung carcinoma'], ['Mitochondrial DNA depletion syndrome, encephalomyopathic form with methylmalonic aciduria', 'Mitochondrial DNA depletion syndrome, encephalomyopathic form', 'Mitochondrial DNA depletion syndrome', 'Leigh syndrome', 'mean corpuscular hemoglobin'], ['childhood onset asthma', 'hepatocellular carcinoma', 'eosinophil count', 'celiac disease', 'red blood cell distribution width'], ['risk-taking behaviour', 'obese body mass index status', 'C-reactive protein measurement', 'breast cancer', 'lip and oral cavity carcinoma'], ['sex hormone-binding globulin measurement', 'heel bone mineral density', 'breast cancer', 'Uterine leiomyoma', 'uterine fibroid'], ['vitamin D measurement', 'blood metabolite measurement', 'cholelithiasis', 'triglyceride measurement', 'sleep duration'], ['Multiple sulfatase deficiency', 'heel bone mineral density', 'self reported educational attainment', 'high density lipoprotein cholesterol measurement', 'genetic disorder'], ['platelet component distribution width', 'rheumatoid arthritis', 'acute myeloid leukemia', 'Spina bifida - hypospadias', 'Craniorachischisis'], ['Imagawa-Matsumoto syndrome', 'prostate adenocarcinoma', 'Weaver syndrome', 'breast carcinoma', 'viral disease'], ['epilepsy', 'Seizure', 'Bilateral tonic-clonic seizure', 'botulism', "Parkinson's disease"], ['neurodevelopmental disorder with ataxia, hypotonia, and microcephaly', 'Intellectual disability', 'Lower limb spasticity', 'Microcephaly', 'mental retardation'], ['sprain', 'muscle strain', 'dislocation', 'ankle brachial index', 'female infertility'], ['habitual abortion', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation', 'head and neck squamous cell carcinoma', 'male infertility', 'acute myeloid leukemia'], ['cutaneous Leishmaniasis', 'autoimmune thrombocytopenic purpura', 'arthritis', 'colitis', 'Primary immunodeficiency due to a defect in adaptive immunity'], ['Atypical juvenile parkinsonism', 'Early infantile epileptic encephalopathy', 'genetic disorder', 'undetermined early-onset epileptic encephalopathy', 'Young adult-onset Parkinsonism'], ['intraocular pressure measurement', 'platelet component distribution width', 'age-related hearing impairment', 'glioblastoma multiforme', 'breast adenocarcinoma'], ['visual perception measurement', 'FEV/FEC ratio', 'Ehlers-Danlos syndrome, progeroid type', 'melanoma', 'neoplasm'], ['Non-steroidal anti-inflammatory and antirheumatic product use measurement', 'hematocrit', 'red blood cell density measurement', 'hemoglobin measurement', 'myeloid white cell count'], ['chronotype measurement', 'mathematical ability', 'body mass index', 'drug use measurement', 'astrocytoma'], ['X-linked non-syndromic intellectual disability', 'mental retardation', 'genetic disorder', 'neoplasm', 'breast cancer'], ['infantile hypotonia-oculomotor anomalies-hyperkinetic movements-developmental delay syndrome', 'botulism', 'Intellectual disability', 'cognitive function measurement', 'Autosomal recessive non-syndromic intellectual disability'], ['Congenital myasthenic syndromes', 'Presynaptic congenital myasthenic syndromes', 'botulism', 'acute myeloid leukemia', 'Muscle weakness'], ['Autosomal dominant secondary polycythemia', 'blood pressure', 'Primary familial polycythemia', 'Wiskott-Aldrich syndrome', 'diabetes mellitus, permanent neonatal 4'], ['glioblastoma multiforme', 'lung adenocarcinoma', 'cutaneous melanoma', 'endometrial cancer', 'esophageal cancer'], ['lung adenocarcinoma', 'cutaneous melanoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'neuroblastoma'], ['pancreatic adenocarcinoma', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'breast cancer'], ['risk-taking behaviour', 'feeling nervous measurement', 'anxiety', 'emotional symptom measurement', 'social interaction measurement'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Isolated congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'Delayed puberty', 'hypogonadotropic hypogonadism'], ['osteoarthritis, knee', 'mean reticulocyte volume', 'gait measurement', 'body height', 'joint disease'], ['Chemotherapy-induced nausea and vomiting', 'post operative nausea and vomiting', 'ovarian cancer', 'cancer', 'Vomiting'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Kallmann syndrome', 'Isolated congenital hypogonadotropic hypogonadism', 'age at menarche', 'testosterone measurement'], ['Gelatinous drop-like corneal dystrophy', 'triple-negative breast cancer', 'breast cancer', 'metastatic malignant neoplasm', 'prostate cancer'], ['breast adenocarcinoma', 'lymphocyte count', 'monocyte count', 'hematocrit', 'platelet count'], ['Microcephaly', 'Partial agenesis of the corpus callosum', 'Severe global developmental delay', 'lymphocyte count', 'platelet count'], ['blood protein measurement', 'red blood cell distribution width', 'cerebrospinal fluid biomarker measurement', 'triglyceride measurement', 'proprotein convertase subtilisin/kexin type 7 measurement'], ['T-cell acute lymphoblastic leukemia', 'Cutaneous T-cell lymphoma', 'B-cell acute lymphoblastic leukemia', 'unspecified peripheral T-cell lymphoma', 'mean corpuscular volume'], ['Immunodeficiency by defective expression of HLA class 1', 'Immunodeficiency by defective expression of HLA class 2', 'inflammatory bowel disease', 'neoplasm', 'melanoma'], ['Immunodeficiency by defective expression of HLA class 1', 'Immunodeficiency by defective expression of HLA class 2', 'neoplasm', 'body mass index', 'Reticular dysgenesis'], ['Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Frontotemporal dementia', 'motor neuron disease'], ['sensory perception of bitter taste', 'alcohol consumption measurement', 'diet measurement', 'tea consumption measurement', 'bitter alcoholic beverage consumption measurement'], ['type II diabetes mellitus', 'mean platelet volume', 'mean corpuscular volume', 'erythrocyte count', 'mean corpuscular hemoglobin'], ['neutrophil count', 'myeloid white cell count', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'leukocyte count'], ['platelet count', 'platelet crit', 'attention deficit hyperactivity disorder', 'neoplasm', 'hepatocellular carcinoma'], ['Micro syndrome', 'mental retardation', 'developmental disability', 'Hepatitis', 'Seizure'], ['DOORS syndrome', 'Familial infantile myoclonic epilepsy', "Rolandic epilepsy - paroxysmal exercise-induced dystonia - writer's cramp", 'Malignant migrating partial seizures of infancy', 'Progressive myoclonic epilepsy with dystonia'], ['mean platelet volume', 'vital capacity', 'platelet component distribution width', 'electrocardiography', 'sitting height measurement'], ['head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'chronic lymphocytic leukemia'], ['Sanjad-Sakati syndrome', 'Kenny-Caffey syndrome', 'Autosomal recessive Kenny-Caffey syndrome', 'encephalopathy, progressive, with amyotrophy and optic atrophy', 'genetic disorder'], ['Frontotemporal dementia with motor neuron disease', 'Herpetic encephalitis', 'amyotrophic lateral sclerosis', 'Frontotemporal dementia', 'Pick disease'], ['hypothyroidism, congenital, nongoitrous', 'viral disease', 'Central congenital hypothyroidism', 'breast adenocarcinoma', 'eosinophilic pneumonia'], ['mean corpuscular hemoglobin', 'C-reactive protein measurement', 'arterial stiffness measurement', 'neoplasm', 'colorectal carcinoma'], ['22q11.2 deletion syndrome', 'Conotruncal heart malformations', 'Tetralogy of Fallot', 'Shprintzen-Goldberg syndrome', 'Double outlet right ventricle'], ['Coxopodopatellar syndrome', 'pulmonary arterial hypertension', 'Autosomal recessive amelia', 'Heritable pulmonary arterial hypertension', 'Idiopathic pulmonary arterial hypertension'], ['Sacral agenesis-abnormal ossification of the vertebral bodies-persistent notochordal canal syndrome', 'Chordoma', 'Neural tube closure defect', 'Isolated spina bifida', 'spina bifida'], ['neoplasm', 'hepatocellular carcinoma', 'cancer', 'cervical cancer', 'esophageal squamous cell carcinoma'], ['Autosomal agammaglobulinemia', 'Isolated agammaglobulinemia', 'Agammaglobulinemia', 'primary immunodeficiency disease', 'acute lymphoblastic leukemia'], ['Pitt-Hopkins syndrome', 'Fuchs endothelial corneal dystrophy', 'schizophrenia', 'genetic disorder', 'risk-taking behaviour'], ['colorectal adenocarcinoma', 'type II diabetes mellitus', 'diabetes mellitus', 'breast carcinoma', 'body mass index'], ['birth weight', 'infect', 'infection', 'Autosomal recessive spondylocostal dysostosis', 'Poland syndrome'], ['type II diabetes mellitus', 'basal cell carcinoma', 'chronic hepatitis B virus infection', 'toxic epidermal necrolysis', 'Stevens-Johnson syndrome'], ['Developmental delay with variable intellectual impairment and behavioural abnormalities', 'Intellectual disability', 'developmental delay with variable intellectual impairment and behavioral abnormalities', 'Neurodevelopmental abnormality', 'genetic disorder'], ['pulse pressure measurement', 'systolic blood pressure', 'platelet count', 'eosinophil count', 'cardiovascular disease'], ['chronic lymphocytic leukemia', 'lymphoid neoplasm', 'mosaic loss of chromosome Y measurement', 'platelet crit', 'platelet count'], ['Protein R deficiency', 'vitamin B12 measurement', 'pernicious anemia', 'head and neck squamous cell carcinoma', 'hypertrophic cardiomyopathy'], ['Treacher-Collins syndrome', 'Crouzon disease', 'Microcephaly', 'lumbar disc degeneration', 'Isolated anencephaly/exencephaly'], ['Hypertryptophanemia', 'blood metabolite measurement', 'hypospadias', 'hepatocellular carcinoma', 'myeloid white cell count'], ['Mucocutaneous venous malformations', 'Congenital glaucoma', 'neoplasm', 'metastatic colorectal cancer', 'thyroid cancer'], ['TELO2-related intellectual disability-neurodevelopmental disorder', 'genetic disorder', 'rheumatoid arthritis', 'Microcephaly', 'Niemann-Pick disease type C'], ['Agammaglobulinemia', 'glioblastoma multiforme', 'neoplasm', 'Acute hepatic failure', 'papillary thyroid carcinoma'], ['cancer', 'age at menopause', 'body height', 'multiple sclerosis', 'sphingolipid measurement'], ['vital capacity', 'lymphocyte percentage of leukocytes', 'forced expiratory volume', 'neutrophil count', 'leukocyte count'], ['eosinophil count', 'chronic obstructive pulmonary disease', 'infect', 'infection', 'Hematuria'], ['head and neck squamous cell carcinoma', 'ovarian cancer', 'lung adenocarcinoma', 'esophageal cancer', 'squamous cell lung carcinoma'], ['Dyskeratosis congenita', 'pulmonary fibrosis and/or bone marrow failure, telomere-related, 1', 'idiopathic pulmonary fibrosis', 'Aplastic anemia', 'idiopathic aplastic anemia'], ['intraocular pressure measurement', 'glaucoma', 'body mass index', 'neoplasm', 'chronotype measurement'], ['mean platelet volume', 'self reported educational attainment', 'PR interval', 'lifestyle measurement', 'platelet crit'], ['myelodysplastic syndrome', 'prostate carcinoma', 'acute myeloid leukemia', 'breast carcinoma', 'lung carcinoma'], ['Congenital atransferrinemia', 'iron biomarker measurement', 'red blood cell distribution width', 'total iron binding capacity', 'alcohol drinking'], ['glioblastoma multiforme', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'ovarian cancer'], ['high density lipoprotein cholesterol measurement', 'neoplasm', 'hepatocellular carcinoma', 'chronic kidney disease', 'breast carcinoma'], ['Autosomal recessive spastic paraplegia type 57', 'Hereditary motor and sensory neuropathy, Okinawa type', 'Autosomal recessive complex spastic paraplegia', 'Charcot-Marie-Tooth disease type 2', 'anaplastic large cell lymphoma'], ['intracranial hemorrhage', 'postmenopausal atrophic vaginitis', 'lymphocyte count', 'lung adenocarcinoma', 'leukocyte count'], ['neoplasm', 'hepatocellular carcinoma', 'breast cancer', 'colorectal carcinoma', 'urinary bladder cancer'], ['low grade glioma', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'glioblastoma multiforme', 'esophageal squamous cell carcinoma'], ['Primary immunodeficiency due to a defect in adaptive immunity', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'Primary immunodeficiency', 'mean corpuscular hemoglobin'], ['Familial thyroid dyshormonogenesis', "Hashimoto's thyroiditis", 'hypothyroidism', 'Neonatal diabetes - congenital hypothyroidism - congenital glaucoma - hepatic fibrosis - polycystic kidneys', 'Congenital hypothyroidism'], ['cancer', 'grip strength measurement', 'osteoarthritis', 'tooth agenesis', 'Oligodontia'], ['Camurati-Engelmann disease', 'inflammatory bowel disease, immunodeficiency, and encephalopathy', 'cancer', 'Cystic fibrosis', 'Autosomal recessive early-onset inflammatory bowel disease'], ['Familial thoracic aortic aneurysm and aortic dissection', 'Loeys-Dietz syndrome', 'Holt-Oram syndrome', 'Abnormality of the cardiovascular system', 'BMI-adjusted waist circumference'], ['Lattice corneal dystrophy type I', 'Granular corneal dystrophy type I', 'Microcystic corneal dystrophy', 'Granular corneal dystrophy type II', 'Reis-Bücklers corneal dystrophy'], ['Loeys-Dietz syndrome', 'Familial thoracic aortic aneurysm and aortic dissection', 'Multiple keratoacanthoma, Ferguson-Smith type', 'cancer', 'Rare disease with thoracic aortic aneurysm and aortic dissection'], ['Loeys-Dietz syndrome', 'Familial thoracic aortic aneurysm and aortic dissection', 'Marfan syndrome', 'Marfan syndrome type 2', 'Lynch syndrome'], ['obesity', 'myeloid white cell count', 'protein measurement', 'platelet count', 'electrocardiography'], ['Uncombable hair syndrome', 'basal cell carcinoma', 'skin cancer', 'non-melanoma skin carcinoma', 'keratinocyte carcinoma'], ['self reported educational attainment', 'mathematical ability', 'eosinophil count', 'adolescent idiopathic scoliosis', 'lean body mass'], ['Autosomal recessive dopa-responsive dystonia', 'tyrosine hydroxylase deficiency', 'Dopa-responsive dystonia', 'Dystonia', 'Intellectual disability'], ['thrombomodulin-related bleeding disorder', 'Atypical hemolytic-uremic syndrome', 'Atypical hemolytic-uremic syndrome with thrombomodulin anomaly', 'Venous thrombosis', 'thrombomodulin measurement'], ['autism spectrum disorder', 'cortical surface area measurement', 'brain volume measurement', 'neuroimaging measurement', 'cortical thickness'], ['essential thrombocythemia', 'Congenital amegakaryocytic thrombocytopenia', 'Familial thrombocytosis', 'Hereditary isolated aplastic anemia', 'Hereditary thrombocytosis with transverse limb defect'], ['neoplasm', 'Androgen insensitivity syndrome', 'Åland Islands eye disease', 'breast cancer', 'hepatocellular carcinoma'], ['mathematical ability', 'glomerular filtration rate', 'neoplasm', 'hepatocellular carcinoma', 'liver cancer'], ['hair shape measurement', 'balding measurement', 'smoking status measurement', 'risk-taking behaviour', 'skin aging measurement'], ['breast adenocarcinoma', 'low grade glioma', 'ovarian cancer', 'late-onset Alzheimers disease', 'glioblastoma multiforme'], ['head and neck squamous cell carcinoma', 'endometrial cancer', 'cutaneous melanoma', 'Sepsis', 'neoplasm'], ['heel bone mineral density', 'body height', 'gut microbiome measurement', 'blood protein measurement', 'hip bone size'], ["Sorsby's fundus dystrophy", 'Retinal dystrophy', 'metalloproteinase inhibitor 3 measurement', 'macular degeneration', 'Cerebral arteriovenous malformation'], ['red blood cell distribution width', 'eosinophil count', 'triglyceride measurement', 'wellbeing measurement', 'type II diabetes mellitus'], ['body mass index', 'childhood onset asthma', 'body height', 'ovarian carcinoma', 'age at menopause'], ['Primary immunodeficiency', 'bacteremia, susceptibility', 'appendicular lean mass', 'neutrophil percentage of granulocytes', 'cutaneous melanoma'], ['Progressive familial intrahepatic cholestasis', 'Familial hypercholanemia', 'Progressive familial intrahepatic cholestasis type 2', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'genetic disorder'], ['Red cell distribution width', 'mean corpuscular volume', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin'], ['neoplasm', 'melanoma', 'cancer', 'glioma', 'Autosomal recessive early-onset inflammatory bowel disease'], ['self reported educational attainment', 'lymphocyte count', 'lymphocyte percentage of leukocytes', 'body mass index', 'balding measurement'], ['age at menopause', 'mean platelet volume', 'vital capacity', 'body height', 'appendicular lean mass'], ['Interauricular communication', 'Genetic cardiac anomaly', 'refractive error measurement', 'Atrial septal defect, ostium primum type', 'Atrial septal defect, ostium secundum type'], ['adolescent idiopathic scoliosis', 'breast adenocarcinoma', 'cutaneous melanoma', 'Proximal amyotrophy', 'Kennedy disease'], ['Primary immunodeficiency', 'childhood onset asthma', 'asthma', 'allergy', 'seasonal allergic rhinitis'], ['Tuberculosis', 'Primary immunodeficiency', 'lung adenocarcinoma', 'glioblastoma multiforme', 'cutaneous melanoma'], ['Herpetic encephalitis', "Hashimoto's thyroiditis", 'Primary immunodeficiency', 'HIV-1 infection', 'drug use measurement'], ['pneumonia', 'influenza', 'COVID-19', 'Primary immunodeficiency', 'toll-like receptor 4:Lymphocyte antigen 96 complex measurement'], ['Primary immunodeficiency', 'monocyte percentage of leukocytes', 'monocyte count', 'legionellosis', 'chronic lymphocytic leukemia'], ['Primary immunodeficiency', 'childhood onset asthma', 'asthma', 'allergy', 'lung adenocarcinoma'], ['rheumatoid arthritis', 'COVID-19', 'actinic keratosis', 'systemic lupus erythematosus', 'basal cell carcinoma'], ['osteoarthritis', 'chronic lymphocytic leukemia', 'ovarian cancer', 'neoplasm', 'cancer'], ['rheumatoid arthritis', 'COVID-19', 'systemic lupus erythematosus', 'obesity', 'IGA glomerulonephritis'], ['glioblastoma multiforme', 'Hydrolethalus', 'Hodgkins lymphoma', 'lactose intolerance adult type', 'Hirschsprung disease'], ['low density lipoprotein cholesterol measurement', 'type II diabetes mellitus', 'triglyceride measurement', 'vitamin D measurement', 'hematocrit'], ['protein measurement', 'vital capacity', 'intelligence', 'forced expiratory volume', 'smoking behaviour measurement'], ['breast adenocarcinoma', 'pancreatic adenocarcinoma', 'squamous cell lung carcinoma', 'cutaneous melanoma', 'infection'], ['central corneal thickness', 'testosterone measurement', 'breast cancer', 'neoplasm', 'cyst'], ['body mass index', 'visual perception measurement', 'platelet count', 'risk-taking behaviour', 'Helicobacter pylori infectious disease'], ['mean corpuscular hemoglobin', 'mean corpuscular volume', 'Blackfan-Diamond anemia', 'Microcytic anemia with liver iron overload', 'Alpha-thalassemia - myelodysplastic syndrome'], ['neoplasm', 'Hereditary renal hypouricemia', 'protein measurement', 'cerebellar ataxia', 'Ataxia'], ['neoplasm', 'breast cancer', 'endometrial cancer', 'cancer', 'leiomyoma'], ['HbA1c measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'hemoglobin measurement', 'Uncombable hair syndrome'], ['cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'age at menarche', 'lean body mass'], ['prostate carcinoma', 'COVID-19', 'benign prostatic hyperplasia', 'prostate adenocarcinoma', 'severe acute respiratory syndrome'], ['Autosomal recessive cerebral atrophy', 'non-small cell lung carcinoma', 'neoplasm', 'cancer', 'breast cancer'], ['IRIDA syndrome', 'Hypochromic microcytic anemia', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'red blood cell distribution width'], ['reticulocyte count', 'idiopathic pulmonary fibrosis', 'non-small cell lung carcinoma', 'infect', 'infection'], ['total cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'triglyceride measurement', 'chronic kidney disease', 'urinary metabolite measurement'], ['DNA methylation', 'neoplasm', 'pes', 'prostate cancer', 'colorectal carcinoma'], ['Congenital enteropathy due to enteropeptidase deficiency', 'self reported educational attainment', 'acute myeloid leukemia', 'anemia (disease)', 'bipolar disorder'], ['neoplasm', 'breast cancer', 'nonpapillary renal cell carcinoma', 'colorectal carcinoma', 'lung adenocarcinoma'], ['hepatocellular carcinoma', 'Severe combined immunodeficiency due to CORO1A deficiency', 'attention deficit hyperactivity disorder', 'lean body mass', 'vital capacity'], ['Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'deafness', 'blood protein measurement', 'tenascin measurement', 'protein measurement'], ['psoriatic arthritis', 'rheumatoid arthritis', 'psoriasis', "Crohn's disease", 'ankylosing spondylitis'], ['breast cancer', 'cancer', 'folate deficiency', 'sporadic amyotrophic lateral sclerosis', 'serine/threonine-protein kinase 17B measurement'], ['cathepsin L1 measurement', 'inflammatory bowel disease', 'glioma', 'gonorrhea', 'esophageal squamous cell carcinoma'], ['Autoimmune lymphoproliferative syndrome', 'Hodgkins lymphoma', 'diffuse large B-cell lymphoma', 'systemic lupus erythematosus', 'rheumatoid arthritis'], ['Tumor necrosis factor receptor 1 associated periodic syndrome', 'multiple sclerosis', "Behcet's syndrome", 'lymphocyte count', 'CHARGE syndrome'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'tumor necrosis factor receptor II measurement', 'glioblastoma multiforme'], ['Primary immunodeficiency', 'Primary immunodeficiency due to a defect in adaptive immunity', 'Combined T and B cell immunodeficiency', 'neoplasm', 'systemic lupus erythematosus'], ['ulcerative colitis', "Crohn's disease", 'inflammatory bowel disease', 'psoriasis', 'ankylosing spondylitis'], ['Hodgkins lymphoma', 'anaplastic large cell lymphoma', 'lymphoma', 'neoplasm', 'eosinophil count'], ['rheumatoid arthritis', 'hair color', 'diffuse large B-cell lymphoma', 'ulcerative colitis', 'pancreatic adenocarcinoma'], ['age-related macular degeneration', 'monocyte count', 'central serous retinopathy', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['mean corpuscular volume', 'TNF-related apoptosis-inducing ligand receptor 2 measurement', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'esophageal cancer'], ['Juvenile Paget disease', 'eosinophil count', 'bone density', 'alkaline phosphatase measurement', 'eosinophil percentage of leukocytes'], ['breast cancer', 'hepatocellular carcinoma', 'rheumatoid arthritis', 'neoplasm', 'melanoma'], ['Common variable immunodeficiency', 'immunodeficiency, common variable, 2', 'Primary immunodeficiency', 'selective IgA deficiency disease', 'multiple myeloma'], ['Common variable immunodeficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'coronary artery disease', 'immunodeficiency, common variable, 2', 'atherosclerosis'], ['diffuse large B-cell lymphoma', 'neoplasm of mature B-cells', 'basal cell carcinoma', 'lymphoid neoplasm', 'lung carcinoma'], ['multiple myeloma', 'neoplasm', 'pancreatic carcinoma', 'cutaneous melanoma', 'gastric carcinoma'], ['drug use measurement', 'neoplasm', 'Miyoshi myopathy', 'systemic lupus erythematosus', 'infect'], ['mean platelet volume', 'breast adenocarcinoma', 'platelet count', 'head and neck squamous cell carcinoma', 'neoplasm'], ['Autosomal recessive lower motor neuron disease with childhood onset', 'Distal spinal muscular atrophy', 'lung cancer', 'colitis', 'prostate neoplasm'], ['systemic lupus erythematosus', 'asthma', 'rheumatoid arthritis', 'allergy', 'Eczema'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'inflammatory bowel disease', "Crohn's disease", 'sum of eosinophil and basophil counts'], ['neoplasm', 'colorectal adenocarcinoma', 'colon carcinoma', 'colonic neoplasm', 'malignant colon neoplasm'], ['TNF-related apoptosis-inducing ligand measurement', 'sex hormone-binding globulin measurement', 'monocyte percentage of leukocytes', 'platelet count', 'monocyte count'], ['Autosomal recessive malignant osteopetrosis', 'autosomal recessive osteopetrosis 2', 'osteoporosis', 'postmenopausal osteoporosis', 'giant cell tumor'], ['serum non-albumin protein measurement', 'alcohol consumption measurement', 'IGA glomerulonephritis', 'blood protein measurement', 'albumin:globulin ratio measurement'], ['systemic lupus erythematosus', 'monocyte count', 'platelet count', 'myeloid white cell count', 'neutrophil count'], ['multiple sclerosis', 'tumor necrosis factor ligand superfamily member 14 measurement', 'monocyte count', 'blood protein measurement', 'monocyte percentage of leukocytes'], ["Crohn's disease", 'inflammatory bowel disease', 'ulcerative colitis', 'psoriasis', 'ankylosing spondylitis'], ["Crohn's disease", 'celiac disease', 'ulcerative colitis', 'ankylosing spondylitis', 'psoriasis'], ['Autosomal recessive non-syndromic intellectual disability', 'body mass index', 'lymphocyte count', 'lymphocyte percentage of leukocytes', 'arterial stiffness measurement'], ['Frontotemporal dementia', 'Infantile myofibromatosis', 'reticulum cell sarcoma', 'Familial atypical multiple mole melanoma syndrome', 'Epidermodysplasia verruciformis'], ['Infantile-onset mesial temporal lobe epilepsy with severe cognitive regression', "Parkinson's disease", 'hematocrit', 'mean corpuscular hemoglobin', 'breast cancer'], ['cancer', 'alcohol drinking', 'systolic blood pressure', 'ovarian cancer', 'mathematical ability'], ['sex hormone-binding globulin measurement', 'cancer', 'testosterone measurement', 'neutrophil count', 'alkaline phosphatase measurement'], ['Arthrogryposis - hyperkeratosis, lethal form', 'Pelvic dysplasia - arthrogryposis of lower limbs', 'age-related macular degeneration', 'obesity', 'diabetes mellitus'], ['mean corpuscular hemoglobin concentration', 'mean platelet volume', 'red blood cell distribution width', 'platelet count', 'hemoglobin measurement'], ['reticulocyte count', 'chronic obstructive pulmonary disease', 'forced expiratory volume', 'breast carcinoma', 'vital capacity'], ['reticulocyte count', 'prostate carcinoma', 'reticulocyte measurement', 'basal cell carcinoma', 'myeloid white cell count'], ['acute myeloid leukemia', 'neoplasm', 'breast cancer', 'acute lymphoblastic leukemia', 'adult acute lymphoblastic leukemia'], ['Autosomal recessive limb-girdle muscular dystrophy', 'Abnormality of refraction', 'prostate carcinoma', 'hepatocellular carcinoma', 'mean corpuscular volume'], ['Li-Fraumeni syndrome', 'head and neck squamous cell carcinoma', 'osteosarcoma', 'colorectal adenocarcinoma', 'basal cell carcinoma'], ['EEC syndrome', 'Ankyloblepharon - ectodermal defects - cleft lip/palate', 'ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3', 'ADULT syndrome', 'Split hand-split foot malformation'], ['acute myeloid leukemia', 'ovarian cancer', 'lung adenocarcinoma', 'urinary bladder cancer', 'gastric adenocarcinoma'], ['lean body mass', 'fat body mass', 'body mass index', 'body weight', 'base metabolic rate measurement'], ['hemoglobin measurement', 'hematocrit', 'red blood cell density measurement', 'drug use measurement', 'asthma'], ['reaction time measurement', 'risk-taking behaviour', 'hypertrophic cardiomyopathy', 'waist-hip ratio', 'smoking status measurement'], ['Triose phosphate-isomerase deficiency', 'multiple myeloma', 'acute myeloid leukemia', 'B-cell non-Hodgkins lymphoma', 'endometrial cancer'], ['Digitotalar dysmorphism', 'Nemaline myopathy', 'Cap myopathy', 'Distal arthrogryposis', 'Sheldon-Hall syndrome'], ['Congenital fiber-type disproportion myopathy', 'Nemaline myopathy', 'Cap myopathy', 'anaplastic large cell lymphoma', 'papillary thyroid carcinoma'], ['Familial thyroid dyshormonogenesis', 'Abnormality of the thyroid gland', 'Graves disease', 'Graves ophthalmopathy', 'nodular goiter'], ['CLN2 disease', 'Childhood-onset autosomal recessive slowly progressive spinocerebellar ataxia', 'Neuronal ceroid lipofuscinosis', 'Juvenile neuronal ceroid lipofuscinosis', 'Late infantile neuronal ceroid lipofuscinosis'], ['glomerular filtration rate', 'appendicular lean mass', 'diabetes mellitus', 'Abnormal urine potassium concentration', 'peptic esophagitis'], ['HIV infection', 'influenza', 'viral disease', 'papillary thyroid carcinoma', 'Benign Thyroid Gland Neoplasm'], ['blood protein measurement', 'systemic mastocytosis', 'appendicular lean mass', 'tryptase beta-2 measurement', 'DNA methylation'], ['eosinophil count', 'response to drug', 'hypertension', 'appendicular lean mass', 'myeloid white cell count'], ['red blood cell density measurement', 'low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'glioblastoma multiforme'], ['platelet count', 'platelet crit', 'eosinophil count', 'mean platelet volume', 'alcohol abuse'], ['rheumatoid arthritis', 'lymphocyte count', 'myeloid white cell count', 'asthma', 'leukocyte count'], ['genetic disorder', 'ovarian cancer', 'breast adenocarcinoma', 'lung adenocarcinoma', 'ovarian carcinoma'], ['Herpetic encephalitis', 'herpes simplex encephalitis, susceptibility to, 3', 'Primary immunodeficiency', 'multiple myeloma', 'lean body mass'], ['Chronic mucocutaneous candidosis', 'rheumatoid arthritis', 'psoriasis', 'discoid lupus erythematosus', 'psoriatic arthritis'], ['Autosomal dominant hypohidrotic ectodermal dysplasia', 'glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'esophageal cancer'], ['Symbol', 'nan', 'nan', 'nan', 'nan'], ['blood protein measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Sepsis'], ['Nasu-Hakola disease', 'dementia (disease)', 'dystonic disorder', 'C-reactive protein measurement', 'lung adenocarcinoma'], ['mean corpuscular volume', 'acute myeloid leukemia', 'blood protein measurement', 'monocyte count', 'leukocyte count'], ['monocyte count', 'myeloid white cell count', 'leukocyte count', 'blood protein measurement', 'reticulocyte count'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil count', 'myeloid white cell count', 'neutrophil percentage of granulocytes'], ['Aicardi-Goutières syndrome', 'systemic lupus erythematosus', 'Chilblain lupus', 'Retinal vasculopathy and cerebral leukodystrophy', 'Hereditary vascular retinopathy'], ['acute myeloid leukemia', 'neoplasm', 'brucellosis', 'hepatocellular carcinoma', 'T-cell acute lymphoblastic leukemia'], ['sex hormone-binding globulin measurement', 'body height', 'glioblastoma multiforme', 'ovarian cancer', 'head and neck squamous cell carcinoma'], ['neoplasm', 'X-linked retinal dysplasia', 'Ehlers-Danlos syndrome, progeroid type', 'rheumatoid arthritis', 'refractory anemia'], ['lung adenocarcinoma', 'ovarian cancer', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma'], ['schizophrenia', 'thyroid gland adenocarcinoma', 'autism spectrum disorder', 'bipolar disorder', 'unipolar depression'], ['Autosomal recessive limb-girdle muscular dystrophy type 2H', 'Bardet-Biedl syndrome', 'Autosomal recessive limb-girdle muscular dystrophy', 'Limb-girdle muscular dystrophy', 'myopathy'], ['clear cell renal carcinoma', 'risk-taking behaviour', 'mosquito bite reaction itch intensity measurement', 'colorectal carcinoma', 'glioma'], ['spinal muscular atrophy with congenital bone fractures 1', 'prenatal-onset spinal muscular atrophy with congenital bone fractures', 'amyotrophic lateral sclerosis', 'axial length measurement', "Alzheimer's disease"], ['glioblastoma multiforme', 'neoplasm', 'Hyper-IgM syndrome type 3', 'Hepatic veno-occlusive disease - immunodeficiency', 'Hyper-IgM syndrome type 5'], ['combined oxidative phosphorylation deficiency 35', 'developmental disability', 'genetic disorder', 'Macrocephaly', 'Epileptic encephalopathy'], ['mean platelet volume', 'platelet count', 'thyroid cancer', 'thyroid carcinoma', 'platelet component distribution width'], ['Familial episodic pain syndrome with predominantly upper body involvement', 'arthritis', 'pain', 'Familial episodic pain syndrome', 'Myalgia'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'glioblastoma multiforme', 'ovarian cancer'], ['Spinocerebellar ataxia type 41', 'Autosomal dominant cerebellar ataxia', 'childhood onset asthma', 'acute myeloid leukemia', 'breast adenocarcinoma'], ['smoking status measurement', 'smoking initiation', 'protein measurement', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'pancreatic adenocarcinoma', 'cutaneous melanoma', 'mental retardation'], ['focal segmental glomerulosclerosis', 'Familial idiopathic steroid-resistant nephrotic syndrome', 'response to prednisolone', 'myeloid white cell count', 'kidney failure'], ['Familial progressive cardiac conduction defect', 'Erythrokeratoderma variabilis progressiva', 'Progressive symmetric erythrokeratodermia', 'Erythrokeratodermia variabilis', 'sudden cardiac arrest'], ['pain', 'arthritis', 'Myalgia', 'frozen shoulder', 'Muscle spasm'], ['migraine disorder', 'pain', 'Fever', 'osteoarthritis', 'arthritis'], ['Mutilating palmoplantar keratoderma with periorificial keratotic plaques', 'Non-epidermolytic palmoplantar keratoderma', 'breast adenocarcinoma', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['Metatropic dysplasia', 'Spondylometaphyseal dysplasia, Kozlowski type', 'Autosomal dominant Charcot-Marie-Tooth disease type 2C', 'Autosomal dominant congenital benign spinal muscular atrophy', 'Familial digital arthropathy-brachydactyly'], ['hyperparathyroidism, transient neonatal', 'Neonatal severe primary hyperparathyroidism', 'hyperparathyroidism', 'anxiety measurement', 'glioblastoma multiforme'], ['developmental delay with or without dysmorphic facies and autism', 'deafness, autosomal dominant 75', 'Global developmental delay', 'autism', 'Intellectual disability'], ['Tuberous sclerosis', 'Lymphangioleiomyomatosis', 'Isolated focal cortical dysplasia type II', 'urinary bladder cancer', 'Inherited cancer-predisposing syndrome'], ['HIV infection', 'viral disease', 'infect', 'infection', 'hepatocellular carcinoma'], ['Hypothyroidism due to TSH receptor mutations', 'Familial hyperthyroidism due to mutations in TSH receptor', 'Familial gestational hyperthyroidism', 'thyroid neoplasm', 'Thyroid hypoplasia'], ['Abnormality of the ear', 'protein measurement', 'endometrial neoplasm', 'colorectal cancer', 'lymphocyte count'], ['vitamin D measurement', 'factor VII measurement', 'self reported educational attainment', 'osteoarthritis', 'body mass index'], ['asthma', 'eosinophil count', 'Nasal Cavity Polyp', 'childhood onset asthma', 'Eczema'], ['heart failure', 'smoking status measurement', 'response to drug', 'hypertension', 'overweight body mass index status'], ['intellectual disability, X-linked 58', 'X-linked non-syndromic intellectual disability', 'mental retardation', 'breast adenocarcinoma', 'Intellectual disability'], ['Familial exudative vitreoretinopathy', 'bone density', 'Norrie disease', 'Persistent hyperplastic primary vitreous', 'Vitreoretinopathy'], ['hematocrit', 'hemoglobin measurement', 'urate measurement', 'systemic lupus erythematosus', 'glomerular filtration rate'], ['monocyte count', 'myeloid white cell count', 'leukocyte count', 'monocyte percentage of leukocytes', 'neutrophil count'], ['attention deficit hyperactivity disorder', 'intellectual disability, autosomal dominant 45', 'X-linked dominant intellectual disability - epilepsy syndrome', 'X-linked retinal dysplasia', 'Severe early-childhood-onset retinal dystrophy'], ['gastric carcinoma', 'lung adenocarcinoma', 'colorectal carcinoma', 'risky sexual behaviour measurement', 'appendicular lean mass'], ['macula measurement', 'pancreatic neoplasm', 'hepatocellular carcinoma', 'neoplasm', 'glioblastoma multiforme'], ['Tibial muscular dystrophy', 'Early-onset myopathy with fatal cardiomyopathy', 'Hereditary proximal myopathy with early respiratory failure', 'Familial dilated cardiomyopathy', 'dilated cardiomyopathy'], ['Ataxia with vitamin E deficiency', 'hereditary ataxia', 'diet measurement', 'blood protein measurement', 'C-reactive protein measurement'], ['ATTRV122I amyloidosis', 'Familial amyloid polyneuropathy', 'Familial transthyretin-related amyloidosis', 'familial amyloid neuropathy', 'cardiomyopathy'], ['Retinitis pigmentosa', 'autism spectrum disorder', 'diet measurement', 'hearing loss', 'age-related hearing impairment'], ['Congenital fibrosis of extraocular muscles', 'Cortical dysgenesis with pontocerebellar hypoplasia due to TUBB3 mutation', 'head and neck squamous cell carcinoma', 'ovarian cancer', 'pancreatic adenocarcinoma'], ['Autosomal recessive chorioretinopathy-microcephaly', 'ovarian cancer', 'breast adenocarcinoma', 'glycine measurement', 'eosinophil count'], ['Autosomal recessive chorioretinopathy-microcephaly', 'Microcephaly - lymphedema - chorioretinopathy', 'Autosomal recessive primary microcephaly', 'Isolated congenital microcephaly', 'ovarian cancer'], ['red blood cell distribution width', 'body height', 'FEV/FEC ratio', 'sitting height measurement', 'forced expiratory volume'], ['Saethre-Chotzen syndrome', 'Isolated oxycephaly', 'Isolated scaphocephaly', 'Isolated brachycephaly', 'Syndromic craniosynostosis'], ['Autosomal dominant progressive external ophthalmoplegia', 'Perrault syndrome', 'Infantile onset spinocerebellar ataxia', 'Mitochondrial DNA depletion syndrome, hepatocerebral form due to DGUOK deficiency', 'Mitochondrial DNA depletion syndrome'], ['pancreatic adenocarcinoma', 'Benign familial chorea', 'Cholesterol-ester transfer protein deficiency', 'Testicular regression syndrome', 'Homozygous familial hypercholesterolemia'], ['mean corpuscular volume', 'sex hormone-binding globulin measurement', 'mean corpuscular hemoglobin concentration', 'body height', 'mean corpuscular hemoglobin'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'haptoglobin measurement', 'hemoglobin measurement', 'hematocrit'], ['acute promyelocytic leukemia', 'neoplasm', 'diphtheria', 'rheumatic disease', 'leukemia'], ['Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'rheumatoid arthritis', 'ulcerative colitis', 'ankylosing spondylitis', 'immune system disease'], ['colorectal carcinoma', 'breast carcinoma', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'neoplasm'], ['Oculocutaneous albinism type 1', 'Oculocutaneous albinism type 1A', 'Oculocutaneous albinism type 1B', 'Temperature-sensitive oculocutaneous albinism type 1', 'Ocular albinism with congenital sensorineural deafness'], ['urinary albumin to creatinine ratio', 'Moderate albuminuria', 'metabolic syndrome', 'blood protein measurement', 'neoplasm'], ['Nasu-Hakola disease', 'dementia (disease)', 'late-onset Alzheimers disease', "Parkinson's disease", 'memory impairment'], ['Oculocutaneous albinism type 3', 'Oculocutaneous albinism', 'hair color', 'Piebaldism', 'cutaneous melanoma'], ['acute myeloid leukemia', 'lung adenocarcinoma', 'myelodysplastic syndrome', 'Uterine Carcinosarcoma', 'chronic myelomonocytic leukemia'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'influenza'], ['total cholesterol measurement', 'high density lipoprotein cholesterol measurement', 'monocyte count', 'lymphocyte count', 'sex hormone-binding globulin measurement'], ['HIV infection', 'cancer', 'listeriosis', 'T-cell leukemia', 'Tuberculosis'], ['HIV infection', 'listeriosis', 'cancer', 'T-cell leukemia', 'Cystic fibrosis'], ['platelet count', 'self reported educational attainment', 'sex hormone-binding globulin measurement', 'insulin like growth factor measurement', 'platelet crit'], ['red blood cell distribution width', 'mean reticulocyte volume', 'platelet crit', 'reticulocyte measurement', 'platelet count'], ['colorectal carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'breast carcinoma', 'hepatocellular carcinoma'], ['Angelman syndrome', 'genetic disorder', 'Intellectual disability', 'mental retardation', 'Global developmental delay'], ['Short stature due to GHSR deficiency', 'oral cavity carcinoma', 'head and neck carcinoma', 'esophageal adenocarcinoma', 'esophageal squamous cell carcinoma'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'alcohol drinking', 'triglyceride measurement', 'alcohol consumption measurement'], ['Early-onset progressive neurodegeneration - blindness - ataxia - spasticity', 'Young adult-onset Parkinsonism', 'Spastic paraplegia', 'glioblastoma multiforme', 'non-small cell lung carcinoma'], ['low grade glioma', 'heart failure', 'colorectal carcinoma', 'Hand-foot syndrome', 'hydrops fetalis (disease)'], ['triglyceride measurement', 'sleep duration', 'diabetes mellitus', 'Partial pancreatic agenesis', 'Asperger syndrome'], ['obesity', 'coronary artery disease', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'Duane retraction syndrome'], ['Gaucher disease', 'Niemann-Pick disease type C', 'red blood cell distribution width', 'neutrophil count', 'Gaucher disease type 1'], ['developmental and epileptic encephalopathy, 84', 'Epileptic encephalopathy', 'hemorrhoid', 'peripheral arterial disease', 'traffic air pollution measurement'], ['Crigler-Najjar syndrome type 1', 'Crigler-Najjar syndrome type 2', 'Gilbert syndrome', 'Crigler-Najjar syndrome', 'Transient familial neonatal hyperbilirubinemia'], ['cutaneous melanoma', 'lung adenocarcinoma', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'testosterone measurement'], ['erythrocyte count', 'hemoglobin measurement', 'red blood cell density measurement', 'hematocrit', 'glomerular filtration rate'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'asthma', 'seasonal allergic rhinitis', 'eosinophil percentage of granulocytes'], ['Pachyonychia congenita', 'blood protein measurement', 'midbrain', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm'], ['breast adenocarcinoma', 'cancer', 'glioma', 'acute myeloid leukemia', 'nonpapillary renal cell carcinoma'], ['Familial juvenile hyperuricemic nephropathy type 1', 'Autosomal dominant medullary cystic kidney disease with or without hyperuricemia', 'Medullary Cystic Kidney Disease Type II', 'Autosomal dominant medullary cystic kidney disease with hyperuricemia', 'hypertension'], ['chronotype measurement', 'vital capacity', 'acute myeloid leukemia', 'breast adenocarcinoma', 'ovarian cancer'], ["Alzheimer's disease", 'Early-onset autosomal dominant Alzheimer disease', 'fat body mass', 'refractive error measurement', 'type II diabetes mellitus'], ['Hyper-IgM syndrome type 5', 'Hyper-IgM syndrome without susceptibility to opportunistic infections', 'Hyper-IgM syndrome type 2', 'infect', 'infection'], ['protein measurement', 'Hemoglobin E - beta-thalassemia', 'self reported educational attainment', 'Adult-onset autosomal recessive sideroblastic anemia', 'Delta-beta-thalassemia'], ['urinary bladder cancer', 'Dent disease type 1', 'Ochoa syndrome', 'Hereditary renal hypouricemia', 'Primary hyperoxaluria type 2'], ['myeloid white cell count', 'leukocyte count', 'cutaneous melanoma', 'neutrophil count', 'glioma'], ['vaginal discharge', 'candidiasis', 'neoplasm', 'gastric cancer', 'cancer'], ['cardiomyopathy', 'Propionic acidemia', 'Isolated NADH-CoQ reductase deficiency', 'mitochondrial complex 3 deficiency, nuclear type 10', 'Isolated CoQ-cytochrome C reductase deficiency'], ['sex hormone-binding globulin measurement', 'reticulocyte count', 'sleep duration', 'triglyceride measurement', 'mean corpuscular volume'], ['Retinitis pigmentosa', 'Usher syndrome type 2', 'Usher syndrome', 'hearing loss', 'Retinal dystrophy'], ['mean corpuscular volume', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'triglyceride measurement'], ['sex hormone-binding globulin measurement', 'triglyceride measurement', 'heel bone mineral density', 'low density lipoprotein cholesterol measurement', 'vitamin D measurement'], ['Hao-Fountain syndrome', 'platelet count', 'mean platelet volume', 'platelet component distribution width', 'lymphocyte count'], ['vital capacity', 'triglyceride measurement', 'ovarian cancer', 'age at menarche', 'low grade glioma'], ['Congenital intrauterine infection-like syndrome', 'ovarian cancer', 'lung adenocarcinoma', 'ovarian carcinoma', 'glioblastoma multiforme'], ['low grade glioma', 'ovarian cancer', 'breast adenocarcinoma', 'body height', 'reticulocyte count'], ['endometrial cancer', 'diabetes mellitus', 'ovarian carcinoma', 'colorectal carcinoma', 'cancer'], ['chronotype measurement', 'sitting height measurement', 'head and neck squamous cell carcinoma', 'body fat percentage', 'migraine disorder'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'low grade glioma', 'heart failure', 'Capillary malformation - arteriovenous malformation'], ['neurodevelopmental disorder with hypotonia and autistic features with or without hyperkinetic movements', 'Intellectual disability', 'botulism', 'Global developmental delay', 'Cerebral visual impairment'], ['eosinophil percentage of leukocytes', 'eosinophil count', 'glioblastoma multiforme', 'lung adenocarcinoma', 'cyst'], ['urinary albumin to creatinine ratio', 'coronary artery disease', 'prostate carcinoma', 'lean body mass', 'eosinophil count'], ['spina bifida', 'Isolated spina bifida', 'Neural tube closure defect', 'body height', 'neuroimaging measurement'], ['neoplasm', 'lung cancer', 'cancer', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy'], ['lymphocyte count', 'monocyte count', 'neutrophil percentage of leukocytes', 'leukocyte count', 'lymphocyte percentage of leukocytes'], ['Wagner disease', 'white matter microstructure measurement', 'appendicular lean mass', 'body fat percentage', 'lean body mass'], ['Inclusion body myopathy with Paget disease of bone and frontotemporal dementia', 'Frontotemporal dementia with motor neuron disease', 'amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'Autosomal dominant Charcot-Marie-Tooth disease type 2'], ['cup-to-disc ratio measurement', 'glioblastoma multiforme', 'acute myeloid leukemia', 'gastric carcinoma', 'multiple myeloma'], ['diabetic retinopathy', 'age-related macular degeneration', 'wet macular degeneration', 'diabetic macular edema', 'glioblastoma multiforme'], ['age-related macular degeneration', 'proliferative diabetic retinopathy', 'retinal detachment', 'diabetic macular edema', 'pathological myopia'], ['Milroy disease', 'age-related macular degeneration', 'proliferative diabetic retinopathy', 'retinal detachment', 'testosterone measurement'], ['neoplasm', 'Lymphangioleiomyomatosis', 'breast carcinoma', 'endometrial cancer', 'hemophagocytic syndrome'], ['vitamin D measurement', 'amyotrophic lateral sclerosis', 'axial length measurement', "Alzheimer's disease", 'Lewy body dementia'], ['Von Hippel-Lindau disease', 'Chuvash erythrocytosis', 'adrenal gland pheochromocytoma', 'nonpapillary renal cell carcinoma', 'papillary renal cell carcinoma'], ['Non-syndromic male infertility due to sperm motility disorder', 'neoplasm', 'Primary familial polycythemia', 'HbA1c measurement', 'mean corpuscular hemoglobin concentration'], ['Pulverulent cataract', 'Non-syndromic congenital cataract', 'cataract', 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement'], ['chronotype measurement', 'circadian rhythm', 'sum of eosinophil and basophil counts', 'lymphocyte count', 'monocyte count'], ['medulloblastoma', 'experimental autoimmune encephalomyelitis', 'lean body mass', 'Spinocerebellar ataxia type 4', 'body fat percentage'], ['Hereditary combined deficiency of vitamin K-dependent clotting factors', 'atrial fibrillation', 'vitamin K-dependent clotting factors, combined deficiency of, type 2', 'deep vein thrombosis', 'cholesterol embolism'], ['hypertension', 'colitis', "Barrett's esophagus", 'esophageal adenocarcinoma', 'Bartsocas-Papas syndrome'], ['inflammatory bowel disease', 'prostate carcinoma', 'heel bone mineral density', 'bone density', 'electrocardiography'], ['psoriasis', 'nonpapillary renal cell carcinoma', 'heel bone mineral density', 'electrocardiography', 'bone density'], ['age at onset', 'Myopia', 'visual perception measurement', 'glioblastoma multiforme', 'testosterone measurement'], ['Cohen syndrome', 'genetic disorder', 'Microcephaly', 'Intellectual disability', 'mental retardation'], ['pontocerebellar hypoplasia type 13', 'visual perception measurement', 'neoplasm', 'Sepsis', 'malaria'], ['Keratoconus', 'craniofacial anomalies and anterior segment dysgenesis syndrome', 'Posterior polymorphous corneal dystrophy', 'mean corpuscular hemoglobin', 'alcohol consumption measurement'], ['breast adenocarcinoma', 'lung adenocarcinoma', 'blood protein measurement', 'low density lipoprotein cholesterol measurement', 'heel bone mineral density'], ['acute myeloid leukemia', 'intelligence', 'Posterior polymorphous corneal dystrophy', 'X-linked endothelial corneal dystrophy', 'Fuchs endothelial corneal dystrophy'], ['Von Willebrand disease', 'Von Willebrand disease type 2B', 'Von Willebrand disease type 2A', 'Von Willebrand disease type 2N', 'Von Willebrand disease type 2M'], ['Wiskott-Aldrich syndrome', 'X-linked thrombocytopenia with normal platelets', 'Hereditary thrombocytopenia with normal platelets', 'X-linked severe congenital neutropenia', 'Thrombocytopenia'], ['endometrial cancer', 'clear cell renal carcinoma', 'neuroblastoma', 'osteosarcoma', 'Ewing sarcoma'], ['red blood cell distribution width', 'platelet component distribution width', 'nan', 'nan', 'nan'], ['Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'Short stature due to primary acid-labile subunit deficiency', 'Transient neonatal diabetes mellitus', 'Hyperinsulinism due to INSR deficiency', 'Obesity due to melanocortin 4 receptor deficiency'], ['cutaneous squamous cell carcinoma', 'glioblastoma multiforme', 'endometrial cancer', 'breast adenocarcinoma', 'lung adenocarcinoma'], ['blood protein measurement', 'ovarian cancer', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'Herpes Zoster'], ['neoplasm', 'ovarian cancer', 'renal fibrosis', 'chronic kidney disease', 'endometrial cancer'], ['Wolfram syndrome', 'Wolfram-like syndrome', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'type II diabetes mellitus', 'Non-syndromic congenital cataract'], ['heel bone mineral density', 'urinary albumin to creatinine ratio', 'Moderate albuminuria', 'albuminuria', 'bone quantitative ultrasound measurement'], ['bone density', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'hematocrit', 'erythrocyte count'], ['Hereditary sensory and autonomic neuropathy type 2', 'Pseudohypoaldosteronism type 2C', 'body mass index', 'body fat percentage', 'Pseudohypoaldosteronism type 2'], ['mean platelet volume', 'neuroimaging measurement', 'mathematical ability', 'brain volume measurement', 'brain measurement'], ['Atypical Mayer-Rokitansky-Küster-Hauser syndrome', 'Mayer-Rokitansky-Küster-Hauser syndrome', 'SERKAL syndrome', 'body height', 'lean body mass'], ['Werner syndrome', 'colorectal adenocarcinoma', 'gastric carcinoma', 'colorectal cancer', 'kidney neoplasm'], ['Denys-Drash syndrome', 'Frasier syndrome', 'Nephroblastoma', 'Meacham syndrome', 'Wilms tumor 1'], ['Autosomal recessive cerebellar ataxia-epilepsy-intellectual disability syndrome due to WWOX deficiency', 'Early infantile epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'esophageal squamous cell carcinoma', 'West syndrome'], ['neoplasm', 'cancer', 'prostate cancer', 'gastric cancer', 'neuroblastoma'], ['hematocrit', 'hemoglobin measurement', 'hair shape measurement', 'mean corpuscular hemoglobin', 'pancreatic carcinoma'], ['adolescent idiopathic scoliosis', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'Hepatitis'], ['ulcer disease', 'glioblastoma multiforme', 'C-X-C motif chemokine 16 measurement', 'breast adenocarcinoma', 'autoimmune disease'], ['Xanthinuria type I', 'Hereditary xanthinuria', 'Xanthinuria type II', 'gout', 'leukemia'], ['X-linked lymphoproliferative disease', 'inflammatory bowel disease', 'Splenomegaly', 'Lymphoproliferative syndrome', 'Sepsis'], ['monocyte count', 'monocyte percentage of leukocytes', 'apolipoprotein A 1 measurement', 'sex hormone-binding globulin measurement', 'Left ventricular noncompaction'], ['McLeod neuroacanthocytosis syndrome', 'genetic disorder', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma'], ['chronic lymphocytic leukemia', 'multiple myeloma', 'HIV infection', 'influenza', 'gastric carcinoma'], ['Intellectual disability-coarse face-macrocephaly-cerebellar hypotrophy syndrome', 'Adult-onset autosomal recessive cerebellar ataxia', 'breast adenocarcinoma', 'ovarian cancer', 'body height'], ['Fanconi anemia', 'Inherited cancer-predisposing syndrome', 'acute myeloid leukemia', 'mean platelet volume', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation'], ['HIV infection', 'lung adenocarcinoma', 'breast adenocarcinoma', 'childhood onset asthma', 'endometrial cancer'], ['Spondylo-ocular syndrome', 'Pseudoxanthoma elasticum', 'genetic disorder', 'skin pigmentation', 'Autosomal recessive polycystic kidney disease'], ['Uveal coloboma - cleft lip and palate - intellectual disability', 'sex hormone-binding globulin measurement', 'polycystic ovary syndrome', 'pulse pressure measurement', 'birth weight'], ['Autosomal dominant intermediate Charcot-Marie-Tooth disease type C', 'Charcot-Marie-Tooth disease', 'Primary amenorrhea', 'retinal degeneration', 'Hepatic steatosis'], ['birth weight', 'HbA1c measurement', 'fasting blood glucose measurement', 'type II diabetes mellitus', 'serum metabolite measurement'], ['Leber hereditary optic neuropathy', 'Autosomal recessive isolated optic atrophy', 'Autosomal recessive syndromic optic atrophy', 'platelet count', 'platelet crit'], ['cancer', 'Noonan syndrome', 'Costello syndrome', 'hypertrophic cardiomyopathy', 'lean body mass'], ['endometrial stromal sarcoma', 'clear cell sarcoma', 'synovial sarcoma', 'bile duct carcinoma', 'undifferentiated pleomorphic sarcoma'], ['lean body mass', 'fat body mass', 'gut microbiome measurement', 'taxonomic microbiome measurement', 'body weight'], ['reticulocyte measurement', 'body mass index', 'reticulocyte count', 'chronotype measurement', 'risk-taking behaviour'], ['Combined immunodeficiency due to ZAP70 deficiency', 'autoimmune disease', 'Combined T and B cell immunodeficiency', 'Severe combined immunodeficiency', 'X-Linked Combined Immunodeficiency Diseases'], ['mean corpuscular volume', 'body fat percentage', 'erythrocyte count', 'mean corpuscular hemoglobin', 'body mass index'], ['sex hormone-binding globulin measurement', 'testosterone measurement', 'asthma', 'body mass index', 'hematocrit'], ['response to vaccine', 'lean body mass', 'visual perception measurement', 'chronic hepatitis B virus infection', 'cardiovascular disease'], ['body height', 'body mass index', 'self reported educational attainment', 'neoplasm', 'gait measurement'], ['lymphocyte count', 'neuroticism measurement', 'mathematical ability', 'feeling tense measurement', 'eosinophil percentage of leukocytes'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'asthma', 'Dupuytren Contracture', 'lymphocyte count'], ['Intellectual disability-developmental delay-contractures syndrome', 'genetic disorder', 'X-linked syndromic intellectual disability', 'Autosomal recessive spastic paraplegia type 25', 'Juvenile amyotrophic lateral sclerosis'], ['schizophrenia', 'self reported educational attainment', 'cutaneous melanoma', 'pancreatic adenocarcinoma', 'visual perception measurement'], ['Mowat-Wilson syndrome', 'genetic disorder', 'mean corpuscular hemoglobin concentration', 'renal cell carcinoma', 'self reported educational attainment'], ['neoplasm', 'prostate cancer', 'cancer', 'hepatocellular carcinoma', 'pancreatic carcinoma'], ['eosinophil count', 'ulcerative colitis', 'psoriasis', 'eosinophil percentage of leukocytes', 'Hepatic fibrosis'], ['systolic blood pressure', 'chronotype measurement', 'pulse pressure measurement', 'Agents acting on the renin-angiotensin system use measurement', 'appendicular lean mass'], ['red blood cell distribution width', 'erythrocyte count', 'red blood cell density measurement', 'mean corpuscular hemoglobin', 'age at menopause'], ['cancer', 'breast adenocarcinoma', 'infect', 'infection', 'atrial fibrillation'], ['Hand-foot syndrome', 'hydrops fetalis (disease)', 'rheumatoid arthritis', 'obesity', 'Hypoglycemia'], ['mean corpuscular volume', 'body height', 'lean body mass', 'forced expiratory volume', 'infectious diarrheal disease'], ['particulate matter air pollution measurement', 'unipolar depression', 'C-reactive protein measurement', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'depressive symptom measurement'], ['platelet count', 'mean platelet volume', 'platelet crit', 'mean corpuscular volume', 'Abnormality of refraction'], ['cancer', 'leukemia', 'myelodysplastic syndrome', 'precursor lymphoblastic lymphoma/leukemia', 'body mass index'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'neoplasm', 'breast cancer', 'mathematical ability'], ['cancer', 'neoplasm', 'systemic juvenile idiopathic arthritis', 'chronic kidney disease', 'Zika virus infectious disease'], ['particulate matter air pollution measurement', 'neoplasm', 'breast cancer', 'unipolar depression', 'aortic stenosis'], ['body height', 'platelet component distribution width', 'pancreatic adenocarcinoma', 'lung cancer', 'nan'], ['Brittle cornea syndrome', 'Keratoconus', 'lean body mass', 'central corneal thickness', 'appendicular lean mass'], ['hematocrit', 'sex hormone-binding globulin measurement', 'smoking status measurement', 'body height', 'grip strength measurement'], ['head and neck squamous cell carcinoma', 'Zinc deficiency', 'hematocrit', 'neoplasm', 'sex hormone-binding globulin measurement'], ['Myoclonus-dystonia syndrome', 'Huntington disease-like 2', 'X-linked dominant intellectual disability - epilepsy syndrome', 'Spinocerebellar ataxia type 14', 'FRAXE intellectual disability'], ['dehydroepiandrosterone sulphate measurement', 'blood metabolite measurement', 'body fat percentage', 'platelet count', 'smoking behavior'], ['Blackfan-Diamond anemia', 'appendicular lean mass', 'neutrophil count', 'alcohol dependence', 'amyotrophic lateral sclerosis'], ['appendicular lean mass', 'fat body mass', 'lean body mass', 'Congenital contractural arachnodactyly', 'cholangiocarcinoma'], ['prostate carcinoma', 'visual perception measurement', 'Ataxia-telangiectasia', 'schizophrenia', 'self reported educational attainment'], ['mean corpuscular hemoglobin concentration', 'Down syndrome', 'mean corpuscular hemoglobin', 'body height', 'appendicular lean mass'], ['balding measurement', 'type II diabetes mellitus', 'hemoglobin measurement', 'hair shape measurement', 'hematocrit'], ['sitting height measurement', 'rheumatoid arthritis', 'neutrophil count', 'lymphocyte count', 'diet measurement']]
    #print(list(pepega1["Unnamed: 1"]))
    
    ### NAN NAN NAN Diseases like ABCD1 need to be filled in
    '''
    diseases123 = list(pepega1["Unnamed: 1"])
    count = 2
    for gene in diseases123:
        #print(gene)
        if not (isinstance(gene, str)):
            print(count)
        count += 1
    #print(isinstance(diseases123[63], str))
    '''
    ######
    ######               HDAC15 in BIOMARIN DATA IS INCORRECT: SHOULD BE DMD. NO OF PATENTS IS 4.
    ######
    '''
    for gene in range(len(disease_genes)):
        if (disease_genes[gene] not in unique_genes):
            continue
        heck = []
        for i in range(5):
            x = list(pepega1["Unnamed: " + str(i+1)])
            heck.append(x[gene])
        diseases.append(heck)
        #print(heck)
    print(diseases)
    '''
    
    keys = list(genes_with_patents_sorted)
    values = list(genes_with_patents_sorted.values())
    disease1, disease2, disease3, disease4, disease5 = ([] for i in range(5))
    filler = [None] * 3349
    distribution = list(genes_with_companies.values())
    sorted_distribution = []
    for gene in keys:
        sorted_distribution.append(genes_with_companies[gene])
    #print(sorted_distribution)
    print(len(sorted_distribution))
    
    #Ipsen, Pfizer, Moderna, Perrigo, CSL, Celgene, Genmab, Galapagos, BioNTech, Biomarin, Biogen, Astellas_Pharma, Seattle_Genetics, Roche, Bayer, Novartis, Incyte, Allergan, Lundbeck, Shionogi_Co, Servier, Mitsubishi, Sumitomo_Dainippon, Vertex, Eisai, Regeneron, Alexion, Otsuka, Daiichi_Sankyo, Boehringer, Novo_Nordisk, Eli_Lilly, Gilead, Amgen, AstraZeneca, Sanofi, GlaxoSmithKline, Abbvie, Merck_Co, Merck_KGaA, Johnson_Johnson  = ({} for i in range(41))
    dictionary = {}
    for company in res:
        CompanyDictionary(company, dictionary, sorted_distribution, keys)
    print(len(list(dictionary["Roche"].values())))
    
            
    
    for gene in keys:
        index = unique_genes.index(gene)
        associated_diseases = diseases[index]
        disease1.append(associated_diseases[0])
        disease2.append(associated_diseases[1])
        disease3.append(associated_diseases[2])
        disease4.append(associated_diseases[3])
        disease5.append(associated_diseases[4])
  
    
    df = pd.DataFrame({'Targets': keys, 'Total Number of Patents': values, 'Distribution': sorted_distribution, 'Indication 1': disease1, 'Publications 1': filler, 'Indication 2': disease2, 'Publications 2': filler, 'Indication 3': disease3, 'Publications 3': filler, 'Indication 4': disease4, 'Publications 4': filler, 'Indication 5': disease5, 'Publications 5': filler})
    writer = pd.ExcelWriter('formatted.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Genes', index=False)
    writer.save()
    
    dp = pd.DataFrame({'Targets': keys, 'Total Number of Patents': values, 'Roche': list(dictionary["Roche"].values()), 'Novartis': list(dictionary["Novartis"].values()), 'Bayer': list(dictionary["Bayer"].values()), 'AbbVie': list(dictionary["AbbVie"].values())})
    writer1 = pd.ExcelWriter('other.xlsx', engine='xlsxwriter')
    dp.to_excel(writer1, sheet_name='Genes', index=False)
    writer1.save()

    #writer = pd.ExcelWriter('test.xlsx', engine='openpyxl')
    #wb = writer.book

In [ ]:
'''
pepega1 = pd.read_excel("Gene_Conditions_All.xls")
    disease_genes = list(pepega1["Unnamed: 0"])
    diseases12345 = []
    #print(list(pepega1["Unnamed: 1"]))
    
    
    for gene in range(len(disease_genes)):
        heck = []
        for i in range(10):
            x = list(pepega1["Unnamed: " + str(i+1)])
            heck.append(x[gene])
        #diseases12345.append(heck)
        print(heck)
'''